## Combine CSV files
### All files were downloaded from FanGraphs and contain relief pitcher data from 2015-2023

In [152]:
import pandas as pd
import os

def load_data(base_path):
    
    # Dictionary to hold all data
    data = {}
    
    # Parameters of a randomly generated, late game situation
    innings = ['6th_Inning', '7th_Inning', '8th_Inning', '9th_Inning']
    outs = ['0_Outs', '1_Out', '2_Outs']
    base_states = ['Bases_Empty', 'Runners_On', 'RISP', 'Bases_Loaded']
    
    # A generic and specific prediction will be made
    general = 'General'
    out_spec = 'Out_Specific'
    
    # Thresholds to ignore pitchers w/o much experience
    min_games = 5
    min_ip = 1
    
    # Iterate over innings, outs, and base_states
    for inning in innings:
        data[inning] = {}
        for out in outs:
            data[inning][out] = {}
                
            for base_state in base_states:
                
                # Construct the paths to the CSV files
                stnd_path = os.path.join(base_path, inning, out_spec, out, base_state, 'standard.csv')
                adv_path = os.path.join(base_path, inning, out_spec, out, base_state, 'advanced.csv')
                bb_path = os.path.join(base_path, inning, out_spec, out, base_state, 'batted_balls.csv')
                
                # Load the CSV files if they exist
                stnd_df = pd.read_csv(stnd_path) if os.path.exists(stnd_path) else pd.DataFrame()
                adv_df = pd.read_csv(adv_path) if os.path.exists(adv_path) else pd.DataFrame()
                bb_df = pd.read_csv(bb_path) if os.path.exists(bb_path) else pd.DataFrame()
                
                # Filter data based on minimum thresholds
                stnd_df = stnd_df[stnd_df['G'] >= min_games]
                adv_df = adv_df[adv_df['IP'] >= min_ip] 
                bb_df = bb_df[bb_df['IP'] >= min_ip]
               
                # Merge the dataframes on pitcher's name
                combined_df = pd.merge(stnd_df, adv_df, on='Name', how='outer')
                combined_df = pd.merge(combined_df, bb_df, on='Name', how='outer')
                
                combined_df.fillna(0 , inplace=True)
                
                # Store combined dataframe in the dict
                data[inning][out][base_state] = combined_df
                
        # Load General data
        gen_path = os.path.join(base_path, inning, general)
        if os.path.exists(gen_path):
            data[inning]['General'] = {}

            # Load the general CSV files
            stnd_path_gen = os.path.join(gen_path, 'standard.csv')
            adv_path_gen = os.path.join(gen_path, 'advanced.csv')
            bb_path_gen = os.path.join(gen_path, 'batted_balls.csv')

            stnd_df_gen = pd.read_csv(stnd_path_gen) if os.path.exists(stnd_path_gen) else pd.DataFrame()
            adv_df_gen = pd.read_csv(adv_path_gen) if os.path.exists(adv_path_gen) else pd.DataFrame()
            bb_df_gen = pd.read_csv(bb_path_gen) if os.path.exists(bb_path_gen) else pd.DataFrame()

            # Filter data based on minimum thresholds
            stnd_df_gen = stnd_df_gen[stnd_df_gen['G'] >= min_games]
            adv_df_gen = adv_df_gen[adv_df_gen['IP'] >= min_ip]
            bb_df_gen = bb_df_gen[bb_df_gen['IP'] >= min_ip]

            # Merge the dataframes on pitcher's name
            combined_df_gen = pd.merge(stnd_df_gen, adv_df_gen, on='Name', how='outer')
            combined_df_gen = pd.merge(combined_df_gen, bb_df_gen, on='Name', how='outer')

            combined_df_gen.fillna(0, inplace=True)

            # Store combined general dataframe in the dict
            data[inning]['General'] = combined_df_gen 
            
    return data


# Navigate to dir and load everything
script_dir = os.getcwd()
base_path = os.path.join(script_dir, 'Situational_Pitching/Versus_All')
data = load_data(base_path)

## Predict the best relief pitcher for a late game situation using a Random Forest Regressor

In [153]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Combines the data by adding needed columns for the situation #
#
# Params:
#   data (dict)      - created from all CSV files across all situations
#   inning (int)     - the inning for situation
#   outs (int)       - number of outs in situation
#   base_state (int) - represents runners on base (empty, runners on, RISP, or loaded)
#
# Returns:
#   1. Pandas dataframe of all data relevant to the situation
#   2. List of all pitcher names
#                                                              #
def create_df(data, inning, outs, base_state):
    
    # Store the relevant data and pitchers' names
    all_data = []
    names = []
    
    base_state_map = {
        'Bases_Empty': 0,
        'Runners_On': 1,
        'RISP': 2,
        'Bases_Loaded': 3
    }
    # Iterate over innings, outs, & base_states and
    # add context to the data if it matches the chosen situation.
    # Otherwise, ignore it
    for inning_key, out_data in data.items():
        inning_num = int(inning_key.split('_')[0][0])
        if inning is not None and inning_num != inning:
            continue

        for outs_key, base_state_data in out_data.items():
            if outs_key == 'General':
                continue
            outs_num = int(outs_key.split('_')[0])
            if outs is not None and outs_num != outs:
                continue

            for base_state_key, df in base_state_data.items():
                base_state_int = base_state_map.get(base_state_key, -1)
                if base_state is not None and base_state_int != base_state:
                    continue

                if not df.empty:
                    df['BaseState'] = base_state_int
                    df['Inning'] = inning_num
                    df['Outs'] = outs_num
                   
                    all_data.append(df)
                    names.extend(df['Name'].tolist())
                    
    # Ensure dataframe isn't empty
    if not all_data:
        print("No data collected.")
        
    # Combine everything
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Return df and names to keep track of everyone's stats
    return combined_df, names


# Clean and prepare the data #
#
# Params:
#   combined_df (Pandas dataframe)  - created from all CSV files across all situations
#   names (list)                    - the inning for situation
#   weights (dictionary)            - essential stats and their respective weight / value
#
# Returns:
#   Training & testing splits along with the corresponding player names
#                            #
def create_splits(combined_df, names, weights):
    
    # Determine normalization parameters
    norm_stats = ['K-BB%', 'OBP', 'ERA', 'BABIP', 'FIP', 'LOB%', 'K%', 'WHIP', 'Hard%']
    norm_params = get_norm_params(combined_df, norm_stats)

    # Separate features and target variable
    X = combined_df.drop(['playerId', 'Name', 'Season_x', 'Tm_x'], axis=1, errors='ignore')
    
    # Create a new column using important stats based on the situation
    combined_df['Effectiveness'] = combined_df.apply(lambda row: calc_effectiveness(row, weights, norm_params), axis=1)
    Y = combined_df['Effectiveness']
    
    # Identify non-numeric columns & handle if needed
    non_nums = X.select_dtypes(include=['object']).columns
    for col in non_nums:
        X[col] = X[col].astype('category').cat.codes

    # Convert remaining columns to & fill w/ placeholders
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

    # Align Y with X after dropping rows w/ NAN values
    Y = Y[X.index]
    
    # Ensure player_ids and player_names are aligned
    names = [names[i] for i in X.index]
    
    # Ensure there is sufficient data
    if X.shape[0] == 0 or Y.shape[0] == 0:
        raise ValueError("No data available after preprocessing.")

    return train_test_split(X, Y, names, test_size=0.2, random_state=42)


# Choose metrics to focus on & weight them based on a SPECIFIC situation #
#
# Params:
#   inning (int)     - the inning for situation
#   outs (int)       - number of outs in situation
#   base_state (int) - represents runners on base (empty, runners on, RISP, or loaded)
#
# Returns:
#   Dictionary of essential stats & their respective weights
#                                                                        #
def calc_weights(inning, outs, base_state):
    
    weights = {}
    
    # Weight levels
        # High: 0.6 - 0.9
        # Medium: 0.3 - 0.5
        # Low: 0.1 - 0.3 
    
    # 6th or 7th
    if inning < 8:
        
        # Bases empty or runner on
        # Low Leverage (6th / 7th inning, bases empty / runners on, large lead/deficit, long relief)
        if base_state < 2:
            weights['K-BB%'] = 0.40
            weights['OBP'] = -0.25
            weights['ERA'] = -0.45
            weights['BABIP'] = -0.20
            weights['FIP'] = 0.40
    
            
        # RISP or bases loaded
        # Medium Leverage (6th / 7th inning, RISP or worse, CLOSE GAME, long / short term relief)
        else:
            
            # Threatening spot
            if outs < 2:
                weights['LOB%'] = 0.80
                weights['K%'] = 0.70
                weights['WHIP'] = -0.65
                weights['ERA'] = -0.40
                weights['BABIP'] = -0.20
                weights['FIP'] = 0.52
            
            else:
                weights['LOB%'] = 0.72
                weights['K-BB%'] = 0.70
                weights['WHIP'] = -0.50
                weights['ERA'] = -0.40
                weights['BABIP'] = -0.35
                weights['FIP'] = 0.43
                

    # 8th or 9th
    # High Leverage: (8th / 9th inning, RISP or worse, close game, short term relief)
    else:
        
        # Bases empty or runners on 
        if base_state < 2:
            
            # Slightly threatening: 0 or 1 out
            if outs < 2: 
                weights['LOB%'] = 0.67
                weights['K-BB%'] = 0.25
                weights['OBP'] = -0.75
                weights['ERA'] = -0.40
                weights['BABIP'] = -0.33
                weights['FIP'] = 0.80
                
            else:
                weights['LOB%'] = 0.60
                weights['K-BB%'] = 0.25
                weights['ERA'] = -0.30
                weights['BABIP'] = -0.45
                weights['FIP'] = 0.74
            
        # RISP or bases loaded
        else:
            
            # 0/1 Out:
            if outs < 2:
                weights['LOB%'] = 0.85
                weights['K%'] = 0.63
                weights['WHIP'] = -0.90
                weights['ERA'] = -0.20
                weights['Hard%'] = -0.70
                weights['FIP'] = 0.82
                
            # 2 Outs:
            else:
                weights['LOB%'] = 0.80
                weights['K%'] = 0.57
                weights['WHIP'] = -0.85
                weights['ERA'] = -0.17
                weights['Hard%'] = -0.65
                weights['FIP'] = 0.77

    return weights


# Choose metrics to focus on & weight them based on a GENERAL situation #
#
# Params:
#   inning (int) - the inning for situation
#
# Returns:
#   Dictionary of essential stats & their respective weights
#                                                                       #
def calc_weights_gen(inning):
    
    # Store the stats and their respective weights
    weights = {}
    
    # Weight levels:
        # High: 0.6 - 0.9
        # Medium: 0.3 - 0.5
        # Low: 0.1 - 0.3 
    
    # 6th or 7th
    if inning < 8:
        weights['K-BB%'] = 0.40
        weights['OBP'] = -0.25
        weights['ERA'] = -0.45
        weights['BABIP'] = -0.20
        weights['FIP'] = 0.40        

    # 8th or 9th
    else:
        weights['LOB%'] = 0.77
        weights['K-BB%'] = 0.25
        weights['OBP'] = -0.75
        weights['ERA'] = -0.40
        weights['BABIP'] = -0.33
        weights['FIP'] = 0.80
                
    return weights


# Generate a late game situation specifying inning, number of outs, and runner amount #
#
# Returns:
#   Pandas dataframe with the above mentioned information
#                                                                                     #
def generate_random_situation():
    
    # Pick innning between 6th - 9th
    inning = np.random.randint(6, 10)  
    
    # 0 - 2 outs
    outs = np.random.randint(0, 3) 
    
    # Pick runner amount:
    #   0 = bases empty
    #   1 = runners on
    #   2 = RISP
    #   3 = bases loaded
    base_state = np.random.randint(0, 4)  
    
    situation = {
        'Inning': [inning],
        'Outs': [outs],
        'BaseState': [base_state]
    }
    
    situation_df = pd.DataFrame(situation)
    
    return situation_df


# ----- Z-Score Normalization ----- #

# Calculate custom effectiveness score based on preferred stats #
#
# Params:
#   row (Pandas object) - contains all data from a single row of the dataframe
#   weights (dict)      - essential stats & their respective weights
#   norm_params (list)  - all metrics to be normalized
#
# Returns:
#   Target statistic for model to determine the best pitcher
#                                                               #
def calc_effectiveness(row, weights, norm_params):
    effectiveness = 0
    missing = []
    
    # Iterate & normalize if applicable
    for stat, weight in weights.items():
        if stat in row and not pd.isnull(row[stat]):
            if stat in norm_params:
                mean, std_dev = norm_params[stat]
                normalized_stat = normalize(row[stat], mean, std_dev)
                effectiveness += normalized_stat * weight
            else:
                effectiveness += row[stat] * weight
        else:
            missing.append(stat)
    
    if missing:
        print(f"Warning: Missing stats in row: {missing}")
    
    return effectiveness


# Normalize all stats contributing to the effectiveness score #
#
# Params:
#   stat (float)     - current statistic
#   mean (float)     - mean of the stat across all pitchers
#   std_dev (float)  - standard deviation of the stat across all pitchers
#
# Returns:
#   z-score / normalized version of the statistic
#                                                             #
def normalize(stat, mean, std_dev):
    # Avoid division by zero, return 0 if std_dev is 0
    if std_dev == 0:
        return 0
    
    return (stat - mean) / std_dev


# Normalize all stats contributing to the effectiveness score #
#
# Params:
#   df (Pandas dataframe) - combined dataset
#   stats (list)          - list of all stats to be normalized
#
# Returns:
#   Dictionary of stats & respective tuple of (mean, std_dev)
#                                                             #
def get_norm_params(df, stats):
    
    # Store mean & std dev of each stat
    norm_params = {}
    
    # Calculate mean & std dev for all relevant stats
    for stat in stats:
        mean = df[stat].mean()
        std_dev = df[stat].std()
        norm_params[stat] = (mean, std_dev)
        
    return norm_params
# --------------------------------- #


# Normalize all stats contributing to the effectiveness score #
#
# Params:
#   situation_df (Pandas dataframe) - combined dataset
#   model (RandomForestRegressor)   - list of all stats to be normalized
#   X_train (Pandas dataframe)      - training data used to fit the model
#   names_train (list)              - all pitcher names corresponding to training data 
#
# Returns:
#   Name of the most effective pitcher for the situation
#                                                             #
def predict_pitcher(situation_df, model, X_train, names_train):
    
    # Predict the effectiveness
    prediction = model.predict(situation_df)
    
    # Get prediction
    predicted_effectiveness = prediction[0]
    
    # Find the closest match to the prediction
    closest_idx = np.abs(model.predict(X_train) - predicted_effectiveness).argmin()
    best_pitcher = names_train[closest_idx]
    
    return best_pitcher



# Specific Situation (inning, outs, and runners specified)
rand_situation = generate_random_situation()
print("Situation: ", rand_situation)

# Parameters for df generation
inning = rand_situation['Inning'].iloc[0]
outs = rand_situation['Outs'].iloc[0]
base_state = rand_situation['BaseState'].iloc[0]

# Weight stats based on the severity of the situation
stat_weights = calc_weights(inning, outs, base_state)

# Add context to data
df, names = create_df(data, inning, outs, base_state)

# Generic Situation (inning specified only)
stat_weights_gen = calc_weights_gen(inning)
situation = {
    'Inning': [inning],
}
    
# Creating a DataFrame to match the format of X
situation_df_gen = pd.DataFrame(situation)

# Use specific CSVs for each inning, out, and runner situation
X_train, X_test, y_train, y_test, names_train, names_test = create_splits(df, names, stat_weights)

# Use general CSVs for each inning
X_train_gen, X_test_gen, y_train_gen, y_test_gen, names_train_gen, names_test_gen = create_splits(df, names, stat_weights_gen)

# Fill in other columns with null vals
for col in X_train.columns:
    if col not in rand_situation.columns:
        rand_situation[col] = 0

# Fill in other columns with null vals
for col in X_train_gen.columns:
    if col not in situation_df_gen.columns:
        situation_df_gen[col] = 0
               
# Specific situation model 
spec_model = RandomForestRegressor()
spec_model.fit(X_train, y_train)

# Generic situation model
gen_model = RandomForestRegressor()
gen_model.fit(X_train_gen, y_train_gen)

# Make the predictions
best_pitcher_spec = predict_pitcher(rand_situation, spec_model, X_train, names_train)
best_pitcher_gen = predict_pitcher(situation_df_gen, gen_model, X_train_gen, names_train_gen)

print("Best pitcher for the specific situation:", best_pitcher_spec)
print("Best pitcher for the inning:", best_pitcher_gen)

Situation:     Inning  Outs  BaseState
0       8     1          2
Best pitcher for the specific situation: Keury Mella
Best pitcher for the inning: Luis F. Ortiz


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


## Matchup History Analysis (WIP)
### Example: Get all of Zach Wheeler's matchup history

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Run in headless mode
opts = Options()
opts.add_argument("--headless=new")
driver = webdriver.Chrome(options=opts)

# BaseballSavant ID numbers of all 30 MLB teams
team_ids = [108,109,110,111,112,113,114,115,116,117,118,119,120,121,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,158]

# Loop through each team ID and update the URL
for team_id in team_ids:
    
    # Example: Stats of all hitters on all teams vs. Zach Wheeler
    url = f'https://baseballsavant.mlb.com/player_matchup?type=batter&teamPitching=143&teamBatting={team_id}&player_id=554430'
    
    # Open the webpage
    driver.get(url)
    
    # Wait for player_table and team_table to be visible
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'player_table')))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'team_table')))
    
    # Find the player table
    player_table = driver.find_element(by=By.ID, value='player_table')
    
    # Find the team table
    team_table = driver.find_element(by=By.ID, value='team_table')
    
    # Extract data from player table
    player_data = player_table.text
    
    # Extract data from team table
    team_data = team_table.text
    
    # Print or process the scraped data for the current team
    print(f"Data for team ID {team_id}:")
    print(player_data)
    print(team_data)

# Close the WebDriver
driver.quit()

## Get IDs of all pitchers on all MLB teams (BaseballSavant)

In [ ]:
# Loading team webpages is very unstable.
# A majority of the time webpages will not load at all, 
# and the needed data is not visible.
# To save time, the staffs dictionary is saved.

# BaseballSavant ID numbers of all 30 MLB teams
team_ids = [108,109,110,111,112,113,114,115,116,117,118,119,120,121,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,158]

# Pitching staffs of all teams
staffs = {}

# Loop through each team ID and update the URL
for team_id in team_ids:
    
    url = f'https://baseballsavant.mlb.com/team/{team_id}'
  
    driver = webdriver.Chrome(options=opts)

    driver.get(url)

    # Find the statcastPitching section
    pitching_section = driver.find_element(By.ID, 'statcastPitching')

    # Use a set to store unique player IDs
    player_ids = set()

    # Find all player ID elements under the statcastPitching section
    player_id_elements = pitching_section.find_elements(By.CSS_SELECTOR, '.statcast-generic')


    for element in player_id_elements:
        player_id = element.get_attribute('id').split('_')[1]
        player_ids.add(player_id)
        
    staffs[team_id] = list(player_ids)


driver.quit()
print(staffs)

In [ ]:
staffs = {108: ['667755', '519326', '663776', '608032', '472610', '660761', '542585', '643256', '542881', '670871', '672282', '681217', '672710', '519043', '656288', '608334'], 109: ['657044', '669459', '677944', '679885', '686796', '612434', '694297', '573009', '656756', '686826', '543518', '656464', '676714', '682175', '518876', '663531', '674072', '666818', '685314', '623149', '669194', '668678'], 110: ['657097', '607644', '600986', '680570', '680694', '669211', '669203', '665152', '663989', '669169', '518886', '608344', '543056', '622253', '669330', '544150', '672335', '666974'], 111: ['657514', '601713', '445276', '641745', '686580', '592155', '676477', '502624', '677161', '678394', '455119', '670174', '681867', '676710', '663462', '683822', '669711', '656557'], 112: ['622075', '605155', '543294', '669713', '668338', '687863', '657006', '592791', '676962', '640470', '643410', '624522', '681432', '593576', '665871', '694037', '684007', '696136', '621016', '657240', '592767'], 113: ['571656', '641941', '666157', '458677', '608371', '607259', '668933', '594580', '518585', '664747', '622088', '593423', '686730', '668881', '671096', '608718', '668984'], 114: ['605130', '680704', '669212', '670766', '676440', '657265', '682120', '641149', '671922', '656529', '671106', '683769', '675540', '661403', '471911', '669456', '658648', '663474', '594902'], 115: ['663372', '621389', '571539', '607536', '683627', '663567', '656222', '615698', '641712', '641755', '683232', '596295', '664875', '595928', '663562', '656234', '680767'], 116: ['669438', '663554', '676684', '656638', '687922', '689225', '663947', '681857', '666214', '656412', '628317', '656427', '666159', '671345', '669373', '571946', '605177'], 118: ['680742', '623433', '519293', '669674', '670167', '663738', '607625', '608717', '694361', '672582', '663903', '681882', '663704', '685107', '666142', '672578', '608379', '679525'], 119: ['642232', '681402', '607192', '621111', '670990', '694813', '676568', '681911', '518489', '641401', '623465', '656420', '808967', '676272', '669165', '595014', '607455', '523260', '543339', '675627', '622694', '598286', '689017', '572020'], 120: ['571670', '680730', '607179', '571578', '669022', '671131', '672851', '640448', '663432', '676395', '571882', '592866', '680686', '640451', '663623', '606930'], 121: ['621242', '656731', '518617', '500779', '657585', '493603', '656887', '676130', '642121', '548384', '673380', '605288', '640455', '664192', '605347', '622663', '677020', '664849', '622098', '701643', '681035'], 134: ['669796', '669387', '641726', '663559', '656924', '676702', '670059', '670280', '594835', '527048', '693312', '656605', '682990', '547973', '694973', '621366', '683003', '682847'], 135: ['673513', '656302', '605397', '669093', '506433', '660853', '650633', '663158', '666745', '670810', '681190', '663568', '676680', '663362', '593974', '670970'], 136: ['660825', '693433', '642100', '674285', '676092', '682243', '592773', '642048', '662253', '669912', '657508', '669302', '608723', '676969', '675989', '676106', '622491', '669923', '663423'], 137: ['678906', '690986', '694738', '663855', '605483', '573124', '669062', '687014', '666808', '696131', '676775', '643511', '663546', '678495', '592426', '676254', '657277'], 139: ['671737', '664208', '641793', '542888', '606965', '687330', '663992', '664076', '670955', '655889', '641302', '650644', '592094', '621097', '682052', '621107', '686752', '621363', '664126'], 140: ['681982', '592351', '666720', '571760', '656271', '668390', '502085', '674003', '643493', '547179', '489446', '656641', '543135', '622250', '570632', '641312', '642546', '669391', '622786', '683004', '600917', '641540'], 141: ['666201', '554340', '621244', '579328', '605447', '592332', '663893', '650893', '647315', '663878', '684320', '657024', '669270', '670950', '605135', '641835', '670102', '643338', '502179'], 142: ['595345', '660896', '622251', '641154', '592779', '681892', '676961', '686973', '663978', '650960', '661395', '663485', '641927', '678692', '657746', '573204', '643377', '680573'], 144: ['642216', '675911', '519242', '450203', '678061', '554431', '623474', '572955', '621345', '678226', '700363', '628452', '625643', '669276', '608331', '670241', '641729', '693821', '607481', '445926'], 145: ['621051', '605121', '676979', '608678', '656629', '623167', '686563', '672860', '657612', '605154', '607200', '605182', '694363', '621383', '680897', '673929', '647336'], 146: ['676534', '572143', '677960', '669622', '666200', '664350', '665795', '676974', '677053', '682610', '666277', '663773', '642629', '669432', '640462', '686539', '666129', '681871', '656945', '687424'], 147: ['543859', '664776', '690925', '677076', '573186', '607074', '592454', '641482', '641656', '595897', '661563', '657571', '676760', '672782', '605280', '624647', '596133', '642528', '657376', '547001', '642701'], 117: ['664299', '668874', '661527', '606160', '666215', '681293', '434378', '664285', '623352', '656786', '641585', '669395', '650556', '669854', '686613', '681869', '605463', '519151'], 158: ['657649', '489119', '656730', '607067', '682842', '663542', '676879', '660600', '606303', '669084', '669952', '665625', '688107', '669060', '676083', '643361', '642547', '571948', '668964', '656266', '605452', '596001'], 133: ['683155', '666204', '676664', '547184', '621112', '621053', '605156', '663687', '695243', '548389', '674370', '666205', '519008', '671212', '518397', '668674', '622072', '687765', '613534'], 138: ['676617', '571927', '667463', '688297', '681517', '681676', '543243', '571945', '687798', '502043', '669467', '668941', '668868', '606149', '669461', '552640', '664854', '458681'], 143: ['554430', '656793', '621237', '650911', '621381', '592836', '624133', '614179', '628708', '605513', '656546', '605400', '689147', '622554', '656814', '642397']}

## Gather matchup history of every pitcher vs. every hitter (BaseballSavant)

In [ ]:
# Run in headless mode
opts = Options()
opts.add_argument("--headless=new")
driver = webdriver.Chrome(options=opts)

# BaseballSavant ID numbers of all 30 MLB teams
team_ids = [108,109,110,111,112,113,114,115,116,117,118,119,120,121,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,158]

# Dictionary to store the results
results = {}

# Go through all pitching teams
for team_pitching in team_ids:
   
    # Go through all batting teams
    for team_batting in team_ids:
        
        # Don't compare pitchers and hitters on the same team
        if team_batting == team_pitching:
            continue  

        # Iterate over all pitchers on the current team
        for pitcher_id in staffs[team_pitching]:
            
            # Set the URL with the current teamPitching and teamBatting values
            url = f'https://baseballsavant.mlb.com/player_matchup?type=batter&teamPitching={team_pitching}&teamBatting={team_batting}&player_id={pitcher_id}'
            driver.get(url)

            # Find the player table
            player_table = driver.find_element(by=By.ID, value='player_table')

            # Find the team table
            team_table = driver.find_element(by=By.ID, value='team_table')

            # Extract data from player table
            player_data = player_table.text

            # Extract data from team table
            team_data = team_table.text

            # Store the results in the dictionary
            if team_pitching not in results:
                results[team_pitching] = {}
            if team_batting not in results[team_pitching]:
                results[team_pitching][team_batting] = []
            results[team_pitching][team_batting].append({
                'pitcher_id': pitcher_id,
                'player_data': player_data,
                'team_data': team_data
            })
            
driver.quit()
print(results)

In [ ]:
all_matchups = {108: {109: [{'pitcher_id': '667755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoriano, José 34 30 6 0 0 2 7 20.6 30.9 4 11.8 0.2 0.4 0.307 0.252 0.533 0.37 92.9 -2.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerdomo, Geraldo 4 3 1 0 0 0 1 25 0 1 25 0.333 0.333 0.396 0.177 0.199 0.298 88.1 -14.5\nPederson, Joc 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.291 0.98 0.516 91.4 -12.8\nMoreno, Gabriel 3 1 0 0 0 0 0 0 33.3 2 66.7 0 0 0.461 0.34 0.94 0.64 97.5 27\nMarte, Ketel 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.222 0.388 0.427 0.357 99.8 2.7\nAlexander, Blaze 3 3 0 0 0 0 2 66.7 62.5 0 0 0 0 0 0.047 0.052 0.045 69.8 -5\nMcCarthy, Jake 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.047 0.052 0.042 75.7 -27.5\nGurriel Jr., Lourdes 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.12 0.128 0.109 100 -18\nWalker, Christian 3 3 2 0 0 1 0 0 25 0 0 0.667 1.667 0.991 0.4 1.408 0.729 103.2 15.7\nSuárez, Eugenio 3 2 1 0 0 1 0 0 50 1 33.3 0.5 2 0.9 0.355 1.091 0.62 99.6 29\nCarroll, Corbin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.476 0.573 0.451 89.5 -8.3\nGrichuk, Randal 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '519326', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrickland, Hunter 50 48 16 3 0 3 6 12 28.6 2 4 0.333 0.583 0.398 0.328 0.597 0.403 89.2 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPederson, Joc 11 11 4 0 0 2 1 9.1 15 0 0 0.364 0.909 0.52 0.309 0.869 0.481 93.9 14.9\nSuárez, Eugenio 8 6 2 1 0 0 0 0 30.8 2 25 0.333 0.5 0.438 0.329 0.528 0.449 90.3 38.7\nGrichuk, Randal 7 7 1 0 0 1 2 28.6 50 0 0 0.143 0.571 0.283 0.25 0.677 0.378 88 25.8\nMarte, Ketel 6 6 2 0 0 0 1 16.7 12.5 0 0 0.333 0.333 0.293 0.277 0.342 0.277 85.3 11\nNewman, Kevin 5 5 2 0 0 0 0 0 30 0 0 0.4 0.4 0.352 0.454 0.592 0.449 90.4 8\nWalker, Christian 3 3 1 1 0 0 1 33.3 62.5 0 0 0.333 0.667 0.42 0.258 0.324 0.251 96.5 24\nBarnhart, Tucker 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.176 0.274 0.371 0.279 75 20.3\nGurriel Jr., Lourdes 2 2 2 0 0 0 0 0 25 0 0 1 1 0.884 0.762 0.865 0.717 86.4 12\nPerdomo, Geraldo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.294 0.623 0.384 97.4 23\nMoreno, Gabriel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.709 0.937 0.706 101 15\nCarroll, Corbin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.02 0.04 0.03 84.5 45'}, {'pitcher_id': '663776', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSandoval, Patrick 33 30 11 2 0 1 9 27.3 25.8 3 9.1 0.367 0.533 0.413 0.283 0.41 0.337 84.5 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 17 16 5 0 0 1 6 35.3 22.9 1 5.9 0.313 0.5 0.366 0.236 0.395 0.295 86.7 22.5\nWalker, Christian 5 5 1 1 0 0 1 20 20 0 0 0.2 0.4 0.248 0.22 0.299 0.229 79.7 3.5\nMarte, Ketel 3 3 2 0 0 0 1 33.3 50 0 0 0.667 0.667 0.586 0.427 0.481 0.408 84.8 -25\nGrichuk, Randal 3 3 1 0 0 0 0 0 33.3 0 0 0.333 0.333 0.294 0.611 0.817 0.618 86.5 23\nBarnhart, Tucker 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.435 0.133 0.15 0.125 82.9 3\nGurriel Jr., Lourdes 2 1 1 1 0 0 0 0 0 1 50 1 2 0.975 0.252 0.294 0.46 76.2 5\nPederson, Joc 1 0 0 0 0 0 0 0 0 1 100 0.699 0.699'}, {'pitcher_id': '608032', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstévez, Carlos 63 55 10 2 0 1 17 27 25 6 9.5 0.182 0.273 0.245 0.214 0.309 0.275 91.8 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Ketel 18 13 4 2 0 0 2 11.1 11.1 4 22.2 0.308 0.462 0.427 0.216 0.27 0.346 89.9 6.4\nPederson, Joc 14 12 3 0 0 1 6 42.9 29.6 1 7.1 0.25 0.5 0.316 0.257 0.499 0.325 102.4 11.3\nWalker, Christian 12 11 1 0 0 0 4 33.3 37 1 8.3 0.091 0.091 0.132 0.236 0.318 0.274 96 2.6\nPerdomo, Geraldo 5 5 0 0 0 0 1 20 14.3 0 0 0 0 0 0.076 0.117 0.083 87.4 25.5\nMcCarthy, Jake 2 2 1 0 0 0 0 0 10 0 0 0.5 0.5 0.441 0.376 0.425 0.352 95.1 -8.5\nBarnhart, Tucker 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.124 0.133 0.117 101.4 -4\nNewman, Kevin 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.279 0.349 0.292 70.3 13.8\nGrichuk, Randal 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.038 0.04 0.034 82.9 -21\nCarroll, Corbin 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.226 0.257 0.216 106.2 -3\nSuárez, Eugenio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.271 0.323 0.244 89.8 -5\nGurriel Jr., Lourdes 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '472610', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Luis 60 50 15 3 0 1 13 21.7 27.1 9 15 0.3 0.42 0.368 0.256 0.403 0.346 84.2 0.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPederson, Joc 11 9 3 1 0 1 1 9.1 11.1 2 18.2 0.333 0.778 0.508 0.411 1.004 0.61 90.4 -4.9\nGrichuk, Randal 9 9 3 1 0 0 3 33.3 26.7 0 0 0.333 0.444 0.335 0.145 0.162 0.135 82.4 25.2\nWalker, Christian 8 6 2 0 0 0 0 0 20 2 25 0.333 0.333 0.393 0.362 0.446 0.448 87.3 2.3\nMarte, Ketel 6 4 1 0 0 0 1 16.7 25 2 33.3 0.25 0.25 0.377 0.2 0.355 0.387 94 3\nNewman, Kevin 6 5 1 0 0 0 2 33.3 50 1 16.7 0.2 0.2 0.262 0.096 0.103 0.188 71 -28.7\nSuárez, Eugenio 5 5 1 1 0 0 3 60 40 0 0 0.2 0.4 0.248 0.148 0.21 0.152 81.2 33.5\nCarroll, Corbin 5 4 1 0 0 0 1 20 25 1 20 0.25 0.25 0.315 0.217 0.225 0.297 91.8 -23\nBarnhart, Tucker 3 3 0 0 0 0 1 33.3 42.9 0 0 0 0 0 0.097 0.111 0.096 84.9 -11\nMcCarthy, Jake 2 1 1 0 0 0 0 0 33.3 1 50 1 1 0.787 0.72 0.765 0.671 76.4 15\nMoreno, Gabriel 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.576 0.618 0.533 75.1 -18\nGurriel Jr., Lourdes 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.335 0.387 0.317 104.3 6\nPerdomo, Geraldo 1 0 0 0 0 0 0 0 0 0 0 29.3 -2'}, {'pitcher_id': '660761', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, José 43 40 16 3 2 4 6 14 21 2 4.7 0.4 0.875 0.537 0.368 0.693 0.463 91 11.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 14 13 4 0 0 1 3 21.4 24.1 1 7.1 0.308 0.538 0.377 0.291 0.613 0.395 90.3 18.3\nGurriel Jr., Lourdes 7 7 4 1 0 1 0 0 28.6 0 0 0.571 1.143 0.702 0.4 0.529 0.404 83.6 8\nGrichuk, Randal 7 7 3 1 1 1 1 14.3 18.2 0 0 0.429 1.286 0.702 0.485 1.078 0.657 101.1 21\nWalker, Christian 4 4 0 0 0 0 2 50 37.5 0 0 0 0 0 0.126 0.139 0.114 81.2 -2\nMoreno, Gabriel 2 2 1 1 0 0 0 0 16.7 0 0 0.5 1 0.635 0.365 0.627 0.411 99.8 4\nNewman, Kevin 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.936 0.567 1.454 0.827 88.4 6\nMarte, Ketel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.218 0.241 0.45 86.3 -2\nBarnhart, Tucker 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.051 0.046 75.6 -28\nCarroll, Corbin 1 1 1 0 1 0 0 0 0 0 0 1 3 1.612 0.729 1.027 0.728 106.2 14\nAlexander, Blaze 1 0 0 0 0 0 0 0 0 0 0 0.724 0.724\nPerdomo, Geraldo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.814 1.014 0.803 103.4 10'}, {'pitcher_id': '542585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCisnero, José 14 14 2 1 0 0 4 28.6 26.1 0 0 0.143 0.214 0.152 0.161 0.225 0.166 89.7 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 6 6 1 0 0 0 3 50 45.5 0 0 0.167 0.167 0.147 0.175 0.237 0.179 92.1 28\nNewman, Kevin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.239 0.202 94.9 -9\nWalker, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.151 0.47 0.258 93.2 32\nGrichuk, Randal 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.414 0.21 0.259 0.198 82.6 1\nGurriel Jr., Lourdes 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.002 0.002 0.002 90 80'}, {'pitcher_id': '643256', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCimber, Adam 18 15 6 2 0 0 4 22.2 13.2 3 16.7 0.4 0.533 0.45 0.298 0.474 0.392 92 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 4 3 2 0 0 0 1 25 8.3 1 25 0.667 0.667 0.61 0.244 0.272 0.342 89.1 -9.5\nPederson, Joc 3 2 1 1 0 0 1 33.3 20 1 33.3 0.5 1 0.646 0.345\nSuárez, Eugenio 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.056 0.072 0.053 89.7 23.5\nWalker, Christian 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.163 0.196 0.151 83.6 23\nMarte, Ketel 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69\nGurriel Jr., Lourdes 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.568 0.677 0.535 105.4 0\nNewman, Kevin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.477 0.569 0.459 80.9 28\nMoreno, Gabriel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.86 2.725 1.471 106.2 22'}, {'pitcher_id': '542881', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tyler 137 125 38 13 0 5 28 20.4 22.6 8 5.8 0.304 0.528 0.376 0.254 0.389 0.307 86.3 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Ketel 35 30 13 6 0 2 2 5.7 6.7 3 8.6 0.433 0.833 0.552 0.322 0.55 0.414 89.3 18\nWalker, Christian 33 29 7 3 0 1 6 18.2 28.1 2 6.1 0.241 0.448 0.319 0.264 0.462 0.334 88.5 28.2\nGrichuk, Randal 14 13 4 3 0 1 5 35.7 45.2 1 7.1 0.308 0.769 0.463 0.251 0.367 0.296 89.6 10.9\nSuárez, Eugenio 11 11 2 0 0 1 4 36.4 20 0 0 0.182 0.455 0.26 0.127 0.212 0.144 76.4 27\nPederson, Joc 14 13 5 1 0 0 3 21.4 30.4 1 7.1 0.385 0.462 0.389 0.205 0.25 0.23 88.8 18.8\nPerdomo, Geraldo 10 9 2 0 0 0 2 20 26.7 1 10 0.222 0.222 0.246 0.275 0.323 0.305 81.5 10.6\nMcCarthy, Jake 11 11 1 0 0 0 4 36.4 13.3 0 0 0.091 0.091 0.08 0.11 0.122 0.101 80.2 -4.1\nBarnhart, Tucker 4 4 2 0 0 0 1 25 30 0 0 0.5 0.5 0.442 0.352 0.394 0.33 85 4.5\nGurriel Jr., Lourdes 3 3 0 0 0 0 1 33.3 11.1 0 0 0 0 0 0.121 0.146 0.117 65.1 10\nMoreno, Gabriel 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.905 1.047 0.862 87.4 14.5'}, {'pitcher_id': '670871', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672282', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDetmers, Reid 26 24 5 3 0 0 8 30.8 28.8 2 7.7 0.208 0.333 0.265 0.214 0.322 0.266 94.4 19.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 12 12 2 1 0 0 4 33.3 34.5 0 0 0.167 0.25 0.177 0.161 0.24 0.172 92.6 16.8\nGurriel Jr., Lourdes 6 4 2 2 0 0 0 0 0 2 33.3 0.5 1 0.648 0.28 0.596 0.474 96.4 27\nMarte, Ketel 3 3 0 0 0 0 2 66.7 30 0 0 0 0 0 0.001 0.002 0.001 93.2 48\nWalker, Christian 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.25 0.293 0.235 98.9 10\nBarnhart, Tucker 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.442 0.671 0.789 0.648 96.1 6.5'}, {'pitcher_id': '681217', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSilseth, Chase 5 5 1 1 0 0 2 40 44.4 0 0 0.2 0.4 0.252 0.203 0.263 0.202 82 -8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.019 0.019 0.018 54 -30\nGurriel Jr., Lourdes 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.478 0.629 0.479 96 3'}, {'pitcher_id': '672710', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, Roansy 31 26 7 3 0 1 5 16.1 18.8 5 16.1 0.269 0.5 0.384 0.285 0.404 0.361 91.7 20.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Christian 6 6 0 0 0 0 4 66.7 41.7 0 0 0 0 0 0.09 0.103 0.084 101.7 24.5\nMarte, Ketel 7 7 2 2 0 0 1 14.3 18.2 0 0 0.286 0.571 0.36 0.366 0.465 0.363 93.3 21\nPerdomo, Geraldo 4 2 1 0 0 0 0 0 0 2 50 0.5 0.5 0.567 0.245 0.282 0.463 100.3 31.5\nGurriel Jr., Lourdes 5 5 1 1 0 0 0 0 11.1 0 0 0.2 0.4 0.252 0.508 0.69 0.516 88.1 14\nCarroll, Corbin 3 2 1 0 0 1 0 0 0 1 33.3 0.5 2 0.9 0.254 0.889 0.544 97.3 32.5\nGrichuk, Randal 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.011 0.015 0.35 88.1 60\nMoreno, Gabriel 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nNewman, Kevin 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.889 0.303 0.305 0.273 77.1 10\nPederson, Joc 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.386 0.44 0.364 80 9\nSuárez, Eugenio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.075 0.089 0.07 80.7 -14'}, {'pitcher_id': '519043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoore, Matt 61 54 15 5 0 2 12 19.7 20.4 7 11.5 0.278 0.481 0.363 0.31 0.565 0.405 92.3 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPederson, Joc 18 14 3 1 0 1 5 27.8 23.1 4 22.2 0.214 0.5 0.383 0.26 0.628 0.437 96.7 8.6\nGrichuk, Randal 13 12 3 1 0 0 2 15.4 37.5 1 7.7 0.25 0.333 0.284 0.252 0.417 0.315 89.7 16.8\nSuárez, Eugenio 11 11 2 0 0 0 2 18.2 15.8 0 0 0.182 0.182 0.16 0.372 0.573 0.406 93.9 14.2\nMarte, Ketel 10 10 6 3 0 1 1 10 6.7 0 0 0.6 1.2 0.756 0.38 0.781 0.48 93.8 18.8\nGurriel Jr., Lourdes 4 4 0 0 0 0 1 25 11.1 0 0 0 0 0 0.343 0.421 0.331 83.4 20\nWalker, Christian 3 2 1 0 0 0 1 33.3 22.2 1 33.3 0.5 0.5 0.524 0.288 0.311 0.405 108 4\nBarnhart, Tucker 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nCarroll, Corbin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.237 0.288 0.232 61.6 46'}, {'pitcher_id': '656288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanning, Griffin 56 50 10 2 0 3 10 17.9 26.7 5 8.9 0.2 0.42 0.309 0.248 0.454 0.341 88.4 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Christian 9 8 4 0 0 2 3 33.3 30.4 1 11.1 0.5 1.25 0.737 0.381 1.075 0.61 88.2 18\nMarte, Ketel 6 5 1 1 0 0 2 33.3 36.4 1 16.7 0.2 0.4 0.328 0.195 0.238 0.274 91.1 11.7\nGurriel Jr., Lourdes 8 8 2 0 0 1 1 12.5 20 0 0 0.25 0.625 0.361 0.154 0.423 0.238 86.6 19.4\nPederson, Joc 6 5 0 0 0 0 1 16.7 35.7 1 16.7 0 0 0.115 0.149 0.172 0.236 86.7 8.7\nGrichuk, Randal 6 6 1 0 0 0 1 16.7 30.8 0 0 0.167 0.167 0.145 0.271 0.303 0.25 80.7 8.8\nMcCarthy, Jake 5 4 0 0 0 0 0 0 12.5 1 20 0 0 0.139 0.248 0.543 0.4 90.5 29.8\nSuárez, Eugenio 4 3 0 0 0 0 2 50 50 0 0 0 0 0.181 0.001 0.001 0.182 94.6 50\nCarroll, Corbin 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.368 0.419 0.342 89.8 -2\nNewman, Kevin 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.535 0.408 0.478 0.386 95.1 19.3\nPerdomo, Geraldo 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.344 0.396 0.45 79.5 -25\nMoreno, Gabriel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.18 0.286 0.193 95.5 7.5'}, {'pitcher_id': '608334', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFulmer, Carson 18 13 2 2 0 0 3 16.7 33.3 3 16.7 0.154 0.308 0.297 0.207 0.259 0.305 89.7 2.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 3 3 1 1 0 0 1 33.3 22.2 0 0 0.333 0.667 0.423 0.307 0.426 0.319 91.1 -14.5\nCarroll, Corbin 2 1 1 1 0 0 0 0 33.3 1 50 1 2 0.981 0.697 0.729 0.66 69.2 18\nMarte, Ketel 2 1 0 0 0 0 0 0 66.7 1 50 0 0 0.346 0.037 0.127 0.378 99.2 44\nAlexander, Blaze 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.108 0.128 0.104 89.5 -26\nPerdomo, Geraldo 2 1 0 0 0 0 0 0 50 1 50 0 0 0.346 0.339 0.363 0.499 99.7 -3\nMoreno, Gabriel 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSuárez, Eugenio 2 1 0 0 0 0 1 50 100 0 0 0 0 0.362 0.362\nWalker, Christian 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.126 0.054 91.6 35.5\nGurriel Jr., Lourdes 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.384 0.433 0.357 102 -2\nNewman, Kevin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.058 0.071 72.8 -15'}], 110: [{'pitcher_id': '667755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoriano, José 13 12 1 0 0 0 2 15.4 31.8 1 7.7 0.083 0.083 0.122 0.223 0.264 0.249 85.9 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHays, Austin 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nHenderson, Gunnar 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.097 0.115 0.091 87.1 -23\nSantander, Anthony 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.453 0.551 0.434 85.7 28.5\nMcCann, James 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.237 0.273 0.223 97.7 -5\nWestburg, Jordan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.157 0.159 0.144 68.7 -59\nMountcastle, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.096 0.103 0.086 90.8 -19\nMullins, Cedric 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.078 0.051 83.1 35\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.84 1.007 0.796 91.7 16\nCowser, Colton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.052 0.053 0.047 81.3 -19'}, {'pitcher_id': '519326', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrickland, Hunter 6 6 1 0 0 1 1 16.7 18.2 0 0 0.167 0.667 0.334 0.271 0.738 0.414 95.7 22.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.01 0.017 0.012 91.6 53\nWestburg, Jordan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nO'Hearn, Ryan 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.94 3.654 1.849 106.3 27\nSantander, Anthony 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.586 0.65 0.54 98.8 7\nUrías, Ramón 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.078 0.092 0.073 90.1 -26"}, {'pitcher_id': '663776', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSandoval, Patrick 65 54 18 2 0 0 11 16.9 30.3 10 15.4 0.333 0.37 0.363 0.284 0.406 0.359 88.7 10.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMountcastle, Ryan 10 7 2 0 0 0 3 30 26.7 3 30 0.286 0.286 0.385 0.257 0.313 0.382 78.2 32.5\nUrías, Ramón 9 8 3 0 0 0 1 11.1 30 1 11.1 0.375 0.375 0.371 0.347 0.632 0.454 79.3 -2.5\nHays, Austin 11 11 5 1 0 0 3 27.3 39.1 0 0 0.455 0.545 0.434 0.25 0.291 0.241 92.6 -9.1\nRutschman, Adley 8 6 2 0 0 0 0 0 16.7 2 25 0.333 0.333 0.394 0.313 0.344 0.387 88 24.2\nSantander, Anthony 8 6 3 0 0 0 1 12.5 23.1 1 12.5 0.5 0.5 0.418 0.452 0.886 0.542 97.1 13\nMcCann, James 5 4 1 0 0 0 0 0 36.4 1 20 0.25 0.25 0.313 0.315 0.457 0.404 97 2.8\nMateo, Jorge 4 3 1 1 0 0 1 25 30 1 25 0.333 0.667 0.491 0.23 0.244 0.332 94.1 -5\nHenderson, Gunnar 5 4 0 0 0 0 1 20 30 1 20 0 0 0.138 0.112 0.129 0.219 85.1 40\nMullins, Cedric 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.028 0.04 0.031 86.8 14.7\nWestburg, Jordan 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.969 1.077 0.898 81 17'}, {'pitcher_id': '608032', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstévez, Carlos 17 16 4 1 0 1 6 35.3 33.3 1 5.9 0.25 0.5 0.34 0.27 0.586 0.377 90.7 0.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHenderson, Gunnar 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.123 0.124 0.114 85.2 -59\nRutschman, Adley 2 1 0 0 0 0 1 50 25 1 50 0 0 0.348 0.348\nMullins, Cedric 3 3 1 1 0 0 0 0 42.9 0 0 0.333 0.667 0.415 0.251 0.44 0.29 89.5 32\nO'Hearn, Ryan 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.985 0.816 2.271 1.258 102.9 24.3\nMountcastle, Ryan 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.294 0.19 0.23 0.189 88.1 -13.5\nMcCann, James 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHays, Austin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSantander, Anthony 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.18 0.18 0.164 68.1 -79"}, {'pitcher_id': '472610', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Luis 23 23 10 3 0 1 4 17.4 23.9 0 0 0.435 0.696 0.48 0.336 0.543 0.374 93.7 1.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWestburg, Jordan 4 4 1 0 0 0 0 0 30 0 0 0.25 0.25 0.222 0.22 0.249 0.202 92.7 -11\nSantander, Anthony 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.29 0.097 0.158 0.108 91.1 3.5\nHays, Austin 3 3 2 0 0 1 1 33.3 50 0 0 0.667 1.667 0.964 0.388 0.767 0.488 101.4 21.5\nMullins, Cedric 4 4 2 2 0 0 1 25 14.3 0 0 0.5 1 0.628 0.576 0.704 0.556 104.2 8\nUrías, Ramón 3 3 1 0 0 0 0 0 40 0 0 0.333 0.333 0.294 0.364 1.1 0.597 87.3 -18\nMountcastle, Ryan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.591 0.684 0.563 106.2 4\nCowser, Colton 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nO'Hearn, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.127 0.311 0.167 91.6 26\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.11 0.11 0.103 67.7 2\nHenderson, Gunnar 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.874 1.169 0.887 104.7 13\nMcCann, James 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.295 0.336 0.266 80.4 6"}, {'pitcher_id': '660761', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, José 51 44 12 5 0 1 7 13.7 17.6 6 11.8 0.273 0.455 0.363 0.27 0.474 0.367 88.2 18.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 8 6 2 1 0 0 0 0 26.3 2 25 0.333 0.5 0.441 0.191 0.247 0.314 71.2 30\nMateo, Jorge 7 6 1 1 0 0 2 28.6 6.3 1 14.3 0.167 0.333 0.28 0.173 0.583 0.364 94.5 35.8\nSantander, Anthony 8 6 2 0 0 1 1 12.5 18.2 1 12.5 0.333 0.833 0.546 0.369 0.981 0.589 99 20.2\nUrías, Ramón 4 4 1 0 0 0 0 0 14.3 0 0 0.25 0.25 0.221 0.441 0.718 0.503 74.4 -3\nRutschman, Adley 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.394 0.343 0.378 0.412 90.9 -4\nMountcastle, Ryan 6 6 1 1 0 0 1 16.7 25 0 0 0.167 0.333 0.21 0.195 0.33 0.22 90.2 22.8\nMcCann, James 4 4 2 2 0 0 1 25 14.3 0 0 0.5 1 0.622 0.271 0.304 0.25 97.6 22\nHays, Austin 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.184 0.247 0.188 94.2 15.5\nCowser, Colton 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nHenderson, Gunnar 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.331 0.357 0.306 90.7 23.7\nWestburg, Jordan 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.441 0.477 0.399 94.6 5'}, {'pitcher_id': '542585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCisnero, José 41 30 5 1 0 2 8 19.5 23.9 10 24.4 0.167 0.4 0.342 0.242 0.459 0.397 92.9 12.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 6 5 1 0 0 1 1 16.7 16.7 1 16.7 0.2 0.8 0.463 0.211 0.636 0.408 92.3 7.8\nHays, Austin 5 2 2 0 0 1 0 0 0 3 60 1 2.5 0.991 0.597 1.721 0.795 89.6 23\nO'Hearn, Ryan 6 6 2 1 0 0 2 33.3 28.6 0 0 0.333 0.5 0.358 0.297 0.42 0.31 85.6 28.5\nRutschman, Adley 3 0 0 0 0 0 0 0 50 3 100 0.692 0.692\nMountcastle, Ryan 3 1 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0 0 0.397 101.1 38\nUrías, Ramón 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.158 0.164 0.327 105.7 -12\nMcCann, James 4 4 0 0 0 0 0 0 28.6 0 0 0 0 0 0.267 0.425 0.291 92.6 6.3\nSantander, Anthony 3 3 0 0 0 0 0 0 12.5 0 0 0 0 0 0.106 0.299 0.159 95.4 24.7\nWestburg, Jordan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.377 0.426 0.355 82.1 21\nCowser, Colton 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nMateo, Jorge 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.261 0.286 0.237 100.1 -16.7\nStowers, Kyle 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHenderson, Gunnar 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0"}, {'pitcher_id': '643256', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCimber, Adam 50 43 9 2 0 0 10 20 12.7 4 8 0.209 0.256 0.249 0.256 0.397 0.34 90.3 11.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMountcastle, Ryan 9 8 2 1 0 0 2 22.2 7.1 0 0 0.25 0.375 0.238 0.209 0.543 0.399 95.8 10.6\nMcCann, James 9 8 2 0 0 0 0 0 0 1 11.1 0.25 0.25 0.273 0.128 0.152 0.188 88.5 -5\nMullins, Cedric 6 5 0 0 0 0 3 50 41.7 0 0 0 0 0.12 0.002 0.003 0.122 82.5 63.5\nHenderson, Gunnar 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.295 0.221 0.248 0.208 100.6 6\nMateo, Jorge 4 2 1 0 0 0 0 0 0 1 25 0.5 0.5 0.525 0.222 0.242 0.367 65 5\nHays, Austin 5 5 0 0 0 0 1 20 0 0 0 0 0 0 0.39 0.605 0.423 94.4 11.8\nSantander, Anthony 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0.396 0.682 0.465 92 27.8\nRutschman, Adley 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.88 1.086 0.778 100.2 11\nCowser, Colton 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.977 1.118 0.804 89.3 14\nUrías, Ramón 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.272 0.306 0.251 103.6 4\nO'Hearn, Ryan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nStowers, Kyle 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.152 0.152 0.141 87.9 -5\nWestburg, Jordan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.729 0.855 0.696 102.3 9"}, {'pitcher_id': '542881', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tyler 45 37 8 2 0 2 10 22.2 27.4 6 13.3 0.216 0.432 0.334 0.249 0.447 0.351 88 26.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMountcastle, Ryan 6 4 0 0 0 0 2 33.3 18.8 1 16.7 0 0 0.115 0.004 0.007 0.124 78.5 55.7\nSantander, Anthony 6 3 2 0 0 1 0 0 37.5 3 50 0.667 1.667 0.829 0.602 1.563 0.789 102.7 27.7\nRutschman, Adley 6 5 1 0 0 0 0 0 0 1 16.7 0.2 0.2 0.264 0.283 0.347 0.342 84.7 28.2\nMateo, Jorge 6 6 0 0 0 0 2 33.3 33.3 0 0 0 0 0 0.23 0.334 0.239 84.3 11\nMcCann, James 5 5 0 0 0 0 2 40 38.5 0 0 0 0 0 0.029 0.029 0.026 83.4 22.7\nHenderson, Gunnar 5 4 3 1 0 1 1 20 40 0 0 0.75 1.75 0.992 0.392 0.695 0.519 92.3 15\nHays, Austin 3 3 2 1 0 0 0 0 16.7 0 0 0.667 1 0.709 0.417 0.611 0.446 92.4 26\nWestburg, Jordan 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.301 0.843 0.471 90.3 15\nMullins, Cedric 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.237 0.261 0.215 87.8 38\nCowser, Colton 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.346 0.346'}, {'pitcher_id': '670871', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZuñiga, Guillermo 10 8 2 0 1 1 1 10 22.2 2 20 0.25 0.875 0.508 0.223 0.477 0.372 98.7 22.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRutschman, Adley 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.003 0.009 0.348 96.7 50\nHenderson, Gunnar 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.806 0.433 0.509 0.409 105.5 -7.5\nUrías, Ramón 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nMountcastle, Ryan 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.317 0.712 0.423 98.4 23\nSantander, Anthony 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.314 1.157 0.604 98 32\nMateo, Jorge 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHays, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.198 0.125 86.5 27\nO'Hearn, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.183 0.723 0.364 100.3 38"}, {'pitcher_id': '672282', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDetmers, Reid 98 88 16 3 1 2 20 20.4 25.8 9 9.2 0.182 0.307 0.259 0.229 0.367 0.299 87.9 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHenderson, Gunnar 10 8 3 1 1 0 2 20 25 2 20 0.375 0.75 0.513 0.224 0.278 0.312 90.1 12\nMullins, Cedric 11 10 0 0 0 0 4 36.4 24 0 0 0 0 0.066 0.169 0.241 0.227 86 28.8\nSantander, Anthony 12 9 0 0 0 0 2 16.7 35.3 3 25 0 0 0.172 0.116 0.276 0.294 84.5 6\nMateo, Jorge 12 11 2 0 0 0 1 8.3 21.1 1 8.3 0.182 0.182 0.205 0.241 0.298 0.271 85.9 5.5\nMountcastle, Ryan 11 11 3 0 0 0 2 18.2 26.7 0 0 0.273 0.273 0.241 0.326 0.358 0.301 88.1 -1.4\nRutschman, Adley 10 9 3 1 0 0 1 10 14.3 1 10 0.333 0.444 0.374 0.351 0.664 0.454 91.4 11\nHays, Austin 9 9 1 0 0 0 3 33.3 25 0 0 0.111 0.111 0.098 0.124 0.162 0.124 81.4 33.2\nWestburg, Jordan 8 6 1 1 0 0 2 25 41.7 2 25 0.167 0.333 0.329 0.297 0.478 0.418 93 19.8\nUrías, Ramón 7 7 1 0 0 0 0 0 0 0 0 0.143 0.143 0.126 0.226 0.267 0.219 89.4 5\nMcCann, James 5 5 1 0 0 1 2 40 44.4 0 0 0.2 0.8 0.417 0.166 0.519 0.278 89.2 30.7\nCowser, Colton 3 3 1 0 0 1 1 33.3 42.9 0 0 0.333 1.333 0.695 0.296 1.072 0.557 94.7 34.5'}, {'pitcher_id': '681217', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSilseth, Chase 26 25 11 3 0 2 5 19.2 20 1 3.8 0.44 0.8 0.532 0.303 0.506 0.363 84.8 6.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 5 5 1 0 0 0 1 20 12.5 0 0 0.2 0.2 0.177 0.166 0.187 0.164 77.7 -11.5\nRutschman, Adley 4 4 2 0 0 1 0 0 0 0 0 0.5 1.25 0.739 0.403 0.741 0.492 82.4 15\nSantander, Anthony 4 4 2 1 0 0 2 50 22.2 0 0 0.5 0.75 0.536 0.316 0.741 0.452 89 31\nMateo, Jorge 4 4 2 0 0 0 1 25 20 0 0 0.5 0.5 0.442 0.336 0.383 0.322 92 -3\nHenderson, Gunnar 2 1 1 1 0 0 0 0 50 1 50 1 2 0.97 0.403 0.599 0.551 95.9 18\nHays, Austin 4 4 1 0 0 0 0 0 16.7 0 0 0.25 0.25 0.221 0.292 0.307 0.265 74.1 -3.5\nMountcastle, Ryan 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.631 0.385 0.879 0.532 110 18\nO'Hearn, Ryan 1 1 1 0 0 1 0 0 33.3 0 0 1 4 2.004 0.18 0.658 0.324 100 38"}, {'pitcher_id': '672710', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, Roansy 33 32 5 0 0 2 2 6.1 13.7 1 3 0.156 0.344 0.223 0.254 0.544 0.342 92.4 23', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 4 4 0 0 0 0 1 25 30 0 0 0 0 0 0.214 0.237 0.198 95.9 12.3\nRutschman, Adley 4 3 2 0 0 1 1 25 20 1 25 0.667 1.667 0.896 0.538 0.992 0.658 105.8 15\nMountcastle, Ryan 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.26 0.784 0.423 94.2 17\nMateo, Jorge 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.231 0.164 86.1 16.8\nHenderson, Gunnar 4 4 1 0 0 1 0 0 33.3 0 0 0.25 1 0.501 0.254 0.903 0.466 95.4 39.3\nHays, Austin 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.174 0.204 0.166 84.2 4.3\nO'Hearn, Ryan 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.38 0.244 89.5 25\nSantander, Anthony 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.222 0.406 0.956 0.561 99.1 30\nWestburg, Jordan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.011 0.014 0.011 87.3 51\nMcCann, James 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 81.7 68"}, {'pitcher_id': '519043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoore, Matt 17 14 4 1 0 1 2 11.8 22.7 2 11.8 0.286 0.571 0.424 0.397 0.654 0.498 83.1 0.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 8 8 1 0 0 0 2 25 13.6 0 0 0.125 0.125 0.11 0.343 0.384 0.33 86.3 -8.5\nRutschman, Adley 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.829 0.867 0.717 71.1 29\nMullins, Cedric 2 1 1 1 0 0 0 0 0 1 50 1 2 0.975 0.649 0.957 0.69 102.6 15\nMateo, Jorge 2 1 0 0 0 0 0 0 50 0 0 0 0 0.36 0.076 0.083 0.397 62.6 -3\nHays, Austin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.227 0.229 0.209 63.3 -67\nHenderson, Gunnar 1 1 1 0 0 1 0 0 50 0 0 1 4 2.085 0.966 3.855 1.944 109.4 31\nMountcastle, Ryan 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.063 0.089 0.066 70.6 57'}, {'pitcher_id': '656288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanning, Griffin 75 67 21 6 1 2 9 12 16.2 6 8 0.313 0.522 0.383 0.288 0.461 0.35 91.3 22.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantander, Anthony 12 12 3 2 0 0 2 16.7 11.1 0 0 0.25 0.417 0.283 0.216 0.286 0.212 93 38.5\nO'Hearn, Ryan 9 7 2 1 0 0 1 11.1 25 2 22.2 0.286 0.429 0.394 0.329 0.456 0.421 90.7 -1\nRutschman, Adley 9 9 4 1 0 0 0 0 11.1 0 0 0.444 0.556 0.434 0.411 0.588 0.428 87.2 21.9\nMullins, Cedric 9 8 1 0 0 0 3 33.3 17.6 1 11.1 0.125 0.125 0.175 0.1 0.125 0.163 85.3 24.4\nHenderson, Gunnar 9 8 4 0 1 1 1 11.1 15.8 0 0 0.5 1.125 0.603 0.492 0.92 0.532 99.9 14\nMountcastle, Ryan 8 6 3 2 0 0 0 0 0 2 25 0.5 0.833 0.601 0.409 0.736 0.534 100.2 15.8\nHays, Austin 7 6 2 0 0 1 1 14.3 18.8 1 14.3 0.333 0.833 0.511 0.254 0.57 0.392 90.5 35.6\nMcCann, James 3 2 1 0 0 0 1 33.3 66.7 0 0 0.5 0.5 0.53 0.064 0.093 0.288 75.7 47\nUrías, Ramón 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.235 0.417 0.273 89.4 8\nWestburg, Jordan 4 4 1 0 0 0 0 0 20 0 0 0.25 0.25 0.222 0.241 0.287 0.234 88.3 40.3\nCowser, Colton 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.105 0.087 75.9 -5.5"}, {'pitcher_id': '608334', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFulmer, Carson 11 9 1 0 0 0 5 45.5 28 1 9.1 0.111 0.111 0.21 0.115 0.137 0.218 85.7 -2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 4 4 0 0 0 0 4 100 27.3 0 0 0 0 0 0\nMountcastle, Ryan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCowser, Colton 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.463 0.538 0.437 98.4 2\nMullins, Cedric 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nHenderson, Gunnar 1 0 0 0 0 0 0 0 50 0 0 0.724 0.724\nSantander, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.229 0.206 55.9 -51\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.158 0.082 98.4 43\nWestburg, Jordan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.288 0.306 0.259 90.2 -3'}]}, 109: {108: [{'pitcher_id': '657044', 'player_data': '', 'team_data': ''}, {'pitcher_id': '669459', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Kyle 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.019 0.019 0.017 57 75', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuillorme, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.037 0.033 28.6 69\nPillar, Kevin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 85.4 81'}, {'pitcher_id': '677944', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCecconi, Slade 18 18 10 3 0 2 4 22.2 20.5 0 0 0.556 1.056 0.69 0.298 0.568 0.364 94.3 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThaiss, Matt 2 2 0 0 0 0 2 100 28.6 0 0 0 0 0 0\nCalhoun, Willie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.248 0.288 0.239 94.1 30\nGuillorme, Luis 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.095 0.112 0.087 89.4 22.5\nAdell, Jo 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.043 0.5 2 1 108.1 25\nNeto, Zach 2 2 2 0 0 1 0 0 20 0 0 1 2.5 1.487 0.551 0.903 0.617 101.4 22\nRengifo, Luis 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.079 0.084 0.072 85.9 -21.5\nMoniak, Mickey 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.444 0.354 0.39 0.321 94.9 10\nSchanuel, Nolan 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.079 0.39 0.785 0.499 100 9.5\nPillar, Kevin 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.467 0.546 0.441 87.6 39'}, {'pitcher_id': '679885', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Justin 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.296 0.235 0.271 0.225 92 0.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStefanic, Michael 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRengifo, Luis 0 0 0 0 0 0 0 0\nAdell, Jo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.142 0.159 0.132 77.1 -2\nNeto, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.563 0.653 0.542 106.9 3'}, {'pitcher_id': '686796', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '612434', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastro, Miguel 21 19 2 0 0 0 5 23.8 25 0 0 0.105 0.105 0.153 0.196 0.235 0.237 88.7 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 11 11 2 0 0 0 1 9.1 20 0 0 0.182 0.182 0.16 0.291 0.33 0.269 87.4 10.9\nRengifo, Luis 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.135 0.149 0.125 92.8 24\nTucker, Cole 2 1 0 0 0 0 1 50 50 0 0 0 0 0.363 0.363\nCalhoun, Willie 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nWard, Taylor 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.004 0.009 0.006 86.9 42\nThaiss, Matt 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.107 0.375 0.456 95.2 37'}, {'pitcher_id': '694297', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPfaadt, Brandon 26 23 6 2 0 0 8 30.8 26.4 3 11.5 0.261 0.348 0.314 0.273 0.366 0.322 90.6 8.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdell, Jo 3 3 0 0 0 0 2 66.7 27.3 0 0 0 0 0 0.005 0.012 0.008 95.6 53\nMoniak, Mickey 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.657 0.743 0.639 87.3 7\nO'Hoppe, Logan 3 3 1 1 0 0 2 66.7 71.4 0 0 0.333 0.667 0.423 0.162 0.278 0.187 101.2 17\nWard, Taylor 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.001 0.004 0.002 97.7 50\nStefanic, Michael 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.477 0.535 0.57 98.3 3\nPillar, Kevin 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.343 0.377 0.311 94.9 10.5\nNeto, Zach 3 3 1 1 0 0 1 33.3 28.6 0 0 0.333 0.667 0.423 0.238 0.619 0.351 92.1 -8.5\nSchanuel, Nolan 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.523 0.567 0.545 82.1 9\nRengifo, Luis 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.593 0.395 0.467 0.371 84.4 -11.7"}, {'pitcher_id': '573009', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMantiply, Joe 6 5 1 0 0 0 2 33.3 36.4 1 16.7 0.2 0.2 0.262 0.405 0.519 0.448 97.8 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoniak, Mickey 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nAdell, Jo 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nGuillorme, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.597 0.691 0.562 100.3 6\nWard, Taylor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.743 0.982 0.744 96.6 15\nNeto, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.683 0.924 0.688 96.5 16'}, {'pitcher_id': '656756', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontgomery, Jordan 49 47 10 2 0 2 8 16.3 20.7 1 2 0.213 0.383 0.259 0.241 0.438 0.293 85.6 16.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 18 17 3 1 0 0 3 16.7 15.8 0 0 0.176 0.235 0.168 0.201 0.305 0.217 83 30.2\nRengifo, Luis 9 9 3 0 0 1 1 11.1 20 0 0 0.333 0.667 0.427 0.312 0.631 0.394 87.5 13.1\nWard, Taylor 6 5 2 1 0 1 1 16.7 25 1 16.7 0.4 1.2 0.675 0.361 1.11 0.616 97.3 21.5\nCalhoun, Willie 5 5 0 0 0 0 0 0 11.1 0 0 0 0 0 0.29 0.321 0.265 93.8 12.8\nStefanic, Michael 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.593 0.259 0.305 0.241 91.4 13\nAdell, Jo 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nO'Hoppe, Logan 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.026 0.031 0.024 80.9 -32\nSchanuel, Nolan 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.114 0.121 0.104 61 -8.5\nNeto, Zach 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.39 0.673 0.457 71.1 -13.5"}, {'pitcher_id': '686826', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJarvis, Bryce 10 9 3 1 0 0 1 10 7.1 1 10 0.333 0.444 0.374 0.421 0.845 0.546 94.4 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRengifo, Luis 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.096 0.107 0.089 93.3 -10\nCalhoun, Willie 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nStefanic, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.407 0.27 92.7 21\nThaiss, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.991 3.935 1.989 109.7 25\nNeto, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.065 0.06 69.3 -20\nAdell, Jo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.894 1.175 0.891 110.6 10\nSchanuel, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.426 0.466 0.385 92 3\nMoniak, Mickey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.903 1.191 0.896 105.7 12\nPillar, Kevin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.148 0.098 81.5 30'}, {'pitcher_id': '543518', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcGough, Scott 14 10 3 0 0 0 2 14.3 30.8 4 28.6 0.3 0.3 0.389 0.179 0.2 0.318 82.4 -7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThaiss, Matt 2 0 0 0 0 0 0 0 100 2 100 0.694 0.694\nSchanuel, Nolan 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.173 0.176 0.157 61.9 -30.5\nRengifo, Luis 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.444 0.329 0.382 0.319 91.5 10\nWard, Taylor 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nMoniak, Mickey 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nPillar, Kevin 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.108 0.131 0.101 91.8 23\nNeto, Zach 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.072 0.082 0.065 82.9 -18\nAdell, Jo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGuillorme, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.062 0.056 87.4 -42\nCalhoun, Willie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.437 0.477 0.402 90 5'}, {'pitcher_id': '656464', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGinkel, Kevin 12 10 3 1 0 0 1 8.3 13.6 1 8.3 0.3 0.4 0.369 0.263 0.412 0.356 89.4 14', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 4 3 1 0 0 0 1 25 16.7 1 25 0.333 0.333 0.39 0.137 0.161 0.269 79.7 15.5\nGuillorme, Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.707 0.748 0.636 76.7 15\nRengifo, Luis 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.126 0.214 0.139 90.5 25\nSchanuel, Nolan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.289 0.216 89.6 7.5\nWard, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.276 0.147 97.4 41\nThaiss, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.132 0.104 94 -19\nStefanic, Michael 1 0 0 0 0 0 0 0 0 0 0.724 0.724\nO'Hoppe, Logan 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.766 1.687 1.009 107.8 18"}, {'pitcher_id': '676714', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHughes, Brandon 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.386 0.421 0.353 90.6 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuillorme, Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nThaiss, Matt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.771 0.843 0.706 90.6 11'}, {'pitcher_id': '682175', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJacques, Joe 3 2 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.074 0.104 0.069 55.3 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCalhoun, Willie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nPillar, Kevin 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.149 0.209 0.138 76.7 1\nGuillorme, Luis 1 0 0 0 0 0 0 0 0 0 0 33.9 -3'}, {'pitcher_id': '518876', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Merrill 21 21 2 1 0 1 8 38.1 29.3 0 0 0.095 0.286 0.151 0.179 0.289 0.196 88.7 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 13 13 2 1 0 1 3 23.1 28 0 0 0.154 0.462 0.245 0.24 0.407 0.27 89.5 25.2\nCalhoun, Willie 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.024 0.044 0.027 78.4 50\nWard, Taylor 2 2 0 0 0 0 2 100 42.9 0 0 0 0 0 0\nTucker, Cole 3 3 0 0 0 0 2 66.7 20 0 0 0 0 0 0.187 0.216 0.172 101.8 5'}, {'pitcher_id': '663531', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAllen, Logan 6 6 1 0 1 0 0 0 0 0 0 0.167 0.5 0.267 0.265 0.52 0.335 83.5 0.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdell, Jo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.1 0.074 78 35\nPillar, Kevin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.231 0.245 0.212 76.4 -28.3\nWard, Taylor 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.801 0.409 1.143 0.649 96.9 27'}, {'pitcher_id': '674072', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHenry, Tommy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.424 0.618 0.442 98.4 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.424 0.618 0.442 98.4 24'}, {'pitcher_id': '666818', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '685314', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '623149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSewald, Paul 14 11 2 0 0 0 6 42.9 34.3 1 7.1 0.182 0.182 0.227 0.188 0.224 0.248 88.2 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 3 2 2 0 0 0 0 0 33.3 0 0 1 1 0.587 0.696 0.794 0.465 92 19.3\nCalhoun, Willie 3 3 0 0 0 0 2 66.7 30 0 0 0 0 0 0.096 0.15 0.099 93.4 20\nMoniak, Mickey 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nAdell, Jo 2 1 0 0 0 0 1 50 50 0 0 0 0 0.362 0.362\nThaiss, Matt 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nPillar, Kevin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRengifo, Luis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nStefanic, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.138 0.154 0.124 81 -2\nNeto, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.245 0.271 0.244 78.5 3'}, {'pitcher_id': '669194', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Ryne 14 13 3 2 0 0 1 7.1 9.5 0 0 0.231 0.385 0.293 0.189 0.333 0.256 88 22.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRengifo, Luis 3 3 1 1 0 0 1 33.3 14.3 0 0 0.333 0.667 0.415 0.144 0.308 0.187 93.8 26.5\nWard, Taylor 3 2 1 1 0 0 0 0 25 0 0 0.5 1 0.657 0.35 0.89 0.579 100.9 29\nMoniak, Mickey 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.053 0.054 0.049 87.7 32.3\nThaiss, Matt 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.074 0.051 72.8 6.7\nGuillorme, Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.5 0.617 0.489 92.4 21.5'}, {'pitcher_id': '668678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallen, Zac 16 16 2 0 0 1 7 43.8 31 0 0 0.125 0.313 0.18 0.184 0.325 0.214 91.9 30', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 6 6 0 0 0 0 3 50 38.5 0 0 0 0 0 0.116 0.126 0.105 90.5 48\nMoniak, Mickey 3 3 1 0 0 1 1 33.3 16.7 0 0 0.333 1.333 0.668 0.256 0.926 0.477 97 29\nWard, Taylor 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.235 0.251 0.213 86.2 5.5\nThaiss, Matt 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.257 0.303 0.243 94.7 28.5\nMoore, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}], 110: [{'pitcher_id': '657044', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThompson, Ryan 35 32 9 1 1 0 10 28.6 29 2 5.7 0.281 0.375 0.317 0.216 0.314 0.269 89.1 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHays, Austin 9 9 3 0 1 0 4 44.4 33.3 0 0 0.333 0.556 0.371 0.324 0.618 0.399 92.6 7.8\nSantander, Anthony 7 6 2 0 0 0 1 14.3 23.1 0 0 0.333 0.333 0.355 0.28 0.308 0.325 85.2 10.8\nMountcastle, Ryan 8 7 2 0 0 0 1 12.5 22.2 1 12.5 0.286 0.286 0.307 0.238 0.275 0.282 91.3 6.7\nMullins, Cedric 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.136 0.142 0.122 91.6 24.5\nUrías, Ramón 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.344 0.344\nCowser, Colton 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMateo, Jorge 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRutschman, Adley 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.051 0.063 0.049 78.4 25.5\nHenderson, Gunnar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.119 0.11 101 -23\nWestburg, Jordan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.031 0.053 0.033 83.2 32'}, {'pitcher_id': '669459', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Kyle 13 11 7 3 0 0 1 7.7 37.5 1 7.7 0.636 0.909 0.667 0.391 0.487 0.431 90.9 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 4 2 1 0 0 0 0 0 25 1 25 0.5 0.5 0.574 0.241 0.268 0.464 84.3 29\nSantander, Anthony 2 2 2 0 0 0 0 0 42.9 0 0 1 1 0.879 0.76 0.818 0.698 68.4 21\nMountcastle, Ryan 2 2 1 1 0 0 1 50 60 0 0 0.5 1 0.621 0.357 0.54 0.376 108.8 14\nWestburg, Jordan 2 2 2 2 0 0 0 0 25 0 0 1 2 1.244 0.674 0.814 0.643 107.1 5\nUrías, Ramón 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.072 0.076 0.068 92.6 6\nMcCann, James 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.326 0.179 95.5 38'}, {'pitcher_id': '677944', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCecconi, Slade 17 17 8 3 0 1 4 23.5 16.1 0 0 0.471 0.824 0.545 0.356 0.627 0.415 95.6 15.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRutschman, Adley 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.323 0.427 0.327 100.8 39\nHenderson, Gunnar 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nO'Hearn, Ryan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.476 0.604 0.47 93.4 -0.5\nSantander, Anthony 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.389 0.563 0.402 93 34.5\nMcCann, James 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.497 0.659 0.503 95.4 21\nHays, Austin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.411 1.15 0.643 85.8 -16.5\nMullins, Cedric 2 2 1 0 0 1 1 50 40 0 0 0.5 2 1.002 0.303 1.103 0.558 101.4 29\nUrías, Ramón 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.063 0.463 0.614 0.457 102.8 7.5"}, {'pitcher_id': '679885', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Justin 9 7 0 0 0 0 5 55.6 52.9 2 22.2 0 0 0.154 0.107 0.116 0.23 76.1 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCowser, Colton 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.346 0.537 0.579 0.59 75.6 14\nSantander, Anthony 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.107 0.117 0.097 76.5 4\nWestburg, Jordan 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nRutschman, Adley 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMullins, Cedric 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMountcastle, Ryan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '686796', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '612434', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastro, Miguel 21 15 3 0 0 0 6 28.6 32.4 4 19 0.2 0.2 0.284 0.256 0.324 0.343 84 12.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 6 5 1 0 0 0 1 16.7 16.7 0 0 0.2 0.2 0.174 0.369 0.456 0.351 85.8 15.8\nHays, Austin 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.23 0.23\nMateo, Jorge 3 1 0 0 0 0 1 33.3 66.7 1 33.3 0 0 0.344 0.344\nSantander, Anthony 2 1 0 0 0 0 1 50 60 1 50 0 0 0.344 0.344\nO'Hearn, Ryan 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.162 0.409 0.238 87.6 20\nMountcastle, Ryan 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.653 0.693 0.638 72 16\nUrías, Ramón 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.059 0.059 0.056 92 -21\nMullins, Cedric 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.906 0.957 0.827 74 18"}, {'pitcher_id': '694297', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPfaadt, Brandon 25 23 5 1 0 0 0 0 13.3 2 8 0.217 0.261 0.248 0.254 0.409 0.313 90.4 18.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nO'Hearn, Ryan 3 2 1 0 0 0 0 0 25 1 33.3 0.5 0.5 0.527 0.25 0.304 0.392 103.7 25.5\nCowser, Colton 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.256 0.743 0.407 88.7 4.3\nHenderson, Gunnar 3 2 0 0 0 0 0 0 16.7 1 33.3 0 0 0.231 0.358 0.412 0.45 95.3 1.5\nMountcastle, Ryan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.293 0.543 0.349 94 14\nWestburg, Jordan 3 3 1 1 0 0 0 0 28.6 0 0 0.333 0.667 0.423 0.389 0.556 0.392 87.5 2.7\nRutschman, Adley 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.445 0.56 0.435 97.3 4.3\nMullins, Cedric 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.07 0.078 0.063 85.5 28\nSantander, Anthony 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.079 0.088 0.076 81.2 49.7\nMateo, Jorge 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.169 0.094 81.7 48"}, {'pitcher_id': '573009', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMantiply, Joe 10 10 0 0 0 0 1 10 16.7 0 0 0 0 0 0.158 0.181 0.149 80 -5.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHenderson, Gunnar 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.144 0.145 0.131 49.3 -58\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.118 0.123 0.106 91.5 -18\nSantander, Anthony 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.056 0.044 84.4 5.5\nO'Hearn, Ryan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.342 0.399 0.329 88.2 25.5\nMullins, Cedric 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.055 0.052 66.2 -24\nHays, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.282 0.355 0.259 95.6 -4\nMateo, Jorge 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.079 0.063 72 -11"}, {'pitcher_id': '656756', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontgomery, Jordan 142 132 38 10 0 5 31 21.8 26.5 5 3.5 0.288 0.477 0.337 0.251 0.432 0.308 85.4 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHays, Austin 27 24 6 2 0 1 3 11.1 25 1 3.7 0.25 0.458 0.331 0.184 0.323 0.253 83.4 10.7\nMullins, Cedric 27 26 6 0 0 0 8 29.6 18 0 0 0.231 0.231 0.203 0.276 0.356 0.274 80.2 7.2\nMountcastle, Ryan 23 20 6 1 0 1 6 26.1 37 1 4.3 0.3 0.5 0.325 0.202 0.448 0.288 86.2 13.8\nUrías, Ramón 19 17 5 1 0 0 2 10.5 20 2 10.5 0.294 0.353 0.325 0.251 0.326 0.296 89.2 13\nSantander, Anthony 18 18 5 2 0 1 5 27.8 28.6 0 0 0.278 0.556 0.353 0.248 0.438 0.293 88.7 31.6\nMateo, Jorge 12 11 5 2 0 0 1 8.3 30.8 1 8.3 0.455 0.636 0.487 0.398 0.506 0.421 82.7 27\nMcCann, James 6 6 2 1 0 1 3 50 21.4 0 0 0.333 1 0.545 0.266 0.853 0.456 90 1.3\nRutschman, Adley 5 5 0 0 0 0 2 40 42.9 0 0 0 0 0 0.128 0.141 0.118 89.9 16.3\nHenderson, Gunnar 3 3 2 0 0 1 1 33.3 33.3 0 0 0.667 1.667 0.962 0.42 1.252 0.685 96.7 22.5\nWestburg, Jordan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.464 1.241 0.701 88.9 -1'}, {'pitcher_id': '686826', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJarvis, Bryce 12 9 2 0 0 0 2 16.7 19.2 3 25 0.222 0.222 0.321 0.216 0.305 0.341 89.4 16.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantander, Anthony 2 0 0 0 0 0 0 0 0 2 100 0.694 0.694\nMountcastle, Ryan 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 92.1 47\nHenderson, Gunnar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.058 0.039 76.4 57\nUrías, Ramón 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nMullins, Cedric 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.666 0.746 0.612 98.5 9\nCowser, Colton 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.254 0.274 0.229 101.6 -11\nWestburg, Jordan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.196 0.222 0.188 76.3 -2\nO'Hearn, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.594 1.215 0.754 104.8 19"}, {'pitcher_id': '543518', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcGough, Scott 8 7 2 0 0 1 2 25 38.5 1 12.5 0.286 0.714 0.448 0.302 0.622 0.423 87 39.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nHays, Austin 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.98 1.031 0.889 73 22\nO'Hearn, Ryan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nUrías, Ramón 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSantander, Anthony 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.001 0.001 0 69.3 86\nRutschman, Adley 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.603 2.351 1.187 104.1 35\nHenderson, Gunnar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.309 0.155 95.2 34\nMcCann, James 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.431 0.662 0.461 93.3 19"}, {'pitcher_id': '656464', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGinkel, Kevin 14 14 5 3 0 2 3 21.4 20 0 0 0.357 1 0.559 0.28 0.663 0.389 95.2 21.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.237 0.542 0.319 100 22\nUrías, Ramón 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nHays, Austin 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nSantander, Anthony 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.31 0.694 0.415 91.3 37\nHenderson, Gunnar 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.244 0.831 2.186 1.239 108 20\nCowser, Colton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.329 1.199 0.627 99.3 34\nMateo, Jorge 2 2 2 1 0 1 0 0 0 0 0 1 3 1.624 0.633 1.387 0.838 94.8 22\nMountcastle, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.059 0.063 0.056 74.1 -29\nO'Hearn, Ryan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.113 0.233 0.136 92 51.5\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.122 0.122 0.113 109.5 -28"}, {'pitcher_id': '676714', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHughes, Brandon 6 5 2 0 0 1 0 0 18.2 0 0 0.4 1 0.613 0.344 0.647 0.471 84.1 23.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 93.7 61\nHays, Austin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.119 0.141 0.108 80.6 -10\nMullins, Cedric 2 1 0 0 0 0 0 0 50 0 0 0 0 0.36 0.277 0.277 0.485 51 23\nMateo, Jorge 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.478 0.661 1.408 0.873 97.6 22'}, {'pitcher_id': '682175', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJacques, Joe 11 7 2 1 0 0 1 9.1 11.8 2 18.2 0.286 0.429 0.386 0.199 0.276 0.391 89 1.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantander, Anthony 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.228 0.228 0.451 89.3 -59\nHenderson, Gunnar 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.001 0.001 0 63 69\nO'Hearn, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.044 0.044 80.6 -36\nRutschman, Adley 1 0 0 0 0 0 0 0 0 0 0 0 0.558 100.9 17\nUrías, Ramón 2 1 1 0 0 0 0 0 0 0 0 1 1 0.804 0.454 0.504 0.572 99.5 2\nWestburg, Jordan 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMcCann, James 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nHays, Austin 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.466 0.88 0.568 100.6 18"}, {'pitcher_id': '518876', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Merrill 9 9 3 1 0 1 3 33.3 17.6 0 0 0.333 0.778 0.45 0.381 0.838 0.498 97.7 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMateo, Jorge 2 2 1 1 0 0 1 50 22.2 0 0 0.5 1 0.621 0.207 0.397 0.242 100.9 20\nMcCann, James 5 5 0 0 0 0 2 40 20 0 0 0 0 0 0.21 0.319 0.226 95.4 16.7\nSantander, Anthony 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.405 0.984 2.575 1.435 99.4 19.5'}, {'pitcher_id': '663531', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAllen, Logan 6 6 4 1 0 0 0 0 0 0 0 0.667 0.833 0.65 0.387 0.553 0.398 92.9 30.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantander, Anthony 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.58 0.449 0.67 0.471 90.4 33\nHenderson, Gunnar 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.494 0.792 0.543 99.6 17\nMateo, Jorge 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.482 0.518 0.434 103.8 -3\nMullins, Cedric 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 82.7 68'}, {'pitcher_id': '674072', 'player_data': '', 'team_data': 'Tommy Henry has not faced anyone on this team.'}, {'pitcher_id': '666818', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrías, Luis 4 3 0 0 0 0 0 0 9.1 0 0 0 0 0.182 0.466 0.532 0.509 103.7 -3.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nO'Hearn, Ryan 1 1 0 0 0 0 0 0 12.5 0 0 0 0 0 0.096 0.096 0.088 100 -22\nHenderson, Gunnar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.646 0.735 0.604 104.9 6\nRutschman, Adley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.657 0.766 0.618 106.1 6\nSantander, Anthony 1 0 0 0 0 0 0 0 0 0 0.726 0.726"}, {'pitcher_id': '685314', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '623149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSewald, Paul 19 18 4 2 0 0 5 26.3 24.5 1 5.3 0.222 0.333 0.262 0.203 0.259 0.226 87.4 34.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMullins, Cedric 5 5 3 2 0 0 1 20 33.3 0 0 0.6 1 0.678 0.331 0.421 0.33 90.9 32\nRutschman, Adley 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.24 0.331 0.24 87.9 7\nMateo, Jorge 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.353 0.427 0.339 83.5 46\nMountcastle, Ryan 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.344 0.344\nWestburg, Jordan 2 2 1 0 0 0 1 50 11.1 0 0 0.5 0.5 0.444 0.214 0.265 0.213 99.2 1\nSantander, Anthony 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.005 0.006 0.006 77.1 85\nUrías, Ramón 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.056 0.077 0.051 85.1 30\nHays, Austin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.009 0.011 0.009 83.2 71'}, {'pitcher_id': '669194', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Ryne 20 20 10 3 1 1 2 10 6.7 0 0 0.5 0.9 0.598 0.411 0.643 0.447 96.2 7.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantander, Anthony 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.084 0.089 0.076 99.9 -17\nWestburg, Jordan 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.599 0.838 0.613 107.3 6.5\nCowser, Colton 2 2 1 0 0 0 0 0 28.6 0 0 0.5 0.5 0.444 0.372 0.597 0.396 87.6 8.5\nMountcastle, Ryan 3 3 3 2 1 0 0 0 0 0 0 1 2.333 1.384 0.592 0.77 0.588 104 10\nHenderson, Gunnar 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.991 0.549 1.403 0.806 97.8 30.7\nMateo, Jorge 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.583 0.656 0.541 92.1 -0.5\nO'Hearn, Ryan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.427 0.571 0.433 100.3 2.3\nRutschman, Adley 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.079 0.09 0.071 75.6 0"}, {'pitcher_id': '668678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallen, Zac 51 48 10 3 0 1 12 23.5 16.8 3 5.9 0.208 0.333 0.26 0.199 0.299 0.242 90.7 2.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCann, James 9 9 0 0 0 0 5 55.6 18.8 0 0 0 0 0 0.091 0.18 0.112 89.2 14.8\nRutschman, Adley 6 5 1 0 0 1 0 0 11.1 1 16.7 0.2 0.8 0.464 0.244 0.504 0.376 91.6 19.6\nMountcastle, Ryan 6 6 0 0 0 0 2 33.3 25 0 0 0 0 0 0.083 0.139 0.094 86 1.3\nSantander, Anthony 6 6 2 1 0 0 1 16.7 28.6 0 0 0.333 0.5 0.359 0.193 0.231 0.184 85.1 -17.8\nO'Hearn, Ryan 8 8 3 1 0 0 1 12.5 9.1 0 0 0.375 0.5 0.377 0.261 0.37 0.273 92.8 3\nUrías, Ramón 4 3 1 0 0 0 0 0 11.1 1 25 0.333 0.333 0.394 0.316 0.352 0.393 92.7 -11.7\nHenderson, Gunnar 6 6 2 0 0 0 1 16.7 0 0 0 0.333 0.333 0.294 0.345 0.463 0.351 97 12.8\nMullins, Cedric 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.048 0.05 0.046 97.3 -32\nCowser, Colton 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.032 0.034 0.029 67.4 -7\nWestburg, Jordan 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.574 1.034 0.686 104.2 16"}]}, 110: {108: [{'pitcher_id': '657097', 'player_data': '', 'team_data': ''}, {'pitcher_id': '607644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeans, John 10 10 3 1 0 0 1 10 17.6 0 0 0.3 0.4 0.3 0.149 0.194 0.148 79.4 26', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCalhoun, Willie 5 5 1 1 0 0 1 20 33.3 0 0 0.2 0.4 0.248 0.08 0.154 0.095 84.5 14.8\nPillar, Kevin 5 5 2 0 0 0 0 0 0 0 0 0.4 0.4 0.352 0.218 0.235 0.201 75.4 35'}, {'pitcher_id': '600986', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVieira, Thyago 11 9 1 1 0 0 1 9.1 7.1 2 18.2 0.111 0.222 0.241 0.117 0.405 0.3 84.8 33.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThaiss, Matt 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.006 0.012 0.35 93 54\nCalhoun, Willie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.053 0.041 80.5 11.5\nAdell, Jo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 62.1 61\nSchanuel, Nolan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGuillorme, Luis 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nNeto, Zach 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.869 3.415 1.732 105.1 32\nTucker, Cole 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 97.9 51\nMoniak, Mickey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.049 0.036 86.2 57\nRengifo, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.056 0.05 73.1 -10'}, {'pitcher_id': '680570', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodriguez, Grayson 50 44 18 6 0 1 12 24 29.1 5 10 0.409 0.614 0.458 0.358 0.624 0.445 92 13.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 9 8 3 1 0 1 2 22.2 23.1 1 11.1 0.375 0.875 0.548 0.418 1.227 0.682 92.6 33.5\nO'Hoppe, Logan 8 7 4 1 0 0 0 0 9.1 1 12.5 0.571 0.714 0.578 0.539 0.76 0.57 93 17\nSchanuel, Nolan 8 7 2 0 0 0 2 25 30 1 12.5 0.286 0.286 0.308 0.328 0.41 0.367 90.5 -5\nRengifo, Luis 7 7 4 1 0 0 2 28.6 46.2 0 0 0.571 0.714 0.56 0.349 0.457 0.349 92.9 -1\nNeto, Zach 6 5 2 1 0 0 1 16.7 0 0 0 0.4 0.6 0.36 0.294 0.342 0.307 84.4 3.8\nMoniak, Mickey 5 5 1 1 0 0 2 40 50 0 0 0.2 0.4 0.249 0.116 0.214 0.136 94.3 32\nStefanic, Michael 3 2 0 0 0 0 2 66.7 25 1 33.3 0 0 0.232 0.232\nThaiss, Matt 2 1 0 0 0 0 1 50 75 1 50 0 0 0.348 0.348\nAdell, Jo 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.915 1.732 1.121 103.5 16"}, {'pitcher_id': '680694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradish, Kyle 10 10 1 0 0 0 2 20 20 0 0 0.1 0.1 0.088 0.224 0.293 0.221 93 20.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.244 0.348 0.247 90.7 21.5\nCalhoun, Willie 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.075 0.156 0.091 89.9 48.5\nNeto, Zach 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.23 0.265 0.221 95 18\nThaiss, Matt 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.409 0.446 0.377 96.6 -5'}, {'pitcher_id': '669211', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAkin, Keegan 11 10 2 0 0 0 4 36.4 43.8 1 9.1 0.2 0.2 0.222 0.103 0.124 0.153 81.4 22.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 4 3 1 0 0 0 2 50 42.9 1 25 0.333 0.333 0.392 0.195 0.224 0.312 100.9 4\nAdell, Jo 4 4 1 0 0 0 0 0 33.3 0 0 0.25 0.25 0.22 0.058 0.086 0.06 85.8 31.3\nRengifo, Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMoniak, Mickey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.213 0.222 0.195 44.7 7\nSchanuel, Nolan 1 1 0 0 0 0 1 100 0 0 0 0 0 0'}, {'pitcher_id': '669203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurnes, Corbin 30 28 2 0 0 0 11 36.7 35.8 1 3.3 0.071 0.071 0.106 0.105 0.143 0.149 83.2 6.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTucker, Cole 7 7 1 0 0 0 3 42.9 41.7 0 0 0.143 0.143 0.124 0.154 0.172 0.148 86.8 -7.5\nGuillorme, Luis 6 6 0 0 0 0 2 33.3 30 0 0 0 0 0 0.09 0.098 0.082 80.8 -4\nNeto, Zach 5 4 1 0 0 0 2 40 50 0 0 0.25 0.25 0.322 0.169 0.197 0.275 77.4 11.5\nWard, Taylor 4 4 0 0 0 0 3 75 42.9 0 0 0 0 0 0.001 0.001 0.001 89.4 73\nMoniak, Mickey 2 1 0 0 0 0 0 0 20 1 50 0 0 0.344 0.084 0.087 0.384 80.5 -36\nSchanuel, Nolan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.025 0.036 0.022 87.3 29\nPillar, Kevin 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.091 0.103 0.085 77.7 -17.5\nO'Hoppe, Logan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.529 0.285 88.6 47.5"}, {'pitcher_id': '665152', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKremer, Dean 46 42 14 1 0 1 10 21.7 20 3 6.5 0.333 0.429 0.362 0.29 0.49 0.363 88.9 10.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 7 7 1 0 0 0 1 14.3 9.5 0 0 0.143 0.143 0.126 0.427 0.897 0.551 97.9 10.7\nRengifo, Luis 9 8 2 0 0 0 2 22.2 15 1 11.1 0.25 0.25 0.273 0.27 0.391 0.327 96.6 12.2\nSchanuel, Nolan 6 5 2 0 0 0 0 0 16.7 1 16.7 0.4 0.4 0.411 0.354 0.407 0.394 85.4 12.2\nMoniak, Mickey 7 7 2 0 0 1 1 14.3 14.3 0 0 0.286 0.714 0.412 0.291 0.685 0.408 72.8 -0.3\nO'Hoppe, Logan 4 3 1 1 0 0 2 50 50 1 25 0.333 0.667 0.485 0.212 0.479 0.385 105.9 19\nNeto, Zach 4 4 1 0 0 0 3 75 50 0 0 0.25 0.25 0.221 0.071 0.079 0.066 74.6 9\nThaiss, Matt 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.588 0.229 0.265 0.217 94.8 14.3\nAdell, Jo 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.293 0.325 0.27 98.7 7\nCalhoun, Willie 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.227 0.262 0.212 83.1 32.5\nStefanic, Michael 2 1 1 0 0 0 0 0 0 0 0 1 1 0.802 0.574 0.638 0.626 85.2 11"}, {'pitcher_id': '663989', 'player_data': '', 'team_data': 'Nick Vespi has not faced anyone on this team.'}, {'pitcher_id': '669169', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeasley, Jonathan 5 4 0 0 0 0 2 40 42.9 1 20 0 0 0.138 0.166 0.251 0.28 91 -5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.23 0.31 0.48 0.452 105.4 15\nRengifo, Luis 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.021 0.021 0.022 76.5 -26'}, {'pitcher_id': '518886', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKimbrel, Craig 31 25 4 1 0 0 8 25.8 38.3 2 6.5 0.16 0.2 0.224 0.232 0.277 0.305 86.5 7.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPillar, Kevin 12 11 0 0 0 0 6 50 46.7 0 0 0 0 0 0.13 0.145 0.117 76.1 -11.7\nGuillorme, Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.53 0.695 0.524 91.8 17\nRengifo, Luis 3 1 0 0 0 0 0 0 40 1 33.3 0 0 0.231 0.081 0.083 0.544 94.9 2.5\nO'Hoppe, Logan 3 3 1 0 0 0 0 0 40 0 0 0.333 0.333 0.296 0.482 0.586 0.456 87.1 18.3\nAdell, Jo 2 0 0 0 0 0 0 0 33.3 1 50 0.708 0.708\nTucker, Cole 4 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.49 0.39 0.436 0.449 97.8 16.7\nSchanuel, Nolan 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.046 0.047 0.044 83.6 28.5\nWard, Taylor 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nThaiss, Matt 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0"}, {'pitcher_id': '608344', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Cole 52 48 17 1 0 3 10 19.2 18 4 7.7 0.354 0.563 0.417 0.297 0.536 0.38 91.1 10.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 17 15 6 0 0 1 3 17.6 14.3 2 11.8 0.4 0.6 0.462 0.313 0.526 0.397 89.8 18.5\nRengifo, Luis 18 17 8 0 0 2 1 5.6 13.5 1 5.6 0.471 0.824 0.563 0.406 0.814 0.527 91.2 -0.7\nAdell, Jo 10 10 1 0 0 0 6 60 42.9 0 0 0.1 0.1 0.088 0.073 0.102 0.076 96.2 12.5\nThaiss, Matt 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.393 0.285 0.358 0.382 84.3 19.3\nCalhoun, Willie 2 2 1 1 0 0 0 0 20 0 0 0.5 1 0.621 0.54 0.947 0.623 98.2 12.5\nO'Hoppe, Logan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.012 0.009 90.1 64"}, {'pitcher_id': '543056', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCoulombe, Danny 15 15 3 0 0 1 5 33.3 28.6 0 0 0.2 0.4 0.251 0.143 0.246 0.165 86.9 13.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRengifo, Luis 4 4 1 0 0 0 1 25 30 0 0 0.25 0.25 0.221 0.088 0.101 0.083 82.2 42.7\nThaiss, Matt 2 2 0 0 0 0 1 50 37.5 0 0 0 0 0 0.083 0.083 0.074 55 -46\nO'Hoppe, Logan 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.217 0.244 0.202 91.5 -6.5\nWard, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.166 0.194 0.149 87.3 -8\nNeto, Zach 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nMoniak, Mickey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.011 0.029 0.017 90.3 40\nPillar, Kevin 2 2 1 0 0 1 1 50 33.3 0 0 0.5 2 1.002 0.24 0.915 0.464 100.6 33\nSchanuel, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCalhoun, Willie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.877 0.533 0.578 0.491 105.8 1"}, {'pitcher_id': '622253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTate, Dillon 15 12 2 1 0 0 2 13.3 25 2 13.3 0.167 0.25 0.284 0.192 0.209 0.283 94.2 -8.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRengifo, Luis 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.164 0.178 0.15 89 -10\nWard, Taylor 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.246 0.262 0.224 89.9 8\nO'Hoppe, Logan 2 0 0 0 0 0 0 0 0 1 50 0.708 0.708\nAdell, Jo 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.287 0.317 0.266 100.8 -5\nNeto, Zach 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.141 0.158 0.13 95.9 -21\nSchanuel, Nolan 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nPillar, Kevin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCalhoun, Willie 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.176 0.179 0.182 101.5 -21"}, {'pitcher_id': '669330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWells, Tyler 31 30 9 0 0 1 6 19.4 31.1 1 3.2 0.3 0.4 0.318 0.281 0.514 0.348 86.7 7.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRengifo, Luis 6 6 2 0 0 0 0 0 8.3 0 0 0.333 0.333 0.295 0.231 0.321 0.237 77.8 -15.3\nWard, Taylor 6 5 2 0 0 1 1 16.7 45.5 1 16.7 0.4 1 0.61 0.342 0.942 0.557 92.1 16.3\nMoniak, Mickey 3 3 1 0 0 0 2 66.7 66.7 0 0 0.333 0.333 0.294 0.023 0.028 0.021 77.3 46\nAdell, Jo 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.385 0.756 0.479 111.9 16\nSchanuel, Nolan 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.472 0.5 0.43 79.3 -7.3\nNeto, Zach 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.222 0.312 0.557 0.368 91.3 13.3\nO'Hoppe, Logan 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.444 0.453 0.551 0.432 101.2 13\nPillar, Kevin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.339 1.038 0.563 89.5 49.5\nStefanic, Michael 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.04 0.035 86.1 13\nCalhoun, Willie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.19 0.193 0.171 91 -9"}, {'pitcher_id': '544150', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Albert 16 14 4 1 0 0 2 12.5 26.7 1 6.3 0.286 0.357 0.308 0.277 0.354 0.3 89.3 3.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 3 3 0 0 0 0 0 0 14.3 0 0 0 0 0 0.382 0.651 0.433 96.6 16.3\nSchanuel, Nolan 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.129 0.108 66.7 -23\nO'Hoppe, Logan 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.226 0.247 0.205 102.4 0\nNeto, Zach 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.442 0.483 0.407 86.8 5.5\nPillar, Kevin 2 1 1 0 0 0 0 0 0 1 50 1 1 0.785 0.453 0.492 0.549 101.4 1\nMoniak, Mickey 2 2 0 0 0 0 1 50 80 0 0 0 0 0 0.3 0.334 0.276 97.5 9\nAdell, Jo 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.056 0.062 0.051 98.4 21"}, {'pitcher_id': '672335', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Cionel 17 16 3 0 1 0 5 29.4 28.6 1 5.9 0.188 0.313 0.24 0.206 0.274 0.238 90.7 9.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWard, Taylor 5 5 2 0 0 0 2 40 20 0 0 0.4 0.4 0.354 0.248 0.275 0.234 97.7 -4.7\nAdell, Jo 3 3 1 0 1 0 1 33.3 27.3 0 0 0.333 1 0.537 0.446 0.713 0.497 106.8 14.5\nPillar, Kevin 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.09 0.095 0.089 101 -18\nThaiss, Matt 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.005 0.005 0.005 49.7 59\nRengifo, Luis 1 0 0 0 0 0 0 0 1 100 0.699 0.699\nMoniak, Mickey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.353 0.429 0.318 79.6 7\nCalhoun, Willie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.093 0.058 90 36\nO'Hoppe, Logan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.003 90.7 50\nNeto, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.133 0.135 0.124 79.6 -48"}, {'pitcher_id': '666974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCano, Yennier 14 14 3 1 0 1 2 14.3 36.4 0 0 0.214 0.5 0.301 0.263 0.514 0.326 88.7 6.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNeto, Zach 3 3 1 0 0 1 1 33.3 60 0 0 0.333 1.333 0.695 0.35 1.175 0.621 99.2 9\nRengifo, Luis 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.19 0.241 0.19 90.5 3.7\nSchanuel, Nolan 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.966 1.127 0.917 91.1 14\nWard, Taylor 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.095 0.082 87.5 -31.5\nCalhoun, Willie 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.324 0.652 0.401 94.4 4.5\nO'Hoppe, Logan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.223 0.299 0.222 74.3 39\nAdell, Jo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMoniak, Mickey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.037 0.037 0.033 64.8 54"}], 109: [{'pitcher_id': '657097', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWebb, Jacob 25 23 1 0 0 0 6 24 28.9 2 8 0.043 0.043 0.091 0.157 0.193 0.194 91.7 12.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNewman, Kevin 6 6 1 0 0 0 0 0 8.3 0 0 0.167 0.167 0.147 0.251 0.298 0.238 90.8 15.7\nMarte, Ketel 4 4 0 0 0 0 2 50 57.1 0 0 0 0 0 0.123 0.157 0.119 88.2 17.5\nGrichuk, Randal 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.132 0.142 0.123 91.4 -22.3\nWalker, Christian 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.011 0.017 0.352 89.9 56\nSuárez, Eugenio 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.351 0.401 0.323 101.1 9\nPederson, Joc 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nGurriel Jr., Lourdes 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.049 0.111 0.064 91 33\nCarroll, Corbin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.183 0.152 99.8 -20'}, {'pitcher_id': '607644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeans, John 41 38 7 2 1 1 9 22 23.4 1 2.4 0.184 0.368 0.251 0.236 0.435 0.314 90.1 25.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 12 12 2 1 0 0 3 25 35.7 0 0 0.167 0.25 0.18 0.24 0.295 0.232 90.3 20.1\nGrichuk, Randal 7 6 0 0 0 0 1 14.3 11.5 1 14.3 0 0 0.099 0.125 0.267 0.241 87.3 25.6\nAlexander, Blaze 2 2 1 0 0 0 1 50 9.1 0 0 0.5 0.5 0.444 0.202 0.202 0.181 60.7 35\nMarte, Ketel 5 4 2 0 0 1 0 0 30 0 0 0.5 1.25 0.736 0.543 1.272 0.749 98 30.3\nWalker, Christian 4 4 0 0 0 0 2 50 30.8 0 0 0 0 0 0.045 0.055 0.043 71 29\nCarroll, Corbin 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.32 0.465 0.448 101.8 18.7\nPederson, Joc 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.127 0.143 0.115 72.3 8\nMoreno, Gabriel 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.114 0.314 0.177 95.3 46.5\nSuárez, Eugenio 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.005 0.009 0.006 85.2 45\nNewman, Kevin 2 2 2 1 1 0 0 0 0 0 0 1 2.5 1.441 0.723 1.898 1.087 103.3 19'}, {'pitcher_id': '600986', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVieira, Thyago 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.007 0.012 0.364 95.2 60', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.012 0.007 95.2 60\nGrichuk, Randal 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72'}, {'pitcher_id': '680570', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodriguez, Grayson 2 2 1 0 0 1 1 50 75 0 0 0.5 2 1.002 0.172 0.503 0.275 97.1 27', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 2 2 1 0 0 1 1 50 75 0 0 0.5 2 1.002 0.172 0.503 0.275 97.1 27'}, {'pitcher_id': '680694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradish, Kyle 34 29 12 2 0 0 6 17.6 27.6 4 11.8 0.414 0.483 0.436 0.318 0.429 0.378 92.4 2.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 13 12 5 0 0 0 1 7.7 4.8 1 7.7 0.417 0.417 0.393 0.383 0.485 0.403 96.1 1.8\nSuárez, Eugenio 8 8 4 1 0 0 3 37.5 42.1 0 0 0.5 0.625 0.487 0.441 0.656 0.472 96.2 16.8\nMarte, Ketel 3 2 2 1 0 0 0 0 33.3 1 33.3 1 1.5 0.941 0.124 0.168 0.308 91.9 -23.5\nPerdomo, Geraldo 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nMoreno, Gabriel 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.232 0.107 0.128 0.298 66.4 -23\nCarroll, Corbin 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.094 0.124 0.089 96.8 11\nWalker, Christian 2 1 0 0 0 0 0 0 50 0 0 0 0 0.363 0.351 0.409 0.527 77.9 27'}, {'pitcher_id': '669211', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAkin, Keegan 31 29 9 4 0 1 5 16.1 17.6 2 6.5 0.31 0.552 0.383 0.321 0.71 0.447 93.6 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 11 10 3 2 0 1 2 18.2 13.3 1 9.1 0.3 0.8 0.468 0.371 0.726 0.478 97.7 10.8\nGrichuk, Randal 10 9 2 0 0 0 2 20 20.8 1 10 0.222 0.222 0.245 0.165 0.362 0.269 88.7 31.1\nSuárez, Eugenio 4 4 2 2 0 0 0 0 16.7 0 0 0.5 1 0.626 0.666 1.933 1.074 102.1 8.5\nPederson, Joc 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.164 0.554 0.29 94.8 1\nCarroll, Corbin 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nWalker, Christian 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.339 0.363 0.305 100.2 -3\nNewman, Kevin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.225 0.247 0.212 71.8 8\nBarnhart, Tucker 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.54 0.623 0.523 73.1 31'}, {'pitcher_id': '669203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurnes, Corbin 141 129 33 4 0 4 33 23.4 31 10 7.1 0.256 0.38 0.302 0.249 0.384 0.303 86.7 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPederson, Joc 21 20 6 1 0 1 9 42.9 44.2 1 4.8 0.3 0.5 0.352 0.21 0.366 0.265 86.8 2.7\nNewman, Kevin 25 24 3 0 0 0 4 16 18.6 1 4 0.125 0.125 0.134 0.212 0.25 0.223 80.6 29.5\nSuárez, Eugenio 12 9 2 0 0 0 4 33.3 54.5 3 25 0.222 0.222 0.319 0.231 0.332 0.356 83.5 10.2\nCarroll, Corbin 13 11 2 0 0 1 3 23.1 23.1 2 15.4 0.182 0.455 0.329 0.247 0.576 0.399 91.7 12.3\nWalker, Christian 12 12 4 1 0 0 2 16.7 37 0 0 0.333 0.417 0.324 0.239 0.343 0.252 87.6 32.7\nMarte, Ketel 14 13 4 0 0 1 1 7.1 19 1 7.1 0.308 0.538 0.381 0.342 0.526 0.395 89.4 13.3\nPerdomo, Geraldo 10 8 3 1 0 0 2 20 16.7 2 20 0.375 0.5 0.44 0.314 0.354 0.37 91.2 9.2\nMoreno, Gabriel 11 11 3 0 0 1 3 27.3 43.8 0 0 0.273 0.545 0.343 0.226 0.574 0.326 89.9 21.8\nBarnhart, Tucker 9 8 2 0 0 0 1 11.1 21.4 0 0 0.25 0.25 0.195 0.304 0.343 0.251 83 12\nGurriel Jr., Lourdes 8 7 2 1 0 0 2 25 16.7 0 0 0.286 0.429 0.358 0.171 0.212 0.235 94.4 8.4\nMcCarthy, Jake 6 6 2 0 0 0 2 33.3 33.3 0 0 0.333 0.333 0.294 0.359 0.428 0.351 84.3 6.8'}, {'pitcher_id': '665152', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKremer, Dean 46 39 10 3 0 3 14 30.4 18.4 5 10.9 0.256 0.564 0.367 0.237 0.442 0.383 90 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Eugenio 9 8 2 1 0 0 3 33.3 36.4 1 11.1 0.25 0.375 0.316 0.241 0.308 0.288 83.8 21.6\nGurriel Jr., Lourdes 9 8 3 1 0 2 3 33.3 16.7 1 11.1 0.375 1.25 0.663 0.263 0.778 0.451 95.3 38.2\nGrichuk, Randal 6 6 1 0 0 0 1 16.7 7.7 0 0 0.167 0.167 0.147 0.172 0.189 0.156 92.2 -18.2\nNewman, Kevin 5 4 2 0 0 0 0 0 0 1 20 0.5 0.5 0.493 0.56 0.638 0.557 84.6 5.5\nWalker, Christian 3 2 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0 0 0.648 107.2 26\nPederson, Joc 3 2 0 0 0 0 2 66.7 20 1 33.3 0 0 0.231 0.231\nBarnhart, Tucker 2 1 0 0 0 0 1 50 0 1 50 0 0 0.346 0.346\nMarte, Ketel 3 2 0 0 0 0 2 66.7 60 0 0 0 0 0 0 0 0.358 99.8 29\nCarroll, Corbin 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.303 0.382 0.298 92.7 11.3\nMcCarthy, Jake 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.695 0.347 1.237 0.642 83.1 47.7'}, {'pitcher_id': '663989', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVespi, Nick 4 4 2 1 0 0 0 0 28.6 0 0 0.5 0.75 0.536 0.416 0.436 0.377 81.7 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoreno, Gabriel 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.32 0.341 0.289 79.2 31\nSuárez, Eugenio 1 1 1 1 0 0 0 0 50 0 0 1 2 1.261 0.617 0.624 0.555 65.3 20\nGurriel Jr., Lourdes 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.409 0.441 0.374 103 -1'}, {'pitcher_id': '669169', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeasley, Jonathan 26 19 8 2 0 1 1 3.8 20.6 7 26.9 0.421 0.684 0.532 0.245 0.384 0.386 85.8 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerdomo, Geraldo 4 2 0 0 0 0 0 0 28.6 2 50 0 0 0.344 0.016 0.025 0.354 75.6 66\nBarnhart, Tucker 5 3 3 1 0 1 0 0 0 2 40 1 2.333 1.119 0.542 1.237 0.728 95.6 21.3\nMcCarthy, Jake 4 2 1 1 0 0 1 25 50 2 50 0.5 1 0.66 0.358 0.443 0.519 98.8 15\nMarte, Ketel 6 5 2 0 0 0 0 0 16.7 1 16.7 0.4 0.4 0.409 0.376 0.437 0.415 85.8 18.8\nWalker, Christian 5 5 1 0 0 0 0 0 12.5 0 0 0.2 0.2 0.177 0.077 0.088 0.078 77.6 -27.8\nPederson, Joc 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.442 0.01 0.016 0.011 95.5 55.5'}, {'pitcher_id': '518886', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKimbrel, Craig 68 57 10 2 0 1 14 20.6 28.5 7 10.3 0.175 0.263 0.272 0.194 0.295 0.291 90.1 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 9 7 1 0 0 0 2 22.2 27.8 2 22.2 0.143 0.143 0.252 0.142 0.152 0.258 88.6 5.2\nGurriel Jr., Lourdes 8 7 1 1 0 0 1 12.5 15.4 1 12.5 0.143 0.286 0.242 0.134 0.237 0.224 93.4 32.3\nPerdomo, Geraldo 6 4 1 1 0 0 3 50 36.4 2 33.3 0.25 0.5 0.438 0.029 0.052 0.253 86.2 26\nWalker, Christian 8 7 2 0 0 1 2 25 38.9 0 0 0.286 0.714 0.46 0.339 0.701 0.476 95.9 12.2\nSuárez, Eugenio 6 6 0 0 0 0 2 33.3 33.3 0 0 0 0 0 0.087 0.103 0.08 85.4 4\nNewman, Kevin 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.12 0.139 0.12 86.5 -13.7\nPederson, Joc 7 7 1 0 0 0 1 14.3 25 0 0 0.143 0.143 0.126 0.131 0.18 0.136 88.3 27.8\nMarte, Ketel 6 6 3 0 0 0 0 0 12.5 0 0 0.5 0.5 0.441 0.345 0.487 0.358 92.5 23.5\nMcCarthy, Jake 5 4 0 0 0 0 1 20 36.4 0 0 0 0 0.144 0.152 0.338 0.311 82.6 -24\nMoreno, Gabriel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.069 0.069 0.381 86.2 -15\nCarroll, Corbin 3 0 0 0 0 0 0 0 100 1 33.3 0.712 0.712\nAlexander, Blaze 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.606 0.682 0.565 101.4 6\nBarnhart, Tucker 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.612 0.683 0.565 92 9'}, {'pitcher_id': '608344', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Cole 57 56 17 2 1 5 13 22.8 21.4 1 1.8 0.304 0.643 0.403 0.289 0.602 0.38 89.1 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 13 13 5 0 1 1 4 30.8 20.7 0 0 0.385 0.769 0.48 0.314 0.608 0.387 95 14\nSuárez, Eugenio 7 6 2 0 0 2 3 42.9 33.3 1 14.3 0.333 1.333 0.691 0.31 1.123 0.609 103.3 28\nGrichuk, Randal 7 7 2 1 0 0 0 0 6.3 0 0 0.286 0.429 0.304 0.209 0.324 0.228 85.5 19.4\nCarroll, Corbin 6 6 1 0 0 0 0 0 10 0 0 0.167 0.167 0.147 0.285 0.301 0.259 79.5 -4.8\nWalker, Christian 6 6 3 0 0 1 2 33.3 40 0 0 0.5 1 0.629 0.413 0.809 0.514 94.1 17\nMarte, Ketel 6 6 2 0 0 1 1 16.7 12.5 0 0 0.333 0.833 0.495 0.321 0.772 0.453 89.4 -11\nMoreno, Gabriel 4 4 1 0 0 0 2 50 55.6 0 0 0.25 0.25 0.221 0.238 0.281 0.225 83 24\nNewman, Kevin 5 5 1 1 0 0 0 0 0 0 0 0.2 0.4 0.249 0.224 0.526 0.312 85.5 34\nAlexander, Blaze 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.276 0.847 0.458 99.6 27\nBarnhart, Tucker 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.051 0.046 73.3 -23'}, {'pitcher_id': '543056', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCoulombe, Danny 12 12 3 2 0 0 4 33.3 24 0 0 0.25 0.417 0.282 0.278 0.495 0.328 96.2 4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Ketel 4 4 1 0 0 0 1 25 27.3 0 0 0.25 0.25 0.22 0.185 0.195 0.167 87.6 -7.7\nGrichuk, Randal 3 3 1 1 0 0 1 33.3 25 0 0 0.333 0.667 0.419 0.364 0.552 0.398 109.8 3\nPederson, Joc 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nCarroll, Corbin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.417 0.447 0.375 101 -4\nNewman, Kevin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGurriel Jr., Lourdes 1 1 1 1 0 0 0 0 50 0 0 1 2 1.242 0.449 0.579 0.45 83.9 22\nWalker, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.634 2.473 1.243 102.6 32'}, {'pitcher_id': '622253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTate, Dillon 25 24 5 1 0 1 7 28 38.1 1 4 0.208 0.375 0.262 0.208 0.292 0.234 83.4 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGurriel Jr., Lourdes 13 13 2 0 0 0 4 30.8 50 0 0 0.154 0.154 0.135 0.17 0.2 0.161 82.7 16.9\nGrichuk, Randal 9 9 3 1 0 1 3 33.3 29.4 0 0 0.333 0.778 0.456 0.292 0.473 0.324 90.2 4.7\nMoreno, Gabriel 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nPederson, Joc 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.08 0.069 58 -20\nSuárez, Eugenio 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.075 0.075 0.073 74.9 -8'}, {'pitcher_id': '669330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWells, Tyler 18 18 6 1 0 1 3 16.7 21.2 0 0 0.333 0.556 0.376 0.209 0.383 0.249 88.8 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 5 5 2 1 0 0 2 40 40 0 0 0.4 0.6 0.424 0.377 0.527 0.387 104 13.3\nGurriel Jr., Lourdes 6 6 2 0 0 1 0 0 0 0 0 0.333 0.833 0.481 0.223 0.609 0.339 88.6 12.8\nBarnhart, Tucker 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.295 0.108 0.121 0.101 73.8 -11\nSuárez, Eugenio 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.003 0.009 0.005 96.8 48\nMoreno, Gabriel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.103 0.107 0.096 85.1 29'}, {'pitcher_id': '544150', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Albert 18 16 4 1 0 0 3 16.7 21.9 2 11.1 0.25 0.313 0.294 0.196 0.266 0.252 91.6 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Ketel 4 3 1 0 0 0 0 0 14.3 1 25 0.333 0.333 0.392 0.112 0.129 0.251 87.5 -1\nGrichuk, Randal 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.293 0.303 0.321 0.277 103.5 -2\nWalker, Christian 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.346 0.133 0.151 0.408 90.6 -17\nGurriel Jr., Lourdes 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.075 0.088 0.068 86.8 -8\nMcCarthy, Jake 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.261 0.281 0.235 88.3 -5\nNewman, Kevin 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.183 0.723 0.364 99.6 38\nCarroll, Corbin 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSuárez, Eugenio 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.529 0.686 0.519 94.5 4\nMoreno, Gabriel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.083 0.051 78.8 32\nBarnhart, Tucker 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.118 0.07 88.1 30'}, {'pitcher_id': '672335', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Cionel 24 21 8 2 0 0 5 20.8 35.1 2 8.3 0.381 0.476 0.412 0.224 0.309 0.286 93.7 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrichuk, Randal 5 5 2 2 0 0 1 20 45.5 0 0 0.4 0.8 0.497 0.418 0.714 0.465 99.9 19.3\nGurriel Jr., Lourdes 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.29 0.046 0.046 0.042 83.2 -21.5\nNewman, Kevin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.111 0.126 0.102 87.4 5\nPederson, Joc 2 1 1 0 0 0 0 0 0 0 0 1 1 0.801 0.48 0.553 0.591 98.7 1\nWalker, Christian 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nCarroll, Corbin 2 2 2 0 0 0 0 0 0 0 0 1 1 0.886 0.236 0.261 0.217 89 -11\nSuárez, Eugenio 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.036 0.045 0.033 85.2 -18\nBarnhart, Tucker 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nMoreno, Gabriel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.386 0.454 0.364 91.9 3\nPerdomo, Geraldo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMarte, Ketel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.37 0.395 0.339 108.7 -6.5'}, {'pitcher_id': '666974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCano, Yennier 15 13 4 1 0 1 1 6.7 27.6 2 13.3 0.308 0.615 0.429 0.328 0.464 0.389 89.6 -17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoreno, Gabriel 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.538 0.342 0.404 0.328 81.9 -33\nSuárez, Eugenio 3 1 0 0 0 0 0 0 20 2 66.7 0 0 0.463 0.307 0.307 0.556 86.2 2\nAlexander, Blaze 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.088 0.09 0.081 90.5 -49\nCarroll, Corbin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.116 0.145 0.108 82.5 -15\nPerdomo, Geraldo 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.048 0.048 0.047 79.2 -37\nPederson, Joc 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.826 1.045 0.806 110.4 9\nWalker, Christian 1 1 1 0 0 1 0 0 66.7 0 0 1 4 2.004 0.834 2.055 1.196 110.9 18\nMarte, Ketel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.305 0.356 0.278 89.3 2\nGurriel Jr., Lourdes 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.277 0.277 0.251 98.3 -6'}]}, 112: {113: [{'pitcher_id': '622075', 'player_data': '', 'team_data': ''}, {'pitcher_id': '605155', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrewer, Colten 2 1 0 0 0 0 0 0 50 1 50 0 0 0.349 0.533 1.167 0.707 100.7 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspinal, Santiago 2 1 0 0 0 0 0 0 50 1 50 0 0 0.349 0.533 1.167 0.707 100.7 21'}, {'pitcher_id': '543294', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHendricks, Kyle 49 46 13 2 0 0 11 22.4 22.4 3 6.1 0.283 0.326 0.291 0.207 0.246 0.227 81.9 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 14 14 4 0 0 0 5 35.7 28.6 0 0 0.286 0.286 0.252 0.196 0.219 0.182 81 11.8\nCandelario, Jeimer 11 11 2 2 0 0 2 18.2 26.1 0 0 0.182 0.364 0.227 0.113 0.155 0.115 79.2 27.4\nFriedl, TJ 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.069 0.082 0.064 76.3 20\nBenson, Will 3 1 1 0 0 0 0 0 16.7 2 66.7 1 1 0.758 0.969 1.077 0.761 80.7 17\nStephenson, Tyler 4 3 1 0 0 0 1 25 0 1 25 0.333 0.333 0.393 0.328 0.424 0.419 96.9 -15.5\nFraley, Jake 4 4 1 0 0 0 0 0 14.3 0 0 0.25 0.25 0.222 0.103 0.118 0.095 84 15\nSuter, Brent 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.027 0.027 0.026 63.1 -38\nDe La Cruz, Elly 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.008 0.016 0.009 88.5 38\nMartini, Nick 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.58 0.569 0.674 0.528 93.7 9.3\nHurtubise, Jacob 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.926 1.107 0.891 96.5 12\nSteer, Spencer 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.26 0.261 0.234 43.8 -35'}, {'pitcher_id': '669713', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWesneski, Hayden 50 45 9 3 0 1 18 36 28.1 2 4 0.2 0.333 0.265 0.154 0.226 0.207 83.4 7.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 10 10 1 1 0 0 4 40 33.3 0 0 0.1 0.2 0.126 0.114 0.173 0.126 84.4 17.2\nIndia, Jonathan 8 6 1 1 0 0 2 25 18.2 0 0 0.167 0.333 0.336 0.095 0.148 0.259 89 -8.5\nFriedl, TJ 6 5 3 0 0 1 1 16.7 12.5 0 0 0.6 1.2 0.754 0.263 0.489 0.32 60.2 1\nFraley, Jake 8 8 1 0 0 0 3 37.5 27.8 0 0 0.125 0.125 0.11 0.156 0.172 0.143 84.3 13.2\nBenson, Will 4 3 0 0 0 0 3 75 57.1 1 25 0 0 0.174 0.174\nStephenson, Tyler 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.329 0.522 0.357 97.5 23.3\nDe La Cruz, Elly 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.232 0.232\nCandelario, Jeimer 4 4 2 1 0 0 0 0 14.3 0 0 0.5 0.75 0.533 0.34 0.387 0.316 90.1 -4.5\nMartini, Nick 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nMaile, Luke 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFairchild, Stuart 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nEspinal, Santiago 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.311 0.59 0.365 98.5 20'}, {'pitcher_id': '668338', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Tyson 10 7 1 1 0 0 1 10 23.8 3 30 0.143 0.286 0.335 0.169 0.296 0.347 94.1 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFraley, Jake 2 1 0 0 0 0 0 0 28.6 1 50 0 0 0.348 0.011 0.019 0.354 85.4 45\nIndia, Jonathan 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.003 0.005 0.35 87.8 47\nFairchild, Stuart 1 0 0 0 0 0 0 0 40 1 100 0.692 0.692\nStephenson, Tyler 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.269 0.142 92.2 36\nSteer, Spencer 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.255 0.262 0.232 96.4 -13\nBenson, Will 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.754 1.25 0.855 110.7 14'}, {'pitcher_id': '687863', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHodge, Porter 9 6 1 1 0 0 0 0 0 3 33.3 0.167 0.333 0.372 0.306 0.385 0.43 85.1 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.869 1.019 0.757 98.6 11\nStephenson, Tyler 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.083 0.165 0.398 89.8 30\nHurtubise, Jacob 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nFraley, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.047 0.041 64.8 -28\nSteer, Spencer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 87.8 72\nEspinal, Santiago 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.053 0.046 67.5 -15\nFriedl, TJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.786 1.022 0.774 101.8 14'}, {'pitcher_id': '657006', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteele, Justin 94 85 25 4 1 2 22 23.4 25.8 6 6.4 0.294 0.435 0.338 0.264 0.387 0.312 85.8 14.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 18 16 3 0 0 1 4 22.2 14.3 1 5.6 0.188 0.375 0.248 0.237 0.384 0.302 82 15\nStephenson, Tyler 17 16 9 2 0 1 1 5.9 28.1 1 5.9 0.563 0.875 0.618 0.413 0.751 0.504 91.4 18.6\nDe La Cruz, Elly 9 8 0 0 0 0 6 66.7 43.8 1 11.1 0 0 0.077 0.012 0.019 0.089 77.3 11\nSteer, Spencer 12 12 5 2 1 0 3 25 25.9 0 0 0.417 0.75 0.488 0.353 0.462 0.348 95.2 7.3\nFriedl, TJ 9 6 1 0 0 0 2 22.2 18.8 2 22.2 0.167 0.167 0.333 0.154 0.184 0.33 73.8 18.3\nFairchild, Stuart 12 11 3 0 0 0 3 25 27.3 1 8.3 0.273 0.273 0.279 0.221 0.244 0.244 87.4 5.1\nEspinal, Santiago 8 7 1 0 0 0 0 0 0 0 0 0.143 0.143 0.126 0.294 0.349 0.28 78.1 9.4\nCandelario, Jeimer 6 6 2 0 0 0 3 50 55.6 0 0 0.333 0.333 0.296 0.319 0.364 0.297 80.8 18.7\nMaile, Luke 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.135 0.205 0.141 89.4 36'}, {'pitcher_id': '592791', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaillon, Jameson 71 67 18 5 0 5 21 29.6 26.7 3 4.2 0.269 0.567 0.368 0.25 0.466 0.325 88.5 18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspinal, Santiago 11 11 2 0 0 0 5 45.5 33.3 0 0 0.182 0.182 0.161 0.219 0.269 0.209 90.1 12.7\nSteer, Spencer 8 8 2 1 0 1 3 37.5 18.8 0 0 0.25 0.75 0.406 0.156 0.491 0.264 75.8 34.8\nCandelario, Jeimer 10 10 3 1 0 1 3 30 28 0 0 0.3 0.7 0.413 0.238 0.427 0.284 90.9 5.1\nFraley, Jake 8 7 0 0 0 0 2 25 18.8 1 12.5 0 0 0.087 0.117 0.213 0.208 83.3 34.8\nDe La Cruz, Elly 6 5 1 0 0 1 4 66.7 33.3 1 16.7 0.2 0.8 0.45 0.191 0.76 0.435 107.6 29\nFriedl, TJ 9 8 3 1 0 1 1 11.1 14.3 1 11.1 0.375 0.875 0.536 0.361 0.581 0.432 96.4 8\nBenson, Will 5 4 2 1 0 0 1 20 30.8 0 0 0.5 0.75 0.57 0.492 0.595 0.523 98.4 8.3\nStephenson, Tyler 4 4 4 1 0 1 0 0 12.5 0 0 1 2 1.253 0.496 1.251 0.721 98.7 15.5\nMartini, Nick 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.078 0.198 0.108 96 26\nIndia, Jonathan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.703 0.818 0.657 94.3 13.5\nFairchild, Stuart 2 2 0 0 0 0 0 0 66.7 0 0 0 0 0 0.068 0.071 0.062 80.1 32.5\nMaile, Luke 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.062 0.084 0.068 73.9 24\nSuter, Brent 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.191 0.191 0.248 34.8 46'}, {'pitcher_id': '676962', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Ben 38 33 9 1 0 3 9 23.7 29.7 2 5.3 0.273 0.576 0.37 0.2 0.341 0.292 84.9 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Elly 5 5 0 0 0 0 3 60 37.5 0 0 0 0 0 0.089 0.099 0.082 81.3 31\nSteer, Spencer 5 5 1 0 0 0 1 20 12.5 0 0 0.2 0.2 0.178 0.256 0.425 0.288 84.9 17.3\nCandelario, Jeimer 5 5 3 0 0 1 0 0 22.2 0 0 0.6 1.2 0.773 0.192 0.297 0.213 79.9 0.8\nIndia, Jonathan 3 0 0 0 0 0 0 0 50 2 66.7 0.461 0.589 96.8 23\nHurtubise, Jacob 4 3 0 0 0 0 0 0 0 0 0 0 0 0.181 0.133 0.477 0.365 76.9 54.3\nStephenson, Tyler 4 4 1 0 0 0 2 50 37.5 0 0 0.25 0.25 0.222 0.27 0.308 0.252 90.4 -1\nFriedl, TJ 5 5 2 0 0 2 0 0 14.3 0 0 0.4 1.6 0.834 0.231 0.604 0.348 94.1 24\nBenson, Will 3 3 0 0 0 0 3 100 88.9 0 0 0 0 0 0\nFraley, Jake 4 3 2 1 0 0 0 0 0 0 0 0.667 1 0.54 0.432 0.486 0.548 81.7 5.8'}, {'pitcher_id': '640470', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlzolay, Adbert 41 37 6 2 0 2 16 39 27.8 3 7.3 0.162 0.378 0.269 0.163 0.285 0.239 86.2 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 11 9 2 0 0 0 2 18.2 14.3 1 9.1 0.222 0.222 0.289 0.245 0.271 0.314 85.2 -3.1\nStephenson, Tyler 13 12 2 2 0 0 5 38.5 33.3 1 7.7 0.167 0.333 0.244 0.135 0.17 0.173 85.4 2.7\nBenson, Will 3 3 0 0 0 0 3 100 44.4 0 0 0 0 0 0\nFraley, Jake 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nFairchild, Stuart 3 3 1 0 0 1 1 33.3 33.3 0 0 0.333 1.333 0.668 0.205 0.624 0.339 92 6\nSteer, Spencer 4 3 0 0 0 0 0 0 0 1 25 0 0 0.172 0.023 0.024 0.188 76.7 46.7\nDe La Cruz, Elly 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.336 0.369 0.31 97.7 8\nCandelario, Jeimer 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMaile, Luke 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMartini, Nick 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.86 3.372 1.697 105.1 30'}, {'pitcher_id': '643410', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeiter Jr., Mark 39 35 8 3 0 1 15 38.5 38.4 2 5.1 0.229 0.4 0.299 0.197 0.294 0.248 86.6 7.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFriedl, TJ 7 5 1 0 0 1 4 57.1 46.7 1 14.3 0.2 0.8 0.45 0.096 0.366 0.271 68.3 -18\nFraley, Jake 6 6 2 2 0 0 2 33.3 25 0 0 0.333 0.667 0.415 0.284 0.396 0.29 96.8 10\nIndia, Jonathan 4 4 2 0 0 0 1 25 0 0 0 0.5 0.5 0.442 0.417 0.46 0.382 85.6 10.3\nDe La Cruz, Elly 4 3 1 0 0 0 1 25 42.9 1 25 0.333 0.333 0.394 0.388 0.61 0.494 106.6 12.5\nBenson, Will 4 4 0 0 0 0 4 100 88.9 0 0 0 0 0 0\nSteer, Spencer 3 2 0 0 0 0 1 33.3 42.9 0 0 0 0 0.242 0.01 0.013 0.248 84.2 34\nEspinal, Santiago 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.265 0.445 0.292 63.9 -6.5\nCandelario, Jeimer 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.08 0.086 0.072 93.7 14.5\nStephenson, Tyler 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.083 0.091 0.075 85.3 -5\nSuter, Brent 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.88 0.653 0.71 0.588 77.7 14\nHurtubise, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.092 0.103 0.085 83.4 -12\nMaile, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.187 0.242 0.171 92.7 -6\nMartini, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.016 0.009 85.5 62'}, {'pitcher_id': '624522', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThompson, Keegan 30 27 6 4 0 0 10 33.3 38.1 2 6.7 0.222 0.37 0.296 0.215 0.266 0.26 84.4 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 7 7 1 0 0 0 3 42.9 45.5 0 0 0.143 0.143 0.126 0.134 0.144 0.122 78.1 -5.3\nStephenson, Tyler 4 3 1 1 0 0 2 50 50 1 25 0.333 0.667 0.484 0.305 0.358 0.391 91.6 15\nSteer, Spencer 4 3 1 1 0 0 1 25 57.1 1 25 0.333 0.667 0.485 0.164 0.252 0.305 79.7 16.5\nFraley, Jake 3 3 1 1 0 0 1 33.3 16.7 0 0 0.333 0.667 0.42 0.409 0.474 0.391 89.6 10.5\nDe La Cruz, Elly 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nFriedl, TJ 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.251 0.174 85.4 11\nMaile, Luke 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMartini, Nick 1 0 0 0 0 0 0 0 0 0 0 0.726 0.726\nFairchild, Stuart 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.052 0.093 0.061 82.9 44.5\nBenson, Will 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.063 0.71 0.855 0.683 93.2 11'}, {'pitcher_id': '681432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLittle, Luke 3 2 1 1 0 0 0 0 33.3 1 33.3 0.5 1 0.654 0.58 0.906 0.649 98.9 21.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.226 0.608 0.333 94.8 30\nFriedl, TJ 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.934 1.204 0.921 103 13'}, {'pitcher_id': '593576', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNeris, Héctor 27 23 5 1 0 1 10 37 23.4 2 7.4 0.217 0.391 0.31 0.192 0.333 0.278 86.9 9.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 6 5 1 0 0 0 3 50 15.4 1 16.7 0.2 0.2 0.263 0.191 0.231 0.27 91.8 -5\nIndia, Jonathan 6 5 1 1 0 0 3 50 55.6 0 0 0.2 0.4 0.333 0.159 0.3 0.281 92.6 10.5\nEspinal, Santiago 5 4 0 0 0 0 2 40 20 0 0 0 0 0 0.028 0.034 0.028 69.9 7.3\nStephenson, Tyler 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.669 0.244 0.835 0.434 90.3 20\nFriedl, TJ 2 1 1 0 0 0 0 0 33.3 1 50 1 1 0.792 0.248 0.274 0.46 96.1 -9\nSteer, Spencer 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.444 0.491 0.513 0.443 73.5 21\nMaile, Luke 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDe La Cruz, Elly 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.297 0.527 0.353 99 17'}, {'pitcher_id': '665871', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAssad, Javier 106 93 24 4 0 2 28 26.4 22.6 10 9.4 0.258 0.366 0.315 0.289 0.432 0.35 85.3 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 17 14 3 1 0 0 6 35.3 35.7 3 17.6 0.214 0.286 0.299 0.176 0.249 0.274 89.4 16\nFriedl, TJ 14 11 1 0 0 0 4 28.6 25 3 21.4 0.091 0.091 0.212 0.185 0.228 0.291 93.8 22.3\nIndia, Jonathan 11 11 2 0 0 0 3 27.3 15.8 0 0 0.182 0.182 0.161 0.239 0.318 0.24 88.1 19.9\nFraley, Jake 15 13 5 0 0 1 2 13.3 10.7 2 13.3 0.385 0.615 0.466 0.354 0.509 0.419 78.8 22.8\nDe La Cruz, Elly 11 11 3 1 0 1 5 45.5 31.6 0 0 0.273 0.636 0.383 0.203 0.459 0.275 90.4 8.7\nCandelario, Jeimer 7 7 1 0 0 0 1 14.3 14.3 0 0 0.143 0.143 0.127 0.29 0.351 0.279 78.3 -4.5\nBenson, Will 8 7 3 1 0 0 3 37.5 22.2 1 12.5 0.429 0.571 0.467 0.421 0.953 0.586 93.8 23.8\nFairchild, Stuart 4 3 1 1 0 0 1 25 27.3 1 25 0.333 0.667 0.485 0.308 0.406 0.406 95.2 39.5\nStephenson, Tyler 7 6 4 0 0 0 1 14.3 37.5 0 0 0.667 0.667 0.611 0.515 0.597 0.514 85.5 10.2\nMartini, Nick 5 5 1 0 0 0 0 0 16.7 0 0 0.2 0.2 0.178 0.601 0.831 0.615 92.8 17.6\nMaile, Luke 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.228 0.244 0.208 82.8 -3\nHurtubise, Jacob 3 1 0 0 0 0 1 33.3 0 0 0 0 0 0 0 0 0 28 0.5'}, {'pitcher_id': '694037', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPalencia, Daniel 11 11 3 0 1 0 4 36.4 26.1 0 0 0.273 0.455 0.303 0.242 0.332 0.245 100.2 16.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.441 0\nStephenson, Tyler 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.441 0.557 0.633 0.52 102.2 3.5\nFraley, Jake 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMaile, Luke 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMartini, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.15 0.32 0.198 95 36\nFriedl, TJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.571 0.655 0.53 103.8 4\nDe La Cruz, Elly 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBenson, Will 1 1 1 0 1 0 0 0 0 0 0 1 3 1.569 0.437 0.763 0.485 103.1 17'}, {'pitcher_id': '684007', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nImanaga, Shota 23 22 5 0 0 1 7 30.4 30 1 4.3 0.227 0.364 0.275 0.27 0.47 0.328 86 28.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairchild, Stuart 3 3 0 0 0 0 2 66.7 37.5 0 0 0 0 0 0.038 0.129 0.068 95.9 37\nFriedl, TJ 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0 0 0 84.2 73\nSteer, Spencer 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.455 0.536 0.514 84.3 14.5\nCandelario, Jeimer 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.331 0.699 0.422 83.6 25.5\nDe La Cruz, Elly 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.039 0.064 0.042 86.4 44.7\nMaile, Luke 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.043 0.425 1.575 0.814 103.3 27\nEspinal, Santiago 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.672 0.765 0.627 79.9 8.7\nIndia, Jonathan 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.296 0.315 0.381 0.302 86.1 25'}, {'pitcher_id': '696136', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWicks, Jordan 22 17 6 1 0 0 2 9.1 16.7 3 13.6 0.353 0.412 0.389 0.35 0.44 0.396 78.6 4.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Elly 5 3 1 0 0 0 1 20 11.1 1 20 0.333 0.333 0.396 0.227 0.401 0.372 74.4 -4.3\nSteer, Spencer 4 4 3 1 0 0 0 0 16.7 0 0 0.75 1 0.759 0.696 0.834 0.67 85.7 16.8\nFairchild, Stuart 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.013 0.035 0.018 90.7 57\nStephenson, Tyler 3 2 1 0 0 0 0 0 33.3 1 33.3 0.5 0.5 0.526 0.583 0.72 0.614 86.8 10.5\nIndia, Jonathan 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.159 0.172 0.145 81.2 -8.5\nFriedl, TJ 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.182 0.208 0.429 75.8 2\nCandelario, Jeimer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.46 0.522 0.425 71.7 36\nFraley, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.318 0.319 0.287 66.2 -55\nBenson, Will 1 0 0 0 0 0 0 0 0 0 0 38.9 -71'}, {'pitcher_id': '621016', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCuas, Jose 16 13 4 0 1 0 4 25 33.3 2 12.5 0.308 0.462 0.396 0.187 0.247 0.283 90.5 2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.1 0.321 0.433 92.2 32\nFriedl, TJ 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.204 0.219 0.184 89 5\nStephenson, Tyler 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.163 0.167 0.147 97.7 4\nCandelario, Jeimer 2 1 1 0 1 0 0 0 66.7 0 0 1 3 1.145 0.054 0.112 0.393 85.4 29\nFairchild, Stuart 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFraley, Jake 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.506 0.688 0.51 101.2 2\nBenson, Will 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nEspinal, Santiago 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.169 0.189 0.157 92.7 -16\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.059 0.066 0.06 61.7 -24\nIndia, Jonathan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.143 0.203 0.129 93.5 -16'}, {'pitcher_id': '657240', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerryweather, Julian 16 14 4 1 0 2 4 25 42.3 1 6.3 0.286 0.786 0.472 0.251 0.579 0.391 88.3 23.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 5 4 1 0 0 1 2 40 50 1 20 0.25 1 0.54 0.237 0.823 0.483 97.8 27\nFraley, Jake 3 3 1 0 0 0 1 33.3 60 0 0 0.333 0.333 0.294 0.321 0.354 0.292 67.7 12\nStephenson, Tyler 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 0 0 0 0 0 0\nCandelario, Jeimer 2 2 1 0 0 1 0 0 50 0 0 0.5 2 1.002 0.235 0.884 0.45 87.5 52\nFriedl, TJ 1 0 0 0 0 0 0 0 100 0 0 0.726 0.726\nFairchild, Stuart 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.46 0.724 0.496 99 17\nIndia, Jonathan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.231 0.458 0.276 96.2 22\nBenson, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.193 0.219 0.177 93.4 -11'}, {'pitcher_id': '592767', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmyly, Drew 93 80 18 6 1 3 21 22.6 23.6 9 9.7 0.225 0.438 0.337 0.183 0.258 0.262 82 18.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 17 13 3 1 0 0 4 23.5 16.7 2 11.8 0.231 0.308 0.343 0.194 0.238 0.31 89 12.7\nStephenson, Tyler 19 15 3 1 0 0 3 15.8 14.8 4 21.1 0.2 0.267 0.304 0.188 0.272 0.302 86.7 34.4\nSteer, Spencer 12 11 3 1 0 1 0 0 9.7 1 8.3 0.273 0.636 0.404 0.246 0.4 0.311 85.3 33.6\nFairchild, Stuart 9 8 2 1 0 0 3 33.3 36.4 0 0 0.25 0.375 0.317 0.223 0.325 0.291 71.5 19.6\nCandelario, Jeimer 8 7 1 0 0 1 2 25 30 0 0 0.143 0.571 0.341 0.103 0.15 0.185 80.7 19\nBenson, Will 4 3 0 0 0 0 3 75 55.6 1 25 0 0 0.174 0.174\nMaile, Luke 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.044 0.044 0.041 60.7 21\nFraley, Jake 5 5 2 1 0 0 0 0 18.2 0 0 0.4 0.6 0.427 0.196 0.213 0.184 71.6 -8.6\nFriedl, TJ 4 4 3 1 1 0 1 25 33.3 0 0 0.75 1.5 0.924 0.356 0.427 0.342 74.6 -3\nDe La Cruz, Elly 4 3 0 0 0 0 2 50 75 1 25 0 0 0.173 0.049 0.057 0.206 87.7 -14\nEspinal, Santiago 4 4 1 0 0 1 1 25 37.5 0 0 0.25 1 0.521 0.293 0.533 0.356 87.4 -11\nHurtubise, Jacob 1 1 0 0 0 0 1 100 16.7 0 0 0 0 0 0\nMartini, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.087 0.081 92.5 18.5'}], 114: [{'pitcher_id': '622075', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlmonte, Yency 5 4 1 0 0 0 2 40 10 0 0 0.25 0.25 0.32 0.188 0.217 0.285 96.6 -0.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 3 2 0 0 0 0 2 66.7 25 0 0 0 0 0.243 0.243\nNaylor, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.375 0.435 0.348 96.6 -0.5'}, {'pitcher_id': '605155', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrewer, Colten 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.126 0.15 0.117 76.6 -6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.282 0.355 0.259 95.7 -4\nGiménez, Andrés 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.05 0.052 0.05 67.2 -23\nNaylor, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.173 0.192 0.159 67 7'}, {'pitcher_id': '543294', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHendricks, Kyle 16 14 2 0 0 1 5 31.3 17.2 1 6.3 0.143 0.357 0.225 0.206 0.29 0.232 90.3 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 10 9 1 0 0 1 3 30 18.2 0 0 0.111 0.444 0.203 0.204 0.278 0.189 89.7 23.3\nNaylor, Josh 5 4 1 0 0 0 2 40 16.7 1 20 0.25 0.25 0.314 0.184 0.295 0.3 89.4 7.5\nLively, Ben 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.307 0.369 0.316 96.1 -1'}, {'pitcher_id': '669713', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWesneski, Hayden 4 4 1 0 0 0 1 25 50 0 0 0.25 0.25 0.221 0.17 0.197 0.161 90 -10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGiménez, Andrés 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.17 0.196 0.157 91.7 -15\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.099 0.123 0.093 98.1 -24\nFry, David 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nNaylor, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.411 0.469 0.394 80.2 9'}, {'pitcher_id': '668338', 'player_data': '', 'team_data': 'Tyson Miller has not faced anyone on this team.'}, {'pitcher_id': '687863', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657006', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteele, Justin 15 13 1 0 0 0 4 26.7 31 1 6.7 0.077 0.077 0.154 0.145 0.164 0.212 79.7 -1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFry, David 3 2 1 0 0 0 1 33.3 33.3 1 33.3 0.5 0.5 0.526 0.33 0.377 0.439 94.1 9\nFreeman, Tyler 3 2 0 0 0 0 1 33.3 33.3 0 0 0 0 0.242 0.029 0.029 0.26 58.6 -17\nGiménez, Andrés 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.22 0.251 0.206 78.1 5.7\nNaylor, Bo 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.032 0.032 0.03 82.7 -21\nBrennan, Will 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.147 0.166 0.135 82.6 -0.7\nNaylor, Josh 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '592791', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaillon, Jameson 29 28 9 4 0 1 7 24.1 22.2 0 0 0.321 0.571 0.388 0.254 0.397 0.295 84.3 9.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Josh 8 8 2 1 0 0 2 25 10.5 0 0 0.25 0.375 0.268 0.236 0.338 0.247 86.4 -12.5\nGiménez, Andrés 7 6 3 1 0 1 3 42.9 23.5 0 0 0.5 1.167 0.694 0.268 0.699 0.446 91.5 0.3\nHedges, Austin 6 6 1 1 0 0 1 16.7 30.8 0 0 0.167 0.333 0.205 0.072 0.112 0.075 76 57.8\nBrennan, Will 3 3 1 0 0 0 0 0 50 0 0 0.333 0.333 0.294 0.5 0.541 0.462 81.9 2\nKwan, Steven 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.715 0.428 0.505 0.416 82.3 -8.3\nFry, David 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.197 0.207 0.179 105.7 -5'}, {'pitcher_id': '676962', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '640470', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlzolay, Adbert 9 8 4 0 0 1 0 0 20 0 0 0.5 0.875 0.597 0.426 0.929 0.587 87.7 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Josh 5 5 3 0 0 1 0 0 16.7 0 0 0.6 1.2 0.754 0.518 1.249 0.73 103.2 6.2\nHedges, Austin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.28 0.46 0.317 76.7 34\nGiménez, Andrés 2 1 0 0 0 0 0 0 66.7 0 0 0 0 0.361 0.257 0.264 0.498 32 -35'}, {'pitcher_id': '643410', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeiter Jr., Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.034 0.036 65.3 -22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.034 0.036 65.3 -22'}, {'pitcher_id': '624522', 'player_data': '', 'team_data': 'Keegan Thompson has not faced anyone on this team.'}, {'pitcher_id': '681432', 'player_data': '', 'team_data': 'Luke Little has not faced anyone on this team.'}, {'pitcher_id': '593576', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNeris, Héctor 11 9 3 0 0 0 2 18.2 31.8 1 9.1 0.333 0.333 0.301 0.245 0.287 0.253 84.7 39.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 3 1 1 0 0 0 0 0 50 1 33.3 1 1 0.519 0.847 0.901 0.487 80.7 39\nNaylor, Josh 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.435 0.183 0.211 0.172 77.9 36\nKwan, Steven 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nArias, Gabriel 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.001 0.001 0 92.3 71\nGiménez, Andrés 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.477 0.585 0.464 89.2 29.5\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.086 0.051 89.4 33'}, {'pitcher_id': '665871', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAssad, Javier 6 5 0 0 0 0 2 33.3 17.6 1 16.7 0 0 0.116 0.278 0.344 0.34 85.9 23.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.051 0.096 0.379 80.6 33\nNaylor, Josh 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nGiménez, Andrés 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.526 0.658 0.515 87.2 20\nNaylor, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.814 0.967 0.77 89.8 17'}, {'pitcher_id': '694037', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '684007', 'player_data': '', 'team_data': 'Shota Imanaga has not faced anyone on this team.'}, {'pitcher_id': '696136', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '621016', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCuas, Jose 10 9 2 1 0 0 4 40 55.6 1 10 0.222 0.333 0.282 0.136 0.187 0.196 83.7 -3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nFry, David 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nBrennan, Will 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.622 0.497 0.575 0.469 79.2 -14\nHedges, Austin 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.442 0.065 0.073 0.062 82.4 -13\nNaylor, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.39 0.2 95.2 37\nGiménez, Andrés 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nNaylor, Bo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '657240', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerryweather, Julian 5 4 2 0 0 0 1 20 8.3 0 0 0.5 0.5 0.353 0.425 1.018 0.599 100.1 -3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Bo 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nHedges, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 -63\nGiménez, Andrés 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.7 2.426 1.3 101.8 27\nNaylor, Josh 1 0 0 0 0 0 0 0 0 0 0 0 0.575 101.7 19\nBrennan, Will 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.574 0.629 0.522 96.7 5'}, {'pitcher_id': '592767', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmyly, Drew 7 6 1 0 0 1 1 14.3 23.1 0 0 0.167 0.667 0.334 0.213 0.447 0.275 83.2 11.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 5 5 1 0 0 1 1 20 33.3 0 0 0.2 0.8 0.401 0.241 0.518 0.315 94.7 34.8\nCarrasco, Carlos 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.093 0.075 60.3 -35.5'}]}, 113: {112: [{'pitcher_id': '571656', 'player_data': '', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomes, Yan 14 14 1 1 0 0 5 35.7 38.9 0 0 0.071 0.143 0.089 0.153 0.171 0.144 78.4 11.3\nMorel, Christopher 7 6 1 1 0 0 2 28.6 26.7 1 14.3 0.167 0.333 0.277 0.247 0.433 0.347 84.7 5.3\nHapp, Ian 7 6 2 1 0 0 2 28.6 56.3 1 14.3 0.333 0.5 0.403 0.221 0.402 0.327 98.1 3.5\nSwanson, Dansby 5 4 0 0 0 0 2 40 30.8 1 20 0 0 0.139 0.041 0.043 0.171 78 -26.5\nHoerner, Nico 7 7 3 2 0 0 0 0 0 0 0 0.429 0.714 0.484 0.242 0.333 0.247 90.3 2.9\nBote, David 4 3 1 1 0 0 1 25 33.3 1 25 0.333 0.667 0.482 0.375 1.05 0.607 101 25\nSuzuki, Seiya 5 5 1 0 0 0 1 20 28.6 0 0 0.2 0.2 0.178 0.188 0.356 0.226 80.7 0\nWisdom, Patrick 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.399 1.569 0.788 105.3 32\nBellinger, Cody 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.222 0.358 0.399 0.335 82.7 -4\nTauchman, Mike 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 67.5 87'}, {'pitcher_id': '641941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPagán, Emilio 19 18 4 1 0 2 5 26.3 33.3 0 0 0.222 0.611 0.321 0.204 0.53 0.29 91.2 33.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBellinger, Cody 7 7 2 0 0 2 2 28.6 52.9 0 0 0.286 1.143 0.564 0.275 1.038 0.521 98.6 46.6\nGomes, Yan 3 3 1 1 0 0 0 0 7.7 0 0 0.333 0.667 0.423 0.17 0.304 0.196 84.2 38\nWisdom, Patrick 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.01 0.012 0.009 97 68\nTauchman, Mike 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.011 0.011 0.01 64.4 59\nMorel, Christopher 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.097 0.099 0.087 60.8 -46\nBusch, Michael 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBote, David 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.44 0.484 0.409 105.5 -1\nHapp, Ian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.569 0.642 0.527 110.1 2\nSwanson, Dansby 1 0 0 0 0 0 0 0 0 0 0 0 0.14 93.5 39'}, {'pitcher_id': '666157', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLodolo, Nick 82 68 16 3 0 1 18 22 22 10 12.2 0.235 0.324 0.321 0.197 0.275 0.295 89.1 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuzuki, Seiya 12 6 2 1 0 0 2 16.7 0 6 50 0.333 0.5 0.524 0.12 0.138 0.428 94.5 29\nWisdom, Patrick 12 10 2 0 0 0 6 50 33.3 0 0 0.2 0.2 0.268 0.044 0.053 0.166 90.5 24.3\nHapp, Ian 12 10 2 1 0 0 4 33.3 42.9 1 8.3 0.2 0.3 0.297 0.127 0.148 0.225 85.1 18.8\nHoerner, Nico 9 8 3 0 0 0 0 0 0 1 11.1 0.375 0.375 0.372 0.327 0.477 0.404 83.5 28.2\nMorel, Christopher 10 8 2 0 0 0 1 10 23.1 1 10 0.25 0.25 0.319 0.284 0.315 0.36 90.1 14.2\nBote, David 8 7 0 0 0 0 2 25 33.3 1 12.5 0 0 0.086 0.146 0.196 0.213 96.4 22.4\nSwanson, Dansby 6 6 1 0 0 1 2 33.3 33.3 0 0 0.167 0.667 0.348 0.28 0.593 0.365 96.9 8.8\nBellinger, Cody 6 6 2 0 0 0 1 16.7 12.5 0 0 0.333 0.333 0.296 0.33 0.398 0.312 84.1 32.4\nAmaya, Miguel 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.088 0.076 79.3 -11.8\nBusch, Michael 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.597 0.973 0.656 103.9 16.5'}, {'pitcher_id': '458677', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Justin 25 24 2 0 0 0 10 40 30.2 0 0 0.083 0.083 0.1 0.119 0.159 0.148 73.4 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTauchman, Mike 6 6 1 0 0 0 3 50 26.7 0 0 0.167 0.167 0.148 0.192 0.293 0.202 77.2 14.7\nSwanson, Dansby 4 4 1 0 0 0 3 75 33.3 0 0 0.25 0.25 0.221 0.195 0.237 0.202 68.8 34\nGomes, Yan 5 4 0 0 0 0 2 40 16.7 0 0 0 0 0.144 0.115 0.143 0.239 70.8 32\nBellinger, Cody 4 4 0 0 0 0 1 25 50 0 0 0 0 0 0.025 0.033 0.028 78.8 16.7\nHapp, Ian 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.076 0.083 0.075 62.2 20.7\nBote, David 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWisdom, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.079 0.073 88.8 -30\nMorel, Christopher 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.066 0.075 0.063 73.2 -17'}, {'pitcher_id': '608371', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSims, Lucas 48 42 11 2 0 2 16 33.3 29.3 6 12.5 0.262 0.452 0.352 0.246 0.376 0.32 84.8 9.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 9 8 1 0 0 0 4 44.4 46.2 1 11.1 0.125 0.125 0.175 0.284 0.325 0.315 98.8 0.5\nHoerner, Nico 6 4 1 0 0 0 2 33.3 42.9 2 33.3 0.25 0.25 0.379 0.156 0.18 0.332 75.7 -15\nSwanson, Dansby 7 7 2 0 0 1 1 14.3 15 0 0 0.286 0.714 0.425 0.246 0.408 0.279 86.5 14.2\nBellinger, Cody 6 6 2 0 0 1 0 0 9.1 0 0 0.333 0.833 0.477 0.491 0.98 0.612 88.4 13.7\nWisdom, Patrick 3 2 0 0 0 0 2 66.7 12.5 1 33.3 0 0 0.232 0.232\nMorel, Christopher 4 3 0 0 0 0 3 75 100 1 25 0 0 0.173 0.173\nSuzuki, Seiya 4 3 2 1 0 0 1 25 33.3 1 25 0.667 1 0.706 0.337 0.558 0.455 94.8 20\nGomes, Yan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.003 0.004 0.003 86.7 65\nAmaya, Miguel 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nBote, David 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.094 0.094 0.096 69.1 8.5\nTauchman, Mike 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.063 0.627 0.779 0.617 82.2 -20\nCrow-Armstrong, Pete 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.295 0.303 0.266 30.5 36'}, {'pitcher_id': '607259', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Nick 35 35 6 3 0 0 11 31.4 26.4 0 0 0.171 0.257 0.183 0.193 0.293 0.208 84.9 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuzuki, Seiya 4 4 0 0 0 0 2 50 20 0 0 0 0 0 0.197 0.208 0.178 99.7 -4.5\nSwanson, Dansby 6 6 2 2 0 0 1 16.7 11.1 0 0 0.333 0.667 0.416 0.274 0.319 0.257 78.1 -2\nHapp, Ian 6 6 1 0 0 0 1 16.7 40 0 0 0.167 0.167 0.147 0.132 0.25 0.159 80.9 -1.2\nBellinger, Cody 4 4 1 1 0 0 2 50 44.4 0 0 0.25 0.5 0.317 0.283 0.709 0.415 100.3 12.5\nWisdom, Patrick 4 4 1 0 0 0 1 25 10 0 0 0.25 0.25 0.221 0.16 0.233 0.169 92.3 34\nHoerner, Nico 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.273 0.301 0.255 76.8 21\nGomes, Yan 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.2 0.334 0.224 96.3 18\nBusch, Michael 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.01 0.01 0.009 83.3 59\nAmaya, Miguel 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.058 0.058 0.053 71.6 -9\nTauchman, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0\nMorel, Christopher 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '668933', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAshcraft, Graham 109 94 27 5 0 5 22 20.2 25 11 10.1 0.287 0.5 0.386 0.259 0.432 0.351 89.8 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 16 13 5 1 0 1 5 31.3 24.2 3 18.8 0.385 0.692 0.503 0.274 0.4 0.367 92.3 20.4\nSuzuki, Seiya 16 13 1 0 0 0 3 18.8 14.3 2 12.5 0.077 0.077 0.187 0.16 0.232 0.266 87.5 -4.2\nHoerner, Nico 17 15 3 1 0 0 1 5.9 16.7 2 11.8 0.2 0.267 0.259 0.283 0.374 0.333 91.4 8.2\nWisdom, Patrick 12 11 3 0 0 2 4 33.3 22.9 0 0 0.273 0.818 0.473 0.245 0.684 0.416 95.4 7.4\nMorel, Christopher 12 11 4 1 0 0 2 16.7 33.3 1 8.3 0.364 0.455 0.384 0.238 0.306 0.277 85.9 13.3\nGomes, Yan 8 5 1 1 0 0 2 25 43.8 1 12.5 0.2 0.4 0.425 0.169 0.226 0.377 74.3 18.3\nTauchman, Mike 8 7 3 0 0 0 2 25 30 1 12.5 0.429 0.429 0.418 0.26 0.294 0.299 90.9 6.6\nSwanson, Dansby 7 6 3 1 0 0 1 14.3 30 1 14.3 0.5 0.667 0.533 0.442 0.679 0.51 85.6 0.6\nBellinger, Cody 5 5 1 0 0 1 0 0 0 0 0 0.2 0.8 0.401 0.383 0.991 0.565 93.8 2.4\nBote, David 4 4 2 0 0 1 0 0 11.1 0 0 0.5 1.25 0.739 0.317 0.547 0.371 93.2 33.5\nAmaya, Miguel 2 2 0 0 0 0 2 100 83.3 0 0 0 0 0 0\nCrow-Armstrong, Pete 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.305 0.754 0.435 92.9 2.5'}, {'pitcher_id': '594580', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoll, Sam 24 23 3 1 1 0 10 41.7 24.1 1 4.2 0.13 0.261 0.183 0.113 0.234 0.17 90.7 6.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBellinger, Cody 8 8 0 0 0 0 5 62.5 37.5 0 0 0 0 0 0.04 0.042 0.037 74.1 -30.3\nHapp, Ian 4 4 1 0 0 0 2 50 8.3 0 0 0.25 0.25 0.221 0.067 0.069 0.061 98.6 -24.5\nSuzuki, Seiya 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.163 0.39 0.467 92.4 26\nWisdom, Patrick 2 2 1 0 1 0 0 0 20 0 0 0.5 1.5 0.784 0.291 1.107 0.561 100.7 48.5\nMorel, Christopher 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.007 0.007 0.006 85 59\nGomes, Yan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.196 0.216 0.178 107.3 -6\nTauchman, Mike 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.622 0.243 0.636 0.371 98.7 24\nHoerner, Nico 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.15 0.106 77 39\nSwanson, Dansby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.268 0.296 0.254 97.8 -15'}, {'pitcher_id': '518585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCruz, Fernando 44 39 13 5 0 2 11 25 26.7 4 9.1 0.333 0.615 0.419 0.244 0.443 0.324 88.7 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 7 6 2 1 0 0 2 28.6 17.6 1 14.3 0.333 0.5 0.403 0.192 0.462 0.341 79.4 0.7\nMorel, Christopher 4 3 0 0 0 0 3 75 75 1 25 0 0 0.172 0.172\nSwanson, Dansby 5 5 2 2 0 0 2 40 33.3 0 0 0.4 0.8 0.498 0.24 0.514 0.313 93.5 28\nHoerner, Nico 6 6 3 0 0 2 0 0 12.5 0 0 0.5 1.5 0.826 0.401 0.932 0.559 89.5 30.8\nSuzuki, Seiya 4 3 1 0 0 0 2 50 60 1 25 0.333 0.333 0.393 0.065 0.066 0.217 96.4 -14\nBellinger, Cody 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.395 0.292 0.332 0.376 94.7 8\nWisdom, Patrick 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.074 0.074 0.067 56.9 40\nTauchman, Mike 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.423 0.221 0.537 0.309 97.2 29\nBusch, Michael 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.6 0.668 0.552 96.8 9\nBote, David 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.382 0.456 0.365 102.7 9\nCrow-Armstrong, Pete 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.216 0.316 0.226 92.6 22.5\nGomes, Yan 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.294 0.483 0.558 0.352 85.8 9.7'}, {'pitcher_id': '664747', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Alexis 45 40 10 3 0 1 15 33.3 35.2 4 8.9 0.25 0.4 0.326 0.206 0.325 0.282 90.2 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuzuki, Seiya 8 8 3 2 0 1 2 25 38.5 0 0 0.375 1 0.573 0.207 0.336 0.229 88.4 10\nHapp, Ian 8 8 1 0 0 0 3 37.5 30 0 0 0.125 0.125 0.11 0.059 0.122 0.077 92.8 15.8\nHoerner, Nico 7 6 2 0 0 0 3 42.9 40 1 14.3 0.333 0.333 0.351 0.252 0.277 0.298 97.2 3\nBusch, Michael 4 1 0 0 0 0 1 25 37.5 3 75 0 0 0.519 0.519\nSwanson, Dansby 4 4 1 0 0 0 1 25 12.5 0 0 0.25 0.25 0.221 0.161 0.312 0.201 83.7 10\nWisdom, Patrick 3 3 0 0 0 0 2 66.7 44.4 0 0 0 0 0 0.005 0.008 0.006 91.1 50\nCrow-Armstrong, Pete 4 4 1 0 0 0 1 25 50 0 0 0.25 0.25 0.222 0.44 0.525 0.419 89.7 24.3\nBellinger, Cody 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.211 0.838 0.429 107.3 42\nTauchman, Mike 2 1 1 0 0 0 0 0 0 0 0 1 1 0.804 0.849 1.072 0.783 98.9 14\nMorel, Christopher 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAmaya, Miguel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.549 1.164 0.718 101.8 21\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.174 0.16 47.1 -42'}, {'pitcher_id': '622088', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAntone, Tejay 17 17 4 0 0 3 4 23.5 34.6 0 0 0.235 0.765 0.403 0.264 0.614 0.366 88.3 -3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 5 5 2 0 0 2 1 20 36.4 0 0 0.4 1.6 0.797 0.256 0.756 0.414 101 8.8\nBote, David 5 5 1 0 0 1 3 60 66.7 0 0 0.2 0.8 0.396 0.217 0.819 0.415 91.1 8\nHoerner, Nico 5 5 1 0 0 0 0 0 14.3 0 0 0.2 0.2 0.177 0.365 0.434 0.353 82.6 -26.2\nSwanson, Dansby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.112 0.116 0.103 78.5 -9\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.197 0.278 0.213 70.2 50'}, {'pitcher_id': '593423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontas, Frankie 20 16 6 4 0 0 7 35 30 4 20 0.375 0.625 0.48 0.347 0.443 0.413 91.2 13.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBellinger, Cody 6 5 2 1 0 0 2 33.3 41.7 1 16.7 0.4 0.6 0.471 0.43 0.497 0.457 92.3 9.3\nGomes, Yan 4 4 0 0 0 0 3 75 33.3 0 0 0 0 0 0.027 0.08 0.043 91.9 33\nTauchman, Mike 2 1 1 0 0 0 0 0 16.7 1 50 1 1 0.791 0.914 1.009 0.769 80.1 16\nSuzuki, Seiya 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.635 0.291 0.512 0.336 106.4 16\nHapp, Ian 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.866 1.172 0.871 106.4 13\nBusch, Michael 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nSwanson, Dansby 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nCrow-Armstrong, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.114 0.088 72.1 -3\nMorel, Christopher 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nAmaya, Miguel 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.834 0.973 0.784 86.9 18'}, {'pitcher_id': '686730', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpiers, Carson 36 31 8 1 0 0 12 33.3 24.7 3 8.3 0.258 0.29 0.285 0.184 0.225 0.231 85.6 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuzuki, Seiya 4 4 1 0 0 0 2 50 27.3 0 0 0.25 0.25 0.222 0.202 0.249 0.195 79.8 -10\nBellinger, Cody 4 4 1 0 0 0 2 50 41.7 0 0 0.25 0.25 0.221 0.278 0.356 0.274 82.1 20\nTauchman, Mike 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.426 0.509 0.407 91.6 -8.8\nHapp, Ian 5 3 1 0 0 0 1 20 25 1 20 0.333 0.333 0.315 0.254 0.29 0.284 90.6 14.3\nSwanson, Dansby 5 5 2 1 0 0 1 20 7.7 0 0 0.4 0.6 0.432 0.191 0.24 0.184 90.6 20.3\nAmaya, Miguel 4 2 0 0 0 0 1 25 16.7 1 25 0 0 0.355 0.046 0.056 0.378 69.3 -2\nBusch, Michael 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.002 0.004 0.002 86.6 43\nHoerner, Nico 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nMorel, Christopher 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.296 0.091 0.108 0.085 69.4 31.5\nCrow-Armstrong, Pete 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.002 0.003 0.002 94.3 48'}, {'pitcher_id': '668881', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Hunter 106 90 19 6 1 6 26 24.5 26.2 12 11.3 0.211 0.5 0.353 0.257 0.447 0.357 92.8 25.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 15 11 2 1 0 1 5 33.3 37.1 3 20 0.182 0.545 0.409 0.181 0.368 0.356 97 36.7\nHoerner, Nico 15 13 3 1 0 1 1 6.7 6.7 1 6.7 0.231 0.538 0.376 0.275 0.339 0.326 83 20.3\nSwanson, Dansby 11 10 0 0 0 0 6 54.5 40 1 9.1 0 0 0.063 0.079 0.092 0.129 94 2.5\nMorel, Christopher 14 11 3 1 0 1 2 14.3 18.2 2 14.3 0.273 0.636 0.453 0.346 0.652 0.485 94.2 23.8\nSuzuki, Seiya 11 11 5 1 1 2 2 18.2 20 0 0 0.455 1.273 0.721 0.405 0.975 0.575 97.7 27.6\nTauchman, Mike 9 7 0 0 0 0 3 33.3 28.6 2 22.2 0 0 0.154 0.05 0.057 0.19 88 41.5\nBellinger, Cody 10 9 4 1 0 1 2 20 31.3 0 0 0.444 0.889 0.511 0.294 0.522 0.349 93.5 33.3\nCrow-Armstrong, Pete 6 5 0 0 0 0 3 50 20 1 16.7 0 0 0.115 0.101 0.242 0.235 98.8 43.5\nWisdom, Patrick 5 4 0 0 0 0 1 20 53.8 1 20 0 0 0.138 0.2 0.402 0.345 97.8 28.3\nAmaya, Miguel 6 5 1 0 0 0 0 0 22.2 1 16.7 0.2 0.2 0.264 0.578 0.682 0.577 100.1 2.6\nBote, David 2 2 0 0 0 0 1 50 14.3 0 0 0 0 0 0.217 0.272 0.199 95.4 1\nGomes, Yan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.434 0.527 0.425 82 48.5'}, {'pitcher_id': '671096', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbbott, Andrew 60 51 13 6 0 1 12 20 22.6 7 11.7 0.255 0.431 0.328 0.285 0.47 0.377 91.2 21.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorel, Christopher 7 6 0 0 0 0 5 71.4 52.9 1 14.3 0 0 0.099 0.124 0.155 0.202 105.8 9\nHapp, Ian 8 5 0 0 0 0 0 0 8.3 2 25 0 0 0.173 0.359 0.598 0.491 84.9 -3.7\nSwanson, Dansby 8 5 3 2 0 1 0 0 16.7 3 37.5 0.6 1.6 0.828 0.497 0.987 0.651 95.3 19.4\nBellinger, Cody 8 8 4 1 0 0 2 25 18.8 0 0 0.5 0.625 0.488 0.304 0.49 0.337 86.4 25\nGomes, Yan 7 7 2 0 0 0 1 14.3 10 0 0 0.286 0.286 0.252 0.534 0.746 0.551 94 13.2\nHoerner, Nico 5 5 2 2 0 0 1 20 11.1 0 0 0.4 0.8 0.498 0.183 0.283 0.195 94.2 16.8\nWisdom, Patrick 5 5 0 0 0 0 3 60 44.4 0 0 0 0 0 0.007 0.017 0.009 99.8 48.5\nTauchman, Mike 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.348 0.961 0.538 88.6 34.3\nSuzuki, Seiya 5 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.177 0.129 0.152 0.276 97 33\nBusch, Michael 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.014 0.022 0.354 84.4 62\nBote, David 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.635 0.412 0.473 0.392 76.1 38.5'}, {'pitcher_id': '608718', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuter, Brent 66 61 11 1 0 0 14 21.2 26.7 4 6.1 0.18 0.197 0.205 0.18 0.215 0.215 81.4 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 21 20 4 0 0 0 8 38.1 31.7 1 4.8 0.2 0.2 0.2 0.223 0.284 0.244 80.1 6.6\nHoerner, Nico 6 5 2 0 0 0 0 0 33.3 1 16.7 0.4 0.4 0.41 0.095 0.101 0.189 82.1 14\nBote, David 6 6 0 0 0 0 0 0 11.1 0 0 0 0 0 0.199 0.212 0.193 84.2 -34.2\nGomes, Yan 5 5 0 0 0 0 3 60 50 0 0 0 0 0 0.089 0.108 0.083 77 32.5\nMorel, Christopher 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.028 0.03 0.029 82.9 -32\nSwanson, Dansby 4 3 2 0 0 0 0 0 30 1 25 0.667 0.667 0.612 0.365 0.367 0.42 77.3 -14.7\nWisdom, Patrick 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.414 0.396 0.572 0.418 102.8 -0.7\nSuzuki, Seiya 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.221 0.272 0.302 0.258 84.4 -12.7\nBellinger, Cody 3 2 0 0 0 0 0 0 25 1 33.3 0 0 0.232 0.003 0.003 0.234 86.9 56\nAmaya, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.086 0.072 89.8 -19\nTauchman, Mike 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.096 0.1 0.406 71.1 -6\nHendricks, Kyle 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.12 0.14 0.113 91.6 13.5\nTaillon, Jameson 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.438 0.082 0.082 0.076 46.9 -13.5\nCrow-Armstrong, Pete 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.187 0.149 68.2 18.5\nLeiter Jr., Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.056 0.05 84.1 -26'}, {'pitcher_id': '668984', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 114: [{'pitcher_id': '571656', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Buck 18 14 1 0 0 0 2 11.1 24.3 1 5.6 0.071 0.071 0.099 0.084 0.123 0.129 74.7 28.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Josh 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.08 0.091 0.073 89.2 30\nHedges, Austin 4 2 0 0 0 0 0 0 0 0 0 0 0 0 0.01 0.021 0.055 75.6 36.8\nKwan, Steven 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.026 0.046 0.362 82.3 43\nRocchio, Brayan 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.096 0.121 0.093 84.7 18\nFreeman, Tyler 2 1 0 0 0 0 0 0 28.6 0 0 0 0 0 0.003 0.003 0.003 37.9 52\nGiménez, Andrés 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.112 0.301 0.163 92 1.5\nSchneemann, Daniel 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nNaylor, Bo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.209 0.215 0.188 53.6 22\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.266 0.299 0.246 74.8 9'}, {'pitcher_id': '641941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPagán, Emilio 31 28 7 3 0 1 6 19.4 28.6 3 9.7 0.25 0.464 0.341 0.305 0.526 0.388 87.6 21.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 10 7 1 1 0 0 1 10 27.8 3 30 0.143 0.286 0.333 0.168 0.248 0.328 85.2 35.3\nGiménez, Andrés 7 7 1 0 0 1 1 14.3 21.4 0 0 0.143 0.571 0.296 0.317 0.735 0.447 90.3 25.2\nBrennan, Will 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.294 0.399 0.454 0.372 85.8 9\nNaylor, Josh 4 4 3 2 0 0 1 25 28.6 0 0 0.75 1.25 0.852 0.663 1.253 0.821 104.2 15.3\nFreeman, Tyler 2 2 0 0 0 0 0 0 28.6 0 0 0 0 0 0.163 0.197 0.153 57.5 -11.5\nHedges, Austin 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nArias, Gabriel 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.322 0.366 0.303 91.4 20.3'}, {'pitcher_id': '666157', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLodolo, Nick 33 26 5 1 0 0 7 21.2 28.6 5 15.2 0.192 0.231 0.295 0.257 0.315 0.344 86.9 5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 7 5 0 0 0 0 3 42.9 46.7 1 14.3 0 0 0.202 0.164 0.195 0.316 86.4 18\nKwan, Steven 7 6 2 0 0 0 1 14.3 18.2 1 14.3 0.333 0.333 0.352 0.439 0.584 0.479 87.8 13.2\nFry, David 3 1 0 0 0 0 0 0 25 2 66.7 0 0 0.461 0.537 0.583 0.622 90.3 9\nFreeman, Tyler 4 4 0 0 0 0 1 25 16.7 0 0 0 0 0 0.291 0.345 0.275 81.7 6.7\nArias, Gabriel 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.296 0.132 0.14 0.122 88.9 30.5\nNaylor, Josh 3 2 1 0 0 0 1 33.3 50 0 0 0.5 0.5 0.538 0.036 0.037 0.263 92.7 -28\nGiménez, Andrés 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.116 0.09 80.7 -21.7\nRocchio, Brayan 3 2 1 1 0 0 0 0 0 1 33.3 0.5 1 0.654 0.384 0.444 0.469 96.1 1'}, {'pitcher_id': '458677', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Justin 7 7 2 1 0 0 3 42.9 42.9 0 0 0.286 0.429 0.308 0.24 0.292 0.224 86.1 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGiménez, Andrés 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.554 0.647 0.506 98.6 14\nArias, Gabriel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nNaylor, Bo 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRocchio, Brayan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.283 0.314 0.255 56 31\nKwan, Steven 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.291 0.433 0.303 91.2 21'}, {'pitcher_id': '608371', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSims, Lucas 13 11 0 0 0 0 4 30.8 30.8 1 7.7 0 0 0.109 0.054 0.07 0.154 80.3 32.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Josh 2 1 0 0 0 0 0 0 20 1 50 0 0 0.346 0.203 0.203 0.438 56.5 -55\nHedges, Austin 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0 0 0 92.1 49\nFry, David 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.144 0.171 0.13 71.5 42\nGiménez, Andrés 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0.361 0.014 0.031 0.369 84.4 35\nFreeman, Tyler 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.003 0.007 0.005 82.8 65\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.162 0.1 88.9 31\nNaylor, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.021 0.013 85.6 61'}, {'pitcher_id': '607259', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Nick 19 18 8 3 0 1 2 10.5 22 1 5.3 0.444 0.778 0.526 0.299 0.62 0.399 81.4 1.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 3 3 2 0 0 1 0 0 16.7 0 0 0.667 1.667 0.952 0.441 1.339 0.714 94.2 -0.7\nFry, David 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.212 0.635 0.338 84.9 37.7\nRocchio, Brayan 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nGiménez, Andrés 2 2 1 0 0 0 0 0 14.3 0 0 0.5 0.5 0.441 0.532 0.614 0.5 71.8 22.5\nKwan, Steven 1 1 1 1 0 0 0 0 14.3 0 0 1 2 1.27 0.138 0.161 0.124 87.7 -10\nNaylor, Josh 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.296 0.18 0.213 0.17 72.9 -10.3\nArias, Gabriel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.253 0.255 0.232 41.2 -70\nManzardo, Kyle 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBrennan, Will 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFreeman, Tyler 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.33 0.36 0.309 85.5 -20.5\nNaylor, Bo 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.769 2.231 1.249 102.9 23'}, {'pitcher_id': '668933', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAshcraft, Graham 21 20 2 0 0 0 3 14.3 18.6 0 0 0.1 0.1 0.088 0.194 0.215 0.18 74.1 -1.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 4 4 2 0 0 0 0 0 9.1 0 0 0.5 0.5 0.441 0.568 0.615 0.521 84.8 4.5\nGiménez, Andrés 4 4 0 0 0 0 1 25 25 0 0 0 0 0 0.082 0.084 0.075 57.5 28.3\nArias, Gabriel 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.162 0.164 0.148 82.2 -44.3\nNaylor, Bo 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.129 0.167 0.128 93.8 9\nHedges, Austin 4 3 0 0 0 0 1 25 28.6 0 0 0 0 0 0.018 0.022 0.018 57.6 11\nBrennan, Will 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.117 0.147 0.109 71.7 -14.3'}, {'pitcher_id': '594580', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMoll, Sam 29 28 4 0 0 0 8 27.6 23.7 1 3.4 0.143 0.143 0.146 0.165 0.195 0.177 88.8 -3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Josh 5 5 1 0 0 0 1 20 12.5 0 0 0.2 0.2 0.177 0.317 0.333 0.29 84.6 -15.3\nGiménez, Andrés 4 4 0 0 0 0 2 50 30.8 0 0 0 0 0 0.101 0.124 0.094 85 10\nNaylor, Bo 4 3 0 0 0 0 2 50 42.9 1 25 0 0 0.173 0.231 -70\nKwan, Steven 5 5 2 0 0 0 0 0 0 0 0 0.4 0.4 0.354 0.199 0.215 0.182 88.1 -14.2\nHedges, Austin 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.294 0.198 0.352 0.231 95 20.5\nRocchio, Brayan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.075 0.119 0.083 88.2 24\nBrennan, Will 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.15 0.131 92.7 12\nArias, Gabriel 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.029 0.029 0.025 82.9 -23\nFreeman, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.379 0.381 0.344 104.4 -6\nManzardo, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 86.9 44'}, {'pitcher_id': '518585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCruz, Fernando 8 7 2 0 0 1 2 25 31.6 1 12.5 0.286 0.714 0.458 0.311 0.775 0.477 94.8 19.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Tyler 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.05 0.094 0.056 91.8 26\nGiménez, Andrés 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nRocchio, Brayan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.217 0.288 0.197 97.4 -14\nBrennan, Will 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nKwan, Steven 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.857 1.038 0.825 92.4 16\nNaylor, Josh 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.503 1.955 0.996 96.2 34.5'}, {'pitcher_id': '664747', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Alexis 9 6 0 0 0 0 0 0 10 3 33.3 0 0 0.23 0.201 0.243 0.357 92.1 31', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 5 3 0 0 0 0 0 0 10 2 40 0 0 0.275 0.065 0.106 0.317 86.7 42\nManzardo, Kyle 1 1 0 0 0 0 0 0 14.3 0 0 0 0 0 0.009 0.023 0.013 96.8 52\nHedges, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.469 0.525 0.561 89 7\nNaylor, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.534 0.592 0.491 106.8 1'}, {'pitcher_id': '622088', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAntone, Tejay 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.08 0.239 0.13 94.9 29', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGiménez, Andrés 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nNaylor, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.24 0.718 0.391 94.9 29'}, {'pitcher_id': '593423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontas, Frankie 22 21 6 1 0 1 2 9.1 23.7 1 4.5 0.286 0.476 0.343 0.244 0.392 0.292 89.7 10.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGiménez, Andrés 7 6 3 1 0 1 0 0 30.8 1 14.3 0.5 1.167 0.701 0.292 0.715 0.462 90.7 14\nHedges, Austin 6 6 1 0 0 0 2 33.3 38.5 0 0 0.167 0.167 0.147 0.277 0.333 0.263 89.6 -5.8\nNaylor, Josh 6 6 1 0 0 0 0 0 0 0 0 0.167 0.167 0.147 0.205 0.239 0.195 89.1 29.7\nKwan, Steven 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.158 0.166 0.149 89 -13.3'}, {'pitcher_id': '686730', 'player_data': '', 'team_data': 'Carson Spiers has not faced anyone on this team.'}, {'pitcher_id': '668881', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Hunter 14 12 4 0 0 2 3 21.4 28.6 2 14.3 0.333 0.833 0.512 0.221 0.576 0.377 92.3 18.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNaylor, Bo 2 1 1 0 0 1 0 0 25 1 50 1 4 1.35 0.397 1.435 0.711 100.4 33\nHedges, Austin 2 1 0 0 0 0 1 50 25 1 50 0 0 0.348 0.348\nArias, Gabriel 2 2 0 0 0 0 2 100 57.1 0 0 0 0 0 0\nGiménez, Andrés 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.443 0.719 2.195 1.179 104.5 23\nKwan, Steven 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.265 0.203 85 17.5\nNaylor, Josh 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.042 0.107 0.057 87.5 38\nBrennan, Will 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.159 0.171 0.145 88.5 -11'}, {'pitcher_id': '671096', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbbott, Andrew 16 16 6 0 0 1 3 18.8 20 0 0 0.375 0.563 0.401 0.364 0.623 0.417 90.5 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKwan, Steven 5 5 1 0 0 0 1 20 23.1 0 0 0.2 0.2 0.177 0.379 0.443 0.361 86.5 28.8\nFreeman, Tyler 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.294 0.078 0.113 0.076 84.9 12\nRocchio, Brayan 3 3 2 0 0 0 0 0 12.5 0 0 0.667 0.667 0.588 0.456 0.522 0.428 90.6 22.7\nArias, Gabriel 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.002 0.811 2.175 1.222 98 20\nNaylor, Bo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nNaylor, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.353 0.753 0.456 96.4 -2.5'}, {'pitcher_id': '608718', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuter, Brent 11 11 1 0 0 0 1 9.1 11.1 0 0 0.091 0.091 0.081 0.192 0.215 0.18 75.1 15.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHedges, Austin 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.026 0.026 0.024 74.9 42.7\nKwan, Steven 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.507 0.558 0.467 78.4 26.5\nNaylor, Josh 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.34 0.363 0.316 78.3 -31\nBrennan, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.173 0.098 91.4 29\nFreeman, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.004 74 81\nGiménez, Andrés 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.237 0.215 47.1 -77'}, {'pitcher_id': '668984', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}]}, 114: {112: [{'pitcher_id': '605130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarlow, Scott 6 6 0 0 0 0 2 33.3 33.3 0 0 0 0 0 0.083 0.095 0.077 76.6 -13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Dansby 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.025 0.032 0.028 66.1 -18\nHapp, Ian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.076 0.084 0.072 78.1 -20\nWisdom, Patrick 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.041 0.044 0.036 68.4 -13\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.29 0.333 0.261 93.8 -2'}, {'pitcher_id': '680704', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSandlin, Nick 5 5 0 0 0 0 4 80 50 0 0 0 0 0 0.014 0.016 0.014 66.9 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHapp, Ian 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nWisdom, Patrick 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMorel, Christopher 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBote, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.072 0.08 0.071 66.9 -9\nHoerner, Nico 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '669212', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorgan, Eli 7 7 1 0 0 0 2 28.6 21.4 0 0 0.143 0.143 0.126 0.316 0.574 0.374 94.8 13.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWisdom, Patrick 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.38 0.468 0.37 102.3 9\nBellinger, Cody 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.369 0.506 0.367 92.6 13.5\nHendricks, Kyle 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSwanson, Dansby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.237 0.215 86 -2\nSuzuki, Seiya 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.48 1.831 0.928 100.5 33'}, {'pitcher_id': '670766', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676440', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBibee, Tanner 16 14 2 1 0 0 7 43.8 28.9 2 12.5 0.143 0.214 0.22 0.086 0.099 0.156 84 -2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuzuki, Seiya 3 2 0 0 0 0 2 66.7 44.4 1 33.3 0 0 0.232 0.232\nHapp, Ian 3 2 1 0 0 0 0 0 11.1 1 33.3 0.5 0.5 0.526 0.177 0.179 0.34 78.1 -39\nMorel, Christopher 3 3 0 0 0 0 3 100 66.7 0 0 0 0 0 0\nSwanson, Dansby 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nHoerner, Nico 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.228 0.252 0.205 81.1 11.7\nBellinger, Cody 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.083 0.132 0.082 94.4 13.5'}, {'pitcher_id': '657265', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrzelecki, Peter 10 7 1 1 0 0 2 20 50 1 10 0.143 0.286 0.267 0.184 0.417 0.355 95.5 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomes, Yan 2 1 0 0 0 0 0 0 25 0 0 0 0 0 0.107 0.128 0.232 94.9 2\nWisdom, Patrick 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.42 0.306 0.699 0.424 95.7 24.7\nHapp, Ian 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nSuzuki, Seiya 2 0 0 0 0 0 0 0 100 1 50 0.704 0.704\nHoerner, Nico 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.263 0.694 0.403 96 27'}, {'pitcher_id': '682120', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerrin, Tim 3 3 2 1 0 0 1 33.3 42.9 0 0 0.667 1 0.709 0.133 0.17 0.123 80 -3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomes, Yan 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSwanson, Dansby 1 1 1 1 0 0 0 0 50 0 0 1 2 1.244 0.264 0.311 0.242 97.1 -10\nBellinger, Cody 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.136 0.199 0.128 62.8 4'}, {'pitcher_id': '641149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParsons, Wes 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.023 0.023 0.021 81.5 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBote, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.069 0.062 82.9 -21\nBellinger, Cody 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 80 69\nSwanson, Dansby 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '671922', 'player_data': '', 'team_data': 'Cade Smith has not faced anyone on this team.'}, {'pitcher_id': '656529', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHentges, Sam 10 9 2 1 0 0 4 40 43.5 1 10 0.222 0.333 0.282 0.115 0.18 0.184 93.8 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBote, David 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nSwanson, Dansby 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.051 0.053 0.049 97.2 -30\nBusch, Michael 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nTauchman, Mike 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.046 0.106 0.065 85.2 31\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.142 0.142 0.132 90 -10\nAmaya, Miguel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBellinger, Cody 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.373 0.634 0.428 98.4 27'}, {'pitcher_id': '671106', 'player_data': '', 'team_data': 'Logan Allen has not faced anyone on this team.'}, {'pitcher_id': '683769', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGaddis, Hunter 3 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.036 0.057 0.028 86.6 48', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBusch, Michael 3 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.036 0.057 0.028 86.6 48'}, {'pitcher_id': '675540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurry, Xzavion 10 8 2 0 0 1 0 0 16.7 0 0 0.25 0.625 0.361 0.3 0.665 0.421 86 38.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorel, Christopher 2 1 1 0 0 1 0 0 40 0 0 1 4 1.002 0.877 3.415 1.002 98.7 26.5\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.015 0.011 92 62\nHoerner, Nico 2 1 1 0 0 0 0 0 0 0 0 1 1 0.804 0.8 0.885 0.741 71.4 29\nSwanson, Dansby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.286 0.315 0.257 60.7 43\nBellinger, Cody 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.004 0.003 75.3 79\nHapp, Ian 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.18 0.144 91.6 25\nTauchman, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.325 0.168 93.8 33'}, {'pitcher_id': '661403', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClase, Emmanuel 7 6 2 0 0 0 0 0 10 0 0 0.333 0.333 0.252 0.141 0.201 0.134 86.2 6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorel, Christopher 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.246 0.25 0.226 102.4 -15\nTauchman, Mike 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.13 0.132 0.122 91.1 -19\nHapp, Ian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.011 0.01 77.4 60\nSuzuki, Seiya 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.218 0.242 0.198 95.2 -10\nBellinger, Cody 1 0 0 0 0 0 0 0 0 0 0 0 0.043 87.4 31\nHoerner, Nico 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.084 0.077 56.3 -32\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.16 0.487 0.265 93.6 29'}, {'pitcher_id': '471911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarrasco, Carlos 77 70 16 4 0 2 19 24.7 29.5 7 9.1 0.229 0.371 0.295 0.234 0.375 0.299 89.3 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Dansby 16 15 2 1 0 1 4 25 30.8 1 6.3 0.133 0.4 0.247 0.263 0.472 0.336 92.5 22.5\nHapp, Ian 13 13 3 1 0 0 4 30.8 36.4 0 0 0.231 0.308 0.231 0.162 0.321 0.202 85.9 0.6\nGomes, Yan 7 7 3 1 0 0 1 14.3 25 0 0 0.429 0.571 0.43 0.319 0.452 0.327 89.4 9.5\nTauchman, Mike 6 5 1 0 0 0 2 33.3 33.3 1 16.7 0.2 0.2 0.263 0.264 0.301 0.321 91.9 -5.7\nHoerner, Nico 8 8 3 0 0 0 1 12.5 16.7 0 0 0.375 0.375 0.331 0.397 0.456 0.377 80 0\nMorel, Christopher 7 5 0 0 0 0 2 28.6 35.7 2 28.6 0 0 0.198 0.141 0.155 0.291 94.4 18\nSuzuki, Seiya 6 5 2 0 0 0 2 33.3 30 1 16.7 0.4 0.4 0.41 0.118 0.143 0.207 93.7 8.3\nWisdom, Patrick 5 4 0 0 0 0 0 0 0 1 20 0 0 0.138 0.122 0.207 0.248 89.2 17.8\nBellinger, Cody 5 5 2 1 0 1 1 20 40 0 0 0.4 1.2 0.649 0.333 0.838 0.478 101.1 17\nBote, David 2 1 0 0 0 0 1 50 1 50 0 0 0.349 0.349\nSmyly, Drew 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.04 0.046 0.036 62.4 -5'}, {'pitcher_id': '669456', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBieber, Shane 8 7 0 0 0 0 4 50 36.4 1 12.5 0 0 0.086 0.081 0.085 0.151 77.8 0.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBote, David 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.231 0.059 0.059 0.269 41 -74\nBellinger, Cody 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.003 0.003 0.003 90.5 75\nSwanson, Dansby 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.22 0.234 0.198 101.9 1'}, {'pitcher_id': '658648', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAvila, Pedro 15 11 3 1 1 0 5 33.3 39.1 3 20 0.273 0.545 0.39 0.192 0.278 0.302 90.5 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBellinger, Cody 3 2 0 0 0 0 2 66.7 50 0 0 0 0 0 0 0 0.074 94.4 25\nHapp, Ian 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.346 0.346\nSuzuki, Seiya 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346\nTauchman, Mike 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.311 0.337 0.28 92 -3\nHoerner, Nico 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nBusch, Michael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nAmaya, Miguel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.47 0.526 0.439 82.2 3\nMorel, Christopher 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.248 0.282 0.226 94.3 -6\nSwanson, Dansby 1 1 1 0 1 0 0 0 0 0 0 1 3 1.612 0.297 1.054 0.561 96.6 30'}, {'pitcher_id': '663474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcKenzie, Triston 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTauchman, Mike 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346'}, {'pitcher_id': '594902', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLively, Ben 25 23 9 0 0 4 5 20 28.9 2 8 0.391 0.913 0.553 0.304 0.656 0.422 92.4 23.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Dansby 7 7 3 0 0 2 2 28.6 30 0 0 0.429 1.286 0.698 0.355 0.722 0.445 97.8 32.2\nHapp, Ian 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.232 0.04 0.117 0.273 92.2 33\nTauchman, Mike 3 2 2 0 0 1 0 0 25 1 33.3 1 2.5 1.194 0.921 2.259 1.113 94.7 22\nHendricks, Kyle 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nBellinger, Cody 3 3 3 0 0 1 0 0 20 0 0 1 2 1.256 0.48 1.325 0.742 96.3 7\nHoerner, Nico 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.249 0.283 0.231 98.2 4.3\nMorel, Christopher 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.108 0.123 0.1 87.6 33.7\nGomes, Yan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.073 0.078 0.069 45.5 47'}], 113: [{'pitcher_id': '605130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarlow, Scott 31 26 3 1 0 0 8 25.8 32.1 3 9.7 0.115 0.154 0.194 0.137 0.185 0.215 80.1 17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 17 14 1 0 0 0 6 35.3 35.5 2 11.8 0.071 0.071 0.176 0.145 0.193 0.246 86.6 3\nEspinal, Santiago 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.036 0.028 74.5 48.5\nBenson, Will 2 1 0 0 0 0 1 50 75 1 50 0 0 0.348 0.348\nMaile, Luke 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.05 0.09 0.057 79.5 44.5\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 86.9 45\nStephenson, Tyler 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.42 0.48 0.399 77 40.5\nIndia, Jonathan 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.317 0.629 0.395 96.4 20\nSteer, Spencer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.179 0.199 0.164 88.9 -7\nFriedl, TJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFairchild, Stuart 1 0 0 0 0 0 0 0 0 0 0 18.4 15\nFraley, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.058 0.051 77.4 -36'}, {'pitcher_id': '680704', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSandlin, Nick 18 16 4 0 0 2 4 22.2 30 1 5.6 0.25 0.625 0.385 0.15 0.338 0.233 80 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStephenson, Tyler 3 2 1 0 0 1 1 33.3 25 1 33.3 0.5 2 0.921 0.222 0.82 0.518 99.9 32\nIndia, Jonathan 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.092 0.106 0.087 83.6 -8\nFriedl, TJ 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.387 0.805 0.502 75.6 -0.5\nSteer, Spencer 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.444 0.069 0.084 0.064 94 -21\nEspinal, Santiago 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.099 0.238 0.141 60.1 25.3\nCandelario, Jeimer 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.113 0.145 0.103 91.7 26\nFraley, Jake 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.173 0.45 0.262 82.2 19\nHurtubise, Jacob 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '669212', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorgan, Eli 22 19 4 2 0 1 4 18.2 29.4 2 9.1 0.211 0.474 0.343 0.202 0.494 0.346 85.7 21.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 7 6 2 1 0 0 1 14.3 11.1 1 14.3 0.333 0.5 0.401 0.369 1.003 0.579 95.9 15\nEspinal, Santiago 5 4 1 1 0 0 1 20 28.6 1 20 0.25 0.5 0.391 0.217 0.236 0.3 74.1 -3.3\nFriedl, TJ 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.013 0.009 82.2 72.5\nFraley, Jake 1 1 1 0 0 1 0 0 20 0 0 1 4 2.072 0.571 2.197 1.151 102.6 34\nStephenson, Tyler 2 1 0 0 0 0 1 50 100 0 0 0 0 0.36 0.36\nSteer, Spencer 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBenson, Will 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.058 0.062 0.058 61.8 -30\nFairchild, Stuart 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.026 0.017 93.2 44\nMaile, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.004 90.5 72\nIndia, Jonathan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.101 0.102 0.097 70.7 -1'}, {'pitcher_id': '670766', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676440', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBibee, Tanner 24 22 4 0 0 2 12 50 31.5 1 4.2 0.182 0.455 0.307 0.149 0.307 0.234 83.1 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 3 3 2 0 0 2 1 33.3 7.7 0 0 0.667 2.667 1.39 0.35 1.316 0.671 100.5 31\nDe La Cruz, Elly 3 3 0 0 0 0 3 100 60 0 0 0 0 0 0\nFriedl, TJ 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.296 0.409 0.426 0.371 57.1 0\nEspinal, Santiago 3 2 0 0 0 0 1 33.3 33.3 0 0 0 0 0.242 0.087 0.296 0.345 96.3 34\nStephenson, Tyler 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.296 0.322 0.484 89.6 0\nHurtubise, Jacob 2 2 0 0 0 0 2 100 57.1 0 0 0 0 0 0\nBenson, Will 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.444 0.086 0.098 0.08 92 -13\nFraley, Jake 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.069 0.081 0.064 63.4 3\nIndia, Jonathan 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nSteer, Spencer 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.112 0.132 0.105 87.1 23.5'}, {'pitcher_id': '657265', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrzelecki, Peter 11 10 1 0 0 0 4 36.4 33.3 0 0 0.1 0.1 0.146 0.102 0.121 0.157 76.3 26.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIndia, Jonathan 3 2 0 0 0 0 0 0 0 0 0 0 0 0.242 0.214 0.271 0.384 65.7 16.5\nFraley, Jake 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.007 0.012 0.009 87.3 51\nSteer, Spencer 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.442 0.256 0.287 0.244 67.3 40\nStephenson, Tyler 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFriedl, TJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.063 0.061 77.5 -14\nEspinal, Santiago 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFairchild, Stuart 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.004 0.003 94.6 51'}, {'pitcher_id': '682120', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerrin, Tim 6 5 1 0 0 0 2 33.3 46.2 1 16.7 0.2 0.2 0.262 0.335 0.44 0.398 86.2 -18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairchild, Stuart 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nCandelario, Jeimer 1 1 1 0 0 0 0 0 20 0 0 1 1 0.883 0.926 1.324 0.971 107 12\nEspinal, Santiago 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nStephenson, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.523 0.652 0.522 101.7 2\nFriedl, TJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.224 0.225 0.202 50 -69'}, {'pitcher_id': '641149', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '671922', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Cade 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.029 0.029 0.025 96.4 -35', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCandelario, Jeimer 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.057 0.057 0.051 96.4 -35'}, {'pitcher_id': '656529', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHentges, Sam 19 15 4 1 0 1 3 15.8 34.6 4 21.1 0.267 0.533 0.409 0.278 0.471 0.395 88.1 6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 6 5 2 1 0 1 0 0 22.2 1 16.7 0.4 1.2 0.656 0.426 0.968 0.595 93.9 0.8\nEspinal, Santiago 3 2 1 0 0 0 0 0 25 1 33.3 0.5 0.5 0.523 0.343 0.362 0.437 86.7 2.5\nIndia, Jonathan 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.23 0.261 0.375 73 28\nDe La Cruz, Elly 2 1 0 0 0 0 1 50 0 1 50 0 0 0.348 0.348\nStephenson, Tyler 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.24 0.268 0.217 97.3 -8.5\nFriedl, TJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFraley, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSteer, Spencer 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.409 0.443 0.368 73.1 32'}, {'pitcher_id': '671106', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAllen, Logan 12 10 2 0 0 0 3 25 31.6 1 8.3 0.2 0.2 0.205 0.2 0.294 0.306 95.2 6.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 3 2 0 0 0 0 0 0 50 1 33.3 0 0 0.232 0.141 0.177 0.321 83.3 19.5\nDe La Cruz, Elly 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.131 0.16 0.121 98.2 -20.5\nFairchild, Stuart 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.291 0.326 0.268 101 5\nStephenson, Tyler 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.197 0.226 0.179 100.6 -2\nEspinal, Santiago 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.343 1.007 0.725 98.6 26'}, {'pitcher_id': '683769', 'player_data': '', 'team_data': 'Hunter Gaddis has not faced anyone on this team.'}, {'pitcher_id': '675540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurry, Xzavion 8 7 2 0 0 1 1 12.5 10 1 12.5 0.286 0.714 0.448 0.344 0.934 0.546 90.3 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Elly 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.525 2.029 1.026 97.1 1.5\nFriedl, TJ 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBenson, Will 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.269 0.269 0.469 54.1 30\nIndia, Jonathan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.201 0.108 91.1 31\nSteer, Spencer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.326 1.213 0.619 98.8 34\nMaile, Luke 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.689 0.8 0.65 103.8 7'}, {'pitcher_id': '661403', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClase, Emmanuel 25 24 5 0 0 0 9 36 30 1 4 0.208 0.208 0.205 0.212 0.245 0.221 76.8 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 9 8 1 0 0 0 3 33.3 35.3 1 11.1 0.125 0.125 0.175 0.17 0.203 0.22 78.1 -6.2\nEspinal, Santiago 3 3 0 0 0 0 2 66.7 44.4 0 0 0 0 0 0.116 0.147 0.12 70.7 42\nStephenson, Tyler 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nMartini, Nick 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.337 0.373 0.308 95.5 -4.5\nFriedl, TJ 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.223 0.295 0.227 82.2 20.5\nFraley, Jake 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.06 0.06 0.054 47.1 36\nIndia, Jonathan 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.389 0.446 0.371 74 27\nDe La Cruz, Elly 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.217 0.224 0.202 78 -50\nSteer, Spencer 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.986 1.059 0.908 76.3 19\nFairchild, Stuart 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.166 0.188 0.153 59.7 3'}, {'pitcher_id': '471911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarrasco, Carlos 30 27 10 1 0 2 7 23.3 30.9 3 10 0.37 0.63 0.452 0.332 0.573 0.415 86.8 16.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 19 17 6 1 0 0 3 15.8 25.6 2 10.5 0.353 0.412 0.37 0.326 0.36 0.342 87.3 19.1\nIndia, Jonathan 6 5 2 0 0 1 3 50 55.6 1 16.7 0.4 1 0.597 0.347 0.93 0.555 88.4 24.5\nFraley, Jake 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.691 0.51 1.515 0.828 82.2 11\nMaile, Luke 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.44 0.074 0.075 0.067 91 -16'}, {'pitcher_id': '669456', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBieber, Shane 69 67 17 3 0 2 25 36.2 31 2 2.9 0.254 0.388 0.286 0.231 0.361 0.268 88.8 9.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 33 32 7 0 0 1 13 39.4 37.3 1 3 0.219 0.313 0.242 0.244 0.376 0.28 84.3 6.5\nIndia, Jonathan 7 7 2 2 0 0 3 42.9 30.8 0 0 0.286 0.571 0.358 0.219 0.257 0.209 93.6 7.3\nFraley, Jake 7 7 2 1 0 1 1 14.3 21.4 0 0 0.286 0.857 0.467 0.358 0.913 0.528 93.3 29.2\nMaile, Luke 4 4 0 0 0 0 1 25 37.5 0 0 0 0 0 0.115 0.129 0.108 102.1 36\nEspinal, Santiago 5 4 2 0 0 0 1 20 16.7 1 20 0.5 0.5 0.491 0.14 0.149 0.242 87.2 -5\nFriedl, TJ 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.294 0.305 0.333 0.283 98.4 1\nSteer, Spencer 3 3 1 0 0 0 2 66.7 60 0 0 0.333 0.333 0.294 0.291 0.319 0.274 68.7 30\nMartini, Nick 1 1 0 0 0 0 1 100 20 0 0 0 0 0 0\nBenson, Will 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.051 0.053 0.049 88.7 -20\nStephenson, Tyler 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.036 0.036 0.034 65.3 -10\nDe La Cruz, Elly 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.344 0.374 0.317 104.6 -9.5'}, {'pitcher_id': '658648', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAvila, Pedro 8 7 1 0 0 0 2 25 40 0 0 0.143 0.143 0.201 0.212 0.242 0.263 73.9 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSteer, Spencer 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.974 1.046 0.903 69.9 22\nIndia, Jonathan 1 0 0 0 0 0 0 0 100 0 0 0.726 0.726\nFraley, Jake 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.044 0.068 0.045 69.6 -19\nMaile, Luke 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFriedl, TJ 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.191 0.152 87.2 20.5\nDe La Cruz, Elly 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBenson, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.199 0.124 55.7 2'}, {'pitcher_id': '663474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcKenzie, Triston 52 47 10 4 0 1 13 25 25.5 5 9.6 0.213 0.362 0.288 0.189 0.38 0.283 86.6 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 18 16 3 2 0 1 5 27.8 32.3 2 11.1 0.188 0.5 0.332 0.151 0.332 0.256 81.5 12.2\nIndia, Jonathan 6 6 0 0 0 0 3 50 29.4 0 0 0 0 0 0.14 0.389 0.213 89.4 34\nEspinal, Santiago 7 7 3 1 0 0 1 14.3 15.4 0 0 0.429 0.571 0.432 0.311 0.414 0.314 92.2 22.8\nFraley, Jake 4 3 1 1 0 0 0 0 11.1 1 25 0.333 0.667 0.49 0.24 0.758 0.48 86.6 22.7\nStephenson, Tyler 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.102 0.123 0.098 80.6 38\nSteer, Spencer 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.296 0.37 0.977 0.557 103.9 14\nDe La Cruz, Elly 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.204 0.221 0.353 104.4 -8\nFriedl, TJ 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.234 0.275 0.217 83.2 14.7\nHurtubise, Jacob 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.031 0.086 0.371 92.2 38\nBenson, Will 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.003 0.003 0.003 64.9 58'}, {'pitcher_id': '594902', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLively, Ben 3 3 3 1 0 0 0 0 0 0 0 1 1.333 1.003 0.26 0.496 0.312 73.2 -6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCandelario, Jeimer 3 3 3 1 0 0 0 0 0 0 0 1 1.333 1.003 0.26 0.496 0.312 73.2 -6.3'}]}, 115: {116: [{'pitcher_id': '663372', 'player_data': '', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcKinstry, Zach 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.116 0.176 0.125 92 4.5\nVierling, Matt 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.003 0.003 0.003 90.6 61\nUrshela, Gio 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.442 0.627 0.698 0.578 103.4 6.5\nCanha, Mark 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.311 0.321 0.28 60.1 31\nKelly, Carson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.646 0.913 0.677 97.8 16'}, {'pitcher_id': '621389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlach, Ty 20 20 6 1 0 0 2 10 13.9 0 0 0.3 0.35 0.281 0.321 0.386 0.305 89.3 6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 6 6 3 0 0 0 0 0 25 0 0 0.5 0.5 0.436 0.445 0.534 0.416 89.1 2.3\nMaeda, Kenta 4 4 1 0 0 0 1 25 11.1 0 0 0.25 0.25 0.219 0.198 0.24 0.191 89 -4\nCanha, Mark 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.394 0.44 0.368 92.5 11.7\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.077 0.069 62.5 51\nVierling, Matt 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.123 0.144 0.118 86.8 -0.7\nBaddoo, Akil 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.64 0.705 0.586 103 7\nKelly, Carson 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nIbáñez, Andy 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.686 1.015 0.729 102.4 15'}, {'pitcher_id': '571539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarasiti, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.335 0.38 0.302 103.9 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.335 0.38 0.302 103.9 -7'}, {'pitcher_id': '607536', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeland, Kyle 38 35 11 4 0 2 5 13.2 15.3 1 2.6 0.314 0.6 0.398 0.294 0.464 0.343 91.6 19.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 16 15 5 1 0 2 3 18.8 29.2 1 6.3 0.333 0.8 0.476 0.3 0.57 0.383 92.5 12.3\nCanha, Mark 11 10 4 3 0 0 1 9.1 5.9 0 0 0.4 0.7 0.486 0.315 0.443 0.363 91.5 24.6\nVierling, Matt 7 7 2 0 0 0 0 0 0 0 0 0.286 0.286 0.253 0.358 0.44 0.346 89.7 28.9\nMaeda, Kenta 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0 0\nUrshela, Gio 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.085 0.077 93.6 7'}, {'pitcher_id': '683627', 'player_data': '', 'team_data': 'Anthony Molina has not faced anyone on this team.'}, {'pitcher_id': '663567', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLambert, Peter 11 9 3 0 1 0 1 9.1 5.9 1 9.1 0.333 0.556 0.401 0.255 0.293 0.278 87.3 14.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcKinstry, Zach 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.045 0.059 0.044 85.3 20\nFlaherty, Jack 2 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.88 0.938 0.792 77.9 21\nIbáñez, Andy 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.033 0.063 0.032 73.9 -4\nKelly, Carson 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nCanha, Mark 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.278 0.318 0.253 90 21\nVierling, Matt 1 1 1 0 1 0 0 0 0 0 0 1 3 1.569 0.427 0.503 0.389 106.3 -7'}, {'pitcher_id': '656222', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBeeks, Jalen 11 10 4 2 0 1 4 36.4 26.3 1 9.1 0.4 0.9 0.55 0.355 0.626 0.437 95.8 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Riley 3 2 1 1 0 0 1 33.3 0 1 33.3 0.5 1 0.65 0.392 0.446 0.472 101.1 10\nUrshela, Gio 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.441 0.29 0.328 0.276 70.8 33\nKelly, Carson 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.608 0.646 1.276 0.782 105.4 17.5\nBaddoo, Akil 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nIbáñez, Andy 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRogers, Jake 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.506 1.76 0.915 98.5 29\nVierling, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.392 0.396 0.356 93.6 -1'}, {'pitcher_id': '615698', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuantrill, Cal 70 65 18 3 0 2 11 15.7 19.5 3 4.3 0.277 0.415 0.32 0.264 0.43 0.322 85.2 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaddoo, Akil 16 15 4 1 0 0 3 18.8 31 1 6.3 0.267 0.333 0.286 0.303 0.55 0.383 89.8 13.7\nGreene, Riley 12 11 2 1 0 0 5 41.7 28.6 1 8.3 0.182 0.273 0.236 0.146 0.215 0.201 85.8 4.3\nUrshela, Gio 13 13 4 0 0 1 0 0 18.2 0 0 0.308 0.538 0.363 0.234 0.447 0.292 83.7 5.5\nCanha, Mark 9 7 1 0 0 0 1 11.1 11.1 1 11.1 0.143 0.143 0.255 0.147 0.154 0.266 77.3 -14\nVierling, Matt 5 5 2 1 0 0 1 20 0 0 0 0.4 0.6 0.425 0.35 0.451 0.348 100.4 -1.8\nMcKinstry, Zach 3 3 1 0 0 1 0 0 12.5 0 0 0.333 1.333 0.668 0.409 1.162 0.637 101.9 29.3\nKelly, Carson 4 4 2 0 0 0 1 25 11.1 0 0 0.5 0.5 0.442 0.333 0.427 0.333 91.1 20.7\nRogers, Jake 5 4 1 0 0 0 0 0 14.3 0 0 0.25 0.25 0.22 0.491 0.557 0.462 67 0.6\nIbáñez, Andy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.23 0.251 0.212 75.7 48'}, {'pitcher_id': '641712', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Dakota 20 18 7 0 0 0 1 5 14.8 2 10 0.389 0.389 0.377 0.383 0.534 0.423 94 11.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 8 8 4 0 0 0 0 0 7.7 0 0 0.5 0.5 0.438 0.491 0.629 0.481 92.9 7.2\nKelly, Carson 7 5 2 0 0 0 1 14.3 33.3 2 28.6 0.4 0.4 0.45 0.26 0.328 0.377 91.4 2.5\nMcKinstry, Zach 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.387 0.611 0.43 95.9 22.3\nVierling, Matt 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.257 0.555 0.339 100.2 32'}, {'pitcher_id': '641755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKinley, Tyler 10 9 4 1 0 0 4 40 33.3 1 10 0.444 0.556 0.457 0.403 0.484 0.415 94.2 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 4 4 1 0 0 0 3 75 50 0 0 0.25 0.25 0.22 0.234 0.267 0.22 85.4 17\nMcKinstry, Zach 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.06 0.707 0.864 0.677 100.1 8.5\nKelly, Carson 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.173 0.208 0.155 92.4 2\nVierling, Matt 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nIbáñez, Andy 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.884 0.931 1.141 0.918 92.9 13'}, {'pitcher_id': '683232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMears, Nick 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '596295', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomber, Austin 42 37 9 1 0 2 6 14.3 18.3 4 9.5 0.243 0.432 0.319 0.352 0.612 0.427 85.9 16.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 11 9 3 0 0 1 0 0 23.5 1 9.1 0.333 0.667 0.412 0.624 1.137 0.672 92.9 16.3\nMcKinstry, Zach 8 7 1 1 0 0 2 25 30.8 1 12.5 0.143 0.286 0.242 0.182 0.387 0.297 83.7 17.8\nCanha, Mark 6 4 1 0 0 0 0 0 12.5 2 33.3 0.25 0.25 0.379 0.302 0.378 0.431 83.8 -3.5\nVierling, Matt 6 6 1 0 0 0 2 33.3 11.1 0 0 0.167 0.167 0.147 0.247 0.283 0.23 76.7 13.5\nIbáñez, Andy 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.33 0.398 0.319 87.4 28.2\nRogers, Jake 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.294 0.088 0.088 0.08 99.4 -9\nFlaherty, Jack 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.488 1.831 0.936 64.7 51\nUrshela, Gio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.537 0.586 0.483 94.1 8'}, {'pitcher_id': '664875', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLawrence, Justin 7 5 3 1 0 0 2 28.6 27.3 2 28.6 0.6 0.8 0.627 0.305 0.445 0.429 88 24.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 4 4 3 1 0 0 1 25 14.3 0 0 0.75 1 0.753 0.382 0.556 0.406 88 24.3\nMcKinstry, Zach 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nBaddoo, Akil 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nVierling, Matt 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '595928', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurtiss, John 8 8 3 1 0 0 2 25 22.2 0 0 0.375 0.5 0.375 0.184 0.211 0.173 89.2 39.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 5 5 1 1 0 0 2 40 26.7 0 0 0.2 0.4 0.248 0.007 0.015 0.008 91.8 53.3\nKelly, Carson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.911 0.998 0.85 73.6 24\nVierling, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.026 0.019 84.9 52\nIbáñez, Andy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.511 0.59 0.48 101.3 2'}, {'pitcher_id': '663562', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656234', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBird, Jake 8 7 1 0 0 1 0 0 7.7 1 12.5 0.143 0.571 0.337 0.169 0.325 0.265 91 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.122 0.132 0.112 88.6 -11\nUrshela, Gio 1 0 0 0 0 0 0 0 33.3 1 100 0.689 0.689\nCanha, Mark 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.114 0.094 86 11.7\nIbáñez, Andy 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.5 1.542 0.81 99.8 26\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.127 0.129 0.119 101.8 -23'}, {'pitcher_id': '680767', 'player_data': '', 'team_data': 'Victor Vodnik has not faced anyone on this team.'}], 117: [{'pitcher_id': '663372', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeltner, Ryan 4 3 1 0 0 0 1 25 25 1 25 0.333 0.333 0.393 0.185 0.285 0.322 76.5 -10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.4 0.697 0.572 101.1 18\nHummel, Cooper 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.078 0.08 0.072 51.9 -38'}, {'pitcher_id': '621389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlach, Ty 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.295 0.223 0.256 0.218 70.4 1.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.606 0.703 0.596 72.4 32\nHummel, Cooper 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.063 0.066 0.059 68.3 -29'}, {'pitcher_id': '571539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarasiti, Matt 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.435 0.508 0.537 0.458 71.3 -7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.87 0.947 0.997 0.852 72 23\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.078 0.064 70.5 -38'}, {'pitcher_id': '607536', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeland, Kyle 44 41 8 3 1 1 7 15.9 23.4 3 6.8 0.195 0.39 0.272 0.224 0.344 0.273 83.8 5.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDubón, Mauricio 16 14 5 2 1 1 0 0 4.8 2 12.5 0.357 0.857 0.517 0.277 0.513 0.376 80.9 25.9\nBregman, Alex 6 6 1 1 0 0 1 16.7 11.1 0 0 0.167 0.333 0.206 0.243 0.351 0.251 92.9 21\nHummel, Cooper 6 6 0 0 0 0 2 33.3 37.5 0 0 0 0 0 0.057 0.059 0.056 85.8 -34.5\nCaratini, Victor 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.242 0.303 0.23 97.5 -5.7\nMcCormick, Chas 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.284 0.539 0.457 105.9 17\nMeyers, Jake 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.588 0.366 0.422 0.344 67.8 -8\nAltuve, Jose 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.145 0.145 0.133 55.9 -51\nDiaz, Yainer 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.152 0.159 0.138 98.2 -10.5'}, {'pitcher_id': '683627', 'player_data': '', 'team_data': 'Anthony Molina has not faced anyone on this team.'}, {'pitcher_id': '663567', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLambert, Peter 13 9 4 0 0 2 0 0 6.7 4 30.8 0.444 1.111 0.645 0.302 0.491 0.45 88.4 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 5 3 2 0 0 1 0 0 20 2 40 0.667 1.667 0.838 0.264 0.535 0.472 93.3 25.7\nAltuve, Jose 6 5 1 0 0 1 0 0 0 1 16.7 0.2 0.8 0.438 0.19 0.359 0.321 88.8 4\nAlvarez, Yordan 2 1 1 0 0 0 0 0 0 1 50 1 1 0.78 0.973 1.024 0.783 71.8 21'}, {'pitcher_id': '656222', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBeeks, Jalen 11 10 4 1 0 0 1 9.1 22.7 0 0 0.4 0.5 0.418 0.27 0.327 0.3 87.6 -17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.349 0.406 0.329 91.7 -6.5\nAltuve, Jose 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.608 0.501 0.678 0.498 103.7 -14\nMcCormick, Chas 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCaratini, Victor 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.411 0.479 0.554 94.1 3\nMeyers, Jake 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.213 0.217 0.195 69.6 -80\nAlvarez, Yordan 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.067 0.078 0.06 58.5 -9\nDiaz, Yainer 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.056 0.056 0.054 74.1 -18\nDubón, Mauricio 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.254 0.274 0.229 101.6 -11'}, {'pitcher_id': '615698', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuantrill, Cal 57 51 14 2 0 2 10 17.5 12.4 5 8.8 0.275 0.431 0.344 0.22 0.359 0.295 88.8 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 14 13 3 0 0 0 2 14.3 0 1 7.1 0.231 0.231 0.238 0.253 0.295 0.276 84.4 12.8\nAlvarez, Yordan 9 6 1 0 0 1 2 22.2 28.6 3 33.3 0.167 0.667 0.461 0.265 0.863 0.537 97.3 27.3\nBregman, Alex 8 8 0 0 0 0 3 37.5 28.6 0 0 0 0 0 0.087 0.096 0.079 84.7 -10.6\nMcCormick, Chas 7 6 2 0 0 1 1 14.3 15.4 0 0 0.333 0.833 0.516 0.206 0.409 0.324 87.1 10\nCaratini, Victor 4 4 2 0 0 0 1 25 9.1 0 0 0.5 0.5 0.435 0.219 0.242 0.217 92.6 -7.7\nCabbage, Trey 4 4 2 1 0 0 1 25 18.2 0 0 0.5 0.75 0.538 0.165 0.291 0.191 86.2 -5.7\nDiaz, Yainer 3 3 2 0 0 0 0 0 10 0 0 0.667 0.667 0.593 0.476 0.568 0.445 101.2 0.3\nPeña, Jeremy 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.215 0.245 0.369 97.2 29.5\nDubón, Mauricio 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.101 0.081 78.6 29.3\nMeyers, Jake 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.073 0.392 0.713 0.459 98.3 10.5'}, {'pitcher_id': '641712', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Dakota 17 15 5 0 0 2 1 5.9 12.8 2 11.8 0.333 0.733 0.465 0.302 0.654 0.429 89.4 -1.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 7 6 1 0 0 0 1 14.3 11.8 1 14.3 0.167 0.167 0.225 0.138 0.141 0.21 88 -23.4\nAltuve, Jose 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.647 0.501 1.395 0.762 95.9 6.7\nAlvarez, Yordan 3 2 2 0 0 1 0 0 16.7 1 33.3 1 2.5 1.166 0.744 1.985 0.957 94.1 22.5\nDubón, Mauricio 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.177 0.204 0.168 84 8.3'}, {'pitcher_id': '641755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKinley, Tyler 10 8 4 1 0 1 1 10 25 1 10 0.5 1 0.582 0.45 1.096 0.608 91.5 29.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDubón, Mauricio 3 2 1 1 0 0 1 33.3 40 0 0 0.5 1 0.423 0.354 0.469 0.333 95.9 18\nDiaz, Yainer 1 0 0 0 0 0 0 0 33.3 1 100 0.692 0.692\nAltuve, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.749 0.877 0.706 103.1 9\nBregman, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.7 2.668 1.337 100.6 30\nPeña, Jeremy 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.003 0.003 0.003 78.8 65\nMcCormick, Chas 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.343 0.377 0.31 64.5 43\nAlvarez, Yordan 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.989 3.763 1.93 113 22\nCaratini, Victor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.145 0.105 80.3 29'}, {'pitcher_id': '683232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMears, Nick 5 5 2 1 0 0 0 0 12.5 0 0 0.4 0.6 0.43 0.353 0.542 0.382 98.2 3.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.459 0.868 0.551 101.3 26\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.068 0.061 82.9 -21\nDiaz, Yainer 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.583 0.698 0.566 103.1 4\nDubón, Mauricio 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.2 0.21 0.18 102.4 -16'}, {'pitcher_id': '596295', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomber, Austin 58 54 15 2 1 4 7 12.1 16.7 2 3.4 0.278 0.574 0.359 0.216 0.416 0.276 85.6 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 9 8 2 0 0 0 0 0 5.3 1 11.1 0.25 0.25 0.273 0.316 0.413 0.358 79.2 13.2\nDubón, Mauricio 11 10 3 0 0 1 1 9.1 20 0 0 0.3 0.6 0.349 0.187 0.395 0.238 87.2 18.3\nHummel, Cooper 7 7 1 0 1 0 1 14.3 10 0 0 0.143 0.429 0.229 0.232 0.274 0.224 87.2 -4\nMcCormick, Chas 6 6 2 0 0 1 2 33.3 25 0 0 0.333 0.833 0.481 0.131 0.328 0.191 70 28.8\nCaratini, Victor 7 6 2 1 0 0 0 0 22.2 1 14.3 0.333 0.5 0.404 0.241 0.513 0.371 88 6.8\nPeña, Jeremy 6 6 1 0 0 1 1 16.7 10 0 0 0.167 0.667 0.348 0.127 0.396 0.211 87.7 31.4\nAltuve, Jose 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.991 0.336 1.065 0.577 98.2 15.5\nAlvarez, Yordan 3 2 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.005 0.009 0.009 84.4 47.5\nMeyers, Jake 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.445 0.549 0.431 94.7 20.3\nDiaz, Yainer 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.147 0.543 0.283 92.2 37'}, {'pitcher_id': '664875', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLawrence, Justin 4 4 1 0 0 1 1 25 50 0 0 0.25 1 0.518 0.168 0.503 0.284 91.6 24.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.036 0.333 1.002 0.565 95.1 18\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.009 0.005 84.6 38\nPeña, Jeremy 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '595928', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurtiss, John 9 8 4 1 0 0 2 22.2 23.5 1 11.1 0.5 0.625 0.51 0.341 0.481 0.394 94.9 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.294 0.3 0.459 0.324 91.3 18.3\nAltuve, Jose 3 2 2 1 0 0 0 0 25 1 33.3 1 1.5 0.943 0.457 0.652 0.557 96.1 -3.5\nDubón, Mauricio 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nCaratini, Victor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.914 1.164 0.904 103.2 12'}, {'pitcher_id': '663562', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656234', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBird, Jake 14 9 4 2 0 0 1 7.1 29.4 3 21.4 0.444 0.667 0.556 0.436 0.641 0.549 100.5 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 3 2 2 1 0 0 0 0 25 1 33.3 1 1.5 0.943 0.449 0.707 0.56 100 20.5\nCaratini, Victor 2 1 0 0 0 0 0 0 0 0 0 0 0 0.362 0.483 0.523 0.585 106.5 0\nHummel, Cooper 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.319 0.415 0.313 101.3 15\nAlvarez, Yordan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.32 0.636 0.4 98.3 21\nAltuve, Jose 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nPeña, Jeremy 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nDiaz, Yainer 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.631 1.306 0.797 106.6 18\nDubón, Mauricio 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.319 0.321 0.507 94.6 -3\nMcCormick, Chas 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.64 0.739 0.616 96.5 6'}, {'pitcher_id': '680767', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVodnik, Victor 6 5 2 0 1 0 1 16.7 27.3 1 16.7 0.4 0.8 0.532 0.285 0.319 0.337 85.5 16.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCormick, Chas 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBregman, Alex 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.006 0.014 0.01 88.7 50\nAltuve, Jose 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.091 0.076 70.7 -1\nDiaz, Yainer 1 1 1 0 1 0 0 0 0 0 0 1 3 1.612 0.957 1.073 0.889 86.4 17\nPeña, Jeremy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.376 0.415 0.354 96.3 -1'}], 118: [{'pitcher_id': '663372', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeltner, Ryan 4 3 0 0 0 0 1 25 30 0 0 0 0 0 0.342 0.479 0.302 102.2 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVelázquez, Nelson 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.09 0.091 0.086 107.8 -20\nNola, Austin 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.846 1.255 0.518 99.4 21.5'}, {'pitcher_id': '621389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlach, Ty 9 7 3 1 0 0 2 22.2 23.5 1 11.1 0.429 0.571 0.46 0.358 0.44 0.395 90.3 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.524 0.713 0.839 0.691 86.1 23\nPerez, Salvador 4 4 1 0 0 0 2 50 50 0 0 0.25 0.25 0.219 0.242 0.324 0.246 94.6 20.5\nLugo, Seth 2 1 1 1 0 0 0 0 25 0 0 1 2 1.232 0.11 0.11 0.106 90.2 -13'}, {'pitcher_id': '571539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarasiti, Matt 7 7 1 0 0 0 2 28.6 44.4 0 0 0.143 0.143 0.126 0.128 0.141 0.118 83.9 -9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.336 0.343 0.309 69.2 -59\nMelendez, MJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.156 0.18 0.145 96.2 -17\nGarcia, Maikel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.022 0.017 82.9 41\nWaters, Drew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.177 0.224 0.162 90 -12\nPerez, Salvador 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.217 0.196 81 -2'}, {'pitcher_id': '607536', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeland, Kyle 46 44 12 3 1 1 2 4.3 13.3 2 4.3 0.273 0.455 0.324 0.278 0.452 0.327 85.4 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 14 13 4 0 0 0 1 7.1 4.8 1 7.1 0.308 0.308 0.301 0.241 0.287 0.263 88 21.5\nWitt Jr., Bobby 6 6 1 0 1 0 0 0 17.6 0 0 0.167 0.5 0.267 0.403 1.012 0.58 94.9 13.5\nFrazier, Adam 6 6 2 1 0 0 1 16.7 10 0 0 0.333 0.5 0.354 0.212 0.279 0.212 77.7 6\nPerez, Salvador 6 6 2 1 0 0 0 0 11.1 0 0 0.333 0.5 0.358 0.318 0.416 0.318 87.8 30.3\nGarcia, Maikel 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.227 0.777 0.406 90.1 20\nPasquantino, Vinnie 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.156 0.231 0.163 79.5 23.7\nFermin, Freddy 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.288 0.34 0.278 88.6 -12.5\nHampson, Garrett 2 1 1 0 0 0 0 0 33.3 1 50 1 1 0.789 0.891 0.953 0.757 65.9 25\nMelendez, MJ 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.232 0.358 0.247 82.1 -7.7\nLugo, Seth 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.185 0.185 0.173 48.2 -70'}, {'pitcher_id': '683627', 'player_data': '', 'team_data': 'Anthony Molina has not faced anyone on this team.'}, {'pitcher_id': '663567', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLambert, Peter 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.466 0.587 0.47 72.2 36', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.466 0.587 0.47 72.2 36'}, {'pitcher_id': '656222', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBeeks, Jalen 10 10 2 2 0 0 1 10 6.7 0 0 0.2 0.4 0.246 0.266 0.331 0.254 91 6.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 4 4 1 1 0 0 1 25 0 0 0 0.25 0.5 0.304 0.333 0.432 0.323 98.7 18.7\nFrazier, Adam 3 3 1 1 0 0 0 0 20 0 0 0.333 0.667 0.415 0.24 0.274 0.223 83 -14\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.203 0.279 0.185 99.1 -12\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.014 0.009 89.8 47\nMelendez, MJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.403 0.469 0.38 84.6 6'}, {'pitcher_id': '615698', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuantrill, Cal 76 74 22 6 1 2 9 11.8 14.9 1 1.3 0.297 0.486 0.34 0.293 0.481 0.339 89.2 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 14 14 4 1 0 1 2 14.3 20 0 0 0.286 0.571 0.358 0.275 0.519 0.333 94.2 25.7\nFrazier, Adam 14 14 4 1 0 0 1 7.1 14.3 0 0 0.286 0.357 0.275 0.265 0.342 0.263 84.6 24\nIsbel, Kyle 12 11 3 2 0 0 3 25 4.8 1 8.3 0.273 0.455 0.338 0.199 0.339 0.262 90.6 18.1\nMelendez, MJ 9 9 2 0 1 0 1 11.1 22.2 0 0 0.222 0.444 0.273 0.387 0.606 0.426 92.4 -1.3\nWitt Jr., Bobby 11 10 3 1 0 0 0 0 5.3 0 0 0.3 0.4 0.341 0.205 0.312 0.267 89.6 26.8\nWaters, Drew 4 4 1 1 0 0 1 25 25 0 0 0.25 0.5 0.311 0.243 0.521 0.318 86.7 2.3\nGarcia, Maikel 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.445 0.547 0.429 101 -3\nVelázquez, Nelson 3 3 1 0 0 1 1 33.3 50 0 0 0.333 1.333 0.668 0.434 1.287 0.706 100.7 18.5\nHampson, Garrett 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.408 0.435 0.381 67.5 2.5\nPasquantino, Vinnie 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.579 0.949 0.66 87 23'}, {'pitcher_id': '641712', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Dakota 35 29 8 0 0 2 3 8.6 15.1 6 17.1 0.276 0.483 0.388 0.308 0.563 0.425 87.5 12.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 14 11 4 0 0 0 0 0 4.5 3 21.4 0.364 0.364 0.399 0.335 0.424 0.409 88 5.6\nPerez, Salvador 5 5 1 0 0 1 1 20 22.2 0 0 0.2 0.8 0.414 0.246 0.858 0.441 91.8 38.5\nNola, Austin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.251 0.299 0.244 77.6 24\nWitt Jr., Bobby 5 4 1 0 0 1 1 20 40 1 20 0.25 1 0.552 0.327 0.759 0.505 101.9 17\nHampson, Garrett 2 1 0 0 0 0 1 50 25 1 50 0 0 0.344 0.344\nVelázquez, Nelson 3 2 0 0 0 0 0 0 40 1 33.3 0 0 0.23 0.235 0.286 0.381 55.8 -20.5\nMelendez, MJ 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.497 0.956 0.615 96.4 9.7'}, {'pitcher_id': '641755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKinley, Tyler 9 7 2 0 0 0 2 22.2 23.5 1 11.1 0.286 0.286 0.272 0.201 0.279 0.239 92.8 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 4 3 2 0 0 0 1 25 60 1 25 0.667 0.667 0.612 0.3 0.323 0.377 83 8\nNola, Austin 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.084 0.043 95.8 7\nHampson, Garrett 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMelendez, MJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.383 0.759 0.487 100.5 20\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.139 0.069 99.1 43'}, {'pitcher_id': '683232', 'player_data': '', 'team_data': 'Nick Mears has not faced anyone on this team.'}, {'pitcher_id': '596295', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomber, Austin 45 38 15 1 0 1 7 15.6 16.3 6 13.3 0.395 0.5 0.435 0.242 0.364 0.327 87.3 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 11 10 2 0 0 0 2 18.2 17.6 0 0 0.2 0.2 0.226 0.151 0.216 0.207 90.5 15.5\nNola, Austin 11 11 6 0 0 0 0 0 0 0 0 0.545 0.545 0.48 0.287 0.363 0.283 84.7 25.9\nFrazier, Adam 8 7 4 1 0 1 1 12.5 9.1 1 12.5 0.571 1.143 0.713 0.379 0.736 0.497 87.5 8.2\nWitt Jr., Bobby 4 2 2 0 0 0 0 0 14.3 2 50 1 1 0.786 0.656 0.77 0.652 91.9 7.5\nMelendez, MJ 4 3 0 0 0 0 2 50 50 1 25 0 0 0.174 0.018 0.025 0.188 65.8 -33\nGarcia, Maikel 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.197 0.385 0.24 100.5 19\nPasquantino, Vinnie 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.13 0.13 0.407 89 -12\nHampson, Garrett 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.003 0.003 0.003 85.9 46\nWaters, Drew 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '664875', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLawrence, Justin 8 8 2 0 0 0 1 12.5 30.8 0 0 0.25 0.25 0.221 0.326 0.373 0.307 94.1 18.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.002 0.003 0.002 82.6 66\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.657 0.766 0.618 105.6 6\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.197 0.228 0.177 80.6 26\nPerez, Salvador 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.38 0.483 0.38 71 41\nMelendez, MJ 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.546 0.609 0.517 109.4 0\nNola, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.313 0.333 0.285 102.6 -7\nWitt Jr., Bobby 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.514 0.564 0.474 106.6 0'}, {'pitcher_id': '595928', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurtiss, John 9 8 4 1 0 0 2 22.2 27.3 1 11.1 0.5 0.625 0.509 0.276 0.384 0.335 89.5 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 4 4 2 1 0 0 1 25 40 0 0 0.5 0.75 0.531 0.338 0.465 0.35 93.9 30.3\nMelendez, MJ 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGarcia, Maikel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.709 1.046 0.751 101.3 15\nWitt Jr., Bobby 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nHampson, Garrett 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.139 0.156 0.156 89.8 -15\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.008 0.007 64.4 74'}, {'pitcher_id': '663562', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656234', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBird, Jake 8 6 0 0 0 0 1 12.5 27.3 2 25 0 0 0.174 0.173 0.2 0.298 90.3 -12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 3 1 0 0 0 0 0 0 0 2 66.7 0 0 0.464 0.244 0.244 0.538 77.5 -57\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.153 0.083 91.9 35\nWaters, Drew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.24 0.214 96 -13\nWitt Jr., Bobby 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.504 0.418 98.6 2\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.056 0.053 87.5 -29'}, {'pitcher_id': '680767', 'player_data': '', 'team_data': 'Victor Vodnik has not faced anyone on this team.'}], 119: [{'pitcher_id': '663372', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeltner, Ryan 48 41 10 2 0 1 5 10.4 11.1 6 12.5 0.244 0.366 0.324 0.241 0.425 0.342 86.9 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 11 8 1 1 0 0 0 0 10.5 2 18.2 0.125 0.25 0.305 0.212 0.327 0.359 83.4 5.3\nBetts, Mookie 7 5 1 0 0 0 2 28.6 22.2 2 28.6 0.2 0.2 0.323 0.371 0.544 0.475 99.7 16\nSmith, Will 6 5 0 0 0 0 0 0 0 1 16.7 0 0 0.116 0.233 0.469 0.364 88.1 20.2\nRojas, Miguel 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0.132 0.188 0.136 79.8 2.2\nTaylor, Chris 4 4 2 0 0 1 0 0 0 0 0 0.5 1.25 0.722 0.468 1.247 0.704 92.2 37\nBiggio, Cavan 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.593 0.188 0.212 0.174 87.3 -3\nBarnes, Austin 2 1 0 0 0 0 0 0 25 1 50 0 0 0.344 0.109 0.2 0.409 82.1 29\nLux, Gavin 4 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.315 0.351 0.593 0.4 94.4 7.3\nHernández, Enrique 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.102 0.117 0.093 82.5 31\nHeyward, Jason 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.068 0.068 0.061 49.7 5\nHernández, Teoscar 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.112 0.114 0.102 106.4 -20'}, {'pitcher_id': '621389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlach, Ty 112 103 22 8 0 2 21 18.8 18 8 7.1 0.214 0.35 0.273 0.255 0.381 0.305 89.1 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Enrique 25 22 5 1 0 1 3 12 7.5 2 8 0.227 0.409 0.316 0.308 0.47 0.375 91.5 14.5\nTaylor, Chris 30 28 8 3 0 1 6 20 26.1 2 6.7 0.286 0.5 0.353 0.313 0.472 0.358 87 10.1\nFreeman, Freddie 11 10 1 1 0 0 2 18.2 18.2 1 9.1 0.1 0.2 0.176 0.267 0.394 0.322 89 0.9\nBarnes, Austin 12 12 1 1 0 0 3 25 20 0 0 0.083 0.167 0.101 0.075 0.127 0.085 86.4 3.8\nHeyward, Jason 10 10 3 0 0 0 2 20 19 0 0 0.3 0.3 0.263 0.207 0.265 0.204 88.9 11.6\nBetts, Mookie 8 7 1 1 0 0 3 37.5 30 1 12.5 0.143 0.286 0.239 0.192 0.39 0.294 94.4 24.5\nRojas, Miguel 5 3 0 0 0 0 0 0 0 2 40 0 0 0.277 0.365 0.495 0.502 95.7 13.7\nLux, Gavin 6 6 1 0 0 0 1 16.7 25 0 0 0.167 0.167 0.147 0.205 0.236 0.197 86.1 18.4\nSmith, Will 4 4 2 1 0 0 1 25 20 0 0 0.5 0.75 0.533 0.36 0.471 0.354 90.7 18\nBuehler, Walker 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.013 0.026 0.015 88 53'}, {'pitcher_id': '571539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarasiti, Matt 8 7 2 0 0 0 3 37.5 27.3 0 0 0.286 0.286 0.221 0.074 0.095 0.071 93.5 1.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeyward, Jason 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.103 0.123 0.095 97.1 -29\nBetts, Mookie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nPages, Andy 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBarnes, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHernández, Enrique 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.148 0.098 81.9 30\nOhtani, Shohei 1 0 0 0 0 0 0 0 0 0 0 0 0.084 94.2 43\nRojas, Miguel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.217 0.274 0.198 97.3 -8'}, {'pitcher_id': '607536', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeland, Kyle 228 198 46 14 0 8 50 21.9 21.2 24 10.5 0.232 0.424 0.323 0.202 0.393 0.304 87.9 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Chris 38 30 7 1 0 2 8 21.1 27.4 7 18.4 0.233 0.467 0.378 0.175 0.354 0.323 81.2 1.8\nFreeman, Freddie 36 31 7 3 0 0 11 30.6 29.7 3 8.3 0.226 0.323 0.279 0.166 0.254 0.235 89.5 16.7\nBetts, Mookie 36 34 7 4 0 2 11 30.6 24.2 2 5.6 0.206 0.5 0.314 0.231 0.552 0.345 93.6 25.9\nBarnes, Austin 33 29 3 0 0 1 5 15.2 8.3 4 12.1 0.103 0.207 0.198 0.174 0.293 0.262 83.7 2.4\nHernández, Enrique 27 21 4 1 0 0 3 11.1 10 4 14.8 0.19 0.238 0.273 0.189 0.32 0.311 91.3 9.4\nSmith, Will 19 17 5 2 0 1 4 21.1 16.7 2 10.5 0.294 0.588 0.403 0.207 0.497 0.335 88.4 21.8\nRojas, Miguel 14 13 5 2 0 1 2 14.3 23.3 1 7.1 0.385 0.769 0.495 0.245 0.415 0.311 83.8 25.7\nLux, Gavin 13 12 5 0 0 0 3 23.1 18.2 1 7.7 0.417 0.417 0.392 0.261 0.307 0.281 89.7 0.9\nBuehler, Walker 7 6 1 0 0 0 3 42.9 37.5 0 0 0.167 0.167 0.147 0.129 0.143 0.122 89 -2.3\nOhtani, Shohei 3 3 2 1 0 1 0 0 0 0 0 0.667 2 1.083 0.698 2.291 1.216 107.8 28.3\nHeyward, Jason 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.026 0.028 0.033 78 30'}, {'pitcher_id': '683627', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMolina, Anthony 9 8 2 0 0 0 0 0 8.3 0 0 0.25 0.25 0.198 0.238 0.28 0.223 84.6 19.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.694 0.912 0.677 100 15\nPages, Andy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.01 0.008 81.5 62\nFreeman, Freddie 1 0 0 0 0 0 0 0 0 0 0 0 0.215 88.2 23\nHernández, Teoscar 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.119 0.13 0.109 91.5 -29\nBetts, Mookie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.014 0.01 83.4 65\nHernández, Enrique 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.119 0.09 83.6 -9\nHeyward, Jason 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.843 0.9 0.766 73.9 17\nRojas, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.024 0.022 67.6 61'}, {'pitcher_id': '663567', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLambert, Peter 56 48 20 3 0 2 9 16.1 18.4 5 8.9 0.417 0.604 0.454 0.323 0.485 0.378 89.1 10.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Chris 8 6 3 1 0 0 1 12.5 20.8 0 0 0.5 0.667 0.459 0.517 0.664 0.476 87 16.5\nBarnes, Austin 7 7 1 0 0 0 2 28.6 7.7 0 0 0.143 0.143 0.127 0.181 0.232 0.179 82.1 7.4\nBetts, Mookie 6 4 1 0 0 0 1 16.7 0 2 33.3 0.25 0.25 0.376 0.242 0.313 0.391 82.1 18\nFreeman, Freddie 5 4 1 0 0 0 0 0 16.7 1 20 0.25 0.25 0.315 0.403 0.599 0.479 93.8 20.8\nSmith, Will 6 6 2 0 0 1 2 33.3 28.6 0 0 0.333 0.833 0.479 0.192 0.641 0.34 91.6 3.8\nHernández, Enrique 5 4 2 1 0 0 0 0 25 1 20 0.5 0.75 0.564 0.395 0.611 0.482 93.2 23.5\nHeyward, Jason 6 6 5 1 0 1 0 0 14.3 0 0 0.833 1.5 0.97 0.381 0.603 0.418 97.8 5.2\nOhtani, Shohei 3 2 1 0 0 0 1 33.3 33.3 0 0 0.5 0.5 0.441 0.357 0.432 0.339 110.5 7\nLux, Gavin 3 2 2 0 0 0 0 0 33.3 1 33.3 1 1 0.81 0.93 1.088 0.81 91.8 15\nBuehler, Walker 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.58 0.233 0.25 0.219 84.6 -23.3\nHernández, Teoscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.111 0.162 0.11 82.9 27\nPages, Andy 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRojas, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.185 0.153 52.8 19\nBiggio, Cavan 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '656222', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBeeks, Jalen 60 51 13 3 0 1 15 25 30.9 8 13.3 0.255 0.373 0.332 0.233 0.409 0.336 88.6 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 13 12 4 3 0 0 4 30.8 33.3 1 7.7 0.333 0.583 0.408 0.216 0.327 0.266 92.9 16.1\nBetts, Mookie 10 6 0 0 0 0 1 10 15.4 4 40 0 0 0.277 0.341 0.793 0.563 83.5 9.6\nHernández, Enrique 8 8 2 0 0 0 1 12.5 31.3 0 0 0.25 0.25 0.219 0.189 0.349 0.226 85.9 15.7\nOhtani, Shohei 5 5 2 0 0 1 3 60 50 0 0 0.4 1 0.588 0.288 0.764 0.443 103.9 13\nRojas, Miguel 6 6 1 0 0 0 0 0 9.1 0 0 0.167 0.167 0.145 0.23 0.26 0.222 87.7 -4.3\nBiggio, Cavan 5 2 0 0 0 0 2 40 66.7 2 40 0 0 0.42 0.42\nFreeman, Freddie 6 5 2 0 0 0 1 16.7 22.2 1 16.7 0.4 0.4 0.409 0.342 0.536 0.426 89.6 27.3\nBarnes, Austin 3 3 1 0 0 0 2 66.7 40 0 0 0.333 0.333 0.29 0.131 0.139 0.118 85.2 7\nTaylor, Chris 2 2 0 0 0 0 1 50 80 0 0 0 0 0 0.203 0.205 0.183 104.5 -1\nSmith, Will 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.202 0.235 0.183 72.4 5'}, {'pitcher_id': '615698', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuantrill, Cal 83 76 22 7 2 0 13 15.7 19.7 5 6 0.289 0.434 0.333 0.224 0.362 0.282 87 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 12 9 2 1 0 0 4 33.3 29 3 25 0.222 0.333 0.35 0.123 0.17 0.281 94 39\nFreeman, Freddie 12 12 6 2 1 0 2 16.7 24.2 0 0 0.5 0.833 0.561 0.247 0.427 0.28 94.4 16.9\nSmith, Will 11 11 3 2 0 0 0 0 7.7 0 0 0.273 0.455 0.306 0.273 0.453 0.313 81.4 14.9\nHernández, Teoscar 11 11 2 0 0 0 1 9.1 5.3 0 0 0.182 0.182 0.162 0.24 0.495 0.307 89.1 15.6\nHeyward, Jason 8 6 3 1 0 0 1 12.5 20 2 25 0.5 0.667 0.549 0.353 0.432 0.424 87.4 17\nBiggio, Cavan 4 4 0 0 0 0 2 50 27.3 0 0 0 0 0 0.038 0.046 0.036 85.4 22\nHernández, Enrique 4 4 2 0 0 0 0 0 22.2 0 0 0.5 0.5 0.44 0.297 0.363 0.286 81.5 20\nLux, Gavin 5 5 1 0 1 0 1 20 16.7 0 0 0.2 0.6 0.32 0.145 0.207 0.149 88.8 10.7\nBuehler, Walker 4 4 0 0 0 0 2 50 33.3 0 0 0 0 0 0.138 0.138 0.128 47.1 -35.5\nTaylor, Chris 4 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.497 0.258 0.658 0.47 94.2 39.7\nBetts, Mookie 4 4 0 0 0 0 0 0 16.7 0 0 0 0 0 0.065 0.189 0.101 87.4 52.5\nPages, Andy 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.718 0.83 0.677 101.3 7.5\nBarnes, Austin 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.014 0.134 79.1 40'}, {'pitcher_id': '641712', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Dakota 70 63 13 4 0 1 9 12.9 18.6 5 7.1 0.206 0.317 0.262 0.279 0.441 0.339 88.6 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 12 11 3 1 0 0 2 16.7 22.7 1 8.3 0.273 0.364 0.31 0.382 0.681 0.478 94.4 15.4\nSmith, Will 8 7 2 1 0 0 0 0 7.1 0 0 0.286 0.429 0.359 0.362 0.642 0.46 94.7 16.1\nRojas, Miguel 9 7 2 1 0 0 0 0 15.4 2 22.2 0.286 0.429 0.39 0.422 0.515 0.471 86.2 -1.4\nHeyward, Jason 7 5 0 0 0 0 1 14.3 8.3 1 14.3 0 0 0.099 0.269 0.305 0.292 82.6 8.6\nBetts, Mookie 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.102 0.084 85.5 17.3\nLux, Gavin 6 5 3 1 0 1 0 0 0 1 16.7 0.6 1.4 0.82 0.261 0.631 0.427 87.7 -3.6\nOhtani, Shohei 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.297 0.331 0.289 92.3 -2.3\nTaylor, Chris 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.241 0.266 0.217 81.3 3\nPages, Andy 2 2 1 0 0 0 1 50 20 0 0 0.5 0.5 0.444 0.484 0.522 0.444 77.7 18\nHernández, Teoscar 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.195 0.766 0.389 99.6 50.5\nBiggio, Cavan 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.121 0.152 0.119 82 20\nHernández, Enrique 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.435 0.205 0.227 0.184 87.3 -12.5\nBarnes, Austin 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.05 0.066 0.05 84.1 5.5\nBuehler, Walker 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '641755', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKinley, Tyler 41 33 15 2 1 3 3 7.3 22.8 6 14.6 0.455 0.848 0.555 0.327 0.559 0.426 88.1 20.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBetts, Mookie 9 8 2 1 0 0 0 0 0 1 11.1 0.25 0.375 0.313 0.304 0.405 0.348 87.3 11.4\nFreeman, Freddie 6 4 3 0 0 1 0 0 21.4 1 16.7 0.75 1.5 0.851 0.514 1.031 0.654 96.8 14.5\nTaylor, Chris 7 7 4 0 0 1 1 14.3 18.8 0 0 0.571 1 0.665 0.252 0.352 0.263 74.1 26.5\nSmith, Will 4 3 2 0 0 1 1 25 33.3 1 25 0.667 1.667 0.887 0.295 0.799 0.511 92.3 -14\nRojas, Miguel 3 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.293 0.15 0.222 0.194 86.8 36.3\nLux, Gavin 4 2 1 0 1 0 1 25 33.3 2 50 0.5 1.5 0.739 0.44 0.561 0.563 105.5 11\nBarnes, Austin 4 4 1 1 0 0 0 0 16.7 0 0 0.25 0.5 0.311 0.261 0.745 0.409 92.3 41.3\nHeyward, Jason 2 1 0 0 0 0 0 0 50 1 50 0 0 0.345 0.473 0.561 0.572 103.4 1\nHernández, Enrique 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.48 0.561 0.46 88.6 33.5'}, {'pitcher_id': '683232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMears, Nick 14 12 2 0 0 0 4 28.6 25 1 7.1 0.167 0.167 0.228 0.129 0.152 0.206 85.9 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBetts, Mookie 3 2 0 0 0 0 1 33.3 60 1 33.3 0 0 0.232 0.222 0.239 0.365 98.1 2\nHeyward, Jason 2 2 0 0 0 0 1 50 12.5 0 0 0 0 0 0.007 0.013 0.009 84.9 43\nSmith, Will 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.104 0.065 84.1 38.5\nFreeman, Freddie 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.197 0.218 0.454 86 -3\nLux, Gavin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nPages, Andy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.426 0.5 0.409 85.5 6\nHernández, Enrique 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.092 0.094 0.083 83.2 -47\nHernández, Teoscar 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRojas, Miguel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.262 0.303 0.24 81.5 4'}, {'pitcher_id': '596295', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGomber, Austin 92 77 24 3 1 4 13 14.1 18.2 13 14.1 0.312 0.532 0.407 0.313 0.526 0.406 90.1 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 12 11 6 1 0 2 0 0 6.9 1 8.3 0.545 1.182 0.729 0.573 1.076 0.699 94.6 13.3\nHernández, Enrique 12 10 3 0 0 0 0 0 12 2 16.7 0.3 0.3 0.335 0.29 0.438 0.373 89.1 20.8\nTaylor, Chris 11 8 2 1 0 0 2 18.2 14.3 3 27.3 0.25 0.375 0.381 0.29 0.66 0.476 92.5 26.8\nRojas, Miguel 8 7 3 0 0 0 1 12.5 22.2 1 12.5 0.429 0.429 0.418 0.322 0.37 0.359 87.8 6.8\nSmith, Will 10 9 0 0 0 0 4 40 35.7 0 0 0 0 0.073 0.091 0.11 0.15 81.2 11.2\nBetts, Mookie 10 7 3 1 0 1 0 0 9.1 3 30 0.429 1 0.629 0.377 0.599 0.502 99.5 20.1\nOhtani, Shohei 8 8 3 0 1 1 2 25 29.4 0 0 0.375 1 0.557 0.353 0.784 0.471 90.2 16.7\nHeyward, Jason 6 5 0 0 0 0 1 16.7 22.2 1 16.7 0 0 0.116 0.086 0.135 0.199 80 7.3\nLux, Gavin 4 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.221 0.502 0.66 0.377 98.4 23.5\nBarnes, Austin 5 4 1 0 0 0 2 40 25 1 20 0.25 0.25 0.314 0.171 0.174 0.266 80 -31\nHernández, Teoscar 4 4 1 0 0 0 1 25 16.7 0 0 0.25 0.25 0.221 0.185 0.206 0.17 84.2 -18\nPages, Andy 2 1 1 0 0 0 0 0 25 1 50 1 1 0.791 0.703 0.786 0.679 69.6 35'}, {'pitcher_id': '664875', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLawrence, Justin 29 25 10 2 1 1 6 20.7 25 3 10.3 0.4 0.68 0.466 0.374 0.667 0.469 94.5 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 7 6 2 0 0 0 1 14.3 10 1 14.3 0.333 0.333 0.351 0.488 0.997 0.642 95.6 27.8\nLux, Gavin 5 5 2 1 0 0 1 20 27.3 0 0 0.4 0.6 0.428 0.172 0.249 0.183 95.3 18\nFreeman, Freddie 5 5 3 1 0 1 1 20 33.3 0 0 0.6 1.4 0.83 0.575 1.271 0.778 96.6 20.8\nHeyward, Jason 4 4 2 0 1 0 0 0 20 0 0 0.5 1 0.621 0.456 0.534 0.441 93.2 8.8\nBetts, Mookie 4 3 1 0 0 0 1 25 14.3 0 0 0.333 0.333 0.221 0.289 0.322 0.287 90.4 11.7\nOhtani, Shohei 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nTaylor, Chris 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nBiggio, Cavan 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '595928', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCurtiss, John 21 19 3 0 0 1 3 14.3 31.6 1 4.8 0.158 0.316 0.211 0.246 0.466 0.304 92.4 20.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Austin 3 2 1 0 0 1 0 0 25 1 33.3 0.5 2 0.892 0.7 2.024 0.967 102.4 14\nHernández, Teoscar 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.189 0.314 0.223 82 13.5\nBiggio, Cavan 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.056 0.095 0.061 78.5 12.7\nBetts, Mookie 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.318 0.443 0.328 99.5 18\nSmith, Will 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.005 0.007 0.005 96.1 53.5\nTaylor, Chris 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.065 0.083 0.068 96.1 8.5\nFreeman, Freddie 3 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.293 0.708 0.808 0.442 95.7 24\nLux, Gavin 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.271 0.799 0.428 95.9 28'}, {'pitcher_id': '663562', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, Noah 21 17 4 0 0 0 4 19 19 1 4.8 0.235 0.235 0.27 0.29 0.344 0.351 87.8 10.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 4 2 0 0 0 0 0 0 14.3 0 0 0 0 0.182 0.034 0.035 0.328 92.6 20\nFreeman, Freddie 4 3 1 0 0 0 2 50 33.3 0 0 0.333 0.333 0.402 0.21 0.232 0.326 99.4 7\nBetts, Mookie 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.523 0.68 0.581 81.9 26.5\nOhtani, Shohei 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.155 0.201 0.158 83.3 -19.5\nHernández, Teoscar 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.246 0.29 0.233 81 -3.5\nHeyward, Jason 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.395 0.467 0.38 88.4 28.5\nBarnes, Austin 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRojas, Miguel 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.723 0.803 0.66 91.7 11'}, {'pitcher_id': '656234', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBird, Jake 36 30 8 0 0 1 5 13.9 22.6 2 5.6 0.267 0.367 0.306 0.296 0.434 0.372 89.1 8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBetts, Mookie 8 6 0 0 0 0 3 37.5 27.8 1 12.5 0 0 0.178 0.085 0.207 0.267 85.4 3.3\nFreeman, Freddie 5 4 1 0 0 0 0 0 25 0 0 0.25 0.25 0.177 0.33 0.659 0.451 97.3 15.4\nTaylor, Chris 4 4 2 0 0 0 0 0 11.1 0 0 0.5 0.5 0.442 0.478 0.539 0.453 76 9.3\nLux, Gavin 5 4 1 0 0 0 1 20 40 1 20 0.25 0.25 0.315 0.384 0.431 0.422 101 -1.7\nSmith, Will 3 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.295 0.289 0.36 0.376 97.6 18\nBarnes, Austin 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.534 0.442 0.467 0.506 91.3 4.5\nHernández, Teoscar 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nHernández, Enrique 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.092 0.092 0.083 67.1 -15\nRojas, Miguel 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.237 0.296 0.23 81.8 13.3\nBiggio, Cavan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.694 0.912 0.677 99.8 15\nHeyward, Jason 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.546 1.622 0.9 100.2 25'}, {'pitcher_id': '680767', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVodnik, Victor 8 7 3 1 0 0 1 12.5 27.8 1 12.5 0.429 0.571 0.463 0.248 0.303 0.296 85.9 0.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.05 0.047 87.6 -39\nOhtani, Shohei 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nTaylor, Chris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHernández, Enrique 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.441 0.223 0.358 0.253 100 11.5\nPages, Andy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.909 0.951 0.821 67.5 24\nFreeman, Freddie 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.197 0.254 0.179 71.3 5\nBetts, Mookie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.135 0.148 0.122 89.2 -11'}]}, 116: {115: [{'pitcher_id': '669438', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEnglert, Mason 4 4 1 0 0 0 1 25 14.3 0 0 0.25 0.25 0.221 0.264 0.297 0.247 100.1 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.381 0.388 0.345 98.1 -4\nJones, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMcMahon, Ryan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.669 0.796 0.64 109.9 5\nTovar, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 92.2 44'}, {'pitcher_id': '663554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMize, Casey 10 7 3 0 0 1 4 40 23.8 2 20 0.429 0.857 0.586 0.334 0.737 0.524 95.6 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 10 7 3 0 0 1 4 40 23.8 2 20 0.429 0.857 0.586 0.334 0.737 0.524 95.6 18'}, {'pitcher_id': '676684', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVest, Will 4 4 0 0 0 0 1 25 44.4 0 0 0 0 0 0.128 0.139 0.116 74 1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.242 0.265 0.221 83.4 -26.5\nJones, Nolan 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBlackmon, Charlie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.026 0.023 55.3 56'}, {'pitcher_id': '656638', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLange, Alex 8 8 0 0 0 0 5 62.5 44.4 0 0 0 0 0 0.06 0.088 0.063 84.7 28.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nMcMahon, Ryan 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.026 0.02 81.6 61\nTovar, Ezequiel 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.194 0.386 0.235 94.9 23\nJones, Nolan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.266 0.295 0.245 77.7 2'}, {'pitcher_id': '687922', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '689225', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrieske, Beau 8 8 3 0 0 1 1 12.5 28.6 0 0 0.375 0.75 0.48 0.303 0.487 0.341 94.1 3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.005 0.012 0.008 98.7 53\nBlackmon, Charlie 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.691 0.431 0.835 0.536 99 9\nTrejo, Alan 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.442 0.513 0.637 0.51 81.3 -17.5\nStallings, Jacob 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.094 0.094 0.086 100.8 -20'}, {'pitcher_id': '663947', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolton, Tyler 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoyle, Brenton 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '681857', 'player_data': '', 'team_data': 'Reese Olson has not faced anyone on this team.'}, {'pitcher_id': '666214', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWentz, Joey 4 3 1 0 0 0 1 25 22.2 1 25 0.333 0.333 0.395 0.134 0.178 0.275 89 0.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStallings, Jacob 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nCave, Jake 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.201 0.267 0.202 89 0.5'}, {'pitcher_id': '656412', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '628317', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaeda, Kenta 47 41 12 4 0 2 6 12.8 17.9 5 10.6 0.293 0.537 0.388 0.308 0.596 0.42 88.9 8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 28 24 8 3 0 1 1 3.6 2.5 3 10.7 0.333 0.583 0.427 0.363 0.72 0.489 88.7 10.2\nMcMahon, Ryan 14 12 4 1 0 1 2 14.3 38.1 2 14.3 0.333 0.667 0.449 0.286 0.531 0.394 89.2 5.1\nStallings, Jacob 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nBlach, Ty 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.166 0.263 0.176 88.9 2.5'}, {'pitcher_id': '656427', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlaherty, Jack 49 47 13 3 0 2 10 20.4 26.4 2 4.1 0.277 0.468 0.331 0.289 0.493 0.348 87.3 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 13 12 3 1 0 0 1 7.7 16.7 1 7.7 0.25 0.333 0.284 0.385 0.644 0.455 89.8 10\nBlackmon, Charlie 15 14 5 1 0 1 3 20 33.3 1 6.7 0.357 0.643 0.441 0.267 0.452 0.334 88.6 18.3\nStallings, Jacob 5 5 1 0 0 0 1 20 28.6 0 0 0.2 0.2 0.177 0.135 0.15 0.127 80.2 27.3\nTovar, Ezequiel 5 5 1 0 0 0 1 20 28.6 0 0 0.2 0.2 0.177 0.35 0.432 0.341 78.3 0.3\nJones, Nolan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.257 0.282 0.237 106.9 0\nGomber, Austin 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.033 0.045 0.035 73.9 56\nGoodman, Hunter 3 3 1 0 0 0 1 33.3 60 0 0 0.333 0.333 0.294 0.311 0.869 0.482 93.5 17.5\nDoyle, Brenton 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.622 0.28 0.296 0.252 93.5 8\nToglia, Michael 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.361 1.18 0.635 85.7 25.5'}, {'pitcher_id': '666159', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManning, Matt 15 11 3 2 0 0 3 20 21.4 0 0 0.273 0.455 0.274 0.163 0.257 0.252 93.1 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 4 4 2 1 0 0 1 25 11.1 0 0 0.5 0.75 0.536 0.23 0.31 0.231 90.9 -3.7\nMcMahon, Ryan 3 3 0 0 0 0 1 33.3 22.2 0 0 0 0 0 0.098 0.098 0.09 89 -10.5\nTovar, Ezequiel 3 2 0 0 0 0 1 33.3 75 0 0 0 0 0.242 0.009 0.015 0.248 84.7 54\nMontero, Elehuris 2 1 1 1 0 0 0 0 0 0 0 1 2 0.622 0.329 1.012 0.682 99.3 25\nDoyle, Brenton 2 0 0 0 0 0 0 0 0 0 0 0 0.13 94 37\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.249 0.217 102.3 -5'}, {'pitcher_id': '671345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFoley, Jason 4 3 0 0 0 0 2 50 50 1 25 0 0 0.172 0 0 0.172 96.4 50', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMontero, Elehuris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCave, Jake 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nJones, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 96.4 50'}, {'pitcher_id': '669373', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSkubal, Tarik 8 8 1 0 0 0 3 37.5 25 0 0 0.125 0.125 0.11 0.187 0.342 0.232 89.6 29.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 4 4 0 0 0 0 1 25 30 0 0 0 0 0 0.18 0.458 0.271 94.9 21.7\nStallings, Jacob 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.44 0.39 0.453 0.388 69.1 34\nMcMahon, Ryan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0 0 0 94.5 50'}, {'pitcher_id': '571946', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Shelby 36 33 8 1 1 1 6 16.7 18.1 3 8.3 0.242 0.424 0.315 0.224 0.429 0.313 88.2 24.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 28 26 8 1 1 1 2 7.1 12.7 2 7.1 0.308 0.538 0.38 0.286 0.554 0.38 89.7 26.1\nMcMahon, Ryan 2 1 0 0 0 0 1 50 60 1 50 0 0 0.348 0.348\nDoyle, Brenton 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.031 0.081 0.043 98.7 45\nToglia, Michael 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nJones, Nolan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nMontero, Elehuris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTovar, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.14 0.184 0.126 56 6\nTrejo, Alan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.108 0.079 82.4 -15'}, {'pitcher_id': '605177', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChafin, Andrew 55 51 18 1 0 2 14 25.5 30.2 4 7.3 0.353 0.49 0.383 0.265 0.381 0.31 84.8 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 35 32 10 1 0 1 6 17.1 26.5 3 8.6 0.313 0.438 0.351 0.268 0.373 0.314 82.7 5.4\nMcMahon, Ryan 10 10 5 0 0 0 5 50 47.1 0 0 0.5 0.5 0.435 0.244 0.278 0.227 102.1 1.8\nCave, Jake 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.295 0.249 0.275 0.231 82.7 -7.5\nToglia, Michael 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.674 0.775 0.665 107.3 6\nStallings, Jacob 2 2 1 0 0 1 0 0 16.7 0 0 0.5 2 0.99 0.391 1.395 0.723 84.2 5.5\nDoyle, Brenton 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.162 0.166 0.149 35.7 -66\nMontero, Elehuris 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}], 117: [{'pitcher_id': '669438', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEnglert, Mason 17 16 7 3 0 1 2 11.8 27.8 1 5.9 0.438 0.813 0.544 0.377 0.769 0.494 92 15.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 3 2 2 0 0 0 0 0 14.3 1 33.3 1 1 0.821 0.783 0.976 0.736 92.4 16.5\nPeña, Jeremy 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.271 0.356 0.266 91.8 12.7\nMcCormick, Chas 3 3 1 0 0 1 1 33.3 50 0 0 0.333 1.333 0.695 0.376 1.274 0.677 96.5 14\nDubón, Mauricio 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.537 0.601 0.499 88.3 0.5\nSingleton, Jon 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.006 0.006 0.005 86 46\nMeyers, Jake 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSalazar, César 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.111 0.305 0.175 91.7 29\nAlvarez, Yordan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.052 0.052 0.051 74.9 -14\nCabbage, Trey 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.769 1.939 1.117 106.9 20\nDiaz, Yainer 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.514 1.965 1.012 99.7 31'}, {'pitcher_id': '663554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMize, Casey 40 35 6 1 0 0 5 12.5 17.9 4 10 0.171 0.2 0.23 0.251 0.33 0.305 89.3 7.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 9 8 1 0 0 0 2 22.2 23.5 1 11.1 0.125 0.125 0.176 0.162 0.217 0.22 92.3 20.5\nAltuve, Jose 9 7 1 0 0 0 0 0 7.7 2 22.2 0.143 0.143 0.251 0.288 0.346 0.366 81.9 -3\nBregman, Alex 5 4 1 1 0 0 0 0 0 1 20 0.25 0.5 0.392 0.263 0.38 0.355 98 14.3\nPeña, Jeremy 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.296 0.167 0.186 0.151 85.1 -5\nSingleton, Jon 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.443 0.528 0.417 96.2 1.7\nCaratini, Victor 4 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.404 0.356 0.472 0.455 91.6 -2\nMcCormick, Chas 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.293 0.366 0.615 0.414 99.9 26\nDiaz, Yainer 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.069 0.072 0.061 56.9 45\nMeyers, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.144 0.166 0.131 84.8 -11'}, {'pitcher_id': '676684', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVest, Will 32 27 10 2 0 0 2 6.3 10.7 2 6.3 0.37 0.444 0.354 0.323 0.485 0.366 88.1 8.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 7 6 2 1 0 0 1 14.3 0 0 0 0.333 0.5 0.304 0.32 0.414 0.352 93.2 13\nAltuve, Jose 5 3 2 1 0 0 0 0 0 1 20 0.667 1 0.563 0.444 0.63 0.417 87.2 21.5\nDubón, Mauricio 4 4 1 0 0 0 0 0 22.2 0 0 0.25 0.25 0.221 0.354 0.532 0.383 90.1 11.8\nPeña, Jeremy 4 3 0 0 0 0 1 25 23.1 0 0 0 0 0 0.125 0.141 0.117 57.5 13.3\nMeyers, Jake 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.159 0.165 0.145 79.6 -36\nDiaz, Yainer 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.398 0.479 0.386 99.4 24.7\nSingleton, Jon 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.427 0.66 0.459 103.4 5.5\nAlvarez, Yordan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.532 1.405 0.815 94 28\nCaratini, Victor 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nHummel, Cooper 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.099 0.079 94.2 -39'}, {'pitcher_id': '656638', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLange, Alex 20 17 5 1 0 0 2 10 32.4 2 10 0.294 0.353 0.346 0.214 0.252 0.282 90.5 14.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 5 5 0 0 0 0 2 40 60 0 0 0 0 0 0.019 0.068 0.036 92.9 39.5\nAlvarez, Yordan 3 2 1 1 0 0 0 0 33.3 1 33.3 0.5 1 0.655 0.47 0.524 0.524 93.8 1\nMcCormick, Chas 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.157 0.181 0.145 81.8 5.3\nDiaz, Yainer 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.295 0.342 0.278 91.7 -8.5\nSingleton, Jon 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nBregman, Alex 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.201 0.228 0.186 89.8 43.3\nAltuve, Jose 1 0 0 0 0 0 0 0 0 0 0.72 0.72\nMeyers, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.597 0.644 0.541 101.2 7\nDubón, Mauricio 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.147 0.16 0.134 94.8 -15'}, {'pitcher_id': '687922', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucas, Easton 10 9 3 2 0 0 1 10 11.5 1 10 0.333 0.556 0.412 0.199 0.276 0.254 89.5 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 2 2 1 0 0 0 1 50 14.3 0 0 0.5 0.5 0.444 0.288 0.329 0.269 99.2 5\nBregman, Alex 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.248 0.493 0.315 86.7 14.5\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.335 0.381 0.312 91.1 -2\nSingleton, Jon 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nCaratini, Victor 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.27 0.256 0.285 0.236 101.6 -5\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.077 0.064 85.2 -35\nMeyers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.032 0.018 85.1 34\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.064 0.051 80.8 59'}, {'pitcher_id': '689225', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrieske, Beau 20 17 3 1 0 1 4 20 19 3 15 0.176 0.412 0.314 0.212 0.448 0.34 85.6 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 5 4 1 0 0 0 1 20 25 1 20 0.25 0.25 0.315 0.269 0.323 0.347 84.5 -3\nAlvarez, Yordan 4 4 0 0 0 0 2 50 18.2 0 0 0 0 0 0.021 0.033 0.024 82.2 48\nBregman, Alex 4 2 0 0 0 0 0 0 0 2 50 0 0 0.344 0.188 0.518 0.492 94.1 27\nPeña, Jeremy 4 4 1 1 0 0 0 0 10 0 0 0.25 0.5 0.315 0.217 0.237 0.203 77.9 13.5\nMeyers, Jake 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMcCormick, Chas 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.036 0.6 2.101 1.092 97.6 13.5'}, {'pitcher_id': '663947', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolton, Tyler 13 13 4 1 0 1 1 7.7 12 0 0 0.308 0.615 0.386 0.284 0.389 0.29 89.2 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDubón, Mauricio 2 2 1 0 0 1 0 0 16.7 0 0 0.5 2 1.002 0.201 0.538 0.301 93.5 -3.5\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.244 0.246 0.22 90.3 -4\nCaratini, Victor 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.006 0.01 0.006 84.6 42\nAlvarez, Yordan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.168 0.189 0.151 84.2 -4\nMeyers, Jake 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBregman, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 82.6 72.5\nSingleton, Jon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.516 0.564 0.475 81.1 7\nDiaz, Yainer 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.934 1.119 0.899 96.1 13\nMcCormick, Chas 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.44 0.798 0.521 99.5 18\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.485 0.409 101 1'}, {'pitcher_id': '681857', 'player_data': '', 'team_data': 'Reese Olson has not faced anyone on this team.'}, {'pitcher_id': '666214', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWentz, Joey 13 12 3 0 0 0 4 30.8 22.6 0 0 0.25 0.25 0.26 0.259 0.377 0.306 89.3 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0.36 0.36\nAltuve, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.154 0.458 0.253 85.8 14\nBregman, Alex 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.473 0.671 0.491 87.2 10.3\nMcCormick, Chas 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nSalazar, César 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCabbage, Trey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.16 0.183 0.145 84.4 -3\nMeyers, Jake 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.931 1.089 0.879 98.9 12\nDubón, Mauricio 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.286 0.32 0.26 97.9 -4'}, {'pitcher_id': '656412', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFaedo, Alex 22 19 5 2 0 0 2 9.1 21.4 3 13.6 0.263 0.368 0.328 0.201 0.313 0.283 85.5 27.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 3 2 0 0 0 0 0 0 11.1 1 33.3 0 0 0.232 0.01 0.018 0.24 87.3 44\nMcCormick, Chas 3 3 1 0 0 0 1 33.3 27.3 0 0 0.333 0.333 0.294 0.18 0.197 0.164 60.3 19\nBregman, Alex 3 2 1 1 0 0 0 0 40 1 33.3 0.5 1 0.647 0.217 0.508 0.42 96.1 26\nAltuve, Jose 4 3 0 0 0 0 1 25 40 1 25 0 0 0.174 0.052 0.052 0.209 86.5 28\nSingleton, Jon 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.026 0.024 81.8 48.3\nPeña, Jeremy 3 3 3 1 0 0 0 0 25 0 0 1 1.333 1.003 0.624 0.798 0.619 95.5 -1.7\nDubón, Mauricio 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.203 0.671 0.358 86.9 44\nMeyers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.308 0.33 0.281 88.1 0'}, {'pitcher_id': '628317', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaeda, Kenta 39 34 7 1 0 1 9 23.1 28.7 5 12.8 0.206 0.324 0.285 0.257 0.414 0.337 89.4 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 10 8 2 0 0 0 1 10 22.2 2 20 0.25 0.25 0.315 0.371 0.543 0.452 89.1 25.3\nAltuve, Jose 9 9 1 0 0 1 0 0 19 0 0 0.111 0.444 0.22 0.249 0.57 0.335 91.1 21.8\nMcCormick, Chas 5 5 2 0 0 0 2 40 40 0 0 0.4 0.4 0.352 0.39 0.486 0.384 79.9 -4.3\nAlvarez, Yordan 5 4 2 1 0 0 0 0 0 1 20 0.5 0.75 0.563 0.31 0.446 0.402 92.1 9\nPeña, Jeremy 2 1 0 0 0 0 1 50 28.6 1 50 0 0 0.348 0.348\nMeyers, Jake 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nDubón, Mauricio 3 3 0 0 0 0 3 100 62.5 0 0 0 0 0 0\nCaratini, Victor 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.06 0.069 0.056 83.5 -5\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.209 0.25 0.191 99.7 -9'}, {'pitcher_id': '656427', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlaherty, Jack 79 74 15 2 1 0 23 29.1 26.1 3 3.8 0.203 0.257 0.232 0.225 0.303 0.26 87.5 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 12 10 2 1 0 0 2 16.7 15.8 2 16.7 0.2 0.3 0.293 0.267 0.421 0.36 93.7 7.9\nCaratini, Victor 10 10 1 0 1 0 3 30 27.3 0 0 0.1 0.3 0.161 0.145 0.336 0.208 80.5 5.1\nAltuve, Jose 10 10 3 0 0 0 5 50 18.2 0 0 0.3 0.3 0.265 0.258 0.292 0.242 90.7 3.8\nDubón, Mauricio 10 9 5 1 0 0 0 0 29 0 0 0.556 0.667 0.554 0.425 0.49 0.433 79 25.9\nAlvarez, Yordan 9 8 1 0 0 0 2 22.2 21.4 0 0 0.125 0.125 0.179 0.129 0.145 0.188 88.9 8\nPeña, Jeremy 9 9 1 0 0 0 3 33.3 30.8 0 0 0.111 0.111 0.099 0.199 0.229 0.189 87.4 7.7\nSingleton, Jon 5 5 2 0 0 0 2 40 22.2 0 0 0.4 0.4 0.356 0.175 0.203 0.16 91.4 0.3\nMcCormick, Chas 4 4 0 0 0 0 4 100 55.6 0 0 0 0 0 0\nDiaz, Yainer 6 5 0 0 0 0 0 0 25 1 16.7 0 0 0.115 0.376 0.475 0.423 94.8 5.4\nMeyers, Jake 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.13 0.285 0.167 95.8 25\nCabbage, Trey 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.149 0.17 0.141 85.3 1'}, {'pitcher_id': '666159', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManning, Matt 28 26 3 0 0 0 3 10.7 16.3 1 3.6 0.115 0.115 0.124 0.221 0.382 0.272 89.9 18.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 6 5 1 0 0 0 0 0 0 1 16.7 0.2 0.2 0.263 0.223 0.339 0.318 91.6 27.4\nAlvarez, Yordan 6 6 0 0 0 0 1 16.7 0 0 0 0 0 0 0.336 0.844 0.487 91.5 26.2\nPeña, Jeremy 5 4 0 0 0 0 1 20 11.1 0 0 0 0 0 0.353 0.396 0.329 100.8 -1\nMcCormick, Chas 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.185 0.211 0.172 84.9 5\nDiaz, Yainer 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.085 0.145 0.091 89 50\nMeyers, Jake 2 2 1 0 0 0 0 0 42.9 0 0 0.5 0.5 0.441 0.162 0.176 0.149 78.5 -26.5\nAltuve, Jose 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.045 0.083 0.053 87.2 26.3\nDubón, Mauricio 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.137 0.102 84.4 19.5'}, {'pitcher_id': '671345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFoley, Jason 11 10 2 0 0 0 1 9.1 0 0 0 0.2 0.2 0.161 0.199 0.22 0.209 90.5 -6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.095 0.106 0.088 64.3 4\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.057 0.054 69.1 -21\nSingleton, Jon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.283 0.305 0.255 86.8 1\nCaratini, Victor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.096 0.071 84 -9\nPeña, Jeremy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.322 0.374 0.299 98.7 -7.5\nMeyers, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.323 0.349 0.294 103.8 -11\nMcCormick, Chas 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.232 0.21 101.9 -18\nBregman, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.187 0.198 0.17 100 -15\nDiaz, Yainer 1 0 0 0 0 0 0 0 0 0 0 0 0.468 97.9 17'}, {'pitcher_id': '669373', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSkubal, Tarik 72 65 17 5 1 3 15 20.8 28.8 4 5.6 0.262 0.508 0.354 0.237 0.4 0.307 88.3 14.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 13 11 4 2 0 1 3 23.1 25.9 1 7.7 0.364 0.818 0.531 0.223 0.518 0.372 90.5 33.1\nDiaz, Yainer 6 6 2 0 0 0 1 16.7 28.6 0 0 0.333 0.333 0.296 0.29 0.343 0.279 92.7 -6.6\nAlvarez, Yordan 12 10 2 0 0 0 3 25 6.7 1 8.3 0.2 0.2 0.205 0.183 0.21 0.219 91.8 -1.5\nPeña, Jeremy 8 7 3 1 0 1 1 12.5 35.7 0 0 0.429 1 0.619 0.307 0.779 0.484 94.4 13.8\nBregman, Alex 9 9 3 1 1 0 2 22.2 14.3 0 0 0.333 0.667 0.418 0.248 0.338 0.255 85 37.6\nMcCormick, Chas 6 6 0 0 0 0 2 33.3 53.8 0 0 0 0 0 0.235 0.271 0.22 82.8 13.5\nMeyers, Jake 6 6 1 1 0 0 0 0 33.3 0 0 0.167 0.333 0.212 0.23 0.345 0.25 84 0.7\nDubón, Mauricio 6 5 1 0 0 1 1 16.7 37.5 1 16.7 0.2 0.8 0.463 0.263 0.584 0.406 85.7 18.3\nSingleton, Jon 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nCaratini, Victor 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.308 0.348 0.287 81.8 12'}, {'pitcher_id': '571946', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Shelby 16 12 5 0 0 2 0 0 13 2 12.5 0.417 0.917 0.586 0.402 1.113 0.647 85.8 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 6 3 3 0 0 1 0 0 14.3 2 33.3 1 2 0.963 0.609 1.645 0.808 88.4 14.3\nAltuve, Jose 7 7 1 0 0 0 0 0 0 0 0 0.143 0.143 0.124 0.148 0.158 0.134 77.8 36.7\nAlvarez, Yordan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.07 0.084 0.063 84.4 -14\nPeña, Jeremy 1 0 0 0 0 0 0 0 50 0 0 0.724 0.724\nCaratini, Victor 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.877 3.415 1.732 103.6 29'}, {'pitcher_id': '605177', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChafin, Andrew 13 10 4 0 0 0 3 23.1 30 3 23.1 0.4 0.4 0.431 0.289 0.44 0.399 88.3 -1.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 5 2 1 0 0 0 1 20 66.7 3 60 0.5 0.5 0.592 0.125 0.137 0.461 92.7 -3\nCaratini, Victor 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.293 0.459 0.594 0.453 93.8 16\nAltuve, Jose 2 2 2 0 0 0 0 0 0 0 0 1 1 0.885 0.376 0.521 0.374 95 8\nVerlander, Justin 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBregman, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.349 1.145 0.621 97.7 30\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.16 0.16 50 -83'}], 118: [{'pitcher_id': '669438', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEnglert, Mason 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.507 0.417 97.6 1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.507 0.417 97.6 1'}, {'pitcher_id': '663554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMize, Casey 54 53 24 4 2 2 6 11.1 21.4 0 0 0.453 0.717 0.504 0.37 0.606 0.421 89.7 13.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 24 23 11 2 0 1 2 8.3 24.5 0 0 0.478 0.696 0.513 0.378 0.592 0.425 88.5 22.5\nWitt Jr., Bobby 8 8 4 0 1 1 1 12.5 10 0 0 0.5 1.125 0.683 0.411 0.844 0.524 98.6 17.9\nFrazier, Adam 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.178 0.256 0.291 0.245 87.7 -4.3\nPasquantino, Vinnie 5 5 1 1 0 0 1 20 27.3 0 0 0.2 0.4 0.254 0.365 0.937 0.542 90.4 18.5\nGarcia, Maikel 5 5 4 0 1 0 0 0 22.2 0 0 0.8 1.2 0.856 0.518 0.692 0.526 97.1 6.8\nBlanco, Dairon 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.136 0.149 0.123 82.2 -18\nIsbel, Kyle 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.533 0.409 0.458 0.38 76.2 -4\nMelendez, MJ 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '676684', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVest, Will 36 32 8 0 1 1 9 25 23.2 2 5.6 0.25 0.406 0.288 0.218 0.33 0.271 88.3 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 6 6 1 0 1 0 0 0 0 0 0 0.167 0.5 0.269 0.224 0.369 0.254 86.1 29\nMelendez, MJ 4 4 1 0 0 1 2 50 40 0 0 0.25 1 0.518 0.264 0.744 0.424 97.9 10\nPasquantino, Vinnie 4 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.221 0.461 0.54 0.425 91 8.3\nPerez, Salvador 4 4 0 0 0 0 1 25 37.5 0 0 0 0 0 0.073 0.08 0.066 87 24.7\nIsbel, Kyle 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.525 0.364 0.486 0.468 99.1 14.5\nFrazier, Adam 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.305 0.363 0.294 77.5 33\nGarcia, Maikel 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.825 0.394 0.453 0.472 97.2 -2.5\nVelázquez, Nelson 1 1 0 0 0 0 1 100 20 0 0 0 0 0 0\nBlanco, Dairon 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nWaters, Drew 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.076 0.08 0.073 71.8 -10.5\nFermin, Freddy 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.294 0.159 0.166 0.143 101.7 -2'}, {'pitcher_id': '656638', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLange, Alex 48 39 12 2 1 0 11 22.9 32.5 5 10.4 0.308 0.41 0.369 0.266 0.37 0.345 89.3 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 9 7 3 1 0 0 3 33.3 33.3 1 11.1 0.429 0.571 0.414 0.204 0.25 0.264 87.3 10.4\nPasquantino, Vinnie 7 7 1 1 0 0 3 42.9 43.8 0 0 0.143 0.286 0.178 0.192 0.279 0.2 98.9 14.3\nGarcia, Maikel 5 3 1 0 0 0 0 0 12.5 1 20 0.333 0.333 0.461 0.389 0.6 0.535 97 14\nPerez, Salvador 7 5 1 0 0 0 1 14.3 27.3 0 0 0.2 0.2 0.333 0.356 0.386 0.44 87.4 14\nFrazier, Adam 4 3 3 0 1 0 0 0 0 1 25 1 1.667 1.015 0.725 0.927 0.712 87.3 19.3\nMelendez, MJ 4 3 0 0 0 0 1 25 50 1 25 0 0 0.172 0.265 0.714 0.473 86.1 -9.5\nIsbel, Kyle 5 5 1 0 0 0 2 40 42.9 0 0 0.2 0.2 0.177 0.029 0.032 0.028 73.7 40.7\nWaters, Drew 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.311 0.418 0.305 85.1 12.5\nVelázquez, Nelson 1 0 0 0 0 0 0 0 25 1 100 0.692 0.692\nLoftin, Nick 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.005 0.003 88.5 48\nBlanco, Dairon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.309 0.333 0.278 101.4 -8\nFermin, Freddy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.296 0.307 0.268 103.5 -12'}, {'pitcher_id': '687922', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '689225', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrieske, Beau 34 32 7 3 1 0 5 14.7 18 2 5.9 0.219 0.375 0.276 0.283 0.442 0.334 93.4 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPasquantino, Vinnie 7 6 1 0 0 0 1 14.3 0 1 14.3 0.167 0.167 0.225 0.258 0.481 0.371 99.4 5.2\nWitt Jr., Bobby 8 8 0 0 0 0 1 12.5 14.3 0 0 0 0 0 0.25 0.37 0.265 92.7 26.6\nMelendez, MJ 7 7 1 0 1 0 2 28.6 38.5 0 0 0.143 0.429 0.229 0.181 0.407 0.245 90.8 25.4\nLoftin, Nick 1 1 1 0 0 0 0 0 14.3 0 0 1 1 0.883 0.843 1.2 0.869 107.1 13\nFermin, Freddy 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.173 0.175 0.426 91 -8\nIsbel, Kyle 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.715 0.479 0.622 0.484 98.9 18.3\nGarcia, Maikel 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.622 0.697 0.823 0.666 98.4 6.5\nVelázquez, Nelson 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nWaters, Drew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.126 0.188 0.132 73.1 12\nBlanco, Dairon 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.156 0.185 0.147 83.6 -8'}, {'pitcher_id': '663947', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolton, Tyler 30 25 7 0 1 0 5 16.7 21.8 2 6.7 0.28 0.36 0.325 0.289 0.386 0.335 92.3 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIsbel, Kyle 4 3 1 0 0 0 2 50 33.3 0 0 0.333 0.333 0.403 0.123 0.13 0.264 104.9 -6\nHampson, Garrett 3 2 1 0 0 0 1 33.3 28.6 1 33.3 0.5 0.5 0.524 0.467 0.639 0.548 107.1 11\nFrazier, Adam 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0.241 0.433 0.581 0.53 97.8 5\nPerez, Salvador 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.374 0.529 0.383 89.1 6.7\nMelendez, MJ 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.131 0.152 0.124 89.7 42\nPasquantino, Vinnie 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.39 0.654 0.432 89.2 27.5\nFermin, Freddy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.289 0.326 0.27 72.8 10\nGarcia, Maikel 3 3 2 0 1 0 0 0 16.7 0 0 0.667 1.333 0.832 0.572 0.759 0.569 98.7 15\nWaters, Drew 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.052 0.075 0.052 78.5 38.5\nVelázquez, Nelson 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nWitt Jr., Bobby 2 1 1 0 0 0 0 0 0 0 0 1 1 0.441 0.469 0.503 0.219 100.2 20\nLoftin, Nick 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '681857', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Reese 68 60 14 1 1 2 20 29.4 37.4 7 10.3 0.233 0.383 0.302 0.259 0.437 0.334 90.3 6.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 10 8 0 0 0 0 6 60 50 2 20 0 0 0.139 0.035 0.037 0.165 91.4 -22\nWitt Jr., Bobby 11 9 5 1 1 1 2 18.2 21.4 1 9.1 0.556 1.222 0.664 0.44 0.892 0.531 97.5 12.3\nGarcia, Maikel 11 10 3 0 0 1 2 18.2 26.7 1 9.1 0.3 0.6 0.406 0.336 0.651 0.439 94.5 9\nPerez, Salvador 11 11 2 0 0 0 2 18.2 40.9 0 0 0.182 0.182 0.161 0.267 0.353 0.27 89.5 7.4\nIsbel, Kyle 6 4 1 0 0 0 1 16.7 27.3 2 33.3 0.25 0.25 0.379 0.408 0.877 0.586 98.4 22\nPasquantino, Vinnie 5 4 0 0 0 0 1 20 11.1 1 20 0 0 0.138 0.211 0.287 0.309 89.4 30\nVelázquez, Nelson 6 6 0 0 0 0 3 50 66.7 0 0 0 0 0 0.062 0.065 0.056 57.7 -40.7\nFrazier, Adam 3 3 1 0 0 0 2 66.7 40 0 0 0.333 0.333 0.296 0.278 0.296 0.254 69.5 33\nWaters, Drew 4 4 1 0 0 0 1 25 50 0 0 0.25 0.25 0.221 0.256 0.326 0.249 90 7.7\nFermin, Freddy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.271 0.282 0.244 102.9 -11'}, {'pitcher_id': '666214', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWentz, Joey 39 36 7 2 0 1 5 12.8 17.6 3 7.7 0.194 0.333 0.262 0.32 0.493 0.376 85.2 8.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 7 7 3 1 0 1 1 14.3 21.7 0 0 0.429 1 0.604 0.443 0.954 0.586 90.7 7.8\nMelendez, MJ 8 8 1 0 0 0 2 25 18.2 0 0 0.125 0.125 0.111 0.289 0.322 0.267 85.5 14.7\nPasquantino, Vinnie 7 5 1 1 0 0 0 0 12.5 2 28.6 0.2 0.4 0.377 0.227 0.329 0.364 85.7 6\nWitt Jr., Bobby 7 7 0 0 0 0 0 0 10 0 0 0 0 0 0.422 0.657 0.467 86.6 5.3\nFrazier, Adam 4 3 0 0 0 0 0 0 0 1 25 0 0 0.174 0.137 0.137 0.268 68.4 -8\nWaters, Drew 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.011 0.015 0.011 79.5 40\nIsbel, Kyle 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.083 0.102 0.078 76.7 33\nFermin, Freddy 1 1 1 0 0 0 0 0 25 0 0 1 1 0.889 0.734 0.846 0.704 88.3 12\nGarcia, Maikel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.68 0.768 0.641 98.4 8'}, {'pitcher_id': '656412', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFaedo, Alex 11 8 3 1 0 0 0 0 6.3 3 27.3 0.375 0.5 0.463 0.45 0.545 0.503 96.9 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIsbel, Kyle 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.392 0.525 0.391 93.3 39.5\nPasquantino, Vinnie 2 0 0 0 0 0 0 0 2 100 0.689 0.689\nWitt Jr., Bobby 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.715 0.508 0.617 0.491 100.1 7.3\nMelendez, MJ 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.791 0.927 0.722 95 11\nBlanco, Dairon 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.224 0.251 0.208 90.2 -8\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.273 0.28 0.246 102.8 -12'}, {'pitcher_id': '628317', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaeda, Kenta 52 50 9 1 0 3 17 32.7 29.1 2 3.8 0.18 0.38 0.249 0.203 0.378 0.261 88.3 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 6 6 1 0 0 1 3 50 20 0 0 0.167 0.667 0.334 0.229 0.736 0.391 95.5 16.7\nPerez, Salvador 9 8 1 0 0 0 5 55.6 55 1 11.1 0.125 0.125 0.175 0.082 0.088 0.144 89 55\nHampson, Garrett 8 8 2 0 0 0 2 25 27.3 0 0 0.25 0.25 0.217 0.282 0.354 0.274 81 37.5\nGarcia, Maikel 6 6 1 1 0 0 2 33.3 20 0 0 0.167 0.333 0.207 0.207 0.309 0.221 89.2 18.5\nMelendez, MJ 6 6 1 0 0 0 3 50 50 0 0 0.167 0.167 0.147 0.153 0.168 0.139 99.8 17.7\nBlanco, Dairon 4 4 1 0 0 0 0 0 11.1 0 0 0.25 0.25 0.221 0.096 0.101 0.089 84.8 2.2\nFermin, Freddy 2 2 1 0 0 1 1 50 0 0 0 0.5 2 1.002 0.395 1.45 0.736 102.6 27\nFrazier, Adam 6 6 1 0 0 1 0 0 14.3 0 0 0.167 0.667 0.323 0.284 0.646 0.382 84.8 4.2\nIsbel, Kyle 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.046 0.055 0.045 83.7 8\nWaters, Drew 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.669 0.729 0.652 100.7 8'}, {'pitcher_id': '656427', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlaherty, Jack 43 39 9 2 0 0 14 32.6 25.6 3 7 0.231 0.282 0.254 0.213 0.299 0.254 86.6 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 27 24 5 1 0 0 6 22.2 20.8 2 7.4 0.208 0.25 0.237 0.194 0.247 0.23 84.3 7.3\nNola, Austin 6 6 2 1 0 0 3 50 11.1 0 0 0.333 0.5 0.348 0.293 0.579 0.362 102.7 8.3\nPerez, Salvador 5 4 1 0 0 0 1 20 50 1 20 0.25 0.25 0.314 0.4 0.469 0.444 90 23.3\nHampson, Garrett 5 5 1 0 0 0 4 80 36.4 0 0 0.2 0.2 0.176 0.064 0.072 0.058 72.5 8'}, {'pitcher_id': '666159', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManning, Matt 40 36 9 1 0 0 4 10 12.2 4 10 0.25 0.278 0.277 0.312 0.428 0.355 90 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 8 7 1 0 0 0 0 0 11.1 1 12.5 0.143 0.143 0.196 0.347 0.383 0.366 92.7 22\nWitt Jr., Bobby 10 9 3 1 0 0 0 0 10.5 1 10 0.333 0.444 0.372 0.268 0.382 0.32 83.5 17.2\nPerez, Salvador 7 7 3 0 0 0 1 14.3 10 0 0 0.429 0.429 0.379 0.405 0.445 0.374 96.8 4\nIsbel, Kyle 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.173 0.281 0.192 93.9 5\nWaters, Drew 4 3 0 0 0 0 1 25 16.7 1 25 0 0 0.172 0.294 0.462 0.412 85.1 21.5\nFermin, Freddy 2 2 2 0 0 0 0 0 16.7 0 0 1 1 0.883 0.516 0.57 0.467 91.8 1\nPasquantino, Vinnie 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.327 1.03 0.606 89.5 8\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.18 0.106 84.9 -15'}, {'pitcher_id': '671345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFoley, Jason 48 43 10 0 0 2 4 8.3 14.7 4 8.3 0.233 0.372 0.307 0.325 0.573 0.415 89.8 5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 8 7 2 0 0 0 0 0 18.8 1 12.5 0.286 0.286 0.307 0.326 0.417 0.367 91.8 -4.1\nWitt Jr., Bobby 11 11 2 0 0 1 1 9.1 0 0 0 0.182 0.455 0.27 0.314 0.627 0.398 86.1 16.4\nGarcia, Maikel 5 3 1 0 0 0 0 0 0 2 40 0.333 0.333 0.455 0.302 0.347 0.453 100.9 -18.3\nMelendez, MJ 5 5 1 0 0 0 2 40 50 0 0 0.2 0.2 0.177 0.15 0.169 0.138 92.2 12.3\nIsbel, Kyle 5 5 1 0 0 1 0 0 0 0 0 0.2 0.8 0.414 0.355 0.727 0.453 95.8 20.8\nPasquantino, Vinnie 7 6 2 0 0 0 0 0 0 0 0 0.333 0.333 0.355 0.626 1.343 0.814 93 15\nFrazier, Adam 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.11 0.18 0.124 83.1 -7\nBlanco, Dairon 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.146 0.148 0.132 48.5 -63\nWaters, Drew 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nFermin, Freddy 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.272 0.298 0.249 84.3 -8.5'}, {'pitcher_id': '669373', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSkubal, Tarik 113 108 27 7 2 4 32 28.3 30.1 4 3.5 0.25 0.463 0.311 0.247 0.463 0.314 89.9 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 24 24 8 2 0 3 9 37.5 39.6 0 0 0.333 0.792 0.464 0.283 0.756 0.427 94.5 16.7\nWitt Jr., Bobby 19 19 3 0 0 0 3 15.8 17.1 0 0 0.158 0.158 0.14 0.259 0.32 0.253 84.8 11.9\nGarcia, Maikel 11 10 2 1 0 0 5 45.5 27.3 0 0 0.2 0.3 0.196 0.196 0.28 0.219 91.2 8.8\nBlanco, Dairon 8 8 3 0 1 0 3 37.5 25 0 0 0.375 0.625 0.418 0.282 0.384 0.284 85.1 -11\nHampson, Garrett 7 7 3 2 0 0 1 14.3 20 0 0 0.429 0.714 0.486 0.311 0.523 0.353 92.8 15.7\nVelázquez, Nelson 7 6 1 0 0 1 3 42.9 46.2 1 14.3 0.167 0.667 0.397 0.172 0.674 0.389 91.2 50.7\nFermin, Freddy 7 7 2 2 0 0 3 42.9 52.6 0 0 0.286 0.571 0.359 0.286 0.823 0.456 98.3 34.3\nPasquantino, Vinnie 8 6 1 0 0 0 0 0 25 2 25 0.167 0.167 0.284 0.265 0.297 0.359 96.8 3.3\nIsbel, Kyle 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.177 0.139 0.16 0.131 74.3 -18\nMelendez, MJ 6 6 1 0 0 0 2 33.3 35.7 0 0 0.167 0.167 0.147 0.181 0.206 0.171 95.9 1.2\nFrazier, Adam 4 4 1 0 0 0 0 0 20 0 0 0.25 0.25 0.22 0.347 0.414 0.339 91.1 -1.8\nNola, Austin 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.014 0.019 0.239 75.2 57\nWaters, Drew 3 3 1 0 1 0 1 33.3 14.3 0 0 0.333 1 0.523 0.266 0.318 0.252 87.1 40.5\nLoftin, Nick 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '571946', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Shelby 9 9 2 0 0 0 2 22.2 26.7 0 0 0.222 0.222 0.198 0.167 0.292 0.193 87.5 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 3 3 0 0 0 0 1 33.3 42.9 0 0 0 0 0 0.237 0.477 0.294 95 12.5\nPerez, Salvador 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.594\nNola, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.292 0.319 0.274 101.5 -12\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 58.4 69'}, {'pitcher_id': '605177', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChafin, Andrew 35 31 5 0 0 0 11 31.4 30.9 4 11.4 0.161 0.161 0.205 0.209 0.354 0.29 83.1 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 7 5 1 0 0 0 1 14.3 12.5 2 28.6 0.2 0.2 0.322 0.267 0.466 0.415 84.3 20\nMelendez, MJ 6 5 0 0 0 0 3 50 50 1 16.7 0 0 0.116 0.093 0.109 0.189 89.7 31.5\nPerez, Salvador 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.23 0.263 0.836 0.523 98.7 30\nPasquantino, Vinnie 4 4 2 0 0 0 1 25 16.7 0 0 0.5 0.5 0.442 0.436 0.877 0.554 87.4 17.7\nIsbel, Kyle 6 6 2 0 0 0 2 33.3 40 0 0 0.333 0.333 0.294 0.205 0.255 0.201 84.9 1.3\nWitt Jr., Bobby 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.135 0.135 0.121 68.1 11\nHampson, Garrett 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.083 0.151 0.092 86 27\nNola, Austin 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nFermin, Freddy 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVelázquez, Nelson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.119 0.1 66.2 -4\nWaters, Drew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.148 0.182 0.139 63.4 -17\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.42 0.488 0.398 97.8 -1'}], 119: [{'pitcher_id': '669438', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEnglert, Mason 6 6 1 0 0 0 4 66.7 50 0 0 0.167 0.167 0.147 0.013 0.024 0.015 81.4 32', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 3 3 0 0 0 0 3 100 44.4 0 0 0 0 0 0\nHernández, Teoscar 3 3 1 0 0 0 1 33.3 60 0 0 0.333 0.333 0.294 0.032 0.059 0.037 81.4 32'}, {'pitcher_id': '663554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMize, Casey 16 12 4 0 0 3 3 18.8 34.3 4 25 0.333 1.083 0.609 0.317 0.796 0.52 93.7 8.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Enrique 6 5 1 0 0 1 1 16.7 46.2 1 16.7 0.2 0.8 0.45 0.377 1.01 0.595 92.8 13.3\nOhtani, Shohei 6 5 2 0 0 1 2 33.3 33.3 1 16.7 0.4 1 0.596 0.262 0.549 0.396 97.2 4.7\nBiggio, Cavan 2 1 1 0 0 1 0 0 20 1 50 1 4 1.389 0.551 1.693 0.804 99.7 27\nHeyward, Jason 2 1 0 0 0 0 0 0 0 1 50 0 0 0.349 0.057 0.064 0.383 81.4 -20'}, {'pitcher_id': '676684', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVest, Will 18 17 4 2 0 0 5 27.8 25.9 1 5.6 0.235 0.353 0.274 0.332 0.45 0.359 94.7 22.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Chris 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nBiggio, Cavan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.291 0.312 0.265 97.8 18.7\nOhtani, Shohei 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSmith, Will 2 2 2 1 0 0 0 0 25 0 0 1 1.5 1.063 0.923 1.132 0.901 95.5 13.5\nHernández, Teoscar 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.379 0.942 0.546 105.7 20\nLux, Gavin 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.068 0.073 0.07 90.7 -15\nBetts, Mookie 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.42 0.537 0.418 93.7 43\nFreeman, Freddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.651 0.733 0.604 99.2 7\nHeyward, Jason 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.54 0.614 0.503 94.8 6\nHernández, Enrique 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 74.2 82'}, {'pitcher_id': '656638', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLange, Alex 15 12 2 1 0 0 4 26.7 43.5 2 13.3 0.167 0.25 0.284 0.201 0.342 0.332 94.3 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 3 3 1 1 0 0 2 66.7 42.9 0 0 0.333 0.667 0.42 0.228 0.656 0.372 102.4 24\nHeyward, Jason 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.271 0.303 0.476 94.8 -5\nHernández, Enrique 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.172 0.197 0.334 97.1 21.5\nBetts, Mookie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.529 0.673 0.498 94.9 17\nHernández, Teoscar 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFreeman, Freddie 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.193 0.214 0.175 86.8 12\nOhtani, Shohei 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSmith, Will 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '687922', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucas, Easton 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.244 0.569 0.634 0.517 104.5 4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.244 0.569 0.634 0.517 104.5 4'}, {'pitcher_id': '689225', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrieske, Beau 25 24 5 1 0 1 3 12 16.7 1 4 0.208 0.375 0.267 0.293 0.535 0.365 90.7 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 3 3 1 0 0 0 0 0 30 0 0 0.333 0.333 0.295 0.574 0.709 0.561 96.6 4.7\nFreeman, Freddie 4 3 0 0 0 0 0 0 0 1 25 0 0 0.172 0.054 0.089 0.217 85.5 17\nBetts, Mookie 4 4 3 0 0 1 0 0 0 0 0 0.75 1.5 0.96 0.549 0.988 0.654 96.9 19.5\nTaylor, Chris 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.631 0.452 1.573 0.844 107.4 23\nLux, Gavin 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.04 0.042 0.039 73.4 28\nBiggio, Cavan 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.03 0.046 0.028 77.1 15\nBarnes, Austin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.108 0.208 0.134 89.3 48.5\nRojas, Miguel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.391 0.446 0.359 90.6 1\nOhtani, Shohei 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSmith, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.46 0.507 0.425 105.2 -1\nHeyward, Jason 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.409 1.323 0.705 97.9 29'}, {'pitcher_id': '663947', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolton, Tyler 11 11 1 0 0 0 4 36.4 30 0 0 0.091 0.091 0.08 0.219 0.271 0.212 84 -0.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBetts, Mookie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.334 0.363 0.3 77.2 10.5\nHeyward, Jason 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.066 0.066 0.062 70.3 -27.5\nBiggio, Cavan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.19 0.385 0.24 100.4 20\nRojas, Miguel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHernández, Teoscar 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSmith, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.534 0.598 0.495 104.1 2\nTaylor, Chris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFreeman, Freddie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.697 0.752 0.629 88.6 11'}, {'pitcher_id': '681857', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Reese 15 15 0 0 0 0 5 33.3 25.8 0 0 0 0 0 0.094 0.105 0.087 77.2 -10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Miguel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.193 0.213 0.175 71.5 -10.5\nFreeman, Freddie 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.064 0.078 0.06 85.3 -14\nBetts, Mookie 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.091 0.102 0.085 90.6 28.5\nHernández, Teoscar 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.064 0.074 0.061 74.3 -1.5\nBarnes, Austin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.051 0.061 0.046 82.6 -44\nHeyward, Jason 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.132 0.132 0.121 46 -62'}, {'pitcher_id': '666214', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWentz, Joey 4 2 1 0 0 0 0 0 36.4 1 25 0.5 0.5 0.396 0.107 0.125 0.314 80.4 31.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 2 1 0 0 0 0 0 0 37.5 0 0 0 0 0 0.006 0.009 0.185 90.6 41.5\nHernández, Enrique 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nBiggio, Cavan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.209 0.241 0.191 60.1 12'}, {'pitcher_id': '656412', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFaedo, Alex 2 2 2 0 0 1 0 0 50 0 0 1 2.5 1.443 0.438 1.54 0.799 85.8 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 2 2 2 0 0 1 0 0 50 0 0 1 2.5 1.443 0.438 1.54 0.799 85.8 14.5'}, {'pitcher_id': '628317', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaeda, Kenta 49 45 6 2 1 1 17 34.7 34.8 3 6.1 0.133 0.289 0.205 0.191 0.329 0.248 86.1 22.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 9 6 0 0 0 0 4 44.4 35.7 3 33.3 0 0 0.173 0.119 0.24 0.283 83.2 34\nHeyward, Jason 11 10 2 0 1 0 1 9.1 21.1 0 0 0.2 0.4 0.288 0.28 0.406 0.333 88.4 7.3\nOhtani, Shohei 8 8 2 0 0 1 4 50 43.8 0 0 0.25 0.625 0.351 0.209 0.479 0.276 84.6 17.3\nBiggio, Cavan 5 5 0 0 0 0 2 40 28.6 0 0 0 0 0 0.046 0.056 0.047 67.8 28.7\nBetts, Mookie 5 5 2 2 0 0 1 20 11.1 0 0 0.4 0.8 0.493 0.275 0.562 0.35 91.4 41.8\nHernández, Teoscar 5 5 0 0 0 0 3 60 50 0 0 0 0 0 0.167 0.21 0.163 101.4 30.5\nRojas, Miguel 5 5 0 0 0 0 2 40 50 0 0 0 0 0 0.191 0.258 0.192 84.9 24.7\nHernández, Enrique 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.017 0.031 0.02 84.5 51'}, {'pitcher_id': '656427', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlaherty, Jack 91 76 19 2 0 5 21 23.1 27.2 11 12.1 0.25 0.474 0.365 0.247 0.434 0.352 87.5 10.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeyward, Jason 27 21 5 2 0 1 4 14.8 21 5 18.5 0.238 0.476 0.371 0.202 0.325 0.313 82.1 12.4\nFreeman, Freddie 20 19 7 0 0 1 3 15 23.1 1 5 0.368 0.526 0.4 0.386 0.586 0.433 88.6 5.6\nRojas, Miguel 8 8 1 0 0 0 1 12.5 5.6 0 0 0.125 0.125 0.11 0.24 0.262 0.221 78.3 -3.6\nSmith, Will 8 6 1 0 0 0 2 25 40 2 25 0.167 0.167 0.284 0.081 0.088 0.229 86.7 21.8\nBetts, Mookie 5 3 1 0 0 1 0 0 28.6 1 20 0.333 1.333 0.684 0.361 1.277 0.68 93.8 18.7\nTaylor, Chris 4 4 1 0 0 1 2 50 50 0 0 0.25 1 0.502 0.394 1.128 0.622 102.4 17.5\nBuehler, Walker 4 4 0 0 0 0 4 100 60 0 0 0 0 0 0\nBarnes, Austin 4 3 0 0 0 0 2 50 28.6 0 0 0 0 0.18 0.009 0.018 0.189 88.8 60\nHernández, Enrique 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.44 0.106 0.109 0.096 103.8 -24\nOhtani, Shohei 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.654 0.784 0.659 113 4\nHernández, Teoscar 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.277 0.53 0.332 102.5 26\nLux, Gavin 2 2 1 0 0 1 1 50 33.3 0 0 0.5 2 1.003 0.18 0.712 0.363 105 41\nBiggio, Cavan 2 0 0 0 0 0 0 0 1 50 0.711 0.711'}, {'pitcher_id': '666159', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManning, Matt 23 18 3 0 0 2 5 21.7 30.8 5 21.7 0.167 0.5 0.363 0.228 0.638 0.428 94.3 27', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 12 8 3 0 0 2 2 16.7 35 4 33.3 0.375 1.125 0.638 0.411 1.178 0.662 97.9 17.5\nBiggio, Cavan 5 4 0 0 0 0 1 20 0 1 20 0 0 0.139 0.077 0.117 0.206 92.8 11.7\nHernández, Teoscar 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.017 0.051 0.026 95.3 38\nHernández, Enrique 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.154 0.48 0.263 88.4 57.7'}, {'pitcher_id': '671345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFoley, Jason 10 9 3 1 0 0 4 40 18.2 1 10 0.333 0.444 0.372 0.246 0.303 0.283 92.1 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 4 4 1 0 0 0 3 75 44.4 0 0 0.25 0.25 0.221 0.073 0.09 0.068 100.8 -8\nBiggio, Cavan 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nHernández, Enrique 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.631 0.466 0.571 0.459 93 13\nFreeman, Freddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.203 0.224 0.185 99.9 -16\nBetts, Mookie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.717 0.884 0.695 90.3 18\nSmith, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.073 0.117 0.068 76.5 -17'}, {'pitcher_id': '669373', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSkubal, Tarik 14 13 4 3 0 0 3 21.4 31.6 1 7.1 0.308 0.538 0.381 0.289 0.548 0.38 95.7 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Enrique 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.231 0.196 0.776 0.494 102 37\nHernández, Teoscar 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.073 0.435 0.479 0.404 97.5 -0.5\nOhtani, Shohei 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.157 0.269 0.174 94 4.5\nBiggio, Cavan 3 3 1 1 0 0 1 33.3 33.3 0 0 0.333 0.667 0.42 0.238 0.704 0.395 88.3 21\nHeyward, Jason 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.414 0.436 0.564 0.44 98.5 1.7'}, {'pitcher_id': '571946', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Shelby 45 39 10 1 0 3 4 8.9 10.8 6 13.3 0.256 0.513 0.377 0.238 0.453 0.332 89.7 10.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeyward, Jason 17 15 3 0 0 1 3 17.6 26.1 2 11.8 0.2 0.4 0.309 0.082 0.106 0.085 79.2 25.8\nFreeman, Freddie 16 13 5 1 0 2 1 6.3 4 3 18.8 0.385 0.923 0.578 0.388 1.392 0.708 100.6 20\nRojas, Miguel 6 5 1 0 0 0 0 0 0 1 16.7 0.2 0.2 0.261 0.236 0.264 0.34 93.8 -22.3\nBetts, Mookie 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.176 0.337 0.474 0.353 89.1 6\nHernández, Teoscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.746 0.911 0.716 109.3 8'}, {'pitcher_id': '605177', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChafin, Andrew 50 40 9 2 0 2 8 16 32.2 7 14 0.225 0.425 0.344 0.281 0.458 0.38 89.7 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 13 9 1 1 0 0 1 7.7 36.4 3 23.1 0.111 0.222 0.311 0.159 0.233 0.333 87.9 11.1\nHernández, Enrique 9 8 3 1 0 0 2 22.2 28.6 0 0 0.375 0.5 0.33 0.468 0.858 0.504 92.2 24\nBarnes, Austin 5 3 1 0 0 0 1 20 11.1 1 20 0.333 0.333 0.457 0.425 0.507 0.527 101.5 4.5\nOhtani, Shohei 5 4 2 0 0 1 1 20 50 1 20 0.5 1.25 0.729 0.393 0.762 0.529 101.6 16\nSmith, Will 4 4 1 0 0 0 0 0 37.5 0 0 0.25 0.25 0.217 0.306 0.317 0.277 80.4 22.8\nHeyward, Jason 5 5 0 0 0 0 1 20 37.5 0 0 0 0 0 0.065 0.069 0.062 84.5 28.7\nTaylor, Chris 4 4 1 0 0 1 1 25 50 0 0 0.25 1 0.501 0.374 0.725 0.465 96.4 13.7\nBetts, Mookie 3 2 0 0 0 0 1 33.3 14.3 1 33.3 0 0 0.231 0.05 0.111 0.272 92.2 28\nRojas, Miguel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.069 0.069 0.383 63.5 -2'}]}, 117: {115: [{'pitcher_id': '664299', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJavier, Cristian 11 11 3 2 0 1 2 18.2 14.3 0 0 0.273 0.727 0.406 0.197 0.464 0.273 90.8 34.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 3 3 2 2 0 0 1 33.3 14.3 0 0 0.667 1.333 0.829 0.301 0.432 0.316 88.4 26.5\nMcMahon, Ryan 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.66 0.38 1.168 0.625 98.2 27\nBlackmon, Charlie 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.026 0.063 0.038 83.5 49.5\nStallings, Jacob 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.024 0.055 0.032 89.7 40.5'}, {'pitcher_id': '668874', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661527', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Seth 5 4 1 0 0 0 3 60 46.2 1 20 0.25 0.25 0.315 0.216 0.272 0.309 103.2 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTovar, Ezequiel 2 2 0 0 0 0 2 100 44.4 0 0 0 0 0 0\nGoodman, Hunter 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nJones, Nolan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCave, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.866 1.086 0.854 103.2 10'}, {'pitcher_id': '606160', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontero, Rafael 13 12 4 2 0 1 1 7.7 17.9 1 7.7 0.333 0.75 0.463 0.407 0.731 0.497 92 13.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 6 6 3 1 0 1 1 16.7 20 0 0 0.5 1.167 0.681 0.54 1.021 0.646 90.1 23\nMcMahon, Ryan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.619 0.52 1.18 0.701 102.3 14.5\nToglia, Michael 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nTovar, Ezequiel 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.129 0.158 0.118 88 12.5\nCave, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.68 0.768 0.64 101.4 7\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.066 0.074 0.059 76.1 -9'}, {'pitcher_id': '666215', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681293', 'player_data': '', 'team_data': 'Spencer Arrighetti has not faced anyone on this team.'}, {'pitcher_id': '434378', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerlander, Justin 37 35 8 3 0 1 15 40.5 38.7 2 5.4 0.229 0.4 0.288 0.203 0.343 0.262 89.7 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 16 15 4 1 0 1 4 25 24.1 1 6.3 0.267 0.533 0.356 0.206 0.634 0.377 92.6 18.6\nCave, Jake 7 7 0 0 0 0 7 100 64.7 0 0 0 0 0 0\nMcMahon, Ryan 5 5 1 0 0 0 3 60 53.8 0 0 0.2 0.2 0.177 0.124 0.139 0.112 95.3 -2.5\nJones, Nolan 5 5 2 1 0 0 1 20 40 0 0 0.4 0.6 0.425 0.418 0.465 0.386 86 20.5\nTovar, Ezequiel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.057 0.059 0.376 74.7 -14\nTrejo, Alan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.641 0.733 0.606 91.7 6'}, {'pitcher_id': '664285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nValdez, Framber 23 18 4 1 0 0 7 30.4 30.8 2 8.7 0.222 0.278 0.294 0.183 0.254 0.273 82.8 2.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 8 4 2 0 0 0 2 25 27.3 2 25 0.5 0.5 0.576 0.249 0.293 0.474 100.1 -0.5\nJones, Nolan 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.635 0.14 0.421 0.232 95.8 28\nBlackmon, Charlie 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.318 0.348 0.298 69.9 -4\nDoyle, Brenton 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.119 0.123 0.109 36.5 -73\nTrejo, Alan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.19 0.385 0.24 99.6 20\nMontero, Elehuris 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.033 0.036 0.032 86.5 14\nTovar, Ezequiel 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.186 0.213 0.172 91.4 2\nStallings, Jacob 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0 0 0 0.018 87.8 34'}, {'pitcher_id': '623352', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHader, Josh 40 38 6 0 0 0 14 35 33.7 2 5 0.158 0.158 0.167 0.178 0.249 0.209 86 19.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 11 10 1 0 0 0 2 18.2 30 1 9.1 0.1 0.1 0.143 0.15 0.178 0.197 83 17\nStallings, Jacob 7 6 0 0 0 0 1 14.3 20 1 14.3 0 0 0.1 0.126 0.224 0.223 91.6 40\nMcMahon, Ryan 9 9 2 0 0 0 6 66.7 58.8 0 0 0.222 0.222 0.196 0.129 0.189 0.136 76.8 -1\nMontero, Elehuris 5 5 0 0 0 0 3 60 33.3 0 0 0 0 0 0.119 0.129 0.108 101.9 -6\nJones, Nolan 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.506 0.848 0.586 80.6 32\nDoyle, Brenton 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.59 0.526 0.682 0.514 93.3 25.3\nCave, Jake 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nTovar, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.247 0.159 66.9 5'}, {'pitcher_id': '656786', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrance, J.P. 8 7 1 0 0 0 3 37.5 23.5 1 12.5 0.143 0.143 0.197 0.295 0.37 0.338 95.2 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Nolan 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.087 0.133 0.293 85.7 24\nMcMahon, Ryan 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.171 0.188 0.158 107.4 0\nTovar, Ezequiel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.69 0.88 0.675 93.8 17'}, {'pitcher_id': '669395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nColeman, Dylan 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.344 0.344', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Nolan 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nCave, Jake 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '650556', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbreu, Bryan 9 8 0 0 0 0 7 77.8 47.4 1 11.1 0 0 0.077 0.083 0.094 0.146 101.4 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nCave, Jake 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nBlackmon, Charlie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDoyle, Brenton 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nJones, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.66 0.752 0.622 101.4 7\nMontero, Elehuris 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '669854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlanco, Ronel 20 18 4 0 0 1 7 35 36.6 1 5 0.222 0.389 0.287 0.169 0.355 0.245 91.2 20.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTovar, Ezequiel 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.296 0.223 0.293 0.218 96 16\nBlackmon, Charlie 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.188 0.225 0.182 88.4 7\nDoyle, Brenton 3 3 1 0 0 0 1 33.3 57.1 0 0 0.333 0.333 0.296 0.055 0.065 0.055 84.7 25\nJones, Nolan 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.329 0.38 0.442 96.2 -10\nMcMahon, Ryan 3 2 1 0 0 1 1 33.3 20 0 0 0.5 2 1.043 0.493 1.938 0.984 108.9 25\nGoodman, Hunter 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nMontero, Elehuris 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.001 0.001 0 85.8 66'}, {'pitcher_id': '686613', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Hunter 10 10 3 0 1 1 4 40 36.4 0 0 0.3 0.8 0.446 0.202 0.604 0.326 88.7 5.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Nolan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.06 0.082 0.059 75.7 39\nMcMahon, Ryan 3 3 2 0 0 1 0 0 33.3 0 0 0.667 1.667 0.962 0.372 1.378 0.703 89.2 -16.7\nDoyle, Brenton 2 2 0 0 0 0 2 100 42.9 0 0 0 0 0 0\nTovar, Ezequiel 2 2 1 0 1 0 1 50 50 0 0 0.5 1.5 0.784 0.323 0.754 0.439 105.2 20\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.137 0.23 0.152 83.8 26'}, {'pitcher_id': '681869', 'player_data': '', 'team_data': 'Shawn Dubin has not faced anyone on this team.'}, {'pitcher_id': '605463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tayler 12 12 2 0 0 0 3 25 27.3 0 0 0.167 0.167 0.148 0.172 0.222 0.171 83.8 29.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontero, Elehuris 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.199 0.215 0.181 80.1 40.7\nJones, Nolan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.48 0.657 0.492 95.1 16.5\nBlackmon, Charlie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.112 0.133 0.111 76.5 8.5\nTovar, Ezequiel 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.015 0.018 0.013 77.3 57\nGoodman, Hunter 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nTrejo, Alan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.192 0.101 93.5 35'}, {'pitcher_id': '519151', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPressly, Ryan 10 10 1 0 0 0 6 60 48 0 0 0.1 0.1 0.089 0.197 0.578 0.316 91.4 11.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nTovar, Ezequiel 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.137 0.148 0.124 98.3 -11\nTrejo, Alan 1 1 0 0 0 0 1 100 60 0 0 0 0 0 0\nBlackmon, Charlie 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.373 1.159 0.623 101.8 24\nToglia, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.854 3.048 1.575 103.7 25\nJones, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nStallings, Jacob 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.099 0.119 0.089 61.8 7'}], 116: [{'pitcher_id': '664299', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJavier, Cristian 53 45 10 1 0 1 14 26.4 31.2 8 15.1 0.222 0.311 0.301 0.202 0.349 0.303 85.5 23.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 19 15 3 0 0 1 7 36.8 41.2 4 21.1 0.2 0.4 0.348 0.202 0.53 0.383 95.9 30.8\nGreene, Riley 8 7 2 0 0 0 2 25 17.6 1 12.5 0.286 0.286 0.307 0.178 0.201 0.235 76.5 8.2\nKreidler, Ryan 4 4 0 0 0 0 3 75 33.3 0 0 0 0 0 0.017 0.017 0.016 73.2 -16\nVierling, Matt 4 4 1 0 0 0 0 0 16.7 0 0 0.25 0.25 0.221 0.334 0.455 0.336 96.9 21.5\nKelly, Carson 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.022 0.023 0.026 82.9 18\nBaddoo, Akil 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.014 0.023 0.354 86.5 52\nUrshela, Gio 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.101 0.123 0.091 72.2 12\nIbáñez, Andy 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0 0 0 73.7 84\nMcKinstry, Zach 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.525 0.634 0.664 0.617 65.1 31\nRogers, Jake 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.282 0.35 0.28 79.4 42.5\nPérez, Wenceel 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.571 1.306 0.732 102.3 22\nKeith, Colt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.606 0.673 0.57 107.7 4'}, {'pitcher_id': '668874', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661527', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Seth 18 17 6 1 1 0 4 22.2 23.5 1 5.6 0.353 0.529 0.394 0.298 0.374 0.315 80.8 25', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 2 1 0 0 0 0 1 50 20 1 50 0 0 0.346 0.346\nCanha, Mark 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.046 0.053 0.042 67.5 42.7\nKeith, Colt 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.699 0.98 0.73 83.3 25.5\nKreidler, Ryan 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.215 0.246 0.203 84.8 7\nIbáñez, Andy 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.002 0.002 0.002 71.6 66\nBaddoo, Akil 1 1 1 0 1 0 0 0 25 0 0 1 3 1.612 0.66 0.742 0.611 101.3 8\nGreene, Riley 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.317 0.37 0.301 89.8 43.5\nMalloy, Justyn-Henry 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVierling, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.631 0.9 0.653 97.6 16\nRogers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.253 0.253 0.229 50 -50\nUrshela, Gio 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.923 1.106 0.885 96.3 12'}, {'pitcher_id': '606160', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontero, Rafael 29 25 9 0 2 0 9 31 33.3 4 13.8 0.36 0.52 0.417 0.224 0.296 0.288 87.3 6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 9 7 2 0 1 0 3 33.3 35.3 2 22.2 0.286 0.571 0.424 0.161 0.297 0.302 96.3 12.8\nUrshela, Gio 5 5 2 0 0 0 2 40 36.4 0 0 0.4 0.4 0.353 0.178 0.187 0.16 72.7 4.3\nGreene, Riley 3 2 1 0 1 0 1 33.3 50 1 33.3 0.5 1.5 0.754 0.446 0.628 0.536 107.9 12\nIbáñez, Andy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.28 0.314 0.258 87.3 15.7\nBaddoo, Akil 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.035 0.037 0.031 82.1 -23\nVierling, Matt 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.27 0.307 0.252 95.2 6\nMcKinstry, Zach 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.029 0.03 0.026 69.9 -29\nKreidler, Ryan 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nKelly, Carson 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.92 1.142 0.89 96.1 14\nPérez, Wenceel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.267 0.295 0.24 79.5 5'}, {'pitcher_id': '666215', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArrighetti, Spencer 12 9 5 1 0 3 2 16.7 15 2 16.7 0.556 1.667 0.877 0.307 1.003 0.578 93.6 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Wenceel 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.134 0.146 0.124 98.2 -8\nVierling, Matt 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.043 0.533 1.879 0.986 89.3 36.5\nGreene, Riley 2 2 2 0 0 2 0 0 0 0 0 1 4 2.085 0.583 2.264 1.151 105.2 33\nBaddoo, Akil 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nCanha, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.102 0.112 0.094 73 -7\nMcKinstry, Zach 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nKeith, Colt 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.27 0.163 0.337 0.206 95.2 23\nMalloy, Justyn-Henry 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nKelly, Carson 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '434378', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerlander, Justin 81 71 16 4 0 2 19 23.5 19.6 6 7.4 0.225 0.366 0.306 0.212 0.414 0.316 86.8 23.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 20 16 5 1 0 1 4 20 22.2 1 5 0.313 0.563 0.433 0.227 0.451 0.366 85 37.4\nUrshela, Gio 16 14 1 0 0 0 2 12.5 10.7 2 12.5 0.071 0.071 0.141 0.173 0.285 0.256 87.9 14.4\nGreene, Riley 9 8 3 0 0 1 2 22.2 15 1 11.1 0.375 0.75 0.496 0.342 0.86 0.516 88.9 24.3\nIbáñez, Andy 12 12 4 2 0 0 2 16.7 5.3 0 0 0.333 0.5 0.357 0.248 0.334 0.251 83.4 29.4\nVierling, Matt 7 7 0 0 0 0 2 28.6 22.2 0 0 0 0 0 0.122 0.175 0.125 86.6 14.6\nKelly, Carson 5 4 2 1 0 0 2 40 37.5 1 20 0.5 0.75 0.567 0.273 0.435 0.385 98.6 21.5\nMcKinstry, Zach 5 4 0 0 0 0 2 40 42.9 1 20 0 0 0.139 0.084 0.297 0.258 98.9 37.5\nRogers, Jake 4 3 0 0 0 0 3 75 41.7 0 0 0 0 0.182 0.182\nKeith, Colt 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.332 1.053 0.572 81.1 -3'}, {'pitcher_id': '664285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nValdez, Framber 103 89 20 1 0 0 24 23.3 26.8 10 9.7 0.225 0.236 0.254 0.233 0.291 0.284 89.6 -5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 23 20 4 0 0 0 5 21.7 29.4 2 8.7 0.2 0.2 0.213 0.248 0.286 0.297 94.6 -3.4\nVierling, Matt 17 14 3 1 0 0 3 17.6 17.2 2 11.8 0.214 0.286 0.276 0.223 0.247 0.268 88.8 -20\nIbáñez, Andy 17 14 4 0 0 0 3 17.6 30 3 17.6 0.286 0.286 0.331 0.301 0.358 0.359 94.8 1.7\nGreene, Riley 13 11 3 0 0 0 4 30.8 23.8 2 15.4 0.273 0.273 0.31 0.192 0.239 0.264 89.6 -10.9\nRogers, Jake 11 8 2 0 0 0 4 36.4 43.8 1 9.1 0.25 0.25 0.319 0.162 0.21 0.269 80.3 -6\nUrshela, Gio 11 11 2 0 0 0 3 27.3 19 0 0 0.182 0.182 0.16 0.199 0.229 0.187 84.1 -8.1\nKreidler, Ryan 6 6 1 0 0 0 1 16.7 30 0 0 0.167 0.167 0.147 0.203 0.48 0.284 83.2 14\nKelly, Carson 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.063 0.083 0.058 91.4 -16\nPérez, Wenceel 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.296 0.505 0.598 0.479 88.3 17.3'}, {'pitcher_id': '623352', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHader, Josh 18 17 1 0 0 1 8 44.4 32.6 1 5.6 0.059 0.235 0.154 0.13 0.447 0.263 88.6 33.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Jake 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.011 0.018 0.24 96.8 60.5\nKelly, Carson 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.052 0.058 0.049 70.5 -14\nCanha, Mark 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nVierling, Matt 3 3 1 0 0 1 1 33.3 14.3 0 0 0.333 1.333 0.691 0.368 1.259 0.679 90.8 16.5\nIbáñez, Andy 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.001 0.001 0 79 76\nBaddoo, Akil 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGreene, Riley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.909 3.596 1.82 107.6 33\nUrshela, Gio 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFlaherty, Jack 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nPérez, Wenceel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.019 0.017 94.5 68'}, {'pitcher_id': '656786', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrance, J.P. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.497 0.781 0.538 86.4 22.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.497 0.781 0.538 86.4 22.5'}, {'pitcher_id': '669395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nColeman, Dylan 19 16 2 0 0 0 3 15.8 20.7 3 15.8 0.125 0.125 0.202 0.093 0.125 0.191 73.9 -11.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 6 4 0 0 0 0 0 0 25 2 33.3 0 0 0.23 0.06 0.061 0.269 66.8 -6.3\nBaddoo, Akil 4 4 1 0 0 0 1 25 16.7 0 0 0.25 0.25 0.221 0.092 0.095 0.087 58.8 -0.7\nGreene, Riley 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.166 0.267 0.186 89.7 -18.4\nKreidler, Ryan 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nIbáñez, Andy 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.044 0.046 0.045 68.5 -27\nRogers, Jake 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '650556', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbreu, Bryan 26 22 7 1 0 2 5 19.2 18.9 4 15.4 0.318 0.636 0.443 0.29 0.536 0.402 89.4 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 9 8 4 0 0 1 1 11.1 12.5 1 11.1 0.5 0.875 0.586 0.351 0.669 0.451 90.9 8\nCanha, Mark 5 4 1 0 0 0 2 40 50 1 20 0.25 0.25 0.314 0.287 0.347 0.362 88 8\nIbáñez, Andy 3 2 0 0 0 0 0 0 16.7 1 33.3 0 0 0.231 0.236 0.258 0.393 85.2 -15.5\nGreene, Riley 2 2 2 1 0 1 0 0 0 0 0 1 3 1.637 0.694 1.92 1.077 103 20\nMcKinstry, Zach 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.005 0.007 0.005 90.5 51\nVierling, Matt 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.185 0.23 0.191 88 36.5\nRogers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.193 0.219 0.176 65.5 5\nPérez, Wenceel 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nKelly, Carson 1 0 0 0 0 0 0 0 1 100 0.699 0.699'}, {'pitcher_id': '669854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlanco, Ronel 10 10 2 0 0 0 3 30 35.3 0 0 0.2 0.2 0.177 0.232 0.318 0.236 94.3 24.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nUrshela, Gio 3 3 2 0 0 0 1 33.3 20 0 0 0.667 0.667 0.588 0.344 0.469 0.35 98.8 17.5\nKelly, Carson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.246 0.153 89.4 42\nMcKinstry, Zach 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.526 0.633 0.499 97.7 1\nBaddoo, Akil 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.01 0.006 88.7 51'}, {'pitcher_id': '686613', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Hunter 68 64 15 2 1 0 25 36.8 25.4 3 4.4 0.234 0.297 0.248 0.177 0.27 0.211 87.1 9.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Riley 14 11 3 1 0 0 5 35.7 30.4 2 14.3 0.273 0.364 0.314 0.256 0.604 0.382 93.2 12.6\nVierling, Matt 9 9 3 0 0 0 2 22.2 11.1 0 0 0.333 0.333 0.295 0.286 0.315 0.263 88.1 -1.4\nMcKinstry, Zach 7 6 1 0 0 0 1 14.3 16.7 1 14.3 0.167 0.167 0.225 0.065 0.07 0.151 78.8 12.8\nBaddoo, Akil 5 5 1 0 1 0 3 60 25 0 0 0.2 0.6 0.32 0.122 0.29 0.171 98.4 31.5\nUrshela, Gio 5 5 1 0 0 0 1 20 10 0 0 0.2 0.2 0.178 0.137 0.174 0.134 86.3 3\nPérez, Wenceel 5 5 0 0 0 0 3 60 40 0 0 0 0 0 0.03 0.042 0.032 82.3 -6\nRogers, Jake 5 5 1 0 0 0 2 40 50 0 0 0.2 0.2 0.178 0.093 0.107 0.089 87.1 -11\nCanha, Mark 5 5 0 0 0 0 3 60 33.3 0 0 0 0 0 0.033 0.038 0.03 54 45.5\nKeith, Colt 5 5 3 1 0 0 1 20 14.3 0 0 0.6 0.8 0.609 0.434 0.527 0.414 92.2 21.8\nMalloy, Justyn-Henry 3 3 1 0 0 0 2 66.7 40 0 0 0.333 0.333 0.296 0.055 0.057 0.049 90.5 -7\nKreidler, Ryan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.021 0.038 0.025 80.5 34\nKelly, Carson 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.364 0.414 0.342 103.1 3.5'}, {'pitcher_id': '681869', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDubin, Shawn 15 12 7 1 0 0 0 0 7.7 3 20 0.583 0.667 0.579 0.364 0.485 0.434 90.6 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 2 2 1 0 0 0 0 0 14.3 0 0 0.5 0.5 0.444 0.195 0.203 0.179 73.6 -8.5\nCanha, Mark 3 2 2 1 0 0 0 0 0 1 33.3 1 1.5 0.95 0.467 0.816 0.6 99.9 10\nGreene, Riley 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.554 0.73 0.55 100.3 10\nKeith, Colt 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.63 0.743 0.599 91.3 9\nPérez, Wenceel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.29 0.352 0.287 92.2 28\nBaddoo, Akil 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nMcKinstry, Zach 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nKelly, Carson 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.067 0.07 0.06 71.5 -7\nMalloy, Justyn-Henry 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.065 0.036 101.1 47'}, {'pitcher_id': '605463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tayler 8 7 2 0 0 2 3 37.5 42.9 0 0 0.286 1.143 0.593 0.213 0.805 0.441 95.1 39.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMalloy, Justyn-Henry 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nKeith, Colt 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.043 0.247 0.837 0.439 96.2 33\nMcKinstry, Zach 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVierling, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nUrshela, Gio 1 1 1 0 0 1 0 0 0 0 0 1 4 1.94 0.993 3.959 1.925 108.2 28\nCanha, Mark 1 0 0 0 0 0 0 0 0 0 0.719 0.719\nBaddoo, Akil 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 79.9 65'}, {'pitcher_id': '519151', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPressly, Ryan 24 24 3 0 0 1 10 41.7 39.6 0 0 0.125 0.25 0.157 0.19 0.417 0.252 89.9 17.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 5 5 0 0 0 0 2 40 41.7 0 0 0 0 0 0.037 0.06 0.043 86.9 -4.7\nBaddoo, Akil 3 3 0 0 0 0 2 66.7 28.6 0 0 0 0 0 0.263 0.875 0.462 103 24\nCanha, Mark 5 5 1 0 0 0 1 20 28.6 0 0 0.2 0.2 0.176 0.185 0.391 0.241 87.1 42.8\nRogers, Jake 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nIbáñez, Andy 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.003 0.526 1.671 0.885 94.8 27.5\nKelly, Carson 2 2 0 0 0 0 1 50 80 0 0 0 0 0 0.285 0.325 0.267 104.3 5\nGreene, Riley 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.25 0.271 0.226 91.1 7\nVierling, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMcKinstry, Zach 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.273 0.292 0.248 80.5 1'}], 118: [{'pitcher_id': '664299', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJavier, Cristian 58 49 13 2 1 2 12 20.7 26.2 6 10.3 0.265 0.469 0.34 0.246 0.495 0.362 85.5 24.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 13 11 3 1 0 0 2 15.4 10 2 15.4 0.273 0.364 0.339 0.178 0.212 0.249 72.9 29.2\nMelendez, MJ 9 6 2 0 0 0 3 33.3 50 3 33.3 0.333 0.333 0.426 0.212 0.244 0.362 95.4 24.7\nWitt Jr., Bobby 6 5 3 0 1 1 0 0 15.4 1 16.7 0.6 1.6 0.866 0.54 1.675 0.869 94.5 24\nPasquantino, Vinnie 6 6 0 0 0 0 1 16.7 12.5 0 0 0 0 0 0.049 0.094 0.059 89 26.8\nPerez, Salvador 7 6 1 0 0 0 1 14.3 23.1 0 0 0.167 0.167 0.127 0.285 0.439 0.348 92.3 22\nVelázquez, Nelson 5 5 3 1 0 1 1 20 40 0 0 0.6 1.4 0.833 0.458 1.094 0.647 87.9 31\nGarcia, Maikel 3 2 0 0 0 0 1 33.3 40 0 0 0 0 0 0.298 1.048 0.588 99.3 30.5\nLoftin, Nick 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.042 0.056 0.041 69.2 53\nWaters, Drew 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.123 0.146 0.113 99.9 -9\nNola, Austin 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.27 0.271 0.243 79.9 32.5\nHampson, Garrett 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nIsbel, Kyle 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.337 0.374 0.316 63.5 -24.5'}, {'pitcher_id': '668874', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661527', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Seth 17 16 2 0 0 0 4 23.5 30.3 1 5.9 0.125 0.125 0.144 0.119 0.22 0.176 87.1 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.442 0.07 0.072 0.063 66.7 50\nWitt Jr., Bobby 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.081 0.087 0.073 92.3 33\nFrazier, Adam 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.295 0.1 0.108 0.093 86.4 12.3\nPasquantino, Vinnie 2 1 0 0 0 0 0 0 50 1 50 0 0 0.344 0.12 0.136 0.4 90.6 -17\nIsbel, Kyle 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.317 1.053 0.563 95.9 6.5\nVelázquez, Nelson 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarcia, Maikel 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.059 0.063 0.053 68.1 -22\nPerez, Salvador 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.029 0.015 86.4 33\nFermin, Freddy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.391 0.457 0.371 98.2 -1'}, {'pitcher_id': '606160', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontero, Rafael 31 26 7 2 0 0 9 29 20.3 3 9.7 0.269 0.346 0.325 0.196 0.297 0.275 87.5 15.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 3 2 1 1 0 0 1 33.3 8.3 0 0 0.5 1 0.662 0.13 0.302 0.363 95.6 25\nFrazier, Adam 6 4 1 0 0 0 0 0 22.2 1 16.7 0.25 0.25 0.314 0.192 0.309 0.303 79.1 11.8\nVelázquez, Nelson 3 2 0 0 0 0 2 66.7 28.6 1 33.3 0 0 0.232 0.232\nNola, Austin 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.29 0.175 0.189 0.158 76.1 36.5\nWitt Jr., Bobby 4 4 2 0 0 0 1 25 14.3 0 0 0.5 0.5 0.443 0.395 0.456 0.371 100 0.7\nMelendez, MJ 3 2 1 0 0 0 0 0 25 1 33.3 0.5 0.5 0.527 0.266 0.294 0.395 88.3 -0.5\nPasquantino, Vinnie 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.635 0.382 0.556 0.395 107.1 14\nIsbel, Kyle 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.151 0.413 0.231 87.6 24.3\nGarcia, Maikel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWaters, Drew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.073 0.131 0.081 84.7 28'}, {'pitcher_id': '666215', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArrighetti, Spencer 17 14 6 1 0 0 3 17.6 22.6 3 17.6 0.429 0.5 0.458 0.263 0.332 0.335 84.4 25.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Maikel 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.257 0.304 0.245 85.5 48.5\nWitt Jr., Bobby 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nMelendez, MJ 2 1 0 0 0 0 0 0 50 1 50 0 0 0.346 0.143 0.515 0.478 104.6 45\nIsbel, Kyle 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.444 0.032 0.034 0.029 59 -27\nPerez, Salvador 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.497 0.513 0.448 82.4 47\nPasquantino, Vinnie 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.143 0.308 0.442 92.4 24\nFrazier, Adam 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.418 0.517 0.414 85.8 19.5\nVelázquez, Nelson 2 2 2 0 0 0 0 0 33.3 0 0 1 1 0.889 0.364 0.39 0.327 82.7 6.5'}, {'pitcher_id': '434378', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerlander, Justin 105 96 33 8 1 2 16 15.2 17.6 5 4.8 0.344 0.51 0.376 0.272 0.413 0.325 85.7 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 67 63 26 8 1 2 9 13.4 17.5 1 1.5 0.413 0.667 0.456 0.304 0.504 0.367 84.7 14.8\nFrazier, Adam 25 23 7 0 0 0 1 4 13.7 2 8 0.304 0.304 0.303 0.301 0.383 0.33 85.2 13\nMelendez, MJ 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.23 0.178 0.179 0.337 102.9 -5\nNola, Austin 4 4 0 0 0 0 2 50 20 0 0 0 0 0 0.101 0.117 0.092 88 23.5\nPasquantino, Vinnie 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.133 0.477 0.254 99.4 31\nIsbel, Kyle 3 1 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0 0 0.239 86.2 32'}, {'pitcher_id': '664285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nValdez, Framber 75 65 12 0 1 1 21 28 25 5 6.7 0.185 0.262 0.241 0.226 0.358 0.307 91.6 10.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 13 12 2 0 0 0 5 38.5 23.1 0 0 0.167 0.167 0.191 0.176 0.217 0.211 82.6 12.7\nWitt Jr., Bobby 9 9 1 0 1 0 4 44.4 26.7 0 0 0.111 0.333 0.174 0.114 0.198 0.131 93.6 10\nNola, Austin 8 5 1 0 0 0 0 0 12.5 2 25 0.2 0.2 0.374 0.314 0.456 0.466 93 -1.8\nHampson, Garrett 7 7 0 0 0 0 1 14.3 7.7 0 0 0 0 0 0.306 0.485 0.334 91.6 9\nMelendez, MJ 6 5 2 0 0 0 3 50 30.8 1 16.7 0.4 0.4 0.409 0.268 0.295 0.321 85.1 11\nLoftin, Nick 6 4 2 0 0 0 1 16.7 20 0 0 0.5 0.5 0.415 0.314 0.35 0.496 92.5 31.2\nPerez, Salvador 6 5 1 0 0 1 2 33.3 36.4 1 16.7 0.2 0.8 0.46 0.232 0.683 0.436 89.2 -14.3\nGarcia, Maikel 6 6 1 0 0 0 1 16.7 22.2 0 0 0.167 0.167 0.147 0.263 0.42 0.29 96.4 9.8\nVelázquez, Nelson 6 5 1 0 0 0 0 0 16.7 1 16.7 0.2 0.2 0.263 0.297 0.651 0.449 95.2 23\nWaters, Drew 3 2 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0 0 0.034 94.4 38\nIsbel, Kyle 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.294 0.14 0.148 0.129 98.2 -14.5\nBlanco, Dairon 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.304 0.353 0.288 93.7 8'}, {'pitcher_id': '623352', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHader, Josh 31 27 7 1 2 1 10 32.3 26.1 4 12.9 0.259 0.556 0.38 0.218 0.445 0.332 85.5 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 12 10 2 1 1 0 6 50 22.6 2 16.7 0.2 0.5 0.347 0.18 0.425 0.328 78.7 -4.8\nPerez, Salvador 4 4 2 0 0 1 1 25 20 0 0 0.5 1.25 0.716 0.318 0.764 0.445 84.6 3\nVelázquez, Nelson 3 2 0 0 0 0 1 33.3 22.2 1 33.3 0 0 0.232 0.097 0.107 0.292 86.7 -3\nHampson, Garrett 4 4 2 0 1 0 2 50 50 0 0 0.5 1 0.618 0.345 0.739 0.46 101.6 15.5\nWitt Jr., Bobby 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.017 0.045 0.027 90.8 39\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.055 0.043 72.4 -28\nIsbel, Kyle 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nNola, Austin 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.236 0.252 0.212 78.8 39\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.346 0.539 0.35 97.8 19\nWaters, Drew 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.327 0.35 0.297 85.5 1'}, {'pitcher_id': '656786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMushinski, Parker 13 11 3 1 0 0 1 7.7 24 1 7.7 0.273 0.364 0.288 0.348 0.403 0.364 89.2 19.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.121 0.134 0.11 85.8 -8\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.023 0.024 0.021 97.5 57\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.223 0.284 0.221 71.7 46\nGarcia, Maikel 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nPerez, Salvador 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.017 0.032 0.021 101.6 58\nPasquantino, Vinnie 2 1 1 0 0 0 0 0 0 0 0 1 1 0.444 0.831 0.952 0.602 91.9 17.5\nFermin, Freddy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.626 0.699 0.581 96.7 6\nVelázquez, Nelson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.262 0.223 95.9 -14\nHampson, Garrett 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.549 0.682 0.546 79.5 25\nMelendez, MJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.96 1.098 0.894 83.3 17'}, {'pitcher_id': '641585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrance, J.P. 32 25 8 1 0 0 3 9.4 16.7 5 15.6 0.32 0.36 0.341 0.277 0.421 0.359 90.6 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 6 5 2 0 0 0 1 16.7 7.1 1 16.7 0.4 0.4 0.41 0.279 0.322 0.337 87.2 14.8\nMelendez, MJ 6 4 1 0 0 0 1 16.7 7.7 1 16.7 0.25 0.25 0.263 0.305 0.362 0.401 93 -0.3\nIsbel, Kyle 4 3 1 0 0 0 0 0 14.3 1 25 0.333 0.333 0.395 0.191 0.223 0.313 84.5 46.3\nGarcia, Maikel 6 4 2 0 0 0 0 0 20 2 33.3 0.5 0.5 0.526 0.391 0.945 0.598 94.9 2\nVelázquez, Nelson 4 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.023 0.033 0.02 91 56.8\nWaters, Drew 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.441 0.34 0.491 0.36 97 16\nBlanco, Dairon 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.292 0.31 0.27 84.2 2\nPerez, Salvador 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.411 0.66 0.446 95.8 19.5'}, {'pitcher_id': '669395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nColeman, Dylan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.186 0.142 86.9 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.186 0.142 86.9 16'}, {'pitcher_id': '650556', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbreu, Bryan 26 20 3 0 0 1 7 26.9 34.1 5 19.2 0.15 0.3 0.309 0.216 0.419 0.363 88.3 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 7 5 1 0 0 0 1 14.3 21.4 2 28.6 0.2 0.2 0.324 0.143 0.168 0.291 75.2 19.5\nWitt Jr., Bobby 4 4 1 0 0 1 2 50 55.6 0 0 0.25 1 0.521 0.307 1.163 0.591 102.3 31\nMelendez, MJ 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.23 0.229 0.265 0.373 91.7 4\nGarcia, Maikel 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.2 0.262 0.193 97.6 13.5\nPasquantino, Vinnie 3 1 0 0 0 0 0 0 0 1 33.3 0 0 0.471 0.414 0.545 0.601 93.7 19\nLoftin, Nick 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nIsbel, Kyle 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.444 0.303 0.337 0.276 106.8 5\nPerez, Salvador 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.093 0.093 0.084 70.5 -43\nFermin, Freddy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 84.6 70'}, {'pitcher_id': '669854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlanco, Ronel 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.05 0.05 0.05 69.3 -14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.05 0.05 0.05 69.3 -14'}, {'pitcher_id': '686613', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Hunter 31 28 16 1 0 4 2 6.5 9.3 3 9.7 0.571 1.036 0.684 0.44 0.82 0.546 95 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 8 7 1 0 0 0 0 0 0 1 12.5 0.143 0.143 0.197 0.283 0.32 0.313 89.6 8\nWitt Jr., Bobby 4 3 2 0 0 1 0 0 0 1 25 0.667 1.667 0.918 0.473 1.245 0.713 98 -0.7\nVelázquez, Nelson 3 3 3 0 0 2 0 0 0 0 0 1 3 1.632 0.731 2.112 1.163 100.5 28.3\nPerez, Salvador 4 4 2 0 0 1 1 25 16.7 0 0 0.5 1.25 0.723 0.337 1.079 0.574 92.9 30.3\nFermin, Freddy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.319 0.365 0.294 93.9 -1\nMelendez, MJ 2 1 1 1 0 0 0 0 50 1 50 1 2 0.981 0.934 1.109 0.789 100.3 12\nWaters, Drew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGarcia, Maikel 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.91 1.147 0.891 101 13\nLoftin, Nick 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.59 0.233 0.258 0.21 98.4 -16.3\nIsbel, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.072 0.076 0.067 89.2 -32\nPasquantino, Vinnie 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.768 0.861 0.714 93.5 11'}, {'pitcher_id': '681869', 'player_data': '', 'team_data': 'Shawn Dubin has not faced anyone on this team.'}, {'pitcher_id': '605463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tayler 11 11 1 0 0 0 2 18.2 23.8 0 0 0.091 0.091 0.081 0.155 0.207 0.156 80.4 1.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.205 0.238 0.194 93.6 2\nFermin, Freddy 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.2 0.234 0.181 75.3 33\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 83 62\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.106 0.083 70.7 -2\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.067 0.068 0.06 69.8 -10\nVelázquez, Nelson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.071 0.058 77.1 -30\nPerez, Salvador 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBlanco, Dairon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.209 0.596 0.329 94.5 28\nIsbel, Kyle 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.399 0.459 0.371 93.7 1\nMelendez, MJ 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.264 0.265 0.241 66.1 -72'}, {'pitcher_id': '519151', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPressly, Ryan 31 28 6 0 0 1 6 19.4 19.6 3 9.7 0.214 0.321 0.278 0.257 0.296 0.298 90.7 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 16 16 3 0 0 1 2 12.5 15.4 0 0 0.188 0.375 0.243 0.259 0.294 0.241 91.2 12.8\nFrazier, Adam 8 7 2 0 0 0 1 12.5 12.5 1 12.5 0.286 0.286 0.307 0.283 0.329 0.323 86.3 3.3\nMelendez, MJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.369 0.458 0.381 101.2 -1\nWaters, Drew 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nNola, Austin 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.35 0.409 0.344 102.2 7\nIsbel, Kyle 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nHampson, Garrett 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nVelázquez, Nelson 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}], 119: [{'pitcher_id': '664299', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJavier, Cristian 53 48 11 1 2 3 14 26.4 31.6 5 9.4 0.229 0.521 0.347 0.256 0.481 0.349 86.5 27.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 24 22 6 0 2 2 6 25 38.8 2 8.3 0.273 0.727 0.433 0.326 0.639 0.433 87.4 8.7\nBiggio, Cavan 7 5 1 0 0 1 0 0 0 2 28.6 0.2 0.8 0.484 0.304 0.898 0.549 89 42\nHernández, Teoscar 8 8 1 0 0 0 2 25 42.9 0 0 0.125 0.125 0.11 0.181 0.27 0.194 86.2 47\nBetts, Mookie 4 4 0 0 0 0 2 50 0 0 0 0 0 0 0.105 0.116 0.099 89.1 26\nHernández, Enrique 4 4 2 1 0 0 1 25 25 0 0 0.5 0.75 0.53 0.226 0.29 0.222 85.8 45\nTaylor, Chris 3 3 0 0 0 0 3 100 80 0 0 0 0 0 0\nSmith, Will 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.873 0.921 0.799 65.1 26\nBarnes, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.349 0 0 0.349 78.8 65'}, {'pitcher_id': '668874', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661527', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, Seth 6 6 0 0 0 0 1 16.7 8.3 0 0 0 0 0 0.149 0.28 0.179 90.2 42.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.01 0.022 0.014 106.1 57\nBiggio, Cavan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.366 1.055 0.566 97.8 26\nSmith, Will 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.48 0.535 0.445 104.1 0\nHernández, Enrique 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.015 0.024 0.017 71.6 64'}, {'pitcher_id': '606160', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMontero, Rafael 50 44 13 1 0 2 14 28 29.2 4 8 0.295 0.455 0.365 0.24 0.434 0.335 89.8 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 12 9 0 0 0 0 4 33.3 37 3 25 0 0 0.173 0.135 0.319 0.311 93 24\nRojas, Miguel 10 10 5 0 0 0 2 20 24 0 0 0.5 0.5 0.438 0.349 0.375 0.325 80.2 3.5\nFreeman, Freddie 6 6 2 1 0 0 2 33.3 35.7 0 0 0.333 0.5 0.353 0.284 0.378 0.285 94.8 25.3\nHernández, Teoscar 6 6 3 0 0 0 2 33.3 30 0 0 0.5 0.5 0.441 0.228 0.262 0.208 95.7 -2.8\nBetts, Mookie 4 2 0 0 0 0 1 25 20 0 0 0 0 0.362 0.09 0.292 0.438 96 33\nSmith, Will 4 3 1 0 0 1 1 25 0 1 25 0.333 1.333 0.674 0.279 1.032 0.571 96 31\nHernández, Enrique 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.066 0.147 0.086 92.4 8\nTaylor, Chris 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.293 0.179 0.208 0.176 81 -5.5\nBiggio, Cavan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.08 0.065 85.3 -16\nLux, Gavin 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.951 3.814 1.92 106.5 32'}, {'pitcher_id': '666215', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681293', 'player_data': '', 'team_data': 'Spencer Arrighetti has not faced anyone on this team.'}, {'pitcher_id': '434378', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerlander, Justin 102 93 18 6 0 2 25 24.5 18.4 8 7.8 0.194 0.323 0.253 0.271 0.427 0.326 88.2 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBetts, Mookie 32 30 3 1 0 0 3 9.4 8.8 2 6.3 0.1 0.133 0.137 0.2 0.277 0.235 90.3 25.7\nOhtani, Shohei 23 23 5 2 0 2 10 43.5 30.4 0 0 0.217 0.565 0.319 0.307 0.68 0.411 87.5 11.3\nTaylor, Chris 9 7 3 1 0 0 2 22.2 17.6 2 22.2 0.429 0.571 0.486 0.329 0.399 0.396 86.5 24.2\nHernández, Teoscar 7 7 3 1 0 0 3 42.9 33.3 0 0 0.429 0.571 0.429 0.444 0.744 0.506 101.5 14\nFreeman, Freddie 6 4 1 1 0 0 1 16.7 21.4 1 16.7 0.25 0.5 0.323 0.429 0.478 0.382 86.2 33.5\nBarnes, Austin 7 7 2 0 0 0 0 0 0 0 0 0.286 0.286 0.25 0.351 0.408 0.33 86.9 17.1\nRojas, Miguel 7 6 1 0 0 0 0 0 0 1 14.3 0.167 0.167 0.226 0.419 0.441 0.443 76.2 27.5\nBiggio, Cavan 3 2 0 0 0 0 2 66.7 25 1 33.3 0 0 0.23 0.23\nHeyward, Jason 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.157 0.157 0.418 92.6 -18\nSmith, Will 3 3 0 0 0 0 3 100 80 0 0 0 0 0 0\nHernández, Enrique 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.07 0.08 0.063 72 -18.5'}, {'pitcher_id': '664285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nValdez, Framber 99 87 17 2 0 7 24 24.2 28.2 10 10.1 0.195 0.46 0.316 0.246 0.497 0.353 91.6 5.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 40 34 4 0 0 1 12 30 32.4 5 12.5 0.118 0.206 0.208 0.158 0.261 0.245 87.9 -3.6\nHernández, Enrique 16 16 5 0 0 2 3 18.8 8.7 0 0 0.313 0.688 0.414 0.283 0.682 0.401 93.5 4.5\nHernández, Teoscar 15 13 3 2 0 1 4 26.7 45.8 2 13.3 0.231 0.615 0.387 0.225 0.43 0.331 94.9 7.7\nBiggio, Cavan 5 4 2 0 0 1 1 20 12.5 1 20 0.5 1.25 0.7 0.403 0.91 0.557 98.4 23.3\nBetts, Mookie 6 5 0 0 0 0 2 33.3 50 1 16.7 0 0 0.116 0.136 0.198 0.235 88.3 26.7\nFreeman, Freddie 4 3 1 0 0 1 0 0 0 1 25 0.333 1.333 0.675 0.548 1.594 0.826 101.2 21\nTaylor, Chris 4 4 1 0 0 1 1 25 42.9 0 0 0.25 1 0.495 0.482 1 0.615 89.9 20.3\nRojas, Miguel 3 2 0 0 0 0 0 0 14.3 0 0 0 0 0.24 0.185 0.206 0.354 92.1 -18\nSmith, Will 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.582 0.99 0.667 89.8 6\nLux, Gavin 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.006 0.015 0.009 87.3 54'}, {'pitcher_id': '623352', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHader, Josh 75 65 14 1 0 5 21 28 33.1 8 10.7 0.215 0.462 0.325 0.232 0.471 0.339 89.7 26.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Chris 14 12 3 0 0 0 4 28.6 46.7 2 14.3 0.25 0.25 0.286 0.272 0.318 0.318 83.4 26.1\nFreeman, Freddie 13 11 4 1 0 2 3 23.1 36.4 2 15.4 0.364 1 0.575 0.303 0.903 0.523 98.2 30.3\nHeyward, Jason 11 10 2 0 0 1 3 27.3 26.1 0 0 0.2 0.5 0.286 0.269 0.575 0.354 90.6 22\nSmith, Will 6 4 0 0 0 0 0 0 11.1 2 33.3 0 0 0.232 0.2 0.3 0.376 93.2 29.8\nHernández, Enrique 8 6 1 0 0 1 3 37.5 50 2 25 0.167 0.667 0.416 0.173 0.46 0.368 89.5 36.3\nBarnes, Austin 7 7 1 0 0 0 3 42.9 33.3 0 0 0.143 0.143 0.126 0.167 0.288 0.193 87.2 38.8\nBetts, Mookie 6 6 1 0 0 1 1 16.7 16.7 0 0 0.167 0.667 0.334 0.204 0.562 0.317 95.4 33\nHernández, Teoscar 3 3 0 0 0 0 1 33.3 30 0 0 0 0 0 0.129 0.146 0.12 90.3 -9\nRojas, Miguel 5 5 2 0 0 0 2 40 33.3 0 0 0.4 0.4 0.35 0.243 0.294 0.235 71 3\nLux, Gavin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBuehler, Walker 1 0 0 0 0 0 0 0 0 0 0'}, {'pitcher_id': '656786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMushinski, Parker 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.717 2.239 1.198 102.1 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.717 2.239 1.198 102.1 14'}, {'pitcher_id': '641585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrance, J.P. 26 24 8 2 0 1 5 19.2 23.3 1 3.8 0.333 0.542 0.369 0.365 0.812 0.492 95.8 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 9 9 3 1 0 0 2 22.2 18.8 0 0 0.333 0.444 0.334 0.405 0.649 0.451 91.9 14.4\nHernández, Teoscar 5 5 2 1 0 0 2 40 33.3 0 0 0.4 0.6 0.425 0.338 0.896 0.507 102.3 17.7\nFreeman, Freddie 3 3 0 0 0 0 1 33.3 57.1 0 0 0 0 0 0.096 0.111 0.087 91.6 25\nSmith, Will 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.329 1.164 0.606 97.2 32.3\nRojas, Miguel 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.82 0.534 0.6 0.562 94.4 5\nBetts, Mookie 3 2 1 0 0 1 0 0 0 0 0 0.5 2 0.668 0.542 2.077 0.808 100.7 33.3'}, {'pitcher_id': '669395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nColeman, Dylan 8 6 4 1 0 0 0 0 20 1 12.5 0.667 0.833 0.575 0.407 0.499 0.385 84.8 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.294 0.447 0.513 0.42 80.6 9.7\nFreeman, Freddie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.05 0.05 0.05 71.9 -13\nHernández, Enrique 1 0 0 0 0 0 0 0 100 1 100 0.689 0.689\nHernández, Teoscar 1 0 0 0 0 0 0 0 0 0 0 0 0.021 88.1 60\nBetts, Mookie 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.714 1.025 0.752 105 14\nOhtani, Shohei 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.338 0.381 0.308 86.7 1'}, {'pitcher_id': '650556', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbreu, Bryan 20 14 1 0 0 0 5 25 36.4 5 25 0.071 0.071 0.218 0.095 0.113 0.244 92 24.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 5 5 0 0 0 0 3 60 61.5 0 0 0 0 0 0.011 0.016 0.013 92.5 60\nOhtani, Shohei 7 6 1 0 0 0 1 14.3 14.3 1 14.3 0.167 0.167 0.226 0.175 0.209 0.241 92.9 12.6\nSmith, Will 2 1 0 0 0 0 0 0 16.7 1 50 0 0 0.348 0.212 0.23 0.449 90.2 -6\nTaylor, Chris 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nRojas, Miguel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nLux, Gavin 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nFreeman, Freddie 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nHeyward, Jason 1 0 0 0 0 0 0 0 0 0 0 0 0.146 89.8 23\nBetts, Mookie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.013 0.008 90.4 42'}, {'pitcher_id': '669854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlanco, Ronel 23 22 3 0 0 2 6 26.1 36.4 1 4.3 0.136 0.409 0.243 0.158 0.345 0.231 83.8 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 3 3 1 0 0 1 1 33.3 28.6 0 0 0.333 1.333 0.668 0.321 1.022 0.553 82.3 -6.5\nFreeman, Freddie 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.102 0.129 0.092 87.4 21.5\nOhtani, Shohei 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.14 0.33 0.198 91.7 38.3\nHernández, Teoscar 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.232 0.24 0.244 0.378 61.5 -54\nBiggio, Cavan 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.097 0.111 0.091 93.8 36.5\nBetts, Mookie 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.079 0.239 0.127 79.4 4.5\nRojas, Miguel 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.005 0.011 0.007 86 45\nHeyward, Jason 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.002 0.316 0.627 0.396 85.3 0.5'}, {'pitcher_id': '686613', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Hunter 20 17 4 3 0 1 6 30 19.6 3 15 0.235 0.588 0.391 0.237 0.589 0.393 91.9 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 4 3 0 0 0 0 2 50 50 1 25 0 0 0.174 0.23 0.469 0.393 107.8 17\nSmith, Will 3 2 0 0 0 0 1 33.3 12.5 1 33.3 0 0 0.232 0.043 0.044 0.259 80.4 -11\nRojas, Miguel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.02 0.038 0.361 83.9 50\nOhtani, Shohei 3 3 2 2 0 0 0 0 0 0 0 0.667 1.333 0.829 0.381 0.98 0.557 98 20.3\nBetts, Mookie 3 3 1 0 0 1 1 33.3 16.7 0 0 0.333 1.333 0.668 0.37 1.056 0.58 96.2 10\nFreeman, Freddie 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.415 0.233 0.688 0.377 90.4 19.5\nHeyward, Jason 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.136 0.154 0.128 72.1 9'}, {'pitcher_id': '681869', 'player_data': '', 'team_data': 'Shawn Dubin has not faced anyone on this team.'}, {'pitcher_id': '605463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tayler 3 3 2 2 0 0 0 0 0 0 0 0.667 1.333 0.82 0.351 0.542 0.383 101.8 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Enrique 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.683 1.135 0.771 105.9 15\nOhtani, Shohei 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.313 0.341 0.3 108 -10\nRojas, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.149 0.078 91.4 32'}, {'pitcher_id': '519151', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPressly, Ryan 46 44 10 1 0 3 12 26.1 24.7 1 2.2 0.227 0.455 0.287 0.22 0.411 0.274 90.2 18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 15 15 4 0 0 2 8 53.3 44.8 0 0 0.267 0.667 0.385 0.2 0.5 0.287 97.1 21.7\nBetts, Mookie 9 7 1 0 0 0 0 0 7.1 1 11.1 0.143 0.143 0.173 0.197 0.225 0.243 83.9 15.8\nFreeman, Freddie 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.121 0.121 0.109 38 -15\nHernández, Teoscar 5 5 1 0 0 0 1 20 28.6 0 0 0.2 0.2 0.177 0.142 0.172 0.136 94.6 38.5\nHeyward, Jason 4 4 1 0 0 0 1 25 22.2 0 0 0.25 0.25 0.22 0.219 0.248 0.212 89.1 -1\nHernández, Enrique 4 4 2 0 0 1 0 0 0 0 0 0.5 1.25 0.721 0.433 1.136 0.646 90 38.8\nLux, Gavin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRojas, Miguel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.278 0.307 0.251 99.1 -9.5\nTaylor, Chris 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.616 0.368 0.377 0.342 101 -1.5'}]}, 118: {115: [{'pitcher_id': '680742', 'player_data': '', 'team_data': ''}, {'pitcher_id': '623433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Nick 3 3 0 0 0 0 2 66.7 37.5 0 0 0 0 0 0.071 0.076 0.076 105.7 -17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.106 0.114 0.114 105.7 -17\nJones, Nolan 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0'}, {'pitcher_id': '519293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 16 15 5 1 0 0 5 31.3 33.3 1 6.3 0.333 0.4 0.341 0.258 0.404 0.309 93.7 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 8 8 3 0 0 0 2 25 14.3 0 0 0.375 0.375 0.33 0.38 0.638 0.428 97.9 12.5\nMcMahon, Ryan 6 5 2 1 0 0 2 33.3 55.6 1 16.7 0.4 0.6 0.469 0.21 0.28 0.289 88.4 9.3\nCave, Jake 1 1 0 0 0 0 1 100 60 0 0 0 0 0 0\nJones, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.024 0.02 93 71'}, {'pitcher_id': '669674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLong, Sam 8 8 2 0 0 0 1 12.5 15.4 0 0 0.25 0.25 0.221 0.156 0.181 0.151 89.5 19.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.002 0.002 0.002 92.4 56\nToglia, Michael 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.135 0.166 0.128 81.6 29\nMcMahon, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.249 0.282 0.25 98.4 -11\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.557 0.639 0.532 98.5 4\nMontero, Elehuris 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.04 0.027 85.5 40\nJones, Nolan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.152 0.154 0.142 88.8 -9'}, {'pitcher_id': '670167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchreiber, John 4 4 1 0 0 0 2 50 33.3 0 0 0.25 0.25 0.221 0.049 0.051 0.045 76.6 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.013 0.022 0.016 84.4 59\nJones, Nolan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.181 0.182 0.166 68.8 -44'}, {'pitcher_id': '663738', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynch IV, Daniel 20 16 6 0 1 0 6 30 35.7 2 10 0.375 0.5 0.404 0.286 0.368 0.338 92.5 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 6 4 2 0 0 0 1 16.7 50 2 33.3 0.5 0.5 0.524 0.459 0.556 0.527 90.9 22.7\nBlackmon, Charlie 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.242 0.127 0.317 0.366 96.2 25\nTrejo, Alan 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nDoyle, Brenton 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0 0 0 0.045 90.7 27\nMontero, Elehuris 3 3 1 0 1 0 1 33.3 66.7 0 0 0.333 1 0.523 0.147 0.275 0.177 93.4 25.5\nTovar, Ezequiel 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.664 0.721 0.613 81.8 12\nCave, Jake 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.357 0.379 0.331 103.7 -5'}, {'pitcher_id': '607625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLugo, Seth 60 54 15 4 1 3 16 26.7 21.8 6 10 0.278 0.556 0.382 0.277 0.506 0.366 92.4 16.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 13 12 5 1 0 2 4 30.8 23.1 1 7.7 0.417 1 0.593 0.369 0.765 0.489 100.4 21.5\nBlackmon, Charlie 14 13 3 1 1 0 3 21.4 19 1 7.1 0.231 0.462 0.312 0.255 0.416 0.316 88.9 17.9\nTovar, Ezequiel 8 7 2 0 0 0 3 37.5 13.3 1 12.5 0.286 0.286 0.308 0.19 0.297 0.269 87.3 19.8\nMontero, Elehuris 7 6 2 1 0 0 2 28.6 33.3 1 14.3 0.333 0.5 0.403 0.27 0.339 0.327 89.1 5.5\nDoyle, Brenton 6 5 0 0 0 0 1 16.7 22.2 1 16.7 0 0 0.116 0.186 0.217 0.257 85.7 11\nStallings, Jacob 6 5 1 0 0 1 1 16.7 14.3 1 16.7 0.2 0.8 0.45 0.299 0.823 0.496 97.4 30.5\nJones, Nolan 3 3 2 1 0 0 1 33.3 25 0 0 0.667 1 0.709 0.503 1.03 0.64 105 17.5\nToglia, Michael 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.105 0.106 0.096 85.5 -9'}, {'pitcher_id': '608717', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Chris 31 30 13 0 0 2 4 12.9 26.4 1 3.2 0.433 0.633 0.463 0.337 0.538 0.383 87.1 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 20 20 8 0 0 2 2 10 27.3 0 0 0.4 0.7 0.464 0.302 0.547 0.357 85.8 15.2\nMcMahon, Ryan 5 4 4 0 0 0 0 0 42.9 1 20 1 1 0.841 0.714 0.833 0.68 88.9 12.5\nStallings, Jacob 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.061 0.21 0.108 93.1 37\nCave, Jake 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTovar, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.086 0.081 86.8 -24\nMontero, Elehuris 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.943 1.134 0.92 91.9 14'}, {'pitcher_id': '694361', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672582', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZerpa, Angel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.055 0.052 87.6 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.055 0.052 87.6 5'}, {'pitcher_id': '663903', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSinger, Brady 19 19 5 1 0 0 5 26.3 23.1 0 0 0.263 0.316 0.252 0.276 0.368 0.278 92.2 0.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Nolan 5 5 4 1 0 0 0 0 0 0 0 0.8 1 0.782 0.663 0.872 0.662 102.7 11.8\nMcMahon, Ryan 3 3 0 0 0 0 2 66.7 22.2 0 0 0 0 0 0.007 0.018 0.009 95.4 43\nTovar, Ezequiel 2 2 0 0 0 0 1 50 14.3 0 0 0 0 0 0.029 0.029 0.028 88.3 -41\nCave, Jake 4 4 1 0 0 0 1 25 40 0 0 0.25 0.25 0.221 0.321 0.473 0.34 94.3 5.3\nMontero, Elehuris 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.056 0.056 0.053 74 -25\nDoyle, Brenton 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.154 0.173 0.142 81.6 -9.5'}, {'pitcher_id': '681882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSelby, Colin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663704', 'player_data': '', 'team_data': 'James McArthur has not faced anyone on this team.'}, {'pitcher_id': '685107', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666142', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRagans, Cole 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.294 0.326 0.35 0.296 91.4 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTovar, Ezequiel 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.56 0.607 0.507 81 12\nMcMahon, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.082 0.078 96.5 -30\nBlackmon, Charlie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.336 0.36 0.302 96.8 -3'}, {'pitcher_id': '672578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Carlos 12 11 5 1 0 1 2 16.7 25.8 1 8.3 0.455 0.818 0.554 0.382 0.631 0.453 93.3 22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 4 3 2 0 0 0 0 0 18.2 1 25 0.667 0.667 0.614 0.77 0.953 0.739 94.3 13\nBlackmon, Charlie 4 4 1 0 0 1 1 25 30 0 0 0.25 1 0.518 0.161 0.421 0.243 92.9 24\nCave, Jake 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.003 0.003 0.003 92.3 42\nStallings, Jacob 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nDoyle, Brenton 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.743 0.853 0.698 85.8 19\nTovar, Ezequiel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.5 1.542 0.81 100.4 26'}, {'pitcher_id': '608379', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWacha, Michael 51 49 13 3 0 0 11 21.6 28.3 2 3.9 0.265 0.327 0.273 0.234 0.335 0.264 85.2 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 25 25 9 1 0 0 2 8 17.9 0 0 0.36 0.4 0.332 0.355 0.526 0.377 85.4 10\nMcMahon, Ryan 10 8 1 1 0 0 3 30 44 2 20 0.125 0.25 0.262 0.144 0.176 0.25 87.6 25\nMontero, Elehuris 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.177 0.186 0.288 0.202 87.3 28.7\nTovar, Ezequiel 5 5 1 1 0 0 1 20 27.3 0 0 0.2 0.4 0.249 0.105 0.17 0.113 77.6 -2.8\nDoyle, Brenton 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nCave, Jake 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.009 0.013 0.01 84.8 45\nJones, Nolan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.343 0.344 0.308 89.9 1.5'}, {'pitcher_id': '679525', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Alec 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.003 0.003 0.003 87.5 45', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.003 0.003 0.003 87.5 45'}], 116: [{'pitcher_id': '680742', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBowlan, Jonathan 4 4 0 0 0 0 0 0 12.5 0 0 0 0 0 0.079 0.122 0.084 81 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.129 0.247 0.155 92.3 25\nBaddoo, Akil 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.062 0.036 88.8 34\nIbáñez, Andy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.118 0.142 0.11 50.9 -2\nRogers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.038 0.038 0.037 91.8 -37'}, {'pitcher_id': '623433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Nick 21 19 4 1 0 0 9 42.9 32 2 9.5 0.211 0.263 0.253 0.148 0.252 0.219 91.1 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 4 4 0 0 0 0 2 50 27.3 0 0 0 0 0 0.031 0.035 0.029 89.9 19\nUrshela, Gio 3 2 0 0 0 0 2 66.7 33.3 1 33.3 0 0 0.233 0.233\nIbáñez, Andy 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.588 0.354 0.413 0.334 99.9 2\nKelly, Carson 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nRogers, Jake 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.032 0.042 0.029 68.2 -12\nCanha, Mark 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.275 0.824 0.447 96.9 29\nPérez, Wenceel 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.069 0.094 0.072 78.4 54\nGreene, Riley 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nKeith, Colt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.299 0.35 0.271 90.9 0\nMcKinstry, Zach 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.376 0.4 0.343 100.2 -3'}, {'pitcher_id': '519293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 15 12 6 1 0 1 0 0 20 3 20 0.5 0.833 0.588 0.461 0.829 0.573 89.3 13.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 4 4 2 0 0 1 0 0 0 0 0 0.5 1.25 0.702 0.394 0.818 0.497 82.7 -1.3\nCanha, Mark 2 0 0 0 0 0 0 0 50 2 100 0.69 0.69\nVierling, Matt 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.332 0.526 0.356 97.1 33.5\nKeith, Colt 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.923 0.989 0.769 72.4 19\nRogers, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.344 0.374 0.322 93.5 1\nBaddoo, Akil 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.37 0.374 0.338 81.4 6\nUrshela, Gio 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.51 0.765 0.548 99.1 23\nIbáñez, Andy 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.643 2.354 1.213 101.5 29'}, {'pitcher_id': '669674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLong, Sam 8 8 2 1 0 0 2 25 28.6 0 0 0.25 0.375 0.268 0.204 0.346 0.236 79.9 20.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.42 0.202 0.518 0.304 79.3 34.7\nVierling, Matt 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.44 0.466 0.53 0.44 81.3 19\nPérez, Wenceel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nUrshela, Gio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.072 0.111 0.068 72.2 -38\nCanha, Mark 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.02 0.043 0.026 88 36'}, {'pitcher_id': '670167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchreiber, John 11 11 2 1 0 0 4 36.4 5.3 0 0 0.182 0.273 0.195 0.137 0.176 0.136 91.6 32.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Riley 3 3 0 0 0 0 2 66.7 20 0 0 0 0 0 0.115 0.123 0.103 105.5 -7\nRogers, Jake 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.015 0.025 0.018 85.7 57\nUrshela, Gio 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.233 0.299 0.229 85.7 38\nPérez, Wenceel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.009 0.008 84.2 63\nCanha, Mark 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.327 0.455 0.34 97.2 20\nBaddoo, Akil 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '663738', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynch IV, Daniel 50 45 11 2 0 0 9 18 26.1 3 6 0.244 0.289 0.251 0.302 0.361 0.318 89.7 9.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 15 13 2 0 0 0 3 20 25.7 1 6.7 0.154 0.154 0.164 0.183 0.228 0.218 90.4 12.7\nGreene, Riley 9 8 3 2 0 0 3 33.3 29.4 1 11.1 0.375 0.625 0.455 0.338 0.409 0.368 87.4 -4.6\nKreidler, Ryan 6 5 1 0 0 0 1 16.7 44.4 1 16.7 0.2 0.2 0.262 0.459 0.583 0.494 97.9 10\nRogers, Jake 5 5 0 0 0 0 1 20 22.2 0 0 0 0 0 0.294 0.318 0.268 91.8 -6.8\nBaddoo, Akil 5 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.176 0.442 0.5 0.442 87.4 27.4\nIbáñez, Andy 6 6 2 0 0 0 0 0 14.3 0 0 0.333 0.333 0.294 0.234 0.281 0.222 86.1 10\nVierling, Matt 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.529 0.608 0.496 88.4 6.3\nCanha, Mark 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '607625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLugo, Seth 35 32 5 0 0 1 16 45.7 35.5 2 5.7 0.156 0.25 0.198 0.181 0.272 0.218 89.9 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 7 6 2 0 0 0 1 14.3 30.8 0 0 0.333 0.333 0.253 0.409 0.496 0.334 88.6 25\nGreene, Riley 6 6 0 0 0 0 4 66.7 28.6 0 0 0 0 0 0.135 0.164 0.132 89.7 22.5\nUrshela, Gio 4 3 0 0 0 0 2 50 50 1 25 0 0 0.175 0.103 0.175 0.26 94.1 20\nRogers, Jake 4 4 0 0 0 0 3 75 28.6 0 0 0 0 0 0 0 0 57 61\nKeith, Colt 3 2 0 0 0 0 0 0 14.3 1 33.3 0 0 0.231 0.307 0.643 0.494 94.9 -10.5\nMcKinstry, Zach 4 4 2 0 0 1 1 25 40 0 0 0.5 1.25 0.722 0.243 0.537 0.324 96.7 -6.7\nBaddoo, Akil 3 3 0 0 0 0 3 100 71.4 0 0 0 0 0 0\nCanha, Mark 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.296 0.281 0.337 0.266 92.7 43.5\nKelly, Carson 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '608717', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Chris 18 17 3 0 0 1 7 38.9 36.4 1 5.6 0.176 0.353 0.253 0.231 0.408 0.294 93.8 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 4 4 1 0 0 1 1 25 25 0 0 0.25 1 0.521 0.236 0.369 0.264 95.6 -21\nBaddoo, Akil 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nKelly, Carson 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.047 0.179 0.09 98.7 41\nRogers, Jake 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.346 0.346\nKeith, Colt 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.14 0.3 0.186 93.1 23\nCanha, Mark 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.488 0.559 0.458 88.7 14\nUrshela, Gio 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.514 1.738 0.916 99.2 28\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.126 0.077 88.6 31\nGreene, Riley 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.983 1.122 0.917 88.3 15\nPérez, Wenceel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.171 0.519 0.263 94.6 31'}, {'pitcher_id': '694361', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKlein, Will 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.057 0.063 0.052 89.2 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMcKinstry, Zach 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nKeith, Colt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.172 0.188 0.155 89.2 -9'}, {'pitcher_id': '672582', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZerpa, Angel 11 9 2 1 0 0 0 0 10.5 2 18.2 0.222 0.333 0.32 0.25 0.316 0.326 91.6 1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.594 0.681 0.629 111 1\nKeith, Colt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.248 0.274 0.225 96.3 -9\nGreene, Riley 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.164 0.167 0.147 83.9 -45.5\nRogers, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.041 0.068 0.041 84.2 17\nUrshela, Gio 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nIbáñez, Andy 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.459 0.581 0.449 94.7 24\nKelly, Carson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.259 0.139 92 31'}, {'pitcher_id': '663903', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSinger, Brady 72 62 18 2 1 1 13 18.1 18.1 10 13.9 0.29 0.403 0.353 0.259 0.381 0.334 89.3 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Riley 15 12 3 0 0 0 4 26.7 17.6 3 20 0.25 0.25 0.315 0.204 0.417 0.347 96 2.8\nBaddoo, Akil 15 11 5 2 0 0 3 20 20 4 26.7 0.455 0.636 0.526 0.332 0.428 0.426 85.4 17.6\nMcKinstry, Zach 9 9 2 0 1 0 2 22.2 28.6 0 0 0.222 0.444 0.277 0.233 0.357 0.254 94.3 22.9\nCanha, Mark 6 5 0 0 0 0 2 33.3 27.3 1 16.7 0 0 0.115 0.073 0.08 0.175 86.5 -18\nVierling, Matt 5 5 3 0 0 1 0 0 0 0 0 0.6 1.2 0.754 0.277 0.657 0.388 91.2 3\nIbáñez, Andy 6 6 1 0 0 0 0 0 0 0 0 0.167 0.167 0.147 0.185 0.215 0.181 91.3 -16.3\nUrshela, Gio 4 4 1 0 0 0 1 25 0 0 0 0.25 0.25 0.22 0.172 0.204 0.162 85.4 -26\nKreidler, Ryan 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.473 0.536 0.453 78.9 18\nPérez, Wenceel 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.447 0.501 0.503 94.2 1\nRogers, Jake 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.388 0.455 0.366 73.7 22.3\nKelly, Carson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.482 0.412 78.1 -1.5\nKeith, Colt 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.443 0.52 0.561 101 1'}, {'pitcher_id': '681882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSelby, Colin 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696'}, {'pitcher_id': '663704', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcArthur, James 3 3 0 0 0 0 0 0 14.3 0 0 0 0 0 0.223 0.245 0.203 89.1 18.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.017 0.009 99.4 48\nMcKinstry, Zach 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.32 0.363 0.295 63.5 17\nGreene, Riley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.34 0.356 0.306 104.3 -9'}, {'pitcher_id': '685107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVeneziano, Anthony 5 4 1 0 1 0 0 0 14.3 1 20 0.25 0.75 0.453 0.254 0.595 0.412 81.4 39.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.183 0.261 0.443 68.6 49\nVierling, Matt 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.784 0.167 0.287 0.183 78.6 41.5\nIbáñez, Andy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.5 1.542 0.81 99.7 26'}, {'pitcher_id': '666142', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRagans, Cole 34 29 2 0 1 0 15 44.1 33.3 5 14.7 0.069 0.138 0.175 0.098 0.137 0.188 84 19.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVierling, Matt 6 4 0 0 0 0 3 50 71.4 2 33.3 0 0 0.231 0.014 0.021 0.239 81.3 31\nUrshela, Gio 7 7 1 0 1 0 2 28.6 31.3 0 0 0.143 0.429 0.229 0.039 0.056 0.038 83.6 41\nIbáñez, Andy 6 5 0 0 0 0 2 33.3 20 1 16.7 0 0 0.116 0.194 0.226 0.271 87.3 16\nRogers, Jake 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nCanha, Mark 3 3 0 0 0 0 3 100 57.1 0 0 0 0 0 0\nGreene, Riley 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.419 0.691 0.471 79.9 7.3\nKelly, Carson 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.232 0.232\nKeith, Colt 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.037 0.049 0.035 77.2 -12\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.202 0.204 0.187 97.7 -16'}, {'pitcher_id': '672578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Carlos 26 22 5 0 0 0 6 23.1 22.4 3 11.5 0.227 0.227 0.249 0.234 0.32 0.281 88.3 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaddoo, Akil 6 5 0 0 0 0 0 0 21.4 1 16.7 0 0 0.115 0.283 0.503 0.39 88.5 23.2\nGreene, Riley 3 3 0 0 0 0 2 66.7 18.2 0 0 0 0 0 0.076 0.11 0.078 82 26\nUrshela, Gio 4 3 1 0 0 0 1 25 28.6 0 0 0.333 0.333 0.221 0.138 0.177 0.103 93.5 19.7\nIbáñez, Andy 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.531 0.607 0.503 70.2 28\nVierling, Matt 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.249 0.262 0.225 93.1 0.5\nCanha, Mark 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.817 0.252 0.286 0.382 92.1 -5.5\nKreidler, Ryan 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nMcKinstry, Zach 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.054 0.093 0.059 90.3 24\nRogers, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nKelly, Carson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.666 0.905 0.675 95.4 16'}, {'pitcher_id': '608379', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWacha, Michael 68 65 20 2 0 3 11 16.2 22.8 1 1.5 0.308 0.477 0.344 0.304 0.491 0.347 89 6.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 17 17 4 0 0 1 4 23.5 18.5 0 0 0.235 0.412 0.274 0.279 0.526 0.341 94 -1.9\nCanha, Mark 12 12 3 1 0 0 1 8.3 13.6 0 0 0.25 0.333 0.253 0.27 0.394 0.285 88.9 7.9\nGreene, Riley 9 7 2 0 0 0 2 22.2 20 1 11.1 0.286 0.286 0.274 0.283 0.328 0.299 87.9 11\nPérez, Wenceel 6 6 2 0 0 1 0 0 18.2 0 0 0.333 0.833 0.496 0.299 0.495 0.34 86.6 2.2\nKeith, Colt 6 6 3 1 0 0 1 16.7 25 0 0 0.5 0.667 0.508 0.353 0.426 0.337 90.2 20.6\nBaddoo, Akil 4 4 0 0 0 0 2 50 44.4 0 0 0 0 0 0.171 0.659 0.333 98.3 47\nMaeda, Kenta 3 3 1 0 0 0 0 0 30.8 0 0 0.333 0.333 0.292 0.113 0.147 0.117 66.7 -0.7\nRogers, Jake 5 5 3 0 0 1 0 0 22.2 0 0 0.6 1.2 0.773 0.603 1.024 0.692 85.6 16\nMcKinstry, Zach 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.296 0.353 0.411 0.332 87.8 -4.5\nVierling, Matt 3 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.538 0.4 0.5 0.506 97.5 -12.5'}, {'pitcher_id': '679525', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Alec 19 17 5 0 1 0 4 21.1 28.1 2 10.5 0.294 0.412 0.345 0.279 0.429 0.344 88.8 17.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 3 2 0 0 0 0 1 33.3 66.7 1 33.3 0 0 0.231 0.222 0.339 0.387 97.6 17\nKeith, Colt 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.593 0.359 0.485 0.369 97.8 10.5\nKelly, Carson 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346\nPérez, Wenceel 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.333 0.383 0.316 68.5 17.7\nGreene, Riley 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.29 0.587 0.367 100.2 20\nVierling, Matt 3 3 2 0 1 0 0 0 20 0 0 0.667 1.333 0.834 0.422 0.718 0.486 97.1 14\nMcKinstry, Zach 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.041 0.051 0.036 81.9 28'}], 117: [{'pitcher_id': '680742', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '623433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Nick 30 25 10 1 0 1 7 23.3 25.5 5 16.7 0.4 0.56 0.458 0.368 0.599 0.457 93.6 14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 8 6 2 0 0 1 2 25 0 2 25 0.333 0.833 0.527 0.347 0.712 0.502 92.5 22.8\nBregman, Alex 7 6 2 0 0 0 2 28.6 15.4 1 14.3 0.333 0.333 0.35 0.257 0.38 0.336 81.2 21.3\nPeña, Jeremy 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.036 0.039 0.032 91.3 -19\nDubón, Mauricio 3 3 2 0 0 0 0 0 16.7 0 0 0.667 0.667 0.59 0.522 0.829 0.576 92.9 11.3\nAlvarez, Yordan 4 3 2 0 0 0 1 25 66.7 1 25 0.667 0.667 0.613 0.569 0.715 0.587 106.2 10\nDiaz, Yainer 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.369 0.897 0.517 102.3 13.5\nMeyers, Jake 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.666 0.77 0.636 102.9 6\nSingleton, Jon 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nCaratini, Victor 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.783 1.101 0.811 100.8 14'}, {'pitcher_id': '519293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 22 19 5 1 0 1 2 9.1 16.7 2 9.1 0.263 0.474 0.364 0.182 0.35 0.29 85.2 5.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 7 7 1 0 0 0 1 14.3 28.6 0 0 0.143 0.143 0.126 0.199 0.226 0.19 75.4 4.5\nBregman, Alex 5 3 1 0 0 0 0 0 0 2 40 0.333 0.333 0.453 0.22 0.644 0.488 95.2 26.3\nCaratini, Victor 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.096 0.101 0.087 84.5 -21.5\nHummel, Cooper 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nDiaz, Yainer 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.169 0.331 0.206 92.5 24\nMcCormick, Chas 2 2 2 1 0 1 0 0 0 0 0 1 3 1.624 0.311 1.079 0.557 94.9 27.5\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.221 0.236 0.211 90.8 -7\nPeña, Jeremy 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '669674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLong, Sam 9 8 1 0 0 0 1 11.1 18.8 1 11.1 0.125 0.125 0.175 0.152 0.349 0.259 94.2 32.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.164 0.208 0.335 96.7 28.5\nPeña, Jeremy 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.015 0.042 0.022 94.8 42\nBregman, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.018 0.042 0.027 86.8 39\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.417 1.288 0.699 98.1 27\nMeyers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.403 0.917 0.526 99.3 23'}, {'pitcher_id': '670167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchreiber, John 14 12 1 1 0 0 3 21.4 21.4 2 14.3 0.083 0.167 0.188 0.179 0.246 0.256 87.2 4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.282 0.371 0.283 85.2 1.5\nPeña, Jeremy 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.143 0.145 0.132 71.4 -51\nAlvarez, Yordan 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.234 0.242 0.371 82.9 1\nCaratini, Victor 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.103 0.111 0.093 94.2 17.5\nMcCormick, Chas 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nBregman, Alex 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDiaz, Yainer 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nDubón, Mauricio 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.569 1.132 0.709 105.1 18\nSingleton, Jon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.081 0.049 84.2 30'}, {'pitcher_id': '663738', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynch IV, Daniel 16 15 4 3 0 0 3 18.8 18.5 1 6.3 0.267 0.467 0.331 0.294 0.537 0.372 89.2 6.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 6 6 1 1 0 0 1 16.7 11.1 0 0 0.167 0.333 0.207 0.192 0.216 0.186 81.8 1.8\nMeyers, Jake 4 4 2 1 0 0 1 25 12.5 0 0 0.5 0.75 0.53 0.406 0.647 0.442 104.3 15.3\nMcCormick, Chas 3 2 1 1 0 0 1 33.3 40 1 33.3 0.5 1 0.644 0.446 1.637 0.79 103.9 26\nAlvarez, Yordan 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.246 0.299 0.236 81.3 -1'}, {'pitcher_id': '607625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLugo, Seth 46 41 17 5 0 1 5 10.9 10.6 5 10.9 0.415 0.61 0.467 0.304 0.536 0.392 90.2 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 9 7 3 1 0 0 2 22.2 7.1 2 22.2 0.429 0.571 0.49 0.24 0.455 0.379 99 17.6\nCaratini, Victor 4 4 3 0 0 0 0 0 7.7 0 0 0.75 0.75 0.659 0.425 0.525 0.408 95.8 19\nBregman, Alex 9 9 3 2 0 0 2 22.2 11.8 0 0 0.333 0.556 0.377 0.271 0.521 0.333 93.7 10.9\nAlvarez, Yordan 6 5 3 1 0 1 0 0 0 1 16.7 0.6 1.4 0.805 0.633 1.515 0.856 89.6 24.8\nDiaz, Yainer 3 2 2 1 0 0 0 0 40 1 33.3 1 1.5 0.95 0.433 0.697 0.552 100 16\nPeña, Jeremy 6 6 2 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.23 0.252 0.213 81.1 -10.7\nMeyers, Jake 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.109 0.147 0.113 83.1 -2\nDubón, Mauricio 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.07 0.072 0.382 65.1 -3\nMcCormick, Chas 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.275 0.339 0.268 87.7 14\nHummel, Cooper 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '608717', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Chris 35 31 10 3 0 1 4 11.4 9.4 4 11.4 0.323 0.516 0.391 0.362 0.506 0.416 91 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 7 6 4 1 0 1 0 0 0 1 14.3 0.667 1.333 0.802 0.792 1.06 0.77 92.4 17.3\nAltuve, Jose 7 5 3 1 0 0 0 0 0 2 28.6 0.6 0.8 0.621 0.508 0.565 0.542 92.6 -8\nCaratini, Victor 6 6 2 1 0 0 0 0 11.1 0 0 0.333 0.5 0.357 0.344 0.406 0.332 94.9 7.2\nDubón, Mauricio 4 4 1 0 0 0 0 0 10 0 0 0.25 0.25 0.221 0.365 0.697 0.447 91.7 25.3\nPeña, Jeremy 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.191 0.211 0.176 74.1 21.3\nMcCormick, Chas 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.223 0.461 0.282 100.7 20\nDiaz, Yainer 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.09 0.329 0.162 100.4 38\nAlvarez, Yordan 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.238 0.256 0.464 88.8 -3'}, {'pitcher_id': '694361', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672582', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZerpa, Angel 10 10 2 1 0 0 3 30 25 0 0 0.2 0.3 0.213 0.375 0.596 0.414 101.4 3.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.568 0.757 0.572 101.6 7.5\nBregman, Alex 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.071 0.082 0.065 103.9 -26\nAlvarez, Yordan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.776 1.621 1.002 102.2 18.5\nAltuve, Jose 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSingleton, Jon 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nMcCormick, Chas 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.657 0.751 0.613 100.5 8\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.267 0.293 0.249 98 -8'}, {'pitcher_id': '663903', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSinger, Brady 48 46 11 1 0 2 11 22.9 16.9 2 4.2 0.239 0.391 0.289 0.272 0.475 0.332 91.3 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 12 12 2 0 0 1 4 33.3 22.2 0 0 0.167 0.417 0.246 0.182 0.451 0.268 95.9 18.9\nAltuve, Jose 12 12 3 1 0 0 1 8.3 5.9 0 0 0.25 0.333 0.253 0.341 0.455 0.341 85.5 8.2\nMeyers, Jake 6 5 2 0 0 0 2 33.3 25 1 16.7 0.4 0.4 0.408 0.353 0.416 0.393 96.6 9\nBregman, Alex 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.188 0.211 0.176 92.9 16\nDiaz, Yainer 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.612 2.124 1.101 101.6 9.5\nPeña, Jeremy 4 4 1 0 0 0 1 25 40 0 0 0.25 0.25 0.222 0.2 0.269 0.198 89.6 7\nMcCormick, Chas 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.293 0.28 0.382 0.284 87.9 -12.5\nDubón, Mauricio 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.444 0.269 0.291 0.242 90.7 9\nCaratini, Victor 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.031 0.033 0.028 72.5 -2\nSingleton, Jon 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.5 1.112 0.689 104 19'}, {'pitcher_id': '681882', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663704', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcArthur, James 19 19 3 0 0 0 6 31.6 34.5 0 0 0.158 0.158 0.14 0.164 0.204 0.161 81.8 2.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.222 0.209 0.236 0.195 89.3 15\nAltuve, Jose 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.578 0.641 0.542 65.8 -27.5\nBregman, Alex 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.202 0.347 0.239 81.6 -15.7\nAlvarez, Yordan 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nCaratini, Victor 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nDiaz, Yainer 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.036 0.027 83.4 31.3\nDubón, Mauricio 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMeyers, Jake 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMcCormick, Chas 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.253 0.193 84.6 -4'}, {'pitcher_id': '685107', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666142', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRagans, Cole 86 73 21 4 1 2 14 16.3 27.2 12 14 0.288 0.452 0.365 0.272 0.461 0.364 86.4 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 15 14 4 0 0 0 3 20 38.1 1 6.7 0.286 0.286 0.282 0.28 0.325 0.293 83.5 -13.6\nBregman, Alex 15 10 2 0 0 1 1 6.7 13 5 33.3 0.2 0.5 0.428 0.233 0.434 0.417 90.5 23.7\nAltuve, Jose 14 10 1 0 0 0 1 7.1 11.8 4 28.6 0.1 0.1 0.261 0.173 0.247 0.328 76.5 27.1\nAlvarez, Yordan 12 11 6 3 0 0 1 8.3 16.7 1 8.3 0.545 0.818 0.593 0.388 0.734 0.49 93.9 -0.4\nMcCormick, Chas 9 8 1 0 0 0 4 44.4 47.8 1 11.1 0.125 0.125 0.176 0.131 0.333 0.248 81.2 13\nDubón, Mauricio 10 10 4 1 1 0 1 10 15 0 0 0.4 0.7 0.46 0.428 0.617 0.451 89.7 12.6\nDiaz, Yainer 7 6 2 0 0 1 2 28.6 42.9 0 0 0.333 0.833 0.413 0.262 0.748 0.385 88.7 3.2\nMeyers, Jake 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.221 0.184 0.215 0.175 83.5 38'}, {'pitcher_id': '672578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Carlos 23 20 8 3 0 0 0 0 7.1 2 8.7 0.4 0.55 0.445 0.319 0.504 0.4 87.7 15.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Yordan 6 5 2 1 0 0 0 0 0 1 16.7 0.4 0.6 0.47 0.345 0.682 0.478 92.1 16.4\nMcCormick, Chas 5 4 2 1 0 0 0 0 33.3 1 20 0.5 0.75 0.564 0.401 0.77 0.538 97.7 21.3\nAltuve, Jose 4 4 3 1 0 0 0 0 0 0 0 0.75 1 0.751 0.312 0.352 0.291 80.1 5.5\nCaratini, Victor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.003 83.8 66\nBregman, Alex 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.419 0.475 0.395 85.6 16.3\nMeyers, Jake 2 1 0 0 0 0 0 0 0 0 0 0 0 0.361 0.121 0.136 0.421 84.8 -13\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.366 0.56 0.393 90.8 20\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.061 0.061 0.059 66.4 -9'}, {'pitcher_id': '608379', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWacha, Michael 18 17 4 2 0 1 5 27.8 17.5 0 0 0.235 0.529 0.296 0.292 0.513 0.343 85.5 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 6 6 1 1 0 0 1 16.7 8.3 0 0 0.167 0.333 0.203 0.386 0.551 0.397 98.1 14.8\nAltuve, Jose 5 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.174 0.174 0.184 0.205 71.7 22.4\nAlvarez, Yordan 4 4 2 1 0 1 2 50 27.3 0 0 0.5 1.5 0.812 0.474 1.151 0.673 94.9 21\nMcCormick, Chas 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.032 0.034 0.043 73.1 -28\nMontero, Rafael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '679525', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Alec 19 17 3 0 0 1 5 26.3 25.8 0 0 0.176 0.353 0.25 0.235 0.524 0.338 81 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.233 0.252 0.213 77.6 -15.3\nPeña, Jeremy 3 2 0 0 0 0 2 66.7 50 0 0 0 0 0 0 0 0 11.3 44\nAlvarez, Yordan 2 1 1 0 0 1 0 0 0 0 0 1 4 1.365 0.96 3.811 1.318 109.2 28\nBregman, Alex 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.184 0.678 0.344 98.4 35\nDubón, Mauricio 3 3 1 0 0 0 0 0 12.5 0 0 0.333 0.333 0.294 0.372 0.514 0.381 97.4 5.3\nDiaz, Yainer 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.2 0.227 0.187 72.3 8\nMcCormick, Chas 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.037 0.039 0.035 66.7 -12'}], 119: [{'pitcher_id': '680742', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBowlan, Jonathan 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.251 0.279 0.459 73 40', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.251 0.279 0.459 73 40'}, {'pitcher_id': '623433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Nick 31 30 7 3 0 1 6 19.4 20.3 1 3.2 0.233 0.433 0.291 0.271 0.533 0.346 90.3 15.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 5 5 0 0 0 0 1 20 33.3 0 0 0 0 0 0.264 0.416 0.298 91.3 18.8\nBetts, Mookie 4 4 2 1 0 0 1 25 20 0 0 0.5 0.75 0.527 0.371 0.439 0.342 95.9 9.7\nFreeman, Freddie 5 5 1 0 0 0 1 20 9.1 0 0 0.2 0.2 0.174 0.284 0.546 0.337 95.6 18.5\nSmith, Will 4 4 2 0 0 1 2 50 33.3 0 0 0.5 1.25 0.715 0.339 0.919 0.515 101 17.5\nBiggio, Cavan 4 4 1 1 0 0 0 0 14.3 0 0 0.25 0.5 0.31 0.235 0.494 0.301 84.5 21.3\nTaylor, Chris 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.619 0.567 1.651 0.892 100.5 25.5\nHernández, Enrique 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.146 0.146 0.133 66.1 14\nHeyward, Jason 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.074 0.077 0.077 87.6 -20\nRojas, Miguel 1 0 0 0 0 0 0 0 1 100 0.699 0.699\nLux, Gavin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBarnes, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.027 0.021 83.6 43'}, {'pitcher_id': '519293', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Will 71 63 16 2 0 3 22 31 31.2 6 8.5 0.254 0.429 0.335 0.276 0.547 0.378 88.5 16.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeyward, Jason 15 12 3 0 0 0 4 26.7 34.5 2 13.3 0.25 0.25 0.316 0.205 0.227 0.283 80.2 4.7\nFreeman, Freddie 10 9 2 1 0 0 3 30 27.3 1 10 0.222 0.333 0.284 0.406 0.664 0.457 82.6 22.4\nTaylor, Chris 9 7 1 0 0 0 4 44.4 52.9 2 22.2 0.143 0.143 0.251 0.082 0.098 0.217 89.6 9.7\nBarnes, Austin 6 5 2 0 0 0 1 16.7 7.1 1 16.7 0.4 0.4 0.407 0.412 0.449 0.425 86.6 7\nSmith, Will 7 7 1 0 0 1 3 42.9 46.2 0 0 0.143 0.571 0.283 0.28 1.035 0.53 99.3 34.3\nHernández, Teoscar 4 4 1 0 0 0 3 75 40 0 0 0.25 0.25 0.22 0.015 0.022 0.015 77.9 49\nRojas, Miguel 6 5 1 0 0 0 1 16.7 25 0 0 0.2 0.2 0.267 0.376 0.555 0.451 92 24\nHernández, Enrique 4 4 2 0 0 1 1 25 20 0 0 0.5 1.25 0.724 0.523 1.175 0.708 105.3 8.7\nBetts, Mookie 4 4 1 0 0 0 1 25 12.5 0 0 0.25 0.25 0.222 0.136 0.151 0.128 83.4 15\nOhtani, Shohei 4 4 1 0 0 1 1 25 33.3 0 0 0.25 1 0.501 0.354 1.119 0.596 96.2 28.7\nBiggio, Cavan 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.621 0.366 1.109 0.597 82.1 3'}, {'pitcher_id': '669674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLong, Sam 37 33 9 0 1 1 6 16.2 29.7 2 5.4 0.273 0.424 0.322 0.304 0.448 0.35 87.2 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 8 8 4 0 1 0 0 0 16.7 0 0 0.5 0.75 0.531 0.451 0.541 0.436 82.6 8.1\nSmith, Will 5 4 1 0 0 0 1 20 33.3 0 0 0.25 0.25 0.321 0.279 0.368 0.375 83.5 8\nBetts, Mookie 6 5 0 0 0 0 0 0 0 0 0 0 0 0 0.15 0.301 0.171 95.8 45\nTaylor, Chris 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.352 0.386 0.321 93.4 4.5\nOhtani, Shohei 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.231 0.002 0.002 0.232 83.7 65\nLux, Gavin 3 3 2 0 0 0 1 33.3 20 0 0 0.667 0.667 0.589 0.168 0.176 0.155 79 -27.5\nHernández, Teoscar 2 2 1 0 0 1 1 50 66.7 0 0 0.5 2 1.002 0.466 1.708 0.876 106 24\nRojas, Miguel 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.403 0.428 0.367 75.1 -23\nHernández, Enrique 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.441 0.222 0.307 0.228 90.4 11.5\nBiggio, Cavan 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nPages, Andy 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.666 0.753 0.625 99.6 9\nBuehler, Walker 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '670167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchreiber, John 19 17 4 1 0 0 5 26.3 28.6 2 10.5 0.235 0.294 0.278 0.255 0.378 0.313 89 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.392 0.421 0.359 79.7 -11.7\nHernández, Teoscar 3 3 0 0 0 0 2 66.7 75 0 0 0 0 0 0.117 0.131 0.105 74.3 32\nBetts, Mookie 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.287 0.815 0.531 95.4 49\nBarnes, Austin 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nHeyward, Jason 2 2 2 1 0 0 0 0 25 0 0 1 1.5 1.06 0.445 0.596 0.435 97.9 10.5\nTaylor, Chris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSmith, Will 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nLux, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.194 0.334 0.218 93.2 22\nHernández, Enrique 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.001 0.001 0 81.7 70\nRojas, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.226 0.293 0.212 86.3 23\nFreeman, Freddie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.926 1.324 0.971 106.9 12'}, {'pitcher_id': '663738', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynch IV, Daniel 34 26 3 0 0 1 4 11.8 26.3 7 20.6 0.115 0.231 0.253 0.247 0.565 0.409 90.5 17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 6 5 1 0 0 0 2 33.3 26.7 1 16.7 0.2 0.2 0.262 0.275 0.34 0.337 93 13\nHernández, Teoscar 8 7 1 0 0 1 1 12.5 36.4 1 12.5 0.143 0.571 0.337 0.289 0.868 0.5 94.4 30\nBetts, Mookie 6 3 0 0 0 0 0 0 28.6 2 33.3 0 0 0.231 0.164 0.205 0.351 84.8 15.5\nSmith, Will 6 6 0 0 0 0 0 0 10 0 0 0 0 0 0.242 0.844 0.452 91.9 26.3\nRojas, Miguel 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.33 0.375 0.309 85.7 -1.7\nOhtani, Shohei 3 1 1 0 0 0 0 0 33.3 2 66.7 1 1 0.754 0.079 0.101 0.484 89.3 -36\nTaylor, Chris 2 1 0 0 0 0 1 50 75 1 50 0 0 0.344 0.344'}, {'pitcher_id': '607625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLugo, Seth 100 92 24 5 0 3 18 18 19.9 7 7 0.261 0.413 0.32 0.302 0.493 0.368 89.7 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 18 16 5 1 0 0 1 5.6 8.1 2 11.1 0.313 0.375 0.343 0.412 0.579 0.457 92.4 6.7\nHeyward, Jason 13 10 3 1 0 0 3 23.1 36.4 2 15.4 0.3 0.4 0.393 0.246 0.333 0.353 82.8 -2\nBetts, Mookie 8 7 3 0 0 1 2 25 11.8 1 12.5 0.429 0.857 0.558 0.328 0.668 0.458 92.3 11.2\nRojas, Miguel 14 14 3 0 0 0 2 14.3 14.3 0 0 0.214 0.214 0.189 0.241 0.303 0.238 84.5 16\nBiggio, Cavan 7 7 1 0 0 0 4 57.1 26.7 0 0 0.143 0.143 0.126 0.11 0.12 0.102 83.9 0\nTaylor, Chris 6 6 2 1 0 1 2 33.3 25 0 0 0.333 1 0.54 0.4 1.135 0.632 90.4 36.3\nOhtani, Shohei 6 6 0 0 0 0 0 0 30.8 0 0 0 0 0 0.257 0.28 0.238 95 -13.3\nHernández, Teoscar 7 7 2 0 0 0 1 14.3 29.4 0 0 0.286 0.286 0.254 0.336 0.552 0.377 89.2 9\nSmith, Will 6 6 2 0 0 1 1 16.7 23.1 0 0 0.333 0.833 0.492 0.343 0.73 0.455 94.2 24.2\nHernández, Enrique 3 2 1 1 0 0 0 0 25 1 33.3 0.5 1 0.638 0.23 0.412 0.408 96.2 7\nLux, Gavin 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.178 0.333 0.514 0.359 100.4 9.3\nBarnes, Austin 4 3 0 0 0 0 1 25 0 1 25 0 0 0.172 0.321 0.353 0.392 76.9 11.5\nPages, Andy 3 3 1 1 0 0 0 0 20 0 0 0.333 0.667 0.423 0.292 0.596 0.371 85.2 10.7'}, {'pitcher_id': '608717', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Chris 72 64 20 4 1 4 14 19.4 22.2 7 9.7 0.313 0.594 0.404 0.277 0.547 0.382 90.5 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Chris 18 15 5 1 0 2 4 22.2 34.2 3 16.7 0.333 0.8 0.506 0.339 0.786 0.505 92.7 21.8\nHeyward, Jason 15 12 3 1 0 0 4 26.7 25 2 13.3 0.25 0.333 0.29 0.215 0.251 0.289 86.7 0.4\nHernández, Enrique 6 6 3 0 1 0 1 16.7 7.7 0 0 0.5 0.833 0.556 0.39 0.847 0.516 99.1 19.4\nBarnes, Austin 5 4 0 0 0 0 1 20 14.3 1 20 0 0 0.138 0.01 0.013 0.146 85.3 49.7\nFreeman, Freddie 5 5 3 2 0 1 0 0 0 0 0 0.6 1.6 0.904 0.5 1.471 0.808 98.8 34.6\nRojas, Miguel 6 6 0 0 0 0 0 0 18.2 0 0 0 0 0 0.279 0.326 0.264 87.2 21.2\nSmith, Will 4 4 2 0 0 0 1 25 20 0 0 0.5 0.5 0.44 0.335 0.392 0.317 84.7 21.7\nBetts, Mookie 4 4 1 0 0 1 1 25 44.4 0 0 0.25 1 0.501 0.193 0.63 0.337 94.8 28.3\nHernández, Teoscar 2 1 1 0 0 0 0 0 14.3 1 50 1 1 0.789 0.306 0.334 0.488 72.7 11\nLux, Gavin 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.155 0.236 0.166 94.7 22\nBuehler, Walker 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.44 0.263 0.285 0.239 101.3 3\nOhtani, Shohei 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.036 0.041 0.038 73 18.5'}, {'pitcher_id': '694361', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672582', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZerpa, Angel 8 8 2 0 0 1 6 75 46.7 0 0 0.25 0.625 0.369 0.177 0.515 0.292 108 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 5 5 1 0 0 0 4 80 50 0 0 0.2 0.2 0.177 0.108 0.121 0.102 106.3 3\nOhtani, Shohei 2 2 1 0 0 1 1 50 0 0 0 0.5 2 1.036 0.44 1.758 0.916 109.6 33\nBiggio, Cavan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663903', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSinger, Brady 41 37 11 3 0 1 5 12.2 23.7 4 9.8 0.297 0.459 0.359 0.289 0.443 0.351 86.4 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 6 5 0 0 0 0 1 16.7 26.7 1 16.7 0 0 0.116 0.161 0.213 0.251 87.8 16.5\nBiggio, Cavan 6 5 2 1 0 0 2 33.3 22.2 1 16.7 0.4 0.6 0.475 0.21 0.277 0.29 93.5 12.7\nOhtani, Shohei 6 5 2 1 0 0 0 0 9.1 1 16.7 0.4 0.6 0.47 0.384 0.461 0.428 93.8 -10.4\nBetts, Mookie 7 7 1 1 0 0 1 14.3 22.2 0 0 0.143 0.286 0.178 0.127 0.258 0.158 73.4 16\nHeyward, Jason 5 4 2 0 0 1 0 0 22.2 1 20 0.5 1.25 0.712 0.493 1.262 0.717 87.6 -7.5\nBarnes, Austin 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.307 0.36 0.292 92.6 26.5\nHernández, Enrique 2 2 2 0 0 0 0 0 33.3 0 0 1 1 0.884 0.702 0.79 0.664 87.1 11\nTaylor, Chris 2 2 1 0 0 0 0 0 60 0 0 0.5 0.5 0.442 0.267 0.293 0.251 84.8 20.5\nLux, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.431 0.502 0.399 88.1 14\nHernández, Teoscar 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.176 0.266 0.182 84.2 17'}, {'pitcher_id': '681882', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663704', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcArthur, James 5 5 1 0 0 0 2 40 57.1 0 0 0.2 0.2 0.177 0.142 0.153 0.13 93.6 -8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBiggio, Cavan 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.091 0.093 0.084 97.9 -16.5\nHernández, Teoscar 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.218 0.242 0.199 85.1 8'}, {'pitcher_id': '685107', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666142', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRagans, Cole 36 34 7 0 0 3 7 19.4 24.2 2 5.6 0.206 0.471 0.309 0.241 0.457 0.316 89.6 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOhtani, Shohei 11 10 3 0 0 1 2 18.2 31.6 1 9.1 0.3 0.6 0.407 0.424 0.774 0.522 95.9 6.4\nHernández, Teoscar 3 2 0 0 0 0 2 66.7 33.3 1 33.3 0 0 0.231 0.231\nPages, Andy 3 3 1 0 0 0 0 0 12.5 0 0 0.333 0.333 0.296 0.33 0.358 0.302 81.6 35.3\nRojas, Miguel 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.991 0.372 0.949 0.548 97 26\nBetts, Mookie 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.128 0.188 0.13 97.4 11\nSmith, Will 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.085 0.185 0.11 92.8 24\nHernández, Enrique 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.032 0.053 0.038 89 11.5\nTaylor, Chris 2 2 1 0 0 1 0 0 20 0 0 0.5 2 1.043 0.39 1.125 0.627 87.8 22.5\nFreeman, Freddie 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.049 0.057 0.047 81.6 2.7\nBiggio, Cavan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.085 0.091 0.076 45.9 10'}, {'pitcher_id': '672578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Carlos 22 17 3 2 0 0 3 13.6 24.3 5 22.7 0.176 0.294 0.311 0.22 0.315 0.334 87 26.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Enrique 3 2 0 0 0 0 0 0 22.2 1 33.3 0 0 0.231 0.007 0.009 0.236 75.8 74\nHeyward, Jason 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.231 0.158 0.178 0.335 80.4 21\nLux, Gavin 2 1 0 0 0 0 0 0 25 1 50 0 0 0.344 0.003 0.003 0.346 86.9 68\nBiggio, Cavan 2 2 1 1 0 0 1 50 40 0 0 0.5 1 0.631 0.174 0.261 0.183 94.8 19\nFreeman, Freddie 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.23 0.351 0.401 0.445 101 9\nHernández, Teoscar 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.107 0.167 0.114 89.1 23\nSmith, Will 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.295 0.369 0.423 0.341 80.7 -15\nOhtani, Shohei 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nBetts, Mookie 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.279 0.524 0.334 95 41.5\nTaylor, Chris 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.483 1.009 0.627 102.2 20'}, {'pitcher_id': '608379', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWacha, Michael 98 87 33 6 0 6 16 16.3 22.1 11 11.2 0.379 0.655 0.465 0.359 0.642 0.451 93.1 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFreeman, Freddie 20 15 8 1 0 1 2 10 18.6 5 25 0.533 0.8 0.599 0.476 0.756 0.567 96.1 9.4\nHernández, Enrique 20 20 8 1 0 1 6 30 34 0 0 0.4 0.6 0.423 0.26 0.44 0.293 89.5 15.2\nHeyward, Jason 21 20 3 2 0 0 2 9.5 14.6 1 4.8 0.15 0.25 0.192 0.255 0.437 0.309 93.3 15.6\nHernández, Teoscar 12 11 6 1 0 2 2 16.7 28.6 1 8.3 0.545 1.182 0.729 0.359 0.963 0.569 91.5 18.1\nTaylor, Chris 5 4 2 0 0 0 1 20 11.1 1 20 0.5 0.5 0.489 0.522 0.582 0.536 96.8 6\nOhtani, Shohei 6 6 2 0 0 0 1 16.7 23.1 0 0 0.333 0.333 0.295 0.531 0.649 0.52 94.3 8.6\nSmith, Will 5 4 3 1 0 1 0 0 14.3 1 20 0.75 1.75 0.963 0.635 1.438 0.826 99.6 7.8\nBuehler, Walker 2 2 1 0 0 1 0 0 0 0 0 0.5 2 0.97 0.525 1.66 0.865 101.2 3.5\nBetts, Mookie 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.137 0.191 0.141 80.5 26.5\nPaxton, James 2 1 0 0 0 0 1 50 0 1 50 0 0 0.345\nRojas, Miguel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.345 0.192 0.198 0.437 81.6 0'}, {'pitcher_id': '679525', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Alec 22 19 8 1 1 3 5 22.7 30.6 3 13.6 0.421 1.053 0.616 0.342 0.848 0.52 90.4 20.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Teoscar 6 5 2 0 0 1 2 33.3 50 1 16.7 0.4 1 0.597 0.43 1.09 0.639 87.1 -10.7\nBiggio, Cavan 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.111 0.123 0.298 88.5 -3\nFreeman, Freddie 3 2 0 0 0 0 1 33.3 42.9 1 33.3 0 0 0.232 0.02 0.029 0.246 80.8 54\nSmith, Will 3 3 1 0 1 0 0 0 0 0 0 0.333 1 0.523 0.078 0.149 0.095 87.2 32\nBetts, Mookie 3 3 3 0 0 2 0 0 0 0 0 1 3 1.63 0.916 2.724 1.491 103.6 22.7\nRojas, Miguel 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.003 0.004 0.004 58 75\nHeyward, Jason 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.063 0.553 0.861 0.603 101.9 14'}]}, 119: {115: [{'pitcher_id': '642232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYarbrough, Ryan 30 29 10 4 0 2 8 26.7 28.6 0 0 0.345 0.69 0.441 0.206 0.38 0.262 83.8 14.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoyle, Brenton 3 3 1 1 0 0 1 33.3 20 0 0 0.333 0.667 0.415 0.185 0.275 0.19 92.9 10.5\nJones, Nolan 4 4 1 1 0 0 1 25 33.3 0 0 0.25 0.5 0.311 0.244 0.662 0.375 86.6 6.7\nMontero, Elehuris 4 4 2 1 0 1 2 50 37.5 0 0 0.5 1.5 0.812 0.207 0.617 0.336 86.1 22.5\nTovar, Ezequiel 4 4 3 0 0 0 1 25 50 0 0 0.75 0.75 0.662 0.281 0.313 0.258 71 17\nMcMahon, Ryan 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.004 0.006 0.004 87.6 56\nTrejo, Alan 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.231 0.235 0.208 69.6 25\nBlackmon, Charlie 5 4 2 1 0 1 0 0 0 0 0 0.5 1.5 0.793 0.212 0.484 0.381 89.2 2.5\nCave, Jake 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.435 0.36 0.425 0.339 101.8 9\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.152 0.091 91.5 27\nToglia, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.13 0.154 0.117 82.6 -8'}, {'pitcher_id': '681402', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Gus 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.129 0.253 0.147 97.3 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStallings, Jacob 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.129 0.253 0.147 97.3 23'}, {'pitcher_id': '607192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGlasnow, Tyler 4 4 1 0 0 1 1 25 10 0 0 0.25 1 0.508 0.264 0.792 0.438 85.5 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 1 1 1 0 0 1 0 0 0 0 0 1 4 2.031 0.747 2.332 1.273 102 24\nStallings, Jacob 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.022 0.022 0.02 69 -9\nGomber, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0'}, {'pitcher_id': '621111', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBuehler, Walker 103 95 28 7 1 4 25 24.3 25.8 7 6.8 0.295 0.516 0.36 0.282 0.446 0.336 90.6 8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 54 50 18 4 1 2 9 16.7 20 3 5.6 0.36 0.6 0.41 0.358 0.533 0.396 88.7 6.2\nMcMahon, Ryan 32 30 6 3 0 1 13 40.6 37.3 2 6.3 0.2 0.4 0.276 0.176 0.285 0.228 95.8 2.3\nDoyle, Brenton 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.228 0.607 0.461 95.2 28.5\nQuantrill, Cal 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.435 0.483 0.53 0.437 79.3 18\nCave, Jake 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.11 0.123 0.1 95 -3\nStallings, Jacob 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.184 0.303 0.206 84.8 40.7\nMontero, Elehuris 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.009 0.018 0.239 92 43.5\nTovar, Ezequiel 3 3 3 0 0 1 0 0 25 0 0 1 2 1.288 0.444 1.233 0.687 92 10.3'}, {'pitcher_id': '670990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamírez, Yohan 14 10 2 0 1 1 3 21.4 27.8 3 21.4 0.2 0.7 0.457 0.189 0.4 0.372 92.8 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 4 2 1 0 0 1 1 25 75 2 50 0.5 2 0.843 0.366 1.202 0.66 102.1 25\nMcMahon, Ryan 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.064 0.177 0.092 86.3 10\nMontero, Elehuris 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.002 0.002 0.002 95.3 49\nDoyle, Brenton 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nTovar, Ezequiel 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.25 0.274 0.477 94.2 -7\nCave, Jake 1 1 1 0 1 0 0 0 0 0 0 1 3 1.612 0.386 0.42 0.347 97 -2\nTrejo, Alan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.326 0.367 0.294 88.3 2'}, {'pitcher_id': '694813', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStone, Gavin 16 15 3 1 0 0 5 31.3 27.8 1 6.3 0.2 0.267 0.234 0.235 0.347 0.281 94.4 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTovar, Ezequiel 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.246 0.275 0.231 89.4 -2.5\nBlackmon, Charlie 3 3 1 1 0 0 1 33.3 33.3 0 0 0.333 0.667 0.423 0.077 0.115 0.081 94.1 5.5\nGoodman, Hunter 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.277 0.827 0.457 96.2 17\nDoyle, Brenton 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nMcMahon, Ryan 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nTrejo, Alan 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.33 0.401 0.327 91.9 -9.5\nStallings, Jacob 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.667 0.792 0.651 100.6 6.5'}, {'pitcher_id': '676568', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVesia, Alex 27 26 1 0 0 0 8 29.6 30.4 1 3.7 0.038 0.038 0.058 0.151 0.197 0.171 83.3 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 8 8 1 0 0 0 0 0 15.4 0 0 0.125 0.125 0.11 0.346 0.412 0.336 77.2 7\nMcMahon, Ryan 5 5 0 0 0 0 2 40 44.4 0 0 0 0 0 0.101 0.185 0.118 86.4 10.3\nTrejo, Alan 3 3 0 0 0 0 3 100 33.3 0 0 0 0 0 0\nStallings, Jacob 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.003 0.003 0.346 86.4 65\nJones, Nolan 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.035 0.101 0.054 90.9 31\nToglia, Michael 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.018 0.051 0.028 94.5 41\nTovar, Ezequiel 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.177 0.191 0.16 84.4 -2\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 89 42\nMontero, Elehuris 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 93.4 46'}, {'pitcher_id': '518489', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrasier, Ryan 15 13 1 0 0 0 4 26.7 22.6 2 13.3 0.077 0.077 0.151 0.084 0.139 0.173 86 15.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.145 0.23 0.153 85.8 4\nToglia, Michael 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.01 0.021 0.013 100.7 49\nJones, Nolan 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.232 0.06 0.061 0.269 77.2 -1\nCave, Jake 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.091 0.162 0.103 79.1 6.5\nTovar, Ezequiel 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.24 0.449 0.291 94.5 21\nMcMahon, Ryan 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nDoyle, Brenton 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMontero, Elehuris 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.018 0.011 86.1 42'}, {'pitcher_id': '641401', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrogdon, Connor 11 10 1 0 0 1 4 36.4 37.9 1 9.1 0.1 0.4 0.252 0.266 0.61 0.396 100.1 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 4 3 0 0 0 0 3 75 60 1 25 0 0 0.174 0.174\nStallings, Jacob 1 1 1 0 0 1 0 0 28.6 0 0 1 4 2.072 0.943 3.581 1.885 106.1 25\nTrejo, Alan 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.354 0.704 0.429 99 20.5\nBlackmon, Charlie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.526 0.571 0.478 100.6 3\nDoyle, Brenton 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nTovar, Ezequiel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.241 0.268 0.22 98.1 25'}, {'pitcher_id': '623465', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPhillips, Evan 15 14 1 0 0 0 5 33.3 34.8 0 0 0.071 0.071 0.107 0.098 0.111 0.131 76.5 -5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.177 0.111 0.142 0.101 88.4 -8\nBlackmon, Charlie 3 2 0 0 0 0 1 33.3 40 0 0 0 0 0.241 0.034 0.035 0.261 74.3 -8\nCave, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.079 0.062 74.6 24\nTovar, Ezequiel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nStallings, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.07 0.063 77.2 -6\nJones, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.34 0.343 0.306 65.3 15\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.206 0.211 0.185 57.7 -72\nMontero, Elehuris 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '656420', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeyereisen, J.P. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '808967', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYamamoto, Yoshinobu 14 13 3 1 0 0 3 21.4 23.1 0 0 0.231 0.308 0.218 0.236 0.294 0.266 92 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.2 0.242 0.194 100.6 -5\nMontero, Elehuris 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.131 0.146 0.122 84.7 -2.5\nBlackmon, Charlie 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.318 0.467 0.331 81.3 32\nDoyle, Brenton 2 1 0 0 0 0 1 50 25 0 0 0 0 0 0 0 0.364 98.2 29\nTovar, Ezequiel 3 3 3 1 0 0 0 0 25 0 0 1 1.333 1.016 0.374 0.421 0.35 99.8 -3'}, {'pitcher_id': '676272', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bobby 17 17 6 1 0 1 8 47.1 38.9 0 0 0.353 0.588 0.399 0.206 0.336 0.231 90.3 2.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Nolan 3 3 2 0 0 0 1 33.3 33.3 0 0 0.667 0.667 0.588 0.338 0.386 0.319 80 16.5\nGoodman, Hunter 3 3 0 0 0 0 3 100 75 0 0 0 0 0 0\nBlackmon, Charlie 3 3 2 1 0 0 1 33.3 33.3 0 0 0.667 1 0.709 0.353 0.524 0.371 99.7 12.5\nMontero, Elehuris 3 3 2 0 0 1 1 33.3 16.7 0 0 0.667 1.667 0.962 0.39 0.909 0.537 103.4 24.5\nDoyle, Brenton 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.044 0.048 0.042 95.3 -36\nTovar, Ezequiel 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.054 0.056 0.052 75.8 -23'}, {'pitcher_id': '669165', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '595014', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTreinen, Blake 17 17 2 0 0 0 7 41.2 26.2 0 0 0.118 0.118 0.104 0.193 0.233 0.187 86.5 6.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 8 8 1 0 0 0 2 25 18.5 0 0 0.125 0.125 0.11 0.199 0.226 0.191 80.5 8.2\nMcMahon, Ryan 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.178 0.283 0.369 0.282 94 9\nCave, Jake 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.137 0.154 0.123 100.2 -10\nTovar, Ezequiel 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nStallings, Jacob 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '607455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanda, Anthony 5 3 1 0 0 0 1 20 11.1 2 40 0.333 0.333 0.454 0.371 0.713 0.556 102.5 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.441 0.437 0.585 0.444 104.5 13\nMontero, Elehuris 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nBlackmon, Charlie 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nTovar, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.24 0.971 0.503 100.5 38'}, {'pitcher_id': '523260', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Joe 21 19 2 0 0 0 8 38.1 47.6 2 9.5 0.105 0.105 0.149 0.165 0.212 0.219 92.3 -8.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 9 8 1 0 0 0 1 11.1 37.5 1 11.1 0.125 0.125 0.173 0.22 0.289 0.287 91.9 -8\nMcMahon, Ryan 6 6 1 0 0 0 3 50 50 0 0 0.167 0.167 0.147 0.235 0.296 0.232 95.4 -3.7\nCave, Jake 3 2 0 0 0 0 1 33.3 60 1 33.3 0 0 0.23 0.04 0.044 0.254 84.4 -23\nJones, Nolan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nTovar, Ezequiel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDoyle, Brenton 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '543339', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Daniel 14 13 4 0 0 0 3 21.4 16.7 1 7.1 0.308 0.308 0.301 0.336 0.441 0.36 90.2 28.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 9 8 2 0 0 0 1 11.1 15.4 1 11.1 0.25 0.25 0.272 0.368 0.507 0.409 91 33.7\nMcMahon, Ryan 3 3 2 0 0 0 1 33.3 16.7 0 0 0.667 0.667 0.589 0.301 0.34 0.279 81.6 23.5\nTrejo, Alan 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nDoyle, Brenton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.523 0.652 0.522 102.2 2'}, {'pitcher_id': '675627', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrove, Michael 28 25 6 0 0 1 6 21.4 17.1 3 10.7 0.24 0.36 0.306 0.219 0.298 0.275 88.2 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcMahon, Ryan 10 8 3 0 0 1 2 20 12.5 2 20 0.375 0.75 0.523 0.371 0.561 0.461 97.8 20.3\nTrejo, Alan 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.021 0.022 0.019 79.2 16.5\nStallings, Jacob 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.049 0.049 0.046 96.2 -10\nDoyle, Brenton 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.209 0.24 0.201 86.7 -7.5\nBlackmon, Charlie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.137 0.179 0.139 79.4 17\nJones, Nolan 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.469 0.567 0.578 92.8 3\nTovar, Ezequiel 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.25 0.304 0.239 83.4 36.3\nGoodman, Hunter 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.128 0.129 0.115 74 -45\nMontero, Elehuris 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.037 0.07 0.039 85.2 30'}, {'pitcher_id': '622694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Elieser 12 10 1 0 0 0 5 41.7 19.2 1 8.3 0.1 0.1 0.143 0.105 0.128 0.16 93.4 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStallings, Jacob 6 5 1 0 0 0 2 33.3 7.7 1 16.7 0.2 0.2 0.262 0.163 0.18 0.239 95.2 25.7\nBlach, Ty 3 2 0 0 0 0 1 33.3 0 0 0 0 0 0 0\nHudson, Dakota 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nBlackmon, Charlie 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.127 0.247 0.159 88.2 25'}, {'pitcher_id': '598286', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamirez, Nick 8 8 3 0 0 2 2 25 30.8 0 0 0.375 1.125 0.608 0.402 0.993 0.577 95.2 19.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCave, Jake 4 4 1 0 0 1 2 50 50 0 0 0.25 1 0.495 0.227 0.481 0.3 86.9 24\nBlackmon, Charlie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.195 0.223 0.179 94.6 -10\nJones, Nolan 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 1 4 2 109.5 28\nTrejo, Alan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.931 1.141 0.913 92.6 13\nToglia, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.18 0.658 0.324 100.4 38'}, {'pitcher_id': '689017', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '572020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaxton, James 27 25 6 1 0 1 9 33.3 34 2 7.4 0.24 0.4 0.299 0.168 0.202 0.201 90.4 7.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackmon, Charlie 12 11 4 0 0 1 4 33.3 33.3 1 8.3 0.364 0.636 0.437 0.168 0.217 0.209 88.2 9.4\nMcMahon, Ryan 5 5 1 1 0 0 1 20 0 0 0 0.2 0.4 0.249 0.251 0.298 0.244 90 5.5\nJones, Nolan 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.081 0.082 0.282 94 -11\nTovar, Ezequiel 3 3 0 0 0 0 1 33.3 44.4 0 0 0 0 0 0.149 0.157 0.136 93.2 -5.5\nDoyle, Brenton 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.232 0.251 0.209 102.8 -2\nMontero, Elehuris 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.007 0.013 0.008 86.1 55'}], 116: [{'pitcher_id': '642232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYarbrough, Ryan 32 32 9 3 0 0 7 21.9 23.4 0 0 0.281 0.375 0.282 0.263 0.409 0.287 83.9 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 10 10 3 2 0 0 2 20 21.1 0 0 0.3 0.5 0.336 0.154 0.172 0.144 77.6 -9.6\nCanha, Mark 6 6 1 0 0 0 1 16.7 15.4 0 0 0.167 0.167 0.147 0.278 0.452 0.309 75.4 39.2\nRogers, Jake 5 5 0 0 0 0 3 60 45.5 0 0 0 0 0 0.189 0.699 0.358 90.6 37.5\nBaddoo, Akil 3 3 2 0 0 0 0 0 16.7 0 0 0.667 0.667 0.586 0.542 0.629 0.514 89.1 24.3\nVierling, Matt 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.376 0.442 0.353 90.4 18\nGreene, Riley 3 3 3 1 0 0 0 0 0 0 0 1 1.333 1.003 0.504 0.644 0.495 98.4 -1.3\nIbáñez, Andy 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.002 0.002 0.002 85.7 56'}, {'pitcher_id': '681402', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Gus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.167 0.134 96.6 -18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.167 0.134 96.6 -18'}, {'pitcher_id': '607192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGlasnow, Tyler 25 19 2 1 0 0 8 32 26.7 5 20 0.105 0.158 0.223 0.16 0.337 0.295 88 24.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 17 14 1 1 0 0 4 23.5 26.7 2 11.8 0.071 0.143 0.155 0.209 0.448 0.306 89.4 27.4\nCanha, Mark 6 4 1 0 0 0 3 50 27.3 2 33.3 0.25 0.25 0.377 0.029 0.032 0.249 72.8 -5\nKelly, Carson 1 0 0 0 0 0 0 0 33.3 1 100 0.69 0.69\nFlaherty, Jack 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '621111', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBuehler, Walker 22 20 3 0 0 0 5 22.7 27.8 1 4.5 0.15 0.15 0.184 0.254 0.364 0.304 85.7 19.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 16 14 2 0 0 0 3 18.8 29.2 1 6.3 0.143 0.143 0.198 0.26 0.341 0.312 88.4 15.4\nFlaherty, Jack 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.109 0.152 0.117 72.4 39\nBaddoo, Akil 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.296 0.444 0.318 69.9 34.5\nCanha, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.533 1.167 0.715 100.7 21'}, {'pitcher_id': '670990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamírez, Yohan 16 13 1 0 0 0 4 25 23.1 3 18.8 0.077 0.077 0.185 0.197 0.252 0.287 90.7 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 4 3 0 0 0 0 1 25 25 1 25 0 0 0.175 0.136 0.21 0.283 91.1 46\nKelly, Carson 4 3 0 0 0 0 0 0 0 1 25 0 0 0.172 0.286 0.304 0.368 83.2 9.7\nMcKinstry, Zach 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.529 0.633 0.497 99.7 0.5\nIbáñez, Andy 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.077 0.155 0.104 92.6 -5.5\nUrshela, Gio 1 0 0 0 0 0 0 0 66.7 1 100 0.689 0.689\nBaddoo, Akil 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVierling, Matt 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '694813', 'player_data': '', 'team_data': 'Gavin Stone has not faced anyone on this team.'}, {'pitcher_id': '676568', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVesia, Alex 4 3 0 0 0 0 3 75 62.5 1 25 0 0 0.174 0.174', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 3 2 0 0 0 0 2 66.7 80 1 33.3 0 0 0.232 0.232\nBaddoo, Akil 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '518489', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrasier, Ryan 16 14 4 0 0 0 3 18.8 20 1 6.3 0.286 0.286 0.31 0.424 0.528 0.451 91.9 19.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 6 6 1 0 0 0 1 16.7 11.8 0 0 0.167 0.167 0.147 0.457 0.55 0.439 88.5 15.6\nMcKinstry, Zach 1 0 0 0 0 0 0 0 25 1 100 0.696 0.696\nRogers, Jake 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.129 0.451 0.473 100.9 40\nGreene, Riley 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.977 1.1 0.921 79.4 18\nVierling, Matt 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.025 0.036 0.026 74.8 56\nKreidler, Ryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nKelly, Carson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.914 1.208 0.911 104.8 12\nBaddoo, Akil 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.563 0.635 0.524 104.2 3'}, {'pitcher_id': '641401', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrogdon, Connor 4 4 1 0 0 1 2 50 22.2 0 0 0.25 1 0.518 0.024 0.081 0.044 84.6 53', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.036 0.045 0.161 0.087 94.1 35\nMcKinstry, Zach 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nKelly, Carson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 75.1 71'}, {'pitcher_id': '623465', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPhillips, Evan 10 8 2 0 0 0 2 20 14.3 2 20 0.25 0.25 0.316 0.278 0.297 0.352 86 5.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 4 3 1 0 0 0 1 25 20 1 25 0.333 0.333 0.394 0.138 0.139 0.268 98.5 -13\nKelly, Carson 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.129 0.139 0.116 87.7 -1\nUrshela, Gio 1 0 0 0 0 0 0 0 1 100 0.699 0.699\nIbáñez, Andy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.826 0.92 0.767 81.8 14\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.449 0.468 0.404 63.7 40'}, {'pitcher_id': '656420', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeyereisen, J.P. 6 5 3 0 2 0 0 0 18.2 1 16.7 0.6 1.4 0.784 0.377 0.771 0.521 84.7 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 4 4 2 0 1 0 0 0 12.5 0 0 0.5 1 0.612 0.395 0.874 0.539 85.5 6\nBaddoo, Akil 2 1 1 0 1 0 0 0 33.3 1 50 1 3 1.13 0.306 0.362 0.483 81.3 25'}, {'pitcher_id': '808967', 'player_data': '', 'team_data': 'Yoshinobu Yamamoto has not faced anyone on this team.'}, {'pitcher_id': '676272', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bobby 17 14 1 0 0 0 5 29.4 25.8 1 5.9 0.071 0.071 0.178 0.154 0.212 0.255 88.9 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 4 3 0 0 0 0 0 0 25 0 0 0 0 0.182 0.029 0.041 0.207 84.6 29\nMcKinstry, Zach 3 2 0 0 0 0 2 66.7 25 0 0 0 0 0.242 0.242\nVierling, Matt 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.232 0.347 0.561 0.491 105.1 15\nIbáñez, Andy 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.028 0.043 0.026 90.3 -30\nBaddoo, Akil 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.235 0.272 0.215 85.8 -11\nKelly, Carson 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.409 0.527 0.4 89.9 37.5'}, {'pitcher_id': '669165', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '595014', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTreinen, Blake 9 9 1 0 0 0 4 44.4 33.3 0 0 0.111 0.111 0.097 0.156 0.172 0.144 78.5 -5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 4 4 0 0 0 0 1 25 16.7 0 0 0 0 0 0.089 0.102 0.085 79.8 -11\nCanha, Mark 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.036 0.041 0.032 82.9 -21\nRogers, Jake 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nUrshela, Gio 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.947 1.019 0.859 70.2 26'}, {'pitcher_id': '607455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanda, Anthony 2 2 1 0 0 1 1 50 0 0 0 0.5 2 1.003 0.172 0.604 0.31 98.1 32', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreene, Riley 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nKelly, Carson 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.343 1.207 0.62 98.1 32'}, {'pitcher_id': '523260', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Joe 24 18 5 1 1 1 7 29.2 32.4 4 16.7 0.278 0.611 0.406 0.262 0.513 0.377 95.4 16.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUrshela, Gio 7 6 0 0 0 0 2 28.6 23.1 1 14.3 0 0 0.098 0.247 0.332 0.303 92.2 20.5\nGreene, Riley 3 2 2 1 1 0 0 0 33.3 1 33.3 1 2.5 1.167 0.695 2.036 0.967 101.4 21\nKelly, Carson 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.294 0.164 0.248 0.175 94.5 18\nCanha, Mark 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.293 0.291 92.4 10.7\nMcKinstry, Zach 3 3 2 0 0 1 1 33.3 33.3 0 0 0.667 1.667 0.962 0.34 0.613 0.394 100.6 13.5\nBaddoo, Akil 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMiller, Shelby 2 0 0 0 0 0 0 0 0 1 50 0.689\nKreidler, Ryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRogers, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '543339', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Daniel 10 10 1 0 0 0 4 40 27.8 0 0 0.1 0.1 0.088 0.201 0.367 0.24 95.1 18.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Carson 5 5 0 0 0 0 3 60 33.3 0 0 0 0 0 0.171 0.32 0.206 99.8 20.5\nCanha, Mark 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.295 0.296 0.363 0.292 87.8 -4.5\nUrshela, Gio 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.134 0.49 0.248 97.7 40'}, {'pitcher_id': '675627', 'player_data': '', 'team_data': 'Michael Grove has not faced anyone on this team.'}, {'pitcher_id': '622694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Elieser 8 8 3 1 0 0 0 0 16.7 0 0 0.375 0.5 0.381 0.457 0.622 0.464 97.5 22.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.491 0.682 0.491 96.5 17.5\nMalloy, Justyn-Henry 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.586 0.65 0.54 99 7\nGreene, Riley 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 88.1 43\nPérez, Wenceel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.013 0.009 91.1 59\nVierling, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.591 1.26 0.786 105.3 19\nMcKinstry, Zach 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.557 0.602 0.513 104.3 4\nUrshela, Gio 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.931 1.089 0.879 99 12'}, {'pitcher_id': '598286', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamirez, Nick 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.103 0.141 0.102 82.2 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.077 0.122 0.083 89.1 23\nKelly, Carson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.18 0.14 75.3 4'}, {'pitcher_id': '689017', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '572020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaxton, James 14 12 3 0 0 1 3 21.4 20 1 7.1 0.25 0.5 0.32 0.338 0.631 0.45 90.9 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanha, Mark 12 12 3 0 0 1 3 25 24 0 0 0.25 0.5 0.316 0.338 0.631 0.407 90.2 20.2\nUrshela, Gio 2 0 0 0 0 0 0 0 0 1 50 0.348 0.708 96.5 16'}], 117: [{'pitcher_id': '642232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYarbrough, Ryan 45 39 11 3 0 2 4 8.9 10.3 3 6.7 0.282 0.513 0.373 0.273 0.49 0.365 81.5 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 15 13 4 1 0 1 2 13.3 5 1 6.7 0.308 0.615 0.426 0.275 0.565 0.397 85.9 24.2\nAlvarez, Yordan 10 9 3 1 0 0 2 20 23.8 1 10 0.333 0.444 0.367 0.281 0.532 0.375 93 11.7\nAltuve, Jose 16 14 4 1 0 1 0 0 0 1 6.3 0.286 0.571 0.376 0.3 0.47 0.353 73.7 15.8\nCaratini, Victor 2 1 0 0 0 0 0 0 25 0 0 0 0 0.36 0.113 0.113 0.412 72 -45\nMeyers, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.12 0.146 0.112 81.2 35'}, {'pitcher_id': '681402', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '607192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGlasnow, Tyler 23 21 7 2 0 3 2 8.7 14.3 2 8.7 0.333 0.857 0.498 0.231 0.471 0.327 88.2 14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 9 8 3 1 0 1 1 11.1 18.8 1 11.1 0.375 0.875 0.525 0.232 0.349 0.304 83.9 0.9\nAltuve, Jose 11 10 4 1 0 2 0 0 7.1 1 9.1 0.4 1.1 0.611 0.286 0.693 0.423 92.2 31.9\nAlvarez, Yordan 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.045 0.053 0.045 82.9 -26.5'}, {'pitcher_id': '621111', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBuehler, Walker 27 26 4 1 0 0 7 25.9 12.2 0 0 0.154 0.192 0.17 0.236 0.356 0.271 94 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 8 8 1 0 0 0 2 25 27.3 0 0 0.125 0.125 0.11 0.226 0.257 0.214 96.6 -11.3\nDubón, Mauricio 5 5 1 0 0 0 2 40 18.2 0 0 0.2 0.2 0.174 0.255 0.472 0.307 99 10.7\nAltuve, Jose 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.176 0.346 0.49 0.358 92.9 12\nAlvarez, Yordan 3 3 0 0 0 0 1 33.3 10 0 0 0 0 0 0.205 0.361 0.235 97.8 36\nBregman, Alex 5 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.393 0.204 0.359 0.333 85.5 -0.5\nHummel, Cooper 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '670990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamírez, Yohan 17 15 0 0 0 0 7 41.2 40 1 5.9 0 0 0.084 0.078 0.099 0.152 82 20.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 7 7 0 0 0 0 2 28.6 33.3 0 0 0 0 0 0.153 0.194 0.152 81.1 19.6\nBregman, Alex 4 3 0 0 0 0 3 75 40 1 25 0 0 0.175 0.175\nAlvarez, Yordan 3 2 0 0 0 0 1 33.3 50 0 0 0 0 0.243 0.029 0.035 0.261 72.2 -20\nMcCormick, Chas 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDubón, Mauricio 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.02 0.023 0.018 88.6 34\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.032 0.025 89.9 53'}, {'pitcher_id': '694813', 'player_data': '', 'team_data': 'Gavin Stone has not faced anyone on this team.'}, {'pitcher_id': '676568', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVesia, Alex 4 3 0 0 0 0 2 50 40 1 25 0 0 0.172 0.019 0.02 0.186 69.4 -20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHummel, Cooper 2 1 0 0 0 0 0 0 25 1 50 0 0 0.344 0.058 0.061 0.373 69.4 -20\nDubón, Mauricio 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCaratini, Victor 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '518489', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrasier, Ryan 17 16 4 1 0 0 1 5.9 14.8 1 5.9 0.25 0.313 0.269 0.327 0.435 0.352 90.1 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 5 5 2 0 0 0 0 0 9.1 0 0 0.4 0.4 0.352 0.342 0.436 0.333 82.9 29\nBregman, Alex 6 5 2 1 0 0 0 0 0 1 16.7 0.4 0.6 0.47 0.418 0.614 0.485 97.4 27\nMeyers, Jake 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.166 0.181 0.154 79.5 -29.5\nDiaz, Yainer 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDubón, Mauricio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.92 1.103 0.895 91.4 15\nAlvarez, Yordan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.019 0.011 97.2 50\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.193 0.219 0.177 94.8 -12'}, {'pitcher_id': '641401', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrogdon, Connor 17 17 7 1 0 0 2 11.8 25.6 0 0 0.412 0.471 0.386 0.296 0.447 0.321 88.4 11.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeña, Jeremy 4 4 2 0 0 0 0 0 9.1 0 0 0.5 0.5 0.442 0.416 0.576 0.43 91.7 10.5\nAlvarez, Yordan 4 4 0 0 0 0 2 50 62.5 0 0 0 0 0 0.268 0.325 0.257 96.2 16.5\nMeyers, Jake 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.277 0.304 0.258 84.3 0.5\nBregman, Alex 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.42 0.277 0.807 0.456 85.4 48.7\nCaratini, Victor 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.884 0.047 0.047 0.042 68.9 -30\nAltuve, Jose 1 1 1 0 0 0 0 0 25 0 0 1 1 0.884 0.469 0.525 0.434 89.4 7\nDubón, Mauricio 1 1 1 0 0 0 0 0 50 0 0 1 1 0.884 0.234 0.235 0.216 98 -11\nMcCormick, Chas 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.156 0.143 85.1 -10'}, {'pitcher_id': '623465', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPhillips, Evan 6 4 1 0 0 0 0 0 25 1 16.7 0.25 0.25 0.382 0.288 0.339 0.416 83.5 28.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHummel, Cooper 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.223 0.283 0.216 77.8 30\nAlvarez, Yordan 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nDiaz, Yainer 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.083 0.166 0.099 90.8 26\nMcCormick, Chas 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.834 0.875 0.751 74.3 17\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.034 0.023 91.1 42\nBregman, Alex 1 0 0 0 0 0 0 0 0 0 0.719 0.719'}, {'pitcher_id': '656420', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeyereisen, J.P. 4 3 0 0 0 0 2 50 40 1 25 0 0 0.175 0.004 0.013 0.18 95.2 44', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaratini, Victor 2 1 0 0 0 0 1 50 50 1 50 0 0 0.349 0.349\nMeyers, Jake 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nAltuve, Jose 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.011 0.04 0.021 95.2 44'}, {'pitcher_id': '808967', 'player_data': '', 'team_data': 'Yoshinobu Yamamoto has not faced anyone on this team.'}, {'pitcher_id': '676272', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bobby 13 12 7 2 0 1 1 7.7 10 1 7.7 0.583 1 0.671 0.371 0.534 0.41 83.2 22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.313 0.767 0.441 84 31.7\nAltuve, Jose 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.314 0.345 0.285 72.8 25\nDiaz, Yainer 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.346 0.394 0.322 81.3 18.7\nPeña, Jeremy 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.298 0.356 0.288 99.8 5\nMeyers, Jake 2 1 1 1 0 0 0 0 0 1 50 1 2 0.97 0.934 1.177 0.808 101.8 11'}, {'pitcher_id': '669165', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '595014', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTreinen, Blake 26 22 6 1 0 1 2 7.7 17.4 3 11.5 0.273 0.455 0.336 0.324 0.581 0.405 82.2 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 11 8 1 0 0 0 0 0 13.6 2 18.2 0.125 0.125 0.205 0.228 0.297 0.293 80.9 23.4\nAltuve, Jose 12 11 3 1 0 0 2 16.7 25 1 8.3 0.273 0.364 0.305 0.293 0.476 0.357 82.2 13.3\nCaratini, Victor 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.94 0.697 1.726 1.004 86.4 17.3'}, {'pitcher_id': '607455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanda, Anthony 6 5 3 0 1 0 1 16.7 25 1 16.7 0.6 1 0.666 0.482 0.554 0.491 83.5 18.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 3 2 1 0 0 0 1 33.3 37.5 1 33.3 0.5 0.5 0.523 0.327 0.398 0.439 85.3 19\nBregman, Alex 3 3 2 0 1 0 0 0 0 0 0 0.667 1.333 0.81 0.586 0.658 0.543 82.8 18.3'}, {'pitcher_id': '523260', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Joe 42 34 9 0 0 1 8 19 20.5 5 11.9 0.265 0.353 0.336 0.244 0.411 0.354 87.1 9.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 15 13 6 0 0 1 2 13.3 21.7 1 6.7 0.462 0.692 0.531 0.287 0.328 0.342 77.4 12.4\nBregman, Alex 10 8 1 0 0 0 1 10 10 1 10 0.125 0.125 0.23 0.304 0.68 0.47 94.8 16.3\nSingleton, Jon 5 2 0 0 0 0 1 20 50 3 60 0 0 0.413 0.573 1.101 0.696 103.8 16\nDubón, Mauricio 5 5 1 0 0 0 3 60 50 0 0 0.2 0.2 0.177 0.023 0.024 0.021 61.4 -51.5\nCaratini, Victor 2 1 0 0 0 0 1 50 0 0 0 0 0 0 0 0 0.406 110.1 16\nAlvarez, Yordan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.282 0.577 0.358 98.6 13.3\nMcCormick, Chas 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.011 0.022 0.012 85.3 36\nPeña, Jeremy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.262 0.214 63.8 11'}, {'pitcher_id': '543339', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Daniel 16 16 4 2 0 0 4 25 31 0 0 0.25 0.375 0.268 0.175 0.237 0.178 79.7 1.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 9 9 2 1 0 0 2 22.2 31.6 0 0 0.222 0.333 0.24 0.176 0.217 0.173 86.4 1.7\nAlvarez, Yordan 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.621 0.287 0.547 0.344 105.3 17\nMcCormick, Chas 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBregman, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.031 0.034 0.029 80.8 12.5\nHummel, Cooper 1 1 1 0 0 0 0 0 50 0 0 1 1 0.884 0.297 0.3 0.269 28.1 -45\nMeyers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.28 0.318 0.254 76.4 9'}, {'pitcher_id': '675627', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrove, Michael 9 9 1 1 0 0 3 33.3 23.8 0 0 0.111 0.222 0.14 0.169 0.222 0.167 87.9 26.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHummel, Cooper 5 5 1 1 0 0 2 40 30.8 0 0 0.2 0.4 0.252 0.117 0.193 0.128 91.6 29.3\nPeña, Jeremy 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 68.7 66\nAltuve, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.297 0.321 0.269 92.6 -2\nMcCormick, Chas 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.631 0.713 0.592 91.2 9'}, {'pitcher_id': '622694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Elieser 4 3 1 0 0 1 0 0 33.3 1 25 0.333 1.333 0.657 0.17 0.305 0.323 80.7 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHummel, Cooper 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.344 0.297 0.3 0.48 64 -59\nDubón, Mauricio 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 0.97 0.107 0.307 0.166 89 47.5'}, {'pitcher_id': '598286', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamirez, Nick 19 16 2 0 0 0 6 31.6 34.4 3 15.8 0.125 0.125 0.202 0.09 0.113 0.184 82.1 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBregman, Alex 7 6 0 0 0 0 3 42.9 25 1 14.3 0 0 0.099 0.049 0.065 0.141 85.4 33.7\nAlvarez, Yordan 4 2 1 0 0 0 1 25 42.9 2 50 0.5 0.5 0.567 0.029 0.029 0.361 57 -32\nMcCormick, Chas 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.025 0.028 0.026 77.7 -25\nAltuve, Jose 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.096 0.196 0.118 92.3 31.5\nDiaz, Yainer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.517 0.563 0.465 93.8 7\nCaratini, Victor 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nPeña, Jeremy 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.32 0.324 0.294 61.1 -55\nMeyers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.023 0.015 90.5 59'}, {'pitcher_id': '689017', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '572020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaxton, James 101 89 27 3 0 3 17 16.8 22.5 11 10.9 0.303 0.438 0.346 0.267 0.469 0.343 88.2 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAltuve, Jose 43 37 12 1 0 2 6 14 23.8 6 14 0.324 0.514 0.385 0.309 0.557 0.395 90 9.8\nBregman, Alex 36 33 10 1 0 0 6 16.7 18 3 8.3 0.303 0.333 0.311 0.259 0.42 0.323 89.5 13.4\nAlvarez, Yordan 10 7 1 0 0 0 2 20 16.7 2 20 0.143 0.143 0.226 0.137 0.176 0.246 83.6 6\nMcCormick, Chas 3 3 1 0 0 1 1 33.3 44.4 0 0 0.333 1.333 0.668 0.461 1.406 0.762 98.1 16.5\nDiaz, Yainer 3 3 1 0 0 0 1 33.3 27.3 0 0 0.333 0.333 0.294 0.041 0.048 0.039 73.6 -16.5\nPeña, Jeremy 3 3 1 1 0 0 0 0 16.7 0 0 0.333 0.667 0.415 0.311 0.379 0.302 86.1 -14.7\nMeyers, Jake 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.128 0.169 0.135 65.6 7.5'}], 118: [{'pitcher_id': '642232', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYarbrough, Ryan 34 31 7 3 0 1 6 17.6 25.4 2 5.9 0.226 0.419 0.312 0.26 0.442 0.334 88.2 23.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 6 5 1 1 0 0 2 33.3 50 1 16.7 0.2 0.4 0.323 0.285 0.328 0.338 86.9 6\nWitt Jr., Bobby 5 4 3 1 0 1 0 0 22.2 0 0 0.75 1.75 0.989 0.643 1.289 0.802 92.8 25.8\nMelendez, MJ 5 5 2 1 0 0 1 20 40 0 0 0.4 0.6 0.429 0.253 0.312 0.239 93.2 15.5\nHampson, Garrett 4 4 0 0 0 0 1 25 12.5 0 0 0 0 0 0.059 0.098 0.063 84.9 41.5\nPasquantino, Vinnie 5 5 1 0 0 0 2 40 22.2 0 0 0.2 0.2 0.177 0.242 0.287 0.233 89.3 24.3\nIsbel, Kyle 2 1 0 0 0 0 0 0 25 1 50 0 0 0.344 0.113 0.114 0.399 83.2 -9\nFrazier, Adam 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.066 0.043 87.3 40.3\nNola, Austin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.287 0.939 0.487 84.3 23.3\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.057 0.051 76.5 38'}, {'pitcher_id': '681402', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Gus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.12 0.067 90.7 35', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.12 0.067 90.7 35'}, {'pitcher_id': '607192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGlasnow, Tyler 40 36 10 1 0 1 15 37.5 43.8 3 7.5 0.278 0.389 0.31 0.219 0.371 0.286 89.4 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 11 10 3 1 0 0 4 36.4 50 0 0 0.3 0.4 0.274 0.229 0.368 0.264 91.7 27.1\nWaters, Drew 5 4 1 0 0 0 3 60 63.6 1 20 0.25 0.25 0.316 0.072 0.073 0.192 90.9 -3\nWitt Jr., Bobby 5 5 2 0 0 1 1 20 22.2 0 0 0.4 1 0.577 0.359 1.011 0.563 88 37.3\nFrazier, Adam 4 2 1 0 0 0 0 0 16.7 2 50 0.5 0.5 0.569 0.39 0.405 0.523 77.2 11\nMelendez, MJ 5 5 2 0 0 0 3 60 66.7 0 0 0.4 0.4 0.353 0.19 0.248 0.19 93.8 -1\nIsbel, Kyle 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.15 0.184 0.148 74.4 15\nBlanco, Dairon 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nGarcia, Maikel 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.177 0.263 0.345 0.257 97.9 -3.8'}, {'pitcher_id': '621111', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBuehler, Walker 34 31 6 0 1 0 3 8.8 17.9 3 8.8 0.194 0.258 0.235 0.248 0.307 0.28 85.7 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 15 14 3 0 1 0 1 6.7 19.2 1 6.7 0.214 0.357 0.264 0.287 0.368 0.308 86.7 13.8\nFrazier, Adam 14 13 2 0 0 0 2 14.3 18.8 1 7.1 0.154 0.154 0.174 0.185 0.221 0.214 83.5 17.9\nNola, Austin 5 4 1 0 0 0 0 0 11.1 1 20 0.25 0.25 0.316 0.322 0.372 0.383 88.2 2.3'}, {'pitcher_id': '670990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamírez, Yohan 14 11 1 0 0 0 4 28.6 17.9 1 7.1 0.091 0.091 0.216 0.203 0.292 0.318 84 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 4 3 0 0 0 0 1 25 11.1 0 0 0 0 0.18 0.213 0.244 0.328 77.7 7.5\nHampson, Garrett 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.075 0.082 0.069 109.1 -27\nGarcia, Maikel 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.49 0.538 0.453 78.7 18\nWitt Jr., Bobby 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.044 0.051 0.04 74.9 50\nIsbel, Kyle 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nNola, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.303 1.059 0.536 99.3 33\nBlanco, Dairon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.079 0.063 70.7 -12\nFrazier, Adam 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '694813', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStone, Gavin 26 24 4 0 0 1 3 11.5 19.5 2 7.7 0.167 0.292 0.236 0.23 0.452 0.32 91.4 14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 3 2 1 0 0 0 0 0 50 1 33.3 0.5 0.5 0.527 0.795 0.906 0.729 80.1 21\nLoftin, Nick 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.038 0.053 0.04 74.4 53\nPerez, Salvador 3 3 1 0 0 1 0 0 12.5 0 0 0.333 1.333 0.695 0.373 1.374 0.702 100 13\nPasquantino, Vinnie 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.272 1.025 0.585 99.1 35.5\nFrazier, Adam 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.214 0.275 0.217 96.9 -0.5\nWaters, Drew 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.073 0.147 0.095 83.8 12\nWitt Jr., Bobby 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.085 0.109 0.088 83.7 19\nIsbel, Kyle 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.116 0.093 96.6 -23\nGarcia, Maikel 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.28 0.294 0.253 99.1 16'}, {'pitcher_id': '676568', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVesia, Alex 14 12 1 0 0 0 3 21.4 18.5 2 14.3 0.083 0.083 0.162 0.132 0.166 0.208 78 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 6 5 0 0 0 0 1 16.7 10 1 16.7 0 0 0.115 0.066 0.107 0.174 78.9 20.5\nHampson, Garrett 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nVelázquez, Nelson 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.222 0.239 0.2 83 9\nFermin, Freddy 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.469 0.538 0.433 95.3 2\nWitt Jr., Bobby 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.198 0.125 86.3 27\nPasquantino, Vinnie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.212 0.191 70 -52\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.034 0.031 51.7 52'}, {'pitcher_id': '518489', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrasier, Ryan 8 6 0 0 0 0 4 50 33.3 1 12.5 0 0 0.087 0.165 0.173 0.226 95.4 8.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.571 0.61 0.605 88.1 10\nGarcia, Maikel 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nWitt Jr., Bobby 1 0 0 0 0 0 0 0 0 0 0 0 0.218 92.9 25\nPerez, Salvador 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nIsbel, Kyle 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWaters, Drew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBlanco, Dairon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.421 0.428 0.384 105.2 -9'}, {'pitcher_id': '641401', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrogdon, Connor 5 4 1 0 0 0 1 20 31.3 1 20 0.25 0.25 0.314 0.085 0.137 0.217 81 30', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 3 3 0 0 0 0 1 33.3 30.8 0 0 0 0 0 0.029 0.088 0.046 90.6 39\nVelázquez, Nelson 1 0 0 0 0 0 0 0 50 1 100 0.689 0.689\nFrazier, Adam 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.254 0.283 0.259 61.8 12'}, {'pitcher_id': '623465', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPhillips, Evan 7 7 1 0 0 0 4 57.1 56.3 0 0 0.143 0.143 0.126 0.151 0.193 0.15 94.7 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Austin 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nMelendez, MJ 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVelázquez, Nelson 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.125 0.126 0.116 97 -19\nWaters, Drew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFermin, Freddy 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.063 0.196 0.107 97.9 43\nPerez, Salvador 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.871 1.027 0.83 89.1 17'}, {'pitcher_id': '656420', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFeyereisen, J.P. 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.024 0.025 0.024 85.8 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.042 0.042 0.043 69.5 -22\nPerez, Salvador 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.006 0.007 0.005 102 56'}, {'pitcher_id': '808967', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYamamoto, Yoshinobu 8 7 1 0 0 0 1 12.5 20 1 12.5 0.143 0.143 0.198 0.083 0.093 0.154 91.3 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.251 0.261 0.227 103.4 -13\nPasquantino, Vinnie 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.014 0.009 89.7 46\nBlanco, Dairon 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nFermin, Freddy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 75.8 71\nVelázquez, Nelson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.248 0.285 0.226 97.1 -7\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.008 0.004 93.2 42\nGarcia, Maikel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.075 0.082 0.07 88.7 -26'}, {'pitcher_id': '676272', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bobby 19 18 4 0 0 0 2 10.5 13.5 1 5.3 0.222 0.222 0.222 0.297 0.363 0.308 90.4 5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWitt Jr., Bobby 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.09 0.116 0.083 92.3 -3\nMelendez, MJ 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.096 0.102 0.091 84.2 -6\nWaters, Drew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.221 0.296 0.226 92.9 37\nGarcia, Maikel 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.59 0.729 0.569 98.3 14\nPerez, Salvador 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.347 0.382 0.445 96.7 -2.5\nIsbel, Kyle 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.362 0.514 0.383 84.2 -4.5\nFermin, Freddy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.388 0.437 0.36 86.4 1'}, {'pitcher_id': '669165', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '595014', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTreinen, Blake 20 15 2 1 0 1 3 15 22.5 4 20 0.133 0.4 0.34 0.216 0.444 0.384 88.3 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHampson, Garrett 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.225 0.417 0.267 89.1 27.3\nMelendez, MJ 1 1 1 0 0 1 0 0 11.1 0 0 1 4 2.085 0.471 1.787 0.909 103.2 36\nPerez, Salvador 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0.131 0.141 0.309 84.7 2\nFrazier, Adam 3 2 1 1 0 0 1 33.3 28.6 0 0 0.5 1 0.651 0.246 0.516 0.456 100.5 19\nNola, Austin 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.345 0.25 81.8 -11.4\nVelázquez, Nelson 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nPasquantino, Vinnie 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nFermin, Freddy 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.063 0.163 0.082 94.4 34'}, {'pitcher_id': '607455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanda, Anthony 14 12 2 0 1 0 3 21.4 24.1 1 7.1 0.167 0.333 0.228 0.243 0.322 0.261 86 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.215 0.193 78.8 8.5\nVelázquez, Nelson 2 1 0 0 0 0 1 50 42.9 0 0 0 0 0 0 0 0.015 95.1 44\nBlanco, Dairon 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.124 0.183 0.121 91.1 22\nGarcia, Maikel 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nPerez, Salvador 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.27 0.306 0.249 78 7.5\nPasquantino, Vinnie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nHampson, Garrett 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.806 0.299 0.629 0.39 91.6 39\nMelendez, MJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.305 0.315 0.274 105.2 -12\nWitt Jr., Bobby 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.803 0.884 0.749 71.5 17\nFermin, Freddy'}, {'pitcher_id': '523260', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Joe 36 33 6 1 0 1 7 19.4 20.4 2 5.6 0.182 0.303 0.248 0.185 0.229 0.232 84.2 6.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 9 9 3 0 0 1 0 0 16.7 0 0 0.333 0.667 0.426 0.143 0.304 0.187 87 25.3\nFrazier, Adam 6 5 1 1 0 0 0 0 0 1 16.7 0.2 0.4 0.322 0.309 0.338 0.353 84.9 0.8\nHampson, Garrett 7 6 1 0 0 0 2 28.6 10 0 0 0.167 0.167 0.227 0.164 0.202 0.239 82.6 2.5\nWitt Jr., Bobby 5 5 0 0 0 0 1 20 22.2 0 0 0 0 0 0.08 0.121 0.086 83.6 -3\nMelendez, MJ 4 4 0 0 0 0 3 75 66.7 0 0 0 0 0 0.174 0.182 0.157 71.7 17\nPasquantino, Vinnie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.104 0.082 96.3 6\nGarcia, Maikel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWaters, Drew 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nNola, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.777 0.812 0.699 68.7 20'}, {'pitcher_id': '543339', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Daniel 5 5 1 0 0 0 1 20 12.5 0 0 0.2 0.2 0.177 0.228 0.331 0.245 85.6 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Maikel 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.131 0.132 0.118 76.1 -53\nFermin, Freddy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.183 0.57 0.313 93.8 31\nIsbel, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.529 0.62 0.511 89.7 8\nWitt Jr., Bobby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.201 0.163 82.7 -14'}, {'pitcher_id': '675627', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrove, Michael 10 10 2 1 0 0 3 30 35.3 0 0 0.2 0.3 0.216 0.299 0.361 0.286 89.1 -1.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMelendez, MJ 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nGarcia, Maikel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nWitt Jr., Bobby 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nPasquantino, Vinnie 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.889 0.903 1.191 0.896 105.8 12\nFrazier, Adam 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.328 0.337 0.295 80.7 -19.5\nFermin, Freddy 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.94 1.133 0.909 96.7 12\nHampson, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.112 0.129 0.114 85.3 -17\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.276 0.331 0.25 92.8 -5\nVelázquez, Nelson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.151 0.098 81.9 28'}, {'pitcher_id': '622694', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Elieser 9 8 1 0 0 0 1 11.1 11.8 1 11.1 0.125 0.125 0.174 0.142 0.217 0.212 84.3 34.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 8 7 1 0 0 0 1 12.5 14.3 1 12.5 0.143 0.143 0.196 0.161 0.247 0.238 86.8 26.3\nNola, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 69.1 81'}, {'pitcher_id': '598286', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamirez, Nick 5 5 3 1 0 0 1 20 12.5 0 0 0.6 0.8 0.591 0.396 0.542 0.402 89.3 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrazier, Adam 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.435 0.243 0.292 0.237 99.9 0\nNola, Austin 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.043 0.337 0.589 0.37 88.2 25\nLoftin, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.82 0.947 0.793 80.8 21'}, {'pitcher_id': '689017', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '572020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaxton, James 40 36 13 3 1 2 7 17.5 32.9 4 10 0.361 0.667 0.456 0.405 0.649 0.474 97.6 6.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerez, Salvador 16 16 5 1 0 1 2 12.5 30.6 0 0 0.313 0.563 0.368 0.505 0.731 0.531 98.2 9.5\nNola, Austin 4 2 0 0 0 0 1 25 66.7 2 50 0 0 0.347 0.243 0.422 0.486 100.7 17\nWitt Jr., Bobby 4 4 2 1 0 1 1 25 33.3 0 0 0.5 1.5 0.812 0.462 0.953 0.593 101.6 15.3\nGarcia, Maikel 5 5 3 0 0 0 1 20 22.2 0 0 0.6 0.6 0.53 0.406 0.481 0.385 97.1 0\nFermin, Freddy 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.265 0.366 0.258 98.4 7.5\nBlanco, Dairon 2 2 1 0 1 0 1 50 75 0 0 0.5 1.5 0.784 0.326 0.999 0.534 102.3 24\nMelendez, MJ 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.108 0.121 0.099 82 -16\nLoftin, Nick 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nWaters, Drew 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nVelázquez, Nelson 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.711 1.43 0.889 108.7 17\nIsbel, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.078 0.08 0.073 91.3 -32'}]}, 120: {121: [{'pitcher_id': '571670', 'player_data': '', 'team_data': ''}, {'pitcher_id': '680730', 'player_data': '', 'team_data': 'Mitchell Parker has not faced anyone on this team.'}, {'pitcher_id': '607179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeems, Jordan 32 29 9 2 0 0 9 28.1 26.2 1 3.1 0.31 0.379 0.317 0.2 0.255 0.224 89.4 8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 5 4 3 0 0 0 1 20 14.3 0 0 0.75 0.75 0.676 0.331 0.365 0.388 98.8 1\nBader, Harrison 3 3 1 1 0 0 1 33.3 28.6 0 0 0.333 0.667 0.423 0.281 0.343 0.261 89.5 -5\nLindor, Francisco 5 5 2 0 0 0 2 40 33.3 0 0 0.4 0.4 0.355 0.207 0.238 0.196 94.1 23.3\nAlonso, Pete 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.003 0.005 0.232 98.5 52\nMartinez, J.D. 4 3 1 1 0 0 1 25 27.3 0 0 0.333 0.667 0.311 0.314 0.449 0.257 92.1 8.3\nVientos, Mark 3 3 1 0 0 0 2 66.7 60 0 0 0.333 0.333 0.296 0.039 0.043 0.036 88.4 -13\nTorrens, Luis 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.028 0.029 0.025 80.9 -29\nAlvarez, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.32 0.32 0.288 94.5 -1\nIglesias, Jose 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.226 0.235 0.207 66.7 -4.5\nMarte, Starling 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.181 0.283 0.207 80.3 16.5\nMcNeil, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.134 0.234 0.146 91.6 25\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.209 0.596 0.329 95.2 28'}, {'pitcher_id': '571578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorbin, Patrick 282 254 79 14 2 14 60 21.3 25.2 20 7.1 0.311 0.547 0.387 0.301 0.522 0.376 90.6 8.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 56 46 15 3 0 5 12 21.4 20.6 9 16.1 0.326 0.717 0.466 0.351 0.704 0.478 95.3 14.2\nNimmo, Brandon 43 40 11 0 0 1 13 30.2 38.8 2 4.7 0.275 0.35 0.301 0.27 0.374 0.306 92.3 1.6\nMarte, Starling 44 42 13 2 0 1 10 22.7 22.6 2 4.5 0.31 0.429 0.335 0.248 0.372 0.28 85.6 7.5\nMcNeil, Jeff 41 37 10 1 0 0 5 12.2 19.7 2 4.9 0.27 0.297 0.282 0.296 0.411 0.336 83.7 7.1\nLindor, Francisco 41 41 13 2 0 3 8 19.5 19.4 0 0 0.317 0.585 0.381 0.328 0.603 0.396 92.2 9.5\nBader, Harrison 15 13 3 1 0 1 9 60 58.6 1 6.7 0.231 0.538 0.316 0.196 0.418 0.271 93.5 20.8\nIglesias, Jose 13 13 5 2 0 0 0 0 0 0 0 0.385 0.538 0.395 0.372 0.509 0.381 92.4 10\nAlvarez, Francisco 6 4 1 0 0 1 0 0 30 2 33.3 0.25 1 0.566 0.282 0.997 0.578 89.7 9.5\nVientos, Mark 6 5 2 1 1 0 1 16.7 21.4 0 0 0.4 1 0.59 0.317 0.542 0.423 101.4 -1.3\nTaylor, Tyrone 6 6 1 0 1 0 2 33.3 41.7 0 0 0.167 0.5 0.269 0.17 0.426 0.252 95.8 15.5\nMartinez, J.D. 6 4 2 1 0 0 0 0 14.3 2 33.3 0.5 0.75 0.587 0.361 0.437 0.46 84.3 0.7\nTorrens, Luis 2 2 2 0 0 2 0 0 16.7 0 0 1 4 2.085 0.76 2.663 1.398 102.3 26\nHouser, Adrian 3 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.56 1.041 0.651 104.4 17'}, {'pitcher_id': '669022', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGore, MacKenzie 79 68 22 1 0 4 10 12.7 19.9 9 11.4 0.324 0.515 0.397 0.297 0.527 0.391 87.2 20.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 12 11 3 1 0 0 2 16.7 27.3 1 8.3 0.273 0.364 0.311 0.346 0.523 0.399 92.7 8.8\nMarte, Starling 8 8 4 0 0 0 1 12.5 6.9 0 0 0.5 0.5 0.443 0.461 0.697 0.494 93.7 18.1\nAlonso, Pete 11 10 1 0 0 1 1 9.1 16.7 1 9.1 0.1 0.4 0.245 0.146 0.463 0.288 83 35\nLindor, Francisco 11 9 1 0 0 0 0 0 6.3 1 9.1 0.111 0.111 0.21 0.173 0.191 0.258 79.7 26.9\nMcNeil, Jeff 8 7 6 0 0 0 0 0 15.4 1 12.5 0.857 0.857 0.749 0.403 0.445 0.414 82.3 -6\nMartinez, J.D. 5 4 1 0 0 1 1 20 33.3 1 20 0.25 1 0.539 0.368 0.932 0.567 97.9 23.7\nAlvarez, Francisco 4 2 0 0 0 0 1 25 50 2 50 0 0 0.348 0.033 0.053 0.367 77.3 53\nVientos, Mark 5 3 1 0 0 1 1 20 50 2 40 0.333 1.333 0.694 0.42 1.196 0.681 98.6 20\nTaylor, Tyrone 5 5 1 0 0 1 1 20 25 0 0 0.2 0.8 0.401 0.29 0.854 0.475 94.1 20.3\nIglesias, Jose 4 4 3 0 0 0 0 0 14.3 0 0 0.75 0.75 0.665 0.338 0.37 0.311 80.6 25.5\nBader, Harrison 4 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.221 0.412 0.633 0.366 87.4 39.5\nTorrens, Luis 2 2 0 0 0 0 2 100 42.9 0 0 0 0 0 0'}, {'pitcher_id': '671131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672851', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdon, Joan 43 33 9 2 0 1 6 14 19.7 8 18.6 0.273 0.424 0.391 0.269 0.438 0.394 92.5 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 8 5 2 1 0 0 1 12.5 5.9 3 37.5 0.4 0.6 0.524 0.17 0.195 0.356 89.5 9.3\nNimmo, Brandon 8 5 2 1 0 0 1 12.5 25 3 37.5 0.4 0.6 0.524 0.35 0.681 0.532 96.1 28.3\nAlonso, Pete 6 5 1 0 0 1 2 33.3 33.3 1 16.7 0.2 0.8 0.461 0.341 0.881 0.54 104.7 18.7\nMcNeil, Jeff 7 6 0 0 0 0 0 0 0 1 14.3 0 0 0.098 0.148 0.248 0.242 90.3 17.3\nMarte, Starling 5 4 0 0 0 0 0 0 14.3 0 0 0 0 0.144 0.298 0.345 0.367 85.2 20.8\nIglesias, Jose 4 4 3 0 0 0 0 0 0 0 0 0.75 0.75 0.662 0.41 0.434 0.374 91.2 3\nVientos, Mark 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.294 0.286 0.355 0.278 96.7 -7\nMartinez, J.D. 2 1 0 0 0 0 1 50 75 0 0 0 0 0.361 0.361'}, {'pitcher_id': '640448', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFinnegan, Kyle 48 45 10 1 0 3 6 12.5 19.8 2 4.2 0.222 0.444 0.292 0.273 0.498 0.339 91.4 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 12 11 3 0 0 2 2 16.7 26.7 1 8.3 0.273 0.818 0.472 0.265 0.694 0.422 89.5 36.4\nLindor, Francisco 9 8 1 0 0 1 2 22.2 23.1 0 0 0.125 0.5 0.223 0.183 0.378 0.229 91.2 12.3\nMcNeil, Jeff 9 9 1 0 0 0 0 0 8.7 0 0 0.111 0.111 0.098 0.219 0.333 0.239 94.8 4.9\nNimmo, Brandon 5 5 1 0 0 0 0 0 14.3 0 0 0.2 0.2 0.177 0.393 0.705 0.47 86.4 -3\nIglesias, Jose 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.589 0.588 0.613 0.538 83.2 15.7\nBader, Harrison 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.303 0.897 0.584 96.8 29\nMartinez, J.D. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.105 0.084 87 14\nMarte, Starling 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.123 0.135 0.113 94.5 -9\nAlvarez, Francisco 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVientos, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.393 0.393 0.357 106.9 -7\nTaylor, Tyrone 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.569 1.079 0.676 105.9 17\nStewart, DJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.486 0.544 0.449 98.8 3'}, {'pitcher_id': '663432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRainey, Tanner 31 28 3 0 0 1 6 19.4 27.9 3 9.7 0.107 0.214 0.187 0.207 0.379 0.289 87.8 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 4 4 1 0 0 1 2 50 46.2 0 0 0.25 1 0.485 0.25 0.976 0.481 94 5\nBader, Harrison 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.188 0.214 0.176 89.2 41.3\nMarte, Starling 4 4 0 0 0 0 1 25 50 0 0 0 0 0 0.129 0.181 0.132 82.6 42\nIglesias, Jose 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.354 0.413 0.332 85.9 14.5\nMcNeil, Jeff 4 4 1 0 0 0 0 0 16.7 0 0 0.25 0.25 0.221 0.228 0.246 0.21 89.9 2.3\nMartinez, J.D. 1 0 0 0 0 0 0 0 16.7 1 100 0.692 0.692\nNimmo, Brandon 4 4 0 0 0 0 1 25 16.7 0 0 0 0 0 0.081 0.24 0.131 95.5 35\nLindor, Francisco 4 3 0 0 0 0 0 0 0 1 25 0 0 0.172 0.313 0.516 0.438 79.8 -19\nStewart, DJ 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nVientos, Mark 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.889 0.251 0.267 0.235 80.3 4\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.124 0.073 89.7 33'}, {'pitcher_id': '676395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Robert 5 4 0 0 0 0 2 40 20 1 20 0 0 0.139 0.173 0.204 0.272 83.9 -1.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nLindor, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.631 0.752 0.601 99.9 6\nTaylor, Tyrone 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nNimmo, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.064 0.061 67.8 -9\nMcNeil, Jeff 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '571882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaw, Derek 31 28 7 1 1 0 5 16.1 31 2 6.5 0.25 0.357 0.278 0.24 0.311 0.273 86 14.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 8 7 2 0 0 0 1 12.5 25 1 12.5 0.286 0.286 0.306 0.397 0.572 0.439 91.3 10.2\nMarte, Starling 5 5 2 0 1 0 2 40 27.3 0 0 0.4 0.8 0.499 0.241 0.262 0.226 87.8 -19.3\nAlonso, Pete 3 2 0 0 0 0 1 33.3 42.9 1 33.3 0 0 0.232 0.059 0.059 0.269 83.5 -6\nLindor, Francisco 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.099 0.101 0.089 53.2 11\nStewart, DJ 3 3 1 1 0 0 0 0 50 0 0 0.333 0.667 0.406 0.056 0.119 0.072 78.7 41.3\nAlvarez, Francisco 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.441 0.502 0.615 0.486 88.8 11.5\nNimmo, Brandon 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.179 0.223 0.172 91.2 36\nVientos, Mark 1 0 0 0 0 0 0 0 66.7 0 0 0 0.47 97.8 22\nMcNeil, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.357 0.428 0.321 97.5 -2\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.251 0.212 95.1 -14\nIglesias, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.046 0.032 84.6 58'}, {'pitcher_id': '592866', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilliams, Trevor 99 88 24 6 2 3 18 18.2 21.7 8 8.1 0.273 0.489 0.346 0.247 0.427 0.322 88.8 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 17 15 6 1 0 0 4 23.5 27.6 0 0 0.4 0.467 0.375 0.281 0.384 0.298 88 22.9\nAlonso, Pete 15 13 3 1 0 1 2 13.3 33.3 1 6.7 0.231 0.538 0.32 0.299 0.704 0.435 93.5 29.2\nLindor, Francisco 15 14 4 2 0 1 2 13.3 16 1 6.7 0.286 0.643 0.405 0.225 0.434 0.304 91.7 25.6\nMcNeil, Jeff 16 13 4 1 1 0 2 12.5 22.6 3 18.8 0.308 0.538 0.414 0.284 0.34 0.348 84 18.7\nNimmo, Brandon 11 10 1 0 0 0 4 36.4 4.5 1 9.1 0.1 0.1 0.143 0.218 0.331 0.273 94.7 14.8\nIglesias, Jose 8 8 2 0 1 0 0 0 0 0 0 0.25 0.5 0.305 0.134 0.233 0.16 82.3 -5.3\nMarte, Starling 4 3 0 0 0 0 0 0 11.1 1 25 0 0 0.174 0.11 0.273 0.29 93.4 21.3\nMartinez, J.D. 5 5 2 1 0 1 2 40 40 0 0 0.4 1.2 0.649 0.298 0.829 0.468 99.5 14\nAlvarez, Francisco 3 2 1 0 0 0 1 33.3 40 1 33.3 0.5 0.5 0.526 0.491 0.561 0.539 86.7 15\nQuintana, Jose 3 3 1 0 0 0 1 33.3 60 0 0 0.333 0.333 0.29 0.245 0.247 0.221 71.6 10.5\nTaylor, Tyrone 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.076 0.051 78.9 6.5'}, {'pitcher_id': '680686', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Josiah 72 64 13 4 1 2 17 23.6 29.4 5 6.9 0.203 0.391 0.291 0.203 0.405 0.298 85.6 31.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 12 10 4 1 1 0 3 25 21.1 2 16.7 0.4 0.7 0.502 0.285 0.431 0.368 92.4 25.3\nLindor, Francisco 13 11 0 0 0 0 3 23.1 33.3 2 15.4 0 0 0.106 0.012 0.015 0.116 83.5 54.9\nAlonso, Pete 13 11 3 1 0 1 4 30.8 51.9 1 7.7 0.273 0.636 0.431 0.217 0.485 0.357 90.1 27.3\nMcNeil, Jeff 13 11 4 1 0 1 0 0 3.8 0 0 0.364 0.727 0.448 0.369 0.715 0.458 89.8 35.8\nMarte, Starling 9 9 1 1 0 0 2 22.2 36.8 0 0 0.111 0.222 0.14 0.157 0.229 0.166 76.4 8.6\nStewart, DJ 3 3 0 0 0 0 2 66.7 28.6 0 0 0 0 0 0.001 0.002 0.001 69.1 75\nBader, Harrison 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.502 1.836 0.945 90.9 40.7\nIglesias, Jose 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.2 0.2 0.18 69.7 43\nMartinez, J.D. 2 2 0 0 0 0 2 100 25 0 0 0 0 0 0\nAlvarez, Francisco 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.123 0.123 0.113 71.2 -62'}, {'pitcher_id': '640451', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarvey, Hunter 29 27 6 1 0 1 7 24.1 26.4 1 3.4 0.222 0.37 0.258 0.273 0.521 0.34 92.3 9.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 8 8 1 0 0 1 2 25 15.8 0 0 0.125 0.5 0.247 0.217 0.766 0.401 94.4 18.8\nMcNeil, Jeff 8 6 3 1 0 0 1 12.5 16.7 1 12.5 0.5 0.667 0.466 0.449 0.673 0.465 86.7 17.3\nLindor, Francisco 5 5 0 0 0 0 1 20 46.2 0 0 0 0 0 0.223 0.284 0.214 94.8 5.3\nMartinez, J.D. 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.294 0.245 0.288 0.233 100.9 -11\nNimmo, Brandon 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.253 0.255 0.233 67.4 -64\nMarte, Starling 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.329 0.383 0.309 106.2 6\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.592 0.31 96.1 34\nVientos, Mark 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663623', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Jake 21 19 5 1 0 0 8 38.1 25 0 0 0.263 0.316 0.262 0.208 0.275 0.25 89.7 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 3 3 2 1 0 0 1 33.3 16.7 0 0 0.667 1 0.709 0.239 0.262 0.223 67.1 -29.5\nMartinez, J.D. 5 4 1 0 0 0 2 40 44.4 0 0 0.25 0.25 0.177 0.193 0.243 0.256 105.5 26\nNimmo, Brandon 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.25 0.523 0.328 95.2 21.7\nMarte, Starling 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.441 0.484 0.52 0.449 70.4 26\nAlonso, Pete 3 2 0 0 0 0 2 66.7 50 0 0 0 0 0.242 0.242\nAlvarez, Francisco 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.101 0.129 0.092 78 6\nMcNeil, Jeff 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.18 0.202 0.167 95.7 33.5'}, {'pitcher_id': '606930', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Jacob 34 33 10 4 0 1 8 23.5 33.3 1 2.9 0.303 0.515 0.354 0.27 0.482 0.325 87.9 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 8 8 1 0 0 0 1 12.5 35.7 0 0 0.125 0.125 0.111 0.221 0.314 0.229 81.1 22.3\nMarte, Starling 6 6 2 1 0 0 2 33.3 46.7 0 0 0.333 0.5 0.356 0.168 0.227 0.169 88 28.5\nMartinez, J.D. 3 3 3 1 0 0 0 0 14.3 0 0 1 1.333 1.01 0.399 0.601 0.429 90.8 25.7\nAlonso, Pete 3 3 1 0 0 1 2 66.7 44.4 0 0 0.333 1.333 0.647 0.333 1.329 0.645 110.7 28\nTorrens, Luis 2 1 0 0 0 0 1 50 0 1 50 0 0 0.346 0.346\nStewart, DJ 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.057 0.06 0.052 89.9 11\nNimmo, Brandon 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.448 0.952 0.581 98.7 23\nVientos, Mark 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.176 0.186 0.16 76.2 -35\nMcNeil, Jeff 2 2 2 2 0 0 0 0 0 0 0 1 2 1.229 0.447 0.746 0.485 100.5 17\nTaylor, Tyrone 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLindor, Francisco 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.847 0.953 0.791 76 23\nIglesias, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.383 0.442 0.352 89.3 5'}], 133: [{'pitcher_id': '571670', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFloro, Dylan 13 13 1 0 0 0 7 53.8 42.3 0 0 0.077 0.077 0.068 0.094 0.106 0.09 83.8 -3.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 8 8 0 0 0 0 4 50 41.2 0 0 0 0 0 0.074 0.077 0.069 80 0.3\nBrown, Seth 2 2 1 0 0 0 1 50 75 0 0 0.5 0.5 0.444 0.253 0.315 0.255 88.9 7\nBleday, JJ 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDíaz, Aledmys 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLangeliers, Shea 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.128 0.114 94 -27'}, {'pitcher_id': '680730', 'player_data': '', 'team_data': 'Mitchell Parker has not faced anyone on this team.'}, {'pitcher_id': '607179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeems, Jordan 14 12 5 1 0 0 3 21.4 24 2 14.3 0.417 0.5 0.441 0.377 0.526 0.434 94.9 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 3 2 1 0 0 0 0 0 16.7 1 33.3 0.5 0.5 0.527 0.774 1.011 0.757 107.3 9.5\nGelof, Zack 3 3 1 1 0 0 1 33.3 28.6 0 0 0.333 0.667 0.415 0.369 0.614 0.415 102.5 10.5\nBrown, Seth 2 1 1 0 0 0 0 0 25 1 50 1 1 0.792 0.348 0.385 0.512 95.8 -1\nLangeliers, Shea 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.442 0.517 0.413 93.1 15\nBleday, JJ 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.289 0.289 0.26 72.4 12\nSoderstrom, Tyler 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAndujar, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.151 0.47 0.258 93.1 32\nToro, Abraham 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.197 0.269 0.197 79.9 27'}, {'pitcher_id': '571578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorbin, Patrick 63 58 15 4 0 5 14 22.2 29.5 5 7.9 0.259 0.586 0.373 0.311 0.642 0.419 93 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 44 40 11 3 0 4 10 22.7 27.4 4 9.1 0.275 0.65 0.404 0.298 0.622 0.408 92.4 4.5\nDíaz, Aledmys 10 9 2 0 0 1 2 20 31.6 1 10 0.222 0.556 0.359 0.325 0.685 0.451 91.1 4.9\nAndujar, Miguel 9 9 2 1 0 0 2 22.2 40 0 0 0.222 0.333 0.236 0.357 0.688 0.438 97.3 8.3'}, {'pitcher_id': '669022', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGore, MacKenzie 13 12 3 0 0 0 7 53.8 42.4 1 7.7 0.25 0.25 0.259 0.188 0.224 0.218 91.3 13.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 4 3 0 0 0 0 3 75 60 1 25 0 0 0.174 0.174\nToro, Abraham 3 3 2 0 0 0 1 33.3 33.3 0 0 0.667 0.667 0.593 0.358 0.44 0.344 89.5 8.5\nSchuemann, Max 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.112 0.127 0.105 95.2 -8\nLangeliers, Shea 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.011 0.038 0.018 95.3 41\nGelof, Zack 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.444 0.465 0.517 0.428 87 17'}, {'pitcher_id': '671131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672851', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdon, Joan 13 10 2 0 0 0 2 15.4 17.2 2 15.4 0.2 0.2 0.298 0.264 0.349 0.363 91.4 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAndujar, Miguel 3 2 1 0 0 0 0 0 12.5 1 33.3 0.5 0.5 0.526 0.314 0.349 0.422 85.2 -6\nGelof, Zack 2 1 0 0 0 0 1 50 14.3 1 50 0 0 0.348 0.348\nBrown, Seth 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.285 0.328 0.265 103.5 4\nDavis, J.D. 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0.36 0.113 0.113 0.412 63.5 -1\nBleday, JJ 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.285 0.544 0.337 100.6 29.5\nSoderstrom, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.151 0.089 89.6 28\nLangeliers, Shea 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.689 0.78 0.648 102.9 7'}, {'pitcher_id': '640448', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFinnegan, Kyle 10 10 2 1 0 0 5 50 35 0 0 0.2 0.3 0.215 0.06 0.117 0.073 92.6 28', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 3 3 0 0 0 0 3 100 57.1 0 0 0 0 0 0\nBleday, JJ 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.029 0.089 0.045 92.5 34\nAndujar, Miguel 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.063 0.204 0.107 90.7 49\nSoderstrom, Tyler 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBrown, Seth 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.231 0.378 0.254 93.2 21\nDíaz, Aledmys 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.184 0.206 0.175 96.2 -13'}, {'pitcher_id': '663432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRainey, Tanner 7 6 0 0 0 0 3 42.9 53.3 1 14.3 0 0 0.099 0.086 0.129 0.176 88 45.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 6 5 0 0 0 0 2 33.3 50 1 16.7 0 0 0.115 0.103 0.155 0.205 88 45.3\nMcFarland, T.J. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '676395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Robert 12 12 3 2 0 0 3 25 23.8 0 0 0.25 0.417 0.283 0.212 0.353 0.24 95 29.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.294 0.227 0.286 0.228 93.9 20.5\nToro, Abraham 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.389 0.728 0.471 98.7 7\nGelof, Zack 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSchuemann, Max 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSoderstrom, Tyler 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.528 0.941 0.609 104 17\nBrown, Seth 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.011 0.009 87.4 61\nDíaz, Aledmys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.021 0.014 86.9 55'}, {'pitcher_id': '571882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaw, Derek 11 9 2 0 0 0 1 9.1 31.6 2 18.2 0.222 0.222 0.286 0.341 0.542 0.433 92.4 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLangeliers, Shea 3 1 0 0 0 0 0 0 50 2 66.7 0 0 0.461 0.053 0.055 0.479 84.4 -39\nDíaz, Aledmys 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.562 1.107 0.693 99.8 21\nToro, Abraham 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.087 0.063 84.2 -30\nDavis, J.D. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.19 0.209 0.179 102.2 -17\nBrown, Seth 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAndujar, Miguel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.889 0.996 0.845 75.1 23\nGelof, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.18 0.212 0.162 93.8 -16'}, {'pitcher_id': '592866', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilliams, Trevor 41 37 13 3 0 4 7 17.1 18.2 4 9.8 0.351 0.757 0.482 0.336 0.762 0.478 88 23.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 8 7 3 2 0 1 0 0 23.5 1 12.5 0.429 1.143 0.633 0.433 1.019 0.598 99.8 20.7\nBleday, JJ 8 7 4 1 0 0 1 12.5 9.1 1 12.5 0.571 0.714 0.578 0.39 0.542 0.441 82.8 36.3\nGelof, Zack 6 6 3 0 0 2 2 33.3 27.3 0 0 0.5 1.5 0.815 0.511 1.534 0.839 96.2 31\nRooker, Brent 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.232 0.015 0.021 0.242 78.4 57\nBrown, Seth 4 4 1 0 0 1 2 50 22.2 0 0 0.25 1 0.501 0.179 0.704 0.355 95 36\nDíaz, Aledmys 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.324 0.354 0.301 83.4 -0.3\nSoderstrom, Tyler 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.447 1.486 0.791 68 27\nLangeliers, Shea 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.162 0.193 0.15 81.5 -8.7\nToro, Abraham 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.093 0.112 0.288 80.6 30.5'}, {'pitcher_id': '680686', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Josiah 20 15 2 1 0 0 6 30 35.3 4 20 0.133 0.2 0.246 0.182 0.346 0.312 93.6 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 6 5 1 1 0 0 2 33.3 25 1 16.7 0.2 0.4 0.325 0.036 0.094 0.161 93.6 47\nDavis, J.D. 6 4 1 0 0 0 2 33.3 60 1 16.7 0.25 0.25 0.263 0.322 0.374 0.338 97.4 17.3\nDíaz, Aledmys 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.4 1.051 0.602 99.7 24.3\nAndujar, Miguel 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.034 0.035 0.253 63.9 -36\nToro, Abraham 2 1 0 0 0 0 1 50 50 1 50 0 0 0.344 0.344'}, {'pitcher_id': '640451', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarvey, Hunter 12 11 2 1 0 0 5 41.7 34.3 1 8.3 0.182 0.273 0.237 0.234 0.376 0.298 94.1 20.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Seth 4 4 1 0 0 0 2 50 50 0 0 0.25 0.25 0.222 0.308 0.348 0.289 100.3 2.5\nBleday, JJ 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.189 0.245 0.355 80.7 49\nDavis, J.D. 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nLangeliers, Shea 2 2 1 1 0 0 1 50 20 0 0 0.5 1 0.631 0.3 0.93 0.51 100.8 26\nGelof, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.357 0.393 0.33 101.6 -5'}, {'pitcher_id': '663623', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Jake 39 37 7 1 0 3 8 20.5 21.2 2 5.1 0.189 0.459 0.29 0.254 0.522 0.343 91.1 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 8 7 2 0 0 1 2 25 20 1 12.5 0.286 0.714 0.448 0.288 0.591 0.409 96.4 -5.4\nBleday, JJ 6 6 1 0 0 1 2 33.3 36.4 0 0 0.167 0.667 0.334 0.202 0.759 0.384 98.2 35.3\nBrown, Seth 5 4 1 0 0 0 1 20 0 1 20 0.25 0.25 0.315 0.133 0.185 0.25 87 17.3\nGelof, Zack 6 6 1 1 0 0 0 0 12.5 0 0 0.167 0.333 0.207 0.367 0.528 0.385 83.9 6.3\nLangeliers, Shea 5 5 0 0 0 0 2 40 28.6 0 0 0 0 0 0.201 0.263 0.198 89.5 24.3\nRooker, Brent 3 3 1 0 0 1 0 0 33.3 0 0 0.333 1.333 0.668 0.34 1.228 0.631 93 33\nToro, Abraham 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.047 0.092 0.055 88.2 36\nSoderstrom, Tyler 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.294 0.43 0.479 0.4 93.4 32.7'}, {'pitcher_id': '606930', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Jacob 9 8 4 1 0 1 0 0 6.3 1 11.1 0.5 1 0.629 0.381 0.669 0.474 93.3 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 5 5 2 0 0 1 0 0 8.3 0 0 0.4 1 0.571 0.438 0.866 0.547 92.7 22.4\nDavis, J.D. 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nBrown, Seth 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.183 0.204 0.198 96.9 -17\nAndujar, Miguel 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.337 0.409 0.318 93.1 2'}], 134: [{'pitcher_id': '571670', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFloro, Dylan 34 29 3 1 0 1 11 32.4 26 5 14.7 0.103 0.241 0.223 0.114 0.198 0.215 87.2 11.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 10 10 2 1 0 0 3 30 33.3 0 0 0.2 0.3 0.212 0.134 0.189 0.14 87.9 16.3\nReynolds, Bryan 4 3 0 0 0 0 2 50 20 1 25 0 0 0.173 0.105 0.12 0.244 94.3 -2\nTellez, Rowdy 5 3 0 0 0 0 0 0 0 2 40 0 0 0.276 0.047 0.111 0.315 85.8 49.7\nJoe, Connor 5 3 1 0 0 1 0 0 0 2 40 0.333 1.333 0.679 0.362 0.896 0.588 89.8 4.7\nHayes, Ke'Bryan 5 5 0 0 0 0 2 40 30 0 0 0 0 0 0.047 0.048 0.048 79.2 -19.7\nDelay, Jason 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nOlivares, Edward 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTriolo, Jared 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCruz, Oneil 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nSuwinski, Jack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.2 0.222 0.183 94.8 -11\nGonzales, Nick 0 0 0 0 0 0 0 100 0"}, {'pitcher_id': '680730', 'player_data': '', 'team_data': 'Mitchell Parker has not faced anyone on this team.'}, {'pitcher_id': '607179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeems, Jordan 12 9 2 0 0 0 4 33.3 25 3 25 0.222 0.222 0.321 0.102 0.115 0.244 83.1 1.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 3 3 1 0 0 0 2 66.7 25 0 0 0.333 0.333 0.294 0.055 0.069 0.05 84.7 -5\nTriolo, Jared 3 1 0 0 0 0 0 0 33.3 2 66.7 0 0 0.463 0.146 0.148 0.506 93.5 -21\nSuwinski, Jack 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.503 0.574 0.583 78.8 25\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.099 0.101 0.092 67.3 -40\nReynolds, Bryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTaylor, Michael A. 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nDelay, Jason 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.003 0.004 0.003 91.3 48"}, {'pitcher_id': '571578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorbin, Patrick 153 136 33 5 1 5 35 22.9 28.4 15 9.8 0.243 0.404 0.309 0.267 0.476 0.348 91.8 14.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 58 50 12 3 0 3 9 15.5 17 7 12.1 0.24 0.48 0.338 0.301 0.592 0.409 91 16.6\nGrandal, Yasmani 20 18 6 1 0 0 8 40 40 2 10 0.333 0.389 0.35 0.317 0.487 0.387 94.6 6.1\nReynolds, Bryan 20 16 5 0 0 0 4 20 24.1 3 15 0.313 0.313 0.323 0.177 0.212 0.241 89.7 18.5\nHayes, Ke'Bryan 17 16 4 0 0 0 2 11.8 20.8 1 5.9 0.25 0.25 0.248 0.279 0.42 0.32 90.3 13.6\nJoe, Connor 13 11 2 0 1 1 2 15.4 38.1 2 15.4 0.182 0.636 0.384 0.312 0.69 0.46 95.1 24.2\nTaylor, Michael A. 7 7 3 1 0 1 1 14.3 41.7 0 0 0.429 1 0.59 0.415 0.875 0.533 97.9 17\nCruz, Oneil 4 4 0 0 0 0 3 75 50 0 0 0 0 0 0.067 0.067 0.06 104.4 -16\nTellez, Rowdy 5 5 0 0 0 0 1 20 55.6 0 0 0 0 0 0.222 0.341 0.235 93.2 -6.3\nOlivares, Edward 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.153 0.163 0.139 90.9 -5\nSuwinski, Jack 3 3 0 0 0 0 2 66.7 75 0 0 0 0 0 0.002 0.004 0.002 95.3 52\nTriolo, Jared 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.139 0.14 0.127 69.8 -63\nFalter, Bailey 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0"}, {'pitcher_id': '669022', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGore, MacKenzie 42 31 4 0 0 0 12 28.6 35.2 7 16.7 0.129 0.129 0.251 0.156 0.196 0.304 85.3 13.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 8 6 1 0 0 0 1 12.5 22.2 1 12.5 0.167 0.167 0.287 0.181 0.246 0.32 79.9 27.8\nMcCutchen, Andrew 6 4 0 0 0 0 3 50 12.5 1 16.7 0 0 0.235 0.057 0.057 0.27 95.8 -9\nReynolds, Bryan 6 5 1 0 0 0 2 33.3 50 1 16.7 0.2 0.2 0.264 0.207 0.241 0.279 81.3 21.7\nHayes, Ke'Bryan 6 4 0 0 0 0 1 16.7 12.5 1 16.7 0 0 0.235 0.123 0.142 0.311 78.6 -12.7\nOlivares, Edward 5 5 0 0 0 0 2 40 46.2 0 0 0 0 0 0.17 0.261 0.18 79.3 9.7\nTellez, Rowdy 3 2 1 0 0 0 1 33.3 28.6 1 33.3 0.5 0.5 0.524 0.277 0.327 0.409 98.1 4\nSuwinski, Jack 3 1 0 0 0 0 0 0 50 2 66.7 0 0 0.459 0.014 0.031 0.466 94.6 44\nCruz, Oneil 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.296 0.189 0.207 0.17 95.8 11\nTaylor, Michael A. 2 1 0 0 0 0 1 50 75 0 0 0 0 0 0 0 0.504 108.3 18"}, {'pitcher_id': '671131', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRutledge, Jackson 10 10 5 1 0 1 0 0 5.6 0 0 0.5 0.9 0.59 0.27 0.574 0.354 87.9 34.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 3 3 3 0 0 1 0 0 0 0 0 1 2 1.256 0.678 1.473 0.903 93.1 29.7\nHayes, Ke'Bryan 3 3 1 1 0 0 0 0 16.7 0 0 0.333 0.667 0.415 0.168 0.378 0.225 92.7 46\nTriolo, Jared 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.033 0.036 0.033 73.9 4.5\nSuwinski, Jack 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.047 0.056 0.046 86.8 55"}, {'pitcher_id': '672851', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdon, Joan 24 20 7 1 0 1 3 12.5 19.5 4 16.7 0.35 0.55 0.437 0.346 0.585 0.446 86.2 4.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 7 5 0 0 0 0 1 14.3 23.1 2 28.6 0 0 0.198 0.138 0.336 0.336 90 10.8\nReynolds, Bryan 6 5 2 0 0 1 0 0 9.1 1 16.7 0.4 1 0.609 0.411 0.959 0.6 84.6 8.2\nHayes, Ke'Bryan 6 5 2 0 0 0 2 33.3 14.3 1 16.7 0.4 0.4 0.409 0.379 0.431 0.413 87.1 -14\nSuwinski, Jack 2 2 2 1 0 0 0 0 20 0 0 1 1.5 1.063 0.404 0.645 0.434 90.8 30.5\nGrandal, Yasmani 3 3 1 0 0 0 0 0 40 0 0 0.333 0.333 0.294 0.487 0.593 0.471 79.6 -10"}, {'pitcher_id': '640448', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFinnegan, Kyle 22 20 6 2 0 2 9 40.9 32.6 2 9.1 0.3 0.7 0.44 0.252 0.51 0.353 89.7 27.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 6 4 2 0 0 1 2 33.3 50 2 33.3 0.5 1.25 0.708 0.485 1.242 0.705 88.7 27\nTellez, Rowdy 4 4 0 0 0 0 2 50 16.7 0 0 0 0 0 0.19 0.238 0.187 102 20\nHayes, Ke'Bryan 3 3 1 1 0 0 2 66.7 22.2 0 0 0.333 0.667 0.414 0.133 0.139 0.12 61.7 18\nReynolds, Bryan 4 4 2 0 0 1 1 25 33.3 0 0 0.5 1.25 0.738 0.332 0.725 0.443 92.1 23.7\nSuwinski, Jack 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.003 0.003 0.003 85.4 72\nTaylor, Michael A. 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nJoe, Connor 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.554 0.88 0.601 101.6 16\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.077 0.049 80.9 34"}, {'pitcher_id': '663432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRainey, Tanner 24 18 5 1 0 0 5 20.8 36.7 6 25 0.278 0.333 0.373 0.303 0.594 0.457 92.5 26.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 5 4 0 0 0 0 2 40 26.7 1 20 0 0 0.138 0.14 0.17 0.249 99.2 33.5\nMcCutchen, Andrew 4 1 0 0 0 0 0 0 0 3 75 0 0 0.518 0.006 0.009 0.52 89.1 46\nReynolds, Bryan 4 3 2 1 0 0 0 0 20 1 25 0.667 1 0.707 0.619 1.572 0.852 94.1 33\nHayes, Ke'Bryan 3 2 0 0 0 0 0 0 50 1 33.3 0 0 0.231 0.258 0.287 0.389 84.5 -0.5\nSuwinski, Jack 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.335 1.355 0.688 98.7 56.5\nTaylor, Michael A. 2 2 2 0 0 0 0 0 50 0 0 1 1 0.889 0.773 0.831 0.707 86.4 14.5\nCruz, Oneil 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nGrandal, Yasmani 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTriolo, Jared 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.302 0.346 0.275 93.6 -5"}, {'pitcher_id': '676395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Robert 12 11 2 0 0 1 5 41.7 40 1 8.3 0.182 0.455 0.306 0.151 0.336 0.244 81.5 11', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.023 0.035 0.022 71.7 -19\nTellez, Rowdy 2 1 0 0 0 0 0 0 50 1 50 0 0 0.346 0.46 0.496 0.555 102.8 -1\nSuwinski, Jack 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.695 0.301 0.951 0.514 90.9 53.5\nMcCutchen, Andrew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCruz, Oneil 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nTriolo, Jared 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.038 0.022 95.5 45\nTaylor, Michael A. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.236 0.239 0.212 37 -66"}, {'pitcher_id': '571882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaw, Derek 49 44 14 2 0 1 5 10.2 20.5 4 8.2 0.318 0.432 0.364 0.336 0.543 0.408 88.9 9.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 9 8 0 0 0 0 3 33.3 33.3 1 11.1 0 0 0.077 0.218 0.46 0.327 94.2 23.6\nJoe, Connor 6 6 4 0 0 1 0 0 18.2 0 0 0.667 1.167 0.79 0.55 0.909 0.627 79.2 2.2\nReynolds, Bryan 5 4 1 0 0 0 1 20 30.8 1 20 0.25 0.25 0.315 0.187 0.299 0.303 88.6 29.7\nMcCutchen, Andrew 7 6 3 1 0 0 0 0 27.3 1 14.3 0.5 0.667 0.532 0.454 0.847 0.566 95.9 18.8\nSuwinski, Jack 5 5 1 1 0 0 1 20 12.5 0 0 0.2 0.4 0.252 0.184 0.357 0.23 91.7 7.5\nHayes, Ke'Bryan 6 5 2 0 0 0 0 0 14.3 0 0 0.4 0.4 0.415 0.389 0.407 0.414 87.6 1.2\nTaylor, Michael A. 4 3 1 0 0 0 0 0 28.6 1 25 0.333 0.333 0.395 0.193 0.285 0.328 79.4 1.3\nOlivares, Edward 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.262 0.646 0.373 93.4 -4.5\nTellez, Rowdy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.696 0.753 0.631 88.5 10\nCruz, Oneil 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.406 0.452 0.373 92.7 2.5\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.107 0.082 89 -30"}, {'pitcher_id': '592866', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilliams, Trevor 53 45 12 2 0 0 10 18.9 20 6 11.3 0.267 0.311 0.298 0.236 0.307 0.285 80.7 9.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 9 6 2 0 0 0 3 33.3 26.7 3 33.3 0.333 0.333 0.427 0.133 0.188 0.323 83 -0.3\nGrandal, Yasmani 11 9 3 1 0 0 0 0 0 2 18.2 0.333 0.444 0.398 0.247 0.286 0.317 75.4 10.2\nTaylor, Michael A. 6 5 1 0 0 0 3 50 41.7 0 0 0.2 0.2 0.178 0.21 0.234 0.195 75.6 -13.7\nReynolds, Bryan 8 8 2 0 0 0 1 12.5 15.4 0 0 0.25 0.25 0.22 0.257 0.323 0.249 84 16\nTellez, Rowdy 4 3 1 0 0 0 0 0 25 1 25 0.333 0.333 0.394 0.315 0.321 0.386 66.8 11.3\nOlivares, Edward 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.294 0.187 0.199 0.136 76.5 10.7\nHayes, Ke'Bryan 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.077 0.089 0.07 95.8 -28.5\nTriolo, Jared 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.086 0.287 0.15 96.3 33\nCruz, Oneil 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.205 0.587 0.327 102.7 11.5\nSuwinski, Jack 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.422 0.496 0.412 79.7 36.5\nJoe, Connor 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.656 0.705 0.597 79.6 18"}, {'pitcher_id': '680686', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Josiah 53 47 13 3 0 2 11 20.8 25 5 9.4 0.277 0.468 0.346 0.276 0.466 0.354 88.9 16.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 12 11 6 2 0 1 2 16.7 25 1 8.3 0.545 1 0.655 0.545 0.904 0.624 88.9 20.3\nSuwinski, Jack 9 7 2 0 0 1 3 33.3 25 1 11.1 0.286 0.714 0.398 0.24 0.452 0.348 97 28.8\nJoe, Connor 8 7 0 0 0 0 2 25 35.3 1 12.5 0 0 0.087 0.177 0.28 0.26 91 9.6\nHayes, Ke'Bryan 9 8 0 0 0 0 1 11.1 15.4 1 11.1 0 0 0.077 0.07 0.081 0.139 82.2 -1\nTriolo, Jared 4 4 0 0 0 0 2 50 27.3 0 0 0 0 0 0.14 0.26 0.169 96.2 21.5\nCruz, Oneil 3 3 2 0 0 0 0 0 40 0 0 0.667 0.667 0.593 0.259 0.287 0.239 96.7 13.7\nMcCutchen, Andrew 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0 0 0.346 57 65\nTellez, Rowdy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.123 0.142 0.115 79.4 20.5\nTaylor, Michael A. 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.833 1.857 1.129 93.1 22\nDelay, Jason 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.117 0.137 0.106 88.8 -1"}, {'pitcher_id': '640451', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarvey, Hunter 8 8 0 0 0 0 6 75 47.4 0 0 0 0 0 0.064 0.07 0.059 61.2 10', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nReynolds, Bryan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSuwinski, Jack 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.325 0.328 0.297 34.9 26\nMcCutchen, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nJoe, Connor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.23 0.172 87.4 -6\nHayes, Ke'Bryan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nCruz, Oneil 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0"}, {'pitcher_id': '663623', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Jake 2 1 0 0 0 0 1 50 20 1 50 0 0 0.348 0.348', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 2 1 0 0 0 0 1 50 20 1 50 0 0 0.348 0.348'}, {'pitcher_id': '606930', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Jacob 17 15 1 0 0 0 3 17.6 18.8 2 11.8 0.067 0.067 0.133 0.219 0.461 0.331 90.8 26.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 10 10 1 0 0 0 2 20 17.4 0 0 0.1 0.1 0.088 0.172 0.343 0.211 90.2 24.8\nGrandal, Yasmani 4 2 0 0 0 0 1 25 40 2 50 0 0 0.345 0.377 0.911 0.617 108 19\nJoe, Connor 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.33 0.578 0.392 82.4 34\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.154 0.502 0.27 95.1 31'}], 135: [{'pitcher_id': '571670', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFloro, Dylan 22 21 4 1 0 0 5 22.7 25.6 1 4.5 0.19 0.238 0.206 0.247 0.313 0.261 91.1 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 8 8 0 0 0 0 4 50 50 0 0 0 0 0 0.098 0.109 0.089 90 1.8\nCronenworth, Jake 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.197 0.211 0.18 80.1 21\nMachado, Manny 5 5 2 0 0 0 0 0 16.7 0 0 0.4 0.4 0.351 0.295 0.414 0.302 93.2 28.6\nTatis Jr., Fernando 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.384 0.423 0.528 87 2\nPeralta, David 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.043 0.68 0.942 0.682 104 10\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.4 0.47 0.382 95.8 2'}, {'pitcher_id': '680730', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParker, Mitchell 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.196 0.222 0.179 88.6 -6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.196 0.222 0.179 88.6 -6'}, {'pitcher_id': '607179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeems, Jordan 6 6 1 0 0 1 0 0 0 0 0 0.167 0.667 0.334 0.352 0.81 0.482 88.8 22.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.004 0.009 0.005 84 62\nKim, Ha-Seong 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.9 3.52 1.78 104.6 29\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.067 0.069 0.065 65 -33\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.471 0.5 0.424 87.4 8\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.663 0.752 0.614 107.8 6'}, {'pitcher_id': '571578', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorbin, Patrick 84 74 26 6 0 2 6 7.1 16.4 6 7.1 0.351 0.514 0.389 0.306 0.466 0.358 89 8.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 23 21 7 1 0 1 2 8.7 25.5 1 4.3 0.333 0.524 0.367 0.256 0.414 0.286 88.9 -0.6\nTatis Jr., Fernando 12 9 5 1 0 0 1 8.3 17.6 3 25 0.556 0.667 0.569 0.344 0.451 0.431 92.9 4.3\nProfar, Jurickson 10 7 3 1 0 0 0 0 18.2 2 20 0.429 0.571 0.44 0.384 0.514 0.445 94.1 15.4\nKim, Ha-Seong 12 11 4 1 0 1 0 0 0 0 0 0.364 0.727 0.478 0.376 0.627 0.453 91.8 14.5\nArraez, Luis 6 6 2 1 0 0 1 16.7 0 0 0 0.333 0.5 0.351 0.299 0.361 0.288 85 12.2\nSolano, Donovan 7 7 0 0 0 0 1 14.3 9.1 0 0 0 0 0 0.254 0.42 0.281 91.3 10.7\nCronenworth, Jake 5 4 3 0 0 0 1 20 28.6 0 0 0.75 0.75 0.661 0.352 0.398 0.331 65.5 3\nHigashioka, Kyle 5 5 1 1 0 0 0 0 0 0 0 0.2 0.4 0.249 0.153 0.408 0.23 89.8 12.8\nAzocar, José 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.442 0.564 0.989 0.679 87.5 27.5\nWade, Tyler 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.158 0.129 76.8 -13.5'}, {'pitcher_id': '669022', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGore, MacKenzie 27 22 3 0 0 0 9 33.3 32.8 5 18.5 0.136 0.136 0.227 0.172 0.309 0.294 92.3 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronenworth, Jake 5 2 0 0 0 0 2 40 27.3 3 60 0 0 0.418 0.418\nTatis Jr., Fernando 6 5 1 0 0 0 2 33.3 53.8 1 16.7 0.2 0.2 0.263 0.293 0.584 0.419 91.9 15.7\nKim, Ha-Seong 6 5 0 0 0 0 2 33.3 54.5 1 16.7 0 0 0.116 0.079 0.085 0.177 88.1 11.7\nArraez, Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.229 0.278 0.217 88.4 3.3\nProfar, Jurickson 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.003 0.004 0.003 89.5 55\nMachado, Manny 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.435 0.637 0.46 114.5 11\nHigashioka, Kyle 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.183 0.667 0.343 95.1 34.5'}, {'pitcher_id': '671131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '672851', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdon, Joan 11 10 2 0 0 0 0 0 6.3 1 9.1 0.2 0.2 0.223 0.283 0.339 0.31 80.8 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 6 6 2 0 0 0 0 0 10 0 0 0.333 0.333 0.294 0.371 0.444 0.358 83.6 22\nWade, Tyler 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.252 0.275 0.461 34.9 3\nPeralta, David 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.118 0.149 0.112 90.5 17.7'}, {'pitcher_id': '640448', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFinnegan, Kyle 27 25 6 1 0 0 6 22.2 21.1 2 7.4 0.24 0.28 0.261 0.246 0.443 0.321 87.5 16.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 4 4 2 1 0 0 0 0 7.7 0 0 0.5 0.75 0.535 0.511 0.695 0.519 87.9 14.5\nTatis Jr., Fernando 5 5 2 0 0 0 2 40 35.7 0 0 0.4 0.4 0.352 0.197 0.685 0.359 97.1 47\nCronenworth, Jake 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.293 0.109 0.147 0.112 86.4 12.5\nSolano, Donovan 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.032 0.037 0.029 71.6 -6\nProfar, Jurickson 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.316 0.572 0.382 90.5 4.8\nPeralta, David 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.171 0.63 0.513 98.4 37\nKim, Ha-Seong 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.23 0.049 0.049 0.259 53.4 48\nArraez, Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.55 0.627 0.504 94.2 6\nHigashioka, Kyle 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.054 0.062 0.049 72.4 -19'}, {'pitcher_id': '663432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRainey, Tanner 7 5 2 0 0 0 1 14.3 42.9 2 28.6 0.4 0.4 0.448 0.341 0.41 0.431 97.9 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nPeralta, David 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.337 0.378 0.315 102.6 31.5\nWade, Tyler 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nArraez, Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.947 1.215 0.932 95.9 13\nMachado, Manny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.082 0.082 0.074 90.4 -30'}, {'pitcher_id': '676395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcia, Robert 3 3 0 0 0 0 2 66.7 16.7 0 0 0 0 0 0.017 0.023 0.016 74.9 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 2 2 0 0 0 0 2 100 25 0 0 0 0 0 0\nHigashioka, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.07 0.049 74.9 -7'}, {'pitcher_id': '571882', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaw, Derek 10 9 1 0 0 0 2 20 22.2 1 10 0.111 0.111 0.158 0.216 0.27 0.261 87 1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.34 0.385 0.318 95 -7\nPeralta, David 4 4 0 0 0 0 2 50 28.6 0 0 0 0 0 0.143 0.154 0.139 86.9 -26\nKim, Ha-Seong 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nArraez, Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.336 0.5 0.351 79.1 8.5\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.038 0.025 87.1 60'}, {'pitcher_id': '592866', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilliams, Trevor 66 62 19 2 2 3 4 6.1 10.8 2 3 0.306 0.548 0.38 0.318 0.534 0.383 89.8 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 24 22 8 1 1 1 3 12.5 14 0 0 0.364 0.636 0.446 0.3 0.539 0.39 90.4 4.1\nArraez, Luis 19 18 7 1 0 1 0 0 6.5 1 5.3 0.389 0.611 0.439 0.341 0.502 0.378 88.8 14.9\nMachado, Manny 6 6 1 0 0 0 0 0 14.3 0 0 0.167 0.167 0.145 0.247 0.379 0.27 88.5 25.2\nCronenworth, Jake 4 4 1 0 1 0 1 25 11.1 0 0 0.25 0.75 0.4 0.441 0.707 0.486 100.2 15\nProfar, Jurickson 7 7 1 0 0 0 0 0 12.5 0 0 0.143 0.143 0.124 0.176 0.264 0.178 85.4 36.6\nTatis Jr., Fernando 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.49 0.737 0.523 98.5 9\nKim, Ha-Seong 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.506 1.536 0.825 84.2 14.5\nSolano, Donovan 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689'}, {'pitcher_id': '680686', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Josiah 37 33 7 2 0 1 5 13.5 19.4 4 10.8 0.212 0.364 0.292 0.182 0.312 0.26 85.6 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 6 5 1 0 0 0 1 16.7 33.3 1 16.7 0.2 0.2 0.262 0.085 0.1 0.179 86.8 25.5\nArraez, Luis 6 6 2 1 0 1 0 0 0 0 0 0.333 1 0.541 0.274 0.702 0.397 89.3 13.3\nCronenworth, Jake 6 5 1 0 0 0 0 0 0 1 16.7 0.2 0.2 0.262 0.215 0.367 0.323 88.8 41.6\nProfar, Jurickson 6 5 1 0 0 0 0 0 23.1 1 16.7 0.2 0.2 0.262 0.22 0.253 0.288 73.7 8.6\nKim, Ha-Seong 6 6 0 0 0 0 1 16.7 22.2 0 0 0 0 0 0.084 0.133 0.092 82.8 9.2\nSolano, Donovan 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nPeralta, David 2 1 1 1 0 0 0 0 0 1 50 1 2 0.975 0.54 0.886 0.641 104.3 16\nTatis Jr., Fernando 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.237 0.265 0.217 92.4 -11.5'}, {'pitcher_id': '640451', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarvey, Hunter 16 16 3 0 0 0 3 18.8 18.9 0 0 0.188 0.188 0.166 0.221 0.282 0.218 88.4 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronenworth, Jake 5 5 2 0 0 0 1 20 15.4 0 0 0.4 0.4 0.353 0.294 0.329 0.271 90.8 0.2\nPeralta, David 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.018 0.058 0.028 87.3 63\nArraez, Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.516 0.595 0.487 88.2 4.7\nProfar, Jurickson 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSolano, Donovan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.263 0.694 0.403 96.1 27\nMachado, Manny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.01 0.011 0.009 86.3 51\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.18 0.184 0.168 78.4 -49'}, {'pitcher_id': '663623', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIrvin, Jake 22 20 5 2 0 0 1 4.5 10.3 2 9.1 0.25 0.35 0.298 0.247 0.359 0.3 88.6 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 12 12 3 2 0 0 0 0 0 0 0 0.25 0.417 0.283 0.298 0.468 0.325 91.7 11.3\nPeralta, David 6 6 2 0 0 0 0 0 13.3 0 0 0.333 0.333 0.294 0.219 0.251 0.209 87.1 23\nCronenworth, Jake 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nTatis Jr., Fernando 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.056 0.06 0.376 60.5 -28\nKim, Ha-Seong 0'}, {'pitcher_id': '606930', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBarnes, Jacob 22 20 6 0 0 2 1 4.5 15 2 9.1 0.3 0.6 0.406 0.382 0.783 0.508 93.8 6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 4 3 0 0 0 0 0 0 25 1 25 0 0 0.173 0.326 0.525 0.454 99.9 0.7\nMachado, Manny 4 4 1 0 0 1 0 0 11.1 0 0 0.25 1 0.502 0.429 0.961 0.58 97.2 16.5\nTatis Jr., Fernando 3 3 2 0 0 1 1 33.3 50 0 0 0.667 1.667 0.963 0.381 1.39 0.711 93.3 11\nPeralta, David 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.321 0.368 0.302 82.4 -5.5\nKim, Ha-Seong 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.268 0.27 0.469 91.3 -3\nArraez, Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.487 0.627 0.486 92.2 22\nSolano, Donovan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.209 0.222 0.202 98.2 -6\nWade, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.075 0.12 0.068 70.1 -25\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.25 0.271 0.225 98.6 -13\nCampusano, Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.606 2.207 1.116 100.6 29\nHigashioka, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.789 0.988 0.78 99.5 14'}]}, 121: {120: [{'pitcher_id': '621242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Edwin 22 20 4 2 0 1 7 31.8 30.2 2 9.1 0.2 0.45 0.304 0.242 0.444 0.323 94.6 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 4 3 0 0 0 0 3 75 28.6 1 25 0 0 0.172 0.172\nGarcía Jr., Luis 6 6 1 1 0 0 1 16.7 25 0 0 0.167 0.333 0.207 0.213 0.27 0.211 91.7 12.6\nWinker, Jesse 4 3 2 1 0 1 1 25 20 1 25 0.667 2 0.968 0.551 1.496 0.787 103.3 17.5\nRosario, Eddie 5 5 1 0 0 0 2 40 45.5 0 0 0.2 0.2 0.177 0.256 0.41 0.285 94.7 26\nVargas, Ildemaro 2 2 0 0 0 0 0 0 42.9 0 0 0 0 0 0.3 0.334 0.273 95.8 -5\nRuiz, Keibert 1 1 0 0 0 0 0 0 16.7 0 0 0 0 0 0.026 0.055 0.033 89.6 36'}, {'pitcher_id': '656731', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Tylor 67 59 17 4 0 2 13 19.4 22 7 10.4 0.288 0.458 0.352 0.303 0.537 0.386 91.9 14.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 14 13 2 0 0 1 5 35.7 29.2 1 7.1 0.154 0.385 0.257 0.246 0.452 0.322 97.1 10.6\nRosario, Eddie 15 12 6 2 0 1 2 13.3 17.4 2 13.3 0.5 0.917 0.569 0.294 0.553 0.378 96.3 24.2\nRuiz, Keibert 9 8 3 1 0 0 3 33.3 25 1 11.1 0.375 0.5 0.414 0.396 0.727 0.506 90.4 22.4\nGarcía Jr., Luis 11 10 1 1 0 0 3 27.3 35.3 1 9.1 0.1 0.2 0.176 0.193 0.527 0.333 84.6 21.6\nMeneses, Joey 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.352 0.401 0.328 86.3 13\nVargas, Ildemaro 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.178 0.297 0.2 86.9 -3.7\nAbrams, CJ 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.156 0.193 0.144 95.5 -18.5\nSenzel, Nick 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.596 1.09 0.708 87.7 25\nWinker, Jesse 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.823 0.783 0.861 0.708 89.1 12\nMillas, Drew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.447 0.504 0.408 101.5 -0.5'}, {'pitcher_id': '518617', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDiekman, Jake 7 5 0 0 0 0 4 57.1 40 1 14.3 0 0 0.098 0.135 0.161 0.275 100.5 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.224 0.269 0.222 100.3 4\nWinker, Jesse 2 1 0 0 0 0 1 50 40 1 50 0 0 0.344 0.344\nMeneses, Joey 1 0 0 0 0 0 0 0 0 0 0 0 0.57 100.6 37\nMillas, Drew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '500779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuintana, Jose 81 80 22 4 0 1 14 17.3 16.7 1 1.2 0.275 0.362 0.279 0.239 0.333 0.251 87.1 6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 17 16 4 0 0 0 3 17.6 17.5 1 5.9 0.25 0.25 0.248 0.243 0.289 0.261 84.2 8.4\nSenzel, Nick 18 18 7 0 0 1 0 0 12 0 0 0.389 0.556 0.404 0.279 0.464 0.313 89.8 9.2\nWinker, Jesse 15 15 3 1 0 0 5 33.3 21.9 0 0 0.2 0.267 0.199 0.207 0.3 0.214 87.8 0.1\nVargas, Ildemaro 6 6 1 0 0 0 3 50 26.7 0 0 0.167 0.167 0.147 0.091 0.121 0.09 92.4 -2.7\nThomas, Lane 8 8 2 0 0 0 1 12.5 9.1 0 0 0.25 0.25 0.221 0.339 0.404 0.326 82.7 11.1\nMeneses, Joey 5 5 2 1 0 0 0 0 8.3 0 0 0.4 0.6 0.429 0.314 0.393 0.303 88.2 1.2\nRuiz, Keibert 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.182 0.206 0.171 85.2 19.2\nAbrams, CJ 2 2 0 0 0 0 0 0 22.2 0 0 0 0 0 0.044 0.055 0.044 84.7 0.5\nGarcía Jr., Luis 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.631 0.23 0.568 0.337 99.8 23\nYoung, Jacob 3 3 1 1 0 0 1 33.3 14.3 0 0 0.333 0.667 0.415 0.278 0.324 0.264 84.1 -14.5'}, {'pitcher_id': '657585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Reed 10 8 2 0 0 0 2 20 23.5 2 20 0.25 0.25 0.316 0.242 0.268 0.317 93.4 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.319 0.363 0.298 93.4 27.5\nMeneses, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.267 0.286 0.243 99.1 -5\nRosario, Eddie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRuiz, Keibert 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nWinker, Jesse 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nAbrams, CJ 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.006 0.012 0.349 92.8 43\nYoung, Jacob 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.469 0.533 0.437 94.9 6\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.56 0.588 0.504 86.7 10'}, {'pitcher_id': '493603', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOttavino, Adam 37 33 5 2 0 1 10 27 30.3 3 8.1 0.152 0.303 0.247 0.192 0.299 0.266 87.7 3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 8 7 1 0 0 1 3 37.5 31.3 1 12.5 0.143 0.571 0.345 0.244 0.499 0.362 92.8 12.3\nGarcía Jr., Luis 6 5 1 1 0 0 2 33.3 41.7 1 16.7 0.2 0.4 0.326 0.112 0.214 0.232 89.7 -4.7\nSenzel, Nick 4 3 0 0 0 0 1 25 25 1 25 0 0 0.172 0.048 0.057 0.207 82.1 16.5\nMeneses, Joey 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.094 0.094 0.087 73 -30.5\nRuiz, Keibert 6 5 0 0 0 0 0 0 0 0 0 0 0 0.121 0.119 0.133 0.21 88.3 4\nAbrams, CJ 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.263 0.312 0.254 93.3 -6\nRosario, Eddie 2 2 1 1 0 0 1 50 42.9 0 0 0.5 1 0.622 0.151 0.449 0.236 97 29\nWinker, Jesse 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.613 0.723 0.594 80.2 22.5\nCorbin, Patrick 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0\nVargas, Ildemaro 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.265 0.286 0.239 88.8 0'}, {'pitcher_id': '656887', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReid-Foley, Sean 3 3 2 0 0 0 0 0 16.7 0 0 0.667 0.667 0.587 0.258 0.314 0.244 88.9 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, Keibert 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.086 0.165 0.105 85.6 28\nVargas, Ildemaro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.405 0.462 0.365 99.6 -1\nGarcía Jr., Luis 1 1 1 0 0 0 0 0 50 0 0 1 1 0.879 0.284 0.316 0.262 81.5 4'}, {'pitcher_id': '676130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nButtó, José 24 21 5 1 0 1 2 8.3 12.9 3 12.5 0.238 0.429 0.333 0.3 0.469 0.372 83.1 0.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, Keibert 6 5 2 0 0 1 0 0 0 1 16.7 0.4 1 0.597 0.43 0.923 0.584 85.6 30.4\nMeneses, Joey 6 6 2 0 0 0 1 16.7 22.2 0 0 0.333 0.333 0.294 0.282 0.325 0.263 80.3 1.4\nGarcía Jr., Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.351 0.383 0.323 88.6 -7\nThomas, Lane 2 1 0 0 0 0 0 0 66.7 1 50 0 0 0.348 0.21 0.214 0.445 57.9 -80\nYoung, Jacob 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.622 0.24 0.536 0.321 100.8 21\nAbrams, CJ 5 4 0 0 0 0 0 0 0 1 20 0 0 0.139 0.178 0.211 0.27 81.2 -16.3'}, {'pitcher_id': '642121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSulser, Cole 14 12 3 2 0 0 0 0 16.7 2 14.3 0.25 0.417 0.34 0.331 0.504 0.407 88.5 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, Keibert 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.259 0.329 0.256 82.1 4.7\nMeneses, Joey 2 0 0 0 0 0 0 0 25 2 100 0.689 0.689\nThomas, Lane 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.418 0.855 0.532 100.6 7.5\nCorbin, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.726 0.852 0.721 67.9 36\nRosario, Eddie 3 3 1 1 0 0 0 0 50 0 0 0.333 0.667 0.414 0.207 0.334 0.228 86.4 20\nVargas, Ildemaro 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.3 0.644 0.392 97.3 24\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.022 0.015 98 54\nAbrams, CJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.7 0.828 0.672 91.6 10'}, {'pitcher_id': '548384', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRaley, Brooks 21 19 4 1 0 1 4 19 13.2 1 4.8 0.211 0.421 0.272 0.228 0.349 0.26 88.1 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 5 5 2 1 0 0 0 0 10 0 0 0.4 0.6 0.425 0.42 0.584 0.438 81.4 27.8\nWinker, Jesse 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.183 0.247 0.184 92.7 17.3\nAbrams, CJ 3 3 1 0 0 1 1 33.3 14.3 0 0 0.333 1.333 0.668 0.19 0.552 0.305 96.2 27\nGarcía Jr., Luis 2 1 1 0 0 0 0 0 0 0 0 1 1 0.441 0.731 0.862 0.364 93.5 20.5\nVargas, Ildemaro 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.042 0.043 0.039 92.8 10\nMeneses, Joey 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nThomas, Lane 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRuiz, Keibert 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSenzel, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.078 0.07 66.5 -7'}, {'pitcher_id': '673380', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNúñez, Dedniel 4 4 0 0 0 0 1 25 44.4 0 0 0 0 0 0.061 0.076 0.058 83.3 41.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSenzel, Nick 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.011 0.013 0.011 82.3 80\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.206 0.232 0.186 75.3 5\nMeneses, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.06 0.034 92.4 39'}, {'pitcher_id': '605288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHouser, Adrian 56 53 16 1 0 6 6 10.7 12.1 3 5.4 0.302 0.66 0.413 0.317 0.547 0.383 88.5 11.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 23 22 9 0 0 5 3 13 13.9 1 4.3 0.409 1.091 0.614 0.36 0.728 0.468 92.7 6.7\nSenzel, Nick 12 12 1 0 0 0 2 16.7 10 0 0 0.083 0.083 0.074 0.241 0.304 0.237 86.3 11.4\nRosario, Eddie 10 10 4 1 0 1 1 10 7.1 0 0 0.4 0.8 0.499 0.37 0.659 0.428 90 11.7\nThomas, Lane 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.82 0.786 0.893 0.726 76.7 21\nMillas, Drew 2 1 0 0 0 0 0 0 20 1 50 0 0 0.346 0.48 0.515 0.562 66.2 40\nCorbin, Patrick 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.024 0.027 0.026 85 0\nRuiz, Keibert 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.122 0.064 90.2 32\nVargas, Ildemaro 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.047 0.049 0.043 70.1 -20\nGarcía Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.05 0.099 0.06 88.2 42'}, {'pitcher_id': '640455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManaea, Sean 42 40 11 1 1 0 8 19 7.2 2 4.8 0.275 0.35 0.289 0.273 0.451 0.329 87.1 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeneses, Joey 8 7 1 0 0 0 0 0 8.3 1 12.5 0.143 0.143 0.197 0.321 0.727 0.473 92.9 19.4\nThomas, Lane 8 8 2 1 0 0 3 37.5 9.1 0 0 0.25 0.375 0.266 0.315 0.582 0.382 95.9 14.4\nVargas, Ildemaro 6 6 1 0 1 0 1 16.7 9.1 0 0 0.167 0.5 0.261 0.163 0.338 0.21 82.6 40.4\nGarcía Jr., Luis 5 5 2 0 0 0 2 40 11.1 0 0 0.4 0.4 0.353 0.271 0.353 0.268 82.9 29.3\nRosario, Eddie 5 4 2 0 0 0 0 0 14.3 1 20 0.5 0.5 0.491 0.314 0.368 0.395 78 11.7\nAbrams, CJ 6 6 1 0 0 0 1 16.7 0 0 0 0.167 0.167 0.147 0.296 0.333 0.279 81.4 4.6\nRuiz, Keibert 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.281 0.313 0.26 89.2 -2.7\nWinker, Jesse 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '664192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucchesi, Joey 25 23 7 1 0 0 3 12 14.6 1 4 0.304 0.348 0.301 0.255 0.406 0.3 86.5 2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 5 4 2 0 0 0 1 20 10 1 20 0.5 0.5 0.492 0.359 0.416 0.408 88.9 32.3\nRuiz, Keibert 4 4 2 1 0 0 0 0 14.3 0 0 0.5 0.75 0.532 0.517 1.055 0.658 92.3 22.3\nCorbin, Patrick 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.22 0.133 0.152 0.13 87.4 -19.7\nAbrams, CJ 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.078 0.072 67.4 -31.7\nMeneses, Joey 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.292 0.458 0.321 96.4 4.5\nVargas, Ildemaro 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0 0 0 88.2 45\nWinker, Jesse 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.217 0.386 0.25 96.2 6\nGarcía Jr., Luis 1 0 0 0 0 0 0 0 0 0 0 49.7 -66'}, {'pitcher_id': '605347', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Jorge 20 18 6 1 0 1 0 0 9.8 2 10 0.333 0.556 0.405 0.244 0.519 0.352 90.7 18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 12 11 4 1 0 1 0 0 8.3 1 8.3 0.364 0.727 0.471 0.285 0.707 0.43 95.1 30.5\nWinker, Jesse 3 2 0 0 0 0 0 0 14.3 1 33.3 0 0 0.23 0.164 0.164 0.331 82.1 -17\nVargas, Ildemaro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.206 0.227 0.189 100.1 -14\nMeneses, Joey 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.047 0.066 0.045 74.7 -22\nRuiz, Keibert 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.146 0.287 0.179 93.2 26\nSenzel, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.371 0.406 0.334 62.6 42\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.16 0.25 0.146 91.5 -5'}, {'pitcher_id': '622663', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeverino, Luis 49 47 10 1 0 1 8 16.3 20.5 2 4.1 0.213 0.298 0.238 0.315 0.497 0.359 88.4 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 12 11 3 1 0 1 2 16.7 28.6 1 8.3 0.273 0.636 0.397 0.36 0.793 0.493 93.8 25.4\nThomas, Lane 6 6 0 0 0 0 1 16.7 9.1 0 0 0 0 0 0.227 0.245 0.209 89.7 -4.6\nAbrams, CJ 6 6 0 0 0 0 2 33.3 23.1 0 0 0 0 0 0.147 0.508 0.266 90.7 23.3\nWinker, Jesse 3 3 2 0 0 0 1 33.3 12.5 0 0 0.667 0.667 0.593 0.273 0.321 0.262 93 -3\nRuiz, Keibert 6 6 2 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.635 0.756 0.607 95.5 7.7\nVargas, Ildemaro 5 4 1 0 0 0 0 0 0 1 20 0.25 0.25 0.317 0.279 0.326 0.349 77.7 14\nMeneses, Joey 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.517 0.632 0.493 93.3 12.3\nYoung, Jacob 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.296 0.203 0.223 0.184 77.1 -6.7\nSenzel, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.002 0.002 0.002 72 82\nGarcía Jr., Luis 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.296 0.234 0.244 0.211 70.9 -13'}, {'pitcher_id': '677020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Josh 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.282 0.306 0.259 100.2 -6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.282 0.306 0.259 100.2 -6'}, {'pitcher_id': '664849', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYoung, Danny 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.311 0.353 0.294 83.4 -22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.294 0.303 0.268 62 -51\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.64 0.755 0.615 104.8 7\nNuñez, Nasim 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '622098', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Drew 20 19 6 1 0 1 6 30 20.8 1 5 0.316 0.526 0.377 0.216 0.342 0.261 92.5 26.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 5 5 0 0 0 0 3 60 23.1 0 0 0 0 0 0.001 0.001 0.001 78 71.5\nWinker, Jesse 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.532 0.44 0.688 0.479 91.7 14.5\nRosario, Eddie 2 2 0 0 0 0 2 100 30 0 0 0 0 0 0\nMeneses, Joey 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.442 0.307 0.37 0.289 97.2 -4.5\nGarcía Jr., Luis 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.002 0.002 0.002 87.6 71\nSenzel, Nick 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.036 0.255 0.77 0.427 99.4 34.5\nYoung, Jacob 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nAbrams, CJ 1 1 1 0 0 0 0 0 50 0 0 1 1 0.884 0.426 0.472 0.384 99.1 2\nRuiz, Keibert 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.789 0.988 0.78 100.1 14'}, {'pitcher_id': '701643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHartwig, Grant 3 3 1 0 0 0 2 66.7 27.3 0 0 0.333 0.333 0.294 0.245 0.317 0.243 98.9 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeneses, Joey 1 1 1 0 0 0 0 0 20 0 0 1 1 0.883 0.734 0.952 0.729 98.9 15\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nSenzel, Nick 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '681035', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 133: [{'pitcher_id': '621242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Edwin 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.014 0.017 0.012 75 -22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.021 0.025 0.018 75 -22\nBleday, JJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '656731', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Tylor 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.146 0.165 0.43 79.3 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.146 0.165 0.43 79.3 0'}, {'pitcher_id': '518617', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDiekman, Jake 21 20 0 0 0 0 9 42.9 46.5 1 4.8 0 0 0.033 0.098 0.154 0.136 90.3 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nToro, Abraham 7 6 0 0 0 0 4 57.1 50 1 14.3 0 0 0.099 0.085 0.092 0.165 82.6 15\nDíaz, Aledmys 6 6 0 0 0 0 1 16.7 25 0 0 0 0 0 0.123 0.228 0.146 92.3 12.2\nBrown, Seth 4 4 0 0 0 0 2 50 55.6 0 0 0 0 0 0.109 0.114 0.1 95.9 37.5\nDavis, J.D. 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.097 0.31 0.167 95 30\nRooker, Brent 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAndujar, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.096 0.089 79.8 -21'}, {'pitcher_id': '500779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuintana, Jose 13 10 3 1 0 0 2 15.4 18.2 2 15.4 0.3 0.4 0.365 0.339 0.407 0.39 89.4 8.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCameron, Daz 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.295 0.251 0.302 0.245 99.6 -14.5\nDíaz, Aledmys 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.154 0.263 0.345 94.2 20\nNewcomb, Sean 2 0 0 0 0 0 0 0 0 1 50 0.69 0.69\nDavis, J.D. 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.29 0.476 0.532 0.456 90.3 1\nAndujar, Miguel 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.621 0.452 0.52 0.426 75.5 36.5'}, {'pitcher_id': '657585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Reed 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.6 0.667 0.541 107.6 4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.6 0.667 0.541 107.6 4'}, {'pitcher_id': '493603', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOttavino, Adam 17 13 1 1 0 0 5 29.4 33.3 2 11.8 0.077 0.154 0.238 0.16 0.29 0.31 89.1 38.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 5 5 1 1 0 0 2 40 44.4 0 0 0.2 0.4 0.243 0.324 0.646 0.401 93.9 12\nRooker, Brent 2 0 0 0 0 0 0 0 0 2 100 0.694 0.694\nBrown, Seth 2 1 0 0 0 0 0 0 20 0 0 0 0 0.36 0.419 0.473 0.555 89.5 2\nBleday, JJ 2 1 0 0 0 0 1 50 40 0 0 0 0 0.36 0.36\nLangeliers, Shea 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.003 0.003 0.003 88 69\nToro, Abraham 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.027 0.019 93.1 58\nDíaz, Aledmys 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.002 0.002 0.002 67.1 82'}, {'pitcher_id': '656887', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReid-Foley, Sean 5 5 2 1 0 1 0 0 25 0 0 0.4 1.2 0.656 0.346 0.901 0.519 94.5 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAndujar, Miguel 4 4 2 1 0 1 0 0 28.6 0 0 0.5 1.5 0.82 0.428 1.12 0.643 98.1 2.7\nDavis, J.D. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.027 0.02 80.1 38'}, {'pitcher_id': '676130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nButtó, José 7 5 2 1 0 0 0 0 33.3 2 28.6 0.4 0.6 0.503 0.312 0.494 0.449 84.8 20.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.486 1.272 0.719 99.4 24\nLangeliers, Shea 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.125 0.138 0.118 77.4 -8.5\nRooker, Brent 3 2 1 1 0 0 0 0 33.3 1 33.3 0.5 1 0.647 0.413 0.461 0.49 84.9 48.5'}, {'pitcher_id': '642121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSulser, Cole 6 5 2 0 1 0 0 0 42.9 1 16.7 0.4 0.8 0.521 0.29 0.432 0.372 98.7 15.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nToro, Abraham 3 2 0 0 0 0 0 0 42.9 1 33.3 0 0 0.231 0.379 0.504 0.48 98.9 12.5\nAndujar, Miguel 2 2 2 0 1 0 0 0 50 0 0 1 2 1.219 0.329 0.527 0.369 100.4 5.5\nBrown, Seth 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.034 0.098 0.052 95 40'}, {'pitcher_id': '548384', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRaley, Brooks 6 6 2 0 0 0 1 16.7 0 0 0 0.333 0.333 0.295 0.327 0.374 0.31 90 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nToro, Abraham 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.141 0.193 0.139 91.4 9.5\nAndujar, Miguel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.783 0.871 0.737 92.5 12\nBrown, Seth 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nDíaz, Aledmys 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.116 0.117 0.11 82.3 -48'}, {'pitcher_id': '673380', 'player_data': '', 'team_data': 'Dedniel Núñez has not faced anyone on this team.'}, {'pitcher_id': '605288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHouser, Adrian 13 8 2 1 0 0 2 15.4 14.3 4 30.8 0.25 0.375 0.432 0.098 0.204 0.346 81.7 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRooker, Brent 4 2 0 0 0 0 1 25 18.2 1 25 0 0 0.353 0.05 0.057 0.376 64.1 -2\nBrown, Seth 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.05 0.094 0.269 92 26\nToro, Abraham 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.176 0.53 0.29 75 16.5\nBleday, JJ 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.003 0.003 0.349 84 41\nLangeliers, Shea 2 1 1 1 0 0 0 0 0 1 50 1 2 0.97 0.227 0.271 0.452 99.8 -9'}, {'pitcher_id': '640455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManaea, Sean 35 33 7 1 0 1 10 28.6 29.9 2 5.7 0.212 0.333 0.261 0.202 0.353 0.261 91.5 11.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 7 6 2 0 0 1 2 28.6 33.3 1 14.3 0.333 0.833 0.521 0.272 0.782 0.476 93.1 5.3\nToro, Abraham 7 6 1 1 0 0 2 28.6 30.8 1 14.3 0.167 0.333 0.279 0.07 0.136 0.171 86.1 14.8\nRooker, Brent 6 6 2 0 0 0 1 16.7 36.4 0 0 0.333 0.333 0.294 0.323 0.496 0.349 94 9.4\nBleday, JJ 3 3 0 0 0 0 3 100 50 0 0 0 0 0 0\nDíaz, Aledmys 7 7 2 0 0 0 0 0 11.1 0 0 0.286 0.286 0.252 0.33 0.395 0.311 94.7 15.7\nAndujar, Miguel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.03 0.033 0.037 77.7 21\nGelof, Zack 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nLangeliers, Shea 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nBrown, Seth 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.297 0.321 0.272 98.7 -6'}, {'pitcher_id': '664192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucchesi, Joey 3 3 0 0 0 0 2 66.7 0 0 0 0 0 0 0.038 0.038 0.034 78.1 -5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 3 3 0 0 0 0 2 66.7 0 0 0 0 0 0 0.038 0.038 0.034 78.1 -5'}, {'pitcher_id': '605347', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Jorge 25 22 7 2 1 0 1 4 20 3 12 0.318 0.5 0.386 0.338 0.528 0.409 84.3 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Seth 7 7 3 1 0 0 0 0 22.2 0 0 0.429 0.571 0.422 0.485 0.679 0.496 84.5 4\nDavis, J.D. 4 3 0 0 0 0 0 0 20 1 25 0 0 0.175 0.221 0.234 0.324 75.9 -4\nAndujar, Miguel 4 3 1 0 0 0 0 0 28.6 1 25 0.333 0.333 0.393 0.141 0.149 0.272 73.3 -36.7\nToro, Abraham 4 4 1 0 1 0 1 25 12.5 0 0 0.25 0.75 0.4 0.218 0.785 0.417 93.1 30.3\nDíaz, Aledmys 4 4 1 0 0 0 0 0 25 0 0 0.25 0.25 0.22 0.484 0.594 0.464 95 6\nGelof, Zack 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.131 0.192 0.148 72.2 51\nLangeliers, Shea 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '622663', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeverino, Luis 14 13 3 0 0 0 2 14.3 24 1 7.1 0.231 0.231 0.238 0.24 0.388 0.301 86.9 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 14 13 3 0 0 0 2 14.3 24 1 7.1 0.231 0.231 0.238 0.24 0.388 0.301 86.9 14.7'}, {'pitcher_id': '677020', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '664849', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622098', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Drew 4 3 0 0 0 0 1 25 28.6 1 25 0 0 0.172 0.21 0.246 0.325 67.1 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nToro, Abraham 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nDíaz, Aledmys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.457 0.539 0.437 75.9 28\nDavis, J.D. 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nAndujar, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.199 0.175 58.3 8'}, {'pitcher_id': '701643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHartwig, Grant 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.042 0.042 0.038 92.1 -26', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.042 0.042 0.038 92.1 -26'}, {'pitcher_id': '681035', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 134: [{'pitcher_id': '621242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Edwin 23 17 1 0 0 0 8 34.8 55.6 3 13 0.059 0.059 0.222 0.152 0.217 0.301 88.3 14.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 8 5 0 0 0 0 2 25 60 1 12.5 0 0 0.268 0.062 0.097 0.308 91 23.7\nReynolds, Bryan 5 4 1 0 0 0 1 20 42.9 1 20 0.25 0.25 0.312 0.481 0.675 0.535 88.8 18.7\nHayes, Ke'Bryan 4 3 0 0 0 0 3 75 83.3 0 0 0 0 0.18 0.18\nTellez, Rowdy 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nTaylor, Michael A. 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.089 0.101 0.084 82.5 -30\nJoe, Connor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.198 0.23 0.189 81 0\nCruz, Oneil 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.066 0.177 0.098 91.9 35\nTriolo, Jared 1 0 0 0 0 0 0 0 1 100 0.692 0.692"}, {'pitcher_id': '656731', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Tylor 39 33 14 2 0 2 7 17.9 24.7 6 15.4 0.424 0.667 0.501 0.311 0.471 0.39 90.5 14.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 11 11 3 0 0 1 1 9.1 15.2 0 0 0.273 0.545 0.342 0.285 0.446 0.313 79.9 28.8\nMcCutchen, Andrew 8 5 1 0 0 0 3 37.5 40 3 37.5 0.2 0.2 0.37 0.085 0.086 0.309 90.9 -26\nHayes, Ke'Bryan 10 10 7 1 0 0 1 10 15.8 0 0 0.7 0.8 0.653 0.411 0.606 0.433 98.5 4.4\nSuwinski, Jack 5 3 1 1 0 0 1 20 50 2 40 0.333 0.667 0.527 0.312 0.448 0.475 97.4 27\nTellez, Rowdy 3 3 2 0 0 1 1 33.3 28.6 0 0 0.667 1.667 0.985 0.457 0.828 0.547 99.1 18.5\nJoe, Connor 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.282 0.303 0.477 91.9 -1"}, {'pitcher_id': '518617', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDiekman, Jake 25 21 3 1 0 0 12 48 51.1 4 16 0.143 0.19 0.231 0.097 0.112 0.188 78.8 16.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 6 3 0 0 0 0 3 50 60 3 50 0 0 0.346 0.346\nTaylor, Michael A. 7 7 0 0 0 0 6 85.7 77.8 0 0 0 0 0 0.01 0.011 0.01 59.6 -16\nJoe, Connor 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.437 0.515 0.416 79.4 40\nTellez, Rowdy 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.051 0.051 0.046 102.1 -22\nCruz, Oneil 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nReynolds, Bryan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.02 0.06 0.034 91.1 35\nSuwinski, Jack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.266 0.296 0.243 72.9 9\nOlivares, Edward 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.503 0.512 0.453 59.8 24\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.026 0.039 0.029 81.9 39\nDelay, Jason 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.152 0.171 0.14 83.2 -2"}, {'pitcher_id': '500779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuintana, Jose 114 105 22 2 0 3 29 25.4 25.1 9 7.9 0.21 0.314 0.259 0.285 0.467 0.349 90 9.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 29 26 8 1 0 1 8 27.6 21.9 3 10.3 0.308 0.462 0.368 0.323 0.552 0.403 90.4 10.8\nGrandal, Yasmani 19 17 5 0 0 2 5 26.3 30 2 10.5 0.294 0.647 0.415 0.36 0.768 0.483 93.3 21.8\nReynolds, Bryan 19 18 4 1 0 0 3 15.8 22.2 1 5.3 0.222 0.278 0.239 0.317 0.571 0.389 89.8 3\nJoe, Connor 9 7 1 0 0 0 0 0 13.3 2 22.2 0.143 0.143 0.252 0.171 0.208 0.284 83.7 13\nTellez, Rowdy 8 8 0 0 0 0 5 62.5 50 0 0 0 0 0 0.111 0.116 0.105 87.3 4.3\nCruz, Oneil 6 6 0 0 0 0 2 33.3 33.3 0 0 0 0 0 0.16 0.196 0.162 80.7 2.5\nTaylor, Michael A. 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.219 0.35 0.428 0.341 98.6 13.3\nHayes, Ke'Bryan 5 5 1 0 0 0 0 0 12.5 0 0 0.2 0.2 0.177 0.353 0.379 0.323 91.8 -19\nOlivares, Edward 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.177 0.357 0.451 0.354 86.7 32\nTriolo, Jared 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.133 0.157 0.127 95.7 2\nSuwinski, Jack 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0.178 0.19 0.338 103.2 -7\nDelay, Jason 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.373 0.41 0.347 91.7 -1.3\nKeller, Mitch 1 1 0 0 0 0 1 100 0 0 0 0 0 0"}, {'pitcher_id': '657585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Reed 6 6 1 0 0 0 5 83.3 69.2 0 0 0.167 0.167 0.148 0.066 0.08 0.066 104.4 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCruz, Oneil 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTaylor, Michael A. 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMcCutchen, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nOlivares, Edward 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nJoe, Connor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.397 0.48 0.397 104.4 -1'}, {'pitcher_id': '493603', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOttavino, Adam 45 38 8 1 0 0 15 33.3 36.1 6 13.3 0.211 0.237 0.257 0.167 0.298 0.238 84.2 12.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 15 14 5 0 0 0 2 13.3 21.7 1 6.7 0.357 0.357 0.341 0.249 0.407 0.282 84.8 3\nMcCutchen, Andrew 12 9 1 0 0 0 6 50 45 2 16.7 0.111 0.111 0.189 0.01 0.011 0.01 81 28.3\nTellez, Rowdy 5 4 0 0 0 0 3 60 57.1 1 20 0 0 0.139 0.026 0.026 0.164 74.3 1\nHayes, Ke'Bryan 4 3 0 0 0 0 1 25 42.9 1 25 0 0 0.172 0.257 0.994 0.549 93.5 35.5\nJoe, Connor 2 1 0 0 0 0 1 50 0 1 50 0 0 0.344 0.344\nTaylor, Michael A. 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.511 0.609 0.497 85.7 -11.7\nReynolds, Bryan 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.186 0.208 0.168 84.1 5\nDelay, Jason 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSuwinski, Jack 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 78.3 73"}, {'pitcher_id': '656887', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReid-Foley, Sean 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.293 0.115 0.137 0.105 94.9 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.293 0.115 0.137 0.105 94.9 -2'}, {'pitcher_id': '676130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nButtó, José 8 6 2 1 0 0 2 25 22.7 2 25 0.333 0.5 0.44 0.21 0.299 0.334 101.2 35.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 2 2 1 1 0 0 1 50 20 0 0 0.5 1 0.622 0.332 0.556 0.369 108 15\nMcCutchen, Andrew 2 1 0 0 0 0 1 50 25 1 50 0 0 0.348 0.348\nHayes, Ke'Bryan 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nDelay, Jason 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.003 0.003 88.2 74\nReynolds, Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.042 0.021 102.7 49\nTriolo, Jared 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.569 0.636 0.519 105.7 3"}, {'pitcher_id': '642121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSulser, Cole 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.107 0.119 0.097 96.2 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.161 0.178 0.145 96.2 0\nTaylor, Michael A. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '548384', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRaley, Brooks 21 20 2 0 0 0 10 47.6 41.5 1 4.8 0.1 0.1 0.117 0.137 0.21 0.174 87.3 9.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.219 0.579 0.322 83 7.3\nMcCutchen, Andrew 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.186 0.201 0.169 88.8 -18\nSuwinski, Jack 3 2 0 0 0 0 1 33.3 75 1 33.3 0 0 0.23 0.002 0.002 0.231 82.1 75\nTellez, Rowdy 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nHayes, Ke'Bryan 3 3 1 0 0 0 2 66.7 57.1 0 0 0.333 0.333 0.294 0.094 0.102 0.086 100.2 -6\nCruz, Oneil 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nGrandal, Yasmani 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.221 0.245 0.203 83.6 0\nTaylor, Michael A. 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.442 0.477 0.567 0.465 90.4 14\nDelay, Jason 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.171 0.102 89.9 29"}, {'pitcher_id': '673380', 'player_data': '', 'team_data': 'Dedniel Núñez has not faced anyone on this team.'}, {'pitcher_id': '605288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHouser, Adrian 80 71 16 1 0 3 12 15 14.2 9 11.3 0.225 0.366 0.302 0.268 0.422 0.343 91.1 -1.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 27 24 7 1 0 1 1 3.7 12.8 3 11.1 0.292 0.458 0.36 0.264 0.39 0.329 85.7 6.6\nHayes, Ke'Bryan 20 19 4 0 0 0 3 15 5.7 1 5 0.211 0.211 0.211 0.266 0.333 0.285 96.4 -6.4\nSuwinski, Jack 6 4 1 0 0 1 1 16.7 0 2 33.3 0.25 1 0.577 0.417 1.027 0.641 105.5 3\nMcCutchen, Andrew 7 6 2 0 0 0 0 0 23.1 1 14.3 0.333 0.333 0.352 0.383 0.562 0.445 90 -3.8\nCruz, Oneil 5 4 1 0 0 1 3 60 60 1 20 0.25 1 0.553 0.246 0.961 0.539 110.6 25\nTriolo, Jared 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.157 0.161 0.417 99.1 -21\nJoe, Connor 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.095 0.103 0.085 110.3 -13\nTellez, Rowdy 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.086 0.096 0.08 93.8 22\nGrandal, Yasmani 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.413 0.467 0.391 77.3 -26.3\nOlivares, Edward 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.3 0.319 0.281 83.4 -38\nTaylor, Michael A. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDelay, Jason 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.172 0.172 0.163 96.4 -10"}, {'pitcher_id': '640455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManaea, Sean 56 50 10 0 0 2 12 21.4 22.9 6 10.7 0.2 0.32 0.274 0.269 0.383 0.326 88.8 3.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 10 9 1 0 0 0 3 30 31.8 1 10 0.111 0.111 0.157 0.232 0.272 0.272 89.9 -4.2\nReynolds, Bryan 7 5 2 0 0 1 0 0 15.8 2 28.6 0.4 1 0.619 0.331 0.669 0.497 99.9 0.2\nSuwinski, Jack 9 9 1 0 0 0 2 22.2 14.3 0 0 0.111 0.111 0.098 0.159 0.181 0.148 86 15.1\nHayes, Ke'Bryan 5 4 1 0 0 0 1 20 10 1 20 0.25 0.25 0.315 0.461 0.57 0.492 100.5 7.7\nTaylor, Michael A. 5 5 2 0 0 1 2 40 27.3 0 0 0.4 1 0.591 0.302 0.715 0.429 102.3 3.3\nJoe, Connor 6 6 0 0 0 0 1 16.7 14.3 0 0 0 0 0 0.258 0.279 0.238 75.8 -13.2\nGrandal, Yasmani 4 3 0 0 0 0 1 25 37.5 1 25 0 0 0.173 0.023 0.023 0.193 69.8 -28.5\nOlivares, Edward 5 4 0 0 0 0 1 20 28.6 1 20 0 0 0.139 0.211 0.305 0.32 97.3 -3.7\nTellez, Rowdy 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.589 0.544 0.652 0.526 75.4 40.3\nDelay, Jason 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.41 0.466 0.383 87.4 18"}, {'pitcher_id': '664192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucchesi, Joey 29 25 4 0 1 1 9 31 32.7 2 6.9 0.16 0.36 0.254 0.228 0.442 0.334 91.3 2.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 7 6 1 0 0 0 2 28.6 25 1 14.3 0.167 0.167 0.224 0.198 0.367 0.296 87.2 -0.5\nReynolds, Bryan 6 4 2 0 0 1 0 0 22.2 0 0 0.5 1.25 0.588 0.586 1.203 0.707 99 9.2\nGrandal, Yasmani 6 5 0 0 0 0 1 16.7 33.3 1 16.7 0 0 0.115 0.169 0.582 0.395 85.2 -2.5\nMcCutchen, Andrew 4 4 1 0 1 0 2 50 50 0 0 0.25 0.75 0.392 0.133 0.16 0.129 84.9 4.5\nTellez, Rowdy 3 3 0 0 0 0 3 100 36.4 0 0 0 0 0 0\nJoe, Connor 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.225 0.252 0.205 92.6 -5'}, {'pitcher_id': '605347', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Jorge 21 19 3 1 0 0 6 28.6 17.4 1 4.8 0.158 0.211 0.211 0.222 0.367 0.294 86.3 2.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 9 8 2 0 0 0 4 44.4 28 1 11.1 0.25 0.25 0.272 0.209 0.254 0.257 83.1 -7.8\nGrandal, Yasmani 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.345 0.813 0.475 86 5.3\nTellez, Rowdy 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.42 0.309 0.475 0.335 98.2 28.7\nMcCutchen, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.135 0.153 0.125 74.7 -3\nTriolo, Jared 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.051 0.054 0.047 75.5 -40\nSuwinski, Jack 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nOlivares, Edward 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '622663', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeverino, Luis 53 47 11 1 0 1 11 20.8 18.3 5 9.4 0.234 0.319 0.279 0.212 0.295 0.262 92.6 5.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuwinski, Jack 8 8 0 0 0 0 2 25 15.4 0 0 0 0 0 0.107 0.125 0.1 82.2 20.5\nCruz, Oneil 7 6 3 0 0 0 0 0 6.3 1 14.3 0.5 0.5 0.479 0.432 0.473 0.442 108.2 -5.5\nTellez, Rowdy 7 7 1 0 0 0 3 42.9 28.6 0 0 0.143 0.143 0.127 0.105 0.179 0.12 93.3 23.5\nReynolds, Bryan 8 7 3 1 0 0 1 12.5 11.1 1 12.5 0.429 0.571 0.466 0.288 0.346 0.33 90 8.5\nHayes, Ke'Bryan 5 4 1 0 0 0 2 40 10 0 0 0.25 0.25 0.177 0.111 0.111 0.104 82.3 -19.3\nGrandal, Yasmani 6 5 1 0 0 0 0 0 18.2 1 16.7 0.2 0.2 0.263 0.217 0.236 0.281 96 14.8\nJoe, Connor 3 2 0 0 0 0 0 0 25 1 33.3 0 0 0.231 0.111 0.126 0.298 79.5 -12.5\nOlivares, Edward 3 2 0 0 0 0 1 33.3 16.7 1 33.3 0 0 0.231 0.108 0.121 0.296 105 -16\nDelay, Jason 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.113 0.151 0.103 97.2 -6\nTriolo, Jared 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.032 0.033 0.029 78.9 -37\nTaylor, Michael A. 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.478 0.741 1.924 1.123 101.3 19"}, {'pitcher_id': '677020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Josh 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.374 0.456 0.361 88.6 18', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHayes, Ke'Bryan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.749 0.911 0.722 88.6 18\nSuwinski, Jack 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0"}, {'pitcher_id': '664849', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622098', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Drew 22 19 9 3 0 0 5 22.7 24.5 2 9.1 0.474 0.632 0.473 0.309 0.456 0.365 94.1 19.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 5 5 3 0 0 0 1 20 20 0 0 0.6 0.6 0.53 0.341 0.406 0.324 93.3 21.8\nMcCutchen, Andrew 4 4 1 0 0 0 2 50 25 0 0 0.25 0.25 0.221 0.34 0.466 0.346 86.4 22.5\nSuwinski, Jack 2 1 0 0 0 0 1 50 20 1 50 0 0 0.346 0.346\nTellez, Rowdy 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.295 0.298 0.309 0.268 103.3 0.5\nReynolds, Bryan 2 2 1 1 0 0 0 0 20 0 0 0.5 1 0.622 0.383 1.121 0.611 97.9 26\nGrandal, Yasmani 2 1 1 1 0 0 0 0 60 1 50 1 2 0.967 0.329 0.663 0.544 97.5 22\nCruz, Oneil 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.001 0.001 0 74.5 70\nTriolo, Jared 1 0 0 0 0 0 0 0 0 0 0 0 0.409 94.8 19\nTaylor, Michael A. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.88 0.369 0.402 0.36 98.9 -4\nHayes, Ke'Bryan 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.454 0.534 0.442 97.5 0"}, {'pitcher_id': '701643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHartwig, Grant 9 7 2 1 1 0 1 11.1 23.5 1 11.1 0.286 0.714 0.471 0.299 0.356 0.377 87.9 27.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuwinski, Jack 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.009 0.016 0.353 91.8 55\nDelay, Jason 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.626 0.885 0.638 99.4 16\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.003 0.005 0.003 95.4 50\nGrandal, Yasmani 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nOlivares, Edward 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.646 0.694 0.584 61.7 30\nTellez, Rowdy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.232 0.254 0.209 98.6 -7\nReynolds, Bryan 1 1 1 0 1 0 0 0 50 0 0 1 3 1.569 0.574 0.637 0.528 80.7 23\nTriolo, Jared 1 0 0 0 0 0 0 0 0 0 0.726 0.726"}, {'pitcher_id': '681035', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 135: [{'pitcher_id': '621242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Edwin 34 31 8 1 0 0 13 38.2 25.9 2 5.9 0.258 0.29 0.257 0.222 0.287 0.248 91.8 11.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 9 8 4 0 0 0 2 22.2 10 1 11.1 0.5 0.5 0.469 0.418 0.516 0.437 92.3 10.7\nMachado, Manny 8 7 2 0 0 0 3 37.5 28.6 0 0 0.286 0.286 0.219 0.194 0.225 0.189 90.9 1\nTatis Jr., Fernando 4 4 1 1 0 0 2 50 46.2 0 0 0.25 0.5 0.304 0.228 0.435 0.272 98.8 -1.5\nCronenworth, Jake 3 3 0 0 0 0 3 100 80 0 0 0 0 0 0\nArraez, Luis 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69\nPeralta, David 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.22 0.251 0.29 0.242 96 21.8\nWade, Tyler 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSolano, Donovan 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.129 0.144 0.116 77.5 35\nHigashioka, Kyle 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nKim, Ha-Seong 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '656731', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Tylor 23 22 5 1 0 0 5 21.7 15.9 1 4.3 0.227 0.273 0.238 0.274 0.358 0.293 88.4 12.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 6 6 0 0 0 0 2 33.3 26.7 0 0 0 0 0 0.108 0.137 0.107 88.5 38\nArraez, Luis 6 6 4 1 0 0 0 0 14.3 0 0 0.667 0.833 0.649 0.506 0.706 0.531 90.2 7.5\nProfar, Jurickson 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.294 0.16 0.186 0.145 93.9 -8\nMachado, Manny 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.232 0.305 0.453 0.449 100.1 16\nCronenworth, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.417 0.445 0.376 82.1 5.5\nSolano, Donovan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.026 0.026 0.025 66.1 -4\nWade, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.363 0.408 0.332 89 3'}, {'pitcher_id': '518617', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDiekman, Jake 30 25 6 0 0 2 7 23.3 26.1 4 13.3 0.24 0.48 0.37 0.222 0.549 0.388 92 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 11 9 3 0 0 1 3 27.3 27.6 2 18.2 0.333 0.667 0.469 0.216 0.73 0.444 104.4 24\nPeralta, David 4 4 1 0 0 0 2 50 33.3 0 0 0.25 0.25 0.221 0.063 0.069 0.067 73.6 28.5\nArraez, Luis 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.22 0.214 0.248 0.2 89.6 1.5\nCronenworth, Jake 4 3 0 0 0 0 0 0 0 0 0 0 0 0.181 0.288 0.432 0.412 97 21.3\nKim, Ha-Seong 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nHigashioka, Kyle 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nMachado, Manny 2 2 1 0 0 1 1 50 66.7 0 0 0.5 2 1.036 0.456 1.726 0.909 105 26\nProfar, Jurickson 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.527 1.53 0.854 98.2 25\nMerrill, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.022 0.012 91.6 40'}, {'pitcher_id': '500779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nQuintana, Jose 82 75 16 3 0 5 17 20.7 18.8 6 7.3 0.213 0.453 0.304 0.243 0.427 0.308 85.1 12.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 28 25 5 1 0 1 6 21.4 16.1 3 10.7 0.2 0.36 0.282 0.173 0.306 0.257 88.1 31.7\nProfar, Jurickson 11 10 2 0 0 1 2 18.2 17.4 1 9.1 0.2 0.5 0.326 0.219 0.415 0.305 89.1 7.2\nTatis Jr., Fernando 9 7 3 1 0 1 0 0 9.1 1 11.1 0.429 1 0.531 0.323 0.791 0.432 96 15.6\nKim, Ha-Seong 6 5 1 0 0 1 1 16.7 11.1 1 16.7 0.2 0.8 0.45 0.257 0.661 0.428 81.4 29\nHigashioka, Kyle 5 5 0 0 0 0 4 80 44.4 0 0 0 0 0 0.088 0.109 0.087 71 39\nSolano, Donovan 7 7 2 1 0 0 1 14.3 22.2 0 0 0.286 0.429 0.306 0.329 0.392 0.316 84 -11.5\nCronenworth, Jake 5 5 1 0 0 0 2 40 37.5 0 0 0.2 0.2 0.177 0.193 0.212 0.184 70 28.3\nPeralta, David 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.219 0.465 0.509 0.43 74.3 -3.7\nArraez, Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.238 0.207 88 -15.3\nAzocar, José 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.164 0.167 0.149 59.6 -42.5\nMerrill, Jackson 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.434 1.367 0.74 88 41'}, {'pitcher_id': '657585', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Reed 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.339 0.447 0.331 102.7 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.216 0.39 0.243 100.8 17\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.486 0.412 107.2 -4\nMachado, Manny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.469 0.522 0.428 100 2'}, {'pitcher_id': '493603', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOttavino, Adam 45 37 3 1 0 1 14 31.1 31.3 7 15.6 0.081 0.189 0.199 0.202 0.426 0.335 92.7 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 12 11 0 0 0 0 3 25 25 1 8.3 0 0 0.057 0.102 0.111 0.149 89.2 9.6\nPeralta, David 11 8 1 0 0 1 1 9.1 26.1 2 18.2 0.125 0.5 0.308 0.268 0.855 0.46 92.5 18\nSolano, Donovan 7 6 0 0 0 0 4 57.1 47.1 1 14.3 0 0 0.099 0.348\nProfar, Jurickson 4 3 0 0 0 0 2 50 14.3 1 25 0 0 0.172 0.189 0.687 0.444 101.2 30\nArraez, Luis 3 2 2 1 0 0 0 0 0 1 33.3 1 1.5 0.929 0.682 0.926 0.687 93.2 17\nCronenworth, Jake 3 2 0 0 0 0 2 66.7 75 1 33.3 0 0 0.232 0.232\nTatis Jr., Fernando 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.567 0.805 0.588 108.9 5\nKim, Ha-Seong 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nCampusano, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.158 0.111 77.8 33'}, {'pitcher_id': '656887', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReid-Foley, Sean 5 5 0 0 0 0 1 20 30 0 0 0 0 0 0.201 0.234 0.187 86.5 28.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHigashioka, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 92.7 72\nTatis Jr., Fernando 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.078 0.086 0.072 79.1 -12\nPeralta, David 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.463 0.541 0.431 87.1 27'}, {'pitcher_id': '676130', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nButtó, José 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.503 0.685 0.507 97.5 23.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.503 0.685 0.507 97.5 23.7'}, {'pitcher_id': '642121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSulser, Cole 8 6 2 0 0 0 1 12.5 41.7 2 25 0.333 0.333 0.394 0.29 0.39 0.394 87.3 43.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHigashioka, Kyle 2 0 0 0 0 0 0 0 75 2 100 0.692 0.692\nCampusano, Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nProfar, Jurickson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.874 1.185 0.898 106.1 12\nWade, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.053 0.073 0.054 75.1 57\nKim, Ha-Seong 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.025 0.021 74.1 62\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 81.6 71\nPeralta, David 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.791 1.049 0.787 99.7 14'}, {'pitcher_id': '548384', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRaley, Brooks 17 11 5 1 0 0 3 17.6 14.3 3 17.6 0.455 0.545 0.531 0.307 0.339 0.434 78.2 3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 4 3 2 0 0 0 1 25 20 1 25 0.667 0.667 0.616 0.385 0.427 0.442 79.6 11\nCronenworth, Jake 4 2 1 0 0 0 1 25 0 0 0 0.5 0.5 0.583 0.364 0.413 0.531 87 12\nHigashioka, Kyle 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nKim, Ha-Seong 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.708 0.397 0.431 0.362 81.1 14\nProfar, Jurickson 1 0 0 0 0 0 0 0 0 1 100 0.699 0.699\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.193 0.179 71.2 -47\nPeralta, David 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.114 0.137 0.103 65.1 0\nSolano, Donovan 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72'}, {'pitcher_id': '673380', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNúñez, Dedniel 7 7 0 0 0 0 2 28.6 27.8 0 0 0 0 0 0.164 0.429 0.245 95.6 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.014 0.02 0.016 84 48\nMachado, Manny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.172 0.184 0.155 101.6 -13\nKim, Ha-Seong 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.254 0.264 0.232 103.3 -14\nCronenworth, Jake 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.683 2.477 1.279 101.6 30\nPeralta, David 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMerrill, Jackson 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.06 0.035 87.6 34'}, {'pitcher_id': '605288', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHouser, Adrian 65 57 10 2 0 1 12 18.5 16.8 7 10.8 0.175 0.263 0.242 0.224 0.348 0.293 90.8 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 15 14 3 1 0 1 4 26.7 20.6 1 6.7 0.214 0.5 0.321 0.232 0.363 0.284 89 26.4\nArraez, Luis 10 9 4 1 0 0 2 20 15 1 10 0.444 0.556 0.458 0.315 0.646 0.429 94.2 21.3\nCronenworth, Jake 9 7 0 0 0 0 3 33.3 23.1 1 11.1 0 0 0.086 0.085 0.099 0.154 81.8 -0.3\nMachado, Manny 13 12 1 0 0 0 0 0 9.5 1 7.7 0.083 0.083 0.121 0.25 0.334 0.288 93.5 4.8\nTatis Jr., Fernando 8 6 1 0 0 0 2 25 11.8 2 25 0.167 0.167 0.282 0.205 0.299 0.334 92.8 -3.8\nKim, Ha-Seong 6 5 0 0 0 0 1 16.7 14.3 1 16.7 0 0 0.116 0.114 0.119 0.207 83.4 -3.2\nPeralta, David 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.295 0.322 0.275 90.4 -12.5\nSolano, Donovan 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.343 0.602 0.382 102 0.5'}, {'pitcher_id': '640455', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManaea, Sean 116 110 31 7 1 3 26 22.4 23.2 6 5.2 0.282 0.445 0.33 0.21 0.304 0.247 88.9 4.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 24 23 8 2 1 1 3 12.5 14 1 4.2 0.348 0.652 0.429 0.265 0.416 0.31 88.6 1.9\nMachado, Manny 20 19 7 3 0 0 4 20 22.2 1 5 0.368 0.526 0.397 0.274 0.402 0.316 87.7 4.7\nCronenworth, Jake 11 10 2 1 0 0 2 18.2 21.2 1 9.1 0.2 0.3 0.256 0.119 0.186 0.183 88.3 10\nTatis Jr., Fernando 15 14 2 0 0 0 6 40 35.5 1 6.7 0.143 0.143 0.164 0.083 0.099 0.122 90.5 -20.6\nSolano, Donovan 11 10 3 0 0 0 5 45.5 35.7 1 9.1 0.3 0.3 0.304 0.233 0.259 0.258 90.1 3.8\nKim, Ha-Seong 11 10 1 0 0 0 2 18.2 15 1 9.1 0.1 0.1 0.143 0.126 0.15 0.172 86.8 27.3\nArraez, Luis 9 9 4 0 0 1 1 11.1 20 0 0 0.444 0.778 0.525 0.273 0.345 0.271 86.5 4.9\nPeralta, David 7 7 1 0 0 0 1 14.3 30.8 0 0 0.143 0.143 0.126 0.111 0.125 0.105 90.8 -3.2\nHigashioka, Kyle 4 4 1 0 0 0 2 50 37.5 0 0 0.25 0.25 0.22 0.207 0.294 0.218 99.2 0.5\nAzocar, José 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.212 0.271 0.206 88.2 15.5\nMerrill, Jackson 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.043 0.666 1.603 0.94 98.1 20'}, {'pitcher_id': '664192', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLucchesi, Joey 44 42 8 4 0 2 7 15.9 14.5 1 2.3 0.19 0.429 0.275 0.218 0.458 0.303 89.3 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 14 12 1 1 0 0 2 14.3 17.6 1 7.1 0.083 0.167 0.188 0.187 0.217 0.256 80.9 11.8\nSolano, Donovan 9 9 2 1 0 0 2 22.2 10.5 0 0 0.222 0.333 0.232 0.202 0.449 0.267 89.8 43.4\nMachado, Manny 6 6 3 0 0 2 0 0 7.1 0 0 0.5 1.5 0.819 0.435 1.333 0.721 98.4 33.7\nTatis Jr., Fernando 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.081 0.089 0.078 89.1 15\nProfar, Jurickson 7 7 1 1 0 0 1 14.3 8.3 0 0 0.143 0.286 0.178 0.206 0.349 0.239 91.7 21.3\nKim, Ha-Seong 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.033 0.072 0.043 90.9 30\nCronenworth, Jake 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.621 0.32 0.828 0.467 94.1 6'}, {'pitcher_id': '605347', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Jorge 21 18 7 0 0 0 3 14.3 10.8 2 9.5 0.389 0.389 0.359 0.261 0.295 0.294 91.9 -1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWade, Tyler 6 5 2 0 0 0 2 33.3 22.2 1 16.7 0.4 0.4 0.409 0.157 0.168 0.239 87.5 -2.3\nArraez, Luis 5 5 3 0 0 0 0 0 0 0 0 0.6 0.6 0.525 0.253 0.286 0.238 93.2 -10.4\nProfar, Jurickson 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.147 0.17 0.135 88.4 -11.5\nHigashioka, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.371 0.411 0.336 105.1 -4\nCronenworth, Jake 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nPeralta, David 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.454 0.524 0.431 101.1 -6\nSolano, Donovan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.189 0.17 54.1 42\nMachado, Manny 1 0 0 0 0 0 0 0 0 0 0 0 0.398 98.2 22\nTatis Jr., Fernando 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.74 0.879 0.692 105.3 9'}, {'pitcher_id': '622663', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeverino, Luis 28 26 6 1 0 4 12 42.9 35.6 2 7.1 0.231 0.731 0.413 0.169 0.458 0.291 86.9 24.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 18 17 4 1 0 3 10 55.6 44.8 1 5.6 0.235 0.824 0.444 0.161 0.444 0.276 95.5 27.4\nTatis Jr., Fernando 3 3 1 0 0 1 1 33.3 33.3 0 0 0.333 1.333 0.668 0.355 1.224 0.639 72.5 34\nKim, Ha-Seong 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nCronenworth, Jake 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.03 0.036 0.03 82.9 29\nArraez, Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.259 0.286 0.24 77 -3.5'}, {'pitcher_id': '677020', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Josh 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHigashioka, Kyle 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0'}, {'pitcher_id': '664849', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYoung, Danny 6 6 0 0 0 0 3 50 26.7 0 0 0 0 0 0.131 0.142 0.119 82.7 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKim, Ha-Seong 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.001 0.001 0 91.7 50\nCronenworth, Jake 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.033 0.044 0.031 80.2 -23\nCampusano, Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAzocar, José 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.72 0.765 0.649 76.2 15'}, {'pitcher_id': '622098', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Drew 19 17 3 1 1 0 7 36.8 20.6 1 5.3 0.176 0.353 0.269 0.179 0.255 0.24 93.1 21.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKim, Ha-Seong 3 2 0 0 0 0 2 66.7 40 1 33.3 0 0 0.232 0.232\nMachado, Manny 5 5 1 1 0 0 2 40 30 0 0 0.2 0.4 0.249 0.115 0.262 0.154 93.9 28.7\nProfar, Jurickson 3 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.764 0.403 0.473 0.493 90.8 2\nCronenworth, Jake 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.165 0.19 0.154 92.3 36\nWade, Tyler 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nPeralta, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.288 0.329 0.263 89 2\nTatis Jr., Fernando 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.44 0.435 0.578 0.441 107.9 10\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.021 0.014 86.1 41'}, {'pitcher_id': '701643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHartwig, Grant 5 5 1 0 0 0 0 0 10 0 0 0.2 0.2 0.177 0.315 0.413 0.313 91.1 -10.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 1 1 1 0 0 0 0 0 20 0 0 1 1 0.883 0.934 1.098 0.884 98.6 12\nTatis Jr., Fernando 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.101 0.105 0.097 89.9 -28\nMachado, Manny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.4 0.72 0.449 101.5 18\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.039 0.037 75.8 -28'}, {'pitcher_id': '681035', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}]}, 133: {120: [{'pitcher_id': '683155', 'player_data': '', 'team_data': 'Joey Estes has not faced anyone on this team.'}, {'pitcher_id': '666204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Dany 9 9 2 1 0 1 1 11.1 7.1 0 0 0.222 0.667 0.373 0.22 0.402 0.262 81.5 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.695 0.293 0.774 0.441 88 39\nLipscomb, Trey 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.135 0.158 0.121 84.9 23\nRosario, Eddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.085 0.073 85.9 -20\nYoung, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.309 0.314 0.278 59.1 -65\nGarcía Jr., Luis 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.411 0.492 0.394 68.7 43\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.093 0.049 89.5 37'}, {'pitcher_id': '676664', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSears, JP 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.016 0.016 0.018 71.1 -23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.016 0.016 0.018 71.1 -23'}, {'pitcher_id': '547184', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Michael 7 7 1 1 0 0 0 0 0 0 0 0.143 0.286 0.181 0.308 0.375 0.295 88.9 28.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.296 0.212 89.6 48.3\nRuiz, Keibert 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.316 0.367 0.301 84.5 25\nYoung, Jacob 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.444 0.482 0.4 96.7 -2\nAbrams, CJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.449 0.522 0.425 88 6'}, {'pitcher_id': '621112', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackburn, Paul 45 42 11 1 0 2 7 15.6 16.4 3 6.7 0.262 0.429 0.322 0.232 0.384 0.292 88.6 13.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 12 10 5 1 0 1 0 0 6.7 2 16.7 0.5 0.9 0.615 0.388 0.705 0.503 89.5 25.6\nAbrams, CJ 6 6 1 0 0 0 3 50 38.5 0 0 0.167 0.167 0.147 0.172 0.234 0.175 100.1 2\nMeneses, Joey 6 6 1 0 0 0 0 0 0 0 0 0.167 0.167 0.148 0.288 0.333 0.273 87.3 6.2\nRuiz, Keibert 3 2 1 0 0 1 0 0 0 1 33.3 0.5 2 0.9 0.225 0.682 0.48 90.8 1\nThomas, Lane 6 6 2 0 0 0 1 16.7 0 0 0 0.333 0.333 0.295 0.133 0.173 0.133 83.9 27.8\nRosario, Eddie 5 5 0 0 0 0 3 60 54.5 0 0 0 0 0 0.03 0.031 0.027 69.6 -35.5\nLipscomb, Trey 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.27 0.295 0.247 97.2 -12\nVargas, Ildemaro 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.288 0.489 0.326 91.1 10.7\nGarcía Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.15 0.529 0.283 86.2 49.5'}, {'pitcher_id': '621053', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605156', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrooks, Aaron 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.031 0.031 0.028 82.9 -21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorbin, Patrick 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.031 0.031 0.028 82.9 -21'}, {'pitcher_id': '663687', 'player_data': '', 'team_data': 'Hogan Harris has not faced anyone on this team.'}, {'pitcher_id': '695243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Mason 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '548389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStripling, Ross 12 11 2 0 0 0 3 25 30 1 8.3 0.182 0.182 0.205 0.214 0.266 0.251 89.6 -4.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 7 6 2 0 0 0 1 14.3 30 1 14.3 0.333 0.333 0.351 0.276 0.324 0.327 87.7 -7.2\nRosario, Eddie 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.209 0.306 0.219 97.7 10\nVargas, Ildemaro 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCorbin, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.069 0.062 82.9 -21'}, {'pitcher_id': '674370', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBido, Osvaldo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.186 0.204 0.173 75.5 39.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.186 0.204 0.173 75.5 39.5'}, {'pitcher_id': '666205', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuller, Kyle 25 23 6 2 0 1 4 16 20.4 2 8 0.261 0.478 0.343 0.262 0.393 0.311 90.5 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 4 4 1 0 0 0 1 25 16.7 0 0 0.25 0.25 0.221 0.267 0.285 0.242 91.7 29\nWinker, Jesse 4 4 2 1 0 1 1 25 22.2 0 0 0.5 1.5 0.819 0.298 0.72 0.413 96.9 32.3\nMeneses, Joey 3 2 1 0 0 0 1 33.3 12.5 1 33.3 0.5 0.5 0.524 0.019 0.019 0.242 80.1 -21\nAbrams, CJ 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.255 0.372 0.265 85.2 13\nGarcía Jr., Luis 3 3 1 1 0 0 0 0 14.3 0 0 0.333 0.667 0.423 0.348 0.521 0.371 98.8 0.3\nSenzel, Nick 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.023 0.028 0.358 77.9 59\nLipscomb, Trey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.15 0.15 0.135 86.5 -52\nRosario, Eddie 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVargas, Ildemaro 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.639 0.728 0.598 92 9.5\nYoung, Jacob 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.22 0.299 0.208 89.4 23'}, {'pitcher_id': '519008', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcFarland, T.J. 10 9 3 0 0 1 2 20 16.7 1 10 0.333 0.667 0.446 0.22 0.442 0.32 86.3 -14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.223 0.242 0.201 75.3 -25\nSenzel, Nick 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.291 0.294 0.478 99.8 -9\nAbrams, CJ 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nWinker, Jesse 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.45 1.406 0.766 95.3 -3\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLipscomb, Trey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.124 0.145 0.112 88.2 -10'}, {'pitcher_id': '671212', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBoyle, Joe 16 14 4 1 1 0 4 25 18.5 1 6.3 0.286 0.5 0.357 0.226 0.325 0.266 78.3 12.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.256 0.75 0.413 86.9 34.5\nThomas, Lane 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.231 0.05 0.056 0.262 70.4 -7\nLipscomb, Trey 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.071 0.084 0.068 93.3 -17\nGarcía Jr., Luis 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.151 0.163 0.136 81.4 5\nAbrams, CJ 3 3 2 1 1 0 1 33.3 20 0 0 0.667 1.667 0.961 0.565 0.667 0.535 102.4 10.5\nYoung, Jacob 2 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.285 0.309 0.256 57.2 -23.5\nWinker, Jesse 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.061 0.07 0.056 61.7 55'}, {'pitcher_id': '518397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Scott 14 12 4 0 0 0 3 21.4 24.2 2 14.3 0.333 0.333 0.35 0.293 0.34 0.337 90 -0.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 8 7 2 0 0 0 2 25 29.2 1 12.5 0.286 0.286 0.306 0.22 0.254 0.273 89.2 -5.6\nVargas, Ildemaro 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nWinker, Jesse 2 1 0 0 0 0 0 0 0 1 50 0 0 0.345 0.451 0.532 0.557 87.6 6\nAbrams, CJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.663 0.792 0.633 83.5 21\nMeneses, Joey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.234 0.273 0.211 95.3 -6\nThomas, Lane 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.631 0.699 0.578 97.8 6'}, {'pitcher_id': '668674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nErceg, Lucas 11 9 2 0 0 1 2 18.2 0 1 9.1 0.222 0.556 0.391 0.236 0.476 0.371 90.9 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeneses, Joey 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.074 0.051 82.9 -2.7\nRosario, Eddie 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nAbrams, CJ 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.003 0.003 0.348 87.7 43\nThomas, Lane 2 1 0 0 0 0 1 50 0 0 0 0 0.363 0.363\nWinker, Jesse 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.469 0.563 0.458 95.7 3\nRuiz, Keibert 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.443 0.743 1.75 1.023 102 16.5'}, {'pitcher_id': '622072', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWood, Alex 58 53 20 4 0 3 9 15.5 12.4 3 5.2 0.377 0.623 0.45 0.354 0.572 0.422 88.2 8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 12 12 7 0 0 2 2 16.7 4.5 0 0 0.583 1.083 0.707 0.455 0.891 0.567 91.8 5.9\nCorbin, Patrick 7 7 4 1 0 0 3 42.9 14.3 0 0 0.571 0.714 0.555 0.222 0.287 0.221 89.8 14\nAbrams, CJ 8 8 4 1 0 1 0 0 15.4 0 0 0.5 1 0.63 0.503 0.795 0.552 85.3 13.9\nWinker, Jesse 4 3 0 0 0 0 1 25 40 1 25 0 0 0.173 0.018 0.02 0.186 80.4 11\nMeneses, Joey 4 2 0 0 0 0 0 0 11.1 1 25 0 0 0.355 0.333 0.41 0.516 80.9 18.5\nRuiz, Keibert 5 5 1 1 0 0 1 20 7.7 0 0 0.2 0.4 0.249 0.373 0.463 0.365 80 -7.5\nRosario, Eddie 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.294 0.28 0.315 0.261 76.2 0.7\nVargas, Ildemaro 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.178 0.313 0.404 0.311 96.7 6.8\nYoung, Jacob 3 2 2 1 0 0 0 0 0 0 0 1 1.5 0.961 0.579 0.819 0.644 99.5 15\nSenzel, Nick 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.237 0.749 0.412 90.4 34.5\nLipscomb, Trey 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.101 0.125 0.393 99.4 -28\nGarcía Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.373 0.554 0.396 92.2 12'}, {'pitcher_id': '687765', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpence, Mitch 6 6 4 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.495 0.59 0.476 88 14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbrams, CJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.114 0.12 0.106 88 -25\nWinker, Jesse 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.709 0.937 0.706 101.2 15\nYoung, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 88.6 39\nThomas, Lane 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.594 0.732 0.595 94.8 8\nMeneses, Joey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.583 0.636 0.536 71.5 32\nLipscomb, Trey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.969 1.113 0.912 83.7 16'}, {'pitcher_id': '613534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdams, Austin 4 3 0 0 0 0 2 50 66.7 1 25 0 0 0.173 0.04 0.141 0.227 93.6 35', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nLipscomb, Trey 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRosario, Eddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.12 0.423 0.214 93.6 35'}], 121: [{'pitcher_id': '683155', 'player_data': '', 'team_data': 'Joey Estes has not faced anyone on this team.'}, {'pitcher_id': '666204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Dany 4 3 2 0 0 1 0 0 25 1 25 0.667 1.667 0.896 0.702 1.741 0.931 100.4 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 1 1 1 0 0 1 0 0 50 0 0 1 4 2.004 0.989 3.955 1.988 111.7 29\nMcNeil, Jeff 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMartinez, J.D. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.446 0.499 0.408 83.4 9\nLindor, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.671 0.769 0.634 106 7'}, {'pitcher_id': '676664', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSears, JP 35 33 10 3 1 1 6 17.1 22.6 2 5.7 0.303 0.545 0.375 0.25 0.38 0.294 84.7 19.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 5 5 1 0 0 0 2 40 28.6 0 0 0.2 0.2 0.177 0.185 0.201 0.171 75.2 30.7\nLindor, Francisco 6 6 3 1 0 1 0 0 16.7 0 0 0.5 1.167 0.692 0.275 0.617 0.373 85.3 17\nNimmo, Brandon 6 5 2 0 0 0 1 16.7 12.5 1 16.7 0.4 0.4 0.409 0.387 0.456 0.422 87 25.3\nTorrens, Luis 5 5 1 1 0 0 1 20 20 0 0 0.2 0.4 0.249 0.236 0.267 0.221 82.3 12.5\nBader, Harrison 4 4 2 1 1 0 1 25 20 0 0 0.5 1.25 0.703 0.321 0.599 0.387 93.8 32.3\nMarte, Starling 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.046 0.092 0.059 77.3 14.5\nMcNeil, Jeff 4 3 1 0 0 0 0 0 20 1 25 0.333 0.333 0.393 0.322 0.418 0.408 82.8 11.3\nAlvarez, Francisco 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.088 0.144 0.094 94 7'}, {'pitcher_id': '547184', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '621112', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackburn, Paul 19 19 5 0 0 0 2 10.5 19.4 0 0 0.263 0.263 0.232 0.19 0.221 0.182 76 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 9 9 3 0 0 0 1 11.1 11.8 0 0 0.333 0.333 0.293 0.271 0.318 0.26 79.4 19\nMartinez, J.D. 5 5 2 0 0 0 0 0 25 0 0 0.4 0.4 0.354 0.205 0.237 0.198 74.7 -4.8\nBader, Harrison 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.019 0.021 0.019 76.6 21.5\nTorrens, Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.044 0.04 65.2 26.5'}, {'pitcher_id': '621053', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605156', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrooks, Aaron 17 14 3 1 0 0 2 11.8 26.9 1 5.9 0.214 0.286 0.301 0.293 0.476 0.396 91.8 -3.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 6 6 1 0 0 0 2 33.3 36.4 0 0 0.167 0.167 0.147 0.141 0.152 0.131 84.8 -11.3\nMartinez, J.D. 8 7 1 1 0 0 0 0 23.1 0 0 0.143 0.286 0.242 0.397 0.742 0.51 95.9 0\nNimmo, Brandon 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nMcNeil, Jeff 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.474 0.552 0.455 91.4 6\nMarte, Starling 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72'}, {'pitcher_id': '663687', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarris, Hogan 7 4 3 2 0 0 0 0 14.3 2 28.6 0.75 1.25 0.784 0.672 1.151 0.743 95.4 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 3 3 2 1 0 0 0 0 20 0 0 0.667 1 0.709 0.607 1.199 0.754 94.3 19.7\nMarte, Starling 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nAlonso, Pete 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nLindor, Francisco 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.869 1.005 0.818 98.6 11\nNimmo, Brandon 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '695243', 'player_data': '', 'team_data': 'Mason Miller has not faced anyone on this team.'}, {'pitcher_id': '548389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStripling, Ross 73 68 15 6 0 1 18 24.7 22.1 3 4.1 0.221 0.353 0.274 0.239 0.396 0.298 90 16.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 22 22 4 2 0 1 7 31.8 28.3 0 0 0.182 0.409 0.244 0.222 0.523 0.307 92 20.9\nMarte, Starling 15 11 3 0 0 0 1 6.7 21.4 2 13.3 0.273 0.273 0.365 0.28 0.391 0.401 86.6 22.6\nAlonso, Pete 6 6 1 1 0 0 1 16.7 7.7 0 0 0.167 0.333 0.203 0.321 0.385 0.302 97.3 4.4\nBader, Harrison 4 4 0 0 0 0 2 50 12.5 0 0 0 0 0 0.015 0.027 0.015 73.3 19.5\nNimmo, Brandon 5 5 0 0 0 0 3 60 37.5 0 0 0 0 0 0.034 0.104 0.057 91 59\nLindor, Francisco 4 3 1 1 0 0 0 0 20 1 25 0.333 0.667 0.485 0.342 0.442 0.425 87.5 4.3\nStewart, DJ 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.22 0.421 0.525 0.411 102.5 3\nIglesias, Jose 5 5 2 2 0 0 0 0 0 0 0 0.4 0.8 0.496 0.393 0.64 0.433 90.3 9.4\nMcNeil, Jeff 3 3 2 0 0 0 1 33.3 20 0 0 0.667 0.667 0.588 0.271 0.287 0.249 69.7 21\nAlvarez, Francisco 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.123 0.133 0.111 110.5 -14\nTorrens, Luis 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nTaylor, Tyrone 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.207 0.21 0.189 83.4 -6'}, {'pitcher_id': '674370', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBido, Osvaldo 9 7 1 0 0 1 1 11.1 15 2 22.2 0.143 0.571 0.377 0.256 0.582 0.425 90.7 38.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.402 1.152 0.633 87.4 34.5\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.01 0.007 87.1 67\nMartinez, J.D. 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.24 0.798 0.555 97 33\nAlvarez, Francisco 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nLindor, Francisco 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nMcNeil, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.02 0.015 89.9 45\nVientos, Mark 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.729 0.941 0.726 95.3 16'}, {'pitcher_id': '666205', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuller, Kyle 16 14 2 0 0 1 4 25 30.3 1 6.3 0.143 0.357 0.238 0.222 0.429 0.299 91.6 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 5 5 0 0 0 0 1 20 12.5 0 0 0 0 0 0.162 0.241 0.174 98.8 11.5\nMarte, Starling 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.231 0.231\nTaylor, Tyrone 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.121 0.131 0.116 85.5 24\nTorrens, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.397 0.817 0.494 100.2 21\nBader, Harrison 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.424 1.309 0.687 98.4 5.5\nLindor, Francisco 2 1 0 0 0 0 1 50 66.7 0 0 0 0 0 0 0 0 38.2 3\nMartinez, J.D. 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.879 0.817 1.101 0.82 106.8 13'}, {'pitcher_id': '519008', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcFarland, T.J. 26 23 6 0 1 2 3 11.5 13.9 3 11.5 0.261 0.609 0.402 0.276 0.54 0.378 91 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Starling 4 3 1 0 0 0 1 25 16.7 1 25 0.333 0.333 0.393 0.318 0.354 0.394 86.5 0.5\nIglesias, Jose 5 4 0 0 0 0 1 20 33.3 1 20 0 0 0.138 0.055 0.077 0.053 80.9 -30\nNimmo, Brandon 4 4 2 0 1 1 1 25 20 0 0 0.5 1.75 0.918 0.153 0.474 0.262 92.7 11.3\nBader, Harrison 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.438 0.255 0.403 0.283 99.4 9.5\nTorrens, Luis 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.171 0.254 0.185 95 10.5\nLindor, Francisco 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.243 0.249 0.38 75.2 -3\nMcNeil, Jeff 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.216 0.128 87.5 21.7\nAlonso, Pete 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.036 0.702 2.111 1.155 104.8 24.5\nTaylor, Tyrone 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.894 1.073 0.873 95.3 12'}, {'pitcher_id': '671212', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '518397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Scott 21 21 5 0 1 1 3 14.3 16.7 0 0 0.238 0.476 0.296 0.343 0.567 0.388 93 1.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 6 6 1 0 0 1 1 16.7 23.1 0 0 0.167 0.667 0.336 0.186 0.654 0.338 93.7 24.8\nMartinez, J.D. 4 4 2 0 1 0 1 25 16.7 0 0 0.5 1 0.609 0.657 1.045 0.725 102.4 13.3\nMcNeil, Jeff 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.385 0.437 0.369 80.7 3\nIglesias, Jose 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.259 0.269 0.234 92 -17\nNimmo, Brandon 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.158 0.174 0.149 95.1 -23.5\nBader, Harrison 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMarte, Starling 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.911 1.059 0.863 94.5 12\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.503 0.525 0.453 110.2 -1\nTorrens, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.177 0.177 0.165 66 -62'}, {'pitcher_id': '668674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nErceg, Lucas 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.351 0.351 0.316 72.5 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.351 0.351 0.316 72.5 13'}, {'pitcher_id': '622072', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWood, Alex 77 68 18 4 0 1 16 20.8 21.9 5 6.5 0.265 0.368 0.323 0.237 0.393 0.323 86.1 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Starling 28 25 5 1 0 0 7 25 25.8 2 7.1 0.2 0.24 0.247 0.207 0.252 0.272 79.5 -12.4\nIglesias, Jose 11 10 3 1 0 0 0 0 4.3 0 0 0.3 0.4 0.341 0.264 0.326 0.299 85.8 5.9\nNimmo, Brandon 8 6 2 0 0 0 3 37.5 35.7 1 12.5 0.333 0.333 0.398 0.319 0.39 0.411 97.1 6\nMartinez, J.D. 8 7 3 2 0 0 1 12.5 28.6 0 0 0.429 0.714 0.51 0.371 0.8 0.515 91.9 30.8\nAlonso, Pete 4 3 2 0 0 1 1 25 15.4 1 25 0.667 1.667 0.895 0.507 1.528 0.795 95.8 21\nBader, Harrison 5 5 1 0 0 0 2 40 37.5 0 0 0.2 0.2 0.176 0.17 0.39 0.225 98.8 28.7\nAlvarez, Francisco 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.046 0.074 0.372 78.7 36\nMcNeil, Jeff 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.237 0.279 0.215 77.2 13.7\nTorrens, Luis 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.064 0.121 0.076 78.4 42.5\nLindor, Francisco 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.22 0.074 0.107 0.078 87.6 12.3\nTaylor, Tyrone 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.021 0.074 0.037 95.7 42'}, {'pitcher_id': '687765', 'player_data': '', 'team_data': 'Mitch Spence has not faced anyone on this team.'}, {'pitcher_id': '613534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdams, Austin 13 9 1 0 0 0 5 38.5 33.3 3 23.1 0.111 0.111 0.284 0.094 0.145 0.286 85.6 0.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 5 4 1 0 0 0 2 40 22.2 0 0 0.25 0.25 0.322 0.18 0.294 0.302 92.6 21.5\nAlonso, Pete 4 4 0 0 0 0 3 75 50 0 0 0 0 0 0.012 0.012 0.013 72.6 -26\nMarte, Starling 3 1 0 0 0 0 0 0 0 2 66.7 0 0 0.463 0.077 0.081 0.487 84.8 -15\nMcNeil, Jeff 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696'}], 134: [{'pitcher_id': '683155', 'player_data': '', 'team_data': 'Joey Estes has not faced anyone on this team.'}, {'pitcher_id': '666204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Dany 10 7 0 0 0 0 2 20 41.7 3 30 0 0 0.208 0.038 0.044 0.232 89.7 34.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nJoe, Connor 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nCruz, Oneil 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.003 0.005 0.003 91.7 51\nSuwinski, Jack 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nReynolds, Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.016 0.013 93.8 57\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.188 0.198 0.17 94.9 -10\nOlivares, Edward 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.014 0.022 0.014 80.7 42\nTellez, Rowdy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.063 0.044 87.2 31'}, {'pitcher_id': '676664', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSears, JP 13 10 1 1 0 0 4 30.8 40 3 23.1 0.1 0.2 0.256 0.163 0.212 0.287 77 15.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 3 1 0 0 0 0 0 0 50 2 66.7 0 0 0.464 0.006 0.008 0.466 60.7 87\nReynolds, Bryan 3 3 0 0 0 0 2 66.7 28.6 0 0 0 0 0 0.034 0.035 0.032 65.6 -40\nJoe, Connor 3 2 1 1 0 0 1 33.3 25 1 33.3 0.5 1 0.647 0.344 0.385 0.447 65.2 36\nTaylor, Michael A. 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.03 0.033 0.03 83.4 -30\nHayes, Ke'Bryan 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.389 0.585 0.418 93.7 21"}, {'pitcher_id': '547184', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Michael 13 13 3 1 0 0 3 23.1 11.1 0 0 0.231 0.308 0.232 0.23 0.265 0.213 81.4 14.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTriolo, Jared 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.042 0.06 0.04 79.2 31\nOlivares, Edward 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.533 0.586 0.483 82.2 2\nMcCutchen, Andrew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.161 0.24 0.169 78 23.5\nReynolds, Bryan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nJoe, Connor 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nGonzales, Nick 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.27 0.314 0.246 89.1 -2.5\nCruz, Oneil 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.494 0.533 0.445 81.8 12\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.477 0.487 0.429 69 15\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.019 0.012 85.2 43"}, {'pitcher_id': '621112', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackburn, Paul 9 9 1 0 0 0 1 11.1 10.5 0 0 0.111 0.111 0.098 0.225 0.319 0.237 85.2 23.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 5 5 0 0 0 0 0 0 11.1 0 0 0 0 0 0.197 0.234 0.191 86.5 13\nTellez, Rowdy 3 3 1 0 0 0 1 33.3 11.1 0 0 0.333 0.333 0.294 0.275 0.302 0.256 71.3 39\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.214 0.799 0.407 106.9 45'}, {'pitcher_id': '621053', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605156', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrooks, Aaron 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.457 0.631 0.472 96.1 21.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.076 0.045 86.1 31\nReynolds, Bryan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.874 1.185 0.898 106.1 12"}, {'pitcher_id': '663687', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarris, Hogan 16 14 3 0 0 1 3 18.8 23.8 2 12.5 0.214 0.429 0.323 0.213 0.429 0.324 84.5 12.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 3 1 0 0 0 0 0 0 0 2 66.7 0 0 0.464 0.009 0.019 0.468 95.1 52\nReynolds, Bryan 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.06 0.069 0.057 99.5 -31\nJoe, Connor 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.197 0.218 0.185 75.9 -8\nTaylor, Michael A. 3 3 2 0 0 1 1 33.3 20 0 0 0.667 1.667 0.962 0.401 1.325 0.702 67.3 25.5\nDelay, Jason 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.009 0.011 0.007 94 61\nHayes, Ke'Bryan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.493 0.564 0.465 89.6 31.5"}, {'pitcher_id': '695243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Mason 7 7 0 0 0 0 4 57.1 30.8 0 0 0 0 0 0.08 0.092 0.073 84 11.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 3 3 0 0 0 0 2 66.7 20 0 0 0 0 0 0.005 0.005 0.004 84.9 33\nTriolo, Jared 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.286 0.214 80.7 1\nCruz, Oneil 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSuwinski, Jack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.314 0.344 0.283 86.3 1"}, {'pitcher_id': '548389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStripling, Ross 67 64 15 2 0 3 11 16.4 21.5 3 4.5 0.234 0.406 0.29 0.251 0.411 0.299 92.1 15.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 12 11 6 1 0 2 0 0 13.6 1 8.3 0.545 1.182 0.713 0.387 0.875 0.538 97.2 6.5\nReynolds, Bryan 9 9 1 1 0 0 2 22.2 20 0 0 0.111 0.222 0.141 0.094 0.167 0.108 86.1 23.4\nTellez, Rowdy 5 4 2 0 0 0 0 0 7.7 1 20 0.5 0.5 0.493 0.5 0.633 0.533 100.7 1.8\nTaylor, Michael A. 7 7 1 0 0 0 2 28.6 35.7 0 0 0.143 0.143 0.126 0.171 0.232 0.172 91.1 21.4\nCruz, Oneil 6 6 2 0 0 1 3 50 38.5 0 0 0.333 0.833 0.493 0.225 0.396 0.263 94.6 5.7\nSuwinski, Jack 7 7 0 0 0 0 1 14.3 12.5 0 0 0 0 0 0.102 0.162 0.114 90.3 37.6\nTriolo, Jared 4 4 1 0 0 0 0 0 9.1 0 0 0.25 0.25 0.221 0.351 0.399 0.329 78 11.3\nGrandal, Yasmani 5 4 1 0 0 0 1 20 28.6 1 20 0.25 0.25 0.312 0.319 0.372 0.374 83.6 23\nOlivares, Edward 4 4 1 0 0 0 0 0 12.5 0 0 0.25 0.25 0.221 0.418 0.552 0.418 96.5 7.5\nJoe, Connor 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.134 0.248 0.161 94.6 20.7\nGonzales, Nick 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nHayes, Ke'Bryan 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.267 0.405 0.285 96 20.7"}, {'pitcher_id': '674370', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBido, Osvaldo 4 4 2 0 0 0 1 25 11.1 0 0 0.5 0.5 0.441 0.518 0.613 0.496 94.1 11.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 2 2 1 0 0 0 1 50 20 0 0 0.5 0.5 0.441 0.308 0.354 0.29 104.2 5\nTellez, Rowdy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.729 0.873 0.702 89 15'}, {'pitcher_id': '666205', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuller, Kyle 5 5 2 0 0 0 0 0 22.2 0 0 0.4 0.4 0.353 0.363 0.595 0.41 102.1 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.181 0.215 0.168 99.3 19.5\nOlivares, Edward 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.484 0.847 0.572 104 6'}, {'pitcher_id': '519008', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcFarland, T.J. 39 37 6 1 0 1 11 28.2 20.7 1 2.6 0.162 0.27 0.193 0.2 0.288 0.222 89.4 -0.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 7 5 0 0 0 0 1 14.3 14.3 1 14.3 0 0 0.099 0.08 0.083 0.166 79.8 -5.8\nMcCutchen, Andrew 5 5 2 0 0 0 1 20 25 0 0 0.4 0.4 0.352 0.282 0.317 0.262 93.7 -7\nTaylor, Michael A. 7 7 2 0 0 1 1 14.3 16.7 0 0 0.286 0.714 0.416 0.289 0.586 0.372 84.3 -2.5\nSuwinski, Jack 4 4 0 0 0 0 4 100 100 0 0 0 0 0 0\nTellez, Rowdy 6 6 0 0 0 0 2 33.3 12.5 0 0 0 0 0 0.075 0.104 0.081 93.2 14\nHayes, Ke'Bryan 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.235 0.313 0.229 98.8 -1.3\nOlivares, Edward 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.42 0.338 0.436 0.339 90.9 21.5\nTriolo, Jared 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.791 0.968 0.793 96 11\nReynolds, Bryan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.18 0.195 0.177 94.2 -20.5"}, {'pitcher_id': '671212', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBoyle, Joe 17 14 1 0 0 0 4 23.5 24 3 17.6 0.071 0.071 0.174 0.171 0.297 0.285 93.3 23.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuwinski, Jack 2 1 0 0 0 0 1 50 25 1 50 0 0 0.346 0.346\nHayes, Ke'Bryan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.279 0.358 0.272 95.2 -0.3\nTriolo, Jared 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.081 0.259 0.139 95.3 39\nTaylor, Michael A. 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.371 0.884 0.522 105.6 20\nMcCutchen, Andrew 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.151 0.239 0.429 88.3 24\nTellez, Rowdy 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.032 0.036 0.029 72.8 56\nCruz, Oneil 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.007 0.007 0.006 84.9 59\nReynolds, Bryan 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.423 0.469 0.546 105.7 -3"}, {'pitcher_id': '518397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Scott 13 12 1 0 0 0 3 23.1 38.1 1 7.7 0.083 0.083 0.12 0.155 0.171 0.188 90.1 -17.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 4 3 1 0 0 0 1 25 60 1 25 0.333 0.333 0.39 0.217 0.25 0.326 96.9 -10\nTaylor, Michael A. 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.119 0.121 0.11 103.8 -16\nGonzales, Nick 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.226 0.26 0.209 93 -8\nSuwinski, Jack 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nReynolds, Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.155 0.173 0.158 81.3 -3\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.211 0.213 0.192 80.8 -52\nMcCutchen, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.076 0.079 81 -21\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.083 0.074 84.7 -26"}, {'pitcher_id': '668674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nErceg, Lucas 8 8 3 0 1 0 2 25 26.7 0 0 0.375 0.625 0.417 0.283 0.423 0.302 88.1 -10.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.143 0.143 0.129 62.3 -55.5\nMcCutchen, Andrew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGrandal, Yasmani 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.869 1.005 0.818 99 11\nHayes, Ke'Bryan 2 2 2 0 1 0 0 0 0 0 0 1 2 1.226 0.499 0.931 0.6 106 5.5\nSuwinski, Jack 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.234 0.137 93.1 27"}, {'pitcher_id': '622072', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWood, Alex 143 126 44 9 1 4 35 24.5 23.3 14 9.8 0.349 0.532 0.411 0.286 0.441 0.359 91.6 8.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 43 38 7 2 0 1 10 23.3 24.3 5 11.6 0.184 0.316 0.269 0.249 0.497 0.356 93.8 15.1\nJoe, Connor 21 17 8 0 0 2 0 0 14.7 3 14.3 0.471 0.824 0.583 0.432 0.666 0.518 89.2 17.2\nReynolds, Bryan 23 20 9 2 0 0 4 17.4 21.3 2 8.7 0.45 0.55 0.467 0.292 0.372 0.343 89.3 -9.7\nTaylor, Michael A. 20 19 9 2 0 1 6 30 22.7 1 5 0.474 0.737 0.527 0.358 0.546 0.407 97 11.6\nHayes, Ke'Bryan 12 11 5 2 1 0 5 41.7 26.1 0 0 0.455 0.818 0.487 0.178 0.231 0.223 95 -10.9\nSuwinski, Jack 6 5 1 0 0 0 3 50 30 1 16.7 0.2 0.2 0.263 0.184 0.225 0.263 90.9 10\nCruz, Oneil 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.005 0.008 0.005 86.9 52\nDelay, Jason 4 3 3 1 0 0 0 0 0 1 25 1 1.333 0.927 0.607 0.732 0.6 92.5 8.3\nTriolo, Jared 4 4 2 0 0 0 1 25 0 0 0 0.5 0.5 0.443 0.367 0.403 0.339 78.4 28.7\nOlivares, Edward 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346\nGonzales, Nick 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.038 0.058 0.037 82.1 29\nGrandal, Yasmani 1 1 0 0 0 0 1 100 50 0 0 0 0 0\nKeller, Mitch 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.024 0.024 0.024 77.9 -37"}, {'pitcher_id': '687765', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpence, Mitch 8 8 1 0 0 0 1 12.5 18.2 0 0 0.125 0.125 0.111 0.21 0.283 0.211 85.5 2.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.38 0.492 0.374 82.6 13.5\nMcCutchen, Andrew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.23 0.256 0.219 90.3 -4\nReynolds, Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.134 0.099 75.7 36\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.32 0.591 0.369 98.6 21\nOlivares, Edward 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.156 0.138 85.1 -50\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.121 0.143 0.114 84 -13"}, {'pitcher_id': '613534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdams, Austin 10 8 0 0 0 0 4 40 40 2 20 0 0 0.138 0.098 0.109 0.212 80.5 31', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJoe, Connor 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.01 0.018 0.013 88.9 48\nMcCutchen, Andrew 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.25 0.272 0.232 72.2 14\nReynolds, Bryan 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nGrandal, Yasmani 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nHayes, Ke'Bryan 1 0 0 0 0 0 0 0 1 100 0.692 0.692"}], 135: [{'pitcher_id': '683155', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstes, Joey 27 25 8 3 0 2 4 14.8 15.5 2 7.4 0.32 0.68 0.446 0.256 0.485 0.338 88.3 25.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 6 6 0 0 0 0 3 50 35.7 0 0 0 0 0 0.104 0.114 0.095 85.9 16.3\nMachado, Manny 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.296 0.044 0.067 0.05 80.5 46.5\nProfar, Jurickson 3 2 0 0 0 0 0 0 14.3 1 33.3 0 0 0.231 0.1 0.127 0.295 73.4 40.5\nCronenworth, Jake 3 3 2 1 0 1 0 0 0 0 0 0.667 2 1.118 0.634 1.88 1.025 101.3 22.3\nArraez, Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.36 0.475 0.351 94.4 20.7\nKim, Ha-Seong 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.869 1.175 0.784 106.6 13\nTatis Jr., Fernando 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.695 0.283 0.561 0.356 89.8 33\nMerrill, Jackson 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.149 0.174 0.138 76.9 -4\nCampusano, Luis 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.217 0.363 0.24 85.8 37.5'}, {'pitcher_id': '666204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Dany 11 10 0 0 0 0 5 45.5 35.3 1 9.1 0 0 0.063 0.042 0.078 0.116 94.9 8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKim, Ha-Seong 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.692\nCronenworth, Jake 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.003 0.009 0.004 96.3 45\nSolano, Donovan 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nMachado, Manny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMerrill, Jackson 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nProfar, Jurickson 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.127 0.434 0.228 91.8 31\nCampusano, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.074 0.067 84 -24\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.175 0.175 0.168 107.3 -19'}, {'pitcher_id': '676664', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSears, JP 30 26 9 3 0 0 1 3.3 22.7 3 10 0.346 0.462 0.397 0.376 0.603 0.455 90.1 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 6 4 2 0 0 0 0 0 0 1 16.7 0.5 0.5 0.532 0.419 0.541 0.513 94.7 24.8\nHigashioka, Kyle 4 3 1 1 0 0 0 0 50 1 25 0.333 0.667 0.484 0.355 0.738 0.511 86.5 40.7\nSolano, Donovan 5 4 0 0 0 0 0 0 25 1 20 0 0 0.138 0.161 0.181 0.258 71.8 8.3\nMerrill, Jackson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.175 0.147 64.7 3.5\nCronenworth, Jake 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.118 0.123 0.107 83.5 -1\nMachado, Manny 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.593 0.859 0.619 110.6 1\nTatis Jr., Fernando 3 3 2 1 0 0 0 0 25 0 0 0.667 1 0.719 0.726 1.446 0.915 107.4 8.7\nKim, Ha-Seong 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.311 0.683 0.413 92.5 14.5\nArraez, Luis 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.593 0.426 0.563 0.431 91.7 11'}, {'pitcher_id': '547184', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '621112', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBlackburn, Paul 18 17 7 1 0 0 0 0 3.7 0 0 0.412 0.471 0.402 0.342 0.385 0.336 82.8 1.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWade, Tyler 5 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.32 0.372 0.431 0.414 73.9 -16.8\nProfar, Jurickson 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.294 0.417 0.462 0.383 73.9 -4.3\nSolano, Donovan 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.174 0.194 0.159 95.7 -11\nMachado, Manny 5 5 2 1 0 0 0 0 0 0 0 0.4 0.6 0.422 0.333 0.366 0.305 90.6 23.2\nArraez, Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.356 0.408 0.33 82 6'}, {'pitcher_id': '621053', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605156', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrooks, Aaron 5 5 1 0 0 1 2 40 37.5 0 0 0.2 0.8 0.413 0.419 0.995 0.592 106.9 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.302 0.641 0.397 105.4 6\nMachado, Manny 1 1 1 0 0 1 0 0 0 0 0 1 4 2.065 0.887 2.413 1.373 109.8 18'}, {'pitcher_id': '663687', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarris, Hogan 24 23 4 0 0 2 7 29.2 24.5 1 4.2 0.174 0.435 0.277 0.196 0.448 0.286 86.9 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 4 4 1 0 0 1 2 50 28.6 0 0 0.25 1 0.521 0.222 0.834 0.428 97.7 44\nArraez, Luis 5 5 1 0 0 0 1 20 18.2 0 0 0.2 0.2 0.178 0.169 0.184 0.153 75.3 -11\nCronenworth, Jake 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.8 1.016 0.743 109.2 9\nTatis Jr., Fernando 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.036 0.039 0.032 91.7 -23\nKim, Ha-Seong 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.192 0.21 0.176 90.2 18\nMachado, Manny 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.015 0.022 0.017 98.8 60\nAzocar, José 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.155 0.171 0.14 69.4 10\nMerrill, Jackson 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.512 1.957 1.002 87.9 4\nCampusano, Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.063 0.098 0.064 84.6 8'}, {'pitcher_id': '695243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Mason 6 4 2 1 0 1 1 16.7 20 2 33.3 0.5 1.5 0.787 0.396 0.801 0.565 99.4 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKim, Ha-Seong 2 1 0 0 0 0 0 0 25 1 50 0 0 0.348 0.577 0.643 0.615 103.7 5\nSolano, Donovan 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nCampusano, Luis 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMerrill, Jackson 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.574 2.067 1.067 99.6 30\nTatis Jr., Fernando 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.434 0.494 0.4 95 2'}, {'pitcher_id': '548389', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStripling, Ross 70 64 26 8 0 5 11 15.7 20.9 5 7.1 0.406 0.766 0.502 0.354 0.643 0.441 88.6 10.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 31 29 12 6 0 0 4 12.9 25.5 2 6.5 0.414 0.621 0.453 0.294 0.404 0.327 86.5 6.5\nMachado, Manny 8 7 2 0 0 1 1 12.5 14.3 1 12.5 0.286 0.714 0.445 0.48 0.984 0.623 94.9 17.7\nTatis Jr., Fernando 8 8 5 0 0 2 2 25 23.1 0 0 0.625 1.375 0.821 0.459 1.115 0.643 105 9.7\nHigashioka, Kyle 6 6 1 0 0 1 2 33.3 30 0 0 0.167 0.667 0.334 0.239 0.738 0.4 89.3 12.5\nCronenworth, Jake 5 5 2 2 0 0 1 20 14.3 0 0 0.4 0.8 0.495 0.463 0.991 0.607 94.9 19.3\nWade, Tyler 4 3 1 0 0 0 1 25 25 1 25 0.333 0.333 0.393 0.342 0.406 0.421 81 -4.5\nProfar, Jurickson 4 2 1 0 0 1 0 0 0 1 25 0.5 2 0.893 0.09 0.314 0.343 68.9 29\nArraez, Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.405 0.459 0.382 78.4 1.7\nSolano, Donovan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.92 1.046 0.855 84 14'}, {'pitcher_id': '674370', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBido, Osvaldo 7 6 3 1 0 1 1 14.3 10 0 0 0.5 1.167 0.694 0.332 0.726 0.482 97.3 22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 2 1 0 0 0 0 1 50 25 0 0 0 0 0.363 0.363\nArraez, Luis 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.343 0.99 0.543 97.1 27\nSolano, Donovan 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.063 0.482 0.692 0.508 97.6 14.5'}, {'pitcher_id': '666205', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuller, Kyle 12 9 3 2 0 0 1 8.3 17.9 2 16.7 0.333 0.556 0.396 0.52 1.127 0.665 96.6 22.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 2 2 1 1 0 0 0 0 16.7 0 0 0.5 1 0.621 0.636 1.162 0.759 98.5 20\nTatis Jr., Fernando 2 1 1 0 0 0 0 0 14.3 1 50 1 1 0.785 0.931 1.025 0.775 79.9 19\nKim, Ha-Seong 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.376 0.413 0.378 95.9 9.5\nCronenworth, Jake 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.216 0.429 0.272 93.4 35.5\nMachado, Manny 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.837 3.336 1.189 106 34\nHigashioka, Kyle 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.622 0.415 1.093 0.62 107.7 20'}, {'pitcher_id': '519008', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcFarland, T.J. 24 24 5 0 0 1 5 20.8 20.6 0 0 0.208 0.333 0.228 0.199 0.259 0.197 91 -3.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.137 0.146 0.124 87.7 6.5\nTatis Jr., Fernando 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.125 0.137 0.113 101.5 -5\nCronenworth, Jake 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.293 0.217 0.242 0.205 95.6 -1.5\nSolano, Donovan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.29 0.293 0.326 0.272 92.5 -3\nArraez, Luis 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.1 0.111 0.096 83.2 -14\nHigashioka, Kyle 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nPeralta, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.145 0.158 0.136 98.1 -19\nAzocar, José 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.19 0.193 0.171 58.5 -44\nCampusano, Luis 2 2 1 0 0 1 0 0 0 0 0 0.5 2 0.99 0.299 0.611 0.366 90.3 -8.5\nMachado, Manny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.793 1.073 0.776 109.1 9\nMerrill, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.265 0.135 96.5 38\nKim, Ha-Seong 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.227 0.271 0.209 99.1 -15'}, {'pitcher_id': '671212', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBoyle, Joe 7 5 0 0 0 0 2 28.6 15.4 2 28.6 0 0 0.199 0.018 0.021 0.212 86 30', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAzocar, José 1 0 0 0 0 0 0 0 40 1 100 0.696 0.696\nTatis Jr., Fernando 2 2 0 0 0 0 2 100 0 0 0 0 0 0 0\nProfar, Jurickson 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.017 0.03 0.358 88.7 61\nMachado, Manny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.069 0.063 80.6 -14\nCampusano, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 88.6 43'}, {'pitcher_id': '518397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Scott 33 30 7 1 0 1 6 18.2 12.9 2 6.1 0.233 0.367 0.298 0.251 0.368 0.307 90.3 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 7 6 1 0 0 0 3 42.9 12.5 1 14.3 0.167 0.167 0.223 0.134 0.188 0.218 92.1 8.3\nCronenworth, Jake 8 7 0 0 0 0 2 25 23.1 0 0 0 0 0.09 0.15 0.201 0.236 83 6.3\nKim, Ha-Seong 5 4 2 0 0 0 0 0 16.7 1 20 0.5 0.5 0.492 0.327 0.372 0.382 95.4 -11.3\nArraez, Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.312 0.49 0.343 89.6 8.7\nProfar, Jurickson 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.255 0.283 0.233 96.2 -16\nHigashioka, Kyle 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.043 0.45 1.294 0.72 82.6 -8.5\nSolano, Donovan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.165 0.152 92.2 -4\nTatis Jr., Fernando 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMerrill, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.417 0.551 0.416 86.5 21\nMachado, Manny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.877 0.165 0.17 0.149 90.3 -17\nAzocar, José 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.911 1.059 0.863 94.7 12'}, {'pitcher_id': '668674', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nErceg, Lucas 11 9 3 1 0 1 3 27.3 25 2 18.2 0.333 0.778 0.509 0.193 0.514 0.364 88.3 4.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 4 3 1 0 0 1 1 25 9.1 1 25 0.333 1.333 0.695 0.234 0.768 0.481 99.4 6\nTatis Jr., Fernando 2 1 0 0 0 0 0 0 66.7 1 50 0 0 0.346 0.086 0.133 0.385 77.7 -11\nKim, Ha-Seong 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nCronenworth, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.097 0.08 81.5 -16\nArraez, Luis 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.514 1.738 0.916 99.4 28\nMachado, Manny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nPeralta, David 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.351 0.351 0.316 72.3 13'}, {'pitcher_id': '622072', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWood, Alex 112 103 26 2 1 1 23 20.5 22.3 5 4.5 0.252 0.32 0.274 0.225 0.322 0.269 88.1 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 17 16 5 1 0 1 4 23.5 27 1 5.9 0.313 0.563 0.389 0.298 0.558 0.382 92.9 30.9\nKim, Ha-Seong 15 13 2 0 0 0 2 13.3 10 1 6.7 0.154 0.154 0.212 0.26 0.318 0.314 88.9 13.6\nProfar, Jurickson 15 14 4 0 0 0 2 13.3 22.7 1 6.7 0.286 0.286 0.281 0.226 0.258 0.247 84.8 2.8\nCronenworth, Jake 14 12 3 0 0 0 2 14.3 22.2 0 0 0.25 0.25 0.241 0.272 0.347 0.291 87.7 15.6\nSolano, Donovan 16 16 4 0 1 0 4 25 33.3 0 0 0.25 0.375 0.267 0.239 0.309 0.234 88.1 10.7\nPeralta, David 15 14 3 0 0 0 4 26.7 29.2 1 6.7 0.214 0.214 0.223 0.117 0.129 0.161 86.6 -11.3\nAzocar, José 10 9 2 0 0 0 4 40 25 0 0 0.222 0.222 0.196 0.11 0.122 0.102 76.6 3.5\nTatis Jr., Fernando 6 5 1 1 0 0 1 16.7 6.3 1 16.7 0.2 0.4 0.323 0.19 0.462 0.335 95.6 17\nCampusano, Luis 3 3 2 0 0 0 0 0 28.6 0 0 0.667 0.667 0.589 0.287 0.339 0.267 94.2 -6.3\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.173 0.106 88.5 27'}, {'pitcher_id': '687765', 'player_data': '', 'team_data': 'Mitch Spence has not faced anyone on this team.'}, {'pitcher_id': '613534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdams, Austin 6 5 2 0 0 0 1 16.7 33.3 0 0 0.4 0.4 0.417 0.559 0.728 0.582 81.8 24.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, David 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.241 0.208 0.521 0.441 98.3 25\nSolano, Donovan 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.917 0.987 0.838 75.1 23\nProfar, Jurickson 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.547 0.65 0.502 84.1 21\nMachado, Manny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.914 0.959 0.829 69.5 30'}]}, 134: {120: [{'pitcher_id': '669796', 'player_data': '', 'team_data': ''}, {'pitcher_id': '669387', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMlodzinski, Carmen 6 6 1 1 0 0 3 50 30.8 0 0 0.167 0.333 0.207 0.19 0.37 0.236 100.3 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbrams, CJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.271 1.007 0.524 97.9 35\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nVargas, Ildemaro 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.746 1.087 0.775 104.6 14\nYoung, Jacob 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRosario, Eddie 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMeneses, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.125 0.125 0.114 98.5 -18'}, {'pitcher_id': '641726', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663559', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFalter, Bailey 48 47 11 3 0 0 13 27.1 24.1 1 2.1 0.234 0.298 0.24 0.255 0.389 0.286 87.7 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 9 8 0 0 0 0 2 22.2 25 1 11.1 0 0 0.077 0.181 0.491 0.325 92.1 26\nGarcía Jr., Luis 7 7 2 1 0 0 4 57.1 41.7 0 0 0.286 0.429 0.306 0.193 0.389 0.248 86.8 23.3\nAbrams, CJ 6 6 0 0 0 0 3 50 55.6 0 0 0 0 0 0.233 0.297 0.23 91.2 -12\nRuiz, Keibert 7 7 2 0 0 0 1 14.3 20 0 0 0.286 0.286 0.252 0.287 0.324 0.272 83.6 20.8\nWinker, Jesse 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.295 0.185 0.225 0.179 68.5 -14.5\nMeneses, Joey 5 5 4 2 0 0 0 0 0 0 0 0.8 1.2 0.854 0.418 0.616 0.443 94.3 17\nSenzel, Nick 4 4 1 0 0 0 1 25 0 0 0 0.25 0.25 0.221 0.223 0.267 0.211 88.3 20.3\nVargas, Ildemaro 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.293 0.375 0.295 87.7 -12.5\nYoung, Jacob 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.354 0.398 0.33 78.5 23\nCorbin, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.363 0.452 0.351 90.4 3'}, {'pitcher_id': '656924', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Ryder 6 5 2 0 0 0 2 33.3 14.3 1 16.7 0.4 0.4 0.412 0.177 0.178 0.248 80.4 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbrams, CJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.594 0.595 0.535 72.6 30\nVargas, Ildemaro 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nRosario, Eddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.009 0.004 93.1 43\nMeneses, Joey 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nLipscomb, Trey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.287 0.287 0.258 75.4 -66\nRuiz, Keibert 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '676702', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Hunter 9 9 3 0 0 0 2 22.2 26.7 0 0 0.333 0.333 0.295 0.386 0.442 0.366 95 0.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAbrams, CJ 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.448 0.508 0.42 98 -2\nMeneses, Joey 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.28 0.332 0.272 100.6 3\nThomas, Lane 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.425 0.49 0.406 96 -4.5\nRuiz, Keibert 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.201 0.201 0.181 87.5 -1\nGarcía Jr., Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVargas, Ildemaro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.969 1.114 0.916 89.1 16'}, {'pitcher_id': '670059', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolderman, Colin 9 8 0 0 0 0 1 11.1 26.7 1 11.1 0 0 0.077 0.211 0.329 0.278 87.6 20.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYoung, Jacob 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.172 0.095 91.5 28\nMeneses, Joey 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.269 0.51 0.318 95.6 21.5\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.044 0.044 0.043 72.4 -4\nAbrams, CJ 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nThomas, Lane 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.192 0.342 0.226 96.5 19\nWinker, Jesse 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.089 0.089 0.08 58.6 50\nRuiz, Keibert 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.557 0.623 0.504 103.1 5'}, {'pitcher_id': '670280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBednar, David 19 18 4 1 0 0 4 21.1 21.2 1 5.3 0.222 0.278 0.241 0.154 0.281 0.213 90.8 26', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 5 5 0 0 0 0 2 40 11.1 0 0 0 0 0 0.099 0.262 0.151 94 39.7\nAbrams, CJ 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.08 0.083 0.074 96.4 19.5\nSenzel, Nick 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.293 0.198 0.371 0.24 83 40.3\nRuiz, Keibert 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.138 0.396 0.226 91.8 20.3\nVargas, Ildemaro 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nWinker, Jesse 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.8 0.947 0.754 102.5 10\nMeneses, Joey 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nGarcía Jr., Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.119 0.119 0.111 91.4 -19\nRosario, Eddie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.139 0.102 77.5 33'}, {'pitcher_id': '594835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzales, Marco 19 18 5 0 0 0 3 15.8 5.1 1 5.3 0.278 0.278 0.27 0.291 0.372 0.31 88.8 6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 3 2 1 0 0 0 0 0 9.1 1 33.3 0.5 0.5 0.527 0.617 0.689 0.614 89.8 10.5\nWinker, Jesse 3 3 1 0 0 0 2 66.7 12.5 0 0 0.333 0.333 0.294 0.321 0.361 0.3 83.5 17\nLipscomb, Trey 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.352 0.413 0.337 98 -4\nAbrams, CJ 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.296 0.235 0.265 0.223 91.6 -3.5\nRuiz, Keibert 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.489 0.553 0.455 97.6 3.5\nMeneses, Joey 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.162 0.55 0.29 87.9 6.5\nVargas, Ildemaro 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.142 0.108 79 58.5\nRosario, Eddie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.058 0.051 80.2 -30'}, {'pitcher_id': '527048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Martín 37 35 10 0 0 1 8 21.6 13.1 2 5.4 0.286 0.371 0.309 0.243 0.365 0.283 84.9 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 9 9 2 0 0 0 3 33.3 5.6 0 0 0.222 0.222 0.197 0.129 0.141 0.12 70.6 -4.7\nAbrams, CJ 4 4 2 0 0 1 2 50 30 0 0 0.5 1.25 0.744 0.416 1.125 0.637 104.8 19\nVargas, Ildemaro 3 2 0 0 0 0 1 33.3 16.7 1 33.3 0 0 0.231 0.063 0.081 0.272 71.3 53\nRosario, Eddie 5 5 3 0 0 0 1 20 25 0 0 0.6 0.6 0.526 0.454 0.514 0.422 85.4 9.8\nThomas, Lane 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.025 0.075 0.205 92.8 60.3\nMeneses, Joey 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.316 0.449 0.323 95.7 26.7\nYoung, Jacob 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.067 0.098 0.068 80.9 45.7\nLipscomb, Trey 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.258 0.287 0.242 80.5 -24.5\nSenzel, Nick 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.428 0.515 0.408 92.6 13.3'}, {'pitcher_id': '693312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNicolas, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.201 0.108 90.7 31', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSenzel, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.201 0.108 90.7 31'}, {'pitcher_id': '656605', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKeller, Mitch 80 73 20 5 0 1 19 23.8 25.2 5 6.3 0.274 0.384 0.309 0.237 0.367 0.289 84.4 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 11 11 4 1 0 0 4 36.4 26.9 0 0 0.364 0.455 0.351 0.217 0.264 0.211 78.9 23.2\nMeneses, Joey 6 4 0 0 0 0 2 33.3 26.7 2 33.3 0 0 0.231 0.03 0.03 0.249 81.8 -28\nThomas, Lane 10 9 0 0 0 0 4 40 23.1 1 10 0 0 0.07 0.131 0.156 0.183 96.5 10.8\nRosario, Eddie 10 10 4 0 0 1 4 40 43.5 0 0 0.4 0.7 0.465 0.348 0.695 0.439 89.8 7.2\nAbrams, CJ 8 8 3 0 0 0 1 12.5 7.7 0 0 0.375 0.375 0.332 0.283 0.332 0.271 78 22.4\nRuiz, Keibert 10 9 2 0 0 0 0 0 6.7 1 10 0.222 0.222 0.246 0.199 0.295 0.263 81.5 11.7\nSenzel, Nick 9 8 2 0 0 0 1 11.1 15.4 1 11.1 0.25 0.25 0.271 0.223 0.388 0.319 90.1 7.2\nGarcía Jr., Luis 8 8 4 3 0 0 1 12.5 30.8 0 0 0.5 0.875 0.587 0.462 0.783 0.53 99.4 12.1\nYoung, Jacob 3 2 0 0 0 0 1 33.3 50 0 0 0 0 0 0.038 0.042 0.037 50.8 -27.5\nLipscomb, Trey 3 2 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.021 0.022 0.019 62.2 -24\nVargas, Ildemaro 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.208 0.226 0.193 67.2 -38.5'}, {'pitcher_id': '682990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPriester, Quinn 14 13 4 1 0 1 3 21.4 34.8 1 7.1 0.308 0.615 0.408 0.281 0.456 0.34 94.2 -3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 2 2 1 1 0 0 1 50 40 0 0 0.5 1 0.622 0.223 0.254 0.203 100.7 -2\nMillas, Drew 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.645 1.599 0.918 101.6 20.5\nAbrams, CJ 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.223 0.227 0.449 85.5 -4\nGarcía Jr., Luis 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.12 0.12 0.112 92.8 -18.5\nYoung, Jacob 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.046 0.06 0.043 96.7 -37\nRosario, Eddie 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.464 0.555 0.446 88.3 0.5\nMeneses, Joey 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.218 0.261 0.21 93.7 2'}, {'pitcher_id': '547973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Aroldis 11 11 1 0 0 0 3 27.3 33.3 0 0 0.091 0.091 0.08 0.124 0.14 0.113 83.3 22.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 5 5 0 0 0 0 2 40 45.5 0 0 0 0 0 0.091 0.106 0.083 69.9 56.3\nYoung, Jacob 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nThomas, Lane 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.118 0.106 94.1 24\nMeneses, Joey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.036 0.036 0.035 80.3 -32\nWinker, Jesse 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.463 0.508 0.417 98.6 3\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.177 0.234 0.162 89.3 -5'}, {'pitcher_id': '694973', 'player_data': '', 'team_data': 'Paul Skenes has not faced anyone on this team.'}, {'pitcher_id': '621366', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBorucki, Ryan 13 12 1 0 0 0 3 23.1 38.9 1 7.7 0.083 0.083 0.121 0.201 0.242 0.232 89.7 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 5 5 1 0 0 0 1 20 33.3 0 0 0.2 0.2 0.176 0.232 0.281 0.224 91.9 18.3\nGarcía Jr., Luis 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.149 0.295 0.439 93 26\nAbrams, CJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRuiz, Keibert 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.489 0.536 0.448 72.7 31\nMeneses, Joey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWinker, Jesse 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.313 0.313 0.282 103.2 -5\nMillas, Drew 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.187 0.234 0.174 94.5 -18\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.119 0.119 0.111 76.2 -44'}, {'pitcher_id': '683003', 'player_data': '', 'team_data': 'Jared Jones has not faced anyone on this team.'}, {'pitcher_id': '682847', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis L. 13 11 4 0 0 0 1 7.7 23.5 2 15.4 0.364 0.364 0.379 0.3 0.405 0.367 93.9 -1.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 4 2 1 0 0 0 0 0 40 2 50 0.5 0.5 0.568 0.528 0.679 0.611 90.4 -15.5\nSenzel, Nick 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.07 0.163 0.099 96.2 6.5\nThomas, Lane 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.111 0.26 0.163 90.8 27\nRosario, Eddie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.487 0.628 0.487 92.2 -2\nGarcía Jr., Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.583 0.676 0.563 100.1 8\nLipscomb, Trey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.259 0.297 0.243 97.7 -5\nMeneses, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.101 0.12 0.092 92.8 -19'}], 121: [{'pitcher_id': '669796', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernandez, Jose E. 8 7 2 1 0 1 2 25 23.5 1 12.5 0.286 0.857 0.496 0.474 1.112 0.66 98.2 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.309 0.396 0.302 100 10\nMcNeil, Jeff 2 1 1 1 0 0 0 0 33.3 1 50 1 2 0.983 0.931 1.028 0.779 77 22\nAlvarez, Francisco 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nStewart, DJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.474 1.805 0.904 101.8 34\nAlonso, Pete 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.983 3.762 1.909 112.2 23'}, {'pitcher_id': '669387', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMlodzinski, Carmen 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.233 0.194 70 -14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.126 0.13 0.119 49.4 -29\nMartinez, J.D. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.294 0.335 0.268 90.5 0'}, {'pitcher_id': '641726', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJefferies, Daulton 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.045 0.053 0.04 89.4 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIglesias, Jose 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.045 0.053 0.04 89.4 12.5'}, {'pitcher_id': '663559', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFalter, Bailey 66 58 13 0 0 3 12 18.2 17.3 5 7.6 0.224 0.379 0.301 0.24 0.41 0.318 85 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 11 8 2 0 0 0 1 9.1 25 3 27.3 0.25 0.25 0.348 0.196 0.233 0.322 83.7 37.4\nNimmo, Brandon 12 10 3 0 0 1 4 33.3 18.8 1 8.3 0.3 0.6 0.432 0.268 0.449 0.373 91.3 14.5\nMarte, Starling 9 8 2 0 0 1 2 22.2 17.4 0 0 0.25 0.625 0.41 0.315 0.724 0.465 87.3 28.3\nAlonso, Pete 10 8 1 0 0 1 1 10 12.5 1 10 0.125 0.5 0.276 0.211 0.637 0.352 91.9 26.8\nMcNeil, Jeff 8 8 1 0 0 0 2 25 12.5 0 0 0.125 0.125 0.11 0.214 0.27 0.214 79.6 32\nTaylor, Tyrone 6 6 2 0 0 0 1 16.7 18.2 0 0 0.333 0.333 0.295 0.412 0.475 0.389 72.2 12.8\nAlvarez, Francisco 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.443 0.215 0.231 0.195 86.7 -24.8\nVientos, Mark 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.073 0.081 0.067 78.4 -5\nBader, Harrison 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.057 0.104 0.071 83.1 28\nTorrens, Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.078 0.108 0.075 88.5 12'}, {'pitcher_id': '656924', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Ryder 6 6 0 0 0 0 3 50 46.2 0 0 0 0 0 0.015 0.027 0.019 84.9 42.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.033 0.056 0.041 78.1 36\nMarte, Starling 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.037 0.026 85.3 44\nNimmo, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.013 0.009 91.4 47'}, {'pitcher_id': '676702', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Hunter 10 7 4 1 0 1 1 10 21.4 1 10 0.571 1.143 0.728 0.42 0.922 0.595 99.5 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Tyrone 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.212 0.227 0.191 100.4 -2\nAlvarez, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.025 0.019 84.4 61\nAlonso, Pete 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nNimmo, Brandon 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.443 0.499 0.412 99.8 0\nBader, Harrison 2 1 1 0 0 1 0 0 0 0 0 1 4 1.405 0.914 3.583 1.272 106.3 29\nMcNeil, Jeff 1 0 0 0 0 0 0 0 0 0 0\nLindor, Francisco 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.489 0.536 0.453 99.3 3\nStewart, DJ 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.649 1.357 0.856 106.8 18'}, {'pitcher_id': '670059', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolderman, Colin 15 14 1 0 0 1 3 20 28.6 1 6.7 0.071 0.286 0.18 0.171 0.308 0.233 92.5 2.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.225 0.247 0.367 104.1 -2\nTaylor, Tyrone 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.251 0.274 0.23 100.6 -12\nAlvarez, Francisco 2 2 1 0 0 1 0 0 50 0 0 0.5 2 1.002 0.303 1.046 0.542 95.7 10\nMarte, Starling 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.038 0.132 0.07 96.7 41\nAlonso, Pete 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.056 0.058 0.051 54.3 -35\nMartinez, J.D. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.144 0.162 0.136 91.9 27\nMcNeil, Jeff 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.2 0.221 0.181 94.7 -10\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.166 0.247 0.154 91.5 -12'}, {'pitcher_id': '670280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBednar, David 29 27 7 1 0 0 8 27.6 21.2 2 6.9 0.259 0.296 0.274 0.286 0.435 0.337 88.5 11.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 5 3 1 0 0 0 1 20 27.3 2 40 0.333 0.333 0.453 0.094 0.094 0.33 65.8 -38\nTaylor, Tyrone 5 5 1 0 0 0 0 0 13.3 0 0 0.2 0.2 0.178 0.527 1.013 0.653 93.2 30.4\nBader, Harrison 6 6 0 0 0 0 4 66.7 25 0 0 0 0 0 0.094 0.184 0.118 94.9 12.5\nMcNeil, Jeff 5 5 2 1 0 0 1 20 13.3 0 0 0.4 0.6 0.43 0.243 0.363 0.256 88.9 13.8\nLindor, Francisco 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nAlvarez, Francisco 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.611 0.719 0.581 90.1 4.5\nNimmo, Brandon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nStewart, DJ 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.883 0.583 0.678 0.552 95.5 7\nIglesias, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.247 0.269 0.224 100.6 -14\nMarte, Starling 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.96 1.065 0.898 78.4 19\nMartinez, J.D. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.038 0.026 84.3 50'}, {'pitcher_id': '594835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzales, Marco 55 47 19 4 1 4 8 14.5 15.7 6 10.9 0.404 0.787 0.51 0.305 0.599 0.419 87.1 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 14 13 5 0 0 2 4 28.6 20 1 7.1 0.385 0.846 0.523 0.283 0.794 0.455 94.6 27\nIglesias, Jose 13 12 7 2 1 1 1 7.7 16.7 1 7.7 0.583 1.167 0.724 0.449 0.729 0.517 86 19.8\nLindor, Francisco 7 4 1 0 0 0 0 0 7.7 2 28.6 0.25 0.25 0.323 0.353 0.398 0.472 82.7 -12\nMarte, Starling 9 8 3 1 0 1 1 11.1 7.7 1 11.1 0.375 0.875 0.538 0.247 0.645 0.408 87.5 20.9\nMcNeil, Jeff 3 3 1 0 0 0 1 33.3 7.1 0 0 0.333 0.333 0.295 0.094 0.195 0.123 78.9 17\nAlonso, Pete 3 2 0 0 0 0 0 0 40 1 33.3 0 0 0.23 0.25 0.289 0.388 100.9 17.5\nNimmo, Brandon 4 3 2 1 0 0 1 25 28.6 0 0 0.667 1 0.715 0.177 0.191 0.163 70.8 6.3\nStewart, DJ 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.277 0.297 0.255 87 -0.5'}, {'pitcher_id': '527048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Martín 130 114 29 5 0 2 10 7.7 16.4 13 10 0.254 0.351 0.314 0.284 0.415 0.351 85.3 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 22 17 7 1 0 0 1 4.5 16.7 5 22.7 0.412 0.471 0.455 0.378 0.506 0.452 89.8 11.3\nIglesias, Jose 22 21 5 0 0 1 1 4.5 11.8 0 0 0.238 0.381 0.283 0.312 0.45 0.344 86.4 17\nMartinez, J.D. 14 11 1 0 0 0 2 14.3 19.4 3 21.4 0.091 0.091 0.211 0.339 0.408 0.427 85.1 2.4\nNimmo, Brandon 15 14 4 1 0 0 0 0 4 0 0 0.286 0.357 0.307 0.246 0.4 0.305 85.5 22.1\nMarte, Starling 11 11 6 2 0 1 1 9.1 8.7 0 0 0.545 1 0.658 0.398 0.733 0.479 86.5 2.7\nMcNeil, Jeff 13 13 4 1 0 0 0 0 18.8 0 0 0.308 0.385 0.298 0.273 0.37 0.282 79.8 10.8\nAlonso, Pete 14 12 0 0 0 0 2 14.3 18.8 2 14.3 0 0 0.099 0.185 0.28 0.27 85.5 13.3\nAlvarez, Francisco 4 2 0 0 0 0 0 0 33.3 2 50 0 0 0.346 0.235 0.316 0.466 83.5 12\nTaylor, Tyrone 4 3 2 0 0 0 0 0 25 1 25 0.667 0.667 0.617 0.352 0.427 0.425 80.9 31.7\nTorrens, Luis 5 5 0 0 0 0 2 40 37.5 0 0 0 0 0 0.011 0.011 0.011 83.3 22.7\nVientos, Mark 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.059 0.059 0.055 90.8 -14\nBader, Harrison 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.228 0.397 0.266 75.4 10.5\nStewart, DJ 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.042 0.063 0.384 82.9 -35'}, {'pitcher_id': '693312', 'player_data': '', 'team_data': 'Kyle Nicolas has not faced anyone on this team.'}, {'pitcher_id': '656605', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKeller, Mitch 73 62 7 0 0 2 18 24.7 24.4 8 11 0.113 0.21 0.203 0.169 0.226 0.236 83.8 5.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 12 7 2 0 0 0 2 16.7 22.7 5 41.7 0.286 0.286 0.437 0.25 0.283 0.428 81.1 6.2\nNimmo, Brandon 9 8 1 0 0 0 3 33.3 20 0 0 0.125 0.125 0.11 0.11 0.121 0.102 77.5 -22\nIglesias, Jose 10 10 3 0 0 1 3 30 31.6 0 0 0.3 0.6 0.369 0.191 0.212 0.178 79.9 6.2\nBader, Harrison 8 7 0 0 0 0 2 25 18.2 1 12.5 0 0 0.086 0.121 0.165 0.192 82 12.8\nMcNeil, Jeff 9 9 1 0 0 1 1 11.1 17.6 0 0 0.111 0.444 0.223 0.216 0.379 0.249 85.4 15\nMartinez, J.D. 5 3 0 0 0 0 1 20 44.4 2 40 0 0 0.275 0.264 0.286 0.42 100.5 -2.5\nAlonso, Pete 6 5 0 0 0 0 1 16.7 18.2 0 0 0 0 0 0.265 0.317 0.209 84.7 7.8\nMarte, Starling 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.24 0.168 0.185 0.341 94.4 -5\nAlvarez, Francisco 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.025 0.035 0.025 87.8 -0.5\nTaylor, Tyrone 4 4 0 0 0 0 1 25 16.7 0 0 0 0 0 0.095 0.264 0.149 86.3 22.3\nQuintana, Jose 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0\nVientos, Mark 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.05 0.079 0.051 86.5 25'}, {'pitcher_id': '682990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPriester, Quinn 18 16 5 2 0 0 3 16.7 16.7 1 5.6 0.313 0.438 0.324 0.272 0.364 0.29 92.6 4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.476 0.703 0.51 89.7 -6\nAlonso, Pete 3 2 1 1 0 0 0 0 25 1 33.3 0.5 1 0.647 0.162 0.191 0.331 88.1 -12.5\nVientos, Mark 3 3 0 0 0 0 2 66.7 14.3 0 0 0 0 0 0.033 0.033 0.03 96.7 -25\nAlvarez, Francisco 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.083 0.083 0.076 102.7 -23.5\nMcNeil, Jeff 3 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.294 0.482 0.572 0.346 84.7 34\nLindor, Francisco 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.429 0.612 0.448 98.4 23.3'}, {'pitcher_id': '547973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Aroldis 77 62 15 1 0 3 29 37.7 39.5 11 14.3 0.242 0.403 0.356 0.25 0.467 0.394 93.4 9.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 22 20 6 0 0 1 9 40.9 39 2 9.1 0.3 0.45 0.35 0.303 0.51 0.381 96.3 2\nMarte, Starling 17 15 5 0 0 0 9 52.9 55 1 5.9 0.333 0.333 0.343 0.086 0.09 0.216 77.4 3\nLindor, Francisco 11 11 1 0 0 0 4 36.4 20.8 0 0 0.091 0.091 0.08 0.167 0.191 0.157 89.3 9.9\nStewart, DJ 6 4 0 0 0 0 3 50 55.6 1 16.7 0 0 0.236 0.029 0.029 0.255 90.3 -14\nMcNeil, Jeff 5 2 0 0 0 0 1 20 36.4 3 60 0 0 0.418 0.02 0.038 0.427 85.6 31\nAlvarez, Francisco 3 2 0 0 0 0 1 33.3 16.7 1 33.3 0 0 0.232 0.303 0.721 0.515 103.1 21\nIglesias, Jose 3 1 0 0 0 0 1 33.3 66.7 2 66.7 0 0 0.459 0.459\nTorrens, Luis 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.346 0.731 0.862 0.692 104 8\nNimmo, Brandon 3 1 0 0 0 0 0 0 0 0 0 0 0 0.48 0.68 0.738 0.685 73.4 16\nAlonso, Pete 2 2 1 0 0 1 1 50 50 0 0 0.5 2 1.003 0.447 1.752 0.882 104.8 28\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.377 0.794 0.492 100 21\nVientos, Mark 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.957 3.8 1.912 108 29\nBader, Harrison 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.866 1.172 0.871 106.1 13'}, {'pitcher_id': '694973', 'player_data': '', 'team_data': 'Paul Skenes has not faced anyone on this team.'}, {'pitcher_id': '621366', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBorucki, Ryan 28 25 6 0 0 4 4 14.3 14.6 3 10.7 0.24 0.72 0.424 0.32 0.85 0.506 92.9 9.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 11 10 2 0 0 1 1 9.1 6.7 1 9.1 0.2 0.5 0.327 0.375 0.94 0.562 97.6 16.7\nIglesias, Jose 4 3 0 0 0 0 1 25 28.6 1 25 0 0 0.173 0.014 0.019 0.183 75.4 23.5\nAlonso, Pete 3 3 2 0 0 2 1 33.3 33.3 0 0 0.667 2.667 1.338 0.644 2.389 1.222 108.8 26\nLindor, Francisco 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.39 0.309 0.346 0.387 99.3 -11\nMcNeil, Jeff 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.118 0.119 0.108 68.2 -6\nNimmo, Brandon 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.134 0.177 0.136 83 -19\nStewart, DJ 1 1 1 0 0 1 0 0 33.3 0 0 1 4 2.004 0.714 2.814 1.422 103.9 33'}, {'pitcher_id': '683003', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Jared 13 13 1 1 0 0 7 53.8 42.4 0 0 0.077 0.154 0.098 0.134 0.182 0.136 83.5 1.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarez, Francisco 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nLindor, Francisco 2 2 0 0 0 0 1 50 57.1 0 0 0 0 0 0.059 0.07 0.056 84.3 -8\nAlonso, Pete 2 2 1 1 0 0 0 0 16.7 0 0 0.5 1 0.635 0.4 0.4 0.36 65.1 -12\nNimmo, Brandon 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nMarte, Starling 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.302 0.566 0.365 99 10\nBader, Harrison 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMcNeil, Jeff 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.11 0.149 0.104 88.9 23'}, {'pitcher_id': '682847', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis L. 10 9 2 0 0 0 2 20 25 1 10 0.222 0.222 0.246 0.239 0.413 0.314 81.8 18', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBader, Harrison 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.295 0.047 0.055 0.043 67.9 24.5\nMartinez, J.D. 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.345 1.032 0.556 102.7 23\nLindor, Francisco 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nAlvarez, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.194 0.246 0.19 73.8 42\nAlonso, Pete 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.643 0.698 0.579 109.9 5\nNimmo, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.321 0.355 0.293 82.5 4\nMarte, Starling 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.191 0.144 68.1 3'}], 133: [{'pitcher_id': '669796', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernandez, Jose E. 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.034 0.07 0.044 85 33', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.077 0.175 0.11 89.1 29\nBrown, Seth 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDíaz, Aledmys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.035 0.023 80.8 37'}, {'pitcher_id': '669387', 'player_data': '', 'team_data': 'Carmen Mlodzinski has not faced anyone on this team.'}, {'pitcher_id': '641726', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663559', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFalter, Bailey 19 16 7 3 0 0 2 10.5 17.1 0 0 0.438 0.625 0.423 0.36 0.505 0.428 88.9 22.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 4 3 1 1 0 0 0 0 10 0 0 0.333 0.667 0.491 0.388 0.566 0.485 93.1 11.3\nToro, Abraham 2 2 1 1 0 0 1 50 20 0 0 0.5 1 0.635 0.332 0.503 0.354 103.2 15\nRooker, Brent 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.296 0.385 0.547 0.393 91.5 23\nBleday, JJ 4 4 2 1 0 0 0 0 20 0 0 0.5 0.75 0.538 0.293 0.498 0.337 94.7 37.8\nSchuemann, Max 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.302 0.303 0.48 81.2 -19.5\nLangeliers, Shea 2 1 0 0 0 0 0 0 25 0 0 0 0 0 0.001 0.001 0.386 90.4 44.5\nDíaz, Aledmys 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.614 0.727 0.612 72.9 30\nBrown, Seth 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.689 0.711 0.62 62.7 32'}, {'pitcher_id': '656924', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676702', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '670059', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolderman, Colin 5 4 1 0 0 0 1 20 14.3 1 20 0.25 0.25 0.316 0.24 0.264 0.319 79.8 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.065 0.065 0.271 89.4 -12\nDíaz, Aledmys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.035 0.025 86.4 53\nRooker, Brent 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.811 0.892 0.757 63.6 31'}, {'pitcher_id': '670280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBednar, David 6 4 1 0 0 0 2 33.3 46.2 2 33.3 0.25 0.25 0.378 0.237 0.268 0.377 90.5 34.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nToro, Abraham 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nBleday, JJ 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nMcCann, Kyle 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.946 1.067 0.878 83.1 18\nBrown, Seth 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 97.9 51'}, {'pitcher_id': '594835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzales, Marco 38 34 4 1 0 1 4 10.5 24.1 3 7.9 0.118 0.235 0.188 0.269 0.478 0.349 88.2 19.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 15 14 2 1 0 0 0 0 16.7 1 6.7 0.143 0.214 0.189 0.272 0.467 0.341 88.9 25.7\nAndujar, Miguel 6 6 2 0 0 1 0 0 11.1 0 0 0.333 0.833 0.485 0.393 0.927 0.549 92 11.5\nBrown, Seth 6 5 0 0 0 0 0 0 0 1 16.7 0 0 0.115 0.248 0.324 0.321 79.1 9.2\nLangeliers, Shea 5 4 0 0 0 0 1 20 22.2 0 0 0 0 0 0.252 0.286 0.265 83.3 16.3\nRooker, Brent 3 2 0 0 0 0 2 66.7 100 1 33.3 0 0 0.232 0.232\nDavis, J.D. 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.243 0.463 0.298 105.1 31.5'}, {'pitcher_id': '527048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Martín 55 51 11 2 0 1 12 21.8 22.4 4 7.3 0.216 0.314 0.261 0.248 0.48 0.335 91.3 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 17 16 5 1 0 1 3 17.6 21.4 1 5.9 0.313 0.563 0.391 0.297 0.677 0.423 92.1 21.2\nDavis, J.D. 14 13 3 0 0 0 4 28.6 21.7 1 7.1 0.231 0.231 0.238 0.221 0.37 0.29 91.2 1\nAndujar, Miguel 7 7 1 0 0 0 1 14.3 15.4 0 0 0.143 0.143 0.126 0.261 0.432 0.292 90.2 24\nToro, Abraham 5 4 0 0 0 0 1 20 11.1 1 20 0 0 0.138 0.205 0.265 0.299 83.4 10.7\nLangeliers, Shea 4 4 1 0 0 0 1 25 28.6 0 0 0.25 0.25 0.221 0.254 0.279 0.233 89.8 -3\nRooker, Brent 5 5 1 1 0 0 1 20 18.2 0 0 0.2 0.4 0.249 0.25 0.651 0.363 97.1 14.3\nBrown, Seth 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.123 0.359 0.443 95 33\nGelof, Zack 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '693312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNicolas, Kyle 3 1 0 0 0 0 0 0 25 2 66.7 0 0 0.461 0.003 0.003 0.462 44.1 80', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLangeliers, Shea 1 0 0 0 0 0 0 0 20 1 100 0.692 0.692\nBleday, JJ 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nSchuemann, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 44.1 80'}, {'pitcher_id': '656605', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKeller, Mitch 35 30 10 2 1 2 3 8.6 11.7 5 14.3 0.333 0.667 0.46 0.306 0.62 0.432 87.5 1.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 8 6 2 0 0 1 1 12.5 10 2 25 0.333 0.833 0.545 0.35 0.837 0.546 83.9 -4.8\nBrown, Seth 6 5 1 1 0 0 1 16.7 7.1 1 16.7 0.2 0.4 0.323 0.216 0.611 0.397 94.1 20\nRooker, Brent 6 4 0 0 0 0 0 0 18.2 2 33.3 0 0 0.231 0.231 0.248 0.371 89.4 13\nLangeliers, Shea 5 5 2 1 0 1 0 0 0 0 0 0.4 1.2 0.655 0.415 1.003 0.588 84.4 1.6\nToro, Abraham 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.563 0.632 0.521 88.7 10.5\nDavis, J.D. 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.087 0.093 0.079 68.4 -37\nDíaz, Aledmys 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.327 0.371 0.302 92.8 -8.7\nCameron, Daz 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.801 0.308 0.934 0.513 97 4.5'}, {'pitcher_id': '682990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPriester, Quinn 15 12 3 0 0 1 2 13.3 27.3 3 20 0.25 0.5 0.396 0.322 0.568 0.441 92.9 1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nToro, Abraham 3 3 2 0 0 1 0 0 16.7 0 0 0.667 1.667 0.991 0.601 1.125 0.735 98.8 9.7\nLangeliers, Shea 3 2 0 0 0 0 2 66.7 42.9 1 33.3 0 0 0.231 0.231\nBleday, JJ 3 1 0 0 0 0 0 0 0 2 66.7 0 0 0.461 0.131 0.141 0.501 100.2 -23\nMcCann, Kyle 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.296 0.393 0.807 0.503 96.1 6.3\nBrown, Seth 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.251 0.293 0.236 81.3 -3'}, {'pitcher_id': '547973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Aroldis 14 10 3 1 0 1 5 35.7 41.2 3 21.4 0.3 0.7 0.494 0.259 0.582 0.451 86.4 22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 4 2 0 0 0 0 0 0 25 2 50 0 0 0.347 0.045 0.071 0.373 86.3 20.5\nDavis, J.D. 5 4 3 1 0 1 1 20 37.5 0 0 0.75 1.75 0.966 0.626 1.419 0.824 86.5 23\nGelof, Zack 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nLangeliers, Shea 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBleday, JJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRooker, Brent 1 1 0 0 0 0 1 100 0 0 0 0 0 0'}, {'pitcher_id': '694973', 'player_data': '', 'team_data': 'Paul Skenes has not faced anyone on this team.'}, {'pitcher_id': '621366', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBorucki, Ryan 8 8 3 1 0 0 1 12.5 30.8 0 0 0.375 0.5 0.375 0.373 0.574 0.411 89.7 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAndujar, Miguel 4 4 1 0 0 0 1 25 28.6 0 0 0.25 0.25 0.22 0.212 0.326 0.241 97.2 5.3\nBrown, Seth 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.714 0.808 0.67 74.9 26\nRooker, Brent 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.621 0.236 0.685 0.374 85.7 20\nDavis, J.D. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.954 1.108 0.903 90.4 14'}, {'pitcher_id': '683003', 'player_data': '', 'team_data': 'Jared Jones has not faced anyone on this team.'}, {'pitcher_id': '682847', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis L. 12 11 5 3 0 0 1 8.3 21.1 1 8.3 0.455 0.727 0.518 0.3 0.406 0.332 92.4 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRooker, Brent 3 3 1 1 0 0 0 0 16.7 0 0 0.333 0.667 0.415 0.322 0.485 0.339 98.7 -6\nLangeliers, Shea 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.138 0.158 0.125 81.6 -1.5\nBleday, JJ 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.25 0.276 0.228 83.7 32\nBrown, Seth 3 2 2 1 0 0 0 0 0 1 33.3 1 1.5 0.941 0.653 0.867 0.661 102.3 5\nToro, Abraham 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.117 0.253 0.152 92.9 27'}], 135: [{'pitcher_id': '669796', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernandez, Jose E. 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.336 0.383 0.318 72.5 1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.48 0.547 0.455 82.5 16\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.054 0.044 62.5 -14'}, {'pitcher_id': '669387', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMlodzinski, Carmen 6 5 0 0 0 0 2 33.3 11.1 1 16.7 0 0 0.116 0.031 0.04 0.141 84.8 26.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronenworth, Jake 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.029 0.04 0.029 77.4 51\nKim, Ha-Seong 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nMachado, Manny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.117 0.091 90.3 -20\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 86.7 48'}, {'pitcher_id': '641726', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJefferies, Daulton 18 18 11 4 0 2 0 0 3.2 0 0 0.611 1.167 0.76 0.291 0.507 0.335 88.6 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTatis Jr., Fernando 2 2 1 1 0 0 0 0 14.3 0 0 0.5 1 0.635 0.335 0.613 0.391 98.9 24.5\nWade, Tyler 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.497 0.627 0.49 91.8 20\nMachado, Manny 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.293 0.496 0.324 101.6 -3.5\nCampusano, Luis 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.092 0.229 0.134 81.3 1.5\nArraez, Luis 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.715 0.21 0.388 0.249 88.7 23.3\nProfar, Jurickson 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.385 0.453 0.363 72.3 -28\nCronenworth, Jake 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.357 0.692 0.442 91.1 0.5\nKim, Ha-Seong 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.487 0.246 0.764 0.415 90.5 -7.5\nMerrill, Jackson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.197 0.22 0.177 74.4 5'}, {'pitcher_id': '663559', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFalter, Bailey 8 6 3 1 0 1 1 12.5 18.8 2 25 0.5 1.167 0.7 0.238 0.466 0.395 90.7 33.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronenworth, Jake 2 1 1 0 0 0 0 0 33.3 1 50 1 1 0.784 0.749 0.877 0.697 102.7 9\nArraez, Luis 3 2 1 1 0 0 0 0 0 1 33.3 0.5 1 0.654 0.064 0.107 0.277 88.5 35\nProfar, Jurickson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.011 0.01 73.7 62\nSolano, Donovan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMachado, Manny 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.54 1.69 0.926 100.1 27'}, {'pitcher_id': '656924', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676702', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStratton, Hunter 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.121 0.899 97.9 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.121 0.899 97.9 13'}, {'pitcher_id': '670059', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolderman, Colin 9 6 2 0 0 0 0 0 11.1 3 33.3 0.333 0.333 0.426 0.337 0.83 0.553 90.7 21.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSolano, Donovan 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.506 0.602 0.591 96.8 3\nProfar, Jurickson 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nCronenworth, Jake 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nKim, Ha-Seong 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 83.4 82\nMachado, Manny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.429 0.475 0.4 103.9 -1\nCampusano, Luis 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.003 0.003 68.9 66\nTatis Jr., Fernando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.949 3.762 1.888 107.3 28\nPeralta, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.136 0.126 83.8 -49'}, {'pitcher_id': '670280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBednar, David 28 26 6 2 0 1 6 21.4 37.7 1 3.6 0.231 0.423 0.308 0.207 0.35 0.272 84 -0.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 4 4 1 0 0 0 0 0 23.1 0 0 0.25 0.25 0.22 0.201 0.228 0.19 88.6 -12.8\nMachado, Manny 5 5 1 0 0 1 2 40 50 0 0 0.2 0.8 0.414 0.288 0.816 0.462 90.1 -8.3\nSolano, Donovan 4 4 1 1 0 0 2 50 37.5 0 0 0.25 0.5 0.31 0.127 0.248 0.153 87.3 -6.5\nTatis Jr., Fernando 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.189 0.202 0.171 80.1 35.5\nPeralta, David 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.244 0.29 0.234 77.5 13.7\nCampusano, Luis 2 1 1 0 0 0 0 0 66.7 1 50 1 1 0.787 0.48 0.53 0.563 103.3 1\nCronenworth, Jake 3 2 1 1 0 0 0 0 0 0 0 0.5 1 0.656 0.165 0.253 0.36 86.8 -8.5\nKim, Ha-Seong 2 2 0 0 0 0 0 0 66.7 0 0 0 0 0 0.097 0.117 0.093 70.2 13.5\nArraez, Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.086 0.091 0.079 70.2 -37'}, {'pitcher_id': '594835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzales, Marco 49 46 12 6 0 0 9 18.4 21.4 3 6.1 0.261 0.391 0.3 0.217 0.26 0.24 83.3 19.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 22 20 3 2 0 0 6 27.3 20 2 9.1 0.15 0.25 0.213 0.131 0.181 0.189 85.7 19.6\nMachado, Manny 8 8 4 2 0 0 0 0 28.6 0 0 0.5 0.75 0.529 0.331 0.372 0.308 87.8 23.3\nArraez, Luis 5 4 0 0 0 0 1 20 25 1 20 0 0 0.138 0.285 0.301 0.344 83.4 4.3\nSolano, Donovan 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.707 0.268 0.315 0.254 84.8 7\nPeralta, David 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.53 0.349 0.379 0.319 72.4 38.3\nTatis Jr., Fernando 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.054 0.06 0.06 73.6 22\nCronenworth, Jake 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.557 0.726 0.549 83.4 28\nHigashioka, Kyle 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.024 0.025 0.027 65.6 -16'}, {'pitcher_id': '527048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPérez, Martín 57 49 19 1 0 2 2 3.5 9.6 7 12.3 0.388 0.531 0.44 0.314 0.418 0.371 88.5 3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMachado, Manny 25 20 7 1 0 1 1 4 12.9 5 20 0.35 0.55 0.448 0.35 0.484 0.431 89.3 3.6\nArraez, Luis 9 8 1 0 0 1 0 0 0 1 11.1 0.125 0.5 0.307 0.158 0.249 0.235 91.7 2.5\nCampusano, Luis 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.588 0.292 0.314 0.269 79.9 16.5\nHigashioka, Kyle 5 5 3 0 0 0 0 0 37.5 0 0 0.6 0.6 0.53 0.373 0.45 0.36 92.9 11\nWade, Tyler 4 2 1 0 0 0 0 0 0 1 25 0.5 0.5 0.574 0.172 0.196 0.434 79.3 -36\nTatis Jr., Fernando 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.252 0.552 0.337 91.3 -1\nKim, Ha-Seong 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.563 0.62 0.517 86.3 9.3\nSolano, Donovan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.353 0.395 0.324 86.8 -4\nCronenworth, Jake 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.4 0.454 0.381 77.9 8.5'}, {'pitcher_id': '693312', 'player_data': '', 'team_data': 'Kyle Nicolas has not faced anyone on this team.'}, {'pitcher_id': '656605', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKeller, Mitch 66 59 13 3 0 1 16 24.2 15 6 9.1 0.22 0.322 0.27 0.251 0.356 0.298 89.9 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 13 11 2 1 0 0 3 23.1 16.7 2 15.4 0.182 0.273 0.27 0.247 0.37 0.331 90 14.9\nMachado, Manny 11 9 3 0 0 1 4 36.4 15.8 2 18.2 0.333 0.667 0.469 0.243 0.419 0.356 98.8 3.6\nCronenworth, Jake 11 9 2 1 0 0 1 9.1 6.7 1 9.1 0.222 0.333 0.256 0.275 0.402 0.306 87.1 8.3\nKim, Ha-Seong 9 9 0 0 0 0 2 22.2 25 0 0 0 0 0 0.098 0.125 0.095 84.5 26.7\nArraez, Luis 6 6 1 0 0 0 1 16.7 7.7 0 0 0.167 0.167 0.147 0.246 0.304 0.24 85.1 27.8\nPeralta, David 6 6 3 1 0 0 1 16.7 9.1 0 0 0.5 0.667 0.501 0.42 0.492 0.403 92.8 1.6\nSolano, Donovan 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.294 0.362 0.608 0.414 100 24.5\nHigashioka, Kyle 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nTatis Jr., Fernando 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.294 0.233 0.265 0.217 86.9 4.5\nCampusano, Luis 2 1 0 0 0 0 0 0 25 1 50 0 0 0.346 0.751 1.047 0.727 104.4 14'}, {'pitcher_id': '682990', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPriester, Quinn 13 13 3 0 0 2 1 7.7 4.8 0 0 0.231 0.692 0.376 0.187 0.318 0.213 88.8 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKim, Ha-Seong 3 3 2 0 0 2 0 0 0 0 0 0.667 2.667 1.336 0.314 0.827 0.465 97.6 6.7\nMachado, Manny 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.294 0.07 0.088 0.065 106 -27\nTatis Jr., Fernando 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.105 0.116 0.1 78.3 -14.3\nCronenworth, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.005 0.005 0.005 76.2 71.5\nCampusano, Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.475 0.515 0.435 87 8'}, {'pitcher_id': '547973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Aroldis 21 19 4 3 0 0 6 28.6 28.9 2 9.5 0.211 0.368 0.283 0.322 0.523 0.391 88.8 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nProfar, Jurickson 5 4 0 0 0 0 2 40 40 1 20 0 0 0.138 0.023 0.025 0.188 67.3 32\nSolano, Donovan 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.177 0.463 0.87 0.562 94.9 18.5\nArraez, Luis 4 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.31 0.327 0.451 0.332 83.7 8.5\nKim, Ha-Seong 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nMachado, Manny 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.608 0.542 0.885 0.59 104.7 6.5\nPeralta, David 2 2 1 1 0 0 1 50 60 0 0 0.5 1 0.608 0.693 0.901 0.664 96.2 16\nCronenworth, Jake 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTatis Jr., Fernando 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '694973', 'player_data': '', 'team_data': 'Paul Skenes has not faced anyone on this team.'}, {'pitcher_id': '621366', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBorucki, Ryan 16 15 3 1 0 1 4 25 29.7 1 6.3 0.2 0.467 0.301 0.19 0.393 0.271 89.1 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.22 0.106 0.166 0.113 89 21.7\nWade, Tyler 4 3 0 0 0 0 1 25 44.4 1 25 0 0 0.172 0.159 0.185 0.287 76 54.5\nHigashioka, Kyle 2 2 1 0 0 1 0 0 16.7 0 0 0.5 2 1.003 0.422 1.675 0.849 96.8 38\nCronenworth, Jake 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.14 0.2 0.131 88.4 -10\nKim, Ha-Seong 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.18 0.145 96.1 -17\nSolano, Donovan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nPeralta, David 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.032 0.032 0.03 84.9 -15\nCampusano, Luis 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.751 0.879 0.705 98.6 10'}, {'pitcher_id': '683003', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJones, Jared 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.312 0.625 0.388 93.9 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArraez, Luis 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.312 0.625 0.388 93.9 15.3'}, {'pitcher_id': '682847', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis L. 22 20 6 0 0 2 2 9.1 20.6 1 4.5 0.3 0.6 0.374 0.44 0.878 0.541 95.9 10.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTatis Jr., Fernando 3 3 0 0 0 0 0 0 14.3 0 0 0 0 0 0.462 0.527 0.436 107.7 -20.7\nSolano, Donovan 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.025 0.025 0.023 86.8 -24\nArraez, Luis 4 4 3 0 0 0 0 0 20 0 0 0.75 0.75 0.662 0.663 0.794 0.635 90.7 17.3\nCronenworth, Jake 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.641 0.833 0.663 83.7 23.5\nKim, Ha-Seong 2 1 1 0 0 1 0 0 0 0 0 1 4 1.002 0.9 3.52 0.939 98.1 29\nMachado, Manny 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.243 0.853 0.447 97.5 18\nPeralta, David 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.378 0.968 0.563 100 15.5\nProfar, Jurickson 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.523 1.348 0.769 102.5 16'}]}, 135: {120: [{'pitcher_id': '673513', 'player_data': '', 'team_data': 'Yuki Matsui has not faced anyone on this team.'}, {'pitcher_id': '656302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCease, Dylan 12 10 2 0 0 0 6 50 34.5 2 16.7 0.2 0.2 0.261 0.097 0.111 0.192 91.9 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 5 3 0 0 0 0 3 60 33.3 2 40 0 0 0.278 0.278\nRosario, Eddie 7 7 2 0 0 0 3 42.9 35 0 0 0.286 0.286 0.248 0.138 0.159 0.13 91.9 10'}, {'pitcher_id': '605397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMusgrove, Joe 61 56 11 2 0 1 12 19.7 19.7 2 3.3 0.196 0.286 0.24 0.236 0.353 0.283 84.5 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 15 13 2 0 0 0 3 20 18.2 1 6.7 0.154 0.154 0.211 0.25 0.31 0.303 81.3 -6.5\nRosario, Eddie 9 8 1 0 0 0 4 44.4 29.6 1 11.1 0.125 0.125 0.175 0.044 0.061 0.118 80.9 20.8\nAbrams, CJ 6 6 2 1 0 0 0 0 12.5 0 0 0.333 0.5 0.355 0.506 0.595 0.48 86.7 12.5\nRuiz, Keibert 6 6 0 0 0 0 2 33.3 25 0 0 0 0 0 0.165 0.209 0.162 84.9 17.3\nThomas, Lane 5 5 1 0 0 1 2 40 33.3 0 0 0.2 0.8 0.414 0.18 0.653 0.347 89.2 21.7\nMeneses, Joey 6 6 2 0 0 0 1 16.7 12.5 0 0 0.333 0.333 0.294 0.313 0.399 0.303 93.8 0.4\nSenzel, Nick 5 4 2 1 0 0 0 0 14.3 0 0 0.5 0.75 0.525 0.314 0.341 0.29 72.2 18.5\nGarcía Jr., Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.252 0.808 0.434 92 -0.3\nCorbin, Patrick 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.22 0.166 0.185 0.15 84.4 -26\nVargas, Ildemaro 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.046 0.046 0.384 84.5 -31'}, {'pitcher_id': '669093', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Jeremiah 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.343 0.343 0.504 107.6 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSenzel, Nick 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMeneses, Joey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.343 0.343 0.312 107.6 -9'}, {'pitcher_id': '506433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDarvish, Yu 73 70 14 1 0 4 17 23.3 20.3 3 4.1 0.2 0.386 0.265 0.265 0.475 0.33 87 9.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinker, Jesse 18 15 0 0 0 0 4 22.2 14.7 3 16.7 0 0 0.116 0.236 0.304 0.314 87.8 0.6\nSenzel, Nick 12 12 4 1 0 0 4 33.3 28.6 0 0 0.333 0.417 0.319 0.309 0.38 0.293 84.5 6.9\nRuiz, Keibert 9 9 0 0 0 0 6 66.7 41.7 0 0 0 0 0 0.066 0.199 0.107 79.6 4.3\nMeneses, Joey 10 10 2 0 0 1 1 10 9.5 0 0 0.2 0.5 0.296 0.263 0.578 0.359 84.4 16.7\nThomas, Lane 6 6 3 0 0 1 0 0 0 0 0 0.5 1 0.628 0.355 0.618 0.415 86 16.2\nAbrams, CJ 5 5 1 0 0 1 2 40 38.5 0 0 0.2 0.8 0.401 0.215 0.837 0.422 96.9 33.3\nVargas, Ildemaro 5 5 2 0 0 1 0 0 0 0 0 0.4 1 0.591 0.477 0.95 0.61 88.1 19.4\nRosario, Eddie 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.281 0.45 0.313 88.3 -2.8\nGarcía Jr., Luis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.36 0.557 0.383 94.3 -2'}, {'pitcher_id': '660853', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe Los Santos, Enyel 7 7 0 0 0 0 1 14.3 21.4 0 0 0 0 0 0.168 0.192 0.158 78.7 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.623 0.723 0.598 76.2 26\nMeneses, Joey 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.133 0.133 0.123 59.5 -43\nGarcía Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.166 0.13 79.1 39\nWinker, Jesse 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.03 0.02 85.2 47\nCorbin, Patrick 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nRuiz, Keibert 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.125 0.104 93.3 -14'}, {'pitcher_id': '650633', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, Michael 10 9 0 0 0 0 2 20 20 1 10 0 0 0.07 0.208 0.433 0.312 88.9 -5.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 2 1 0 0 0 0 0 0 25 1 50 0 0 0.348 0.249 0.253 0.46 97 -15\nAbrams, CJ 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.3 1.105 0.564 101.2 30\nMeneses, Joey 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.166 0.185 0.15 103.2 -6\nSenzel, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.102 0.095 74.6 -29\nRosario, Eddie 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.057 0.059 0.056 72.3 -10\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.798 0.521 99.7 18'}, {'pitcher_id': '663158', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, Robert 9 9 2 1 0 1 3 33.3 22.2 0 0 0.222 0.667 0.37 0.163 0.245 0.174 86.3 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.05 0.058 0.047 84.1 -11\nMeneses, Joey 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.165 0.188 0.154 76.5 9\nVargas, Ildemaro 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.231 0.308 0.226 86.9 31\nSenzel, Nick 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.217 0.707 0.384 96.2 30\nRuiz, Keibert 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.312 0.329 0.283 87.2 4'}, {'pitcher_id': '666745', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrito, Jhony 6 6 0 0 0 0 2 33.3 14.3 0 0 0 0 0 0.098 0.161 0.108 89.6 42', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSenzel, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.063 0.231 0.117 100.4 40\nVargas, Ildemaro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 70.1 80\nThomas, Lane 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nAbrams, CJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.476 0.397 105.2 -2\nMeneses, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.029 0.019 82.7 50'}, {'pitcher_id': '670810', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '681190', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVásquez, Randy 2 1 1 0 0 1 0 0 0 1 50 1 4 1.35 0.877 3.401 1.206 104.2 28', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 2 1 1 0 0 1 0 0 0 1 50 1 4 1.35 0.877 3.401 1.206 104.2 28'}, {'pitcher_id': '663568', 'player_data': '', 'team_data': 'Stephen Kolek has not faced anyone on this team.'}, {'pitcher_id': '676680', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCosgrove, Tom 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.471 0.513 0.428 88.2 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSenzel, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.292 0.318 0.268 77.7 7\nRuiz, Keibert 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.649 0.708 0.589 98.7 8'}, {'pitcher_id': '663362', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaldron, Matt 11 10 2 0 0 1 2 18.2 27.8 1 9.1 0.2 0.5 0.326 0.233 0.436 0.318 93 16.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 3 3 2 0 0 1 1 33.3 20 0 0 0.667 1.667 0.962 0.448 1.04 0.615 103.2 19.5\nGarcía Jr., Luis 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.071 0.092 0.381 71 -1\nRuiz, Keibert 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.198 0.247 0.191 93.3 12.5\nMeneses, Joey 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.003 0.005 0.003 95.3 46\nAbrams, CJ 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.256 0.324 0.255 92.4 12'}, {'pitcher_id': '593974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Wandy 6 6 1 0 0 1 3 50 40 0 0 0.167 0.667 0.345 0.149 0.39 0.228 86.5 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Eddie 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nWinker, Jesse 2 2 1 0 0 1 0 0 50 0 0 0.5 2 1.036 0.364 1.085 0.603 100.8 1\nMeneses, Joey 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nSenzel, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.172 0.162 57.9 5'}, {'pitcher_id': '670970', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorejon, Adrian 11 11 3 0 0 0 0 0 6.7 0 0 0.273 0.273 0.241 0.389 0.638 0.441 93.9 18.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThomas, Lane 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.307 0.369 0.297 89.6 36\nMeneses, Joey 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.471 0.836 0.56 95.5 18.7\nRuiz, Keibert 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.383 0.517 0.393 95.5 17.3\nVargas, Ildemaro 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.277 0.297 0.252 90.8 -1.5\nGarcía Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.546 1.622 0.9 99.6 25'}], 121: [{'pitcher_id': '673513', 'player_data': '', 'team_data': 'Yuki Matsui has not faced anyone on this team.'}, {'pitcher_id': '656302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCease, Dylan 34 31 7 0 0 2 9 26.5 27.9 3 8.8 0.226 0.419 0.306 0.182 0.326 0.259 83.1 19.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 8 7 3 0 0 0 2 25 20 1 12.5 0.429 0.429 0.417 0.304 0.341 0.339 79.2 6.8\nLindor, Francisco 7 7 2 0 0 2 1 14.3 21.4 0 0 0.286 1.143 0.56 0.269 0.734 0.41 88.9 18.8\nMarte, Starling 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.023 0.045 0.25 87 31\nAlonso, Pete 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.096 0.109 0.086 79.1 6\nStewart, DJ 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.024 0.028 0.031 78.9 20.5\nTaylor, Tyrone 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.091 0.336 0.175 94.6 53\nIglesias, Jose 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.44 0.355 0.378 0.32 67.4 20\nBader, Harrison 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.037 0.043 0.364 79 44\nMcNeil, Jeff 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.29 0.071 0.092 0.076 80.1 11.7'}, {'pitcher_id': '605397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMusgrove, Joe 118 110 25 5 0 6 31 26.3 26.6 4 3.4 0.227 0.436 0.308 0.275 0.49 0.35 87.6 8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 24 23 7 2 0 2 11 45.8 38.5 1 4.2 0.304 0.652 0.411 0.264 0.534 0.35 90.4 13.9\nMcNeil, Jeff 14 12 2 1 0 0 2 14.3 6.9 0 0 0.167 0.25 0.253 0.316 0.387 0.363 87.9 8.7\nBader, Harrison 12 11 1 0 0 1 6 50 40 1 8.3 0.091 0.364 0.227 0.128 0.359 0.241 91.6 8.4\nIglesias, Jose 15 15 6 2 0 1 2 13.3 25 0 0 0.4 0.733 0.473 0.326 0.574 0.378 78.3 7.4\nNimmo, Brandon 13 12 0 0 0 0 1 7.7 17.4 0 0 0 0 0.055 0.216 0.333 0.277 85.6 -3.5\nAlonso, Pete 15 13 4 0 0 1 2 13.3 30.8 1 6.7 0.308 0.538 0.409 0.42 0.802 0.544 91.3 18.2\nMarte, Starling 12 11 3 0 0 0 1 8.3 14.3 1 8.3 0.273 0.273 0.278 0.311 0.39 0.34 90.2 10.1\nTaylor, Tyrone 3 3 0 0 0 0 1 33.3 10 0 0 0 0 0 0.067 0.102 0.07 87 -3.5\nMartinez, J.D. 3 3 1 0 0 1 1 33.3 14.3 0 0 0.333 1.333 0.668 0.458 1.364 0.742 101.1 13.5\nQuintana, Jose 3 3 0 0 0 0 2 66.7 75 0 0 0 0 0 0.011 0.011 0.01 64.6 -17\nStewart, DJ 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.443 0.562 0.441 96.9 14\nAlvarez, Francisco 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.059 0.069 0.055 96.9 -18'}, {'pitcher_id': '669093', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Jeremiah 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.045 0.047 0.045 75.8 -32.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.041 0.041 0.04 64.6 -29\nMarte, Starling 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.053 0.05 87 -36'}, {'pitcher_id': '506433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDarvish, Yu 160 143 30 4 1 5 39 24.4 27.9 7 4.4 0.21 0.357 0.278 0.216 0.335 0.282 87.1 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLindor, Francisco 27 22 6 1 0 1 7 25.9 32.1 2 7.4 0.273 0.455 0.355 0.244 0.347 0.335 91 15.4\nMarte, Starling 24 23 9 1 0 1 4 16.7 26.7 1 4.2 0.391 0.565 0.417 0.275 0.48 0.333 86 10.8\nAlonso, Pete 20 17 3 0 0 1 6 30 33.3 1 5 0.176 0.353 0.292 0.157 0.346 0.282 91.3 14\nMcNeil, Jeff 22 20 3 1 0 1 4 18.2 26.8 1 4.5 0.15 0.35 0.249 0.2 0.367 0.284 87.9 30.6\nMartinez, J.D. 14 13 2 0 0 1 7 50 31.4 1 7.1 0.154 0.385 0.253 0.171 0.338 0.26 90.3 32.8\nNimmo, Brandon 17 15 2 0 1 0 3 17.6 21.7 1 5.9 0.133 0.267 0.229 0.205 0.243 0.255 90.6 20.7\nBader, Harrison 13 12 0 0 0 0 4 30.8 32 0 0 0 0 0.055 0.066 0.079 0.116 80.8 35\nIglesias, Jose 11 11 4 1 0 0 0 0 8.7 0 0 0.364 0.455 0.349 0.338 0.39 0.318 80 9.1\nTaylor, Tyrone 5 3 0 0 0 0 1 20 25 0 0 0 0 0.145 0.429 0.469 0.402 92.9 19.3\nAlvarez, Francisco 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.1 0.128 0.091 97.3 -8\nTorrens, Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.332 0.366 0.307 68.9 -11.3\nHouser, Adrian 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0'}, {'pitcher_id': '660853', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe Los Santos, Enyel 17 16 4 2 0 1 3 17.6 23.3 1 5.9 0.25 0.563 0.363 0.238 0.499 0.331 90.1 7.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 4 4 2 1 0 1 0 0 8.3 0 0 0.5 1.5 0.832 0.402 1.082 0.61 95.5 13.5\nNimmo, Brandon 4 3 0 0 0 0 1 25 28.6 1 25 0 0 0.172 0.054 0.082 0.217 83 30\nStewart, DJ 1 1 0 0 0 0 1 100 42.9 0 0 0 0 0 0\nAlvarez, Francisco 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.102 0.108 0.093 91.8 -19\nAlonso, Pete 1 1 1 1 0 0 0 0 25 0 0 1 2 1.27 0.654 1.44 0.863 104.8 20\nVientos, Mark 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTaylor, Tyrone 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.075 0.082 0.074 69.3 -16\nMarte, Starling 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.823 1.259 0.885 111.5 13\nLindor, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.05 0.052 0.05 65.8 -20\nIglesias, Jose 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.237 0.365 0.254 88 23'}, {'pitcher_id': '650633', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, Michael 41 38 7 2 0 0 12 29.3 27.7 1 2.4 0.184 0.237 0.221 0.283 0.486 0.357 83 9.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 11 11 2 2 0 0 3 27.3 30.4 0 0 0.182 0.364 0.228 0.303 0.608 0.381 93.3 16.8\nMcNeil, Jeff 6 5 3 0 0 0 0 0 14.3 0 0 0.6 0.6 0.56 0.554 0.67 0.573 83.3 17.2\nStewart, DJ 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.233 0.005 0.013 0.238 84.7 45\nIglesias, Jose 5 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.321 0.181 0.2 0.284 76.1 -9.2\nNimmo, Brandon 4 4 0 0 0 0 3 75 50 0 0 0 0 0 0.033 0.033 0.035 43.9 -73\nAlonso, Pete 5 5 1 0 0 0 2 40 36.4 0 0 0.2 0.2 0.177 0.33 0.672 0.421 77 15.3\nLindor, Francisco 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.296 0.352 0.283 95.9 12\nTaylor, Tyrone 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.364 1.26 0.654 101.8 26\nAlvarez, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.494 0.494 0.445 61.9 37\nMegill, Tylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.035 0.035 0.045 72.3 -31'}, {'pitcher_id': '663158', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, Robert 6 5 0 0 0 0 1 16.7 20 1 16.7 0 0 0.115 0.257 0.356 0.333 79.6 39.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Tyrone 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nLindor, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.691 0.436 100.9 19\nMartinez, J.D. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.223 0.283 0.216 78.2 30\nMcNeil, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.674 0.798 0.651 87.3 19\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.007 0.006 51.8 89'}, {'pitcher_id': '666745', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrito, Jhony 9 9 2 0 0 1 2 22.2 7.7 0 0 0.222 0.556 0.33 0.195 0.523 0.298 85.7 32.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcNeil, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 69 66\nMartinez, J.D. 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.929 3.377 1.755 105.1 25\nStewart, DJ 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nAlonso, Pete 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.46 0.535 0.435 69.1 42\nNimmo, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.236 0.125 93.3 34\nAlvarez, Francisco 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.071 0.127 0.09 78.2 49\nLindor, Francisco 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.336 0.196 93 29\nVientos, Mark 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.088 0.091 0.08 91.9 -18'}, {'pitcher_id': '670810', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGillaspie, Logan 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.282 0.474 0.33 72.8 -15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.321 0.607 0.396 89.2 10.5\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.206 0.209 0.2 39.9 -68'}, {'pitcher_id': '681190', 'player_data': '', 'team_data': 'Randy Vásquez has not faced anyone on this team.'}, {'pitcher_id': '663568', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKolek, Stephen 6 6 3 0 0 0 1 16.7 7.7 0 0 0.5 0.5 0.444 0.304 0.328 0.278 76 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlonso, Pete 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMcNeil, Jeff 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.366 0.417 0.341 67.1 44\nMartinez, J.D. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.182 0.187 0.165 99.5 -14\nNimmo, Brandon 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.949 1.016 0.863 77 18\nBader, Harrison 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.242 0.251 0.218 62.3 -50\nLindor, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.094 0.08 74.3 -8'}, {'pitcher_id': '676680', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCosgrove, Tom 11 8 2 1 0 0 2 18.2 17.6 2 18.2 0.25 0.375 0.386 0.045 0.057 0.224 90.8 33.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMartinez, J.D. 5 3 0 0 0 0 2 40 30 2 40 0 0 0.278 0.009 0.02 0.285 99.7 47\nAlvarez, Francisco 2 1 1 0 0 0 0 0 0 0 0 1 1 0.804 0.221 0.24 0.464 102.4 -10\nStewart, DJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.019 0.013 95.4 52\nNimmo, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.018 0.011 91 46\nMcNeil, Jeff 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.048 0.059 0.045 78 34'}, {'pitcher_id': '663362', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaldron, Matt 26 22 3 1 0 0 2 7.7 13.6 2 7.7 0.136 0.182 0.226 0.233 0.387 0.332 84.3 10.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Tyrone 3 1 0 0 0 0 0 0 12.5 1 33.3 0 0 0.472 0.197 0.212 0.531 106.5 -23\nMarte, Starling 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.312 0.357 0.297 64.7 21.7\nNimmo, Brandon 3 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.538 0.569 1.081 0.704 103.1 14.5\nVientos, Mark 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.089 0.072 78.1 31.3\nMcNeil, Jeff 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.14 0.3 0.439 93.4 23\nMartinez, J.D. 3 3 1 1 0 0 0 0 16.7 0 0 0.333 0.667 0.423 0.362 0.436 0.34 84.5 18\nLindor, Francisco 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.231 0.808 0.423 89.8 0\nAlvarez, Francisco 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.084 0.088 0.077 88.5 -24.5\nAlonso, Pete 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.151 0.169 0.138 80.6 5.7'}, {'pitcher_id': '593974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Wandy 37 33 11 0 0 2 6 16.2 23 3 8.1 0.333 0.515 0.398 0.328 0.53 0.402 91.7 8.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Starling 8 8 3 0 0 1 2 25 38.9 0 0 0.375 0.75 0.471 0.489 0.942 0.601 100.7 7.3\nTorrens, Luis 4 4 1 0 0 0 1 25 22.2 0 0 0.25 0.25 0.22 0.185 0.206 0.173 79.6 -14\nNimmo, Brandon 7 6 2 0 0 0 1 14.3 0 1 14.3 0.333 0.333 0.351 0.22 0.244 0.273 87.8 19\nMartinez, J.D. 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.122 0.131 0.257 85.3 -1.7\nLindor, Francisco 4 4 2 0 0 1 0 0 14.3 0 0 0.5 1.25 0.722 0.547 1.129 0.705 101.9 14.8\nVientos, Mark 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMcNeil, Jeff 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0.363 0.363\nIglesias, Jose 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.88 0.417 0.441 0.381 94 0\nAlonso, Pete 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nTaylor, Tyrone 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.347 0.385 0.319 87.7 34\nAlvarez, Francisco 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.244 0.171 86.9 -4\nStewart, DJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.669 0.759 0.627 77.8 14\nBader, Harrison 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.184 0.211 0.166 83.8 -14'}, {'pitcher_id': '670970', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorejon, Adrian 19 16 4 1 0 0 4 21.1 20 3 15.8 0.25 0.313 0.315 0.296 0.392 0.361 83.3 -2.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNimmo, Brandon 5 4 1 0 0 0 2 40 20 1 20 0.25 0.25 0.317 0.241 0.273 0.321 73.2 -28.5\nAlonso, Pete 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.481 0.507 0.521 64.3 -25.5\nLindor, Francisco 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.289 0.352 0.281 84 12.3\nBader, Harrison 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.036 0.051 0.034 81.1 30\nMcNeil, Jeff 2 2 1 1 0 0 0 0 16.7 0 0 0.5 1 0.631 0.382 0.444 0.368 96.7 -6\nMartinez, J.D. 1 0 0 0 0 0 0 0 66.7 1 100 0.692 0.692\nTorrens, Luis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nAlvarez, Francisco 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.817 1.768 1.078 114.1 16'}], 133: [{'pitcher_id': '673513', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatsui, Yuki 8 7 1 0 0 0 2 25 28.6 1 12.5 0.143 0.143 0.198 0.117 0.14 0.184 90.7 45', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoderstrom, Tyler 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.071 0.109 0.379 88.3 27\nLangeliers, Shea 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGelof, Zack 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAndujar, Miguel 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.114 0.158 0.12 73.5 53\nBleday, JJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.626 0.702 0.588 107.6 5\nSchuemann, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 93.3 64\nBrown, Seth 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.009 0.008 90.7 76'}, {'pitcher_id': '656302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCease, Dylan 76 66 14 2 0 2 23 30.3 32.2 8 10.5 0.212 0.333 0.286 0.206 0.339 0.284 88.8 11.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Seth 15 13 1 0 0 0 6 40 34.3 2 13.3 0.077 0.077 0.151 0.108 0.135 0.184 79.5 3.3\nToro, Abraham 9 8 1 0 0 0 1 11.1 16.7 0 0 0.125 0.125 0.178 0.259 0.33 0.305 88.2 15.6\nCameron, Daz 7 6 0 0 0 0 4 57.1 50 1 14.3 0 0 0.099 0.107 0.299 0.241 92.4 42\nRooker, Brent 6 5 1 0 0 0 4 66.7 83.3 1 16.7 0.2 0.2 0.264 0.186 0.257 0.275 110.9 11\nAndujar, Miguel 5 5 2 1 0 0 2 40 40 0 0 0.4 0.6 0.432 0.146 0.184 0.142 83.3 4.7\nDíaz, Aledmys 5 3 0 0 0 0 0 0 0 2 40 0 0 0.276 0.151 0.163 0.359 79.7 -18\nGelof, Zack 6 6 3 1 0 0 1 16.7 22.2 0 0 0.5 0.667 0.504 0.084 0.099 0.08 75.9 7\nLangeliers, Shea 6 5 1 0 0 0 1 16.7 0 1 16.7 0.2 0.2 0.264 0.346 0.586 0.448 96.4 8.5\nSchuemann, Max 3 2 1 0 0 0 1 33.3 33.3 1 33.3 0.5 0.5 0.527 0.469 0.573 0.534 102.3 12\nBleday, JJ 6 5 1 0 0 0 2 33.3 11.1 0 0 0.2 0.2 0.147 0.192 0.229 0.163 95.4 20\nDavis, J.D. 5 5 2 0 0 1 1 20 36.4 0 0 0.4 1 0.575 0.461 0.808 0.527 102.7 15.3\nSoderstrom, Tyler 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.695 0.314 1.21 0.616 96.1 36'}, {'pitcher_id': '605397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMusgrove, Joe 19 17 4 0 0 2 3 15.8 19.2 0 0 0.235 0.588 0.379 0.228 0.54 0.36 82.5 24.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 9 8 3 0 0 1 2 22.2 23.1 0 0 0.375 0.75 0.49 0.263 0.721 0.44 78.5 35.8\nDíaz, Aledmys 5 4 0 0 0 0 0 0 14.3 0 0 0 0 0.144 0.324 0.587 0.445 89.6 25.5\nBleday, JJ 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.691 0.071 0.259 0.137 92.5 42.5\nRooker, Brent 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.133 0.143 0.122 70.8 -27.5'}, {'pitcher_id': '669093', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Jeremiah 7 5 1 1 0 0 3 42.9 60 2 28.6 0.2 0.4 0.38 0.065 0.107 0.247 75.4 51.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 2 1 0 0 0 0 1 50 60 1 50 0 0 0.348 0.348\nBrown, Seth 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nToro, Abraham 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.004 0.003 52.4 83\nCameron, Daz 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSchuemann, Max 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGelof, Zack 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.323 0.533 0.338 98.4 20'}, {'pitcher_id': '506433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDarvish, Yu 16 15 4 0 0 1 5 31.3 50 1 6.3 0.267 0.467 0.333 0.197 0.385 0.277 84.9 11.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 14 13 4 0 0 1 4 28.6 51.6 1 7.1 0.308 0.538 0.381 0.227 0.443 0.316 84.1 8.1\nDíaz, Aledmys 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.003 0.009 0.004 92.7 43'}, {'pitcher_id': '660853', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe Los Santos, Enyel 15 14 3 1 0 1 4 26.7 23.5 1 6.7 0.214 0.5 0.327 0.229 0.606 0.368 90.8 33', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCameron, Daz 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.003 0.004 0.004 63.5 71\nGelof, Zack 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.093 0.31 0.16 96.6 34\nAndujar, Miguel 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.545 0.597 0.506 106.9 -0.5\nLangeliers, Shea 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.1 0.351 0.436 99.8 41\nBleday, JJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.012 0.009 89.7 64\nRooker, Brent 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nDíaz, Aledmys 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.076 0.082 0.069 67.8 33\nSchuemann, Max 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSoderstrom, Tyler 1 1 1 0 0 1 0 0 50 0 0 1 4 2.085 0.94 3.737 1.896 106.9 30\nBrown, Seth 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.717 2.398 1.265 101.7 25'}, {'pitcher_id': '650633', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, Michael 25 23 7 1 0 0 12 48 33.9 2 8 0.304 0.348 0.319 0.157 0.188 0.194 78.2 10.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 3 2 1 0 0 0 1 33.3 50 1 33.3 0.5 0.5 0.527 0.255 0.285 0.388 84.1 10\nBrown, Seth 3 3 1 0 0 0 2 66.7 22.2 0 0 0.333 0.333 0.295 0.068 0.068 0.063 55.3 -50\nCameron, Daz 2 2 0 0 0 0 0 0 12.5 0 0 0 0 0 0.168 0.206 0.157 74 19.5\nSoderstrom, Tyler 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.346 0.346\nAndujar, Miguel 3 3 1 0 0 0 2 66.7 37.5 0 0 0.333 0.333 0.296 0.06 0.067 0.057 84.3 -5\nToro, Abraham 3 3 0 0 0 0 2 66.7 20 0 0 0 0 0 0.024 0.027 0.022 83.4 -18\nSchuemann, Max 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nDavis, J.D. 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.37 0.409 0.341 75.8 32.5\nLangeliers, Shea 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.488 0.533 0.449 77.5 19\nGelof, Zack 2 2 1 0 0 0 1 50 75 0 0 0.5 0.5 0.444 0.087 0.119 0.089 75.2 38\nDíaz, Aledmys 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.411 0.739 0.489 100.4 18'}, {'pitcher_id': '663158', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, Robert 20 19 2 0 0 0 8 40 26.9 1 5 0.105 0.105 0.123 0.113 0.142 0.141 83.1 21.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 3 3 0 0 0 0 3 100 25 0 0 0 0 0 0\nRooker, Brent 3 3 0 0 0 0 3 100 60 0 0 0 0 0 0\nGelof, Zack 2 1 0 0 0 0 0 0 20 1 50 0 0 0.348 0.058 0.062 0.376 67.2 -18\nToro, Abraham 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.26 0.292 0.243 68.2 41\nBrown, Seth 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.089 0.072 75.7 29.5\nAndujar, Miguel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.86 1.041 0.817 99.3 14\nDavis, J.D. 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.442 0.092 0.092 0.084 91 -4\nSoderstrom, Tyler 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.183 0.454 0.263 95.2 27\nDíaz, Aledmys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.124 0.126 0.114 102.1 -29\nLangeliers, Shea 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.032 0.032 0.029 72.6 51\nSchuemann, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.014 0.008 94.5 42'}, {'pitcher_id': '666745', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrito, Jhony 16 16 6 2 0 1 3 18.8 36.1 0 0 0.375 0.688 0.446 0.331 0.535 0.368 91.9 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRooker, Brent 5 5 2 1 0 0 1 20 33.3 0 0 0.4 0.6 0.425 0.321 0.396 0.311 98.4 11.8\nBleday, JJ 5 5 2 1 0 0 1 20 45.5 0 0 0.4 0.6 0.425 0.424 0.583 0.43 96.1 20.3\nDavis, J.D. 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.441 0.359 0.38 0.323 83.4 4.5\nLangeliers, Shea 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.002 0.002 0.002 85.3 73\nBrown, Seth 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.427 1.45 0.771 82.3 -22'}, {'pitcher_id': '670810', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGillaspie, Logan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.356 0.401 0.332 86 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRooker, Brent 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.663 0.752 0.614 108 6\nAndujar, Miguel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.049 0.049 63.9 -20'}, {'pitcher_id': '681190', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVásquez, Randy 22 19 7 1 0 1 5 22.7 17.5 2 9.1 0.368 0.579 0.437 0.382 0.61 0.45 91.2 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBleday, JJ 3 2 2 0 0 0 0 0 0 1 33.3 1 1 0.823 0.864 1.086 0.799 93.6 13.5\nSoderstrom, Tyler 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.444 0.247 0.265 0.225 106.3 1\nToro, Abraham 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.991 0.683 1.545 0.933 90.7 15.7\nAndujar, Miguel 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.08 0.092 0.08 82.9 33\nMcCann, Kyle 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.04 0.043 0.035 97.9 -25\nRooker, Brent 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.238 0.466 0.291 106.3 40\nSchuemann, Max 2 2 1 0 0 0 1 50 20 0 0 0.5 0.5 0.444 0.483 0.513 0.44 74.5 23\nBrown, Seth 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.92 1.235 0.813 105.9 11\nGelof, Zack 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.06 0.23 0.116 73.1 -10.5'}, {'pitcher_id': '663568', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKolek, Stephen 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.063 0.084 0.064 75 34', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGelof, Zack 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSchuemann, Max 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nToro, Abraham 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.252 0.191 75 34'}, {'pitcher_id': '676680', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCosgrove, Tom 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.489 0.582 0.47 85.7 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrown, Seth 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.923 1.106 0.885 96.3 12\nLangeliers, Shea 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.057 0.054 75.1 -20'}, {'pitcher_id': '663362', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaldron, Matt 13 13 3 0 0 1 3 23.1 24 0 0 0.231 0.462 0.29 0.286 0.487 0.33 81 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.205 0.244 0.194 76.8 13\nBrown, Seth 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.187 0.195 0.171 77.8 -26.5\nDíaz, Aledmys 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.294 0.333 0.356 0.305 76 35.5\nGelof, Zack 3 3 1 0 0 1 1 33.3 40 0 0 0.333 1.333 0.668 0.275 1.015 0.527 90.5 -1\nSoderstrom, Tyler 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.459 0.574 0.45 84 34'}, {'pitcher_id': '593974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Wandy 15 14 6 0 0 0 3 20 30.8 1 6.7 0.429 0.429 0.399 0.257 0.312 0.278 81.4 -4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Aledmys 5 5 0 0 0 0 2 40 45.5 0 0 0 0 0 0.045 0.049 0.043 80.5 -29\nCameron, Daz 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.006 0.008 0.349 73.8 66\nToro, Abraham 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.306 0.307 0.278 61 -3.5\nDavis, J.D. 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.435 0.426 0.625 0.443 111.3 12\nLangeliers, Shea 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.889 0.837 1.063 0.82 112.3 9\nBrown, Seth 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.33 0.331 0.297 35.1 -25\nSoderstrom, Tyler 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.13 0.146 0.117 100.1 -18\nRooker, Brent 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.597 0.712 0.576 99.8 5'}, {'pitcher_id': '670970', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorejon, Adrian 12 11 1 0 0 0 5 41.7 20 1 8.3 0.091 0.091 0.131 0.106 0.118 0.149 92.7 6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDavis, J.D. 4 4 1 0 0 0 2 50 25 0 0 0.25 0.25 0.221 0.169 0.178 0.155 100.1 -4\nBleday, JJ 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.183 0.196 0.165 100.2 -3\nRooker, Brent 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.005 0.013 0.009 89.5 46\nAndujar, Miguel 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nToro, Abraham 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.017 0.04 0.024 89.8 33\nLangeliers, Shea 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.084 0.085 0.08 76.2 -32'}], 134: [{'pitcher_id': '673513', 'player_data': '', 'team_data': 'Yuki Matsui has not faced anyone on this team.'}, {'pitcher_id': '656302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCease, Dylan 44 43 9 2 1 0 12 27.3 36 1 2.3 0.209 0.302 0.228 0.214 0.342 0.247 87.6 6.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 18 18 1 0 0 0 7 38.9 47.5 0 0 0.056 0.056 0.049 0.191 0.361 0.232 81.2 16.7\nOlivares, Edward 16 16 5 2 0 0 4 25 33.3 0 0 0.313 0.438 0.321 0.211 0.31 0.223 87.9 0.2\nReynolds, Bryan 5 5 1 0 0 0 0 0 13.3 0 0 0.2 0.2 0.176 0.263 0.295 0.251 91.9 5\nHayes, Ke'Bryan 5 4 2 0 1 0 1 20 33.3 1 20 0.5 1 0.626 0.272 0.441 0.376 102.6 -2.7"}, {'pitcher_id': '605397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMusgrove, Joe 70 65 19 7 0 1 19 27.1 30.5 3 4.3 0.292 0.446 0.33 0.249 0.393 0.295 87.9 9.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMcCutchen, Andrew 14 14 3 2 0 0 6 42.9 40 0 0 0.214 0.357 0.241 0.237 0.355 0.258 93.7 2.1\nJoe, Connor 9 9 1 1 0 0 3 33.3 15 0 0 0.111 0.222 0.14 0.243 0.275 0.227 85.2 9.8\nGrandal, Yasmani 11 11 5 3 0 1 3 27.3 30.4 0 0 0.455 1 0.587 0.285 0.825 0.441 93.6 25.5\nReynolds, Bryan 8 6 3 1 0 0 1 12.5 21.4 1 12.5 0.5 0.667 0.554 0.33 0.366 0.408 83.4 11.2\nSuwinski, Jack 9 8 3 0 0 0 2 22.2 38.5 1 11.1 0.375 0.375 0.372 0.228 0.295 0.282 77.8 20.3\nHayes, Ke'Bryan 6 6 3 0 0 0 1 16.7 44.4 0 0 0.5 0.5 0.442 0.187 0.22 0.179 95.8 -12.8\nGonzales, Nick 3 2 0 0 0 0 1 33.3 50 0 0 0 0 0 0.272 0.324 0.201 95.4 15\nTellez, Rowdy 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23 0.176 0.178 0.336 71.5 47.5\nTaylor, Michael A. 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.192 0.211 0.176 86.6 -19\nTriolo, Jared 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.247 0.4 0.278 73.8 -20\nDelay, Jason 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.316 0.348 0.291 102.3 6"}, {'pitcher_id': '669093', 'player_data': '', 'team_data': 'Jeremiah Estrada has not faced anyone on this team.'}, {'pitcher_id': '506433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDarvish, Yu 74 64 16 2 0 3 18 24.3 30 10 13.5 0.25 0.422 0.34 0.221 0.349 0.305 87.1 10.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 18 15 1 0 0 1 5 27.8 35.9 3 16.7 0.067 0.267 0.227 0.15 0.282 0.267 79 5.5\nMcCutchen, Andrew 17 12 4 0 0 0 4 23.5 33.3 5 29.4 0.333 0.333 0.412 0.27 0.355 0.421 89.9 18.8\nGrandal, Yasmani 11 10 1 0 0 0 2 18.2 25 1 9.1 0.1 0.1 0.143 0.081 0.122 0.141 89 10.5\nJoe, Connor 9 8 1 0 0 0 2 22.2 25 1 11.1 0.125 0.125 0.175 0.202 0.311 0.268 89.9 19\nTellez, Rowdy 8 8 3 1 0 1 4 50 38.5 0 0 0.375 0.875 0.519 0.266 0.505 0.324 89.3 22\nSuwinski, Jack 6 6 4 1 0 1 0 0 9.1 0 0 0.667 1.333 0.836 0.451 0.738 0.499 90 1.3\nHayes, Ke'Bryan 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.295 0.246 0.269 0.224 103.6 -8.5\nTriolo, Jared 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.396 0.432 0.364 74.2 2.5"}, {'pitcher_id': '660853', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe Los Santos, Enyel 11 9 3 1 0 0 2 18.2 25 2 18.2 0.333 0.444 0.399 0.208 0.305 0.308 80 7.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nReynolds, Bryan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.139 0.152 0.14 59.6 -34.5\nOlivares, Edward 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.086 0.088 0.081 48.3 18\nGrandal, Yasmani 1 0 0 0 0 0 0 0 50 1 100 0.69 0.69\nJoe, Connor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.569 1.132 0.709 105.3 18\nBruihl, Justin 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nHayes, Ke'Bryan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTriolo, Jared 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.769 0.909 0.728 102.5 9\nMcCutchen, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.015 0.01 94.1 49\nTaylor, Michael A. 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.077 0.21 0.117 90.8 30"}, {'pitcher_id': '650633', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, Michael 13 13 1 0 0 1 3 23.1 30.4 0 0 0.077 0.308 0.154 0.231 0.555 0.328 86.3 8.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTellez, Rowdy 6 6 0 0 0 0 2 33.3 41.7 0 0 0 0 0 0.196 0.598 0.331 87.3 -4.8\nTaylor, Michael A. 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.242 0.279 0.227 82.5 14\nOlivares, Edward 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.374 1.195 0.635 98.4 27\nGrandal, Yasmani 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.108 0.123 0.098 69.8 4'}, {'pitcher_id': '663158', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuarez, Robert 13 10 1 0 0 0 4 30.8 20 3 23.1 0.1 0.1 0.227 0.212 0.252 0.316 77.7 23.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHayes, Ke'Bryan 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.23 0.237 0.276 0.381 91.4 6\nMcCutchen, Andrew 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.295 0.304 0.36 0.287 79.6 15.5\nReynolds, Bryan 2 0 0 0 0 0 0 0 0 2 100 0.689 0.689\nTellez, Rowdy 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.013 0.029 0.018 90.1 37\nSuwinski, Jack 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nTriolo, Jared 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.493 0.387 72.3 39\nTaylor, Michael A. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.334 0.334 0.301 53.4 27"}, {'pitcher_id': '666745', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrito, Jhony 7 7 3 0 0 1 1 14.3 18.2 0 0 0.429 0.857 0.538 0.338 0.85 0.485 95.1 9.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 2 2 2 0 0 1 0 0 11.1 0 0 1 2.5 1.443 0.661 1.961 1.06 102.9 12.5\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.317 1.179 0.608 100.9 37\nReynolds, Bryan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nOlivares, Edward 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.249 0.296 0.23 93.5 -7\nTellez, Rowdy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.258 0.31 0.232 94.1 -3\nSuwinski, Jack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.22 0.246 0.201 76.5 3"}, {'pitcher_id': '670810', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGillaspie, Logan 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.171 0.241 0.176 75.3 40', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.171 0.241 0.176 75.3 40'}, {'pitcher_id': '681190', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVásquez, Randy 7 7 1 0 0 0 1 14.3 14.3 0 0 0.143 0.143 0.126 0.206 0.28 0.208 84.3 10.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.038 0.038 0.035 72.2 -36\nJoe, Connor 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.346 0.625 0.391 99.2 19\nOlivares, Edward 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.002 0.002 0.002 95.6 49\nHayes, Ke'Bryan 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.063 0.082 0.059 68.7 54\nSuwinski, Jack 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.954 1.173 0.929 97.6 13"}, {'pitcher_id': '663568', 'player_data': '', 'team_data': 'Stephen Kolek has not faced anyone on this team.'}, {'pitcher_id': '676680', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCosgrove, Tom 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.035 0.035 0.034 75.4 -34', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.035 0.035 0.034 75.4 -34'}, {'pitcher_id': '663362', 'player_data': '', 'team_data': 'Matt Waldron has not faced anyone on this team.'}, {'pitcher_id': '593974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Wandy 21 17 5 1 0 1 3 14.3 34.3 4 19 0.294 0.529 0.408 0.249 0.438 0.364 85.3 0.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrandal, Yasmani 10 8 3 0 0 1 0 0 6.3 2 20 0.375 0.75 0.507 0.352 0.705 0.486 86.7 5.3\nMcCutchen, Andrew 5 4 1 1 0 0 0 0 33.3 1 20 0.25 0.5 0.386 0.21 0.295 0.314 90.5 1\nReynolds, Bryan 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69\nTellez, Rowdy 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.078 0.078 0.07 50 -38\nOlivares, Edward 1 1 1 0 0 0 0 0 50 0 0 1 1 0.879 0.429 0.476 0.388 89.2 5\nCruz, Oneil 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTaylor, Michael A. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '670970', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorejon, Adrian 5 5 2 0 0 1 3 60 41.7 0 0 0.4 1 0.577 0.392 1.019 0.582 93.8 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaylor, Michael A. 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nMcCutchen, Andrew 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.48 0.547 0.455 81.6 16\nSuwinski, Jack 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGonzales, Nick 1 1 1 0 0 1 0 0 25 0 0 1 4 2.004 1 4 2 106 32'}]}, 136: {137: [{'pitcher_id': '660825', 'player_data': '', 'team_data': ''}, {'pitcher_id': '693433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWoo, Bryan 11 9 1 0 0 0 3 27.3 28.6 2 18.2 0.111 0.111 0.207 0.126 0.153 0.224 90 36.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 3 2 1 0 0 0 1 33.3 28.6 1 33.3 0.5 0.5 0.526 0.385 0.455 0.475 102.4 9\nYastrzemski, Mike 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.019 0.027 0.245 92.1 49\nChapman, Matt 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.107 0.131 0.097 83.3 34\nWisely, Brett 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.005 0.012 0.009 87 43'}, {'pitcher_id': '642100', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpeier, Gabe 1 1 1 0 0 1 0 0 0 0 0 1 4 1.979 0.56 2.069 1.054 99.1 31', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCasali, Curt 1 1 1 0 0 1 0 0 0 0 0 1 4 1.979 0.56 2.069 1.054 99.1 31'}, {'pitcher_id': '674285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSalazar, Eduardo 5 4 1 0 0 0 2 40 23.1 1 20 0.25 0.25 0.316 0.246 0.307 0.33 71.8 -3.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCasali, Curt 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nRamos, Heliot 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nEstrada, Thairo 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.293 0.294 0.267 41.7 -22\nYastrzemski, Mike 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.691 0.933 0.689 101.9 15\nChapman, Matt 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '676092', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnider, Collin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.669 0.761 0.628 109 6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.669 0.761 0.628 109 6'}, {'pitcher_id': '682243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bryce 5 4 0 0 0 0 2 40 42.9 1 20 0 0 0.139 0.102 0.366 0.291 86.3 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.018 0.018 0.244 67.9 -25\nChapman, Matt 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.185 0.714 0.361 104.7 40'}, {'pitcher_id': '592773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanek, Ryne 33 30 4 1 0 0 10 30.3 35.1 2 6.1 0.133 0.167 0.159 0.134 0.188 0.169 87.5 26.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 15 13 1 1 0 0 8 53.3 47.2 2 13.3 0.077 0.154 0.175 0.078 0.122 0.168 93.3 33.8\nSoler, Jorge 6 6 1 0 0 0 1 16.7 18.2 0 0 0.167 0.167 0.147 0.146 0.179 0.145 79.9 15.4\nFlores, Wilmer 5 4 1 0 0 0 0 0 30 0 0 0.25 0.25 0.174 0.161 0.181 0.122 93.7 28.6\nConforto, Michael 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.151 0.449 0.236 97.5 29\nAhmed, Nick 1 1 0 0 0 0 0 0 16.7 0 0 0 0 0 0.092 0.11 0.084 68.4 -13\nWisely, Brett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.177 0.201 0.161 91.3 -13\nYastrzemski, Mike 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.897 1.023 0.84 85.8 14\nCasali, Curt 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.004 0.003 91 79\nEstrada, Thairo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 69.9 68'}, {'pitcher_id': '642048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSaucedo, Tayler 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.096 0.096 0.088 83.4 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.096 0.096 0.088 83.4 -9'}, {'pitcher_id': '662253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuñoz, Andrés 5 5 2 1 0 0 2 40 35.7 0 0 0.4 0.6 0.429 0.128 0.134 0.118 94.4 -14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.442 0.024 0.024 0.024 69.4 -16\nYastrzemski, Mike 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nChapman, Matt 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.295 0.312 0.272 107 -14'}, {'pitcher_id': '669912', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnead, Kirby 5 4 0 0 0 0 2 40 25 1 20 0 0 0.138 0.054 0.059 0.179 94.2 10.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSlater, Austin 2 1 0 0 0 0 1 50 25 1 50 0 0 0.344 0.344\nYastrzemski, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.196 0.196 0.178 103 -19\nConforto, Michael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFlores, Wilmer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.04 0.027 85.3 40'}, {'pitcher_id': '657508', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaumann, Mike 9 8 1 1 0 0 2 22.2 20.8 1 11.1 0.125 0.25 0.217 0.271 0.557 0.393 85.7 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 5 5 1 1 0 0 2 40 30 0 0 0.2 0.4 0.252 0.211 0.644 0.355 96.7 32.7\nSoler, Jorge 1 0 0 0 0 0 0 0 33.3 1 100 0.696 0.696\nSlater, Austin 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.282 0.284 0.258 64.4 -56\nWisely, Brett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.024 0.017 87.4 43\nBailey, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.82 0.931 0.788 71.9 28'}, {'pitcher_id': '669302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGilbert, Logan 25 23 2 1 0 0 7 28 32.6 1 4 0.087 0.13 0.142 0.163 0.276 0.225 90.2 22.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 11 9 1 0 0 0 4 36.4 45 1 9.1 0.111 0.111 0.209 0.119 0.242 0.252 88.7 48\nConforto, Michael 4 4 0 0 0 0 0 0 40 0 0 0 0 0 0.173 0.217 0.17 90.3 16.3\nYastrzemski, Mike 4 4 1 1 0 0 1 25 16.7 0 0 0.25 0.5 0.311 0.335 0.619 0.395 88.8 -9\nSoler, Jorge 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.03 0.04 0.028 94.4 6.5\nBailey, Patrick 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.181 0.233 0.172 91.6 35.5'}, {'pitcher_id': '608723', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVoth, Austin 29 25 6 0 0 1 6 20.7 27.3 4 13.8 0.24 0.36 0.317 0.213 0.301 0.287 85.2 22.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 12 9 3 0 0 1 1 8.3 17.9 3 25 0.333 0.667 0.489 0.222 0.393 0.367 88 18.8\nChapman, Matt 5 5 0 0 0 0 2 40 53.3 0 0 0 0 0 0.049 0.056 0.05 70.7 18\nFlores, Wilmer 4 4 3 0 0 0 0 0 0 0 0 0.75 0.75 0.661 0.563 0.677 0.548 92.1 19.3\nSoler, Jorge 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.013 0.023 0.014 79.5 62.5\nEstrada, Thairo 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCasali, Curt 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nBailey, Patrick 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nYastrzemski, Mike 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.506 0.593 0.489 73.8 30\nAhmed, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.29 0.332 0.261 101.6 -9'}, {'pitcher_id': '676969', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nde Geus, Brett 13 9 2 0 0 0 1 7.7 21.7 1 7.7 0.222 0.222 0.355 0.479 0.548 0.531 93.1 -5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 3 1 0 0 0 0 0 0 33.3 1 33.3 0 0 0.471 0.061 0.069 0.493 84.5 -29\nFlores, Wilmer 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.511 0.593 0.483 99 4\nEstrada, Thairo 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0.361 0.352 0.354 0.527 77.2 -58\nYastrzemski, Mike 2 1 1 0 0 0 0 0 0 0 0 1 1 0.8 0.734 0.81 0.695 95.6 10\nCasali, Curt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.906 0.982 0.825 80.6 16\nSlater, Austin 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.406 0.535 0.401 106.2 13\nChapman, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.42 0.462 0.391 102.4 -2'}, {'pitcher_id': '675989', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBolton, Cody 6 6 3 2 0 1 2 33.3 25 0 0 0.5 1.333 0.749 0.518 1.071 0.658 102.5 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYastrzemski, Mike 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nConforto, Michael 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.637 0.69 0.573 90.2 10\nSoler, Jorge 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBailey, Patrick 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.903 3.31 1.693 104.8 25\nSlater, Austin 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.64 1.171 0.739 109 16\nChapman, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.926 1.253 0.943 106 11'}, {'pitcher_id': '676106', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622491', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastillo, Luis 51 45 11 4 0 3 11 21.6 22.5 6 11.8 0.244 0.533 0.367 0.256 0.434 0.341 89.5 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 8 5 2 1 0 1 1 12.5 40 3 37.5 0.4 1.2 0.667 0.425 0.854 0.595 92.8 11\nAhmed, Nick 11 11 0 0 0 0 3 27.3 18.2 0 0 0 0 0 0.161 0.276 0.189 86.8 4\nFlores, Wilmer 7 6 2 0 0 1 0 0 6.7 1 14.3 0.333 0.833 0.513 0.343 0.435 0.389 88.8 14.5\nConforto, Michael 9 8 0 0 0 0 4 44.4 38.5 1 11.1 0 0 0.077 0.064 0.089 0.133 85.7 20.5\nChapman, Matt 9 9 4 2 0 0 2 22.2 21.1 0 0 0.444 0.667 0.475 0.33 0.499 0.358 97.5 22\nYastrzemski, Mike 6 5 3 1 0 1 1 16.7 22.2 1 16.7 0.6 1.4 0.803 0.399 0.858 0.552 82.5 21.3\nCasali, Curt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.094 0.119 0.086 90.9 -13'}, {'pitcher_id': '669923', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKirby, George 6 5 2 0 0 0 1 16.7 18.2 1 16.7 0.4 0.4 0.409 0.256 0.283 0.314 91.2 -14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 3 2 1 0 0 0 1 33.3 14.3 1 33.3 0.5 0.5 0.524 0.277 0.327 0.409 98 4\nSoler, Jorge 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.294 0.242 0.254 0.219 88.9 -21'}, {'pitcher_id': '663423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThornton, Trent 7 6 0 0 0 0 1 14.3 38.5 1 14.3 0 0 0.099 0.184 0.249 0.263 76.4 38.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 4 3 0 0 0 0 1 25 50 1 25 0 0 0.173 0.021 0.025 0.188 74.3 29.5\nSoler, Jorge 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.056 0.191 0.1 83 52.5\nEstrada, Thairo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.931 1.036 0.89 67.7 30'}], 138: [{'pitcher_id': '660825', 'player_data': '', 'team_data': 'Eduard Bazardo has not faced anyone on this team.'}, {'pitcher_id': '693433', 'player_data': '', 'team_data': 'Bryan Woo has not faced anyone on this team.'}, {'pitcher_id': '642100', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpeier, Gabe 3 3 1 0 0 0 0 0 22.2 0 0 0.333 0.333 0.294 0.538 0.841 0.591 92.3 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.774 0.846 0.711 73.6 26\nGorman, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.234 0.234 0.214 100 -8\nGoldschmidt, Paul 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.606 1.442 0.848 103.3 21\nCarlson, Dylan 0 0 0 0 0 0 0 100 0'}, {'pitcher_id': '674285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSalazar, Eduardo 2 1 0 0 0 0 1 50 20 1 50 0 0 0.348 0.348', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurleson, Alec 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nGorman, Nolan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '676092', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnider, Collin 1 0 0 0 0 0 0 0 1 100 0.689 0.689', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 1 0 0 0 0 0 0 0 1 100 0.689 0.689'}, {'pitcher_id': '682243', 'player_data': '', 'team_data': 'Bryce Miller has not faced anyone on this team.'}, {'pitcher_id': '592773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanek, Ryne 8 7 3 0 0 1 3 37.5 29.4 1 12.5 0.429 0.857 0.548 0.365 0.759 0.49 92.1 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 4 3 2 0 0 1 1 25 0 1 25 0.667 1.667 0.876 0.607 1.502 0.814 88.1 25.5\nCrawford, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.367 0.405 0.334 96.2 29.5\nGoldschmidt, Paul 1 1 0 0 0 0 1 100 60 0 0 0 0 0 0\nCarpenter, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '642048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSaucedo, Tayler 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.051 0.063 0.049 88.2 -15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCarlson, Dylan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.102 0.126 0.097 88.2 -15'}, {'pitcher_id': '662253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuñoz, Andrés 5 4 0 0 0 0 0 0 11.1 1 20 0 0 0.138 0.049 0.065 0.175 77.6 40.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.032 0.022 84.1 45.5\nCrawford, Brandon 2 1 0 0 0 0 0 0 0 1 50 0 0 0.345 0.146 0.185 0.412 54.7 14\nCarpenter, Matt 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.009 0.012 0.008 87.3 58'}, {'pitcher_id': '669912', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnead, Kirby 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.344 0.344', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 1 0 0 0 0 0 0 0 100 1 100 0.689 0.689\nCrawford, Brandon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '657508', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaumann, Mike 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '669302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGilbert, Logan 7 7 1 0 0 0 2 28.6 31.3 0 0 0.143 0.143 0.126 0.082 0.11 0.08 83.7 33.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.107 0.144 0.105 82.3 47\nCrawford, Brandon 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.049 0.065 0.045 85.9 13.5'}, {'pitcher_id': '608723', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVoth, Austin 6 6 3 0 0 1 0 0 10 0 0 0.5 1 0.632 0.554 0.944 0.64 91 19.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.619 1.057 0.708 99.8 19.5\nCrawford, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.501 0.601 0.487 82.1 23.5\nArenado, Nolan 1 1 1 0 0 1 0 0 50 0 0 1 4 2.031 0.753 1.972 1.138 103.9 21\nCarlson, Dylan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.334 0.374 0.313 78.3 8'}, {'pitcher_id': '676969', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nde Geus, Brett 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.684 1.849 1.052 95 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.931 3.166 1.665 105.9 23\nCarlson, Dylan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.437 0.533 0.438 84 6'}, {'pitcher_id': '675989', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBolton, Cody 8 4 1 0 0 0 2 25 38.5 4 50 0.25 0.25 0.458 0.368 0.399 0.518 84.9 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.348 0.348\nArenado, Nolan 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.441 0.736 0.798 0.68 84.9 13.5\nCrawford, Brandon 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nBurleson, Alec 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nDonovan, Brendan 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nGorman, Nolan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '676106', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622491', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastillo, Luis 166 146 38 7 1 7 42 25.3 31.3 16 9.6 0.26 0.466 0.35 0.225 0.411 0.317 87.2 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 46 41 9 1 0 3 12 26.1 31.3 5 10.9 0.22 0.463 0.328 0.204 0.465 0.323 90.3 13.2\nCarpenter, Matt 38 30 7 2 0 2 10 26.3 35 6 15.8 0.233 0.5 0.387 0.275 0.472 0.396 82.6 21.4\nArenado, Nolan 29 27 9 0 1 0 5 17.2 25 2 6.9 0.333 0.407 0.344 0.265 0.3 0.279 90.7 3.5\nCarlson, Dylan 23 22 7 2 0 1 7 30.4 36.2 1 4.3 0.318 0.545 0.378 0.182 0.415 0.266 87.1 16.5\nCrawford, Brandon 11 11 2 0 0 1 5 45.5 38.5 0 0 0.182 0.455 0.262 0.163 0.34 0.211 86.8 2.2\nGorman, Nolan 6 5 1 1 0 0 1 16.7 16.7 1 16.7 0.2 0.4 0.323 0.223 0.63 0.409 76.1 15.2\nMikolas, Miles 5 3 1 1 0 0 1 20 16.7 1 20 0.333 0.667 0.477 0.183 0.288 0.323 68.6 -0.3\nDonovan, Brendan 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0.24 0.275 0.316 0.411 98.7 24.5\nLynn, Lance 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.037 0.04 0.033 79.8 20.5\nBurleson, Alec 3 3 2 0 0 0 1 33.3 40 0 0 0.667 0.667 0.588 0.31 0.353 0.286 98.4 4'}, {'pitcher_id': '669923', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKirby, George 16 16 4 0 0 0 4 25 28.6 0 0 0.25 0.25 0.221 0.256 0.361 0.265 86 7.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDonovan, Brendan 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.086 0.087 0.078 77.8 4.5\nGoldschmidt, Paul 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.462 0.937 0.586 106.9 11\nBurleson, Alec 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.294 0.239 0.266 0.217 87.5 16.7\nArenado, Nolan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.305 0.329 0.281 73.5 -0.7\nGorman, Nolan 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nCarpenter, Matt 2 2 2 0 0 0 0 0 16.7 0 0 1 1 0.883 0.414 0.457 0.377 90 3'}, {'pitcher_id': '663423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThornton, Trent 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.174 0.296 0.438 0.306 89.7 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.174 0.296 0.438 0.306 89.7 -4'}], 139: [{'pitcher_id': '660825', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBazardo, Eduard 11 9 1 0 0 0 3 27.3 38.5 1 9.1 0.111 0.111 0.208 0.158 0.285 0.283 86.2 33', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 2 1 0 0 0 0 0 0 40 1 50 0 0 0.344 0.031 0.046 0.36 80 44\nParedes, Isaac 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.021 0.021 0.021 85.3 -31\nSiri, Jose 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.194 0.725 0.371 97.2 45.5\nLowe, Josh 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.001 0.001 0 88.5 71\nLowe, Brandon 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDíaz, Yandy 2 1 0 0 0 0 1 50 66.7 0 0 0 0 0.36 0.36\nRosario, Amed 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.963 1.023 0.884 69 23'}, {'pitcher_id': '693433', 'player_data': '', 'team_data': 'Bryan Woo has not faced anyone on this team.'}, {'pitcher_id': '642100', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpeier, Gabe 6 5 4 1 0 0 0 0 11.1 0 0 0.8 1 0.649 0.474 0.627 0.425 95.5 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.063 0.519 0.857 0.583 108.2 6\nRosario, Amed 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.456 0.469 0.417 86.1 8.5\nWalls, Taylor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.42 0.483 0.378 91.9 1\nSiri, Jose 1 0 0 0 0 0 0 0 0 0 0 0 0.169 92.4 24'}, {'pitcher_id': '674285', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676092', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnider, Collin 11 8 2 0 0 0 1 9.1 15.4 2 18.2 0.25 0.25 0.287 0.37 0.444 0.389 87.7 19.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 4 4 2 0 0 0 1 25 14.3 0 0 0.5 0.5 0.442 0.479 0.588 0.468 89.6 5.3\nParedes, Isaac 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.316 0.334 0.286 100.4 -3\nArozarena, Randy 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nLowe, Brandon 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nDeLuca, Jonny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.274 0.382 0.293 65.7 47\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.479 0.399 91.3 5\nLowe, Josh 1 0 0 0 0 0 0 0 0 0 0 0 0.029 81.8 39\nSiri, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.007 0.005 93.6 55'}, {'pitcher_id': '682243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bryce 20 19 8 2 0 1 6 30 50 1 5 0.421 0.684 0.48 0.399 0.601 0.444 86 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 5 5 2 0 0 0 2 40 40 0 0 0.4 0.4 0.353 0.471 0.775 0.534 98.1 13.7\nLowe, Josh 4 4 3 2 0 0 1 25 54.5 0 0 0.75 1.25 0.843 0.646 0.736 0.609 87.3 13\nLowe, Brandon 3 2 1 0 0 0 0 0 20 1 33.3 0.5 0.5 0.526 0.534 0.621 0.569 84.2 11.5\nArozarena, Randy 2 2 1 0 0 1 1 50 66.7 0 0 0.5 2 1.002 0.449 1.323 0.724 107.8 21\nSiri, Jose 4 4 1 0 0 0 2 50 66.7 0 0 0.25 0.25 0.221 0.155 0.164 0.142 60 5\nWalls, Taylor 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.009 0.012 0.008 91.6 63\nParedes, Isaac 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.038 0.038 0.037 73.4 -24'}, {'pitcher_id': '592773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanek, Ryne 12 11 3 2 0 0 4 33.3 31.8 1 8.3 0.273 0.455 0.338 0.166 0.239 0.217 92.5 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 4 4 0 0 0 0 1 25 27.3 0 0 0 0 0 0.112 0.116 0.103 92.9 -20.7\nArozarena, Randy 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.622 0.08 0.154 0.099 92.1 23\nParedes, Isaac 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.486 0.762 0.603 97.8 17\nLowe, Brandon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSiri, Jose 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCaballero, José 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.68 0.983 0.721 96.5 16\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.105 0.069 82.1 33'}, {'pitcher_id': '642048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSaucedo, Tayler 11 9 4 1 0 1 1 9.1 0 2 18.2 0.444 0.889 0.581 0.303 0.461 0.394 97 -8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalls, Taylor 4 3 1 0 0 0 1 25 0 1 25 0.333 0.333 0.394 0.169 0.18 0.288 94.3 -34.5\nArozarena, Randy 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.621 0.353 0.503 0.355 99.7 4\nDíaz, Yandy 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.442 0.482 0.918 0.604 105.9 7\nParedes, Isaac 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.055 0.052 81.6 -37\nLowe, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.5 0.712 0.519 94.3 18'}, {'pitcher_id': '662253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuñoz, Andrés 13 10 5 1 0 0 2 15.4 50 2 15.4 0.5 0.6 0.529 0.401 0.503 0.464 90.3 -1.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Josh 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.622 0.203 0.241 0.183 98.3 -2\nArozarena, Randy 3 3 2 0 0 0 0 0 40 0 0 0.667 0.667 0.589 0.646 0.919 0.675 97.6 15\nLowe, Brandon 2 1 0 0 0 0 1 50 83.3 0 0 0 0 0.36 0.36\nRosario, Amed 2 2 2 0 0 0 0 0 25 0 0 1 1 0.883 0.646 0.689 0.602 84.8 11\nWalls, Taylor 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nDíaz, Yandy 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.151 0.192 0.412 94.2 -17\nParedes, Isaac 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.221 0.221 0.201 67.1 -60'}, {'pitcher_id': '669912', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnead, Kirby 7 7 2 0 0 0 1 14.3 18.8 0 0 0.286 0.286 0.252 0.218 0.268 0.213 87.7 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.042 0.045 0.04 83.5 6.5\nParedes, Isaac 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.34 0.599 0.394 97.9 19\nLowe, Brandon 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.169 0.196 0.159 78.4 8\nWalls, Taylor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.651 0.679 0.586 73.2 16\nDíaz, Yandy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.115 0.114 109.5 -29'}, {'pitcher_id': '657508', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaumann, Mike 17 14 2 0 0 0 2 11.8 21.9 3 17.6 0.143 0.143 0.227 0.27 0.3 0.331 89.2 -9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 4 3 1 0 0 0 1 25 22.2 1 25 0.333 0.333 0.395 0.177 0.18 0.297 106.1 -20.5\nWalls, Taylor 4 3 0 0 0 0 0 0 12.5 1 25 0 0 0.174 0.213 0.241 0.32 73 0.7\nRosario, Amed 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.26 0.264 0.238 92.2 -31\nParedes, Isaac 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.142 0.16 0.412 82.3 -4\nDíaz, Yandy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.791 0.928 0.762 93.4 11\nSiri, Jose 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLowe, Josh 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.173 0.177 0.162 91.9 -14\nLowe, Brandon 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.196 0.22 0.178 94.7 -13'}, {'pitcher_id': '669302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGilbert, Logan 53 47 16 5 2 0 13 24.5 27.6 6 11.3 0.34 0.532 0.405 0.302 0.388 0.343 91.9 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 11 11 4 2 1 0 6 54.5 36 0 0 0.364 0.727 0.45 0.198 0.278 0.203 88.9 4\nRosario, Amed 15 14 7 2 1 0 1 6.7 16.7 1 6.7 0.5 0.786 0.553 0.44 0.559 0.449 95.2 2.3\nDíaz, Yandy 9 7 2 0 0 0 1 11.1 10.5 2 22.2 0.286 0.286 0.349 0.359 0.397 0.411 100.3 12\nLowe, Brandon 9 7 2 1 0 0 3 33.3 40 2 22.2 0.286 0.429 0.389 0.256 0.312 0.348 87.9 38.8\nWalls, Taylor 4 3 1 0 0 0 1 25 42.9 1 25 0.333 0.333 0.392 0.226 0.318 0.351 60 18\nSiri, Jose 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.042 0.077 0.053 82.4 9.5\nLowe, Josh 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.612 0.42 102.5 28'}, {'pitcher_id': '608723', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVoth, Austin 50 45 9 3 0 2 15 30 31.6 4 8 0.2 0.4 0.298 0.183 0.316 0.261 89.3 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 10 9 1 0 0 1 4 40 38.9 0 0 0.111 0.444 0.28 0.114 0.165 0.182 94.5 26.6\nDíaz, Yandy 8 7 2 1 0 0 3 37.5 37.5 1 12.5 0.286 0.429 0.352 0.304 0.617 0.426 103.5 26.3\nRosario, Amed 9 9 1 0 0 0 2 22.2 28.6 0 0 0.111 0.111 0.098 0.177 0.255 0.189 81.7 3\nLowe, Brandon 8 6 2 1 0 1 0 0 14.3 2 25 0.333 1 0.587 0.313 0.661 0.478 89.1 18.2\nWalls, Taylor 6 5 1 1 0 0 3 50 37.5 1 16.7 0.2 0.4 0.325 0.079 0.152 0.195 95.3 27\nLowe, Josh 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.236 0.265 0.22 76.6 25.3\nParedes, Isaac 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nSiri, Jose 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.295 0.088 0.109 0.086 88.5 2'}, {'pitcher_id': '676969', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nde Geus, Brett 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRortvedt, Ben 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDíaz, Yandy 1 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '675989', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBolton, Cody 5 5 2 0 0 0 1 20 16.7 0 0 0.4 0.4 0.353 0.122 0.138 0.112 84 -20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSiri, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.34 0.389 0.306 95.2 -5\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.132 0.158 0.123 94.8 -27\nLowe, Brandon 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.066 0.07 0.065 61.5 -32\nDíaz, Yandy 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.071 0.071 0.066 84.3 -18'}, {'pitcher_id': '676106', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622491', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastillo, Luis 69 64 16 2 1 3 19 27.5 28.7 5 7.2 0.25 0.453 0.323 0.245 0.377 0.297 89.9 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 22 20 7 0 1 1 4 18.2 20.6 2 9.1 0.35 0.6 0.421 0.271 0.514 0.364 89.2 -8.4\nLowe, Josh 8 7 2 0 0 0 4 50 31.8 1 12.5 0.286 0.286 0.308 0.227 0.29 0.285 97.9 16\nArozarena, Randy 9 7 2 0 0 1 0 0 21.4 2 22.2 0.286 0.714 0.475 0.326 0.461 0.419 92 19\nParedes, Isaac 8 8 2 0 0 1 3 37.5 21.1 0 0 0.25 0.625 0.361 0.197 0.287 0.206 95.8 15.2\nDíaz, Yandy 6 6 1 1 0 0 1 16.7 16.7 0 0 0.167 0.333 0.21 0.445 0.562 0.442 85.3 16.2\nWalls, Taylor 4 4 0 0 0 0 3 75 53.8 0 0 0 0 0 0.031 0.09 0.048 94.7 33\nPalacios, Richie 4 4 0 0 0 0 0 0 15.4 0 0 0 0 0 0.071 0.093 0.066 77.9 15.3\nLowe, Brandon 3 3 1 1 0 0 1 33.3 55.6 0 0 0.333 0.667 0.415 0.368 0.451 0.351 98.3 7.5\nEflin, Zach 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nJackson, Alex 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.293 0.157 0.177 0.142 79.8 1.5'}, {'pitcher_id': '669923', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKirby, George 48 44 6 3 0 0 15 31.3 29.8 3 6.3 0.136 0.205 0.176 0.192 0.331 0.248 90.2 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 9 8 2 1 0 0 3 33.3 43.8 1 11.1 0.25 0.375 0.314 0.341 0.76 0.487 95.9 11.8\nSiri, Jose 6 6 1 1 0 0 3 50 31.3 0 0 0.167 0.333 0.207 0.168 0.235 0.173 95.7 38.3\nLowe, Josh 6 4 0 0 0 0 1 16.7 25 1 16.7 0 0 0.116 0.181 0.59 0.341 91.5 36\nDíaz, Yandy 6 5 1 1 0 0 1 16.7 9.1 1 16.7 0.2 0.4 0.323 0.22 0.26 0.29 75.8 -21.8\nWalls, Taylor 7 7 0 0 0 0 3 42.9 47.1 0 0 0 0 0 0.114 0.139 0.107 90.3 18.3\nParedes, Isaac 3 3 2 0 0 0 1 33.3 18.2 0 0 0.667 0.667 0.588 0.159 0.171 0.143 100.3 29.5\nLowe, Brandon 6 6 0 0 0 0 3 50 22.2 0 0 0 0 0 0.099 0.13 0.096 98.6 4.3\nRosario, Amed 5 5 0 0 0 0 0 0 25 0 0 0 0 0 0.202 0.226 0.19 82.7 14.2'}, {'pitcher_id': '663423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThornton, Trent 33 26 11 4 0 2 6 18.2 18.5 7 21.2 0.423 0.808 0.552 0.412 0.776 0.542 92 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Brandon 10 7 3 1 0 1 4 40 27.3 3 30 0.429 1 0.61 0.358 0.928 0.57 98.7 22.7\nDíaz, Yandy 6 4 1 0 0 0 0 0 8.3 2 33.3 0.25 0.25 0.377 0.362 0.491 0.476 86.4 29.8\nWalls, Taylor 4 2 1 1 0 0 1 25 12.5 2 50 0.5 1 0.656 0.358 0.528 0.533 106.4 14\nParedes, Isaac 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.532 0.544 0.623 0.518 87 5.8\nArozarena, Randy 4 4 2 0 0 1 0 0 22.2 0 0 0.5 1.25 0.738 0.583 1.358 0.825 94.3 3.8\nSiri, Jose 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.631 0.232 0.638 0.359 99.3 26\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.331 0.206 92.8 24\nRosario, Amed 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.891 1.111 0.878 99.8 13\nPalacios, Richie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 75.4 78'}], 140: [{'pitcher_id': '660825', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBazardo, Eduard 21 19 6 0 0 1 3 14.3 17.1 2 9.5 0.316 0.474 0.376 0.294 0.391 0.332 89.7 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 5 5 2 0 0 0 2 40 26.7 0 0 0.4 0.4 0.355 0.293 0.361 0.277 98.8 2.7\nGarcía, Adolis 4 4 2 0 0 1 1 25 16.7 0 0 0.5 1.25 0.74 0.388 0.622 0.433 84 14.7\nTaveras, Leody 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.354 0.486 0.357 98.4 12\nJankowski, Travis 2 1 1 0 0 0 0 0 0 1 50 1 1 0.792 0.338 0.377 0.501 99.7 -5\nGrossman, Robbie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.179 0.21 0.168 89.3 17.5\nSmith, Josh 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nDuran, Ezequiel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.24 0.255 0.22 81.1 -0.5\nLangford, Wyatt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.091 0.076 82.1 -36\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.258 0.284 0.235 69.1 7'}, {'pitcher_id': '693433', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWoo, Bryan 30 23 11 2 0 4 7 23.3 22 6 20 0.478 1.087 0.661 0.382 0.758 0.528 100.4 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 6 4 1 0 0 0 2 33.3 11.8 1 16.7 0.25 0.25 0.384 0.336 0.41 0.448 92.5 17\nTaveras, Leody 5 4 1 0 0 1 1 20 11.1 1 20 0.25 1 0.54 0.211 0.504 0.376 98.9 18.3\nGarcía, Adolis 6 4 1 0 0 1 2 33.3 55.6 2 33.3 0.25 1 0.566 0.264 1.014 0.574 85.3 0.5\nSeager, Corey 6 4 3 1 0 1 0 0 28.6 2 33.3 0.75 1.75 0.92 0.531 0.736 0.595 105.1 13.5\nSemien, Marcus 6 6 4 1 0 1 2 33.3 14.3 0 0 0.667 1.333 0.835 0.497 1.045 0.643 105.2 12.3\nDuran, Ezequiel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.443 0.506 0.402 112.7 -9'}, {'pitcher_id': '642100', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpeier, Gabe 35 35 3 0 0 1 14 40 36.6 0 0 0.086 0.171 0.108 0.168 0.264 0.183 86.2 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 8 8 0 0 0 0 5 62.5 45 0 0 0 0 0 0.031 0.039 0.029 68.9 -8.7\nSeager, Corey 8 8 0 0 0 0 4 50 35.3 0 0 0 0 0 0.258 0.371 0.269 102.1 14.5\nSemien, Marcus 7 7 1 0 0 1 2 28.6 27.3 0 0 0.143 0.571 0.286 0.17 0.461 0.257 82.5 13.2\nGarcía, Adolis 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.177 0.172 0.188 0.158 95.4 24.5\nJankowski, Travis 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.029 0.031 0.029 75 -23\nTaveras, Leody 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.059 0.061 0.057 89.8 -14\nWendzel, Davis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.78 0.87 0.722 80.7 14\nSmith, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.283 0.368 0.289 68.6 46\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.28 0.363 0.261 86 23'}, {'pitcher_id': '674285', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSalazar, Eduardo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.064 0.061 77.2 -39', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.064 0.061 77.2 -39'}, {'pitcher_id': '676092', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnider, Collin 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.312 0.337 0.287 71.2 -0.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHuff, Sam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.217 0.217 0.199 100 -13\nSeager, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.809 0.864 0.744 73 18\nLowe, Nathaniel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.222 0.267 0.206 40.6 -6'}, {'pitcher_id': '682243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Bryce 38 33 10 4 0 3 7 18.4 20.3 5 13.2 0.303 0.697 0.453 0.379 0.901 0.551 93.2 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 7 5 3 0 0 2 0 0 30.8 2 28.6 0.6 1.8 0.897 0.797 2.409 1.134 100.8 18.2\nLowe, Nathaniel 6 5 1 0 0 0 1 16.7 0 1 16.7 0.2 0.2 0.262 0.26 0.412 0.359 84.2 9.8\nSemien, Marcus 8 7 1 1 0 0 2 25 27.3 1 12.5 0.143 0.286 0.242 0.274 0.499 0.37 91.2 31.4\nGarcía, Adolis 6 6 2 1 0 1 2 33.3 15.4 0 0 0.333 1 0.555 0.319 0.891 0.496 99.3 19\nTaveras, Leody 4 4 1 0 0 0 0 0 28.6 0 0 0.25 0.25 0.221 0.355 0.426 0.335 85 33.3\nLangford, Wyatt 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.8 2.75 1.067 102.6 25\nSmith, Josh 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nGrossman, Robbie 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.426 0.98 0.574 97.7 12.5\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.339 0.386 0.306 90.5 0'}, {'pitcher_id': '592773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanek, Ryne 71 61 17 4 0 2 14 19.7 27.1 7 9.9 0.279 0.443 0.352 0.265 0.419 0.352 91.2 16.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 16 12 5 1 0 0 4 25 29 3 18.8 0.417 0.5 0.472 0.251 0.336 0.362 90.8 27.8\nLowe, Nathaniel 8 6 1 0 0 0 1 12.5 30 2 25 0.167 0.167 0.283 0.164 0.33 0.33 91 -4\nGarcía, Adolis 11 11 4 2 0 1 3 27.3 40 0 0 0.364 0.818 0.49 0.362 0.63 0.42 97.1 21\nDuran, Ezequiel 7 7 1 0 0 0 3 42.9 23.8 0 0 0.143 0.143 0.126 0.143 0.174 0.138 83.2 11.2\nSeager, Corey 9 8 2 0 0 0 0 0 29.4 0 0 0.25 0.25 0.277 0.455 0.621 0.503 89.1 5.9\nSmith, Josh 6 6 1 0 0 1 2 33.3 11.8 0 0 0.167 0.667 0.334 0.113 0.382 0.199 88.6 31\nTaveras, Leody 5 4 1 0 0 0 0 0 10 0 0 0.25 0.25 0.177 0.21 0.231 0.34 89.9 14\nGrossman, Robbie 3 2 1 0 0 0 0 0 40 1 33.3 0.5 0.5 0.523 0.403 0.519 0.496 97.8 18\nJankowski, Travis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.12 0.288 0.163 85.7 38.5\nLangford, Wyatt 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.492 0.791 0.539 102.6 5.5\nHuff, Sam 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLorenzen, Michael 1 0 0 0 0 0 0 0 50 1 100 0.69 0.69'}, {'pitcher_id': '642048', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSaucedo, Tayler 24 20 6 2 0 1 0 0 12.2 3 12.5 0.3 0.55 0.415 0.3 0.562 0.418 92.4 2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 5 4 0 0 0 0 0 0 18.2 1 20 0 0 0.138 0.243 0.474 0.377 89.5 -4.5\nSeager, Corey 7 7 2 1 0 0 0 0 0 0 0 0.286 0.429 0.306 0.265 0.445 0.303 96.5 3.6\nGarcía, Adolis 3 2 1 0 0 0 0 0 20 1 33.3 0.5 0.5 0.526 0.255 0.262 0.385 87.6 -39.5\nSemien, Marcus 5 5 2 1 0 0 0 0 11.1 0 0 0.4 0.6 0.425 0.362 0.47 0.354 89.9 3.6\nTaveras, Leody 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.428 1.675 0.846 95.4 47.5\nDuran, Ezequiel 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nSmith, Josh 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '662253', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMuñoz, Andrés 42 38 5 1 0 1 12 28.6 39.8 3 7.1 0.132 0.237 0.209 0.145 0.227 0.211 86.8 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 8 8 1 0 0 0 4 50 47.8 0 0 0.125 0.125 0.11 0.116 0.137 0.113 77.1 -15.8\nSeager, Corey 9 6 0 0 0 0 1 11.1 50 2 22.2 0 0 0.234 0.126 0.143 0.312 93.2 31.2\nLowe, Nathaniel 6 5 1 0 0 0 1 16.7 35.7 1 16.7 0.2 0.2 0.263 0.096 0.114 0.192 84.8 -5.8\nSemien, Marcus 6 6 1 1 0 0 3 50 46.2 0 0 0.167 0.333 0.207 0.154 0.2 0.153 84.2 6.7\nSmith, Josh 4 4 0 0 0 0 0 0 20 0 0 0 0 0 0.31 0.541 0.36 93.4 10.5\nTaveras, Leody 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.069 0.07 0.064 95.1 -32\nWendzel, Davis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDuran, Ezequiel 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.036 0.33 1.066 0.582 93.3 -8\nGrossman, Robbie 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLangford, Wyatt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.063 0.045 79.1 57\nHuff, Sam 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.217 0.282 0.196 81.4 2\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.132 0.135 0.123 74 -42'}, {'pitcher_id': '669912', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnead, Kirby 30 25 8 4 0 1 2 6.7 15.2 4 13.3 0.32 0.6 0.441 0.341 0.51 0.423 83.6 -1.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 8 7 3 0 0 0 1 12.5 11.1 0 0 0.429 0.429 0.421 0.266 0.465 0.364 76.6 -25.5\nGarcía, Adolis 5 5 1 1 0 0 0 0 18.2 0 0 0.2 0.4 0.252 0.388 0.658 0.452 89.5 0.2\nSeager, Corey 8 7 0 0 0 0 0 0 16.7 1 12.5 0 0 0.086 0.379 0.423 0.397 77.2 1.4\nTaveras, Leody 3 1 1 1 0 0 0 0 0 2 66.7 1 2 0.88 0.543 0.613 0.626 103.9 4\nGrossman, Robbie 3 2 1 1 0 0 1 33.3 20 1 33.3 0.5 1 0.65 0.127 0.243 0.333 97 21\nSemien, Marcus 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.215 0.553 0.316 91.3 29.5\nSmith, Josh 1 1 1 0 0 1 0 0 50 0 0 1 4 2.072 0.843 1.029 0.824 91.5 16'}, {'pitcher_id': '657508', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBaumann, Mike 7 7 0 0 0 0 1 14.3 15.4 0 0 0 0 0 0.063 0.072 0.06 85.9 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.118 0.125 0.106 89.2 -18\nSemien, Marcus 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.019 0.023 0.017 75.3 52\nTaveras, Leody 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.243 0.192 98.4 -10\nLowe, Nathaniel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.073 0.084 81.8 -44\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.015 0.006 95.3 48'}, {'pitcher_id': '669302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGilbert, Logan 142 133 29 10 1 3 27 19 24.5 9 6.3 0.218 0.376 0.279 0.272 0.429 0.325 92 16.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 31 28 6 3 0 2 9 29 38.8 3 9.7 0.214 0.536 0.346 0.303 0.597 0.41 98.2 16.9\nLowe, Nathaniel 31 30 10 3 0 0 6 19.4 16.9 1 3.2 0.333 0.433 0.343 0.353 0.503 0.38 91.5 10.9\nSemien, Marcus 26 26 2 0 0 0 4 15.4 22 0 0 0.077 0.077 0.068 0.176 0.22 0.173 87.5 24.8\nSeager, Corey 19 18 6 2 1 0 2 10.5 20.6 1 5.3 0.333 0.556 0.393 0.313 0.527 0.377 92.6 3.6\nTaveras, Leody 11 10 1 0 0 1 2 18.2 30 1 9.1 0.1 0.4 0.245 0.235 0.356 0.289 99.1 9.9\nSmith, Josh 11 9 2 1 0 0 0 0 14.3 2 18.2 0.222 0.333 0.321 0.249 0.309 0.325 86.3 24.6\nGrossman, Robbie 5 5 1 1 0 0 1 20 0 0 0 0.2 0.4 0.248 0.184 0.431 0.252 94 26.5\nLangford, Wyatt 3 2 0 0 0 0 2 66.7 25 1 33.3 0 0 0.231 0.231\nDuran, Ezequiel 5 5 1 0 0 0 1 20 25 0 0 0.2 0.2 0.177 0.277 0.314 0.258 86 28.2'}, {'pitcher_id': '608723', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVoth, Austin 37 32 7 1 0 0 4 10.8 15.4 3 8.1 0.219 0.25 0.272 0.298 0.366 0.345 83.3 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 7 5 2 1 0 0 0 0 7.1 2 28.6 0.4 0.6 0.501 0.382 0.411 0.447 79.8 -3.8\nSemien, Marcus 5 5 0 0 0 0 2 40 18.8 0 0 0 0 0 0.063 0.113 0.072 87 41.7\nSmith, Josh 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.17 0.195 0.155 85.3 18\nGarcía, Adolis 6 6 1 0 0 0 1 16.7 41.7 0 0 0.167 0.167 0.147 0.254 0.357 0.261 79.8 -1\nSeager, Corey 4 2 0 0 0 0 0 0 14.3 1 25 0 0 0.355 0.33 0.488 0.531 94.7 27\nGrossman, Robbie 3 2 0 0 0 0 1 33.3 0 0 0 0 0 0.242 0.064 0.065 0.282 50.3 -36\nTaveras, Leody 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.372 0.467 0.366 88 10.3\nJankowski, Travis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.894 1.073 0.873 95.3 12\nLangford, Wyatt 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.889 0.934 1.004 0.857 71.5 20\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.247 0.277 0.233 95.1 -5\nDuran, Ezequiel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.96 1.095 0.909 81.9 16'}, {'pitcher_id': '676969', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nde Geus, Brett 5 5 1 0 0 0 0 0 14.3 0 0 0.2 0.2 0.176 0.317 0.453 0.334 94.5 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 1 1 1 0 0 0 0 0 25 0 0 1 1 0.879 0.894 1.24 0.922 110.9 11\nJankowski, Travis 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.16 0.18 0.144 74.3 -4\nGarcía, Adolis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.372 0.193 97.4 39\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.121 0.141 0.127 85.6 -14\nLowe, Nathaniel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.307 0.33 0.282 104.4 -7'}, {'pitcher_id': '675989', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBolton, Cody 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKnizner, Andrew 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '676106', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622491', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastillo, Luis 121 110 30 5 0 4 31 25.6 26.1 9 7.4 0.273 0.427 0.33 0.259 0.399 0.313 85.3 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 14 11 4 2 0 1 4 28.6 12.1 2 14.3 0.364 0.818 0.491 0.25 0.489 0.344 97.4 12.8\nSeager, Corey 17 15 3 0 0 0 3 17.6 32.5 2 11.8 0.2 0.2 0.237 0.291 0.458 0.363 86.3 15.9\nGarcía, Adolis 14 14 4 1 0 0 5 35.7 41.4 0 0 0.286 0.357 0.28 0.227 0.328 0.239 86.6 -2\nSemien, Marcus 14 13 4 0 0 1 2 14.3 26.1 1 7.1 0.308 0.538 0.383 0.251 0.5 0.342 84.1 11.9\nKnizner, Andrew 12 11 2 0 0 1 3 25 20 1 8.3 0.182 0.455 0.293 0.089 0.112 0.141 81.7 2.5\nTaveras, Leody 9 7 2 0 0 0 1 11.1 22.7 2 22.2 0.286 0.286 0.35 0.217 0.276 0.32 81.3 35.5\nGrossman, Robbie 12 12 4 0 0 0 5 41.7 40 0 0 0.333 0.333 0.294 0.273 0.352 0.268 85.1 -0.1\nDuran, Ezequiel 6 5 0 0 0 0 3 50 38.5 0 0 0 0 0.121 0.115 0.125 0.209 71.8 52.5\nJankowski, Travis 10 10 3 0 0 0 1 10 6.3 0 0 0.3 0.3 0.264 0.496 0.554 0.46 75.5 12.4\nLangford, Wyatt 5 4 1 0 0 0 0 0 0 1 20 0.25 0.25 0.316 0.519 0.648 0.544 102 -9.3\nSmith, Josh 4 4 3 2 0 1 1 25 16.7 0 0 0.75 2 1.15 0.346 1.062 0.574 96.3 33.3\nUreña, José 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nWendzel, Davis 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.082 0.082 0.073 50.5 23'}, {'pitcher_id': '669923', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKirby, George 112 107 22 3 0 2 25 22.3 19 5 4.5 0.206 0.29 0.236 0.195 0.316 0.24 89.1 8.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 20 18 3 1 0 1 3 15 15.4 2 10 0.167 0.389 0.28 0.169 0.264 0.235 86.9 14.1\nLowe, Nathaniel 18 18 5 1 0 0 7 38.9 24.3 0 0 0.278 0.333 0.266 0.207 0.349 0.236 88.1 8.3\nSeager, Corey 14 13 0 0 0 0 2 14.3 20 1 7.1 0 0 0.049 0.161 0.208 0.199 92.5 16.2\nGarcía, Adolis 18 17 4 0 0 1 2 11.1 17.2 1 5.6 0.235 0.412 0.301 0.306 0.673 0.425 95.5 17.1\nSmith, Josh 9 8 3 1 0 0 3 33.3 12.5 1 11.1 0.375 0.5 0.411 0.171 0.185 0.218 83.3 1.2\nTaveras, Leody 11 11 1 0 0 0 2 18.2 18.2 0 0 0.091 0.091 0.08 0.173 0.268 0.19 94.9 -2.1\nDuran, Ezequiel 9 9 5 0 0 0 1 11.1 13.6 0 0 0.556 0.556 0.491 0.294 0.352 0.281 78 -17.6\nJankowski, Travis 4 4 0 0 0 0 1 25 18.2 0 0 0 0 0 0.153 0.162 0.144 90.6 -14\nGrossman, Robbie 4 4 0 0 0 0 1 25 11.1 0 0 0 0 0 0.014 0.025 0.016 86.9 42.3\nLangford, Wyatt 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.296 0.056 0.064 0.052 96.2 -20\nKnizner, Andrew 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.001 0.001 0 65.2 67'}, {'pitcher_id': '663423', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThornton, Trent 25 21 3 0 0 0 5 20 16.7 4 16 0.143 0.143 0.216 0.235 0.356 0.325 91.2 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 5 4 0 0 0 0 2 40 22.2 1 20 0 0 0.138 0.037 0.063 0.175 73.4 1.5\nLowe, Nathaniel 5 4 1 0 0 0 1 20 12.5 1 20 0.25 0.25 0.315 0.271 0.294 0.338 90.7 3.3\nSemien, Marcus 5 5 2 0 0 0 1 20 28.6 0 0 0.4 0.4 0.352 0.369 0.55 0.397 98.3 11.3\nGarcía, Adolis 4 3 0 0 0 0 1 25 16.7 1 25 0 0 0.172 0.129 0.479 0.361 95.6 57\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.64 0.755 0.615 104.6 7\nSeager, Corey 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nTaveras, Leody 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.127 0.228 0.15 82.5 -6.5\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.586 0.65 0.54 99.4 7\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 86.8 71'}]}, 137: {136: [{'pitcher_id': '678906', 'player_data': '', 'team_data': ''}, {'pitcher_id': '690986', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '694738', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663855', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHicks, Jordan 8 7 3 0 0 0 3 37.5 27.3 1 12.5 0.429 0.429 0.418 0.264 0.306 0.305 88 0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.231 0.112 0.124 0.298 90.1 -6\nRojas, Josh 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.442 0.28 0.385 0.284 95.4 17\nCanzone, Dominic 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nGarver, Mitch 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.072 0.072 0.069 95.7 -31\nHaniger, Mitch 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.991 1.052 0.909 70.7 23'}, {'pitcher_id': '605483', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnell, Blake 35 31 5 0 0 0 11 31.4 31.3 3 8.6 0.161 0.161 0.205 0.185 0.305 0.264 91.4 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 14 13 1 0 0 0 5 35.7 34.8 1 7.1 0.077 0.077 0.113 0.177 0.29 0.234 92.1 24.3\nRojas, Josh 7 6 2 0 0 0 1 14.3 25 1 14.3 0.333 0.333 0.351 0.152 0.183 0.224 92.4 3.6\nGarver, Mitch 8 7 2 0 0 0 2 25 30.8 1 12.5 0.286 0.286 0.304 0.334 0.623 0.437 93 26.8\nRobles, Victor 6 5 0 0 0 0 3 50 31.3 0 0 0 0 0.12 0.037 0.047 0.15 81.9 17.5'}, {'pitcher_id': '573124', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Taylor 13 12 4 1 1 0 3 23.1 21.1 0 0 0.333 0.583 0.356 0.301 0.386 0.291 85.9 26.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 4 4 1 0 0 0 2 50 40 0 0 0.25 0.25 0.221 0.176 0.197 0.161 83.7 42\nGarver, Mitch 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMoore, Dylan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.405 0.608 0.432 96.5 38.5\nCrawford, J.P. 3 3 1 0 1 0 0 0 0 0 0 0.333 1 0.534 0.386 0.517 0.39 84.5 -3\nHaniger, Mitch 1 0 0 0 0 0 0 0 0 0 0 0 0.207 88.2 23\nRodríguez, Julio 1 1 1 0 0 0 0 0 50 0 0 1 1 0.884 0.937 1.074 0.901 80.6 17\nRaleigh, Cal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.004 75.9 72'}, {'pitcher_id': '669062', 'player_data': '', 'team_data': 'Erik Miller has not faced anyone on this team.'}, {'pitcher_id': '687014', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666808', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoval, Camilo 9 8 2 1 0 0 2 22.2 38.1 0 0 0.25 0.375 0.236 0.242 0.331 0.223 93.7 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 5 5 1 0 0 0 1 20 18.2 0 0 0.2 0.2 0.177 0.162 0.188 0.15 95.8 0.5\nRobles, Victor 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nRaleigh, Cal 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.18 0.658 0.324 100.3 38\nRodríguez, Julio 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.946 1.054 0.88 85 17\nCrawford, J.P. 1 0 0 0 0 0 0 0 50 0 0 0 0.053 87.5 29'}, {'pitcher_id': '696131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676775', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinn, Keaton 6 5 2 1 0 0 1 16.7 25 1 16.7 0.4 0.6 0.47 0.213 0.354 0.315 91.6 -15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 3 2 1 1 0 0 1 33.3 33.3 1 33.3 0.5 1 0.647 0.135 0.448 0.389 96.7 32\nRodríguez, Julio 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.294 0.265 0.291 0.242 89.8 -30.7'}, {'pitcher_id': '643511', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Tyler 15 15 7 4 0 0 1 6.7 5.3 0 0 0.467 0.733 0.51 0.37 0.422 0.347 84.7 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 7 7 4 3 0 0 1 14.3 12.5 0 0 0.571 1 0.663 0.413 0.462 0.385 93.2 15.3\nHaniger, Mitch 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.036 0.02 91.7 41\nCrawford, J.P. 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.184 0.185 0.166 42.3 39\nRaley, Luke 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.663 0.719 0.597 81.5 22\nMoore, Dylan 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.621 0.458 0.576 0.445 94.3 18\nRaleigh, Cal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.58 0.685 0.557 102.2 3\nRodríguez, Julio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.123 0.13 0.114 77.7 -9'}, {'pitcher_id': '663546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHjelle, Sean 2 2 1 0 0 0 1 50 12.5 0 0 0.5 0.5 0.442 0.19 0.244 0.178 91.8 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZavala, Seby 2 2 1 0 0 0 1 50 12.5 0 0 0.5 0.5 0.442 0.19 0.244 0.178 91.8 19'}, {'pitcher_id': '678495', 'player_data': '', 'team_data': 'Randy Rodríguez has not faced anyone on this team.'}, {'pitcher_id': '592426', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJackson, Luke 10 8 4 2 0 1 1 10 23.5 2 20 0.5 1.125 0.669 0.395 0.865 0.558 88.8 6.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 7 6 3 1 0 1 0 0 10 1 14.3 0.5 1.167 0.679 0.457 1.018 0.625 86.9 4.8\nCrawford, J.P. 1 0 0 0 0 0 0 0 0 1 100 0.693 0.693\nRojas, Josh 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.42 0.813 0.519 100.7 19\nRaley, Luke 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0'}, {'pitcher_id': '676254', 'player_data': '', 'team_data': 'Ryan Walker has not faced anyone on this team.'}, {'pitcher_id': '657277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWebb, Logan 52 44 11 5 0 0 12 23.1 26.3 5 9.6 0.25 0.364 0.316 0.252 0.383 0.326 90.4 2.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 16 14 4 2 0 0 3 18.8 22.6 1 6.3 0.286 0.429 0.31 0.266 0.444 0.308 92.4 1.6\nMoore, Dylan 9 6 1 1 0 0 4 44.4 44.4 1 11.1 0.167 0.333 0.377 0.181 0.315 0.376 102 8.5\nCrawford, J.P. 9 8 2 1 0 0 2 22.2 16.7 1 11.1 0.25 0.375 0.313 0.167 0.293 0.254 90.3 -3.7\nZavala, Seby 3 3 1 0 0 0 0 0 18.2 0 0 0.333 0.333 0.295 0.49 0.577 0.473 87.5 19.7\nRobles, Victor 3 2 2 0 0 0 0 0 42.9 1 33.3 1 1 0.819 0.546 0.611 0.565 93.4 5.5\nRodríguez, Julio 4 3 0 0 0 0 0 0 20 1 25 0 0 0.174 0.452 0.764 0.553 101.3 0\nRaleigh, Cal 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.002 0.002 0.002 89.7 43\nHaniger, Mitch 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.414 0.082 0.092 0.094 61 4.5\nRaley, Luke 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.39 0.426 0.372 81.9 -32'}], 138: [{'pitcher_id': '678906', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '690986', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '694738', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663855', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHicks, Jordan 11 9 2 1 0 0 2 18.2 23.8 2 18.2 0.222 0.333 0.32 0.171 0.186 0.253 85.8 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 5 4 1 1 0 0 1 20 27.3 1 20 0.25 0.5 0.39 0.06 0.071 0.184 90.1 12\nGoldschmidt, Paul 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.155 0.172 0.14 98.5 -13\nArenado, Nolan 2 1 1 0 0 0 0 0 0 1 50 1 1 0.785 0.106 0.109 0.392 60.9 0\nCarpenter, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.726 0.764 0.657 72.5 17'}, {'pitcher_id': '605483', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnell, Blake 49 41 4 1 0 0 14 28.6 32.5 8 16.3 0.098 0.122 0.193 0.192 0.277 0.28 88.9 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 15 13 1 0 0 0 4 26.7 26.7 2 13.3 0.077 0.077 0.151 0.17 0.211 0.233 81.5 14.1\nGoldschmidt, Paul 9 6 2 1 0 0 2 22.2 30 3 33.3 0.333 0.5 0.469 0.314 0.396 0.437 88.4 21.3\nCrawford, Brandon 10 9 1 0 0 0 3 30 50 1 10 0.111 0.111 0.157 0.269 0.377 0.317 95 14.3\nCarpenter, Matt 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.122 0.435 0.224 89.6 40.3\nCarlson, Dylan 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.037 0.059 0.038 93.5 12\nGorman, Nolan 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.038 0.105 0.271 91.2 30\nDonovan, Brendan 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.235 0.235 0.45 96.2 -7\nWinn, Masyn 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.152 0.163 0.137 99.5 3'}, {'pitcher_id': '573124', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Taylor 15 14 3 0 0 1 5 33.3 28 1 6.7 0.214 0.429 0.302 0.209 0.327 0.261 84.5 26.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 4 3 0 0 0 0 2 50 50 1 25 0 0 0.172 0.059 0.065 0.213 64.5 9\nCarlson, Dylan 3 3 2 0 0 1 0 0 16.7 0 0 0.667 1.667 0.985 0.501 0.826 0.573 84.4 26.7\nArenado, Nolan 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.003 0.003 0.003 83.7 72\nBurleson, Alec 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nCrawford, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.337 0.463 0.345 91.9 14.5\nGorman, Nolan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDonovan, Brendan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.284 0.489 0.327 87.8 24'}, {'pitcher_id': '669062', 'player_data': '', 'team_data': 'Erik Miller has not faced anyone on this team.'}, {'pitcher_id': '687014', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666808', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoval, Camilo 17 14 3 1 0 0 5 29.4 24.4 2 11.8 0.214 0.286 0.301 0.204 0.289 0.301 90.8 5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 5 4 1 0 0 0 1 20 18.2 1 20 0.25 0.25 0.314 0.318 0.359 0.377 102.3 -4\nArenado, Nolan 4 4 0 0 0 0 3 75 42.9 0 0 0 0 0 0.009 0.009 0.009 73.1 -25\nCarpenter, Matt 2 2 1 1 0 0 1 50 20 0 0 0.5 1 0.622 0.021 0.021 0.019 77.9 45\nCarlson, Dylan 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.295 0.486 0.828 0.563 99.7 18\nGorman, Nolan 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.05 0.05 0.372 60.2 -14\nDonovan, Brendan 1 0 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '696131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676775', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinn, Keaton 11 9 1 1 0 0 2 18.2 35 1 9.1 0.111 0.222 0.242 0.154 0.313 0.288 91.8 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 3 3 1 1 0 0 1 33.3 37.5 0 0 0.333 0.667 0.415 0.063 0.105 0.068 86.8 26\nDonovan, Brendan 2 2 0 0 0 0 0 0 60 0 0 0 0 0 0.318 0.93 0.515 92.4 8\nGorman, Nolan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.24 0.267 0.218 91.6 6\nCarlson, Dylan 2 1 0 0 0 0 0 0 0 0 0 0 0 0.363 0.076 0.1 0.398 97.9 -37\nGoldschmidt, Paul 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nBurleson, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.003 94.5 50'}, {'pitcher_id': '643511', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Tyler 24 20 8 3 0 0 2 8.3 21.6 4 16.7 0.4 0.55 0.454 0.281 0.352 0.347 88.9 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 8 6 4 2 0 0 0 0 23.1 2 25 0.667 1 0.702 0.481 0.613 0.526 90.7 4.7\nArenado, Nolan 7 6 0 0 0 0 1 14.3 27.3 1 14.3 0 0 0.098 0.036 0.038 0.127 80 48\nGorman, Nolan 3 2 1 0 0 0 0 0 25 1 33.3 0.5 0.5 0.526 0.38 0.563 0.501 99.3 9.5\nCarpenter, Matt 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.621 0.459 0.529 0.432 97.4 12\nCarlson, Dylan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879\nDonovan, Brendan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.237 0.26 0.217 95.4 25.5\nBurleson, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.087 0.079 79.6 -43'}, {'pitcher_id': '663546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHjelle, Sean 4 1 0 0 0 0 0 0 0 1 25 0 0 0.474 0.166 0.166 0.524 56.5 -50.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurleson, Alec 2 0 0 0 0 0 0 0 0 1 50 0.696 0.696 46.7 -57\nDonovan, Brendan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.166 0.166 0.15 66.3 -44\nCarlson, Dylan 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '678495', 'player_data': '', 'team_data': 'Randy Rodríguez has not faced anyone on this team.'}, {'pitcher_id': '592426', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJackson, Luke 21 20 9 4 0 1 5 23.8 20 1 4.8 0.45 0.8 0.527 0.331 0.64 0.418 93.3 19.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 8 8 4 2 0 1 2 25 13.3 0 0 0.5 1.125 0.658 0.424 1.056 0.602 96.8 25.2\nArenado, Nolan 7 7 4 2 0 0 1 14.3 25 0 0 0.571 0.857 0.605 0.287 0.402 0.295 92.1 13.7\nCarpenter, Matt 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.232 0.155 0.17 0.328 92 0\nCrawford, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.454 0.6 0.454 87 27\nGorman, Nolan 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '676254', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Ryan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.045 0.057 0.041 89.8 -29', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nCarlson, Dylan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.114 0.081 89.8 -29'}, {'pitcher_id': '657277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWebb, Logan 63 59 14 0 0 2 18 28.6 25.2 4 6.3 0.237 0.339 0.275 0.241 0.412 0.303 93.1 -0.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 19 19 6 0 0 0 4 21.1 20.9 0 0 0.316 0.316 0.279 0.256 0.309 0.245 90.4 -0.9\nGoldschmidt, Paul 16 15 4 0 0 1 3 18.8 20 1 6.3 0.267 0.467 0.333 0.373 0.82 0.508 97.6 4\nCarlson, Dylan 11 10 0 0 0 0 3 27.3 25 1 9.1 0 0 0.063 0.156 0.185 0.199 90.5 -2.4\nGorman, Nolan 6 6 0 0 0 0 6 100 47.1 0 0 0 0 0 0\nDonovan, Brendan 5 4 2 0 0 0 1 20 25 1 20 0.5 0.5 0.491 0.214 0.233 0.295 97.5 -13.3\nCarpenter, Matt 2 1 1 0 0 0 0 0 20 1 50 1 1 0.78 0.164 0.167 0.419 90.1 -13\nBurleson, Alec 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.4 1.051 0.602 91.8 4.3\nGray, Sonny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}], 139: [{'pitcher_id': '678906', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTeng, Kai-Wei 9 9 3 0 0 1 0 0 9.1 0 0 0.333 0.667 0.429 0.347 0.587 0.395 86 17.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Isaac 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.128 0.388 0.218 86.2 46\nSiri, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.67 1.229 0.801 105.3 12.5\nArozarena, Randy 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.305 0.522 0.335 87 35\nRosario, Amed 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.261 0.281 0.241 66.6 -18\nDíaz, Yandy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.391 0.447 0.366 84.1 7'}, {'pitcher_id': '690986', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarrison, Kyle 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.221 0.165 0.195 0.159 70.6 -34.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.221 0.165 0.195 0.159 70.6 -34.7'}, {'pitcher_id': '694738', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRoupp, Landen 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0.28 0.657 0.387 95.6 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRortvedt, Ben 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.027 0.023 81.6 46\nCaballero, José 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.035 0.02 90.4 41\nSiri, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.72 2.786 1.41 104.2 33\nDíaz, Yandy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.394 0.424 0.355 104 -3\nArozarena, Randy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.197 0.219 0.18 105.2 -24\nPalacios, Richie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.334 0.451 0.336 88 21'}, {'pitcher_id': '663855', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHicks, Jordan 9 8 1 0 0 0 3 33.3 28.6 1 11.1 0.125 0.125 0.175 0.177 0.189 0.231 83.8 -15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.177 0.181 0.164 101.7 -19\nParedes, Isaac 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.043 0.043 0.369 84 -31\nDíaz, Yandy 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nLowe, Josh 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nWalls, Taylor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.96 1.037 0.889 74.3 23\nRosario, Amed 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.058 0.062 0.058 75.1 -34'}, {'pitcher_id': '605483', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnell, Blake 42 38 5 2 0 1 17 40.5 46.8 3 7.1 0.132 0.263 0.219 0.151 0.246 0.221 84.9 16.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 13 13 1 0 0 1 6 46.2 44.8 0 0 0.077 0.308 0.16 0.132 0.204 0.146 77.6 -2.3\nDíaz, Yandy 12 11 2 2 0 0 4 33.3 44.4 1 8.3 0.182 0.364 0.267 0.185 0.399 0.281 89.7 18.3\nArozarena, Randy 5 5 1 0 0 0 3 60 60 0 0 0.2 0.2 0.178 0.24 0.287 0.23 96.2 4.5\nParedes, Isaac 5 3 1 0 0 0 1 20 50 1 20 0.333 0.333 0.462 0.172 0.18 0.378 75.2 51\nWalls, Taylor 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.002 0.002 0.233 86.1 68\nSiri, Jose 4 4 0 0 0 0 2 50 37.5 0 0 0 0 0 0.072 0.088 0.066 91.6 24'}, {'pitcher_id': '573124', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Taylor 12 12 2 0 0 0 4 33.3 25 0 0 0.167 0.167 0.147 0.11 0.197 0.127 92.4 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 4 4 0 0 0 0 2 50 25 0 0 0 0 0 0.094 0.32 0.16 94.8 -1.5\nRosario, Amed 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.343 0.383 0.314 96 -6.5\nLowe, Brandon 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.01 0.024 0.015 94 49\nJackson, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.084 0.062 76.6 49\nParedes, Isaac 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nArozarena, Randy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.156 0.16 0.14 102.4 -26\nCaballero, José 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.034 0.022 84.4 62'}, {'pitcher_id': '669062', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Erik 3 3 0 0 0 0 2 66.7 22.2 0 0 0 0 0 0.016 0.033 0.022 79 35', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaballero, José 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.049 0.099 0.067 79 35\nSiri, Jose 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRortvedt, Ben 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '687014', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAvila, Nick 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.966 1.105 0.8 81.6 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Isaac 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nPalacios, Richie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.966 1.105 0.907 81.6 17'}, {'pitcher_id': '666808', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoval, Camilo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.969 1.096 0.932 78.1 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.969 1.096 0.932 78.1 19'}, {'pitcher_id': '696131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676775', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinn, Keaton 21 16 3 1 0 0 5 23.8 26.3 3 14.3 0.188 0.25 0.292 0.251 0.343 0.345 89.6 0.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.246 0.264 0.38 94.3 -10\nPalacios, Richie 3 3 0 0 0 0 2 66.7 22.2 0 0 0 0 0 0.007 0.01 0.007 85.8 60\nArozarena, Randy 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.198 0.231 0.354 94.3 -14.5\nRortvedt, Ben 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nSiri, Jose 2 1 0 0 0 0 1 50 100 0 0 0 0 0 0\nRosario, Amed 3 3 2 1 0 0 1 33.3 16.7 0 0 0.667 1 0.719 0.391 0.467 0.37 81.8 -25.5\nParedes, Isaac 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0.241 0.321 0.784 0.545 97.5 8\nCaballero, José 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.444 0.647 0.748 0.616 82.1 12.5'}, {'pitcher_id': '643511', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Tyler 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.459 0.503 0.428 98.7 3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Isaac 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.318 0.318 0.289 93.3 0\nRosario, Amed 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.6 0.688 0.567 104 6'}, {'pitcher_id': '663546', 'player_data': '', 'team_data': 'Sean Hjelle has not faced anyone on this team.'}, {'pitcher_id': '678495', 'player_data': '', 'team_data': 'Randy Rodríguez has not faced anyone on this team.'}, {'pitcher_id': '592426', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJackson, Luke 13 13 3 0 0 1 5 38.5 32 0 0 0.231 0.462 0.289 0.155 0.387 0.225 91.1 -12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 6 6 2 0 0 0 2 33.3 27.3 0 0 0.333 0.333 0.292 0.118 0.162 0.122 90.2 -13\nArozarena, Randy 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.038 0.04 0.035 80.3 -41\nSiri, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.102 0.109 0.098 90.8 -23\nParedes, Isaac 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.173 0.138 91 -14\nDíaz, Yandy 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.931 3.665 1.849 105.8 28'}, {'pitcher_id': '676254', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Ryan 13 11 2 0 0 0 7 53.8 53.1 2 15.4 0.182 0.182 0.243 0.175 0.206 0.248 80.7 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Isaac 3 3 1 0 0 0 2 66.7 75 0 0 0.333 0.333 0.294 0.313 0.351 0.292 81.1 19\nDíaz, Yandy 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nLowe, Brandon 1 0 0 0 0 0 0 0 66.7 1 100 0.696 0.696\nLowe, Josh 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nRosario, Amed 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nArozarena, Randy 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.049 0.053 0.373 73.1 -29\nDeLuca, Jonny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.011 0.017 0.014 65.8 59\nSiri, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.923 1.141 0.894 102.6 11\nCaballero, José 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '657277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWebb, Logan 23 21 5 0 0 0 4 17.4 19.4 2 8.7 0.238 0.238 0.253 0.299 0.392 0.333 84.9 1.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 5 4 0 0 0 0 1 20 18.2 1 20 0 0 0.138 0.091 0.101 0.205 85.9 -18.7\nRortvedt, Ben 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.296 0.33 0.441 0.332 99.4 1.5\nSiri, Jose 3 2 1 0 0 0 1 33.3 66.7 1 33.3 0.5 0.5 0.527 0.213 0.232 0.359 93.4 3\nDíaz, Yandy 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.296 0.442 0.476 0.408 88.8 16\nParedes, Isaac 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.181 0.197 0.165 62.2 5\nCaballero, José 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.489 0.271 78.1 2.7\nPalacios, Richie 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.7 0.852 0.672 98.5 4.7'}], 140: [{'pitcher_id': '678906', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '690986', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '694738', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663855', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHicks, Jordan 7 6 1 1 0 0 2 28.6 27.3 1 14.3 0.167 0.333 0.277 0.191 0.413 0.313 95.5 21.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 2 2 1 1 0 0 0 0 20 0 0 0.5 1 0.622 0.151 0.385 0.218 98.2 33.5\nLowe, Nathaniel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSemien, Marcus 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.1 0.102 0.092 81.6 -1\nGrossman, Robbie 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nTaveras, Leody 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.643 1.504 0.875 104 21'}, {'pitcher_id': '605483', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSnell, Blake 55 49 11 2 0 0 24 43.6 39.5 6 10.9 0.224 0.265 0.265 0.166 0.209 0.22 81.1 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 12 9 4 2 0 0 3 25 17.4 3 25 0.444 0.667 0.528 0.204 0.234 0.316 80.3 13\nSeager, Corey 13 13 2 0 0 0 6 46.2 44.8 0 0 0.154 0.154 0.136 0.193 0.296 0.208 88.3 22.4\nSemien, Marcus 11 9 1 0 0 0 3 27.3 28.6 2 18.2 0.111 0.111 0.206 0.184 0.218 0.27 80.7 4.5\nKnizner, Andrew 5 5 0 0 0 0 4 80 81.8 0 0 0 0 0 0.005 0.007 0.005 81.1 41\nLowe, Nathaniel 3 3 1 0 0 0 2 66.7 22.2 0 0 0.333 0.333 0.294 0.232 0.251 0.21 88.5 11\nDuran, Ezequiel 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nGarcía, Adolis 3 3 1 0 0 0 1 33.3 44.4 0 0 0.333 0.333 0.294 0.113 0.116 0.103 64.1 7.5\nHuff, Sam 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nTaveras, Leody 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.121 0.123 0.112 57.3 -69\nJankowski, Travis 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.439 0.42 0.473 0.386 87.5 16'}, {'pitcher_id': '573124', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Taylor 22 20 1 0 0 0 10 45.5 44.4 2 9.1 0.05 0.05 0.103 0.101 0.115 0.15 83.9 -10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 4 3 0 0 0 0 3 75 37.5 1 25 0 0 0.172 0.172\nLowe, Nathaniel 4 3 0 0 0 0 2 50 66.7 1 25 0 0 0.173 0.073 0.075 0.222 99.2 -15\nGarcía, Adolis 4 4 0 0 0 0 3 75 54.5 0 0 0 0 0 0.034 0.037 0.03 106.8 -24\nSemien, Marcus 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.086 0.121 0.091 87.6 -0.3\nSeager, Corey 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.036 0.038 0.033 81.2 15.5\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.301 0.307 0.279 71.2 -49\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.234 0.234 0.211 32.3 -53\nSmith, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.686 0.776 0.637 104.4 8'}, {'pitcher_id': '669062', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiller, Erik 4 4 1 0 0 0 2 50 22.2 0 0 0.25 0.25 0.222 0.089 0.098 0.083 82.9 -14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.296 0.328 0.278 99.2 -10\nSmith, Josh 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nDuran, Ezequiel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.059 0.065 0.053 66.5 -18\nLowe, Nathaniel 1 1 0 0 0 0 1 100 0 0 0 0 0 0'}, {'pitcher_id': '687014', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '666808', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDoval, Camilo 15 12 2 0 0 0 4 26.7 33.3 3 20 0.167 0.167 0.256 0.106 0.189 0.237 82.8 12.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 2 1 0 0 0 0 1 50 40 1 50 0 0 0.346 0.346\nTaveras, Leody 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.007 0.015 0.009 84.5 35\nJankowski, Travis 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.07 0.072 0.063 68.2 -40\nSeager, Corey 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.322 0.764 0.447 100.9 11\nSemien, Marcus 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nSmith, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.206 0.254 0.194 72.2 45\nGrossman, Robbie 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nKnizner, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarcía, Adolis 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.055 0.062 0.05 72.2 -11\nDuran, Ezequiel 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.111 0.123 0.102 81.9 25'}, {'pitcher_id': '696131', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676775', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinn, Keaton 20 17 6 1 1 1 6 30 36.8 2 10 0.353 0.706 0.487 0.234 0.377 0.326 97 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Josh 3 1 0 0 0 0 1 33.3 75 2 66.7 0 0 0.461 0.461\nDuran, Ezequiel 2 2 0 0 0 0 2 100 22.2 0 0 0 0 0 0\nLowe, Nathaniel 3 3 1 1 0 0 1 33.3 37.5 0 0 0.333 0.667 0.423 0.238 0.389 0.263 99.4 1.5\nTaveras, Leody 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.21 0.459 0.285 101.7 19\nJankowski, Travis 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.444 0.229 0.276 0.226 99.4 2\nGarcía, Adolis 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.325 0.462 0.337 90 6.3\nSemien, Marcus 3 2 1 0 0 1 0 0 33.3 0 0 0.5 2 0.936 0.112 0.266 0.342 96.5 33.5\nLangford, Wyatt 2 2 2 0 1 0 0 0 0 0 0 1 2 1.251 0.592 0.93 0.644 102.2 13.5'}, {'pitcher_id': '643511', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Tyler 23 21 3 1 0 0 4 17.4 22.7 0 0 0.143 0.19 0.162 0.194 0.297 0.233 87.8 1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 4 4 1 1 0 0 0 0 18.2 0 0 0.25 0.5 0.31 0.211 0.428 0.27 80.2 7.5\nSeager, Corey 9 8 0 0 0 0 2 22.2 23.1 0 0 0 0 0.081 0.143 0.187 0.213 92 -9.2\nGrossman, Robbie 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.442 0.361 0.382 0.326 76 -9\nSmith, Josh 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.861 0.546 102.4 5.5\nGarcía, Adolis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.104 0.132 0.103 82.1 17\nKnizner, Andrew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTaveras, Leody 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nLowe, Nathaniel 1 0 0 0 0 0 0 0 0 0 0 0 0.179 93.4 35\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.237 0.284 0.228 94.4 -8'}, {'pitcher_id': '663546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHjelle, Sean 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.068 0.061 54.8 -27', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.068 0.061 54.8 -27'}, {'pitcher_id': '678495', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Randy 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.178 0.159 0.182 0.147 73.2 -9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTaveras, Leody 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.058 0.029 95.5 44\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.052 0.058 0.047 73.2 -27\nLangford, Wyatt 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.292 0.297 0.263 34.1 -59\nDuran, Ezequiel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.439 0.496 0.398 90 3'}, {'pitcher_id': '592426', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJackson, Luke 16 15 3 0 0 1 3 18.8 22.2 1 6.3 0.2 0.4 0.279 0.277 0.461 0.339 89.2 26.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 3 3 2 0 0 1 1 33.3 0 0 0 0.667 1.667 0.964 0.478 1.025 0.626 88.6 27\nSeager, Corey 4 3 0 0 0 0 0 0 16.7 1 25 0 0 0.173 0.225 0.258 0.334 92.5 6.3\nGrossman, Robbie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.439 0.416 0.444 0.385 84.6 37\nJankowski, Travis 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.225 0.26 0.212 98 -1\nGray, Jon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTaveras, Leody 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.013 0.01 88.8 44\nLangford, Wyatt 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 59.1 76\nSmith, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.731 1.556 0.959 108.5 18\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.082 0.043 91.7 37'}, {'pitcher_id': '676254', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Ryan 7 7 1 0 0 0 0 0 7.7 0 0 0.143 0.143 0.126 0.157 0.198 0.154 90.3 1.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.01 0.006 88.8 41\nTaveras, Leody 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.144 0.153 0.132 100.3 -25.5\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.086 0.054 80.5 31\nSmith, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.504 0.418 99.5 2\nKnizner, Andrew 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.08 0.259 0.139 92.1 31\nSemien, Marcus 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.213 0.217 0.195 70.8 -46'}, {'pitcher_id': '657277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWebb, Logan 86 80 16 4 0 0 13 15.1 15.9 3 3.5 0.2 0.25 0.223 0.295 0.395 0.327 88 1.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 16 15 4 0 0 0 1 6.3 13 1 6.3 0.267 0.267 0.265 0.356 0.419 0.359 83 9.7\nGrossman, Robbie 10 8 1 0 0 0 2 20 6.7 1 10 0.125 0.125 0.157 0.234 0.31 0.334 90.9 16.7\nLowe, Nathaniel 10 9 1 0 0 0 2 20 21.7 1 10 0.111 0.111 0.158 0.206 0.233 0.243 90.5 -12.7\nSeager, Corey 17 17 4 2 0 0 1 5.9 18.2 0 0 0.235 0.353 0.25 0.492 0.685 0.506 87.4 -0.7\nGarcía, Adolis 7 7 0 0 0 0 3 42.9 30 0 0 0 0 0 0.196 0.374 0.24 92.8 11.8\nSmith, Josh 6 6 0 0 0 0 2 33.3 30 0 0 0 0 0 0.077 0.082 0.07 78.4 -27.5\nTaveras, Leody 5 5 0 0 0 0 1 20 0 0 0 0 0 0 0.254 0.431 0.29 95.3 3\nDuran, Ezequiel 6 5 3 0 0 0 0 0 14.3 0 0 0.6 0.6 0.564 0.329 0.364 0.375 90.7 -29\nLangford, Wyatt 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.423 0.141 0.148 0.127 99.8 20.5\nKnizner, Andrew 5 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.573 0.232 0.353 0.344 88.9 27\nGray, Jon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.104 0.116 0.094 78.7 -2'}]}, 138: {136: [{'pitcher_id': '676617', 'player_data': '', 'team_data': ''}, {'pitcher_id': '571927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatz, Steven 41 37 10 0 0 1 10 24.4 21.8 4 9.8 0.27 0.351 0.31 0.252 0.407 0.321 85.9 15.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 12 10 3 0 0 1 3 25 30.4 2 16.7 0.3 0.6 0.43 0.327 0.544 0.424 80.9 15.4\nCrawford, J.P. 9 9 3 0 0 0 3 33.3 7.7 0 0 0.333 0.333 0.293 0.118 0.129 0.109 77.5 -4.3\nHaniger, Mitch 8 8 0 0 0 0 2 25 11.1 0 0 0 0 0 0.18 0.466 0.263 92.9 12.8\nRodríguez, Julio 3 2 1 0 0 0 0 0 50 1 33.3 0.5 0.5 0.526 0.37 0.593 0.502 85.2 31.5\nZavala, Seby 2 2 0 0 0 0 2 100 37.5 0 0 0 0 0 0\nMoore, Dylan 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.524 0.468 0.582 0.536 92.6 27\nRaleigh, Cal 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.025 0.073 0.038 98.1 55.5\nGarver, Mitch 2 2 2 0 0 0 0 0 0 0 0 1 1 0.879 0.924 1.111 0.889 89.4 16.5'}, {'pitcher_id': '667463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, John 16 15 5 1 0 0 0 0 14.3 0 0 0.333 0.4 0.298 0.416 0.614 0.418 90 10.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 5 4 1 0 0 0 0 0 11.1 0 0 0.25 0.25 0.177 0.294 0.715 0.35 86.2 12\nMoore, Dylan 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.707 0.611 0.733 0.583 89.6 15\nRodríguez, Julio 2 2 2 0 0 0 0 0 0 0 0 1 1 0.884 0.611 0.767 0.595 105.7 5.5\nHaniger, Mitch 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.503 0.878 0.576 102.3 17\nGarver, Mitch 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.075 0.081 0.068 68.3 -8\nRaley, Luke 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.362 0.418 0.348 94.9 -7\nZavala, Seby 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.517 0.566 0.466 94.6 7\nRaleigh, Cal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.248 0.192 73.2 45'}, {'pitcher_id': '688297', 'player_data': '', 'team_data': 'Chris Roycroft has not faced anyone on this team.'}, {'pitcher_id': '681517', 'player_data': '', 'team_data': 'Kyle Leahy has not faced anyone on this team.'}, {'pitcher_id': '681676', 'player_data': '', 'team_data': 'Ryan Fernandez has not faced anyone on this team.'}, {'pitcher_id': '543243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Sonny 42 34 6 0 0 1 10 23.8 36.9 4 9.5 0.176 0.265 0.288 0.228 0.298 0.321 84.2 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 8 7 1 0 0 0 3 37.5 43.8 1 12.5 0.143 0.143 0.197 0.204 0.24 0.258 86.1 6\nRodríguez, Julio 7 6 0 0 0 0 4 57.1 41.7 1 14.3 0 0 0.099 0.078 0.08 0.161 100.7 -19\nZavala, Seby 6 5 1 0 0 0 2 33.3 58.3 1 16.7 0.2 0.2 0.263 0.131 0.15 0.217 90.3 27\nGarver, Mitch 5 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.69 0.271 0.706 0.524 87.6 7\nCrawford, J.P. 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.177 0.342 0.398 0.326 75.9 36.3\nRaley, Luke 4 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.583 0.433 0.476 0.566 95.6 1\nRaleigh, Cal 3 2 1 0 0 0 0 0 25 1 33.3 0.5 0.5 0.526 0.47 0.587 0.537 95.5 -2\nRobles, Victor 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.321 0.372 0.332 65.1 12.5\nMoore, Dylan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.117 0.109 62.6 -12'}, {'pitcher_id': '571945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMikolas, Miles 36 33 9 2 0 0 6 16.7 22.7 1 2.8 0.273 0.333 0.287 0.239 0.278 0.252 83.6 11.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 11 10 4 0 0 0 2 18.2 25 0 0 0.4 0.4 0.383 0.4 0.452 0.401 81.1 10.8\nCrawford, J.P. 4 3 1 1 0 0 1 25 16.7 1 25 0.333 0.667 0.486 0.299 0.364 0.39 96.8 24.5\nRojas, Josh 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.42 0.174 0.256 0.178 93.1 18.7\nRaleigh, Cal 3 3 1 0 0 0 0 0 12.5 0 0 0.333 0.333 0.294 0.381 0.422 0.354 93.6 19\nCastillo, Luis 5 4 1 0 0 0 2 40 50 0 0 0.25 0.25 0.22 0.111 0.122 0.108 66.6 -9.7\nRodríguez, Julio 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.095 0.102 0.092 73.8 -14.7\nRaley, Luke 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.135 0.172 0.121 82.1 39\nGarver, Mitch 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.091 0.076 84.4 -15\nZavala, Seby 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.011 0.025 0.017 91.8 46.5'}, {'pitcher_id': '687798', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, Nick 1 0 0 0 0 0 0 0 1 100 0.696 0.696', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '502043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGibson, Kyle 82 77 23 5 0 3 19 23.2 27.8 4 4.9 0.299 0.481 0.352 0.341 0.533 0.393 90 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 12 12 5 2 0 0 1 8.3 7.4 0 0 0.417 0.583 0.427 0.315 0.383 0.303 84.8 17.9\nCrawford, J.P. 12 12 6 2 0 0 1 8.3 25 0 0 0.5 0.667 0.498 0.491 0.591 0.469 85.5 2.5\nGarver, Mitch 10 10 2 0 0 0 3 30 37.5 0 0 0.2 0.2 0.176 0.24 0.358 0.257 90.5 17.1\nHaniger, Mitch 13 13 1 0 0 1 5 38.5 26.1 0 0 0.077 0.308 0.149 0.228 0.555 0.322 92.7 -7.1\nMoore, Dylan 6 4 2 0 0 0 1 16.7 28.6 2 33.3 0.5 0.5 0.527 0.398 0.439 0.478 85.5 6\nRobles, Victor 9 8 1 0 0 0 2 22.2 42.1 0 0 0.125 0.125 0.177 0.342 0.417 0.375 92.4 13.2\nRaley, Luke 6 4 1 0 0 0 2 33.3 40 2 33.3 0.25 0.25 0.379 0.381 0.417 0.468 88.4 14\nRodríguez, Julio 6 6 2 1 0 1 2 33.3 30.8 0 0 0.333 1 0.541 0.38 0.818 0.494 105 10.3\nRaleigh, Cal 5 5 2 0 0 1 2 40 44.4 0 0 0.4 1 0.577 0.378 0.995 0.565 101.4 31.7\nCanzone, Dominic 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.401 0.637 0.445 87.1 -3.7'}, {'pitcher_id': '669467', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPallante, Andre 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.524 0.251 0.29 0.38 91.6 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCanzone, Dominic 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.128 0.149 0.115 79.9 -7\nRojas, Josh 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nHaniger, Mitch 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.374 0.431 0.337 103.3 -7'}, {'pitcher_id': '668941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomero, JoJo 4 2 0 0 0 0 1 25 40 1 25 0 0 0.232 0.147 0.162 0.323 65.1 -38.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCrawford, J.P. 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nRodríguez, Julio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.295 0.324 0.272 86.2 0\nRobles, Victor 1 0 0 0 0 0 0 0 0 0 0 44 -77'}, {'pitcher_id': '668868', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThompson, Zack 3 2 1 0 0 0 1 33.3 20 1 33.3 0.5 0.5 0.526 0.477 0.587 0.542 98.3 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nRojas, Josh 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCrawford, J.P. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.954 1.173 0.929 98.3 13'}, {'pitcher_id': '606149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallegos, Giovanny 9 9 0 0 0 0 4 44.4 41.2 0 0 0 0 0 0.064 0.108 0.072 83.8 49', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 4 4 0 0 0 0 2 50 33.3 0 0 0 0 0 0.004 0.008 0.005 80.4 60.5\nRaleigh, Cal 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.007 0.006 74.8 76\nCrawford, J.P. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCanzone, Dominic 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.083 0.241 0.13 91.5 29\nRodríguez, Julio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.466 0.693 0.49 91.9 19'}, {'pitcher_id': '669461', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLiberatore, Matthew 5 4 2 2 0 0 1 20 16.7 1 20 0.5 1 0.637 0.495 1.08 0.665 101.1 6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 3 3 1 1 0 0 1 33.3 20 0 0 0.333 0.667 0.415 0.345 0.397 0.323 95.1 -0.5\nGarver, Mitch 2 1 1 1 0 0 0 0 0 1 50 1 2 0.97 0.946 3.128 1.179 113.2 20'}, {'pitcher_id': '552640', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKittredge, Andrew 13 10 2 0 0 0 5 38.5 42.9 2 15.4 0.2 0.2 0.298 0.101 0.151 0.246 87.8 7.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0.068 0.068 0.273 96.9 -17\nRobles, Victor 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nCrawford, J.P. 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.345 0.062 0.082 0.374 74.8 -15\nRodríguez, Julio 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.299 0.504 0.343 97 13.5\nCanzone, Dominic 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRaleigh, Cal 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nMoore, Dylan 2 1 1 0 0 0 0 0 0 0 0 1 1 0.803 0.217 0.288 0.474 73.3 44'}, {'pitcher_id': '664854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHelsley, Ryan 11 10 1 0 0 0 3 27.3 20 1 9.1 0.1 0.1 0.142 0.146 0.171 0.189 86.9 32.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 4 4 1 0 0 0 1 25 14.3 0 0 0.25 0.25 0.217 0.226 0.281 0.221 82.2 35.3\nGarver, Mitch 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.011 0.013 0.01 93.2 63.5\nRojas, Josh 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.263 0.275 0.388 87.7 -4\nRaleigh, Cal 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '458681', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynn, Lance 53 48 8 2 0 0 22 41.5 32.8 4 7.5 0.167 0.208 0.212 0.135 0.164 0.184 84.6 14.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 17 15 2 0 0 0 7 41.2 30 2 11.8 0.133 0.133 0.184 0.122 0.144 0.188 87.4 4.3\nHaniger, Mitch 14 14 0 0 0 0 9 64.3 55.2 0 0 0 0 0 0.019 0.024 0.019 77.2 31.8\nRodríguez, Julio 6 5 2 1 0 0 1 16.7 13.3 1 16.7 0.4 0.6 0.471 0.339 0.427 0.392 84.1 16.3\nGarver, Mitch 3 2 0 0 0 0 2 66.7 20 1 33.3 0 0 0.231 0.231\nRaleigh, Cal 6 6 3 1 0 0 1 16.7 30 0 0 0.5 0.667 0.504 0.184 0.227 0.171 88.1 17.6\nRaley, Luke 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.215 0.231 0.194 105.7 -2\nMoore, Dylan 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.24 0.123 0.201 0.332 84.8 23\nCastillo, Luis 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.438 0.377 0.42 0.345 71 10.5'}], 137: [{'pitcher_id': '676617', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '571927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatz, Steven 61 54 17 4 1 1 12 19.7 24.8 7 11.5 0.315 0.481 0.38 0.277 0.364 0.325 89 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSlater, Austin 13 11 4 1 0 0 1 7.7 12.5 2 15.4 0.364 0.455 0.405 0.243 0.308 0.31 86.5 2.1\nAhmed, Nick 13 12 3 1 1 0 1 7.7 25 1 7.7 0.25 0.5 0.338 0.29 0.403 0.323 92.8 15.7\nConforto, Michael 8 7 0 0 0 0 3 37.5 26.7 1 12.5 0 0 0.086 0.106 0.119 0.171 81.4 -15\nFlores, Wilmer 8 7 3 0 0 1 1 12.5 20 1 12.5 0.429 0.857 0.564 0.355 0.588 0.443 89 23.8\nChapman, Matt 5 5 1 1 0 0 3 60 40 0 0 0.2 0.4 0.248 0.163 0.175 0.15 78.1 -29.5\nSoler, Jorge 6 4 2 1 0 0 1 16.7 57.1 2 33.3 0.5 0.75 0.587 0.398 0.446 0.478 85.2 20\nEstrada, Thairo 5 5 3 0 0 0 0 0 25 0 0 0.6 0.6 0.53 0.491 0.561 0.463 95.1 -1\nYastrzemski, Mike 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.442 0.357 0.512 0.376 105.5 14\nBailey, Patrick 1 1 0 0 0 0 1 100 0 0 0 0 0 0'}, {'pitcher_id': '667463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, John 10 7 1 0 0 0 2 20 22.2 3 30 0.143 0.143 0.295 0.202 0.215 0.338 94 -12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 2 1 0 0 0 0 1 50 25 1 50 0 0 0.346 0.346\nChapman, Matt 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.231 0.266 0.272 0.392 90.2 -21.5\nSoler, Jorge 2 2 1 0 0 0 1 50 28.6 0 0 0.5 0.5 0.44 0.273 0.298 0.248 107.8 3\nSlater, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.171 0.189 0.433 82.7 -3\nYastrzemski, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.166 0.177 0.149 99.3 -19'}, {'pitcher_id': '688297', 'player_data': '', 'team_data': 'Chris Roycroft has not faced anyone on this team.'}, {'pitcher_id': '681517', 'player_data': '', 'team_data': 'Kyle Leahy has not faced anyone on this team.'}, {'pitcher_id': '681676', 'player_data': '', 'team_data': 'Ryan Fernandez has not faced anyone on this team.'}, {'pitcher_id': '543243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Sonny 72 59 13 2 0 4 15 20.8 22.8 11 15.3 0.22 0.458 0.352 0.216 0.397 0.333 85.5 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 13 9 3 1 0 1 3 23.1 27.8 4 30.8 0.333 0.778 0.531 0.222 0.483 0.416 90 13.7\nConforto, Michael 11 9 3 1 0 1 3 27.3 29.4 2 18.2 0.333 0.778 0.503 0.397 0.715 0.505 98.7 13.5\nFlores, Wilmer 10 9 2 0 0 1 1 10 12.5 1 10 0.222 0.556 0.358 0.257 0.412 0.326 81.9 12.5\nSoler, Jorge 12 10 1 0 0 0 3 25 44.4 1 8.3 0.1 0.1 0.191 0.137 0.154 0.235 67.2 14.5\nYastrzemski, Mike 7 7 3 0 0 1 2 28.6 18.2 0 0 0.429 0.857 0.538 0.238 0.744 0.398 89.4 29\nEstrada, Thairo 5 4 0 0 0 0 1 20 30 1 20 0 0 0.138 0.122 0.13 0.227 83.2 -4\nCasali, Curt 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.229 0.034 0.035 0.249 74 27\nSlater, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.549 0.605 0.596 92.8 8\nWisely, Brett 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.441 0.023 0.024 0.023 90.4 -38\nAhmed, Nick 3 2 0 0 0 0 0 0 20 0 0 0 0 0 0.056 0.071 0.061 85.3 5\nBailey, Patrick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.175 0.195 0.158 102.6 -5\nWebb, Logan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.082 0.072 92.4 25'}, {'pitcher_id': '571945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMikolas, Miles 56 51 14 3 0 2 10 17.9 17.2 4 7.1 0.275 0.451 0.343 0.254 0.437 0.33 89.8 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 13 13 1 1 0 0 5 38.5 26.9 0 0 0.077 0.154 0.096 0.151 0.373 0.216 92.4 29.9\nYastrzemski, Mike 9 8 2 1 0 0 2 22.2 10 1 11.1 0.25 0.375 0.313 0.16 0.267 0.236 91.7 20.5\nEstrada, Thairo 6 4 1 0 0 0 0 0 0 1 16.7 0.25 0.25 0.383 0.463 0.539 0.532 77.5 -21\nFlores, Wilmer 6 5 2 0 0 0 1 16.7 15.4 1 16.7 0.4 0.4 0.411 0.145 0.185 0.228 88 20\nConforto, Michael 6 6 0 0 0 0 1 16.7 33.3 0 0 0 0 0 0.253 0.569 0.335 92.7 9.6\nChapman, Matt 5 4 2 0 0 0 0 0 20 1 20 0.5 0.5 0.491 0.374 0.496 0.439 88.5 23\nCasali, Curt 7 7 3 1 0 0 1 14.3 25 0 0 0.429 0.571 0.432 0.557 0.663 0.526 92.2 15\nAhmed, Nick 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.985 0.283 0.602 0.378 90 12\nSlater, Austin 1 1 1 0 0 1 0 0 0 0 0 1 4 1.94 0.287 1.069 0.549 97.8 35'}, {'pitcher_id': '687798', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, Nick 4 4 3 1 0 0 1 25 33.3 0 0 0.75 1 0.752 0.334 0.389 0.32 93.9 -20.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBailey, Patrick 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.866 1.086 0.854 103.2 10\nYastrzemski, Mike 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSlater, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.3 0.302 0.27 87.5 -64\nConforto, Michael 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.168 0.168 0.155 90.9 -8'}, {'pitcher_id': '502043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGibson, Kyle 98 82 17 5 0 5 23 23.5 29.9 14 14.3 0.207 0.451 0.339 0.24 0.442 0.356 88.2 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 31 23 2 0 0 2 11 35.5 52 7 22.6 0.087 0.348 0.293 0.188 0.458 0.366 93.6 4.3\nChapman, Matt 25 23 2 1 0 1 10 40 33.3 1 4 0.087 0.261 0.19 0.165 0.261 0.229 90.2 15.5\nConforto, Michael 12 9 5 1 0 1 0 0 18.2 3 25 0.556 1 0.655 0.484 0.883 0.603 90.6 19.9\nFlores, Wilmer 12 11 3 1 0 1 1 8.3 13 1 8.3 0.273 0.636 0.409 0.234 0.449 0.331 81.8 22.1\nYastrzemski, Mike 8 6 3 2 0 0 0 0 13.3 2 25 0.5 0.833 0.596 0.514 0.762 0.586 90.1 18.8\nAhmed, Nick 4 4 1 0 0 0 0 0 8.3 0 0 0.25 0.25 0.221 0.174 0.189 0.164 80.9 -21\nCasali, Curt 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.178 0.073 0.073 0.068 67.1 -8\nEstrada, Thairo 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '669467', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPallante, Andre 13 10 3 0 0 1 3 23.1 14.3 3 23.1 0.3 0.6 0.455 0.246 0.438 0.393 92.5 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSlater, Austin 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.344 0.344\nFlores, Wilmer 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.394 0.666 0.568 98.3 18\nChapman, Matt 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.589 0.4 0.506 0.399 112.7 8\nCasali, Curt 2 1 0 0 0 0 1 50 0 1 50 0 0 0.344 0.344\nAhmed, Nick 1 1 1 0 0 1 0 0 50 0 0 1 4 2.072 0.706 1.921 1.1 103 23\nConforto, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.054 0.053 77.4 -21\nYastrzemski, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.211 0.211 0.195 85.1 -1\nEstrada, Thairo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.076 0.055 78.5 51'}, {'pitcher_id': '668941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomero, JoJo 10 8 2 1 0 0 3 30 38.5 1 10 0.25 0.375 0.354 0.24 0.377 0.351 86.8 8.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 5 3 1 0 0 0 2 40 42.9 1 20 0.333 0.333 0.459 0.302 0.365 0.459 91.4 16\nSlater, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nYastrzemski, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.038 0.024 80.6 37\nAhmed, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.451 0.852 0.544 86 -19.5\nCasali, Curt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.089 0.18 0.106 90 27'}, {'pitcher_id': '668868', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThompson, Zack 4 4 2 0 0 1 1 25 12.5 0 0 0.5 1.25 0.722 0.481 0.835 0.56 95.2 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSoler, Jorge 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.423 0.466 0.385 82.1 9\nFlores, Wilmer 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.546 1.622 0.9 100.4 25\nConforto, Michael 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.954 1.251 0.956 103 12'}, {'pitcher_id': '606149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallegos, Giovanny 26 22 5 1 0 1 9 34.6 36.8 3 11.5 0.227 0.409 0.334 0.29 0.586 0.416 94 20.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYastrzemski, Mike 6 5 1 0 0 1 4 66.7 66.7 1 16.7 0.2 0.8 0.449 0.181 0.719 0.416 107 31\nSoler, Jorge 7 7 2 1 0 0 2 28.6 26.7 0 0 0.286 0.429 0.302 0.392 0.892 0.528 99.6 28.4\nCasali, Curt 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.233 0.247 0.265 0.381 88.6 8\nConforto, Michael 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0 0 0.232 80 69\nFlores, Wilmer 2 1 1 0 0 0 0 0 14.3 0 0 1 1 0.803 0.551 0.678 0.628 85.6 20\nEstrada, Thairo 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.295 0.253 0.286 0.229 100.6 -2\nSlater, Austin 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.203 0.206 0.183 94.7 -14\nAhmed, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.72 0.777 0.663 66.8 19'}, {'pitcher_id': '669461', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLiberatore, Matthew 16 13 6 1 0 0 3 18.8 14.3 2 12.5 0.462 0.538 0.486 0.367 0.719 0.503 93.3 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.232 0.063 0.214 0.306 94.8 35\nFlores, Wilmer 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.496 1.227 0.715 88.6 26.7\nBailey, Patrick 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.661 1.359 0.793 101.5 19\nEstrada, Thairo 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.242 0.029 0.096 0.276 93.2 35\nSlater, Austin 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.588 0.267 0.403 0.284 96.5 -3\nSoler, Jorge 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.974 1.111 0.92 83.4 16'}, {'pitcher_id': '552640', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKittredge, Andrew 8 6 2 0 0 0 0 0 36.4 2 25 0.333 0.333 0.392 0.276 0.318 0.372 86.7 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 5 4 2 0 0 0 0 0 37.5 1 20 0.5 0.5 0.489 0.3 0.344 0.369 87.4 23.5\nConforto, Michael 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nFlores, Wilmer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.505 0.424 69.5 13\nSoler, Jorge 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.014 0.025 0.018 101.2 58'}, {'pitcher_id': '664854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHelsley, Ryan 21 19 3 1 0 0 3 14.3 20 2 9.5 0.158 0.211 0.208 0.255 0.418 0.324 84.6 18.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.374 0.509 0.456 89 22\nSlater, Austin 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.344 0.344\nAhmed, Nick 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.217 0.174 0.187 0.157 83.1 22.7\nConforto, Michael 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.356 0.538 0.376 78.5 -31\nCasali, Curt 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.117 0.177 0.127 83.3 25\nSoler, Jorge 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.193 0.22 0.179 72.4 -2\nEstrada, Thairo 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.366 0.415 0.338 83 39\nYastrzemski, Mike 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.223 0.461 0.282 90.3 44.5\nChapman, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.526 2.038 1.068 106.3 38'}, {'pitcher_id': '458681', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynn, Lance 113 95 19 1 1 5 23 20.4 28.1 16 14.2 0.2 0.389 0.317 0.227 0.453 0.353 87.2 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 28 23 2 0 0 2 11 39.3 43.3 5 17.9 0.087 0.348 0.263 0.184 0.542 0.366 91.6 23\nSoler, Jorge 21 18 4 0 1 0 5 23.8 25.6 1 4.8 0.222 0.333 0.302 0.19 0.241 0.263 86.5 17.5\nFlores, Wilmer 19 16 5 0 0 1 0 0 10.5 3 15.8 0.313 0.5 0.401 0.336 0.524 0.425 86.4 12.7\nConforto, Michael 14 11 4 1 0 1 2 14.3 34.6 3 21.4 0.364 0.727 0.505 0.332 0.769 0.508 90.6 23.7\nYastrzemski, Mike 12 9 1 0 0 1 2 16.7 17.6 3 25 0.111 0.444 0.34 0.184 0.524 0.39 92.1 30.7\nAhmed, Nick 11 10 1 0 0 0 3 27.3 30.8 1 9.1 0.1 0.1 0.141 0.09 0.165 0.173 90.6 15\nEstrada, Thairo 8 8 2 0 0 0 0 0 14.3 0 0 0.25 0.25 0.221 0.304 0.358 0.29 72.7 -3.2'}], 139: [{'pitcher_id': '676617', 'player_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nO'Brien, Riley 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.116 0.118 0.11 95.2 -29", 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.077 0.08 0.077 90.3 -41\nLowe, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.157 0.143 100.1 -17'}, {'pitcher_id': '571927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatz, Steven 23 21 6 1 0 2 7 30.4 33.3 2 8.7 0.286 0.619 0.402 0.273 0.592 0.388 89.4 3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 6 6 2 0 0 0 1 16.7 18.2 0 0 0.333 0.333 0.293 0.345 0.38 0.319 87.2 -2.8\nArozarena, Randy 4 4 3 0 0 2 1 25 16.7 0 0 0.75 2.25 1.216 0.622 2.138 1.107 102.8 21.3\nEflin, Zach 5 5 0 0 0 0 3 60 60 0 0 0 0 0 0.016 0.017 0.016 70.8 4.5\nDíaz, Yandy 3 1 1 1 0 0 0 0 0 2 66.7 1 2 0.875 0.063 0.074 0.483 93.1 -27\nCaballero, José 2 2 0 0 0 0 0 0 28.6 0 0 0 0 0 0.2 0.36 0.23 82.2 8\nJackson, Alex 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nLowe, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.64 0.717 0.583 107.7 6'}, {'pitcher_id': '667463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, John 20 20 4 0 0 0 3 15 15.8 0 0 0.2 0.2 0.176 0.209 0.252 0.203 88.1 -11.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Brandon 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.095 0.101 0.086 92.2 -19\nWalls, Taylor 4 4 0 0 0 0 1 25 12.5 0 0 0 0 0 0.058 0.063 0.056 83.9 -21.7\nDíaz, Yandy 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.586 0.532 0.611 0.503 101.5 -9.7\nArozarena, Randy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.188 0.23 0.186 92.3 -15.7\nSiri, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.072 0.055 74 4\nCaballero, José 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.462 0.493 0.418 70.1 -10\nRosario, Amed 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.162 0.123 97.3 -26\nParedes, Isaac 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.357 0.671 0.438 95.1 20'}, {'pitcher_id': '688297', 'player_data': '', 'team_data': 'Chris Roycroft has not faced anyone on this team.'}, {'pitcher_id': '681517', 'player_data': '', 'team_data': 'Kyle Leahy has not faced anyone on this team.'}, {'pitcher_id': '681676', 'player_data': '', 'team_data': 'Ryan Fernandez has not faced anyone on this team.'}, {'pitcher_id': '543243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Sonny 45 44 14 1 0 0 12 26.7 29.8 1 2.2 0.318 0.341 0.298 0.27 0.335 0.273 88.7 8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 22 21 6 0 0 0 5 22.7 21.2 1 4.5 0.286 0.286 0.272 0.274 0.364 0.296 92 5.3\nLowe, Brandon 6 6 1 0 0 0 2 33.3 33.3 0 0 0.167 0.167 0.147 0.144 0.157 0.133 78.2 -7.7\nArozarena, Randy 4 4 1 0 0 0 1 25 40 0 0 0.25 0.25 0.221 0.256 0.282 0.239 83.6 18.3\nWalls, Taylor 2 2 0 0 0 0 2 100 28.6 0 0 0 0 0 0\nDíaz, Yandy 3 3 3 1 0 0 0 0 25 0 0 1 1.333 1.003 0.705 0.891 0.687 93.5 11\nLowe, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.418 0.431 0.377 97.2 -0.5\nSiri, Jose 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.441 0.308 0.374 0.297 99.9 21.5\nJackson, Alex 2 2 0 0 0 0 2 100 83.3 0 0 0 0 0 0\nCaballero, José 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.268 0.278 0.244 64.6 38'}, {'pitcher_id': '571945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMikolas, Miles 30 30 7 0 2 1 6 20 17.6 0 0 0.233 0.467 0.288 0.235 0.39 0.266 87.4 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 6 6 3 0 0 0 1 16.7 0 0 0 0.5 0.5 0.441 0.324 0.349 0.299 87.8 5.8\nArozarena, Randy 6 6 1 0 0 0 1 16.7 30 0 0 0.167 0.167 0.147 0.265 0.297 0.246 88.7 -11.6\nParedes, Isaac 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.218 0.538 0.308 92.6 11.7\nLowe, Brandon 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.001 0.001 0.001 80.1 85\nLowe, Josh 3 3 1 0 1 0 1 33.3 20 0 0 0.333 1 0.523 0.287 1.011 0.528 92.9 42\nSiri, Jose 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.117 0.126 0.107 75.3 22\nWalls, Taylor 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.205 0.335 0.225 79.3 0.7\nRosario, Amed 3 3 1 0 1 0 0 0 0 0 0 0.333 1 0.51 0.347 0.602 0.398 94.5 4'}, {'pitcher_id': '687798', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '502043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGibson, Kyle 83 74 25 3 1 2 15 18.1 29.1 6 7.2 0.338 0.486 0.38 0.304 0.459 0.364 86.3 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Brandon 19 18 4 1 1 0 5 26.3 28.6 0 0 0.222 0.389 0.275 0.172 0.263 0.213 88.8 13.4\nDíaz, Yandy 14 12 4 0 0 0 2 14.3 11.1 2 14.3 0.333 0.333 0.35 0.15 0.165 0.22 79.2 -9.7\nArozarena, Randy 12 10 3 0 0 0 3 25 38.9 1 8.3 0.3 0.3 0.338 0.18 0.234 0.266 87.5 -3.9\nWalls, Taylor 8 8 3 0 0 0 2 25 30.8 0 0 0.375 0.375 0.331 0.457 0.526 0.434 85.6 15.7\nLowe, Josh 6 6 2 0 0 1 0 0 20 0 0 0.333 0.833 0.481 0.358 0.856 0.505 88.8 -4\nSiri, Jose 6 6 0 0 0 0 3 50 66.7 0 0 0 0 0 0.235 0.595 0.345 80.9 -7\nRosario, Amed 5 5 3 1 0 0 0 0 14.3 0 0 0.6 0.8 0.594 0.578 0.667 0.541 84.1 0.4\nCaballero, José 5 3 2 0 0 0 0 0 22.2 1 20 0.667 0.667 0.492 0.67 0.815 0.61 93.1 13.8\nRortvedt, Ben 2 0 0 0 0 0 0 0 50 2 100 0.696 0.696\nParedes, Isaac 3 3 3 1 0 0 0 0 16.7 0 0 1 1.333 1 0.823 0.945 0.777 89.4 15.3\nPalacios, Richie 3 3 1 0 0 1 0 0 25 0 0 0.333 1.333 0.668 0.392 1.125 0.621 88.5 0.7'}, {'pitcher_id': '669467', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPallante, Andre 3 3 3 0 0 0 0 0 20 0 0 1 1 0.883 0.42 0.489 0.398 91.4 -0.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Brandon 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.199 0.201 0.181 83.8 -2\nArozarena, Randy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.923 1.106 0.885 96.3 12\nDíaz, Yandy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.139 0.16 0.127 94.1 -12'}, {'pitcher_id': '668941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomero, JoJo 6 5 1 0 0 0 3 50 37.5 1 16.7 0.2 0.2 0.263 0.175 0.221 0.26 94.8 26', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 2 1 0 0 0 0 1 50 25 1 50 0 0 0.348 0.348\nDíaz, Yandy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.866 1.086 0.854 102.8 10\nSiri, Jose 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nLowe, Brandon 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nParedes, Isaac 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.009 0.019 0.012 86.7 42'}, {'pitcher_id': '668868', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '606149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallegos, Giovanny 9 8 0 0 0 0 3 33.3 33.3 0 0 0 0 0 0.133 0.214 0.213 98.2 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 5 4 0 0 0 0 1 20 25 0 0 0 0 0 0.194 0.221 0.29 98.2 9.8\nEflin, Zach 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.209 0.722 0.391 96.5 31\nDíaz, Yandy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.105 0.076 99.8 -39'}, {'pitcher_id': '669461', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLiberatore, Matthew 15 14 2 1 0 0 5 33.3 33.3 1 6.7 0.143 0.214 0.188 0.173 0.346 0.25 91.7 24.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 3 2 1 1 0 0 1 33.3 28.6 1 33.3 0.5 1 0.647 0.477 0.587 0.542 97.6 13\nDíaz, Yandy 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.015 0.015 0.015 75 -31\nSiri, Jose 3 3 0 0 0 0 2 66.7 28.6 0 0 0 0 0 0.002 0.005 0.003 94 47\nArozarena, Randy 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.294 0.156 0.169 0.142 88.7 35.3\nParedes, Isaac 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.314 1.037 0.548 97.6 29.7'}, {'pitcher_id': '552640', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKittredge, Andrew 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.293 0.225 0.262 0.211 96 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.293 0.225 0.262 0.211 96 20.5'}, {'pitcher_id': '664854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHelsley, Ryan 4 2 0 0 0 0 0 0 33.3 2 50 0 0 0.345 0.474 0.517 0.564 77.7 33', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArozarena, Randy 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nRosario, Amed 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.474 0.517 0.439 77.7 33\nParedes, Isaac 1 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '458681', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynn, Lance 64 61 13 4 0 1 18 28.1 32.8 3 4.7 0.213 0.328 0.252 0.234 0.361 0.276 87.8 1.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 27 27 7 2 0 0 7 25.9 27.5 0 0 0.259 0.333 0.256 0.198 0.255 0.2 89.6 -12.5\nLowe, Brandon 10 9 2 0 0 1 1 10 33.3 1 10 0.222 0.556 0.357 0.253 0.644 0.397 85.8 20.9\nWalls, Taylor 6 4 0 0 0 0 3 50 40 2 33.3 0 0 0.231 0.015 0.018 0.245 72 -25\nArozarena, Randy 6 6 1 1 0 0 3 50 63.6 0 0 0.167 0.333 0.207 0.235 0.297 0.234 99.9 -3.7\nLowe, Josh 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.294 0.501 1.055 0.656 90 30\nDíaz, Yandy 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.092 0.1 0.086 81.6 -9.5\nRortvedt, Ben 4 4 1 0 0 0 1 25 28.6 0 0 0.25 0.25 0.22 0.51 0.591 0.48 84.2 13.7\nParedes, Isaac 3 3 1 1 0 0 1 33.3 16.7 0 0 0.333 0.667 0.415 0.339 0.427 0.332 83.8 4.5\nSiri, Jose 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.164 0.178 0.147 81.6 40'}], 140: [{'pitcher_id': '676617', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '571927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatz, Steven 38 34 6 0 0 0 10 26.3 21.4 2 5.3 0.176 0.176 0.199 0.2 0.236 0.232 85.2 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 10 9 2 0 0 0 2 20 13.3 1 10 0.222 0.222 0.244 0.114 0.126 0.164 84.6 9.7\nJankowski, Travis 5 4 0 0 0 0 2 40 14.3 1 20 0 0 0.138 0.087 0.095 0.202 79.3 -9.5\nGrossman, Robbie 6 5 0 0 0 0 0 0 0 0 0 0 0 0.12 0.19 0.246 0.279 82.5 4.8\nLowe, Nathaniel 5 5 1 0 0 0 2 40 35.7 0 0 0.2 0.2 0.176 0.228 0.255 0.212 82.5 6.3\nSemien, Marcus 4 4 3 0 0 0 0 0 0 0 0 0.75 0.75 0.658 0.655 0.798 0.625 92.9 9.8\nGray, Jon 3 2 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0\nGarcía, Adolis 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.265 0.283 0.238 94.9 7\nUreña, José 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nTaveras, Leody 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 81.8 67'}, {'pitcher_id': '667463', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKing, John 4 3 2 0 0 1 0 0 28.6 0 0 0.667 1.667 0.716 0.289 0.491 0.325 87 29.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 3 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.295 0.337 0.381 0.316 83.3 26.7\nSemien, Marcus 1 1 1 0 0 1 0 0 33.3 0 0 1 4 1.979 0.193 0.712 0.352 97.8 37'}, {'pitcher_id': '688297', 'player_data': '', 'team_data': 'Chris Roycroft has not faced anyone on this team.'}, {'pitcher_id': '681517', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeahy, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.066 0.056 80.3 -22', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.066 0.056 80.3 -22'}, {'pitcher_id': '681676', 'player_data': '', 'team_data': 'Ryan Fernandez has not faced anyone on this team.'}, {'pitcher_id': '543243', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Sonny 96 84 16 4 0 1 19 19.8 21.7 7 7.3 0.19 0.274 0.235 0.238 0.407 0.308 89.9 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 21 18 2 0 0 0 10 47.6 46.2 3 14.3 0.111 0.111 0.183 0.103 0.125 0.201 90.3 1.5\nSemien, Marcus 18 15 3 0 0 0 1 5.6 19.2 2 11.1 0.2 0.2 0.237 0.205 0.384 0.31 84.4 24.2\nSeager, Corey 11 9 2 1 0 0 0 0 13.3 2 18.2 0.222 0.333 0.319 0.324 0.518 0.416 91.8 19.6\nGarcía, Adolis 8 8 2 0 0 1 2 25 27.8 0 0 0.25 0.625 0.369 0.293 0.704 0.412 98.9 15.3\nKnizner, Andrew 7 7 1 1 0 0 2 28.6 25 0 0 0.143 0.286 0.177 0.076 0.169 0.102 88.2 17.4\nLowe, Nathaniel 8 8 2 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.337 0.503 0.363 91 6.4\nTaveras, Leody 7 6 2 2 0 0 1 14.3 16.7 0 0 0.333 0.667 0.355 0.382 0.591 0.389 101.3 9.7\nDuran, Ezequiel 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.177 0.339 0.597 0.402 92.8 6\nEovaldi, Nathan 3 2 0 0 0 0 2 66.7 25 0 0 0 0 0\nSmith, Josh 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.311 0.45 0.332 90.8 38.3\nJankowski, Travis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.191 0.276 0.188 91.7 0\nGray, Jon 2 0 0 0 0 0 0 0 0 0 0 40.6 -44'}, {'pitcher_id': '571945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMikolas, Miles 20 18 6 3 0 0 1 5 13.6 2 10 0.333 0.5 0.386 0.281 0.379 0.307 88.5 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJankowski, Travis 9 9 0 0 0 0 1 11.1 10 0 0 0 0 0 0.186 0.2 0.174 80.9 4.5\nSeager, Corey 8 7 4 1 0 0 0 0 20 1 12.5 0.571 0.714 0.566 0.402 0.61 0.457 97.1 22.3\nGrossman, Robbie 2 1 1 1 0 0 0 0 20 1 50 1 2 0.986\nLorenzen, Michael 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217'}, {'pitcher_id': '687798', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, Nick 7 6 3 2 0 0 1 14.3 20 0 0 0.5 0.833 0.481 0.309 0.41 0.443 93.7 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.207 0.21 0.186 106.6 -2\nTaveras, Leody 1 0 0 0 0 0 0 0 0 0 0 0 1.243 103.5 32\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.058 0.084 0.058 57.6 -25\nGarcía, Adolis 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.476 0.653 0.487 99.5 34.5\nSeager, Corey 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.434 0.651 0.456 95.2 18'}, {'pitcher_id': '502043', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGibson, Kyle 101 94 26 4 0 6 19 18.8 20 6 5.9 0.277 0.511 0.356 0.285 0.541 0.373 90.9 11.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 30 27 5 0 0 0 10 33.3 20.5 2 6.7 0.185 0.185 0.217 0.19 0.269 0.261 83.9 7.3\nSemien, Marcus 29 28 8 3 0 1 3 10.3 13.6 1 3.4 0.286 0.5 0.342 0.317 0.464 0.344 94.3 15.3\nSeager, Corey 13 11 4 0 0 1 3 23.1 36.8 2 15.4 0.364 0.636 0.462 0.285 0.689 0.449 89.4 -0.6\nLowe, Nathaniel 10 9 5 0 0 3 0 0 14.3 1 10 0.556 1.556 0.852 0.508 1.437 0.789 93.7 15.8\nGarcía, Adolis 7 7 1 0 0 1 1 14.3 26.7 0 0 0.143 0.571 0.286 0.264 0.635 0.372 95.9 10.2\nKnizner, Andrew 5 5 2 1 0 0 2 40 22.2 0 0 0.4 0.6 0.426 0.17 0.196 0.162 91.5 15.7\nTaveras, Leody 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.262 0.219 88.1 12\nSmith, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.368 0.493 0.371 86.7 25\nDuran, Ezequiel 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.193 0.591 0.309 78.6 18'}, {'pitcher_id': '669467', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPallante, Andre 6 6 1 0 0 0 3 50 31.3 0 0 0.167 0.167 0.147 0.187 0.211 0.174 93.7 -7.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.494 0.53 0.445 88.9 8\nLowe, Nathaniel 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTaveras, Leody 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGarcía, Adolis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.086 0.076 90.3 -32\nSeager, Corey 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSemien, Marcus 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.543 0.648 0.524 101.9 2'}, {'pitcher_id': '668941', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomero, JoJo 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.021 0.021 0.03 75.8 -26', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.043 0.06 75.8 -26'}, {'pitcher_id': '668868', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '606149', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGallegos, Giovanny 8 7 0 0 0 0 4 50 42.9 0 0 0 0 0.09 0.107 0.129 0.181 67.8 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLorenzen, Michael 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nSeager, Corey 3 2 0 0 0 0 2 66.7 83.3 0 0 0 0 0.24 0.24\nJankowski, Travis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.145 0.148 0.137 56.2 -23\nGrossman, Robbie 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.23 0.304 0.228 91 19'}, {'pitcher_id': '669461', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLiberatore, Matthew 19 18 8 2 0 1 2 10.5 12.5 1 5.3 0.444 0.722 0.507 0.365 0.665 0.447 93.1 7.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.54 0.484 0.694 0.5 95.9 23.5\nLowe, Nathaniel 3 3 1 0 0 1 1 33.3 20 0 0 0.333 1.333 0.668 0.366 1.305 0.675 104.1 -2\nSeager, Corey 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.16 0.24 0.171 80.5 -21.5\nSemien, Marcus 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.213 0.567 0.322 89.6 19.7\nDuran, Ezequiel 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.16 0.164 0.421 103.1 -27\nTaveras, Leody 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.608 0.695 0.567 94.4 0\nGarcía, Adolis 2 2 2 0 0 0 0 0 25 0 0 1 1 0.883 0.524 0.651 0.51 88.5 21'}, {'pitcher_id': '552640', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKittredge, Andrew 16 14 2 1 0 0 2 12.5 18.5 1 6.3 0.143 0.214 0.221 0.179 0.281 0.259 83.5 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 3 1 0 0 0 0 0 0 0 1 33.3 0 0 0.471 0.228 0.228 0.541 56.9 -67\nSemien, Marcus 5 5 0 0 0 0 1 20 30 0 0 0 0 0 0.157 0.19 0.145 83.7 12.5\nGrossman, Robbie 4 4 1 1 0 0 1 25 0 0 0 0.25 0.5 0.312 0.155 0.238 0.173 81.1 33\nSeager, Corey 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.167 0.542 0.276 93.7 33\nGarcía, Adolis 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.124 0.139 0.124 95 -17\nTaveras, Leody 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.289 0.557 0.354 89.5 7'}, {'pitcher_id': '664854', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHelsley, Ryan 7 7 1 0 0 0 3 42.9 33.3 0 0 0.143 0.143 0.124 0.121 0.17 0.127 89.5 22.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.086 0.103 0.083 76.1 49\nDuran, Ezequiel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarcía, Adolis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSeager, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.671 0.426 99.4 18\nLorenzen, Michael 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.207 0.213 0.217 106.4 -25'}, {'pitcher_id': '458681', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLynn, Lance 80 74 18 4 0 1 26 32.5 24.7 5 6.3 0.243 0.338 0.275 0.228 0.336 0.271 89.8 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 27 25 8 3 0 1 7 25.9 20 2 7.4 0.32 0.56 0.389 0.27 0.473 0.341 91.4 15.8\nGrossman, Robbie 31 29 8 1 0 0 8 25.8 20.3 2 6.5 0.276 0.31 0.283 0.291 0.364 0.311 88.6 17.4\nLowe, Nathaniel 6 6 2 0 0 0 3 50 35.3 0 0 0.333 0.333 0.293 0.177 0.235 0.176 101.9 1.3\nSeager, Corey 6 6 0 0 0 0 4 66.7 46.2 0 0 0 0 0 0.01 0.011 0.012 75.3 7\nTaveras, Leody 3 3 0 0 0 0 3 100 57.1 0 0 0 0 0 0\nGarcía, Adolis 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.086 0.235 0.325 89.5 2\nGray, Jon 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0 0 0 80 69\nEovaldi, Nathan 2 2 0 0 0 0 1 50 0 0 0 0 0 0\nUreña, José 1 0 0 0 0 0 0 0 0 0 0'}]}, 139: {136: [{'pitcher_id': '671737', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradley, Taj 21 20 7 4 0 2 3 14.3 19.4 1 4.8 0.35 0.85 0.503 0.231 0.511 0.325 92.9 27.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 6 6 1 1 0 0 1 16.7 20 0 0 0.167 0.333 0.207 0.147 0.178 0.14 86.6 31.4\nRodríguez, Julio 6 6 3 1 0 1 0 0 21.4 0 0 0.5 1.167 0.688 0.284 0.516 0.333 96.4 27.2\nRaleigh, Cal 3 3 1 0 0 1 1 33.3 0 0 0 0.333 1.333 0.668 0.409 1.317 0.699 101.1 22.5\nMoore, Dylan 3 2 1 1 0 0 0 0 0 1 33.3 0.5 1 0.647 0.137 0.414 0.385 96.4 45\nCanzone, Dominic 3 3 1 1 0 0 1 33.3 40 0 0 0.333 0.667 0.415 0.178 0.425 0.244 86.4 3'}, {'pitcher_id': '664208', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaton, Phil 35 32 8 2 0 1 6 17.1 23.1 3 8.6 0.25 0.406 0.314 0.245 0.377 0.304 82.2 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 6 5 1 1 0 0 2 33.3 50 1 16.7 0.2 0.4 0.326 0.101 0.103 0.192 71.9 6.7\nZavala, Seby 3 3 1 0 0 0 1 33.3 27.3 0 0 0.333 0.333 0.295 0.503 0.552 0.467 86.5 12\nRaleigh, Cal 7 7 4 1 0 0 0 0 10 0 0 0.571 0.714 0.555 0.37 0.524 0.379 93.4 23.6\nCrawford, J.P. 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.394 0.396 0.411 0.441 77.7 -10.3\nMoore, Dylan 3 2 0 0 0 0 2 66.7 60 1 33.3 0 0 0.231 0.231\nHaniger, Mitch 5 5 1 0 0 1 0 0 25 0 0 0.2 0.8 0.401 0.298 0.864 0.483 88.4 1.2\nRojas, Josh 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.078 0.101 0.081 76.1 -17.5\nRodríguez, Julio 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.085 0.092 0.083 64.7 -1\nRobles, Victor 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.083 0.095 0.075 76.6 38\nRaley, Luke 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.083 0.086 0.08 62.6 -4'}, {'pitcher_id': '641793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLittell, Zack 23 22 4 0 0 0 2 8.7 17.1 1 4.3 0.182 0.182 0.183 0.235 0.308 0.258 88.6 7.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 6 5 2 0 0 0 0 0 0 1 16.7 0.4 0.4 0.41 0.451 0.628 0.504 89 14.4\nRodríguez, Julio 5 5 0 0 0 0 2 40 37.5 0 0 0 0 0 0.034 0.043 0.035 93.3 -9.7\nRaleigh, Cal 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.294 0.417 0.517 0.412 82.6 -19\nCrawford, J.P. 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.167 0.236 0.172 86.3 16.8\nRobles, Victor 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.44 0.38 0.426 0.369 91.8 -2\nCanzone, Dominic 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.024 0.025 0.022 90 33.3'}, {'pitcher_id': '542888', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArmstrong, Shawn 20 20 4 1 0 1 6 30 24.4 0 0 0.2 0.4 0.252 0.258 0.433 0.294 90.5 20.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 3 3 0 0 0 0 1 33.3 11.1 0 0 0 0 0 0.129 0.255 0.164 95.6 36.5\nCrawford, J.P. 4 4 1 1 0 0 1 25 22.2 0 0 0.25 0.5 0.315 0.274 0.42 0.296 86.5 20.3\nRodríguez, Julio 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.295 0.324 0.423 0.33 85.8 -1\nHaniger, Mitch 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.003 0.354 0.903 0.525 85.9 -2\nMoore, Dylan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.117 0.419 0.203 101 39\nGarver, Mitch 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nZavala, Seby 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nRaleigh, Cal 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.358 0.411 0.335 97.5 36.5\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.363 0.734 0.45 99.5 21\nCanzone, Dominic 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.677 0.749 0.628 78 24'}, {'pitcher_id': '606965', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDevenski, Chris 16 14 1 0 0 0 3 18.8 31 1 6.3 0.071 0.071 0.098 0.252 0.539 0.342 86 34.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.175 0.264 0.678 0.381 87.2 25.3\nRobles, Victor 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.13 0.246 0.18 89.3 30\nMoore, Dylan 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.435 0.972 0.574 88.7 24.5\nCrawford, J.P. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.009 0.005 76.5 60.5\nRaleigh, Cal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.629 0.713 0.592 75.3 27\nZavala, Seby 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nRodríguez, Julio 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.006 0.01 0.006 96.9 52'}, {'pitcher_id': '687330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Kevin 7 7 0 0 0 0 2 28.6 27.3 0 0 0 0 0 0.195 0.337 0.224 87.4 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Julio 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.212 0.67 0.355 99.2 -3.5\nCrawford, J.P. 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.377 0.418 0.347 85 34.5\nRobles, Victor 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarver, Mitch 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nMoore, Dylan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.184 0.186 0.166 68.8 4'}, {'pitcher_id': '663992', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLovelady, Richard 9 8 0 0 0 0 4 44.4 38.9 1 11.1 0 0 0.077 0.099 0.099 0.16 76.3 -20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 6 6 0 0 0 0 2 33.3 40 0 0 0 0 0 0.131 0.131 0.124 76.3 -20\nZavala, Seby 2 1 0 0 0 0 1 50 25 1 50 0 0 0.346 0.346\nHaniger, Mitch 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '664076', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCleavinger, Garrett 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.295 0.179 0.192 0.162 89.4 -7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nZavala, Seby 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.083 0.059 74.5 -16\nRobles, Victor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.474 0.494 0.427 104.3 2'}, {'pitcher_id': '670955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUceta, Edwin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '655889', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Manuel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nZavala, Seby 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '641302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Tyler 36 32 9 1 0 5 10 27.8 24.6 3 8.3 0.281 0.781 0.447 0.274 0.684 0.412 92.9 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 7 6 3 0 0 2 1 14.3 15.8 1 14.3 0.5 1.5 0.807 0.344 1.27 0.669 90.3 14.2\nCrawford, J.P. 6 6 1 1 0 0 3 50 33.3 0 0 0.167 0.333 0.203 0.232 0.312 0.228 96.9 16\nGarver, Mitch 6 5 2 0 0 1 1 16.7 25 0 0 0.4 1 0.481 0.464 1.218 0.575 96.8 26\nRodríguez, Julio 6 5 2 0 0 2 1 16.7 11.1 1 16.7 0.4 1.6 0.794 0.376 1.017 0.604 100.3 10\nZavala, Seby 5 4 1 0 0 0 2 40 33.3 1 20 0.25 0.25 0.315 0.077 0.092 0.194 84.5 36.5\nMoore, Dylan 2 2 0 0 0 0 2 100 57.1 0 0 0 0 0 0\nRaleigh, Cal 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.032 0.029 91.7 7.5\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.68 0.727 0.612 86.2 12\nRaley, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.058 0.062 0.058 70.8 -17'}, {'pitcher_id': '650644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCivale, Aaron 64 58 15 3 0 4 10 15.6 18.5 6 9.4 0.259 0.517 0.358 0.297 0.538 0.386 89.9 8.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 14 12 3 2 0 0 2 14.3 11.1 2 14.3 0.25 0.417 0.34 0.292 0.417 0.361 93 -1.4\nGarver, Mitch 10 9 2 0 0 2 2 20 14.3 1 10 0.222 0.889 0.461 0.379 0.887 0.539 89.9 12.9\nRodríguez, Julio 12 12 5 0 0 1 2 16.7 28.6 0 0 0.417 0.667 0.461 0.316 0.447 0.329 86.4 -1.3\nHaniger, Mitch 9 9 0 0 0 0 3 33.3 26.7 0 0 0 0 0 0.115 0.213 0.138 82.3 4.3\nZavala, Seby 4 3 1 0 0 0 0 0 14.3 1 25 0.333 0.333 0.395 0.312 0.345 0.392 87.2 15.7\nMoore, Dylan 5 5 2 1 0 1 1 20 25 0 0 0.4 1.2 0.663 0.074 0.172 0.108 85 29\nRaleigh, Cal 6 5 1 0 0 0 0 0 12.5 1 16.7 0.2 0.2 0.262 0.723 1.656 0.94 104.3 17.8\nRojas, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.14 0.143 0.128 87.8 21.5\nCanzone, Dominic 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.344 0.365 0.504 95.1 -2'}, {'pitcher_id': '592094', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdam, Jason 11 9 0 0 0 0 6 54.5 33.3 1 9.1 0 0 0.128 0.12 0.352 0.291 88 1.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.013 0.013 0.012 79 -27\nRodríguez, Julio 3 2 0 0 0 0 2 66.7 25 0 0 0 0 0.24 0.24\nCrawford, J.P. 2 2 0 0 0 0 2 100 0 0 0 0 0 0 0\nMoore, Dylan 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.521 1.564 0.877 92.5 16\nHaniger, Mitch 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69'}, {'pitcher_id': '621097', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaguespack, Jacob 5 5 1 0 0 0 1 20 9.1 0 0 0.2 0.2 0.174 0.215 0.24 0.198 89.8 30.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.29 0.351 0.393 0.324 99.4 2\nMoore, Dylan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.01 0.01 0.009 80.2 58.5'}, {'pitcher_id': '682052', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '621107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEflin, Zach 31 29 6 1 0 0 8 25.8 25.5 1 3.2 0.207 0.241 0.211 0.209 0.277 0.227 82.8 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 13 13 2 0 0 0 4 30.8 32 0 0 0.154 0.154 0.135 0.177 0.212 0.169 83.1 19.4\nZavala, Seby 5 5 0 0 0 0 2 40 40 0 0 0 0 0 0.078 0.15 0.094 71.3 -1\nCrawford, J.P. 3 2 1 1 0 0 1 33.3 25 1 33.3 0.5 1 0.647 0.445 0.501 0.507 88.3 13\nRodríguez, Julio 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.294 0.135 0.387 0.216 91.4 14.5\nRaleigh, Cal 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.294 0.354 0.377 0.326 78.1 13\nRojas, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.493 0.589 0.475 95.3 30\nCastillo, Luis 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.039 0.045 0.039 80.4 -35'}, {'pitcher_id': '686752', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPepiot, Ryan 5 3 1 0 0 0 0 0 0 2 40 0.333 0.333 0.452 0.152 0.165 0.359 67.7 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 5 3 1 0 0 0 0 0 0 2 40 0.333 0.333 0.452 0.152 0.165 0.359 67.7 15.3'}, {'pitcher_id': '621363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoche, Colin 18 14 4 0 0 1 3 16.7 27.3 4 22.2 0.286 0.5 0.416 0.306 0.517 0.427 87.1 23.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 7 4 3 0 0 1 0 0 22.2 3 42.9 0.75 1.5 0.844 0.573 1.185 0.721 93.5 10.8\nMoore, Dylan 3 3 0 0 0 0 2 66.7 62.5 0 0 0 0 0 0 0 0 87.2 70\nRodríguez, Julio 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.242 0.259 0.226 69.6 37.5\nRaleigh, Cal 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.04 0.129 0.383 96.9 44\nZavala, Seby 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.226 0.149 78.6 9.5\nRobles, Victor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.923 1.141 0.894 103.4 11'}, {'pitcher_id': '664126', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairbanks, Pete 6 5 0 0 0 0 5 83.3 75 1 16.7 0 0 0.115 0.115', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nRojas, Josh 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMoore, Dylan 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nRodríguez, Julio 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHaniger, Mitch 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarver, Mitch 0 0 0 0 0 0 0 0 0'}], 137: [{'pitcher_id': '671737', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradley, Taj 7 6 1 0 0 0 1 14.3 0 1 14.3 0.167 0.167 0.225 0.162 0.178 0.23 75 39.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 5 4 0 0 0 0 1 20 0 1 20 0 0 0.139 0.016 0.02 0.153 81.8 35\nCasali, Curt 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.455 0.493 0.423 64.9 46'}, {'pitcher_id': '664208', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaton, Phil 34 32 4 0 0 2 12 35.3 33.3 1 2.9 0.125 0.313 0.211 0.161 0.373 0.251 83.3 24.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 5 4 1 0 0 0 1 20 20 0 0 0.25 0.25 0.321 0.423 0.511 0.474 85.7 3.3\nAhmed, Nick 6 6 1 0 0 0 2 33.3 54.5 0 0 0.167 0.167 0.147 0.117 0.179 0.128 76.3 8.5\nFlores, Wilmer 6 6 0 0 0 0 3 50 50 0 0 0 0 0 0.004 0.017 0.009 84.9 60.7\nSoler, Jorge 4 4 0 0 0 0 3 75 57.1 0 0 0 0 0 0.003 0.004 0.003 79.2 61\nSlater, Austin 3 3 1 0 0 1 0 0 11.1 0 0 0.333 1.333 0.647 0.356 1.15 0.593 78.7 17.3\nCasali, Curt 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nEstrada, Thairo 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.235 0.489 0.395 80.7 8.3\nChapman, Matt 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.691 0.315 1.257 0.653 99.4 37.5\nYastrzemski, Mike 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 92.3 44'}, {'pitcher_id': '641793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLittell, Zack 26 24 8 2 0 3 6 23.1 21.6 0 0 0.333 0.792 0.484 0.364 0.8 0.501 96.7 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 11 10 3 1 0 1 3 27.3 26.3 0 0 0.3 0.7 0.439 0.301 0.601 0.404 99 17.6\nSoler, Jorge 6 5 3 1 0 1 2 33.3 23.1 0 0 0.6 1.4 0.81 0.541 1.707 0.885 106.9 23\nConforto, Michael 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.294 0.254 0.306 0.245 76.3 29.5\nFlores, Wilmer 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.317 0.395 0.305 92.7 14\nEstrada, Thairo 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.65 1.289 0.81 104.1 18\nBailey, Patrick 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.162 0.181 0.151 90.5 21'}, {'pitcher_id': '542888', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArmstrong, Shawn 21 21 3 1 0 1 9 42.9 20.8 0 0 0.143 0.333 0.197 0.187 0.322 0.214 94.9 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 8 8 2 0 0 1 2 25 15.8 0 0 0.25 0.625 0.364 0.321 0.53 0.362 100.1 6.7\nSoler, Jorge 4 4 0 0 0 0 2 50 35.7 0 0 0 0 0 0.021 0.023 0.02 82.2 19.5\nFlores, Wilmer 2 2 0 0 0 0 2 100 0 0 0 0 0 0 0\nEstrada, Thairo 3 3 1 1 0 0 1 33.3 20 0 0 0.333 0.667 0.406 0.268 0.46 0.293 97.9 8\nAhmed, Nick 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.237 0.524 0.315 89.1 26\nRamos, Heliot 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nConforto, Michael 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '606965', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDevenski, Chris 21 17 7 2 0 3 3 14.3 25 3 14.3 0.412 1.059 0.629 0.328 0.945 0.556 97 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 7 5 3 1 0 1 1 14.3 23.1 1 14.3 0.6 1.4 0.784 0.353 0.653 0.503 99.6 3.8\nConforto, Michael 3 2 0 0 0 0 0 0 28.6 1 33.3 0 0 0.231 0.555 2.103 0.95 99.8 29.5\nEstrada, Thairo 2 1 1 0 0 1 0 0 20 1 50 1 4 1.389 0.949 3.644 1.28 104.9 27\nSoler, Jorge 3 3 1 0 0 1 1 33.3 40 0 0 0.333 1.333 0.695 0.336 1.336 0.669 100 41\nYastrzemski, Mike 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.062 0.067 0.056 91.6 14.5\nAhmed, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.043 0.044 0.041 90.5 -23\nBailey, Patrick 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.151 0.32 0.193 94.5 27\nSlater, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.379 0.401 0.345 86.7 4'}, {'pitcher_id': '687330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Kevin 9 7 3 0 0 1 0 0 11.8 1 11.1 0.429 0.857 0.577 0.27 0.42 0.388 90.1 -1.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.417 0.539 0.412 95.5 23.5\nChapman, Matt 3 2 0 0 0 0 0 0 25 0 0 0 0 0.242 0.108 0.113 0.309 94.1 12.5\nEstrada, Thairo 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.444 0.272 0.273 0.247 76.4 -58\nBailey, Patrick 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nFlores, Wilmer 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.294 1.091 0.549 98.7 35'}, {'pitcher_id': '663992', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLovelady, Richard 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.29 0.33 0.358 0.303 86.7 4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.8 0.884 0.739 87.1 12\nAhmed, Nick 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSoler, Jorge 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.189 0.17 86.3 -4'}, {'pitcher_id': '664076', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCleavinger, Garrett 12 9 3 0 0 1 4 33.3 33.3 3 25 0.333 0.667 0.487 0.311 0.608 0.462 94 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 4 3 0 0 0 0 2 50 27.3 1 25 0 0 0.172 0.177 0.698 0.437 105.2 38\nAhmed, Nick 3 3 2 0 0 1 0 0 16.7 0 0 0.667 1.667 0.965 0.458 0.756 0.513 86.5 17\nEstrada, Thairo 2 1 0 0 0 0 1 50 60 1 50 0 0 0.346 0.346\nSoler, Jorge 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nSlater, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.897 1.112 0.875 105.1 11\nConforto, Michael 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '670955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUceta, Edwin 13 12 1 1 0 0 3 23.1 25.9 1 7.7 0.083 0.167 0.149 0.155 0.205 0.194 91.6 21.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Thairo 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.004 0.007 0.005 96.3 55\nFlores, Wilmer 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.131 0.117 78.6 38.5\nSlater, Austin 2 1 0 0 0 0 0 0 20 1 50 0 0 0.346 0.122 0.189 0.402 94.8 -21\nYastrzemski, Mike 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.621 0.461 0.567 0.443 96.3 -3\nAhmed, Nick 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nWisely, Brett 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.272 0.42 0.286 93.1 17.5\nConforto, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.018 0.011 97.8 51'}, {'pitcher_id': '655889', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Manuel 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.588 0.617 0.697 0.581 81.6 16.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nYastrzemski, Mike 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.949 1.113 0.9 88.6 14\nChapman, Matt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.903 0.978 0.843 74.5 19'}, {'pitcher_id': '641302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Tyler 24 23 7 1 0 3 3 12.5 20.5 1 4.2 0.304 0.739 0.448 0.253 0.416 0.301 89.1 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 7 7 1 0 0 1 3 42.9 41.2 0 0 0.143 0.571 0.296 0.136 0.257 0.164 92.6 41.3\nSlater, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.374 0.425 0.516 99.8 -1\nSoler, Jorge 4 4 3 1 0 0 0 0 14.3 0 0 0.75 1 0.759 0.329 0.387 0.312 87.6 21\nCasali, Curt 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.66 0.432 0.794 0.52 95.8 14\nFlores, Wilmer 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.198 0.236 0.187 81.8 33.7\nConforto, Michael 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.23 0.907 0.453 99.8 36.5\nEstrada, Thairo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.179 0.18 0.163 62.6 -44.5\nRamos, Heliot 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.48 0.53 0.435 103.4 1'}, {'pitcher_id': '650644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCivale, Aaron 30 26 7 2 0 1 6 20 24.2 3 10 0.269 0.462 0.338 0.189 0.285 0.259 87.8 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 6 5 2 1 0 0 1 16.7 16.7 1 16.7 0.4 0.6 0.464 0.238 0.293 0.302 82.7 2.8\nFlores, Wilmer 5 4 2 1 0 0 0 0 9.1 1 20 0.5 0.75 0.567 0.267 0.594 0.427 76.7 32.5\nSoler, Jorge 6 5 2 0 0 0 1 16.7 18.2 0 0 0.4 0.4 0.294 0.39 0.435 0.373 95.1 3.8\nChapman, Matt 4 3 0 0 0 0 1 25 60 1 25 0 0 0.174 0.008 0.013 0.181 93.8 53.5\nEstrada, Thairo 4 4 1 0 0 1 1 25 50 0 0 0.25 1 0.518 0.12 0.283 0.168 93.5 15.7\nCasali, Curt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.172 0.172 0.158 91.8 -9\nYastrzemski, Mike 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.005 0.007 0.005 88 60\nRamos, Heliot 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.007 0.012 0.009 82.6 40'}, {'pitcher_id': '592094', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdam, Jason 12 12 1 0 0 1 3 25 21.1 0 0 0.083 0.333 0.169 0.256 0.601 0.358 95.1 16.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 5 5 1 0 0 1 2 40 37.5 0 0 0.2 0.8 0.406 0.313 1.007 0.541 101.7 29.3\nConforto, Michael 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.126 0.136 0.114 96 22.5\nEstrada, Thairo 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.292 0.326 0.272 84.1 0\nFlores, Wilmer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.192 0.143 92.8 -8\nCasali, Curt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.393 0.93 0.558 97.7 23'}, {'pitcher_id': '621097', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaguespack, Jacob 20 17 4 1 0 0 6 30 38.1 2 10 0.235 0.294 0.302 0.291 0.375 0.348 88.9 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 5 4 1 0 0 0 1 20 22.2 1 20 0.25 0.25 0.316 0.34 0.578 0.445 97 31.7\nConforto, Michael 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.221 0.286 0.337 0.265 92.1 2\nEstrada, Thairo 4 3 2 1 0 0 1 25 28.6 0 0 0.667 1 0.719 0.491 0.536 0.521 90.2 12\nYastrzemski, Mike 2 1 0 0 0 0 0 0 60 1 50 0 0 0.346 0.626 0.643 0.628 68.4 18\nChapman, Matt 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nBailey, Patrick 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.174 0.23 0.17 80.8 28.5\nAhmed, Nick 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '682052', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Jacob 7 7 2 1 0 0 1 14.3 30.8 0 0 0.286 0.429 0.304 0.402 0.456 0.378 82.4 0.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Thairo 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.622 0.474 0.582 0.458 79.6 -1.5\nSlater, Austin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.415 0.455 0.386 89.6 -1.5\nRamos, Heliot 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nConforto, Michael 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.983 1.07 0.91 76.1 20\nBailey, Patrick 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.051 0.051 0.049 80.1 -13'}, {'pitcher_id': '621107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEflin, Zach 80 74 22 6 0 6 10 12.5 19.5 3 3.8 0.297 0.622 0.396 0.282 0.531 0.359 89.8 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 33 30 10 2 0 3 5 15.2 26.8 2 6.1 0.333 0.7 0.449 0.351 0.737 0.469 92.3 10.3\nSoler, Jorge 11 10 2 2 0 0 1 9.1 18.8 0 0 0.2 0.4 0.295 0.223 0.3 0.27 88.9 -1.6\nFlores, Wilmer 11 11 5 0 0 2 0 0 0 0 0 0.455 1 0.604 0.253 0.538 0.329 86.9 30.6\nYastrzemski, Mike 8 7 1 0 0 1 1 12.5 22.2 1 12.5 0.143 0.571 0.337 0.148 0.446 0.301 91.1 42.7\nAhmed, Nick 9 8 1 1 0 0 1 11.1 16.7 0 0 0.125 0.25 0.138 0.242 0.265 0.213 83 7.9\nCasali, Curt 3 3 2 1 0 0 1 33.3 25 0 0 0.667 1 0.709 0.496 0.674 0.505 100.2 14.5\nChapman, Matt 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.06 0.076 0.06 86.7 -12.3\nSlater, Austin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.587 0.686 0.548 101.6 6'}, {'pitcher_id': '686752', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPepiot, Ryan 22 19 2 1 0 1 7 31.8 34.1 2 9.1 0.105 0.316 0.249 0.213 0.452 0.335 92.3 24.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 4 4 1 1 0 0 2 50 0 0 0 0.25 0.5 0.317 0.256 0.389 0.276 98.3 24\nSoler, Jorge 5 4 0 0 0 0 1 20 55.6 1 20 0 0 0.139 0.284 0.552 0.415 102.2 30\nYastrzemski, Mike 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.004 0.004 0.235 82.3 61\nEstrada, Thairo 4 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.703 0.409 1.367 0.721 100.6 14.7\nChapman, Matt 2 2 0 0 0 0 1 50 57.1 0 0 0 0 0 0.004 0.006 0.005 63.8 77\nBailey, Patrick 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.277 0.305 0.257 87.8 10\nAhmed, Nick 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.046 0.047 0.043 68.4 -36'}, {'pitcher_id': '621363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoche, Colin 10 9 4 0 0 0 3 30 37.5 1 10 0.444 0.444 0.423 0.25 0.349 0.296 93.1 22.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 5 5 3 0 0 0 2 40 62.5 0 0 0.6 0.6 0.53 0.333 0.471 0.341 95.9 25.3\nSlater, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.165 0.209 0.423 83.4 -4\nSoler, Jorge 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBailey, Patrick 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.414 0.545 0.392 94 19\nAhmed, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.03 0.017 93.7 43'}, {'pitcher_id': '664126', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairbanks, Pete 7 5 0 0 0 0 1 14.3 25 1 14.3 0 0 0.203 0.291 0.448 0.428 85.7 1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 2 1 0 0 0 0 0 0 20 0 0 0 0 0.363 0.269 0.896 0.599 96.9 32\nBailey, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.263 0.263 0.237 73.6 -58\nAhmed, Nick 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.603 0.701 0.58 71.1 36\nEstrada, Thairo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.319 0.379 0.29 101 -6\nFlores, Wilmer 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nSoler, Jorge 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}], 138: [{'pitcher_id': '671737', 'player_data': '', 'team_data': 'Taj Bradley has not faced anyone on this team.'}, {'pitcher_id': '664208', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaton, Phil 28 26 8 1 1 0 7 25 32.7 2 7.1 0.308 0.423 0.337 0.275 0.402 0.315 89.1 19.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 8 8 2 0 0 0 2 25 16.7 0 0 0.25 0.25 0.22 0.257 0.488 0.309 92.7 28.2\nArenado, Nolan 7 7 3 1 1 0 1 14.3 46.2 0 0 0.429 0.857 0.521 0.362 0.456 0.351 86.4 10\nGoldschmidt, Paul 6 6 3 0 0 0 2 33.3 44.4 0 0 0.5 0.5 0.44 0.377 0.499 0.375 94.9 23\nCarlson, Dylan 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.231 0.231\nCarpenter, Matt 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.07 0.084 0.274 78.9 30\nDonovan, Brendan 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.148 0.189 0.136 81.8 -2'}, {'pitcher_id': '641793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLittell, Zack 35 33 12 2 0 3 6 17.1 16.2 2 5.7 0.364 0.697 0.465 0.307 0.572 0.393 85.3 23.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarlson, Dylan 7 6 1 0 0 1 2 28.6 28.6 1 14.3 0.167 0.667 0.395 0.26 0.783 0.473 89 18.3\nGoldschmidt, Paul 10 9 6 2 0 1 1 10 17.6 1 10 0.667 1.222 0.791 0.44 0.793 0.539 88.4 17\nArenado, Nolan 10 10 4 0 0 1 0 0 0 0 0 0.4 0.7 0.472 0.401 0.608 0.442 81.4 28.1\nBurleson, Alec 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.294 0.068 0.081 0.063 75.1 36.5\nGorman, Nolan 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.088 0.096 0.081 88.9 20.5\nCrawford, Brandon 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.06 0.213 0.109 98.5 39'}, {'pitcher_id': '542888', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArmstrong, Shawn 7 7 2 0 0 1 1 14.3 18.8 0 0 0.286 0.714 0.403 0.273 0.598 0.359 88.4 27.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarlson, Dylan 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.038 0.043 0.036 80.3 21\nBurleson, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.867 0.537 102.5 19\nGorman, Nolan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDonovan, Brendan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.786 0.918 0.757 84.1 19\nGoldschmidt, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.033 0.064 0.042 82.3 52\nArenado, Nolan 1 1 1 0 0 1 0 0 0 0 0 1 4 1.94 0.573 2.248 1.107 100.9 34'}, {'pitcher_id': '606965', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDevenski, Chris 10 8 1 0 0 0 1 10 12.5 1 10 0.125 0.125 0.156 0.151 0.209 0.188 90.2 21.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.39 0.232 0.352 0.349 92.3 46.7\nArenado, Nolan 3 2 0 0 0 0 0 0 25 0 0 0 0 0 0.153 0.206 0.099 87.9 5.7\nCarpenter, Matt 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nCarlson, Dylan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.014 0.016 0.013 84.6 33\nCrawford, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.189 0.174 96.4 -15'}, {'pitcher_id': '687330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Kevin 9 8 2 0 0 0 3 33.3 23.1 1 11.1 0.25 0.25 0.273 0.219 0.286 0.27 92.1 -3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.527 0.657 0.511 87.2 4.5\nCrawford, Brandon 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nArenado, Nolan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.105 0.105 0.097 94.1 -22\nCarlson, Dylan 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nGorman, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCarpenter, Matt 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nBurleson, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.486 0.762 0.518 98 17'}, {'pitcher_id': '663992', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLovelady, Richard 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.179 0.182 0.166 91.2 -8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.179 0.182 0.166 91.2 -8'}, {'pitcher_id': '664076', 'player_data': '', 'team_data': 'Garrett Cleavinger has not faced anyone on this team.'}, {'pitcher_id': '670955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUceta, Edwin 8 8 2 1 0 1 3 37.5 42.1 0 0 0.25 0.75 0.414 0.258 0.649 0.374 86.2 19.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 3 3 1 1 0 0 1 33.3 37.5 0 0 0.333 0.667 0.414 0.318 1.051 0.555 97.1 35\nCarlson, Dylan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.026 0.037 0.026 59.7 -27\nGoldschmidt, Paul 1 1 1 0 0 1 0 0 66.7 0 0 1 4 2.072 0.634 1.488 0.884 103.5 21\nCarpenter, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.423 0.477 0.395 73.5 33\nArenado, Nolan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '655889', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Manuel 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.003 0.003 0.347 83.5 47', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nGoldschmidt, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 83.5 47'}, {'pitcher_id': '641302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Tyler 5 3 2 1 0 0 0 0 0 1 20 0.667 1 0.709 0.56 1.327 0.756 88 26.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.737 0.846 0.703 76 25\nCarpenter, Matt 1 1 1 1 0 0 0 0 0 0 0 1 2 1.238 0.893 3.032 1.587 103.8 24\nGorman, Nolan 1 0 0 0 0 0 0 0 0 0 0 0.726 0.726\nCrawford, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.102 0.063 84.2 30'}, {'pitcher_id': '650644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCivale, Aaron 12 11 3 1 0 0 4 33.3 37.5 1 8.3 0.273 0.364 0.309 0.123 0.222 0.193 83.8 -5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 5 4 0 0 0 0 3 60 50 1 20 0 0 0.14 0.065 0.12 0.202 94.2 22\nGoldschmidt, Paul 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.413 0.182 0.469 0.269 77.8 -10.3\nCarlson, Dylan 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.252 0.261 0.229 88.5 -1.5\nCrawford, Brandon 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.02 0.02 0.021 82.3 -25'}, {'pitcher_id': '592094', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdam, Jason 17 15 1 0 0 1 8 47.1 25.8 2 11.8 0.067 0.267 0.201 0.157 0.446 0.303 90.7 19.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 7 7 1 0 0 1 5 71.4 10 0 0 0.143 0.571 0.29 0.177 0.549 0.3 93.8 27\nGoldschmidt, Paul 3 2 0 0 0 0 1 33.3 66.7 1 33.3 0 0 0.233 0.064 0.102 0.282 77.1 33\nGorman, Nolan 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.257 1.006 0.511 103.9 37\nArenado, Nolan 2 1 0 0 0 0 1 50 25 1 50 0 0 0.344 0.344\nCarlson, Dylan 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.083 0.152 0.103 81.4 10\nDonovan, Brendan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.313 0.333 0.285 103.4 -7'}, {'pitcher_id': '621097', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '682052', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Jacob 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.193 0.227 0.175 73.8 -2.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.193 0.227 0.175 73.8 -2.5'}, {'pitcher_id': '621107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEflin, Zach 82 77 19 4 0 4 11 13.4 17.6 5 6.1 0.247 0.455 0.318 0.272 0.499 0.347 89.8 18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 21 21 6 2 0 1 2 9.5 18.8 0 0 0.286 0.524 0.339 0.274 0.425 0.298 87 11.2\nGoldschmidt, Paul 16 16 2 0 0 0 5 31.3 21.4 0 0 0.125 0.125 0.11 0.143 0.177 0.139 82.3 18.8\nCrawford, Brandon 17 16 5 0 0 3 1 5.9 15.4 1 5.9 0.313 0.875 0.497 0.394 0.925 0.558 95.1 15.9\nCarlson, Dylan 11 10 3 1 0 0 1 9.1 14.3 1 9.1 0.3 0.4 0.335 0.271 0.341 0.302 89.9 19.1\nCarpenter, Matt 9 6 0 0 0 0 0 0 10 3 33.3 0 0 0.231 0.33 1.073 0.609 97.9 34.7\nGorman, Nolan 4 4 2 0 0 0 2 50 37.5 0 0 0.5 0.5 0.442 0.27 0.305 0.249 95.9 6.5\nBurleson, Alec 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.037 0.072 0.042 87.1 47.3\nDonovan, Brendan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.837 1.028 0.822 102.6 10'}, {'pitcher_id': '686752', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPepiot, Ryan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.178 0.126 88.8 22.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.178 0.126 88.8 22.5'}, {'pitcher_id': '621363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoche, Colin 4 2 0 0 0 0 0 0 0 1 25 0 0 0.354 0.331 0.373 0.504 82 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurleson, Alec 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nCarlson, Dylan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.409 0.453 0.368 88.1 4\nArenado, Nolan 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72\nGoldschmidt, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.252 0.294 0.232 75.9 5'}, {'pitcher_id': '664126', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairbanks, Pete 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.096 0.136 0.102 91.7 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.094 0.173 0.114 81.1 30\nBurleson, Alec 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.098 0.098 0.09 102.2 -21'}], 140: [{'pitcher_id': '671737', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradley, Taj 33 29 8 2 0 1 10 30.3 25 4 12.1 0.276 0.448 0.354 0.301 0.457 0.368 91.1 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 5 3 0 0 0 0 0 0 9.1 2 40 0 0 0.278 0.215 0.366 0.426 101.3 30\nLowe, Nathaniel 5 5 1 0 0 0 4 80 25 0 0 0.2 0.2 0.177 0.156 0.167 0.142 74.7 16\nSeager, Corey 5 5 2 1 0 0 3 60 33.3 0 0 0.4 0.6 0.425 0.233 0.33 0.237 104.1 8\nTaveras, Leody 4 3 0 0 0 0 1 25 40 1 25 0 0 0.174 0.314 0.37 0.391 93.3 10.5\nSemien, Marcus 6 6 3 0 0 1 0 0 10 0 0 0.5 1 0.628 0.612 1.111 0.728 81.3 15\nJankowski, Travis 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.395 0.243 0.262 0.341 90.2 -8\nDuran, Ezequiel 4 4 1 1 0 0 2 50 50 0 0 0.25 0.5 0.311 0.202 0.276 0.202 100 13.5'}, {'pitcher_id': '664208', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaton, Phil 57 52 12 1 0 0 16 28.1 28.6 3 5.3 0.231 0.25 0.242 0.17 0.215 0.204 80.7 7.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 11 10 3 0 0 0 3 27.3 28.6 0 0 0.3 0.3 0.306 0.178 0.258 0.243 86.9 1.9\nSemien, Marcus 9 7 2 0 0 0 2 22.2 30.4 1 11.1 0.286 0.286 0.273 0.211 0.255 0.243 79 6.2\nLowe, Nathaniel 9 8 0 0 0 0 4 44.4 36.4 1 11.1 0 0 0.077 0.032 0.045 0.105 80 52.5\nTaveras, Leody 5 5 0 0 0 0 2 40 40 0 0 0 0 0 0.058 0.061 0.055 83.8 -26.3\nGrossman, Robbie 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.118 0.138 0.106 75.9 31\nJankowski, Travis 5 5 3 1 0 0 0 0 0 0 0 0.6 0.8 0.607 0.431 0.575 0.437 81 11.4\nSmith, Josh 3 2 1 0 0 0 1 33.3 28.6 1 33.3 0.5 0.5 0.525 0.107 0.108 0.298 33.7 -71\nSeager, Corey 6 6 3 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.333 0.385 0.317 87.8 7\nDuran, Ezequiel 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.032 0.047 0.03 78.4 -11\nLorenzen, Michael 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGray, Jon 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.007 0.012 0.008 85.3 44\nLangford, Wyatt 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.187 0.192 0.168 53.4 -43'}, {'pitcher_id': '641793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLittell, Zack 29 28 8 0 1 2 10 34.5 31.5 1 3.4 0.286 0.571 0.367 0.287 0.526 0.354 93.4 25.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 7 6 2 0 0 0 3 42.9 45.5 1 14.3 0.333 0.333 0.35 0.309 0.37 0.353 92 33.7\nSemien, Marcus 7 7 1 0 1 0 1 14.3 25 0 0 0.143 0.429 0.218 0.229 0.652 0.352 92 33\nLowe, Nathaniel 3 3 1 0 0 0 2 66.7 20 0 0 0.333 0.333 0.294 0.062 0.062 0.057 98.1 -16\nGarcía, Adolis 4 4 0 0 0 0 2 50 57.1 0 0 0 0 0 0.118 0.129 0.107 89.5 35\nSeager, Corey 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.353 0.406 0.328 102.9 2\nKnizner, Andrew 3 3 2 0 0 1 1 33.3 40 0 0 0.667 1.667 0.962 0.522 1.156 0.707 86.7 30\nJankowski, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.886 1.124 0.881 96.6 14\nTaveras, Leody 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.397 1.435 0.727 99.8 33'}, {'pitcher_id': '542888', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArmstrong, Shawn 33 30 8 1 0 1 6 18.2 26.8 2 6.1 0.267 0.4 0.323 0.274 0.381 0.319 87.6 17.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 9 8 3 1 0 0 1 11.1 23.1 1 11.1 0.375 0.5 0.412 0.282 0.328 0.313 84.1 25.3\nLowe, Nathaniel 4 3 1 0 0 0 2 50 36.4 1 25 0.333 0.333 0.393 0.326 0.367 0.403 79.2 18\nGarcía, Adolis 4 4 1 0 0 0 2 50 44.4 0 0 0.25 0.25 0.221 0.324 0.376 0.31 84.5 11.5\nSeager, Corey 4 4 0 0 0 0 0 0 33.3 0 0 0 0 0 0.177 0.485 0.271 91.7 40.3\nTaveras, Leody 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.484 0.369 93.7 9\nJankowski, Travis 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.396 0.459 0.378 94.2 -10\nDuran, Ezequiel 2 2 1 0 0 0 0 0 12.5 0 0 0.5 0.5 0.441 0.153 0.188 0.141 73.9 -12.5\nSmith, Josh 2 1 0 0 0 0 0 0 50 0 0 0 0 0.36 0.435 0.488 0.558 95.6 0\nLangford, Wyatt 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGrossman, Robbie 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.155 0.507 0.265 92.8 31'}, {'pitcher_id': '606965', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDevenski, Chris 45 39 10 3 0 2 8 17.8 30.4 6 13.3 0.256 0.487 0.363 0.269 0.525 0.382 89.1 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 14 12 5 3 0 0 5 35.7 40.9 2 14.3 0.417 0.667 0.489 0.291 0.424 0.361 88.6 10.9\nGrossman, Robbie 9 7 1 0 0 0 1 11.1 31.6 2 22.2 0.143 0.143 0.25 0.154 0.168 0.265 78.2 21\nGarcía, Adolis 4 3 2 0 0 2 0 0 37.5 1 25 0.667 2.667 1.196 0.602 2.174 1.018 99.3 17.3\nSeager, Corey 5 5 1 0 0 0 0 0 10 0 0 0.2 0.2 0.177 0.454 0.935 0.576 98.9 27.6\nLowe, Nathaniel 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.143 0.159 0.137 88 -9\nTaveras, Leody 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.33 0.512 0.354 87.8 24.5\nJankowski, Travis 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.003 0.003 0.349 88.3 46\nSmith, Josh 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.16 0.528 0.286 84.9 15.5\nLangford, Wyatt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.015 0.012 82 77\nKnizner, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.272 0.291 0.248 100 -5\nDuran, Ezequiel 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '687330', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelly, Kevin 6 4 1 1 0 0 2 33.3 22.2 1 16.7 0.25 0.5 0.444 0.126 0.233 0.336 88.5 40.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nGarcía, Adolis 1 0 0 0 0 0 0 0 0 0 0 0.726 0.726\nJankowski, Travis 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nTaveras, Leody 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 72.9 64\nLowe, Nathaniel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.5 0.927 0.592 104 17'}, {'pitcher_id': '663992', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLovelady, Richard 4 3 0 0 0 0 2 50 28.6 1 25 0 0 0.174 0.026 0.041 0.192 83.8 -21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nGrossman, Robbie 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nSemien, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.124 0.073 83.8 -21'}, {'pitcher_id': '664076', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCleavinger, Garrett 8 8 4 0 0 0 2 25 26.7 0 0 0.5 0.5 0.444 0.409 0.511 0.401 84 2.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nGarcía, Adolis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.926 1.002 0.851 65.6 27\nSemien, Marcus 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.359 0.424 0.346 81 4\nTaveras, Leody 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.242 0.245 0.218 73.3 -52\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.471 0.8 0.541 99.2 17\nSmith, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.911 1.191 0.91 103.8 13'}, {'pitcher_id': '670955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUceta, Edwin 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.524 0.391 0.426 0.474 73.2 44', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKnizner, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.391 0.426 0.367 73.2 44\nJankowski, Travis 1 0 0 0 0 0 0 0 1 100 0.689 0.689'}, {'pitcher_id': '655889', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641302', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlexander, Tyler 45 42 12 2 0 2 8 17.8 16.9 2 4.4 0.286 0.476 0.351 0.245 0.461 0.327 87.6 9.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 13 12 3 0 0 0 2 15.4 18.8 1 7.7 0.25 0.25 0.257 0.141 0.182 0.182 88.3 22.9\nGarcía, Adolis 6 6 2 0 0 0 1 16.7 31.3 0 0 0.333 0.333 0.294 0.176 0.331 0.217 83.8 10.2\nTaveras, Leody 5 5 2 0 0 0 2 40 22.2 0 0 0.4 0.4 0.354 0.263 0.307 0.249 95.1 5.3\nGrossman, Robbie 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.231 0.032 0.034 0.25 66.8 -7\nSeager, Corey 5 4 0 0 0 0 0 0 11.1 0 0 0 0 0.145 0.288 0.353 0.36 90.7 -8\nLowe, Nathaniel 6 6 2 0 0 1 1 16.7 16.7 0 0 0.333 0.833 0.492 0.328 0.918 0.521 90.7 6.4\nDuran, Ezequiel 4 4 2 1 0 1 1 25 0 0 0 0.5 1.5 0.829 0.46 1.344 0.751 98.1 9.7\nJankowski, Travis 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.94 1.007 0.865 73 19\nSmith, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.152 0.154 0.142 88.5 -9\nHuff, Sam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.14 0.091 53.5 -7'}, {'pitcher_id': '650644', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCivale, Aaron 68 62 16 3 0 3 14 20.6 17.9 4 5.9 0.258 0.452 0.315 0.243 0.396 0.297 90.4 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrossman, Robbie 13 11 3 0 0 1 2 15.4 16.7 2 15.4 0.273 0.545 0.396 0.284 0.621 0.426 91.4 29\nGarcía, Adolis 10 9 1 0 0 0 5 50 25 0 0 0.111 0.111 0.088 0.087 0.121 0.143 82 4\nSemien, Marcus 11 11 4 2 0 0 0 0 0 0 0 0.364 0.545 0.386 0.21 0.283 0.207 89.4 19.7\nSeager, Corey 8 7 2 0 0 1 1 12.5 22.2 1 12.5 0.286 0.714 0.459 0.41 0.698 0.5 100 17.2\nTaveras, Leody 6 5 2 1 0 0 1 16.7 20 1 16.7 0.4 0.6 0.471 0.338 0.37 0.375 88.1 14.5\nLowe, Nathaniel 7 7 3 0 0 1 2 28.6 27.3 0 0 0.429 0.857 0.539 0.306 0.514 0.347 87.6 -0.4\nSmith, Josh 4 3 0 0 0 0 1 25 16.7 0 0 0 0 0 0.143 0.365 0.162 98.1 35.3\nDuran, Ezequiel 4 4 0 0 0 0 1 25 25 0 0 0 0 0 0.09 0.123 0.089 92.7 30.7\nLangford, Wyatt 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.051 0.054 0.047 84.4 30.5\nJankowski, Travis 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.441 0.588 0.736 0.568 87.3 24'}, {'pitcher_id': '592094', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdam, Jason 19 16 4 0 0 0 3 15.8 25.7 3 15.8 0.25 0.25 0.295 0.217 0.283 0.294 80.7 20.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 7 7 2 0 0 0 2 28.6 35.7 0 0 0.286 0.286 0.252 0.291 0.366 0.287 80.8 16\nGarcía, Adolis 6 5 2 0 0 0 0 0 18.2 1 16.7 0.4 0.4 0.41 0.176 0.2 0.256 74.7 37.2\nSemien, Marcus 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.061 0.074 0.065 90.2 -26\nHuff, Sam 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nGrossman, Robbie 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nDuran, Ezequiel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.27 0.333 0.249 88.1 -2\nSeager, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.171 0.49 0.281 93.7 30'}, {'pitcher_id': '621097', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWaguespack, Jacob 6 6 1 0 0 0 0 0 18.2 0 0 0.167 0.167 0.145 0.16 0.205 0.167 84.6 27.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Nathaniel 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.29 0.168 0.194 0.159 76.8 20.7\nSeager, Corey 3 3 0 0 0 0 0 0 10 0 0 0 0 0 0.153 0.217 0.176 92.3 34'}, {'pitcher_id': '682052', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '621107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEflin, Zach 50 47 12 4 0 2 8 16 17.6 3 6 0.255 0.468 0.328 0.269 0.489 0.342 88.6 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSeager, Corey 13 12 5 2 0 1 3 23.1 11.8 1 7.7 0.417 0.833 0.533 0.237 0.526 0.344 90.6 26.4\nGarcía, Adolis 6 6 2 1 0 1 1 16.7 33.3 0 0 0.333 1 0.546 0.468 1.296 0.727 101.5 9.6\nGrossman, Robbie 7 6 1 0 0 0 1 14.3 14.3 1 14.3 0.167 0.167 0.225 0.131 0.15 0.204 81 39\nSemien, Marcus 6 5 0 0 0 0 1 16.7 22.2 1 16.7 0 0 0.115 0.07 0.091 0.173 80.6 42.3\nLowe, Nathaniel 4 4 0 0 0 0 1 25 22.2 0 0 0 0 0 0.226 0.326 0.239 76.5 -20.7\nLangford, Wyatt 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.399 0.585 0.425 94.6 -10\nKnizner, Andrew 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.293 0.337 0.423 0.326 90.1 -5\nTaveras, Leody 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.437 0.509 0.411 95.2 -5.3\nSmith, Josh 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.48 0.561 0.456 85.6 26\nUreña, José 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGray, Jon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.034 0.032 67.8 -27'}, {'pitcher_id': '686752', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPepiot, Ryan 14 11 2 0 0 0 2 14.3 27.3 3 21.4 0.182 0.182 0.275 0.238 0.341 0.345 89.9 29.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Adolis 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.071 0.136 0.088 84.9 58.7\nLangford, Wyatt 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.296 0.339 0.625 0.406 102.4 18.5\nSemien, Marcus 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.231 0.032 0.038 0.251 83.4 21\nJankowski, Travis 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nSeager, Corey 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.664 0.699 0.632 91.4 5.5'}, {'pitcher_id': '621363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoche, Colin 26 23 5 4 0 0 8 30.8 30.4 3 11.5 0.217 0.391 0.304 0.235 0.388 0.314 88.8 21.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 5 4 1 1 0 0 0 0 7.7 1 20 0.25 0.5 0.388 0.276 0.381 0.367 87.8 37.3\nTaveras, Leody 4 4 0 0 0 0 2 50 40 0 0 0 0 0 0.108 0.119 0.1 98.1 -16\nSeager, Corey 4 4 2 2 0 0 1 25 37.5 0 0 0.5 1 0.615 0.298 0.859 0.462 97.1 36\nGarcía, Adolis 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.295 0.539 0.651 0.529 93.3 16\nLowe, Nathaniel 3 3 0 0 0 0 2 66.7 37.5 0 0 0 0 0 0.031 0.037 0.028 63.4 -7\nGrossman, Robbie 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.324 0.474 0.342 80.8 23.7\nDuran, Ezequiel 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nSmith, Josh 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nHuff, Sam 1 0 0 0 0 0 0 0 1 100 0.689 0.689'}, {'pitcher_id': '664126', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFairbanks, Pete 20 17 4 1 0 1 8 40 38.9 3 15 0.235 0.471 0.352 0.221 0.421 0.332 96.1 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSemien, Marcus 5 4 0 0 0 0 2 40 25 1 20 0 0 0.138 0.057 0.065 0.18 86 -21.5\nSeager, Corey 5 4 4 1 0 1 0 0 22.2 1 20 1 2 1.129 0.81 1.612 0.945 100.8 16.8\nGarcía, Adolis 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nLowe, Nathaniel 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.06 0.06 0.055 109.4 -20\nGrossman, Robbie 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSmith, Josh 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nDuran, Ezequiel 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nLangford, Wyatt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.061 0.038 88.3 32\nTaveras, Leody 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.213 0.112 92.4 33'}]}, 140: {136: [{'pitcher_id': '681982', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Grant 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.178 0.229 0.249 0.209 81.7 39', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRaleigh, Cal 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.011 0.013 0.01 76 63\nRaley, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.008 0.007 55.7 83\nMoore, Dylan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.689 0.736 0.62 95.2 9\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.49 0.406 99.7 1'}, {'pitcher_id': '592351', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Jon 81 77 14 2 1 3 28 34.6 28 4 4.9 0.182 0.351 0.249 0.204 0.339 0.256 89.7 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 20 20 4 1 1 0 8 40 28.2 0 0 0.2 0.35 0.231 0.169 0.244 0.179 88.5 11.6\nCrawford, J.P. 14 12 2 0 0 0 3 21.4 20 2 14.3 0.167 0.167 0.224 0.218 0.278 0.286 82.7 19.6\nRodríguez, Julio 15 15 2 0 0 1 7 46.7 34.4 0 0 0.133 0.333 0.197 0.213 0.423 0.271 94.2 13.1\nRaleigh, Cal 9 7 1 0 0 0 1 11.1 10.5 2 22.2 0.143 0.143 0.251 0.184 0.203 0.288 88.2 28.5\nHaniger, Mitch 9 9 3 1 0 2 3 33.3 20 0 0 0.333 1.111 0.602 0.24 0.725 0.393 93.9 0.5\nRaley, Luke 6 6 0 0 0 0 4 66.7 53.8 0 0 0 0 0 0.111 0.118 0.105 102.1 -6.5\nMoore, Dylan 3 3 2 0 0 0 1 33.3 42.9 0 0 0.667 0.667 0.591 0.572 0.692 0.551 94.1 14\nZavala, Seby 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.082 0.095 0.074 89.6 -5\nGarver, Mitch 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.273 0.297 0.245 90.9 -1\nRobles, Victor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 80 69'}, {'pitcher_id': '666720', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Yerry 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.044 0.029 83 36', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.044 0.029 83 36'}, {'pitcher_id': '571760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeaney, Andrew 124 110 30 5 1 6 28 22.6 26.9 12 9.7 0.273 0.5 0.362 0.25 0.471 0.346 87.8 17.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 31 28 9 1 1 3 9 29 34 1 3.2 0.321 0.75 0.443 0.305 0.789 0.462 92.8 16.1\nCrawford, J.P. 26 21 6 1 0 0 1 3.8 9.1 5 19.2 0.286 0.333 0.351 0.334 0.391 0.394 78.8 19.4\nMoore, Dylan 21 18 6 3 0 1 4 19 30 3 14.3 0.333 0.667 0.457 0.269 0.559 0.392 92.5 21.9\nRodríguez, Julio 14 13 3 0 0 0 4 28.6 27.3 1 7.1 0.231 0.231 0.239 0.221 0.269 0.248 98 12.1\nRaleigh, Cal 11 11 0 0 0 0 3 27.3 28.1 0 0 0 0 0 0.1 0.203 0.128 84.7 23.6\nGarver, Mitch 7 5 1 0 0 1 1 14.3 18.8 2 28.6 0.2 0.8 0.496 0.233 0.71 0.475 91.5 13.5\nRobles, Victor 4 4 2 0 0 0 1 25 42.9 0 0 0.5 0.5 0.444 0.203 0.224 0.188 71.6 27.7\nRojas, Josh 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.279 0.325 0.261 93.5 17\nZavala, Seby 4 4 1 0 0 1 3 75 50 0 0 0.25 1 0.502 0\nLocklear, Tyler 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.157 0.17 0.141 95 -4\nBliss, Ryan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.073 0.074 0.068 58.3 -40'}, {'pitcher_id': '656271', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurke, Brock 31 28 3 1 0 1 12 38.7 31.4 3 9.7 0.107 0.25 0.199 0.204 0.36 0.281 84.5 -1.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 7 6 2 0 0 1 2 28.6 12.5 1 14.3 0.333 0.833 0.511 0.287 0.574 0.408 83.1 10\nMoore, Dylan 7 6 1 1 0 0 2 28.6 40 1 14.3 0.167 0.333 0.272 0.301 0.372 0.342 85.4 1.5\nRodríguez, Julio 6 6 0 0 0 0 3 50 18.8 0 0 0 0 0 0.173 0.261 0.185 99.3 -1.7\nRaleigh, Cal 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.248 0.669 0.379 82.6 -15\nHaniger, Mitch 2 2 0 0 0 0 1 50 80 0 0 0 0 0 0.053 0.053 0.049 68.5 -2\nLocklear, Tyler 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nRojas, Josh 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRaley, Luke 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGarver, Mitch 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.042 0.045 0.038 63.2 -22\nCanzone, Dominic 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '668390', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWinn, Cole 7 7 0 0 0 0 4 57.1 44.4 0 0 0 0 0 0.058 0.067 0.053 79.8 3.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarver, Mitch 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.114 0.136 0.103 87 25\nRodríguez, Julio 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMoore, Dylan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRaley, Luke 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.233 0.267 0.213 70.6 5\nHaniger, Mitch 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.06 0.068 0.055 81.7 -19'}, {'pitcher_id': '502085', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, David 9 9 1 0 0 0 5 55.6 37.5 0 0 0.111 0.111 0.097 0.117 0.16 0.117 88.9 29.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRaley, Luke 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.087 0.102 0.082 82.6 28\nHaniger, Mitch 3 3 0 0 0 0 3 100 66.7 0 0 0 0 0 0\nRojas, Josh 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.003 0.004 0.003 85 49\nRodríguez, Julio 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRobles, Victor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.873 1.228 0.887 105.6 13'}, {'pitcher_id': '674003', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradford, Cody 4 4 0 0 0 0 1 25 15.4 0 0 0 0 0 0.093 0.209 0.127 80.5 23.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.189 0.189 0.173 83.5 -53\nCrawford, J.P. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.008 0.007 55.1 81\nRodríguez, Julio 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nRaleigh, Cal 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.177 0.638 0.328 102.9 42'}, {'pitcher_id': '643493', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPruitt, Austin 29 28 9 1 0 3 7 24.1 30 1 3.4 0.321 0.679 0.43 0.284 0.643 0.396 89.9 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 6 6 2 0 0 1 2 33.3 28.6 0 0 0.333 0.833 0.485 0.186 0.567 0.312 81.7 -1.5\nRodríguez, Julio 5 5 3 1 0 1 0 0 18.2 0 0 0.6 1.4 0.843 0.57 1.353 0.789 103.7 9.6\nZavala, Seby 3 3 0 0 0 0 2 66.7 55.6 0 0 0 0 0 0.075 0.135 0.088 94.5 21\nRaleigh, Cal 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.095 0.108 0.087 74.4 35.7\nMoore, Dylan 4 4 2 0 0 0 1 25 42.9 0 0 0.5 0.5 0.442 0.457 0.544 0.447 82 40\nCrawford, J.P. 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.524 0.214 0.264 0.361 91.9 14\nRobles, Victor 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.066 0.244 0.125 100.6 42\nRaley, Luke 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.983 3.778 1.921 108.5 24\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.041 0.026 85.3 35'}, {'pitcher_id': '547179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLorenzen, Michael 40 31 6 1 0 2 10 25 28.8 7 17.5 0.194 0.419 0.349 0.156 0.246 0.281 83.7 16.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Julio 11 9 3 0 0 1 1 9.1 30 2 18.2 0.333 0.667 0.475 0.276 0.365 0.359 89.6 4.1\nCrawford, J.P. 10 6 1 0 0 0 2 20 6.7 3 30 0.167 0.167 0.368 0.131 0.155 0.352 78.1 17.8\nRaleigh, Cal 7 5 1 0 0 1 1 14.3 30.8 2 28.6 0.2 0.8 0.495 0.139 0.234 0.312 86.3 23\nRobles, Victor 4 4 0 0 0 0 4 100 50 0 0 0 0 0 0\nRojas, Josh 4 3 0 0 0 0 0 0 20 0 0 0 0 0 0.028 0.028 0.025 67.3 18.3\nMoore, Dylan 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.004 0.006 0.005 88.4 67\nGarver, Mitch 2 2 1 1 0 0 1 50 75 0 0 0.5 1 0.619 0.397 1.075 0.6 109.8 19'}, {'pitcher_id': '489446', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYates, Kirby 9 7 2 0 0 0 2 22.2 35.3 2 22.2 0.286 0.286 0.349 0.24 0.288 0.333 85.7 16.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 4 3 0 0 0 0 1 25 42.9 1 25 0 0 0.172 0.033 0.079 0.204 90.5 39.5\nRaleigh, Cal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.034 0.039 73.8 -24\nRodríguez, Julio 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nRojas, Josh 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.87 0.593 0.703 0.563 95.9 7\nZavala, Seby 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCanzone, Dominic 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.951 1.044 0.883 77.9 20'}, {'pitcher_id': '656641', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLatz, Jacob 8 8 1 0 0 0 4 50 35 0 0 0.125 0.125 0.111 0.172 0.221 0.17 90.9 -10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.131 0.146 0.118 102.3 -6\nRaleigh, Cal 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nRodríguez, Julio 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.181 0.185 0.166 58.3 -52\nRaley, Luke 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.434 0.466 0.391 105.1 -4\nRojas, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.32 0.636 0.4 97.7 21'}, {'pitcher_id': '543135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEovaldi, Nathan 77 71 15 2 0 3 25 32.5 30.5 6 7.8 0.211 0.366 0.28 0.224 0.37 0.286 91.1 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 11 11 2 0 0 0 5 45.5 26.9 0 0 0.182 0.182 0.159 0.156 0.186 0.15 86.3 15.3\nCrawford, J.P. 15 14 2 0 0 1 3 20 26.9 1 6.7 0.143 0.357 0.239 0.206 0.32 0.254 92.3 16.6\nRaleigh, Cal 13 12 3 1 0 0 2 15.4 14.3 1 7.7 0.25 0.333 0.285 0.336 0.429 0.358 89.8 8.6\nRodríguez, Julio 13 13 3 0 0 1 7 53.8 44.8 0 0 0.231 0.462 0.297 0.191 0.407 0.252 94.1 1.3\nRaley, Luke 8 8 2 1 0 0 4 50 43.8 0 0 0.25 0.375 0.266 0.208 0.428 0.264 98.4 1.3\nGarver, Mitch 6 4 0 0 0 0 2 33.3 45.5 2 33.3 0 0 0.23 0.2 0.216 0.354 98.5 -5.5\nRojas, Josh 6 4 2 0 0 1 0 0 16.7 2 33.3 0.5 1.25 0.712 0.392 1.058 0.622 94.3 1.5\nLocklear, Tyler 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.031 0.032 0.029 64.5 -25\nCanzone, Dominic 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.331 0.355 0.3 80.5 -15\nMoore, Dylan 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '622250', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSborz, Josh 23 21 2 0 0 0 8 34.8 36.6 2 8.7 0.095 0.095 0.137 0.172 0.236 0.22 94.3 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 7 5 2 0 0 0 1 14.3 23.1 2 28.6 0.4 0.4 0.449 0.359 0.397 0.436 97.2 22.3\nRaleigh, Cal 5 5 0 0 0 0 3 60 37.5 0 0 0 0 0 0.147 0.187 0.14 100.8 -2\nRodríguez, Julio 4 4 0 0 0 0 0 0 33.3 0 0 0 0 0 0.142 0.317 0.19 87.7 20.5\nRobles, Victor 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nZavala, Seby 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nCrawford, J.P. 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.144 0.195 0.148 96.9 7\nRojas, Josh 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.074 0.175 0.094 91.7 30\nRaley, Luke 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '570632', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUreña, José 32 30 9 1 0 1 7 21.9 31.8 2 6.3 0.3 0.433 0.339 0.236 0.28 0.254 88.3 1.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 7 7 2 0 0 1 2 28.6 16.7 0 0 0.286 0.714 0.422 0.136 0.193 0.142 78.8 3.2\nCrawford, J.P. 3 3 1 0 0 0 0 0 11.1 0 0 0.333 0.333 0.293 0.307 0.388 0.304 95.1 13\nRobles, Victor 5 4 3 1 0 0 1 20 40 1 20 0.75 1 0.73 0.426 0.449 0.448 70.2 -12\nHaniger, Mitch 3 3 1 0 0 0 0 0 50 0 0 0.333 0.333 0.296 0.39 0.443 0.357 87.6 4\nGarver, Mitch 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.339 0.372 0.444 102.1 -3.5\nLocklear, Tyler 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.009 0.02 0.012 97 45\nCastillo, Luis 3 3 0 0 0 0 3 100 57.1 0 0 0 0 0 0\nMoore, Dylan 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.297 0.334 0.274 104.1 -10\nRodríguez, Julio 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.101 0.125 0.104 87.3 -30\nRaleigh, Cal 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.444 0.313 0.419 0.31 91.6 9\nRaley, Luke 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.311 0.347 0.283 97.5 -4'}, {'pitcher_id': '641312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Shaun 3 2 0 0 0 0 2 66.7 44.4 1 33.3 0 0 0.23 0.23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobles, Victor 2 1 0 0 0 0 1 50 42.9 1 50 0 0 0.345 0.345\nZavala, Seby 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '642546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Jonathan 35 32 9 2 0 1 9 25.7 30.1 3 8.6 0.281 0.438 0.341 0.242 0.429 0.319 93.9 10.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 10 9 4 1 0 0 2 20 42.1 1 10 0.444 0.556 0.458 0.289 0.538 0.383 95.9 3\nMoore, Dylan 8 6 1 0 0 0 1 12.5 14.3 2 25 0.167 0.167 0.282 0.217 0.31 0.34 89.6 6.2\nRaleigh, Cal 4 4 0 0 0 0 4 100 50 0 0 0 0 0 0\nRodríguez, Julio 4 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.311 0.305 0.5 0.342 97.3 -8.3\nHaniger, Mitch 2 2 2 0 0 0 0 0 0 0 0 1 1 0.884 0.627 0.712 0.584 100.2 7\nRojas, Josh 2 2 1 0 0 1 0 0 20 0 0 0.5 2 1.043 0.431 1.402 0.75 96.8 44.5\nGarver, Mitch 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.069 0.152 0.095 92.4 25\nZavala, Seby 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCanzone, Dominic 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 83.1 80\nRaley, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.377 0.489 0.375 82 24'}, {'pitcher_id': '669391', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTinoco, Jesus 12 8 2 0 0 0 2 16.7 44.4 4 33.3 0.25 0.25 0.378 0.138 0.292 0.351 78.6 22.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRojas, Josh 3 2 1 0 0 0 1 33.3 20 1 33.3 0.5 0.5 0.526 0.211 0.235 0.36 98.2 1\nCrawford, J.P. 2 0 0 0 0 0 0 0 100 2 100 0.69 0.69\nLocklear, Tyler 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHaniger, Mitch 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.006 0.006 0.005 67.1 62\nMoore, Dylan 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nRodríguez, Julio 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.195 0.212 0.177 46.1 19\nRaleigh, Cal 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.011 0.031 0.018 96.4 45\nRaley, Luke 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.406 1.55 0.8 101.2 36\nCanzone, Dominic 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.06 0.065 0.057 62.5 -27'}, {'pitcher_id': '683004', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '600917', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeclerc, José 38 32 6 0 0 1 13 34.2 41.5 3 7.9 0.188 0.281 0.23 0.201 0.373 0.273 91.1 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHaniger, Mitch 13 12 3 0 0 0 5 38.5 50 1 7.7 0.25 0.25 0.256 0.142 0.27 0.213 92 26.4\nRodríguez, Julio 7 7 2 0 0 1 1 14.3 27.3 0 0 0.286 0.714 0.422 0.433 0.951 0.588 89.5 16.5\nRaleigh, Cal 7 5 0 0 0 0 3 42.9 50 1 14.3 0 0 0.098 0.131 0.181 0.221 91.2 23\nCrawford, J.P. 4 2 0 0 0 0 0 0 12.5 1 25 0 0 0.23 0.115 0.118 0.3 87.3 17.5\nMoore, Dylan 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nRojas, Josh 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0 0 0 0.023 80.4 39\nRaley, Luke 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nCanzone, Dominic 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.48 0.53 0.435 102.7 1\nGarver, Mitch 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.329 0.368 0.296 101.9 -9'}, {'pitcher_id': '641540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDunning, Dane 95 87 24 5 0 6 24 25.3 26.2 5 5.3 0.276 0.54 0.376 0.265 0.497 0.356 87.5 6.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, J.P. 23 21 7 2 0 0 3 13 6.5 2 8.7 0.333 0.429 0.362 0.304 0.427 0.348 88.1 10.9\nRodríguez, Julio 21 19 4 0 0 2 7 33.3 39.5 1 4.8 0.211 0.526 0.35 0.239 0.544 0.366 84 -11.3\nRaleigh, Cal 18 18 6 2 0 3 3 16.7 28.9 0 0 0.333 0.944 0.531 0.36 0.89 0.521 95.8 9.2\nHaniger, Mitch 8 8 2 0 0 0 4 50 18.8 0 0 0.25 0.25 0.22 0.273 0.314 0.267 79.2 -11\nRaley, Luke 7 6 2 0 0 0 2 28.6 26.7 0 0 0.333 0.333 0.355 0.241 0.333 0.313 72.1 14\nRojas, Josh 5 5 1 1 0 0 0 0 10 0 0 0.2 0.4 0.254 0.131 0.148 0.122 88.3 30\nZavala, Seby 3 3 1 0 0 1 2 66.7 37.5 0 0 0.333 1.333 0.668 0.15 0.532 0.275 99.4 30\nMoore, Dylan 3 2 1 0 0 0 1 33.3 0 1 33.3 0.5 0.5 0.524 0.392 0.451 0.475 97.1 10\nGarver, Mitch 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nCanzone, Dominic 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.029 0.039 0.029 85.5 11.5\nRobles, Victor 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.077 0.08 0.397 71.1 -32'}], 137: [{'pitcher_id': '681982', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Grant 12 10 5 0 0 1 1 8.3 20 2 16.7 0.5 0.8 0.578 0.252 0.373 0.336 86.9 -1.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEstrada, Thairo 3 3 3 0 0 0 0 0 0 0 0 1 1 0.885 0.371 0.412 0.336 80.1 -23.7\nSoler, Jorge 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nYastrzemski, Mike 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRamos, Heliot 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.458 0.508 0.417 100.3 1.5\nSlater, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.065 0.067 0.062 89.7 -24\nBailey, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.026 0.017 93.3 44\nWisely, Brett 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nConforto, Michael 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.329 1.257 0.648 100 36\nFlores, Wilmer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.133 0.073 58 0'}, {'pitcher_id': '592351', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Jon 105 95 19 4 0 0 32 30.5 24.7 6 5.7 0.2 0.242 0.227 0.18 0.263 0.229 86.6 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYastrzemski, Mike 19 16 3 1 0 0 7 36.8 32.3 2 10.5 0.188 0.25 0.267 0.164 0.218 0.25 92.3 10.7\nAhmed, Nick 18 17 5 0 0 0 4 22.2 34.4 0 0 0.294 0.294 0.244 0.189 0.235 0.182 85.1 -0.4\nFlores, Wilmer 17 17 3 0 0 0 2 11.8 15.2 0 0 0.176 0.176 0.155 0.184 0.282 0.199 91 13.3\nConforto, Michael 14 11 1 0 0 0 4 28.6 16 3 21.4 0.091 0.091 0.21 0.125 0.179 0.251 86.9 9.9\nSoler, Jorge 10 10 2 2 0 0 5 50 28.6 0 0 0.2 0.4 0.249 0.187 0.314 0.212 92.7 31.4\nCasali, Curt 9 7 2 0 0 0 3 33.3 16.7 0 0 0.286 0.286 0.309 0.309 0.371 0.347 71.5 3.6\nSlater, Austin 6 5 2 1 0 0 2 33.3 9.1 1 16.7 0.4 0.6 0.463 0.211 0.635 0.399 67.1 12\nEstrada, Thairo 4 4 0 0 0 0 1 25 25 0 0 0 0 0 0.261 0.299 0.245 87.5 10.3\nChapman, Matt 4 4 0 0 0 0 2 50 28.6 0 0 0 0 0 0.073 0.085 0.071 97.2 31\nBailey, Patrick 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.119 0.129 0.109 88.9 1\nWebb, Logan 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.071 0.073 0.064 69.3 -6\nRamos, Heliot 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '666720', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '571760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeaney, Andrew 97 86 21 5 0 4 23 23.7 29.9 10 10.3 0.244 0.442 0.336 0.293 0.528 0.387 90.6 18.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoler, Jorge 18 15 1 1 0 0 4 22.2 29 3 16.7 0.067 0.133 0.185 0.182 0.372 0.317 89.2 6.4\nChapman, Matt 19 18 4 1 0 1 4 21.1 36.8 1 5.3 0.222 0.444 0.297 0.298 0.532 0.366 90.9 30.3\nFlores, Wilmer 17 16 7 2 0 1 3 17.6 16.2 0 0 0.438 0.75 0.515 0.387 0.725 0.484 95.3 24\nSlater, Austin 13 11 1 0 0 0 6 46.2 48 2 15.4 0.091 0.091 0.175 0.228 0.374 0.321 89.2 11.8\nEstrada, Thairo 8 7 2 0 0 1 0 0 15.8 1 12.5 0.286 0.714 0.456 0.332 0.593 0.435 89.6 8.6\nYastrzemski, Mike 6 5 2 0 0 0 2 33.3 30 1 16.7 0.4 0.4 0.408 0.396 0.443 0.424 82.5 7.7\nRamos, Heliot 5 4 2 1 0 1 1 20 36.4 1 20 0.5 1.5 0.81 0.394 1.231 0.668 102.3 27.3\nCasali, Curt 4 3 0 0 0 0 1 25 14.3 1 25 0 0 0.173 0.033 0.057 0.2 89.5 39.5\nConforto, Michael 4 4 1 0 0 0 2 50 60 0 0 0.25 0.25 0.221 0.29 0.388 0.29 103.9 6.5\nAhmed, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.119 0.12 0.109 54.9 7\nBailey, Patrick 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.963 1.134 0.919 90.9 15'}, {'pitcher_id': '656271', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurke, Brock 5 5 2 0 0 1 2 40 38.5 0 0 0.4 1 0.575 0.397 0.852 0.52 86.2 -6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.435 0.384 0.425 0.347 80.3 14\nConforto, Michael 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.276 0.276 0.252 68.1 -54\nRamos, Heliot 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.943 3.133 1.655 110.1 21\nEstrada, Thairo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '668390', 'player_data': '', 'team_data': 'Cole Winn has not faced anyone on this team.'}, {'pitcher_id': '502085', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, David 25 22 7 0 0 5 7 28 30.8 3 12 0.318 1 0.566 0.226 0.682 0.413 93.9 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 6 5 2 0 0 2 1 16.7 10 1 16.7 0.4 1.6 0.806 0.173 0.523 0.357 95.9 45\nSoler, Jorge 6 6 2 0 0 1 2 33.3 60 0 0 0.333 0.833 0.492 0.213 0.672 0.369 93.4 14.8\nBailey, Patrick 3 3 1 0 0 1 1 33.3 0 0 0 0.333 1.333 0.668 0.376 1.379 0.705 103.6 5\nConforto, Michael 3 3 1 0 0 1 1 33.3 20 0 0 0.333 1.333 0.695 0.325 1.141 0.599 89.6 -7\nEstrada, Thairo 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.295 0.2 0.228 0.185 89.4 17.5\nChapman, Matt 2 0 0 0 0 0 0 0 100 2 100 0.691 0.691\nYastrzemski, Mike 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.061 0.061 0.058 86.5 -13'}, {'pitcher_id': '674003', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradford, Cody 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '643493', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPruitt, Austin 24 22 5 1 0 2 3 12.5 18.6 2 8.3 0.227 0.545 0.349 0.199 0.361 0.273 84.3 20.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 5 4 0 0 0 0 0 0 0 1 20 0 0 0.138 0.108 0.135 0.222 78.5 22\nSoler, Jorge 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.011 0.023 0.016 85.2 50.5\nBailey, Patrick 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nChapman, Matt 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.525 0.52 0.742 0.537 104.9 23\nEstrada, Thairo 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.941 0.285 0.524 0.336 66.1 23.3\nSlater, Austin 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.03 0.042 0.028 81.1 10\nAhmed, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.054 0.075 0.051 73.6 -23.5\nYastrzemski, Mike 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.803 2.538 1.382 105.1 23\nWisely, Brett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 84.9 40'}, {'pitcher_id': '547179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLorenzen, Michael 76 73 23 3 0 3 20 26.3 24.1 3 3.9 0.315 0.479 0.356 0.253 0.424 0.306 91.5 13.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlores, Wilmer 15 14 7 0 0 3 3 20 12 1 6.7 0.5 1.143 0.692 0.354 0.806 0.498 94.7 20.5\nConforto, Michael 13 12 3 1 0 0 2 15.4 16.7 1 7.7 0.25 0.333 0.285 0.179 0.224 0.219 88.4 11.7\nSoler, Jorge 12 12 2 1 0 0 7 58.3 39.3 0 0 0.167 0.25 0.178 0.172 0.186 0.162 88.5 3.8\nChapman, Matt 7 6 2 0 0 0 0 0 7.7 1 14.3 0.333 0.333 0.352 0.311 0.618 0.439 92.8 -1.7\nEstrada, Thairo 8 8 3 1 0 0 2 25 25 0 0 0.375 0.5 0.376 0.239 0.439 0.284 92.4 6.2\nYastrzemski, Mike 5 5 2 0 0 0 2 40 20 0 0 0.4 0.4 0.354 0.296 0.332 0.277 83.8 26.3\nBailey, Patrick 6 6 1 0 0 0 2 33.3 20 0 0 0.167 0.167 0.148 0.155 0.187 0.146 92 30.3\nRamos, Heliot 3 3 2 0 0 0 0 0 40 0 0 0.667 0.667 0.593 0.549 0.61 0.509 92.2 4.7\nAhmed, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.27 0.74 0.418 100.3 24\nWisely, Brett 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.023 0.065 0.035 89.2 36\nCasali, Curt 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.459 0.648 0.474 97.5 20\nSlater, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '489446', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYates, Kirby 30 24 2 0 0 1 14 46.7 35.7 4 13.3 0.083 0.208 0.236 0.09 0.176 0.229 88.2 20.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAhmed, Nick 10 8 0 0 0 0 5 50 47.6 2 20 0 0 0.139 0.01 0.017 0.147 88.6 45.7\nConforto, Michael 4 3 0 0 0 0 2 50 16.7 1 25 0 0 0.174 0.106 0.172 0.261 92.2 20\nSlater, Austin 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.23 0.23\nEstrada, Thairo 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0.242 0.298 0.344 0.43 79.1 3\nFlores, Wilmer 4 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.896 0.377 0.94 0.591 92.9 2\nChapman, Matt 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.011 0.026 0.015 87 33\nYastrzemski, Mike 2 2 0 0 0 0 2 100 20 0 0 0 0 0 0\nCasali, Curt 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBailey, Patrick 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '656641', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLatz, Jacob 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYastrzemski, Mike 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0'}, {'pitcher_id': '543135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEovaldi, Nathan 75 69 13 1 0 5 12 16 20.7 5 6.7 0.188 0.42 0.291 0.223 0.451 0.321 88.2 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 16 12 3 0 0 2 3 18.8 18.8 3 18.8 0.25 0.75 0.489 0.223 0.677 0.457 92.9 13.8\nConforto, Michael 11 11 1 0 0 0 2 18.2 40 0 0 0.091 0.091 0.08 0.259 0.371 0.276 90.7 2.6\nFlores, Wilmer 10 10 1 1 0 0 1 10 11.8 0 0 0.1 0.2 0.128 0.144 0.165 0.134 81.8 19.4\nCasali, Curt 5 5 1 0 0 1 0 0 15.4 0 0 0.2 0.8 0.403 0.198 0.512 0.298 74 36\nYastrzemski, Mike 6 5 2 0 0 2 2 33.3 38.5 1 16.7 0.4 1.6 0.786 0.279 1.071 0.561 95.6 33.3\nSoler, Jorge 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.045 0.057 0.042 82.9 40.7\nBailey, Patrick 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.471 0.596 0.542 95.4 20.5\nAhmed, Nick 5 5 1 0 0 0 1 20 0 0 0 0.2 0.2 0.178 0.004 0.004 0.003 83.9 35\nWisely, Brett 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.238 0.288 0.228 83.9 23.3\nBrooks, Trenton 3 3 1 0 0 0 0 0 28.6 0 0 0.333 0.333 0.296 0.431 0.606 0.441 98.7 26\nEstrada, Thairo 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.296 0.105 0.11 0.095 76.2 -25\nRamos, Heliot 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.339 0.713 0.448 102.3 10\nHoward, Spencer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.349 0.393 0.346 83.9 -66\nSlater, Austin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.19 0.237 0.203 95.9 7.5'}, {'pitcher_id': '622250', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSborz, Josh 17 15 0 0 0 0 5 29.4 45.2 2 11.8 0 0 0.081 0.177 0.304 0.26 90.5 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.094 0.109 0.086 78.3 40\nSoler, Jorge 2 0 0 0 0 0 0 0 42.9 2 100 0.692 0.692\nCasali, Curt 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.202 0.234 0.182 87.3 -2.5\nSlater, Austin 2 2 0 0 0 0 1 50 83.3 0 0 0 0 0 0.416 0.537 0.419 101.1 10\nFlores, Wilmer 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.188 0.577 0.305 92.6 11.5\nAhmed, Nick 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.2 0.208 0.182 94.6 -16.5\nYastrzemski, Mike 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nConforto, Michael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBailey, Patrick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.271 1.007 0.524 97.9 35'}, {'pitcher_id': '570632', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUreña, José 63 57 14 3 0 4 7 11.1 16.4 4 6.3 0.246 0.509 0.347 0.294 0.551 0.385 87.8 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nConforto, Michael 25 22 9 2 0 2 2 8 15.2 2 8 0.409 0.773 0.518 0.408 0.794 0.527 93 22.6\nFlores, Wilmer 14 14 2 0 0 1 1 7.1 4.5 0 0 0.143 0.357 0.21 0.192 0.273 0.201 80.4 25.7\nYastrzemski, Mike 9 7 3 1 0 1 0 0 5.9 2 22.2 0.429 1 0.622 0.512 1.116 0.689 96 10.6\nSoler, Jorge 6 6 0 0 0 0 1 16.7 20 0 0 0 0 0 0.148 0.155 0.135 81.4 -21.4\nEstrada, Thairo 3 3 0 0 0 0 3 100 85.7 0 0 0 0 0 0\nChapman, Matt 4 4 0 0 0 0 0 0 20 0 0 0 0 0 0.144 0.32 0.201 80.3 -19.5\nAhmed, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.074 0.065 84.2 -17\nCasali, Curt 1 0 0 0 0 0 0 0 0 0 0'}, {'pitcher_id': '641312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Shaun 19 18 6 0 0 0 3 15.8 30 1 5.3 0.333 0.333 0.312 0.326 0.447 0.351 92.7 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAhmed, Nick 9 9 3 0 0 0 2 22.2 26.7 0 0 0.333 0.333 0.29 0.363 0.519 0.378 90.8 18\nConforto, Michael 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.52 0.37 0.51 0.483 89.5 13.5\nSoler, Jorge 4 4 2 0 0 0 0 0 28.6 0 0 0.5 0.5 0.44 0.464 0.584 0.453 99.3 -0.3\nYastrzemski, Mike 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nFlores, Wilmer 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.013 0.018 0.012 88.5 42\nCasali, Curt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.003 90 42'}, {'pitcher_id': '642546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Jonathan 11 9 0 0 0 0 4 36.4 34.8 2 18.2 0 0 0.126 0.139 0.315 0.279 84.2 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 4 3 0 0 0 0 1 25 40 1 25 0 0 0.172 0.168 0.188 0.288 76 -11.5\nYastrzemski, Mike 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.527 1.993 0.849 99.8 32\nWisely, Brett 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nEstrada, Thairo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.123 0.172 0.119 78.2 31\nSoler, Jorge 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFlores, Wilmer 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRamos, Heliot 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.096 0.109 0.093 91.2 -23'}, {'pitcher_id': '669391', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTinoco, Jesus 15 13 3 0 1 1 4 26.7 26.1 2 13.3 0.231 0.615 0.382 0.232 0.533 0.361 94.8 20.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAhmed, Nick 5 5 1 0 1 0 1 20 0 0 0 0.2 0.6 0.306 0.159 0.372 0.215 88.3 19.3\nYastrzemski, Mike 3 3 2 0 0 1 0 0 12.5 0 0 0.667 1.667 0.936 0.665 1.606 0.916 106.2 11.3\nConforto, Michael 2 1 0 0 0 0 0 0 0 1 50 0 0 0.349 0.001 0.001 0.349 80.3 86\nBailey, Patrick 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nEstrada, Thairo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nChapman, Matt 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSoler, Jorge 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.229 0.25 0.207 101.5 -11'}, {'pitcher_id': '683004', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '600917', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeclerc, José 20 17 1 1 0 0 7 35 26.5 1 5 0.059 0.118 0.169 0.058 0.101 0.163 83.2 31.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 10 7 0 0 0 0 3 30 22.2 1 10 0 0 0.213 0.027 0.079 0.242 79.5 48.8\nBailey, Patrick 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.019 0.019 0.018 67 -28\nFlores, Wilmer 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0 0 0 79.4 85\nSoler, Jorge 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.006 0.01 0.006 97.2 52\nEstrada, Thairo 2 2 1 1 0 0 0 0 20 0 0 0.5 1 0.622 0.323 0.506 0.355 98 5\nRamos, Heliot 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.102 0.112 0.092 74.2 -2\nAhmed, Nick 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSlater, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '641540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDunning, Dane 33 29 8 1 0 1 13 39.4 44.1 4 12.1 0.276 0.414 0.343 0.237 0.354 0.308 85.7 5.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChapman, Matt 12 8 2 1 0 0 4 33.3 22.2 4 33.3 0.25 0.375 0.409 0.279 0.366 0.416 97.9 8.3\nSoler, Jorge 7 7 0 0 0 0 4 57.1 68.8 0 0 0 0 0 0.071 0.123 0.087 82.9 -7\nBailey, Patrick 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.294 0.242 0.282 0.237 71.8 16\nRamos, Heliot 3 3 0 0 0 0 3 100 66.7 0 0 0 0 0 0\nEstrada, Thairo 3 3 2 0 0 0 1 33.3 50 0 0 0.667 0.667 0.588 0.524 0.59 0.49 91.2 11\nConforto, Michael 3 3 2 0 0 1 0 0 20 0 0 0.667 1.667 0.962 0.449 1.113 0.644 93.8 4.3\nAhmed, Nick 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.244 0.262 0.222 62.1 1.5'}], 138: [{'pitcher_id': '681982', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Grant 3 2 2 1 0 0 0 0 25 0 0 1 1.5 0.709 0.596 1.005 0.505 98.9 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 1 0 0 0 0 0 0 0 50 0 0 0 0.153 92.7 22\nArenado, Nolan 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.317 0.884 0.494 96.7 26\nGorman, Nolan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.874 1.126 0.868 107.4 10'}, {'pitcher_id': '592351', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Jon 80 72 19 5 0 5 18 22.5 26.2 4 5 0.264 0.542 0.354 0.228 0.361 0.281 90.5 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 32 29 9 1 0 2 4 12.5 16.4 2 6.3 0.31 0.552 0.395 0.254 0.39 0.315 93 11.2\nGoldschmidt, Paul 22 19 3 1 0 1 7 31.8 33.3 2 9.1 0.158 0.368 0.249 0.155 0.204 0.213 82.4 10.1\nCarpenter, Matt 9 9 4 2 0 1 2 22.2 29.4 0 0 0.444 1 0.599 0.362 0.694 0.442 93.6 17\nGorman, Nolan 3 3 0 0 0 0 3 100 80 0 0 0 0 0 0\nDonovan, Brendan 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.201 0.267 0.2 85.6 3.7\nLynn, Lance 2 1 0 0 0 0 1 50 25 0 0 0 0 0 0\nBurleson, Alec 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.668 0.227 0.609 0.349 95.3 11.3\nArenado, Nolan 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.709 0.383 0.464 0.371 100.9 -6.7\nGray, Sonny 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.217 0.252 0.195 77.6 2'}, {'pitcher_id': '666720', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '571760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeaney, Andrew 26 22 4 1 0 0 4 15.4 22.2 3 11.5 0.182 0.227 0.257 0.215 0.299 0.285 90.8 20.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 9 9 4 1 0 0 2 22.2 27.8 0 0 0.444 0.556 0.432 0.34 0.504 0.366 87.7 4.4\nArenado, Nolan 8 7 0 0 0 0 1 12.5 20 1 12.5 0 0 0.086 0.057 0.077 0.136 90.2 42\nCarpenter, Matt 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.23\nCarlson, Dylan 2 2 0 0 0 0 1 50 22.2 0 0 0 0 0 0.004 0.005 0.004 99.3 46\nDonovan, Brendan 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.344 0.517 0.566 0.577 95.2 7\nGoldschmidt, Paul 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.313 0.333 0.503 103.4 -7'}, {'pitcher_id': '656271', 'player_data': '', 'team_data': 'Brock Burke has not faced anyone on this team.'}, {'pitcher_id': '668390', 'player_data': '', 'team_data': 'Cole Winn has not faced anyone on this team.'}, {'pitcher_id': '502085', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, David 20 18 3 1 0 0 6 30 37.9 1 5 0.167 0.222 0.221 0.171 0.345 0.274 89.3 22.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 5 5 1 1 0 0 1 20 33.3 0 0 0.2 0.4 0.246 0.241 0.294 0.224 85.5 26.3\nGoldschmidt, Paul 5 5 1 0 0 0 1 20 25 0 0 0.2 0.2 0.178 0.336 0.915 0.52 102.1 2.7\nDonovan, Brendan 4 3 0 0 0 0 1 25 33.3 0 0 0 0 0.18 0.039 0.058 0.213 75.5 59.5\nCarpenter, Matt 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0\nCrawford, Brandon 2 1 0 0 0 0 1 50 100 1 50 0 0 0.344 0.344\nGorman, Nolan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.146 0.166 0.138 90.2 -5'}, {'pitcher_id': '674003', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '643493', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPruitt, Austin 7 7 3 0 0 0 1 14.3 16.7 0 0 0.429 0.429 0.378 0.475 0.659 0.489 91.8 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.542 0.662 0.532 78.7 28\nFermín, José 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.169 0.323 0.198 93.8 23\nBurleson, Alec 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.702 1.109 0.775 100.5 17\nArenado, Nolan 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nGoldschmidt, Paul 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.666 0.746 0.612 98.7 9'}, {'pitcher_id': '547179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLorenzen, Michael 58 51 11 1 0 1 13 22.4 34.2 7 12.1 0.216 0.294 0.277 0.189 0.263 0.257 87.3 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCarpenter, Matt 23 19 4 1 0 0 6 26.1 34.2 4 17.4 0.211 0.263 0.29 0.18 0.214 0.263 89.3 21.8\nGoldschmidt, Paul 13 13 2 0 0 0 5 38.5 46.9 0 0 0.154 0.154 0.135 0.15 0.18 0.144 89.1 2.1\nArenado, Nolan 11 10 1 0 0 1 1 9.1 30 1 9.1 0.1 0.4 0.25 0.133 0.351 0.248 89.1 28\nCrawford, Brandon 7 5 3 0 0 0 1 14.3 23.5 2 28.6 0.6 0.6 0.573 0.345 0.405 0.448 94.2 18\nCarlson, Dylan 4 4 1 0 0 0 0 0 14.3 0 0 0.25 0.25 0.221 0.337 0.397 0.328 68 15.8'}, {'pitcher_id': '489446', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYates, Kirby 39 33 6 1 0 0 10 25.6 26.8 6 15.4 0.182 0.212 0.251 0.172 0.234 0.244 88.6 22.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 15 14 3 0 0 0 5 33.3 35.5 1 6.7 0.214 0.214 0.222 0.164 0.203 0.193 91.1 20.3\nCrawford, Brandon 11 8 1 1 0 0 3 27.3 26.9 3 27.3 0.125 0.25 0.3 0.101 0.181 0.278 82.4 17.2\nGoldschmidt, Paul 6 5 2 0 0 0 2 33.3 21.4 1 16.7 0.4 0.4 0.409 0.352 0.457 0.407 92.4 8\nCarpenter, Matt 5 4 0 0 0 0 0 0 0 1 20 0 0 0.138 0.188 0.263 0.196 87.1 31.8\nCarlson, Dylan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.032 0.024 96.8 57\nWinn, Masyn 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.043 0.083 0.052 83.7 31'}, {'pitcher_id': '656641', 'player_data': '', 'team_data': 'Jacob Latz has not faced anyone on this team.'}, {'pitcher_id': '543135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEovaldi, Nathan 53 52 16 3 1 1 12 22.6 22 1 1.9 0.308 0.462 0.339 0.251 0.299 0.242 83.8 10.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 13 12 2 1 0 0 5 38.5 27.3 1 7.7 0.167 0.25 0.218 0.054 0.054 0.049 87.3 -17\nCrawford, Brandon 17 17 6 0 0 1 2 11.8 22.9 0 0 0.353 0.529 0.384 0.402 0.428 0.365 80.5 18\nCarpenter, Matt 12 12 6 1 1 0 2 16.7 4.8 0 0 0.5 0.75 0.538 0.23 0.267 0.221 96.4 2\nArenado, Nolan 7 7 2 1 0 0 1 14.3 33.3 0 0 0.286 0.429 0.304 0.146 0.269 0.181 82.7 13.7\nLynn, Lance 2 2 0 0 0 0 1 50 25 0 0 0 0 0\nGray, Sonny 2 2 0 0 0 0 1 50 25 0 0 0 0 0'}, {'pitcher_id': '622250', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSborz, Josh 8 8 4 1 0 0 1 12.5 23.5 0 0 0.5 0.625 0.479 0.456 0.53 0.429 90.3 3.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 4 4 3 1 0 0 0 0 22.2 0 0 0.75 1 0.741 0.55 0.642 0.515 94.3 5.3\nArenado, Nolan 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.29 0.483 0.559 0.457 84.9 0.7\nBurleson, Alec 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '570632', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUreña, José 64 57 20 4 0 5 3 4.7 13.9 5 7.8 0.351 0.684 0.437 0.351 0.644 0.438 91.7 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 21 19 5 1 0 3 1 4.8 11.1 1 4.8 0.263 0.789 0.416 0.235 0.675 0.368 89.1 22.2\nCrawford, Brandon 16 14 5 1 0 1 0 0 17.2 1 6.3 0.357 0.643 0.409 0.436 0.795 0.541 92.9 17.2\nCarpenter, Matt 8 6 5 1 0 0 0 0 0 2 25 0.833 1 0.768 0.714 0.827 0.672 97.2 10\nGoldschmidt, Paul 8 7 3 1 0 1 1 12.5 15.4 1 12.5 0.429 1 0.6 0.307 0.581 0.41 97.2 9.8\nDonovan, Brendan 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.242 0.267 0.223 89.3 23.5\nLynn, Lance 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.072 0.078 0.065 68.9 0\nBurleson, Alec 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.529 0.646 0.515 99.3 -7.3\nGorman, Nolan 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.094 0.111 0.09 94 4.5\nCarlson, Dylan 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.06 0.061 0.06 59.1 -32'}, {'pitcher_id': '641312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Shaun 8 7 1 0 0 0 0 0 10 1 12.5 0.143 0.143 0.195 0.243 0.333 0.302 90.7 21.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 7 7 1 0 0 0 0 0 0 0 0 0.143 0.143 0.124 0.243 0.333 0.247 90.7 21.7\nGoldschmidt, Paul 1 0 0 0 0 0 0 0 100 1 100 0.69 0.69'}, {'pitcher_id': '642546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Jonathan 4 2 0 0 0 0 1 25 42.9 1 25 0 0 0.175 0.186 0.361 0.284 92.4 30', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArenado, Nolan 3 1 0 0 0 0 0 0 0 1 33.3 0 0 0.233 0.373 0.722 0.379 92.4 30\nCrawford, Brandon 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0'}, {'pitcher_id': '669391', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTinoco, Jesus 6 6 3 0 0 2 1 16.7 10 0 0 0.5 1.5 0.791 0.472 1.38 0.747 97.8 22.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.936 0.423 1.44 0.751 99.1 24.7\nMikolas, Miles 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCarpenter, Matt 1 1 1 0 0 1 0 0 0 0 0 1 4 1.94 0.813 3.124 1.533 101.8 29\nGoldschmidt, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.753 0.834 0.694 89.8 11'}, {'pitcher_id': '683004', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '600917', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeclerc, José 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.176 0.259 0.294 0.245 98.7 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGoldschmidt, Paul 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nArenado, Nolan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.564 0.643 0.535 102 3\nCarpenter, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.17 0.185 0.153 92.3 -18'}, {'pitcher_id': '641540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDunning, Dane 18 17 5 0 0 1 3 16.7 22.2 1 5.6 0.294 0.471 0.346 0.334 0.493 0.37 93.3 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrawford, Brandon 3 3 0 0 0 0 2 66.7 22.2 0 0 0 0 0 0.159 0.171 0.143 97 3\nGoldschmidt, Paul 3 2 1 0 0 0 0 0 28.6 1 33.3 0.5 0.5 0.526 0.393 0.431 0.469 95.6 0.5\nGorman, Nolan 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.061 0.111 0.07 90.4 38.5\nArenado, Nolan 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.668 0.199 0.694 0.361 84.4 20.3\nBurleson, Alec 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.294 0.574 0.688 0.546 92.6 7\nDonovan, Brendan 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.636 0.844 0.631 102.3 11'}], 139: [{'pitcher_id': '681982', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Grant 7 7 2 0 0 2 1 14.3 16.7 0 0 0.286 1.143 0.573 0.263 0.905 0.469 89.3 42', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Isaac 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.007 0.015 0.009 84 37\nArozarena, Randy 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.434 1.288 0.691 99.1 26\nLowe, Josh 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.014 0.022 0.016 88.3 53\nLowe, Brandon 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.937 3.704 1.864 106.6 29\nCaballero, José 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.008 0.007 58.8 81'}, {'pitcher_id': '592351', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGray, Jon 43 40 16 0 1 1 9 20.9 29.6 2 4.7 0.4 0.525 0.422 0.381 0.488 0.404 83.4 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 11 11 4 0 1 0 2 18.2 25 0 0 0.364 0.545 0.387 0.271 0.307 0.26 79.9 -4.7\nDíaz, Yandy 6 5 3 0 0 0 2 33.3 45.5 1 16.7 0.6 0.6 0.556 0.316 0.364 0.365 87.8 -15.7\nWalls, Taylor 6 6 1 0 0 0 3 50 28.6 0 0 0.167 0.167 0.147 0.192 0.215 0.179 87.1 19.7\nCaballero, José 4 3 2 0 0 0 0 0 36.4 0 0 0.667 0.667 0.623 0.578 0.617 0.578 74.4 26.3\nSiri, Jose 6 6 3 0 0 0 0 0 12.5 0 0 0.5 0.5 0.442 0.565 0.663 0.544 85.3 17.2\nParedes, Isaac 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.441 0.592 0.664 0.546 78.5 13\nArozarena, Randy 3 3 1 0 0 1 1 33.3 40 0 0 0.333 1.333 0.691 0.464 1.252 0.729 87.7 31.5\nLowe, Josh 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.2 0.204 0.44 82.5 -1\nLowe, Brandon 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.807 0.955 0.766 96.7 11\nEflin, Zach 1 1 0 0 0 0 1 100 0 0 0 0 0 0'}, {'pitcher_id': '666720', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodríguez, Yerry 7 7 2 0 0 1 2 28.6 41.2 0 0 0.286 0.714 0.425 0.147 0.294 0.183 86.5 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaballero, José 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.186 0.621 0.32 96.7 34\nSiri, Jose 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nArozarena, Randy 1 1 1 0 0 0 0 0 25 0 0 1 1 0.889 0.434 0.52 0.424 96.2 1\nRosario, Amed 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.107 0.137 0.099 54.6 2\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.053 0.049 88.1 -35\nParedes, Isaac 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.251 0.727 0.39 96.8 33'}, {'pitcher_id': '571760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHeaney, Andrew 48 44 9 2 0 1 15 31.3 31.7 3 6.3 0.205 0.318 0.262 0.18 0.282 0.239 87.7 19.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 11 10 3 0 0 0 4 36.4 29.2 1 9.1 0.3 0.3 0.304 0.136 0.212 0.198 93.7 22.5\nRosario, Amed 10 10 2 0 0 0 2 20 22.7 0 0 0.2 0.2 0.176 0.356 0.433 0.344 85.9 3.7\nArozarena, Randy 7 5 0 0 0 0 2 28.6 41.7 2 28.6 0 0 0.199 0.055 0.161 0.26 86.2 49.7\nCaballero, José 8 7 1 1 0 0 3 37.5 20 0 0 0.143 0.286 0.246 0.121 0.253 0.228 89.8 19.5\nSiri, Jose 4 4 0 0 0 0 1 25 54.5 0 0 0 0 0 0.168 0.183 0.154 85.6 13.3\nParedes, Isaac 4 4 2 1 0 1 1 25 0 0 0 0.5 1.5 0.812 0.229 0.579 0.327 88.2 35\nWalls, Taylor 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.002 0.002 0.002 86 69\nDeLuca, Jonny 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.154 0.169 0.141 68.8 -50'}, {'pitcher_id': '656271', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurke, Brock 18 15 2 1 0 0 4 22.2 35.5 2 11.1 0.133 0.2 0.236 0.148 0.256 0.26 87.1 7.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSiri, Jose 4 3 0 0 0 0 2 50 50 1 25 0 0 0.172 0.033 0.064 0.203 83.1 29\nWalls, Taylor 2 1 0 0 0 0 1 50 40 1 50 0 0 0.344 0.344\nDíaz, Yandy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.404 0.787 0.502 95.1 18\nParedes, Isaac 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.131 0.2 0.144 77.8 -14.5\nLowe, Josh 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nArozarena, Randy 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.15 0.152 0.136 87.4 -9\nRosario, Amed 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.631 0.149 0.264 0.165 90.2 33\nLowe, Brandon 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.057 0.059 0.054 78.7 -24\nCaballero, José 1 0 0 0 0 0 0 0 0 0 0.726 0.726'}, {'pitcher_id': '668390', 'player_data': '', 'team_data': 'Cole Winn has not faced anyone on this team.'}, {'pitcher_id': '502085', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobertson, David 28 27 7 1 0 0 8 28.6 39.2 1 3.6 0.259 0.296 0.258 0.244 0.396 0.286 93.4 2.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 7 7 1 0 0 0 1 14.3 23.1 0 0 0.143 0.143 0.126 0.277 0.347 0.269 90.8 -7\nDíaz, Yandy 6 6 1 0 0 0 3 50 55.6 0 0 0.167 0.167 0.146 0.114 0.128 0.104 93.6 32.7\nLowe, Brandon 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.383 1.198 0.648 99.8 -4\nArozarena, Randy 3 2 2 1 0 0 0 0 0 1 33.3 1 1.5 0.939 0.413 0.708 0.545 96 8\nLowe, Josh 3 3 2 0 0 0 1 33.3 57.1 0 0 0.667 0.667 0.588 0.395 0.481 0.38 96.3 9\nSiri, Jose 2 2 1 0 0 0 0 0 60 0 0 0.5 0.5 0.444 0.373 0.498 0.373 92.8 -9\nParedes, Isaac 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nPalacios, Richie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCaballero, José 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRortvedt, Ben 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.053 0.053 0.049 79.8 -18'}, {'pitcher_id': '674003', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBradford, Cody 4 4 1 0 0 1 1 25 20 0 0 0.25 1 0.501 0.268 0.371 0.28 86.5 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nSiri, Jose 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.123 0.445 0.231 95.4 36\nLowe, Josh 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.066 0.07 0.065 73.1 -7\nLowe, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.883 0.971 0.823 90.9 12'}, {'pitcher_id': '643493', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPruitt, Austin 7 7 1 0 0 0 0 0 25 0 0 0.143 0.143 0.126 0.129 0.18 0.129 86.4 32.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 3 3 0 0 0 0 0 0 28.6 0 0 0 0 0 0.091 0.178 0.105 94.1 14.3\nArozarena, Randy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 94.7 75\nLowe, Josh 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.003 0.003 0.003 68.2 67\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.114 0.182 0.121 79.3 31\nParedes, Isaac 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.514 0.538 0.463 80.1 12'}, {'pitcher_id': '547179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLorenzen, Michael 15 14 6 1 0 0 2 13.3 32 0 0 0.429 0.5 0.426 0.334 0.486 0.378 95.1 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 9 9 4 1 0 0 1 11.1 28.6 0 0 0.444 0.556 0.434 0.291 0.492 0.334 93.4 13.3\nArozarena, Randy 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.404 0.455 0.365 92.9 -1\nPalacios, Richie 2 2 2 0 0 0 0 0 33.3 0 0 1 1 0.884 0.72 0.845 0.69 99.6 8\nSiri, Jose 2 1 0 0 0 0 0 0 0 0 0 0 0 0.36 0.215 0.24 0.458 101.3 -19\nJackson, Alex 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '489446', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYates, Kirby 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.294 0.124 0.145 0.114 93.6 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.441 0.186 0.218 0.171 93.6 -2\nCaballero, José 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '656641', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLatz, Jacob 7 5 0 0 0 0 1 14.3 50 1 14.3 0 0 0.202 0.037 0.123 0.25 86.6 50.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.053 0.19 0.1 96.4 45.5\nArozarena, Randy 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nSiri, Jose 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.011 0.023 0.015 85.9 50\nDíaz, Yandy 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.017 0.024 0.021 67.9 60\nParedes, Isaac 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '543135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEovaldi, Nathan 137 127 25 6 0 1 37 27 31.2 9 6.6 0.197 0.268 0.236 0.218 0.305 0.259 86.6 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLowe, Brandon 30 27 5 1 0 1 8 26.7 36.1 3 10 0.185 0.333 0.265 0.258 0.46 0.34 91.8 10.1\nArozarena, Randy 25 24 4 2 0 0 9 36 38.3 1 4 0.167 0.25 0.198 0.191 0.248 0.209 88.3 6\nDíaz, Yandy 25 23 4 0 0 0 3 12 14 2 8 0.174 0.174 0.196 0.272 0.331 0.296 86.2 12.5\nParedes, Isaac 14 13 2 0 0 0 7 50 35.3 1 7.1 0.154 0.154 0.176 0.048 0.049 0.091 75.2 27.8\nLowe, Josh 10 8 4 1 0 0 0 0 20 2 20 0.5 0.625 0.528 0.386 0.588 0.47 90.2 9\nRosario, Amed 12 12 2 0 0 0 3 25 33.3 0 0 0.167 0.167 0.147 0.197 0.231 0.189 84.6 -5.1\nSiri, Jose 8 8 1 0 0 0 4 50 42.1 0 0 0.125 0.125 0.11 0.177 0.241 0.183 88.5 23.8\nCaballero, José 4 3 1 0 0 0 0 0 11.1 0 0 0.333 0.333 0.404 0.309 0.342 0.403 76.4 28.7\nPalacios, Richie 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.124 0.145 0.115 79.9 12.5\nWalls, Taylor 4 4 1 1 0 0 1 25 37.5 0 0 0.25 0.5 0.315 0.145 0.186 0.14 86.7 28\nRortvedt, Ben 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.635 0.253 0.274 0.233 68.3 16'}, {'pitcher_id': '622250', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSborz, Josh 15 12 2 1 0 1 2 13.3 33.3 3 20 0.167 0.5 0.355 0.234 0.521 0.392 85.4 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 3 1 0 0 0 0 0 0 50 2 66.7 0 0 0.461 0.168 0.168 0.512 82.3 -1\nArozarena, Randy 4 3 0 0 0 0 0 0 20 1 25 0 0 0.174 0.09 0.146 0.253 81.3 17.7\nLowe, Brandon 2 2 1 0 0 1 1 50 50 0 0 0.5 2 1.003 0.471 1.815 0.921 105.6 26\nRosario, Amed 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.105 0.29 0.159 88.1 12\nWalls, Taylor 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.92 1.132 0.898 98.4 13\nSiri, Jose 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nParedes, Isaac 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.058 0.058 0.055 85.3 -17\nLowe, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.24 0.216 61.9 -64'}, {'pitcher_id': '570632', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUreña, José 37 33 8 2 0 0 8 21.6 33.3 3 8.1 0.242 0.303 0.272 0.272 0.359 0.31 86.8 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 19 17 6 2 0 0 4 21.1 31 2 10.5 0.353 0.471 0.387 0.37 0.519 0.416 93.1 5.6\nDíaz, Yandy 6 6 1 0 0 0 1 16.7 44.4 0 0 0.167 0.167 0.145 0.241 0.268 0.227 79.7 -12\nRortvedt, Ben 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nLowe, Brandon 4 4 1 0 0 0 0 0 16.7 0 0 0.25 0.25 0.217 0.255 0.296 0.238 85.6 41\nArozarena, Randy 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.033 0.038 0.038 72.7 -20\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.096 0.103 0.104 80.3 -12\nEflin, Zach 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.068 0.068 0.065 65.5 -33'}, {'pitcher_id': '641312', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Shaun 5 4 1 0 0 1 1 20 16.7 1 20 0.25 1 0.526 0.189 0.532 0.375 95.1 16.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.647 0.198 0.649 0.345 93.2 35.5\nLowe, Brandon 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nWalls, Taylor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.164 0.179 0.148 99 -22'}, {'pitcher_id': '642546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Jonathan 5 4 1 0 0 0 2 40 28.6 1 20 0.25 0.25 0.314 0.141 0.157 0.244 101 32.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCaballero, José 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.294 0.184 0.202 0.17 108.4 1\nArozarena, Randy 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nParedes, Isaac 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.023 0.018 93.6 64'}, {'pitcher_id': '669391', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '622786', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTinoco, Jesus 4 4 0 0 0 0 1 25 22.2 0 0 0 0 0 0.133 0.145 0.129 90.4 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRosario, Amed 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.141 0.159 0.142 97.7 -3\nSiri, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.238 0.213 90.4 -4\nDíaz, Yandy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.025 0.016 83.1 49'}, {'pitcher_id': '683004', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '600917', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeclerc, José 26 22 7 2 0 0 3 11.5 10.2 2 7.7 0.318 0.409 0.374 0.289 0.41 0.363 91.8 30.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 7 6 3 1 0 0 0 0 8.3 1 14.3 0.5 0.667 0.525 0.46 0.608 0.494 95.5 20.3\nParedes, Isaac 5 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.538 0.29 0.329 0.452 80.3 52.3\nLowe, Josh 3 2 1 0 0 0 0 0 12.5 1 33.3 0.5 0.5 0.526 0.304 0.543 0.473 101.8 14\nArozarena, Randy 4 4 0 0 0 0 1 25 18.2 0 0 0 0 0 0.033 0.04 0.031 84.5 37.3\nWalls, Taylor 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.051 0.11 0.068 92.2 27\nCaballero, José 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.093 0.363 0.183 93.9 56.5\nSiri, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.737 1.071 0.77 103.6 14\nRosario, Amed 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nPalacios, Richie 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.969 1.112 0.908 90.4 14'}, {'pitcher_id': '641540', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDunning, Dane 63 56 10 1 0 3 16 25.4 28 4 6.3 0.179 0.357 0.269 0.225 0.373 0.306 92.5 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDíaz, Yandy 9 9 1 0 0 0 4 44.4 35.3 0 0 0.111 0.111 0.098 0.09 0.107 0.089 93.4 -6\nLowe, Brandon 11 10 0 0 0 0 2 18.2 13 0 0 0 0 0.066 0.108 0.196 0.182 82.6 26.8\nArozarena, Randy 10 8 0 0 0 0 3 30 28.6 1 10 0 0 0.069 0.208 0.356 0.339 95.2 7.7\nRosario, Amed 8 7 2 0 0 0 0 0 0 1 12.5 0.286 0.286 0.307 0.34 0.428 0.377 97.3 -9.7\nWalls, Taylor 5 5 1 0 0 0 2 40 22.2 0 0 0.2 0.2 0.176 0.215 0.259 0.207 97.7 14.7\nSiri, Jose 6 5 3 1 0 1 1 16.7 36.4 0 0 0.6 1.4 0.827 0.598 0.988 0.688 100.5 9\nRortvedt, Ben 4 4 0 0 0 0 3 75 60 0 0 0 0 0 0.069 0.078 0.062 95.2 -4\nPalacios, Richie 5 4 1 0 0 1 0 0 25 1 20 0.25 1 0.556 0.447 0.971 0.614 91.4 23.3\nLowe, Josh 3 3 2 0 0 1 0 0 33.3 0 0 0.667 1.667 0.962 0.181 0.57 0.306 85.7 11.7\nParedes, Isaac 2 1 0 0 0 0 1 50 100 1 50 0 0 0.349 0.349'}]}, 141: {142: [{'pitcher_id': '666201', 'player_data': '', 'team_data': ''}, {'pitcher_id': '554340', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Yimi 43 42 13 2 0 1 14 32.6 24.7 1 2.3 0.31 0.429 0.324 0.245 0.318 0.254 85 20.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 12 12 4 1 0 1 2 16.7 23.1 0 0 0.333 0.667 0.41 0.349 0.473 0.352 91.6 6.5\nSantana, Carlos 6 6 3 1 0 0 0 0 0 0 0 0.5 0.667 0.504 0.293 0.34 0.283 79.9 34\nCastro, Willi 3 3 0 0 0 0 3 100 37.5 0 0 0 0 0 0\nCorrea, Carlos 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.025 0.026 0.024 72.7 40.5\nKepler, Max 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.16 0.232 0.165 87.7 23\nFarmer, Kyle 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.584 0.435 0.574 0.432 82.6 24\nMiranda, Jose 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.295 0.329 0.466 0.346 95.5 20\nLarnach, Trevor 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.027 0.054 0.029 88.5 28\nVázquez, Christian 3 3 2 0 0 0 1 33.3 16.7 0 0 0.667 0.667 0.588 0.145 0.169 0.142 69.7 26.5\nBuxton, Byron 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.971 1.089 0.801 79.7 19\nPaddack, Chris 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nLewis, Royce 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '621244', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerríos, José 125 105 24 5 1 2 28 22.4 23.4 18 14.4 0.229 0.352 0.306 0.206 0.358 0.311 88.1 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 40 31 8 1 1 1 7 17.5 23.8 9 22.5 0.258 0.452 0.385 0.212 0.451 0.371 92.8 24.3\nCorrea, Carlos 18 15 5 1 0 0 1 5.6 13.8 3 16.7 0.333 0.4 0.381 0.19 0.404 0.323 87.8 10.2\nMargot, Manuel 13 12 5 1 0 0 0 0 11.1 0 0 0.417 0.5 0.369 0.376 0.483 0.405 88.3 8.9\nCastro, Willi 11 9 1 0 0 0 3 27.3 19 1 9.1 0.111 0.111 0.143 0.169 0.196 0.226 84.6 12.3\nBuxton, Byron 12 10 2 1 0 1 4 33.3 43.8 2 16.7 0.2 0.6 0.386 0.295 0.643 0.442 98.2 -0.8\nVázquez, Christian 9 8 2 1 0 0 1 11.1 9.1 1 11.1 0.25 0.375 0.309 0.13 0.151 0.187 74.4 19\nJeffers, Ryan 6 6 0 0 0 0 5 83.3 42.9 0 0 0 0 0 0.074 0.074 0.067 57.6 24\nKepler, Max 4 4 0 0 0 0 1 25 12.5 0 0 0 0 0 0.095 0.112 0.101 84.8 -24\nMiranda, Jose 5 4 1 0 0 0 2 40 37.5 1 20 0.25 0.25 0.315 0.252 0.314 0.337 98.7 -9\nLewis, Royce 2 1 0 0 0 0 0 0 16.7 1 50 0 0 0.348 0.277 0.279 0.472 71.5 7\nLarnach, Trevor 3 3 0 0 0 0 3 100 100 0 0 0 0 0 0\nFarmer, Kyle 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.027 0.027 0.026 76.1 -35'}, {'pitcher_id': '579328', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKikuchi, Yusei 122 114 29 3 2 3 23 18.9 25.7 8 6.6 0.254 0.395 0.304 0.272 0.41 0.319 90.3 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 25 23 11 2 1 0 3 12 18.4 2 8 0.478 0.652 0.5 0.352 0.495 0.392 88.3 6\nMargot, Manuel 21 20 5 0 1 0 5 23.8 28.6 1 4.8 0.25 0.35 0.275 0.283 0.321 0.284 90 11.1\nSantana, Carlos 15 12 3 0 0 1 2 13.3 17.9 3 20 0.25 0.5 0.394 0.305 0.44 0.395 94.8 10\nVázquez, Christian 9 9 0 0 0 0 2 22.2 22.2 0 0 0 0 0 0.195 0.477 0.272 93.9 22.1\nBuxton, Byron 10 9 3 1 0 0 4 40 36.8 1 10 0.333 0.444 0.371 0.315 0.464 0.374 99 5.4\nCastro, Willi 9 9 2 0 0 1 0 0 27.8 0 0 0.222 0.556 0.329 0.296 0.65 0.401 88.8 17.3\nFarmer, Kyle 7 7 0 0 0 0 2 28.6 35.7 0 0 0 0 0 0.067 0.075 0.064 76.2 15.2\nJeffers, Ryan 8 8 0 0 0 0 3 37.5 43.8 0 0 0 0 0 0.214 0.404 0.259 94.9 10.6\nKepler, Max 6 5 1 0 0 0 1 16.7 9.1 1 16.7 0.2 0.2 0.262 0.184 0.283 0.281 92.7 32.8\nMiranda, Jose 5 5 2 0 0 1 0 0 20 0 0 0.4 1 0.592 0.26 0.384 0.273 94.7 18.2\nMartin, Austin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.29 0.325 0.269 85.2 28.3\nLarnach, Trevor 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.077 0.087 0.07 83.5 -4\nLewis, Royce 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.491 0.579 0.476 80.6 8.5'}, {'pitcher_id': '605447', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomano, Jordan 46 41 7 1 0 1 14 30.4 29 3 6.5 0.171 0.268 0.242 0.171 0.314 0.258 90.9 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 6 5 1 0 0 1 1 16.7 17.4 1 16.7 0.2 0.8 0.438 0.24 0.75 0.441 84.6 -8.5\nMargot, Manuel 9 8 3 0 0 0 2 22.2 31.3 1 11.1 0.375 0.375 0.372 0.306 0.49 0.384 98.6 -2\nCorrea, Carlos 6 4 0 0 0 0 1 16.7 30 0 0 0 0 0.241 0.019 0.025 0.254 67.8 44.7\nCastro, Willi 6 6 2 0 0 0 1 16.7 27.3 0 0 0.333 0.333 0.294 0.236 0.293 0.229 91.7 4.8\nKepler, Max 5 5 1 1 0 0 2 40 36.4 0 0 0.2 0.4 0.254 0.178 0.304 0.205 95.6 -4.7\nBuxton, Byron 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.088 0.319 0.164 102.8 42\nSantana, Carlos 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.23 0.005 0.007 0.233 79.2 63\nJeffers, Ryan 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.004 0.008 0.005 98.8 52\nLarnach, Trevor 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nFarmer, Kyle 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.002 0.002 0.002 88.1 77\nLewis, Royce 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.409 0.453 0.37 104.7 -7\nMiranda, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.38 0.691 0.436 100.6 19'}, {'pitcher_id': '592332', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGausman, Kevin 145 131 35 7 0 5 44 30.3 30.3 12 8.3 0.267 0.435 0.333 0.22 0.383 0.293 89.5 11.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 29 26 5 0 0 1 12 41.4 38 2 6.9 0.192 0.308 0.266 0.124 0.155 0.185 89.5 4.3\nCorrea, Carlos 21 17 4 1 0 0 6 28.6 24.1 4 19 0.235 0.294 0.317 0.235 0.262 0.309 83.3 -3.3\nSantana, Carlos 23 19 6 4 0 1 2 8.7 22.2 3 13 0.316 0.684 0.439 0.315 0.622 0.412 93.8 25.7\nKepler, Max 16 15 4 0 0 1 3 18.8 20.6 1 6.3 0.267 0.467 0.335 0.223 0.377 0.282 82.3 6.3\nLarnach, Trevor 8 7 3 0 0 0 4 50 47.1 1 12.5 0.429 0.429 0.42 0.253 0.52 0.368 100.6 7.3\nBuxton, Byron 10 10 1 0 0 0 3 30 22.2 0 0 0.1 0.1 0.088 0.166 0.372 0.231 92.6 26.1\nMiranda, Jose 7 7 2 0 0 0 3 42.9 29.4 0 0 0.286 0.286 0.253 0.257 0.404 0.279 101.4 15.5\nFarmer, Kyle 7 7 2 0 0 0 3 42.9 20 0 0 0.286 0.286 0.248 0.125 0.143 0.119 79.1 0.5\nMargot, Manuel 7 6 1 0 0 0 2 28.6 43.8 1 14.3 0.167 0.167 0.225 0.103 0.128 0.188 87.2 -2.3\nLewis, Royce 4 4 4 0 0 2 0 0 12.5 0 0 1 2.5 1.443 0.655 1.745 0.987 99.4 15.5\nJeffers, Ryan 4 4 0 0 0 0 3 75 28.6 0 0 0 0 0 0.163 0.275 0.182 107.3 15\nCastro, Willi 5 5 2 2 0 0 0 0 25 0 0 0.4 0.8 0.506 0.364 0.768 0.476 81.8 25.6\nLópez, Pablo 4 4 1 0 0 0 3 75 83.3 0 0 0.25 0.25 0.22 0.128 0.142 0.118 105.3 2'}, {'pitcher_id': '663893', 'player_data': '', 'team_data': 'Brendon Little has not faced anyone on this team.'}, {'pitcher_id': '650893', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Génesis 18 16 5 1 0 0 4 22.2 26.5 2 11.1 0.313 0.375 0.344 0.271 0.46 0.354 92.8 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Kyle 7 7 1 0 0 0 2 28.6 16.7 0 0 0.143 0.143 0.124 0.26 0.417 0.288 89.4 14.6\nSantana, Carlos 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.012 0.025 0.015 88 36.5\nCastro, Willi 2 1 1 0 0 0 0 0 66.7 1 50 1 1 0.789 0.486 0.536 0.571 90.7 5\nMargot, Manuel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.576 1.406 0.826 101.4 16.5\nJeffers, Ryan 1 0 0 0 0 0 0 0 1 100 0.699 0.699\nVázquez, Christian 1 1 1 1 0 0 0 0 50 0 0 1 2 1.261 0.303 0.425 0.307 88.1 22\nKepler, Max 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.523 0.574 0.489 108.4 -1'}, {'pitcher_id': '647315', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPop, Zach 9 9 2 1 0 1 2 22.2 23.8 0 0 0.222 0.667 0.373 0.281 0.646 0.386 93.7 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.013 0.022 0.013 81.1 36\nKepler, Max 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.511 0.268 97.2 37\nFarmer, Kyle 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.175 0.217 0.162 102.7 -22\nCastro, Willi 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.227 0.255 0.206 75 5\nJeffers, Ryan 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.769 3.101 1.578 110.8 39\nMiranda, Jose 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nCorrea, Carlos 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.909 1.042 0.848 95.4 12'}, {'pitcher_id': '663878', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPearson, Nate 21 21 6 2 0 0 6 28.6 31 0 0 0.286 0.381 0.288 0.211 0.263 0.204 93.8 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 4 4 2 0 0 0 2 50 46.2 0 0 0.5 0.5 0.44 0.203 0.216 0.185 105.7 -6\nJeffers, Ryan 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.037 0.107 0.059 98.2 51\nVázquez, Christian 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.09 0.097 0.083 82.2 -28\nLarnach, Trevor 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.003 0.003 0.003 99.3 72\nLewis, Royce 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.441 0.433 0.492 0.406 92.9 12\nCorrea, Carlos 3 3 3 2 0 0 0 0 0 0 0 1 1.667 1.132 0.357 0.524 0.371 90.9 17\nSantana, Carlos 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCastro, Willi 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.423 0.471 0.39 97.9 1\nKepler, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.221 0.294 0.204 93.5 -5\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.651 0.725 0.593 96.2 9'}, {'pitcher_id': '684320', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657024', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Erik 42 37 5 1 0 3 10 23.8 20.9 5 11.9 0.135 0.405 0.276 0.223 0.49 0.344 89.8 26.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 9 8 2 0 0 1 0 0 12.5 1 11.1 0.25 0.625 0.406 0.235 0.476 0.348 82.8 40.3\nCorrea, Carlos 7 5 0 0 0 0 0 0 11.1 2 28.6 0 0 0.198 0.513 1.219 0.713 96.1 34.6\nKepler, Max 6 6 2 1 0 1 2 33.3 30 0 0 0.333 1 0.535 0.309 0.488 0.336 95 4\nBuxton, Byron 4 4 1 0 0 1 3 75 18.2 0 0 0.25 1 0.485 0.222 0.825 0.409 104 25\nSantana, Carlos 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.075 0.079 0.071 88.2 21\nVázquez, Christian 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.19 0.475 0.266 94.9 21.7\nJeffers, Ryan 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.232 0.002 0.002 0.233 89.9 62\nLarnach, Trevor 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.084 0.09 0.077 86.2 -27.5\nCastro, Willi 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.045 0.071 0.052 75.5 54\nFarmer, Kyle 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nLewis, Royce 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMiranda, Jose 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '669270', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '670950', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRichards, Trevor 57 54 12 3 0 2 18 31.6 32.8 2 3.5 0.222 0.389 0.269 0.223 0.461 0.304 91.7 27.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 14 13 3 0 0 1 4 28.6 20.7 0 0 0.231 0.462 0.271 0.288 0.592 0.376 90.8 17.3\nMargot, Manuel 10 10 1 1 0 0 2 20 28.6 0 0 0.1 0.2 0.122 0.195 0.397 0.247 88.7 44.6\nFarmer, Kyle 4 4 0 0 0 0 3 75 61.5 0 0 0 0 0 0.008 0.019 0.011 88.6 33\nVázquez, Christian 7 6 4 1 0 0 1 14.3 9.1 1 14.3 0.667 0.833 0.652 0.431 0.564 0.466 95.9 19\nMiranda, Jose 3 2 1 0 0 1 1 33.3 25 1 33.3 0.5 2 0.92 0.204 0.78 0.502 101 35\nCastro, Willi 4 4 0 0 0 0 2 50 60 0 0 0 0 0 0.027 0.029 0.026 90.7 13.5\nCorrea, Carlos 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.293 0.198 0.227 0.189 80.6 44.3\nLarnach, Trevor 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.016 0.024 0.015 87 29\nKepler, Max 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.105 0.219 0.133 96.2 23\nJeffers, Ryan 3 3 1 1 0 0 1 33.3 25 0 0 0.333 0.667 0.414 0.541 2.012 1.036 104 29.5\nBuxton, Byron 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.002 0.002 0.002 90.7 48\nPaddack, Chris 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.377 0.425 0.348 99.9 -5.5'}, {'pitcher_id': '605135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBassitt, Chris 71 57 15 2 0 5 13 18.3 23.6 10 14.1 0.263 0.561 0.403 0.3 0.567 0.425 86.4 15.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 20 15 5 1 0 1 3 15 16 4 20 0.333 0.6 0.469 0.348 0.563 0.468 85.1 17.1\nSantana, Carlos 14 11 3 1 0 2 2 14.3 25 3 21.4 0.273 0.909 0.524 0.361 0.936 0.566 93.5 27.6\nCastro, Willi 8 6 2 0 0 1 2 25 36.8 0 0 0.333 0.833 0.541 0.353 0.689 0.509 85.7 24.3\nKepler, Max 9 7 1 0 0 1 0 0 13.3 1 11.1 0.143 0.571 0.3 0.29 0.719 0.417 89.3 23.4\nFarmer, Kyle 5 4 1 0 0 0 2 40 28.6 1 20 0.25 0.25 0.316 0.26 0.284 0.351 93.1 11\nMargot, Manuel 4 3 1 0 0 0 0 0 20 1 25 0.333 0.333 0.392 0.214 0.234 0.325 79 -22.7\nVázquez, Christian 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.102 0.113 0.095 83.1 -22.5\nLarnach, Trevor 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.586 0.377 0.428 0.357 79.6 18\nBuxton, Byron 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.204 0.222 0.186 70.3 -24.5\nJeffers, Ryan 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.002 0.002 0.002 88.1 73'}, {'pitcher_id': '641835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMayza, Tim 34 31 8 0 1 3 6 17.6 21.4 2 5.9 0.258 0.613 0.392 0.316 0.563 0.403 91.4 9.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 8 8 3 0 0 0 2 25 30.8 0 0 0.375 0.375 0.329 0.356 0.393 0.332 88.4 1.3\nCastro, Willi 6 6 0 0 0 0 1 16.7 16.7 0 0 0 0 0 0.053 0.059 0.051 85.4 3.8\nVázquez, Christian 4 3 1 0 0 0 1 25 20 1 25 0.333 0.333 0.394 0.263 0.323 0.364 96.3 21\nSantana, Carlos 5 4 1 0 0 1 0 0 16.7 0 0 0.25 1 0.558 0.331 0.714 0.496 95.3 25.5\nMargot, Manuel 4 4 1 0 1 0 0 0 0 0 0 0.25 0.75 0.392 0.438 0.548 0.425 95.7 -4\nCorrea, Carlos 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.244 0.263 0.22 77.5 13\nLarnach, Trevor 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.334 0.367 0.301 77 28\nFarmer, Kyle 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nJeffers, Ryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMiranda, Jose 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.949 3.267 1.769 109.2 22\nBuxton, Byron 1 1 1 0 0 1 0 0 0 0 0 1 4 1.98 0.993 3.772 1.891 108.4 23'}, {'pitcher_id': '670102', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrancis, Bowden 6 6 1 0 0 1 1 16.7 28.6 0 0 0.167 0.667 0.334 0.241 0.666 0.367 93.4 1.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJeffers, Ryan 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.035 0.086 0.049 91.1 29\nCorrea, Carlos 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.399 0.459 0.359 103 -7\nKepler, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.124 0.168 0.116 94.2 -22\nCastro, Willi 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.057 0.054 74.2 -24\nLarnach, Trevor 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.797 3.138 1.577 104.5 32'}, {'pitcher_id': '643338', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreen, Chad 55 50 8 1 0 3 15 27.3 26.5 5 9.1 0.16 0.36 0.258 0.247 0.524 0.354 93.3 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 19 17 4 1 0 0 4 21.1 27.9 2 10.5 0.235 0.294 0.277 0.281 0.359 0.32 91.2 6.5\nCorrea, Carlos 12 12 2 0 0 2 2 16.7 17.6 0 0 0.167 0.667 0.331 0.273 0.724 0.407 95.6 13.7\nSantana, Carlos 8 7 1 0 0 1 4 50 29.2 1 12.5 0.143 0.571 0.338 0.16 0.553 0.34 95.8 12\nMargot, Manuel 6 5 0 0 0 0 1 16.7 18.2 1 16.7 0 0 0.115 0.287 0.703 0.45 90.9 38.3\nJeffers, Ryan 4 4 0 0 0 0 2 50 45.5 0 0 0 0 0 0.137 0.507 0.26 91.6 44\nKepler, Max 4 3 1 0 0 0 0 0 20 1 25 0.333 0.333 0.393 0.402 0.662 0.505 96.7 16.3\nBuxton, Byron 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nCastro, Willi 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '502179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspino, Paolo 10 10 3 0 0 1 5 50 22.2 0 0 0.3 0.6 0.383 0.139 0.314 0.19 94.9 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Kyle 5 5 3 0 0 1 1 20 22.2 0 0 0.6 1.2 0.767 0.26 0.597 0.359 97.9 14.8\nPaddack, Chris 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.044 0.08 0.051 83.1 28\nMargot, Manuel 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nCorrea, Carlos 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLópez, Pablo 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}], 143: [{'pitcher_id': '666201', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManoah, Alek 28 24 3 0 0 1 5 17.9 17.9 4 14.3 0.125 0.25 0.234 0.162 0.242 0.246 83.5 16.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Brandon 7 7 0 0 0 0 4 57.1 35.3 0 0 0 0 0 0.147 0.201 0.147 83.8 19\nMerrifield, Whit 6 5 0 0 0 0 0 0 10 1 16.7 0 0 0.115 0.176 0.276 0.269 87.9 22\nStott, Bryson 3 2 0 0 0 0 0 0 14.3 1 33.3 0 0 0.232 0.234 0.496 0.43 96.8 15\nPache, Cristian 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.108 0.111 0.104 65.6 -37.5\nHarper, Bryce 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.066 0.099 0.384 73.2 55\nSchwarber, Kyle 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.014 0.014 0.354 102.8 56\nBohm, Alec 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.186 0.186 0.171 64.8 -20\nSosa, Edmundo 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.002 0.002 0.002 62.2 77\nCastellanos, Nick 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.443 0.419 0.659 0.458 101.2 23'}, {'pitcher_id': '554340', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Yimi 34 30 8 2 0 2 13 38.2 31.3 4 11.8 0.267 0.533 0.375 0.234 0.521 0.358 89 17.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 9 7 3 1 0 0 3 33.3 26.7 2 22.2 0.429 0.571 0.49 0.277 0.549 0.432 96 10\nSchwarber, Kyle 7 6 2 0 0 1 1 14.3 23.1 1 14.3 0.333 0.833 0.512 0.395 0.947 0.576 89.5 24.4\nBohm, Alec 5 5 0 0 0 0 4 80 30.8 0 0 0 0 0 0.026 0.026 0.024 85 -10\nDahl, David 3 3 1 0 0 1 1 33.3 33.3 0 0 0.333 1.333 0.647 0.329 1.258 0.617 96.3 40\nCastellanos, Nick 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.151 0.185 0.148 63.8 6\nMerrifield, Whit 3 3 2 1 0 0 0 0 16.7 0 0 0.667 1 0.707 0.392 0.562 0.412 94.3 15.3\nStott, Bryson 2 1 0 0 0 0 1 50 100 1 50 0 0 0.348 0.348\nMarchán, Rafael 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMarsh, Brandon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '621244', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerríos, José 109 100 33 7 1 4 20 18.3 17.6 6 5.5 0.33 0.54 0.39 0.304 0.474 0.363 87.7 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 54 51 18 5 1 1 4 7.4 7.8 2 3.7 0.353 0.549 0.384 0.368 0.5 0.391 86.9 10.1\nCastellanos, Nick 21 21 8 1 0 1 6 28.6 28.9 0 0 0.381 0.571 0.408 0.272 0.44 0.305 88.3 14.9\nSchwarber, Kyle 9 7 0 0 0 0 3 33.3 28.6 1 11.1 0 0 0.157 0.134 0.374 0.319 94.3 44\nHarper, Bryce 6 4 2 0 0 1 1 16.7 16.7 2 33.3 0.5 1.25 0.726 0.274 0.346 0.415 79.5 5\nMarsh, Brandon 8 8 1 1 0 0 2 25 13.3 0 0 0.125 0.25 0.155 0.19 0.459 0.265 95.7 24.5\nStott, Bryson 5 5 3 0 0 1 2 40 25 0 0 0.6 1.2 0.769 0.432 0.932 0.576 100.5 15.7\nStubbs, Garrett 4 3 1 0 0 0 1 25 40 1 25 0.333 0.333 0.394 0.09 0.094 0.236 57.3 -3\nSosa, Edmundo 2 1 0 0 0 0 1 50 60 0 0 0 0 0.362 0.362'}, {'pitcher_id': '579328', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKikuchi, Yusei 35 34 9 2 0 1 9 25.7 21.8 0 0 0.265 0.412 0.298 0.251 0.403 0.288 87.4 17.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 12 11 4 1 0 1 2 16.7 13.3 0 0 0.364 0.727 0.471 0.291 0.604 0.394 92.1 21.7\nCastellanos, Nick 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.098 0.117 0.098 75.6 -0.3\nSchwarber, Kyle 3 3 0 0 0 0 2 66.7 16.7 0 0 0 0 0 0.03 0.03 0.028 74.7 -40\nSosa, Edmundo 2 2 2 0 0 0 0 0 20 0 0 1 1 0.883 0.556 0.604 0.512 83.9 2\nRojas, Johan 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.622 0.442 0.517 0.413 93.5 15\nMarsh, Brandon 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.295 0.415 0.798 0.507 98.6 19\nStott, Bryson 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.416 0.466 0.379 86.6 23\nHarper, Bryce 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.017 0.025 0.016 79.3 52.5\nBohm, Alec 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.214 0.253 0.196 82.8 24'}, {'pitcher_id': '605447', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomano, Jordan 26 25 8 1 0 0 6 23.1 28.6 1 3.8 0.32 0.36 0.313 0.351 0.468 0.369 91.5 4.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBohm, Alec 7 6 1 0 0 0 1 14.3 20 1 14.3 0.167 0.167 0.226 0.141 0.162 0.215 87.7 -3.6\nMarsh, Brandon 5 5 2 0 0 0 1 20 18.2 0 0 0.4 0.4 0.353 0.39 0.495 0.384 91.2 -5.3\nCastellanos, Nick 3 3 2 0 0 0 1 33.3 33.3 0 0 0.667 0.667 0.59 0.399 0.474 0.382 82 11\nHarper, Bryce 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.294 0.365 0.474 0.365 109.1 -3\nSchwarber, Kyle 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.442 0.383 0.436 0.354 100.1 10\nSosa, Edmundo 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.336 0.365 0.309 89.6 1\nStubbs, Garrett 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMerrifield, Whit 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.026 0.021 64.5 57\nStott, Bryson 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.927 1.11 0.891 97.9 12.5\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.706 1.921 1.1 103.1 23'}, {'pitcher_id': '592332', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGausman, Kevin 113 102 31 5 2 7 25 22.1 23.7 9 8 0.304 0.598 0.396 0.287 0.565 0.377 89.4 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 29 27 9 4 0 2 6 20.7 32.3 2 6.9 0.333 0.704 0.448 0.332 0.64 0.427 92.3 17\nHarper, Bryce 29 23 10 0 0 2 4 13.8 20.3 5 17.2 0.435 0.696 0.498 0.338 0.624 0.436 90.1 7\nSchwarber, Kyle 13 12 3 0 0 3 7 53.8 36.4 1 7.7 0.25 1 0.509 0.264 1.031 0.529 101.1 29.2\nBohm, Alec 11 10 2 0 0 0 1 9.1 11.8 1 9.1 0.2 0.2 0.224 0.258 0.364 0.305 91.2 21.2\nStott, Bryson 9 9 1 1 0 0 2 22.2 7.1 0 0 0.111 0.222 0.138 0.172 0.261 0.18 89 13.9\nDahl, David 5 5 1 0 1 0 2 40 25 0 0 0.2 0.6 0.312 0.146 0.462 0.245 85.3 11.3\nSosa, Edmundo 4 4 0 0 0 0 0 0 21.4 0 0 0 0 0 0.152 0.182 0.143 73.7 10\nNola, Aaron 3 3 2 0 0 0 1 33.3 0 0 0 0.667 0.667 0.58 0.328 0.391 0.312 96 -7\nRojas, Johan 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.252 0.289 0.236 66.3 -33.7\nMarsh, Brandon 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nMerrifield, Whit 2 2 2 0 0 0 0 0 0 0 0 1 1 0.877 0.827 1 0.77 95.1 14.5\nSuárez, Ranger 1 1 1 0 1 0 0 0 50 0 0 1 3 1.578 0.72 0.867 0.663 90.7 17\nWheeler, Zack 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.11 0.082 79.2 44.5'}, {'pitcher_id': '663893', 'player_data': '', 'team_data': 'Brendon Little has not faced anyone on this team.'}, {'pitcher_id': '650893', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Génesis 28 22 8 1 0 1 8 28.6 20.6 4 14.3 0.364 0.545 0.442 0.285 0.558 0.416 93.7 13.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 9 8 1 0 0 0 4 44.4 23.8 1 11.1 0.125 0.125 0.174 0.1 0.248 0.203 92.9 14.5\nStott, Bryson 2 1 0 0 0 0 1 50 25 1 50 0 0 0.344 0.344\nCastellanos, Nick 4 4 2 0 0 0 0 0 10 0 0 0.5 0.5 0.44 0.325 0.59 0.385 97.1 3\nMerrifield, Whit 4 4 2 1 0 0 1 25 25 0 0 0.5 0.75 0.529 0.375 0.601 0.422 90.2 22.3\nBohm, Alec 2 1 0 0 0 0 1 50 22.2 1 50 0 0 0.344 0.344\nHarper, Bryce 4 3 3 0 0 1 0 0 0 0 0 1 2 1.105 0.893 1.844 1.024 93.8 18.3\nNola, Aaron 2 1 0 0 0 0 1 50 0 0 0 0 0 0 0\nStubbs, Garrett 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692'}, {'pitcher_id': '647315', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPop, Zach 20 17 0 0 0 0 5 25 18.8 2 10 0 0 0.069 0.177 0.205 0.226 87.4 2.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBohm, Alec 6 5 0 0 0 0 2 33.3 40 1 16.7 0 0 0.115 0.126 0.154 0.219 77.2 12\nHarper, Bryce 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.215 0.245 0.2 91.7 -2.5\nStott, Bryson 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.436 0.525 0.371 96.2 16.3\nSchwarber, Kyle 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nStubbs, Garrett 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nMarchán, Rafael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSosa, Edmundo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.138 0.163 0.129 84.5 -14\nMarsh, Brandon 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.089 0.078 90.9 15.5\nMerrifield, Whit 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.311 0.32 0.283 63 -51\nCastellanos, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.254 0.264 0.232 103.4 -14'}, {'pitcher_id': '663878', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPearson, Nate 2 2 2 0 0 0 0 0 25 0 0 1 1 0.883 0.811 1.029 0.802 105.3 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSosa, Edmundo 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.689 0.78 0.648 103.3 7\nMarsh, Brandon\nBohm, Alec 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.934 1.278 0.956 107.3 11'}, {'pitcher_id': '684320', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657024', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Erik 21 19 4 1 0 1 6 28.6 28.3 1 4.8 0.211 0.421 0.282 0.219 0.445 0.298 88.7 20.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 6 6 0 0 0 0 2 33.3 26.7 0 0 0 0 0 0.224 0.289 0.219 89.6 27.5\nCastellanos, Nick 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.29 0.186 0.295 0.202 96.8 17.5\nMarsh, Brandon 3 2 1 1 0 0 1 33.3 33.3 1 33.3 0.5 1 0.647 0.245 0.622 0.475 99.9 23\nHarper, Bryce 1 1 0 0 0 0 1 100 20 0 0 0 0 0 0\nSosa, Edmundo 2 1 0 0 0 0 0 0 20 0 0 0 0 0 0.076 0.083 0.075 72.8 -14\nPache, Cristian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.68 0.78 0.662 72.5 29\nStott, Bryson 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.069 0.123 0.084 81.2 9.5\nSchwarber, Kyle 1 1 1 0 0 1 0 0 50 0 0 1 4 1.94 0.86 3.425 1.677 103.6 32\nDahl, David 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.009 0.029 0.015 89.8 38'}, {'pitcher_id': '669270', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKuhnel, Joel 9 8 2 1 0 0 1 11.1 21.4 1 11.1 0.25 0.375 0.315 0.355 0.422 0.378 95.8 2.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 3 3 2 1 0 0 0 0 16.7 0 0 0.667 1 0.715 0.494 0.65 0.489 93.9 16.7\nHarper, Bryce 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69\nSosa, Edmundo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSchwarber, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.255 0.275 0.237 102.1 -11\nStott, Bryson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.614 0.667 0.561 81.3 13\nBohm, Alec 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.244 0.244 0.223 102.9 -17'}, {'pitcher_id': '670950', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRichards, Trevor 45 39 8 4 0 0 15 33.3 26.3 4 8.9 0.205 0.308 0.259 0.172 0.268 0.234 86.5 23.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 14 11 2 1 0 0 7 50 46.2 3 21.4 0.182 0.273 0.297 0.061 0.157 0.218 91.1 29.8\nCastellanos, Nick 8 8 3 2 0 0 3 37.5 34.8 0 0 0.375 0.625 0.416 0.23 0.301 0.228 85.7 21.8\nSchwarber, Kyle 4 4 1 0 0 0 1 25 28.6 0 0 0.25 0.25 0.217 0.293 0.376 0.286 95.9 20.7\nDahl, David 4 4 1 1 0 0 1 25 28.6 0 0 0.25 0.5 0.304 0.159 0.204 0.143 83.2 25.7\nMerrifield, Whit 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.094 0.147 0.103 91 22.3\nMarsh, Brandon 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.307 0.938 0.51 101.2 25\nStott, Bryson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.063 0.097 0.071 78.3 47.5\nWheeler, Zack 2 1 1 0 0 0 0 0 0 0 0 1 1 0.88 0.853 0.971 0.804 53.1 22\nBohm, Alec 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nSosa, Edmundo 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nStrahm, Matt 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nNola, Aaron 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.175 0.189 0.172 92.3 -6\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.333 0.336 0.307 102.1 -6'}, {'pitcher_id': '605135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBassitt, Chris 107 100 26 6 1 1 23 21.5 21 6 5.6 0.26 0.37 0.29 0.23 0.358 0.278 86.9 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 29 28 10 4 0 1 4 13.8 10.7 1 3.4 0.357 0.607 0.415 0.283 0.51 0.351 86.4 17.3\nSchwarber, Kyle 15 13 4 1 0 0 6 40 33.3 1 6.7 0.308 0.385 0.303 0.168 0.223 0.213 89.5 37.9\nBohm, Alec 15 15 2 0 0 0 1 6.7 14.8 0 0 0.133 0.133 0.118 0.235 0.254 0.218 83.8 -4.1\nStott, Bryson 10 9 1 0 1 0 3 30 16.7 1 10 0.111 0.333 0.229 0.146 0.242 0.22 90.8 20.7\nHarper, Bryce 9 8 0 0 0 0 3 33.3 47.6 1 11.1 0 0 0.077 0.175 0.447 0.308 85.7 39\nMerrifield, Whit 9 9 4 0 0 0 2 22.2 13.3 0 0 0.444 0.444 0.389 0.271 0.456 0.307 93.2 13.9\nMarsh, Brandon 8 7 2 1 0 0 2 25 25 1 12.5 0.286 0.429 0.355 0.16 0.243 0.237 89.3 13\nStubbs, Garrett 5 5 0 0 0 0 0 0 7.1 0 0 0 0 0 0.126 0.188 0.13 82.8 23.6\nRojas, Johan 2 2 1 0 0 0 1 50 20 0 0 0.5 0.5 0.444 0.484 0.522 0.444 78.3 18\nSosa, Edmundo 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.346 0.346\nDahl, David 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.586 0.576 0.633 0.524 83.7 14.7'}, {'pitcher_id': '641835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMayza, Tim 33 30 7 2 1 1 6 18.2 20 2 6.1 0.233 0.467 0.306 0.27 0.397 0.302 90.3 10.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 8 7 1 0 0 0 2 25 33.3 1 12.5 0.143 0.143 0.197 0.176 0.185 0.227 81.5 -11.6\nSchwarber, Kyle 8 7 1 1 0 0 0 0 7.1 0 0 0.143 0.286 0.159 0.309 0.4 0.271 92.9 16.4\nMerrifield, Whit 4 3 0 0 0 0 1 25 22.2 1 25 0 0 0.172 0.044 0.077 0.206 86.4 37\nStubbs, Garrett 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.21 0.21 0.191 82.4 0\nDahl, David 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.243 0.27 0.226 102.7 2\nSosa, Edmundo 1 1 1 0 1 0 0 0 50 0 0 1 3 1.612 0.669 2.177 1.169 101.6 25\nStott, Bryson 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.444 0.423 0.501 0.393 96.1 15\nPache, Cristian 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.13 0.133 0.122 98.3 -23\nCastellanos, Nick 2 2 2 1 0 1 0 0 0 0 0 1 3 1.592 0.372 0.921 0.524 97.6 22.5\nMarchán, Rafael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.159 0.123 75.7 36\nBohm, Alec 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.934 1.098 0.884 98.8 12'}, {'pitcher_id': '670102', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrancis, Bowden 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.668 0.461 1.3 0.714 102.9 29.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 1 1 1 0 0 1 0 0 33.3 0 0 1 4 2.004 0.937 3.12 1.647 107.6 22\nStott, Bryson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.437 0.763 0.485 103 17\nCastellanos, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.016 0.01 98.1 50'}, {'pitcher_id': '643338', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreen, Chad 16 15 3 1 0 0 3 18.8 28.6 0 0 0.2 0.267 0.232 0.263 0.578 0.373 89.8 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 7 7 2 0 0 0 1 14.3 27.8 0 0 0.286 0.286 0.25 0.239 0.35 0.249 85.5 22.3\nCastellanos, Nick 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.257 0.832 0.45 85 11.5\nHarper, Bryce 3 3 1 1 0 0 1 33.3 57.1 0 0 0.333 0.667 0.414 0.397 1.124 0.621 96.9 9.5\nSchwarber, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.023 0.016 97.6 56\nDahl, David 1 0 0 0 0 0 0 0 0 0 0 0.719 0.719\nBohm, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.307 0.33 0.282 103.7 -7'}, {'pitcher_id': '502179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspino, Paolo 57 55 15 1 0 3 9 15.8 13.3 2 3.5 0.273 0.455 0.326 0.281 0.518 0.35 89 12.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBohm, Alec 13 12 3 0 0 1 2 15.4 13 1 7.7 0.25 0.5 0.348 0.217 0.3 0.257 94.4 16.6\nSchwarber, Kyle 11 10 4 0 0 2 2 18.2 9.5 1 9.1 0.4 1 0.6 0.464 1.306 0.725 98.4 13.4\nHarper, Bryce 11 11 3 0 0 0 1 9.1 15.8 0 0 0.273 0.273 0.24 0.245 0.447 0.293 83.4 -5.4\nCastellanos, Nick 9 9 1 1 0 0 1 11.1 12.5 0 0 0.111 0.222 0.14 0.144 0.226 0.16 79.3 26.6\nStott, Bryson 6 6 2 0 0 0 1 16.7 11.1 0 0 0.333 0.333 0.295 0.335 0.395 0.319 95.4 9.8\nMarsh, Brandon 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.407 0.449 0.385 86.3 25\nNola, Aaron 2 2 0 0 0 0 2 100 25 0 0 0 0 0 0\nMarchán, Rafael 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.499 0.585 0.466 78.9 12.5'}], 144: [{'pitcher_id': '666201', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManoah, Alek 26 19 5 1 0 1 6 23.1 28 5 19.2 0.263 0.474 0.39 0.254 0.453 0.381 93.7 31.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 5 3 1 0 0 1 1 20 22.2 2 40 0.333 1.333 0.691 0.384 1.264 0.688 100 25\nMurphy, Sean 6 4 1 0 0 0 1 16.7 20 0 0 0.25 0.25 0.267 0.302 0.374 0.321 93.5 28\nDuvall, Adam 5 5 0 0 0 0 2 40 38.5 0 0 0 0 0 0.061 0.076 0.058 82 61\nKelenic, Jarred 5 5 2 0 0 0 2 40 25 0 0 0.4 0.4 0.353 0.24 0.288 0.228 96.8 17.3\nOlson, Matt 3 2 1 1 0 0 0 0 28.6 1 33.3 0.5 1 0.644 0.48 0.752 0.578 101 24\nAnderson, Brian 2 0 0 0 0 0 0 0 33.3 2 100 0.696 0.696'}, {'pitcher_id': '554340', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Yimi 47 40 9 1 0 1 18 38.3 37 6 12.8 0.225 0.325 0.303 0.233 0.406 0.333 92.5 13.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 8 6 2 0 0 0 3 37.5 50 2 25 0.333 0.333 0.394 0.282 0.325 0.371 95.5 0.3\nDuvall, Adam 5 5 1 0 0 0 3 60 38.9 0 0 0.2 0.2 0.176 0.078 0.088 0.07 99.4 27\nAlbies, Ozzie 7 6 1 0 0 0 1 14.3 33.3 1 14.3 0.167 0.167 0.225 0.315 0.516 0.401 93.6 18.8\nRiley, Austin 5 4 1 0 0 0 2 40 53.8 1 20 0.25 0.25 0.315 0.269 0.312 0.339 89.7 11.5\nd'Arnaud, Travis 6 5 1 0 0 1 1 16.7 0 1 16.7 0.2 0.8 0.446 0.27 0.901 0.505 96 22.3\nKelenic, Jarred 3 3 1 0 0 0 2 66.7 37.5 0 0 0.333 0.333 0.293 0.278 0.327 0.264 93.7 12\nArcia, Orlando 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.293 0.306 0.353 0.285 96.1 2\nMurphy, Sean 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.018 0.018 0.018 61.2 -8\nOlson, Matt 2 1 0 0 0 0 1 50 100 1 50 0 0 0.348 0.348\nLaureano, Ramón 2 1 0 0 0 0 1 50 25 0 0 0 0 0.363 0.363\nShort, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.162 0.206 0.15 67.7 4\nAnderson, Brian 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.608 0.397 1.191 0.629 104.3 21"}, {'pitcher_id': '621244', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerríos, José 85 80 19 4 1 3 22 25.9 26.7 3 3.5 0.237 0.425 0.303 0.218 0.45 0.305 85.3 13.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 23 21 6 2 0 1 5 21.7 26.9 1 4.3 0.286 0.524 0.371 0.227 0.528 0.346 83.2 11.7\nLaureano, Ramón 19 17 5 0 0 1 5 26.3 28.2 1 5.3 0.294 0.471 0.368 0.237 0.477 0.347 90.1 7\nDuvall, Adam 8 8 1 1 0 0 5 62.5 43.8 0 0 0.125 0.25 0.155 0.14 0.155 0.131 70.6 18.3\nArcia, Orlando 6 6 1 1 0 0 2 33.3 21.1 0 0 0.167 0.333 0.207 0.262 0.657 0.383 81 18.8\nOzuna, Marcell 6 6 2 0 0 1 1 16.7 33.3 0 0 0.333 0.833 0.481 0.339 0.932 0.523 94.7 23.8\nMurphy, Sean 5 5 0 0 0 0 0 0 18.2 0 0 0 0 0 0.129 0.139 0.121 79.4 -8.6\nAlbies, Ozzie 7 7 3 0 1 0 0 0 12.5 0 0 0.429 0.714 0.467 0.293 0.541 0.344 92.5 24.4\nShort, Zack 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.231 0.004 0.005 0.234 72.2 63\nRiley, Austin 3 3 1 0 0 0 2 66.7 33.3 0 0 0.333 0.333 0.294 0.158 0.174 0.145 84.8 9\nAnderson, Brian 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.074 0.119 0.079 81.8 -4.5\nd'Arnaud, Travis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.274 0.321 0.268 80.6 41"}, {'pitcher_id': '579328', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKikuchi, Yusei 81 71 18 4 0 3 23 28.4 32 7 8.6 0.254 0.437 0.322 0.236 0.374 0.306 90.7 10.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 28 23 7 2 0 0 7 25 28.3 3 10.7 0.304 0.391 0.344 0.265 0.362 0.332 90.8 3.4\nOlson, Matt 26 23 5 1 0 2 5 19.2 32.6 3 11.5 0.217 0.522 0.346 0.209 0.412 0.313 91.1 17.4\nMurphy, Sean 11 10 1 0 0 0 5 45.5 33.3 0 0 0.1 0.1 0.08 0.201 0.22 0.203 94.7 -4.3\nAlbies, Ozzie 3 3 1 0 0 1 2 66.7 22.2 0 0 0.333 1.333 0.668 0.045 0.165 0.083 98.5 39\nRiley, Austin 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.588 0.582 0.667 0.556 77.5 25.5\nd'Arnaud, Travis 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.049 0.049 0.046 92.1 -24\nDuvall, Adam 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nAnderson, Brian 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.06 0.06 0.375 62.2 52\nArcia, Orlando 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.466 0.562 0.451 93 -11.5\nOzuna, Marcell 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.622 0.444 1.425 0.758 95.2 34.5"}, {'pitcher_id': '605447', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomano, Jordan 18 17 6 1 0 1 6 33.3 32.4 1 5.6 0.353 0.588 0.414 0.281 0.49 0.351 86.2 20.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 5 5 2 0 0 0 3 60 30.8 0 0 0.4 0.4 0.352 0.208 0.234 0.204 63.1 -15.5\nDuvall, Adam 3 3 1 1 0 0 0 0 28.6 0 0 0.333 0.667 0.414 0.238 0.274 0.226 82.6 55.3\nOlson, Matt 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.44 0.147 0.154 0.139 108.8 -9\nAnderson, Brian 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.413 0.462 0.38 93.2 3\nKelenic, Jarred 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLaureano, Ramón 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nAlbies, Ozzie 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMurphy, Sean 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.533 2.046 1.034 90.9 41.5\nd'Arnaud, Travis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.827 1.02 0.815 97.8 10"}, {'pitcher_id': '592332', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGausman, Kevin 60 55 15 0 0 3 16 26.7 35 3 5 0.273 0.436 0.31 0.241 0.408 0.294 89.9 1.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Brian 15 13 4 0 0 0 4 26.7 29.6 2 13.3 0.308 0.308 0.326 0.285 0.432 0.359 95.3 5.7\nOlson, Matt 10 10 3 0 0 1 3 30 29.2 0 0 0.3 0.6 0.374 0.347 0.663 0.423 92.1 6.6\nd'Arnaud, Travis 6 5 0 0 0 0 1 16.7 27.3 1 16.7 0 0 0.114 0.192 0.206 0.262 90.7 8.5\nDuvall, Adam 5 4 2 0 0 2 1 20 30.8 0 0 0.5 2 0.797 0.312 1.147 0.524 97.4 29.5\nKelenic, Jarred 5 4 1 0 0 0 2 40 60 0 0 0.25 0.25 0.177 0.192 0.202 0.145 77.5 -2.3\nLaureano, Ramón 4 4 0 0 0 0 1 25 50 0 0 0 0 0 0.12 0.13 0.114 62.9 -16.3\nAlbies, Ozzie 3 3 0 0 0 0 1 33.3 44.4 0 0 0 0 0 0.045 0.05 0.049 83.3 -30.5\nArcia, Orlando 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.583 0.36 0.36 0.324 102.7 -2\nFried, Max 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.115 0.115 0.118 67.2 -58\nShort, Zack 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.035 0.111 0.057 93.5 34\nRiley, Austin 3 3 2 0 0 0 0 0 25 0 0 0.667 0.667 0.586 0.395 0.431 0.361 101.2 -5.3\nMurphy, Sean 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.44 0.075 0.081 0.081 98.8 -17"}, {'pitcher_id': '663893', 'player_data': '', 'team_data': 'Brendon Little has not faced anyone on this team.'}, {'pitcher_id': '650893', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Génesis 15 13 2 1 0 0 2 13.3 29.2 2 13.3 0.154 0.231 0.236 0.281 0.5 0.377 97.1 17.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 2 1 0 0 0 0 1 50 50 1 50 0 0 0.344 0.344\nArcia, Orlando 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.233 0.234 0.396 0.41 97.5 18\nd'Arnaud, Travis 1 1 1 1 0 0 0 0 66.7 0 0 1 2 1.261 0.454 0.979 0.604 101.2 21\nDuvall, Adam 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.134 0.471 0.239 98.8 43\nAlbies, Ozzie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.343 0.393 0.321 99.4 36.5\nAnderson, Brian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.56 1.584 0.867 99.6 25\nOlson, Matt 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.609 0.73 0.587 105.6 6\nKelenic, Jarred 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.072 0.065 87.9 -28\nLaureano, Ramón 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.434 0.494 0.4 95 2\nShort, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.116 0.095 84.8 -15"}, {'pitcher_id': '647315', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPop, Zach 10 10 2 1 0 0 1 10 11.1 0 0 0.2 0.3 0.212 0.262 0.422 0.292 83.1 10.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 4 4 2 1 0 0 0 0 11.1 0 0 0.5 0.75 0.53 0.437 0.616 0.455 82.2 -13.5\nAlbies, Ozzie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.293 0.727 0.42 95.6 28\nKelenic, Jarred 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.07 0.059 85 24.5\nd'Arnaud, Travis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDuvall, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.157 0.16 0.141 58 47"}, {'pitcher_id': '663878', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPearson, Nate 14 12 2 0 0 1 4 28.6 19 1 7.1 0.167 0.417 0.254 0.15 0.303 0.214 92.3 32.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.294 0.303 0.42 0.319 100.1 43.5\nAnderson, Brian 3 3 1 0 0 1 1 33.3 28.6 0 0 0.333 1.333 0.66 0.17 0.636 0.325 100.4 39\nDuvall, Adam 3 2 0 0 0 0 1 33.3 0 0 0 0 0 0 0.006 0.019 0.008 96.4 47.5\nOlson, Matt 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.049 0.052 0.047 79.2 -3\nRiley, Austin 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nShort, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.168 0.212 0.156 94.6 -14\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.11 0.095 63.3 50"}, {'pitcher_id': '684320', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657024', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Erik 21 16 3 1 0 0 6 28.6 23.3 3 14.3 0.188 0.25 0.31 0.115 0.182 0.269 90.4 11.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMurphy, Sean 6 4 0 0 0 0 1 16.7 8.3 0 0 0 0 0.24 0.12 0.231 0.332 88 5.7\nOlson, Matt 2 1 0 0 0 0 1 50 25 1 50 0 0 0.345 0.345\nLaureano, Ramón 4 4 1 1 0 0 3 75 50 0 0 0.25 0.5 0.304 0\nOzuna, Marcell 2 0 0 0 0 0 0 0 0 2 100 0.692 0.692\nDuvall, Adam 3 3 1 0 0 0 1 33.3 60 0 0 0.333 0.333 0.294 0.205 0.291 0.212 101.7 22.5\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.227 0.427 0.285 88.2 23\nArcia, Orlando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.05 0.05 0.049 84 -30\nAlbies, Ozzie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.077 0.057 79.1 50\nAnderson, Brian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.307 0.385 0.279 95.4 -2"}, {'pitcher_id': '669270', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKuhnel, Joel 5 5 0 0 0 0 2 40 25 0 0 0 0 0 0.112 0.12 0.104 86.5 19', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.399 0.417 0.368 97.6 -1\nArcia, Orlando 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nOlson, Matt 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nDuvall, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.181 0.146 92.7 -12\nMartínez, J.P. 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.004 69.2 70'}, {'pitcher_id': '670950', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRichards, Trevor 39 35 5 2 0 1 10 25.6 38.5 3 7.7 0.143 0.286 0.231 0.235 0.487 0.342 90.8 26.3', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 9 9 1 1 0 0 3 33.3 38.9 0 0 0.111 0.222 0.139 0.194 0.26 0.197 85.5 20.7\nAlbies, Ozzie 8 6 1 0 0 0 0 0 20 1 12.5 0.167 0.167 0.285 0.421 0.765 0.546 86.2 28.3\nRiley, Austin 7 7 1 1 0 0 4 57.1 52.9 0 0 0.143 0.286 0.174 0.104 0.121 0.098 94.3 43\nLaureano, Ramón 3 2 0 0 0 0 0 0 40 1 33.3 0 0 0.23 0.57 2.233 0.984 103.6 35\nArcia, Orlando 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.318 0.524 0.352 90.6 25\nOlson, Matt 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.014 0.037 0.358 98.1 48\nShort, Zack 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nMurphy, Sean 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nAnderson, Brian 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nd'Arnaud, Travis 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.267 0.308 0.242 97.8 -5\nKelenic, Jarred 2 2 1 0 0 1 0 0 50 0 0 0.5 2 1.003 0.412 1.42 0.746 92.9 -4.5\nDuvall, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.035 0.026 96.9 56"}, {'pitcher_id': '605135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBassitt, Chris 78 67 15 3 0 3 21 26.9 27.1 7 9 0.224 0.403 0.319 0.218 0.346 0.299 86.9 19.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 12 9 1 0 0 1 3 25 28.6 3 25 0.111 0.444 0.345 0.164 0.401 0.352 90.7 26\nKelenic, Jarred 10 7 0 0 0 0 4 40 27.8 3 30 0 0 0.208 0.002 0.005 0.21 77.4 64.3\nd'Arnaud, Travis 8 7 3 1 0 0 1 12.5 17.6 0 0 0.429 0.571 0.469 0.351 0.376 0.373 78.4 23.2\nRiley, Austin 12 11 3 0 0 1 3 25 40 0 0 0.273 0.545 0.38 0.328 0.6 0.425 90.5 12.1\nDuvall, Adam 8 7 0 0 0 0 3 37.5 20 0 0 0 0 0 0.189 0.195 0.15 88.2 19.4\nArcia, Orlando 6 6 3 0 0 0 1 16.7 9.1 0 0 0.5 0.5 0.437 0.234 0.27 0.218 91.5 19.4\nOzuna, Marcell 6 5 0 0 0 0 2 33.3 25 1 16.7 0 0 0.115 0.023 0.028 0.133 80.3 30\nMurphy, Sean 7 7 2 1 0 0 3 42.9 38.5 0 0 0.286 0.429 0.306 0.359 0.53 0.38 96.1 10.5\nAlbies, Ozzie 6 5 1 0 0 0 1 16.7 40 0 0 0.2 0.2 0.268 0.109 0.138 0.208 81.5 -1.3\nLaureano, Ramón 3 3 2 1 0 1 0 0 0 0 0 0.667 2 1.083 0.392 0.935 0.556 87.7 -1.7"}, {'pitcher_id': '641835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMayza, Tim 12 11 3 1 0 0 2 16.7 27.8 1 8.3 0.273 0.364 0.308 0.292 0.377 0.325 89.9 16.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShort, Zack 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.035 0.056 0.042 78.1 62.5\nOlson, Matt 2 1 0 0 0 0 0 0 50 1 50 0 0 0.346 0.07 0.073 0.377 86.6 -42\nAlbies, Ozzie 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.037 0.127 0.068 97.9 42\nLaureano, Ramón 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.891 1.256 0.915 108.4 12\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.56 0.672 0.547 95.5 5\nOzuna, Marcell 1 1 1 0 0 0 0 0 50 0 0 1 1 0.879 0.74 0.831 0.681 80 22\nMurphy, Sean 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.714 0.84 0.686 111.2 6\nKelenic, Jarred 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.058 0.053 73.2 -20"}, {'pitcher_id': '670102', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrancis, Bowden 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShort, Zack 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '643338', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreen, Chad 22 21 6 0 0 2 6 27.3 26.8 1 4.5 0.286 0.571 0.371 0.222 0.35 0.261 89.2 26.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nd'Arnaud, Travis 6 5 1 0 0 1 0 0 0 1 16.7 0.2 0.8 0.438 0.219 0.521 0.363 85.5 32.2\nAlbies, Ozzie 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.293 0.298 0.342 0.283 77.1 43\nOlson, Matt 3 3 0 0 0 0 1 33.3 57.1 0 0 0 0 0 0.14 0.248 0.159 87.9 44\nOzuna, Marcell 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nMurphy, Sean 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nArcia, Orlando 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.095 0.095 0.091 86.2 -20\nRiley, Austin 2 2 2 0 0 0 0 0 0 0 0 1 1 0.879 0.635 0.7 0.577 103.2 6.5\nAnderson, Brian 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.44 0.254 0.28 0.228 95.9 25\nKelenic, Jarred 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.377 0.911 0.539 96.6 24"}, {'pitcher_id': '502179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspino, Paolo 81 70 25 4 0 8 13 16 24.1 9 11.1 0.357 0.757 0.49 0.35 0.732 0.479 90 17.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuvall, Adam 15 11 4 0 0 2 3 20 32.1 3 20 0.364 0.909 0.57 0.373 0.872 0.565 91.7 28.1\nRiley, Austin 18 18 7 2 0 2 2 11.1 32.3 0 0 0.389 0.833 0.514 0.471 1.016 0.629 91.8 17\nAlbies, Ozzie 8 4 1 0 0 0 1 12.5 20 3 37.5 0.25 0.25 0.369 0.376 0.696 0.486 87.9 21.5\nOzuna, Marcell 12 11 3 0 0 2 2 16.7 15.8 1 8.3 0.273 0.818 0.469 0.336 1.015 0.568 91.1 12\nOlson, Matt 10 9 4 0 0 1 1 10 15.8 1 10 0.444 0.778 0.541 0.33 0.467 0.379 92.8 10.1\nd'Arnaud, Travis 7 7 1 1 0 0 1 14.3 14.3 0 0 0.143 0.286 0.177 0.133 0.18 0.134 81 32.3\nArcia, Orlando 5 4 2 0 0 1 1 20 20 1 20 0.5 1.25 0.729 0.134 0.225 0.258 83.2 -3.7\nFried, Max 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.44 0.393 0.436 0.364 85.1 19\nAnderson, Brian 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.073 0.461 0.752 0.519 92.9 10\nMurphy, Sean 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.277 0.327 0.269 98.3 4"}], 145: [{'pitcher_id': '666201', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nManoah, Alek 53 48 9 3 0 1 10 18.9 19.8 3 5.7 0.188 0.313 0.259 0.189 0.309 0.259 87 15.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 11 10 1 0 0 0 1 9.1 23.8 0 0 0.1 0.1 0.146 0.281 0.352 0.315 91.4 2.1\nVaughn, Andrew 9 9 3 1 0 0 0 0 0 0 0 0.333 0.444 0.336 0.242 0.286 0.232 91.9 13.6\nBenintendi, Andrew 9 8 2 1 0 0 3 33.3 20 1 11.1 0.25 0.375 0.315 0.178 0.367 0.285 88.3 14\nRobert Jr., Luis 7 5 2 1 0 0 0 0 27.3 1 14.3 0.4 0.6 0.508 0.166 0.259 0.325 87.1 20.8\nSheets, Gavin 4 3 0 0 0 0 2 50 50 1 25 0 0 0.172 0.041 0.063 0.201 91.2 -12\nMendick, Danny 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.174 0.121 68 27\nPham, Tommy 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.17 0.127 79.4 31.7\nMaldonado, Martín 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nJulks, Corey 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.002 0.516 1.624 0.873 81.4 22.5\nDeJong, Paul 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nFedde, Erick 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLee, Korey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.038 0.027 82.7 56'}, {'pitcher_id': '554340', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarcía, Yimi 26 24 2 0 0 0 12 46.2 36.7 2 7.7 0.083 0.083 0.121 0.126 0.17 0.172 87.6 20.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 4 4 0 0 0 0 3 75 55.6 0 0 0 0 0 0.101 0.104 0.091 106.5 -3\nRobert Jr., Luis 4 4 1 0 0 0 1 25 42.9 0 0 0.25 0.25 0.22 0.144 0.149 0.132 72.5 0\nLopez, Nicky 5 5 1 0 0 0 2 40 30 0 0 0.2 0.2 0.176 0.307 0.384 0.305 94 17\nDeJong, Paul 2 0 0 0 0 0 0 0 0 2 100 0.691 0.691\nJulks, Corey 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.088 0.289 0.157 94.2 31\nMaldonado, Martín 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.075 0.045 84.6 45.5\nMendick, Danny 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.118 0.204 0.135 92.2 21\nVaughn, Andrew 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.015 0.011 88.9 57\nSosa, Lenyn 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '621244', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerríos, José 163 144 38 9 2 5 37 22.7 24.8 15 9.2 0.264 0.458 0.342 0.275 0.448 0.352 86.6 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 36 32 10 2 1 1 9 25 31.5 4 11.1 0.313 0.531 0.392 0.3 0.526 0.387 85.4 12.7\nLopez, Nicky 25 20 10 3 1 0 3 12 14.6 5 20 0.5 0.75 0.561 0.352 0.465 0.42 89.6 5.1\nSheets, Gavin 17 14 4 2 0 1 3 17.6 31.6 2 11.8 0.286 0.643 0.399 0.216 0.414 0.338 85 12.4\nMendick, Danny 14 13 1 1 0 0 4 28.6 22.6 0 0 0.077 0.154 0.14 0.149 0.19 0.191 69.9 10\nVaughn, Andrew 18 15 4 1 0 1 2 11.1 12.5 1 5.6 0.267 0.533 0.362 0.316 0.511 0.4 89.3 15.2\nRobert Jr., Luis 18 18 3 0 0 1 4 22.2 15.8 0 0 0.167 0.333 0.213 0.348 0.69 0.436 93.3 6.3\nMaldonado, Martín 14 14 2 0 0 0 4 28.6 20.8 0 0 0.143 0.143 0.124 0.158 0.177 0.148 80.6 6\nDeJong, Paul 8 7 3 0 0 1 2 25 26.7 1 12.5 0.429 0.857 0.569 0.41 0.795 0.53 94.5 14.4\nPham, Tommy 6 6 0 0 0 0 4 66.7 54.5 0 0 0 0 0 0.102 0.111 0.094 101.3 -7\nLee, Korey 3 2 1 0 0 0 1 33.3 33.3 1 33.3 0.5 0.5 0.527 0.488 0.524 0.528 73.5 20\nJulks, Corey 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.262 0.286 0.466 99.8 -10\nColás, Oscar 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.015 0.021 0.016 95.9 58'}, {'pitcher_id': '579328', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKikuchi, Yusei 87 75 18 1 0 3 21 24.1 28.5 11 12.6 0.24 0.373 0.32 0.227 0.395 0.323 89.7 10.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 15 11 3 0 0 1 4 26.7 31 3 20 0.273 0.545 0.433 0.249 0.54 0.43 95.4 8.4\nBenintendi, Andrew 14 13 4 1 0 0 3 21.4 16.7 1 7.1 0.308 0.385 0.327 0.216 0.264 0.239 85.4 13.4\nLopez, Nicky 12 11 3 0 0 0 2 16.7 28.6 1 8.3 0.273 0.273 0.277 0.272 0.317 0.293 84.9 0.9\nVaughn, Andrew 10 9 1 0 0 0 1 10 24.1 1 10 0.111 0.111 0.157 0.24 0.495 0.348 88.1 17\nPham, Tommy 10 8 3 0 0 2 3 30 26.7 2 20 0.375 1.125 0.628 0.336 0.888 0.541 102.6 31.2\nMendick, Danny 7 6 2 0 0 0 1 14.3 18.2 1 14.3 0.333 0.333 0.35 0.302 0.338 0.337 85.1 -10\nRobert Jr., Luis 6 6 1 0 0 0 4 66.7 60 0 0 0.167 0.167 0.147 0.072 0.161 0.095 96.3 32.5\nJulks, Corey 3 2 1 0 0 0 0 0 16.7 1 33.3 0.5 0.5 0.527 0.339 0.393 0.444 102.2 -7.5\nDeJong, Paul 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.231 0.231\nLee, Korey 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.057 0.047 84.4 -5.3\nSosa, Lenyn 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.564 0.328 92.4 38.5\nColás, Oscar 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.071 0.071 0.066 80.8 -3'}, {'pitcher_id': '605447', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRomano, Jordan 22 22 6 2 0 1 7 31.8 36.2 0 0 0.273 0.5 0.328 0.308 0.503 0.344 92.4 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 6 6 2 1 0 0 2 33.3 42.9 0 0 0.333 0.5 0.358 0.332 0.389 0.328 91.4 8.8\nBenintendi, Andrew 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.018 0.046 0.027 90.9 36\nSheets, Gavin 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.354 0.469 0.353 100.6 15\nSosa, Lenyn 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.001 0.001 0 65.3 70\nPham, Tommy 2 2 1 1 0 0 1 50 66.7 0 0 0.5 1 0.608 0.45 0.544 0.43 102.5 11\nColás, Oscar 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.199 0.224 0.181 72.9 33\nLee, Korey 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.791 3.139 1.596 107.3 36\nRobert Jr., Luis 2 2 2 0 0 0 0 0 25 0 0 1 1 0.881 0.413 0.472 0.371 98.8 0\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.352 0.529 0.317 94.6 0\nDeJong, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.709 1.473 0.904 107.2 19\nJulks, Corey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLopez, Nicky 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.055 0.077 0.054 82.2 -31'}, {'pitcher_id': '592332', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGausman, Kevin 98 93 29 9 1 4 24 24.5 24.9 4 4.1 0.312 0.559 0.377 0.279 0.419 0.313 91.9 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 23 22 8 1 0 1 6 26.1 18.4 1 4.3 0.364 0.545 0.401 0.302 0.393 0.319 92.5 3.9\nBenintendi, Andrew 21 19 4 1 0 0 3 14.3 28.6 1 4.8 0.211 0.263 0.218 0.241 0.358 0.269 89.8 13.7\nVaughn, Andrew 9 9 3 1 0 1 2 22.2 13.9 0 0 0.333 0.778 0.469 0.243 0.442 0.293 90.5 0.9\nMaldonado, Martín 11 11 0 0 0 0 5 45.5 47.6 0 0 0 0 0 0.148 0.167 0.138 85.9 33.3\nSheets, Gavin 5 4 1 1 0 0 0 0 18.2 1 20 0.25 0.5 0.392 0.309 0.506 0.414 97.7 16.8\nJulks, Corey 7 7 5 1 0 1 0 0 0 0 0 0.714 1.286 0.843 0.619 1.142 0.738 100.1 16.9\nDeJong, Paul 6 6 2 0 0 1 3 50 25 0 0 0.333 0.833 0.476 0.268 0.531 0.33 97.7 12.3\nMendick, Danny 4 3 2 1 0 0 0 0 16.7 1 25 0.667 1 0.709 0.378 0.451 0.443 86 21.7\nLopez, Nicky 5 5 3 2 1 0 1 20 20 0 0 0.6 1.4 0.829 0.37 0.459 0.36 93.3 1.8\nRobert Jr., Luis 3 3 1 1 0 0 1 33.3 37.5 0 0 0.333 0.667 0.42 0.22 0.245 0.21 87 -9\nLee, Korey 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.035 0.066 0.041 88 28\nFedde, Erick 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663893', 'player_data': '', 'team_data': 'Brendon Little has not faced anyone on this team.'}, {'pitcher_id': '650893', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Génesis 24 21 7 1 0 1 2 8.3 17.1 2 8.3 0.333 0.524 0.382 0.212 0.384 0.294 90.1 13.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 6 6 0 0 0 0 1 16.7 28.6 0 0 0 0 0 0.091 0.102 0.084 89 7.8\nJulks, Corey 2 1 1 0 0 0 0 0 11.1 1 50 1 1 0.787 0.346 0.37 0.502 96.5 -2\nPham, Tommy 4 3 2 0 0 1 1 25 25 1 25 0.667 1.667 0.914 0.36 1.258 0.666 92.6 11.5\nLopez, Nicky 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.086 0.109 0.087 85.5 -11.7\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.2 0.285 0.21 91.1 9\nMendick, Danny 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.311 0.522 0.339 83.3 34.5\nDeJong, Paul 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.011 0.017 0.208 93.6 35\nVaughn, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.29 0.34 0.28 90.7 40.5\nLee, Korey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.603 0.665 0.554 101.9 6'}, {'pitcher_id': '647315', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPop, Zach 17 16 4 1 0 0 3 17.6 17.6 1 5.9 0.25 0.313 0.272 0.2 0.346 0.261 83.4 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 3 2 1 1 0 0 0 0 8.3 1 33.3 0.5 1 0.654 0.311 1.06 0.601 81.9 6.5\nPham, Tommy 4 4 2 0 0 0 1 25 0 0 0 0.5 0.5 0.443 0.213 0.223 0.199 66.1 -26.3\nLee, Korey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.144 0.184 0.144 91.6 -12\nJulks, Corey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.851 1.121 0.846 100.2 14\nSheets, Gavin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVaughn, Andrew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.208 0.416 0.27 96.7 0\nRobert Jr., Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFedde, Erick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.117 0.1 83.3 -9\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.034 0.023 90.5 42\nBenintendi, Andrew 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.003 0.003 0.003 88.1 56\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.2 0.234 0.181 74.9 33'}, {'pitcher_id': '663878', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPearson, Nate 13 13 2 0 0 0 3 23.1 22.2 0 0 0.154 0.154 0.136 0.152 0.189 0.148 83.8 33.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.127 0.142 0.115 85.8 32.5\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.102 0.058 85.7 30\nDeJong, Paul 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBenintendi, Andrew 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.444 0.23 0.313 0.23 94.6 18\nRobert Jr., Luis 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.441 0.466 0.517 0.436 72.2 49\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.017 0.014 87.8 45\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.028 0.021 90.4 54\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.103 0.083 72.6 -6\nVaughn, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.04 0.12 0.067 91 35'}, {'pitcher_id': '684320', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657024', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSwanson, Erik 31 29 6 0 0 2 10 32.3 33.3 2 6.5 0.207 0.414 0.29 0.267 0.509 0.35 90.8 26.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 9 9 2 0 0 0 4 44.4 33.3 0 0 0.222 0.222 0.195 0.19 0.29 0.203 84.5 29.4\nMaldonado, Martín 6 5 1 0 0 1 1 16.7 26.7 1 16.7 0.2 0.8 0.449 0.353 0.57 0.447 89.1 20.3\nLopez, Nicky 4 4 2 0 0 0 1 25 40 0 0 0.5 0.5 0.442 0.397 0.458 0.376 88.7 0\nPham, Tommy 4 3 0 0 0 0 1 25 0 1 25 0 0 0.173 0.472 1.224 0.686 103.6 24\nVaughn, Andrew 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.295 1.046 0.544 95.9 48.5\nSheets, Gavin 2 2 1 0 0 1 1 50 50 0 0 0.5 2 1.043 0.086 0.299 0.161 94.2 32\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.5 1.112 0.685 104 19\nJulks, Corey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRobert Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.004 83.4 72\nLee, Korey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '669270', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKuhnel, Joel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.259 0.298 0.245 84.2 23.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.259 0.298 0.245 84.2 23.5'}, {'pitcher_id': '670950', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRichards, Trevor 47 44 8 1 0 0 15 31.9 35.2 3 6.4 0.182 0.205 0.202 0.2 0.268 0.233 87.9 14.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 9 7 1 0 0 0 4 44.4 38.9 2 22.2 0.143 0.143 0.251 0.051 0.051 0.191 61.6 -1.7\nDeJong, Paul 7 7 1 0 0 0 1 14.3 38.5 0 0 0.143 0.143 0.127 0.188 0.307 0.206 89.7 24.2\nPham, Tommy 6 6 3 1 0 0 1 16.7 25 0 0 0.5 0.667 0.5 0.293 0.477 0.327 94.8 12.8\nVaughn, Andrew 4 3 0 0 0 0 2 50 42.9 1 25 0 0 0.173 0.011 0.018 0.182 80.5 46\nLopez, Nicky 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.116 0.181 0.118 89.7 44.5\nLee, Korey 2 2 0 0 0 0 1 50 62.5 0 0 0 0 0 0.259 0.284 0.238 80.5 12\nSheets, Gavin 3 3 2 0 0 0 0 0 33.3 0 0 0.667 0.667 0.589 0.337 0.394 0.329 86 27.7\nMaldonado, Martín 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.195 0.214 0.175 98.1 -4\nRobert Jr., Luis 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nJulks, Corey 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.313 0.349 0.29 96.6 6\nSoroka, Michael 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.416 0.507 0.397 92.7 15\nColás, Oscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.454 0.485 0.409 100.7 1\nFedde, Erick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.387 0.517 0.41 84.2 -6.5'}, {'pitcher_id': '605135', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBassitt, Chris 105 96 20 3 0 2 27 25.7 25 8 7.6 0.208 0.302 0.26 0.214 0.322 0.269 82.4 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 21 18 1 0 0 0 7 33.3 32.1 3 14.3 0.056 0.056 0.141 0.115 0.219 0.219 84.2 4.4\nBenintendi, Andrew 14 13 5 0 0 1 3 21.4 20.8 1 7.1 0.385 0.615 0.446 0.349 0.546 0.403 87.8 17.8\nDeJong, Paul 9 9 1 1 0 0 3 33.3 35 0 0 0.111 0.222 0.135 0.142 0.188 0.139 79 21.2\nSheets, Gavin 6 5 2 0 0 0 0 0 6.7 1 16.7 0.4 0.4 0.412 0.181 0.235 0.266 88.9 10.2\nRobert Jr., Luis 9 8 2 0 0 1 2 22.2 25 1 11.1 0.25 0.625 0.398 0.297 0.687 0.441 88.9 12.7\nLopez, Nicky 11 9 4 0 0 0 1 9.1 15.4 1 9.1 0.444 0.444 0.451 0.304 0.343 0.36 82.3 3.6\nPham, Tommy 8 7 1 0 0 0 3 37.5 33.3 1 12.5 0.143 0.143 0.197 0.224 0.329 0.292 91.1 26.3\nVaughn, Andrew 9 9 2 1 0 0 4 44.4 31.3 0 0 0.222 0.333 0.237 0.211 0.299 0.219 88 2\nJulks, Corey 7 7 1 1 0 0 2 28.6 23.1 0 0 0.143 0.286 0.181 0.141 0.159 0.132 76.2 27\nColás, Oscar 6 6 1 0 0 0 0 0 12.5 0 0 0.167 0.167 0.147 0.259 0.27 0.235 69.4 -24\nSosa, Lenyn 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.216 0.218 0.198 56.6 -47.7\nMendick, Danny 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0'}, {'pitcher_id': '641835', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMayza, Tim 32 29 4 1 0 0 5 15.6 23.6 3 9.4 0.138 0.172 0.187 0.331 0.515 0.395 88.7 4.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 11 9 1 0 0 0 1 9.1 21.7 2 18.2 0.111 0.111 0.205 0.369 0.446 0.42 92.1 6\nLopez, Nicky 8 8 0 0 0 0 2 25 30.8 0 0 0 0 0 0.115 0.125 0.106 80.8 -18\nSheets, Gavin 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nJulks, Corey 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.269 0.408 0.294 84.3 9.7\nVaughn, Andrew 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.613 0.996 0.69 95.2 16.3\nColás, Oscar 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMendick, Danny 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.077 0.546 1.04 0.665 90.3 20\nDeJong, Paul 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.834 0.973 0.784 87.1 18\nLee, Korey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.789 2.671 1.405 102.2 27'}, {'pitcher_id': '670102', 'player_data': '', 'team_data': 'Bowden Francis has not faced anyone on this team.'}, {'pitcher_id': '643338', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGreen, Chad 37 34 10 4 0 0 9 24.3 25.9 3 8.1 0.294 0.412 0.331 0.281 0.494 0.352 89.1 14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 17 15 4 2 0 0 4 23.5 21.9 2 11.8 0.267 0.4 0.328 0.326 0.625 0.429 94 1.9\nMaldonado, Martín 6 6 2 0 0 0 2 33.3 27.8 0 0 0.333 0.333 0.292 0.084 0.109 0.084 75.3 41.3\nPham, Tommy 4 3 1 1 0 0 1 25 25 1 25 0.333 0.667 0.477 0.562 1.279 0.729 102.3 19.5\nLopez, Nicky 4 4 0 0 0 0 1 25 18.2 0 0 0 0 0 0.202 0.22 0.181 78.8 14.3\nVaughn, Andrew 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.879 0.726 0.803 0.669 95 10\nRobert Jr., Luis 1 1 0 0 0 0 1 100 75 0 0 0 0 0 0\nDeJong, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.052 0.037 89 59\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.289 0.327 0.269 75 10\nLee, Korey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.206 0.219 0.185 92.2 -10\nSheets, Gavin 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.423 0.644 0.432 100.9 17'}, {'pitcher_id': '502179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nEspino, Paolo 6 6 2 1 0 1 1 16.7 18.2 0 0 0.333 1 0.552 0.274 0.738 0.419 87.4 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.691 0.39 1.175 0.651 87.7 28.3\nPham, Tommy 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.621 0.225 0.437 0.265 102.1 19\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.028 0.028 0.029 71.9 -24'}]}, 142: {141: [{'pitcher_id': '595345', 'player_data': '', 'team_data': ''}, {'pitcher_id': '660896', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlcala, Jorge 6 6 0 0 0 0 1 16.7 22.2 0 0 0 0 0 0.242 0.381 0.267 79.6 -9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.205 0.224 0.184 74 7\nBichette, Bo 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSpringer, George 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.06 0.067 0.069 65.9 -16\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.78 1.317 0.88 113.3 14\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.26 0.534 0.33 95.9 22\nJansen, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.145 0.145 0.14 49.1 -73'}, {'pitcher_id': '622251', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStaumont, Josh 19 14 3 0 0 1 2 10.5 31.8 5 26.3 0.214 0.429 0.376 0.368 0.656 0.497 91.6 14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJansen, Danny 3 2 2 0 0 0 0 0 33.3 1 33.3 1 1 0.816 0.905 1.095 0.808 93 13\nKiner-Falefa, Isiah 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0.073 0.085 0.277 75.3 0\nGuerrero Jr., Vladimir 4 4 1 0 0 1 0 0 0 0 0 0.25 1 0.485 0.618 1.474 0.853 106.4 22.3\nBichette, Bo 3 3 0 0 0 0 0 0 40 0 0 0 0 0 0.196 0.235 0.188 79.9 -9\nClement, Ernie 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.1 0.177 0.4 88.1 26\nSpringer, George 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nTurner, Justin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.047 0.034 85 57\nVarsho, Daulton 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nKiermaier, Kevin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '641154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Pablo 60 54 14 2 2 2 10 16.7 22.1 4 6.7 0.259 0.481 0.35 0.288 0.425 0.347 85.8 12.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarsho, Daulton 9 8 1 1 0 0 1 11.1 18.2 1 11.1 0.125 0.25 0.217 0.258 0.308 0.303 79.4 4.7\nBichette, Bo 8 8 4 0 1 1 3 37.5 33.3 0 0 0.5 1.125 0.667 0.323 0.684 0.424 89.7 31.8\nKiermaier, Kevin 12 10 5 0 0 1 0 0 6.7 2 16.7 0.5 0.8 0.583 0.382 0.44 0.419 87.1 -13.5\nGuerrero Jr., Vladimir 8 8 1 0 1 0 2 25 33.3 0 0 0.125 0.375 0.196 0.221 0.462 0.286 93.7 24.5\nTurner, Justin 8 7 1 1 0 0 1 12.5 23.5 0 0 0.143 0.286 0.248 0.226 0.349 0.303 87.2 37.2\nGausman, Kevin 6 6 1 0 0 0 2 33.3 18.2 0 0 0.167 0.167 0.145 0.222 0.241 0.208 76.2 -8.7\nSpringer, George 6 6 1 0 0 0 1 16.7 28.6 0 0 0.167 0.167 0.147 0.304 0.407 0.302 87.5 22.6\nKirk, Alejandro 3 1 0 0 0 0 0 0 0 1 33.3 0 0 0.474 0.551 0.608 0.645 74.1 29'}, {'pitcher_id': '592779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStewart, Brock 15 15 3 1 0 0 4 26.7 34.3 0 0 0.2 0.267 0.2 0.208 0.276 0.206 89 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBichette, Bo 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.348 0.351 0.314 67.2 20\nKiermaier, Kevin 3 3 1 0 0 0 0 0 33.3 0 0 0.333 0.333 0.29 0.433 0.49 0.402 85.1 30\nSpringer, George 4 4 1 1 0 0 0 0 14.3 0 0 0.25 0.5 0.311 0.198 0.402 0.241 95.2 20\nKirk, Alejandro 1 1 0 0 0 0 1 100 60 0 0 0 0 0 0\nKiner-Falefa, Isiah 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.122 0.133 0.111 97.5 -12\nVarsho, Daulton 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nGuerrero Jr., Vladimir 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.042 0.045 0.04 89 0'}, {'pitcher_id': '681892', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFunderburk, Kody 5 4 0 0 0 0 1 20 25 0 0 0 0 0 0.071 0.092 0.1 90.2 0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClement, Ernie 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.072 0.075 0.066 85.4 -40\nJansen, Danny 1 0 0 0 0 0 0 0 0 0 0 0 0.222 94.5 25\nKiner-Falefa, Isiah 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nKirk, Alejandro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.063 0.131 0.076 89.8 31\nSpringer, George 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.15 0.163 0.135 91 -13'}, {'pitcher_id': '676961', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '686973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Louie 35 33 11 2 1 2 5 14.3 17 2 5.7 0.333 0.636 0.421 0.311 0.551 0.382 88.9 5.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBichette, Bo 7 7 2 0 0 1 2 28.6 7.7 0 0 0.286 0.714 0.412 0.329 0.851 0.488 92 12.8\nSpringer, George 6 4 3 2 0 0 0 0 23.5 2 33.3 0.75 1.25 0.794 0.371 0.516 0.481 87.9 -5.2\nKirk, Alejandro 5 5 2 0 0 0 0 0 8.3 0 0 0.4 0.4 0.353 0.366 0.432 0.345 90.4 6.2\nVarsho, Daulton 5 5 0 0 0 0 2 40 36.4 0 0 0 0 0 0.044 0.046 0.04 78 23.3\nGuerrero Jr., Vladimir 6 6 2 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.429 0.498 0.404 92.4 -0.5\nKiner-Falefa, Isiah 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.061 0.082 0.063 81.7 -11.7\nKiermaier, Kevin 2 2 2 0 1 1 0 0 40 0 0 1 3.5 1.786 0.81 2.226 1.24 96.7 22.5'}, {'pitcher_id': '663978', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaddack, Chris 24 23 6 1 0 0 5 20.8 24.2 0 0 0.261 0.304 0.265 0.306 0.362 0.308 89.9 10.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 15 14 3 0 0 0 3 20 20.5 0 0 0.214 0.214 0.224 0.259 0.32 0.28 86.2 23.7\nVarsho, Daulton 4 4 2 0 0 0 1 25 35.7 0 0 0.5 0.5 0.44 0.405 0.459 0.379 102.6 -1\nKiner-Falefa, Isiah 4 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.31 0.445 0.507 0.419 90.3 -17.5\nRichards, Trevor 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '650960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuarte, Daniel 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.023 0.035 0.024 80.6 44', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.023 0.035 0.024 80.6 44'}, {'pitcher_id': '661395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuran, Jhoan 26 24 5 1 0 0 8 30.8 43.2 1 3.8 0.208 0.25 0.238 0.214 0.332 0.273 94.7 -5.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 5 5 2 0 0 0 0 0 25 0 0 0.4 0.4 0.353 0.22 0.345 0.25 93.4 -12\nKirk, Alejandro 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.132 0.149 0.122 86.2 5\nBichette, Bo 5 4 1 0 0 0 2 40 50 0 0 0.25 0.25 0.322 0.078 0.084 0.204 105.5 -19.5\nVarsho, Daulton 4 4 0 0 0 0 2 50 77.8 0 0 0 0 0 0.083 0.095 0.076 76.1 -5.5\nSchneider, Davis 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.766 0.945 0.731 99.2 10\nTurner, Justin 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSpringer, George 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.415 0.577 1.188 0.741 100 9\nKiner-Falefa, Isiah 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.271 0.338 0.247 100.7 -10\nKiermaier, Kevin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.118 0.119 0.107 98.3 -14'}, {'pitcher_id': '663485', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSands, Cole 19 17 5 0 0 1 3 15.8 32.4 1 5.3 0.294 0.471 0.371 0.249 0.486 0.354 88.7 26.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBichette, Bo 4 4 1 0 0 0 2 50 66.7 0 0 0.25 0.25 0.222 0.05 0.058 0.046 98.2 8\nKiermaier, Kevin 3 3 1 0 0 0 1 33.3 66.7 0 0 0.333 0.333 0.295 0.382 0.439 0.367 85.1 11.5\nGuerrero Jr., Vladimir 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.011 0.015 0.352 84.6 77\nSchneider, Davis 2 1 1 0 0 0 0 0 0 0 0 1 1 0.806 0.297 0.346 0.499 72.7 39\nKiner-Falefa, Isiah 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.095 0.258 0.148 89.7 18\nKirk, Alejandro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.94 1.077 0.902 82 17\nTurner, Justin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.334 1.055 0.567 97.4 29\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.137 0.392 0.217 93.3 31\nJansen, Danny 1 1 1 0 0 1 0 0 50 0 0 1 4 2.085 0.88 3.059 1.609 105.8 23\nVarsho, Daulton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 70.9 67'}, {'pitcher_id': '641927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOber, Bailey 49 47 9 1 0 0 18 36.7 29.3 1 2 0.191 0.213 0.198 0.204 0.334 0.248 87.7 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 11 11 2 1 0 0 4 36.4 29 0 0 0.182 0.273 0.193 0.273 0.635 0.377 91 29.1\nGuerrero Jr., Vladimir 8 7 1 0 0 0 4 50 37.5 1 12.5 0.143 0.143 0.198 0.202 0.255 0.26 88.6 14.3\nBichette, Bo 8 8 2 0 0 0 3 37.5 13.6 0 0 0.25 0.25 0.22 0.205 0.267 0.201 95 7.6\nKirk, Alejandro 6 6 1 0 0 0 0 0 17.6 0 0 0.167 0.167 0.147 0.319 0.452 0.329 84.7 6.2\nVarsho, Daulton 5 5 1 0 0 0 3 60 31.3 0 0 0.2 0.2 0.177 0.062 0.114 0.073 92.9 4\nClement, Ernie 4 4 1 0 0 0 0 0 11.1 0 0 0.25 0.25 0.22 0.11 0.137 0.11 80 17.8\nSchneider, Davis 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nKiner-Falefa, Isiah 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.007 0.007 0.006 82.6 52\nTurner, Justin 3 2 1 0 0 0 1 33.3 60 0 0 0.5 0.5 0.536 0.414 0.471 0.511 67.8 33'}, {'pitcher_id': '678692', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657746', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Joe 50 50 14 2 0 2 9 18 15.8 0 0 0.28 0.44 0.31 0.337 0.568 0.385 93.8 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 13 13 3 1 0 0 1 7.7 10.3 0 0 0.231 0.308 0.233 0.38 0.648 0.436 87.9 26.3\nGuerrero Jr., Vladimir 6 6 3 0 0 0 2 33.3 30.8 0 0 0.5 0.5 0.443 0.486 0.665 0.494 107.3 11.8\nVarsho, Daulton 5 5 1 0 0 0 1 20 9.1 0 0 0.2 0.2 0.177 0.323 0.382 0.307 92.6 22.3\nClement, Ernie 3 3 1 0 0 0 0 0 23.1 0 0 0.333 0.333 0.296 0.264 0.323 0.256 87.9 15.7\nBichette, Bo 6 6 2 0 0 0 2 33.3 14.3 0 0 0.333 0.333 0.295 0.327 0.38 0.308 100.7 3.3\nKiner-Falefa, Isiah 5 5 3 0 0 2 1 20 14.3 0 0 0.6 1.8 1.009 0.383 0.786 0.494 92.5 14.8\nJansen, Danny 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.094 0.106 0.087 84 8.3\nSchneider, Davis 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.172 0.662 0.339 102.7 35\nSpringer, George 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.369 0.724 0.458 102.4 5.3\nKirk, Alejandro 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.272 0.809 0.445 98 17.7'}, {'pitcher_id': '573204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThielbar, Caleb 21 17 3 0 0 0 1 4.8 9.3 3 14.3 0.176 0.176 0.225 0.273 0.369 0.374 88.1 0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 5 3 1 0 0 0 1 20 16.7 2 40 0.333 0.333 0.453 0.351 0.57 0.512 106.5 0\nClement, Ernie 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.222 0.329 0.373 0.315 77.3 -4.5\nJansen, Danny 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.221 0.386 0.262 83.2 -2.5\nSchneider, Davis 1 0 0 0 0 0 0 0 0 0 0 0 1.009 107.6 18\nVarsho, Daulton 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nKiermaier, Kevin 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.317 0.321 0.291 55.9 -57\nTurner, Justin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.291 0.31 0.262 97.3 -3\nGuerrero Jr., Vladimir 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.297 0.341 0.278 94.6 27\nBichette, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.191 0.206 0.173 101.2 -17\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.007 0.003 92.1 46'}, {'pitcher_id': '643377', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJax, Griffin 39 38 10 1 0 2 6 15.4 20 1 2.6 0.263 0.447 0.311 0.281 0.592 0.373 95.4 6.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 9 9 2 1 0 0 1 11.1 13.3 0 0 0.222 0.333 0.236 0.36 0.76 0.465 99.4 24.1\nBichette, Bo 9 9 2 0 0 0 2 22.2 25 0 0 0.222 0.222 0.196 0.169 0.181 0.155 96.6 -15\nJansen, Danny 5 5 1 0 0 1 2 40 27.3 0 0 0.2 0.8 0.401 0.209 0.617 0.338 98.1 28.3\nKiermaier, Kevin 2 2 1 0 0 0 0 0 28.6 0 0 0.5 0.5 0.44 0.636 1.939 1.046 91.1 18.5\nTurner, Justin 4 4 0 0 0 0 0 0 16.7 0 0 0 0 0 0.118 0.197 0.133 92 6\nVarsho, Daulton 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.497 0.716 0.522 96.5 20\nSpringer, George 3 3 2 0 0 1 1 33.3 40 0 0 0.667 1.667 0.962 0.386 1.249 0.671 88.7 -12\nKirk, Alejandro 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.284 0.297 0.473 100.4 -3\nKiner-Falefa, Isiah 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.123 0.139 0.115 90.8 -15.5\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.447 0.504 0.406 83.6 5'}, {'pitcher_id': '680573', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWoods Richardson, Simeon 19 19 8 2 0 2 1 5.3 17.4 0 0 0.421 0.842 0.54 0.307 0.584 0.375 87 21.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarsho, Daulton 3 3 1 1 0 0 0 0 10 0 0 0.333 0.667 0.423 0.314 0.555 0.366 84.6 38\nGuerrero Jr., Vladimir 3 3 3 0 0 0 0 0 12.5 0 0 1 1 0.889 0.437 0.537 0.42 99.4 6\nSchneider, Davis 3 3 2 1 0 1 1 33.3 50 0 0 0.667 2 1.118 0.23 0.676 0.379 97.2 28.5\nJansen, Danny 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.035 0.118 0.06 86.3 59\nTurner, Justin 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.107 0.137 0.102 85.4 18.5\nKiner-Falefa, Isiah 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.549 0.583 0.503 61.3 -28\nBichette, Bo 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.719 2.018 1.125 101.6 25\nClement, Ernie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.032 0.037 0.03 75.5 22'}], 143: [{'pitcher_id': '595345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOkert, Steven 45 41 14 5 0 0 16 35.6 40 3 6.7 0.341 0.463 0.378 0.252 0.304 0.284 87.2 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 8 8 2 0 0 0 4 50 33.3 0 0 0.25 0.25 0.221 0.195 0.216 0.181 82.7 45\nHarper, Bryce 9 7 3 1 0 0 1 11.1 40 2 22.2 0.429 0.571 0.49 0.44 0.485 0.472 87 18.5\nCastellanos, Nick 6 6 3 1 0 0 3 50 40 0 0 0.5 0.667 0.502 0.245 0.331 0.249 99.3 6.3\nMarsh, Brandon 4 4 2 1 0 0 1 25 25 0 0 0.5 0.75 0.536 0.249 0.397 0.28 99.3 21.3\nStott, Bryson 5 5 1 0 0 0 3 60 62.5 0 0 0.2 0.2 0.177 0.193 0.229 0.187 85 23.5\nBohm, Alec 4 4 1 1 0 0 1 25 33.3 0 0 0.25 0.5 0.311 0.356 0.406 0.333 69.5 14.3\nRojas, Johan 3 2 1 1 0 0 1 33.3 50 0 0 0.5 1 0.657 0.316 0.376 0.442 100 6\nDahl, David 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.44 0.082 0.1 0.082 83.6 23\nSosa, Edmundo 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nPache, Cristian 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nHoffman, Jeff 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nStubbs, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.03 0.022 85.1 58'}, {'pitcher_id': '660896', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlcala, Jorge 13 12 2 1 0 0 3 23.1 28.6 1 7.7 0.167 0.25 0.217 0.258 0.369 0.303 86.6 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 9 8 0 0 0 0 3 33.3 29.4 1 11.1 0 0 0.078 0.023 0.027 0.097 79.5 34.6\nMarsh, Brandon 2 2 1 1 0 0 0 0 28.6 0 0 0.5 1 0.621 0.827 1.37 0.935 101.2 16\nCastellanos, Nick 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.891 1.059 0.856 88 13\nSosa, Edmundo 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.369 0.416 0.332 91.2 -2'}, {'pitcher_id': '622251', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStaumont, Josh 4 3 0 0 0 0 3 75 54.5 1 25 0 0 0.175 0.175', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 2 1 0 0 0 0 1 50 42.9 1 50 0 0 0.349 0.349\nSchwarber, Kyle 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMarsh, Brandon 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '641154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Pablo 87 80 20 5 2 2 16 18.4 33.7 4 4.6 0.25 0.438 0.305 0.295 0.485 0.346 85.8 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 23 19 6 4 1 0 4 17.4 32.6 2 8.7 0.316 0.632 0.411 0.329 0.66 0.431 87.6 15.3\nSchwarber, Kyle 15 15 4 0 0 2 1 6.7 25.9 0 0 0.267 0.667 0.389 0.25 0.536 0.329 87.8 22.6\nBohm, Alec 13 13 2 0 0 0 2 15.4 38.1 0 0 0.154 0.154 0.135 0.306 0.423 0.318 84.8 12.7\nStott, Bryson 8 8 1 1 0 0 1 12.5 28.6 0 0 0.125 0.25 0.158 0.168 0.342 0.216 87.4 21.3\nMerrifield, Whit 6 5 2 0 0 0 1 16.7 25 1 16.7 0.4 0.4 0.406 0.298 0.34 0.345 75.8 8\nCastellanos, Nick 5 4 1 0 0 0 0 0 36.4 1 20 0.25 0.25 0.314 0.431 0.633 0.499 92.8 14\nDahl, David 3 2 0 0 0 0 2 66.7 77.8 0 0 0 0 0 0 0 0 86.2 44\nMarchán, Rafael 4 4 2 0 0 0 0 0 28.6 0 0 0.5 0.5 0.44 0.539 0.618 0.52 81.2 7.8\nStubbs, Garrett 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.502 0.566 0.481 71.3 -11.5\nMarsh, Brandon 2 2 2 0 1 0 0 0 0 0 0 1 2 1.243 0.881 1.038 0.851 88.8 15\nSosa, Edmundo 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.057 0.059 0.053 82 -41\nWheeler, Zack 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nPache, Cristian 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0'}, {'pitcher_id': '592779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStewart, Brock 5 5 2 0 0 0 1 20 37.5 0 0 0.4 0.4 0.351 0.192 0.224 0.184 84.7 38.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDahl, David 4 4 2 0 0 0 1 25 42.9 0 0 0.5 0.5 0.439 0.238 0.278 0.228 81.9 36.3\nMerrifield, Whit 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.007 0.008 0.007 93 45'}, {'pitcher_id': '681892', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '676961', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '686973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Louie 5 5 1 0 0 0 2 40 12.5 0 0 0.2 0.2 0.177 0.265 0.448 0.303 97 23.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 5 5 1 0 0 0 2 40 12.5 0 0 0.2 0.2 0.177 0.265 0.448 0.303 97 23.7'}, {'pitcher_id': '663978', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaddack, Chris 31 30 6 1 0 0 8 25.8 35.6 0 0 0.2 0.233 0.187 0.292 0.507 0.335 87.5 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 9 9 3 0 0 0 2 22.2 31.8 0 0 0.333 0.333 0.29 0.318 0.381 0.301 88 16\nCastellanos, Nick 6 6 2 0 0 0 3 50 70 0 0 0.333 0.333 0.291 0.423 0.775 0.501 90.3 21\nMerrifield, Whit 3 3 1 1 0 0 0 0 16.7 0 0 0.333 0.667 0.42 0.081 0.247 0.137 92.1 42.7\nSchwarber, Kyle 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.169 0.626 0.31 97.3 34.5\nBohm, Alec 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.168 0.174 0.152 85.5 29.5\nWheeler, Zack 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.2 0.221 0.182 86.8 5\nStubbs, Garrett 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.554 0.989 0.649 97.7 20.5\nPache, Cristian 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.242 0.265 0.222 76.8 3\nDahl, David 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.363 1.293 0.655 62.3 -18'}, {'pitcher_id': '650960', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuran, Jhoan 10 6 1 0 0 0 3 30 31.3 3 30 0.167 0.167 0.296 0.312 0.426 0.447 94.3 16.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 7 3 1 0 0 0 1 14.3 12.5 3 42.9 0.333 0.333 0.423 0.429 0.634 0.562 91.9 20.3\nPache, Cristian 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nStubbs, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.583 0.652 0.536 101.4 5'}, {'pitcher_id': '663485', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSands, Cole 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.061 0.087 0.058 59.9 -24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.061 0.087 0.058 59.9 -24'}, {'pitcher_id': '641927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOber, Bailey 17 16 5 1 1 0 4 23.5 14.3 1 5.9 0.313 0.5 0.362 0.252 0.369 0.294 84.8 17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 10 10 4 1 1 0 2 20 5.3 0 0 0.4 0.7 0.458 0.321 0.493 0.351 87.4 7.1\nMarsh, Brandon 2 1 0 0 0 0 1 50 40 1 50 0 0 0.346 0.346\nSchwarber, Kyle 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.44 0.307 0.369 0.291 87.3 19\nCastellanos, Nick 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.07 0.079 0.069 76.8 44'}, {'pitcher_id': '678692', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657746', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Joe 8 7 2 1 0 0 3 37.5 46.7 0 0 0.286 0.429 0.359 0.258 0.358 0.324 89.8 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 8 7 2 1 0 0 3 37.5 46.7 0 0 0.286 0.429 0.359 0.258 0.358 0.324 89.8 12.3'}, {'pitcher_id': '573204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThielbar, Caleb 19 19 1 0 0 0 4 21.1 17.6 0 0 0.053 0.053 0.046 0.142 0.217 0.153 84.4 30.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 7 7 0 0 0 0 1 14.3 16.7 0 0 0 0 0 0.1 0.19 0.124 85 28.2\nBohm, Alec 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.001 0.001 0 69.7 71\nHarper, Bryce 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nSchwarber, Kyle 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.231 0.128 83.2 32\nCastellanos, Nick 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.406 0.538 0.407 97.9 17.5\nStott, Bryson 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.014 0.026 0.017 87.6 59\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.311 0.324 0.28 67.2 12\nDahl, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.603 0.669 0.553 85.7 11'}, {'pitcher_id': '643377', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJax, Griffin 17 14 1 0 0 0 1 5.9 21.9 2 11.8 0.071 0.071 0.176 0.175 0.217 0.265 86.2 34.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 10 9 0 0 0 0 1 10 20.8 1 10 0 0 0.069 0.065 0.098 0.133 85.7 32.8\nSchwarber, Kyle 4 3 0 0 0 0 0 0 20 1 25 0 0 0.173 0.146 0.162 0.273 90.6 52.3\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.78 0.984 0.776 91.3 17\nStubbs, Garrett 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.651 0.685 0.586 72.1 16\nRojas, Johan 1 0 0 0 0 0 0 0 100 0 0 0.726 0.726'}, {'pitcher_id': '680573', 'player_data': '', 'team_data': 'Simeon Woods Richardson has not faced anyone on this team.'}], 144: [{'pitcher_id': '595345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOkert, Steven 51 44 11 5 0 1 16 31.4 25.5 4 7.8 0.25 0.432 0.33 0.176 0.316 0.261 88.5 22.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 9 8 2 1 0 0 4 44.4 21.1 1 11.1 0.25 0.375 0.312 0.047 0.089 0.127 87.8 31.5\nOlson, Matt 10 9 4 1 0 1 1 10 12.5 1 10 0.444 0.889 0.57 0.287 0.673 0.428 91.8 9.8\nAlbies, Ozzie 8 6 0 0 0 0 1 12.5 17.6 0 0 0 0 0.09 0.177 0.327 0.246 85.3 32\nDuvall, Adam 4 3 1 1 0 0 2 50 18.2 1 25 0.333 0.667 0.481 0.056 0.103 0.224 87.2 23\nOzuna, Marcell 5 5 0 0 0 0 2 40 30 0 0 0 0 0 0.156 0.226 0.163 86 36\nd'Arnaud, Travis 3 2 0 0 0 0 0 0 50 1 33.3 0 0 0.231 0.149 0.158 0.32 74.2 26.5\nArcia, Orlando 4 4 2 1 0 0 1 25 22.2 0 0 0.5 0.75 0.53 0.349 0.456 0.347 93.6 11.7\nMurphy, Sean 3 2 0 0 0 0 2 66.7 57.1 0 0 0 0 0.242 0.242\nKelenic, Jarred 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.304 0.353 0.288 94.3 8\nMorton, Charlie 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nLaureano, Ramón 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nShort, Zack 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.466 0.88 0.568 101.4 18"}, {'pitcher_id': '660896', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlcala, Jorge 5 5 1 0 0 1 1 20 20 0 0 0.2 0.8 0.401 0.433 1.065 0.621 98.8 24.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArcia, Orlando 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.587 0.697 0.563 98.5 2.5\nOlson, Matt 1 1 1 0 0 1 0 0 33.3 0 0 1 4 2.007 0.986 3.925 1.972 110.3 27\nLaureano, Ramón 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nShort, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 88 65'}, {'pitcher_id': '622251', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStaumont, Josh 13 13 2 0 0 0 3 23.1 19.2 0 0 0.154 0.154 0.135 0.353 0.581 0.393 89.7 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelenic, Jarred 5 5 0 0 0 0 2 40 44.4 0 0 0 0 0 0.195 0.335 0.222 87 30\nMurphy, Sean 2 2 0 0 0 0 1 50 10 0 0 0 0 0 0.049 0.057 0.043 63 0\nAlbies, Ozzie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.469 1.768 0.91 99.8 32\nOlson, Matt 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.907 1.307 0.942 109.1 10\nOzuna, Marcell 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.473 0.768 0.492 103.2 17\nDuvall, Adam 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.72 0.85 0.677 102 9\nRiley, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.098 0.072 81.2 29\nAnderson, Brian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.863 0.978 0.825 77.2 23'}, {'pitcher_id': '641154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Pablo 110 105 26 5 0 8 31 28.2 30.7 5 4.5 0.248 0.524 0.339 0.24 0.5 0.328 87.1 13.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 19 18 6 0 0 2 4 21.1 23.3 1 5.3 0.333 0.667 0.437 0.312 0.627 0.415 83.5 -4.8\nAlbies, Ozzie 22 22 3 0 0 1 2 9.1 15.7 0 0 0.136 0.273 0.171 0.205 0.319 0.225 82.6 15.3\nRiley, Austin 18 17 3 1 0 1 7 38.9 41.3 1 5.6 0.176 0.412 0.269 0.251 0.532 0.348 89.9 24.5\nd'Arnaud, Travis 14 14 5 0 0 3 7 50 39.3 0 0 0.357 1 0.55 0.256 0.771 0.417 98.2 18\nDuvall, Adam 11 8 3 1 0 0 2 18.2 33.3 3 27.3 0.375 0.5 0.463 0.321 0.766 0.521 84.3 10.3\nLaureano, Ramón 8 8 1 0 0 0 3 37.5 25 0 0 0.125 0.125 0.11 0.025 0.03 0.024 91.2 31.2\nArcia, Orlando 5 5 1 0 0 0 1 20 30 0 0 0.2 0.2 0.176 0.316 0.437 0.316 92.1 15.5\nFried, Max 4 4 2 2 0 0 2 50 44.4 0 0 0.5 1 0.615 0.238 0.484 0.302 101.2 20\nOlson, Matt 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.691 0.33 0.946 0.538 89.3 8\nAnderson, Brian 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.012 0.012 0.012 64.3 -16\nMurphy, Sean 3 3 1 1 0 0 1 33.3 57.1 0 0 0.333 0.667 0.42 0.284 0.325 0.27 88.9 34"}, {'pitcher_id': '592779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStewart, Brock 6 5 1 0 0 0 3 50 40 1 16.7 0.2 0.2 0.262 0.204 0.242 0.276 75.8 1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Brian 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nShort, Zack 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nAlbies, Ozzie 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.161 0.163 0.419 48.3 -8\nDuvall, Adam 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nOzuna, Marcell 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.877 0.86 1.046 0.816 103.3 10'}, {'pitcher_id': '681892', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFunderburk, Kody 5 4 0 0 0 0 2 40 30 0 0 0 0 0.145 0.059 0.07 0.189 98.8 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 4 3 0 0 0 0 1 25 16.7 0 0 0 0 0.181 0.079 0.093 0.237 98.8 15\nShort, Zack 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '676961', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '686973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Louie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.319 0.321 0.29 102.1 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.319 0.321 0.29 102.1 -9'}, {'pitcher_id': '663978', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaddack, Chris 35 34 9 3 1 0 5 14.3 16.5 0 0 0.265 0.412 0.277 0.281 0.381 0.313 86.3 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 8 8 3 1 0 0 3 37.5 19 0 0 0.375 0.5 0.375 0.198 0.258 0.199 85.6 -1.4\nOlson, Matt 5 4 2 0 1 0 0 0 18.2 0 0 0.5 1 0.488 0.601 0.807 0.716 101.3 9.6\nMurphy, Sean 5 5 3 2 0 0 0 0 8.3 0 0 0.6 1 0.677 0.461 0.531 0.448 68.3 5\nAnderson, Brian 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.109 0.205 0.137 81.1 -8\nAlbies, Ozzie 5 5 0 0 0 0 0 0 20 0 0 0 0 0 0.099 0.107 0.092 76.7 44.2\nArcia, Orlando 2 2 0 0 0 0 0 0 12.5 0 0 0 0 0 0.15 0.389 0.225 95 1.5\nKelenic, Jarred 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.147 0.173 0.137 88.6 8\nRiley, Austin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.73 0.851 0.688 103.7 8\nOzuna, Marcell 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.11 0.428 0.215 105 45'}, {'pitcher_id': '650960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuarte, Daniel 13 13 4 1 0 1 4 30.8 26.3 0 0 0.308 0.615 0.387 0.129 0.299 0.178 91.4 14.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlbies, Ozzie 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.004 0.008 0.006 87.1 43\nOzuna, Marcell 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.029 0.029 0.028 61.4 -35\nOlson, Matt 2 2 2 1 0 1 0 0 33.3 0 0 1 3 1.633 0.35 0.893 0.507 99.7 18\nRiley, Austin 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.113 0.334 0.183 93.7 32\nLaureano, Ramón 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nAnderson, Brian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.344 0.344 0.31 101.6 -7\nArcia, Orlando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.087 0.079 82.7 -11\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.249 0.918 0.468 103.1 40"}, {'pitcher_id': '661395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuran, Jhoan 7 6 1 0 0 0 4 57.1 42.1 0 0 0.167 0.167 0.229 0.084 0.085 0.17 61.9 -25', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelenic, Jarred 2 2 0 0 0 0 2 100 44.4 0 0 0 0 0 0\nShort, Zack 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nLaureano, Ramón 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.253 0.254 0.234 61.9 -25\nMurphy, Sean 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72'}, {'pitcher_id': '663485', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSands, Cole 4 4 0 0 0 0 2 50 42.9 0 0 0 0 0 0.076 0.12 0.084 81.2 39', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.03 0.084 0.048 91.2 33\nMartínez, J.P. 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nAnderson, Brian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.243 0.311 0.241 71.2 45'}, {'pitcher_id': '641927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOber, Bailey 17 17 6 0 0 4 6 35.3 39.5 0 0 0.353 1.059 0.575 0.287 0.793 0.445 86.2 22.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKelenic, Jarred 5 5 1 0 0 0 2 40 46.2 0 0 0.2 0.2 0.177 0.224 0.271 0.22 76.1 -15\nAnderson, Brian 4 4 1 0 0 1 2 50 54.5 0 0 0.25 1 0.501 0.265 0.813 0.439 91.2 15.5\nShort, Zack 3 3 2 0 0 1 0 0 40 0 0 0.667 1.667 0.962 0.151 0.356 0.209 84.4 56\nMartínez, J.P. 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.002 0.439 1.708 0.866 104.4 29\nDuvall, Adam 3 3 1 0 0 1 1 33.3 16.7 0 0 0.333 1.333 0.668 0.457 1.461 0.782 90 32'}, {'pitcher_id': '678692', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657746', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Joe 21 21 5 0 1 2 7 33.3 17.3 0 0 0.238 0.619 0.35 0.215 0.508 0.3 85.4 20.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nDuvall, Adam 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.01 0.014 0.009 82.4 54\nLaureano, Ramón 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.29 0.323 0.27 72 31\nAlbies, Ozzie 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.784 0.385 0.621 0.42 95.9 8.5\nArcia, Orlando 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.019 0.019 0.018 70.3 18.5\nOzuna, Marcell 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.441 0.197 0.226 0.179 101.1 -2\nMurphy, Sean 2 2 1 0 0 1 1 50 16.7 0 0 0.5 2 1.002 0.5 2 1 108.2 32\nKelenic, Jarred 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.18 0.117 83 43.5\nRiley, Austin 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.443 0.602 1.758 0.973 81.8 -24\nShort, Zack 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.164 0.188 0.155 85.3 40.5'}, {'pitcher_id': '573204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThielbar, Caleb 11 9 1 0 0 0 4 36.4 28.6 2 18.2 0.111 0.111 0.205 0.135 0.159 0.235 74.2 36.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShort, Zack 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.07 0.075 0.063 58.4 45\nLaureano, Ramón 3 2 0 0 0 0 1 33.3 25 1 33.3 0 0 0.231 0.003 0.004 0.233 92.1 48\nArcia, Orlando 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.098 0.142 0.113 75 29\nMurphy, Sean 1 1 0 0 0 0 1 100 16.7 0 0 0 0 0 0\nOlson, Matt 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.879 0.803 0.916 0.78 70.5 31\nKelenic, Jarred 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689'}, {'pitcher_id': '643377', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJax, Griffin 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.111 0.119 0.101 93.2 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMartínez, J.P. 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nShort, Zack 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.333 0.357 0.302 93.2 -1'}, {'pitcher_id': '680573', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWoods Richardson, Simeon 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.316 0.233 91.8 29', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLaureano, Ramón 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.233 0.316 0.233 91.8 29'}], 145: [{'pitcher_id': '595345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOkert, Steven 13 10 2 1 0 0 2 15.4 21.4 2 15.4 0.2 0.3 0.328 0.306 0.363 0.385 87.2 11.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMendick, Danny 3 2 1 1 0 0 1 33.3 33.3 1 33.3 0.5 1 0.654 0.399 0.434 0.471 78.7 15\nBenintendi, Andrew 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.516 0.639 0.498 90.1 10\nLopez, Nicky 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.199 0.226 0.184 76 18\nVaughn, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.094 0.199 0.119 91.2 26\nDeJong, Paul 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nMaldonado, Martín 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nSheets, Gavin 1 0 0 0 0 0 0 0 0 0 0.724 0.724\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.253 0.293 0.243 96.4 -7\nLee, Korey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.489 0.536 0.453 99.1 3'}, {'pitcher_id': '660896', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlcala, Jorge 19 18 3 0 0 0 4 21.1 33.3 1 5.3 0.167 0.167 0.175 0.206 0.252 0.226 85.4 22.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 5 5 1 0 0 0 2 40 33.3 0 0 0.2 0.2 0.177 0.235 0.271 0.222 87.1 2.7\nMendick, Danny 4 3 0 0 0 0 0 0 10 1 25 0 0 0.173 0.146 0.169 0.278 71.9 -5\nVaughn, Andrew 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.44 0.47 0.573 0.454 91.9 26\nDeJong, Paul 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.06 0.131 0.079 90.1 27\nBenintendi, Andrew 3 3 1 0 0 0 1 33.3 42.9 0 0 0.333 0.333 0.293 0.342 0.418 0.332 83.2 29\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.019 0.013 95.5 52\nRobert Jr., Luis 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0 0 0 89.7 78\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.006 0.004 93.2 50'}, {'pitcher_id': '622251', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStaumont, Josh 21 19 4 0 0 2 7 33.3 40.4 2 9.5 0.211 0.526 0.341 0.18 0.503 0.32 87.1 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRobert Jr., Luis 7 5 0 0 0 0 4 57.1 52.4 2 28.6 0 0 0.199 0.029 0.105 0.238 97.5 38\nVaughn, Andrew 6 6 0 0 0 0 1 16.7 25 0 0 0 0 0 0.258 0.472 0.304 85.4 3.2\nSheets, Gavin 3 3 2 0 0 1 0 0 14.3 0 0 0.667 1.667 0.985 0.186 0.625 0.34 83.1 17.7\nMendick, Danny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSosa, Lenyn 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.071 0.199 0.114 91.3 35\nDeJong, Paul 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.411 0.55 2.066 1.051 90.2 33.5'}, {'pitcher_id': '641154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Pablo 70 66 12 4 0 2 21 30 29.6 4 5.7 0.182 0.333 0.245 0.19 0.286 0.231 83.4 11.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 12 12 2 0 0 0 4 33.3 21.4 0 0 0.167 0.167 0.147 0.266 0.342 0.258 89.2 21.3\nVaughn, Andrew 10 9 3 1 0 0 2 20 22.7 1 10 0.333 0.444 0.371 0.208 0.265 0.251 86 12.7\nSheets, Gavin 10 8 2 1 0 1 2 20 20 2 20 0.25 0.75 0.466 0.215 0.43 0.353 91.5 17.2\nDeJong, Paul 11 10 2 1 0 1 3 27.3 16.7 1 9.1 0.2 0.6 0.361 0.188 0.408 0.29 87.9 18.1\nRobert Jr., Luis 9 9 2 1 0 0 4 44.4 55.6 0 0 0.222 0.333 0.236 0.155 0.191 0.151 73.3 -17.6\nMaldonado, Martín 5 5 0 0 0 0 1 20 41.7 0 0 0 0 0 0.234 0.263 0.222 66.1 6.5\nPham, Tommy 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.084 0.085 0.078 61 -67\nColás, Oscar 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.013 0.013 0.013 75.4 -25\nLopez, Nicky 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.202 0.369 0.232 77.5 17\nSosa, Lenyn 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.059 0.084 0.06 83.7 41.5\nMendick, Danny 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.237 0.271 0.223 92.2 41.5'}, {'pitcher_id': '592779', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStewart, Brock 20 19 2 0 0 1 9 45 39 1 5 0.105 0.263 0.183 0.201 0.363 0.261 81.4 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.022 0.024 0.02 82.9 -21\nPham, Tommy 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.07 0.074 0.075 87.5 -11\nMendick, Danny 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.216 0.225 0.195 62.5 25\nVaughn, Andrew 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nBenintendi, Andrew 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.317 0.818 0.457 100.8 22\nDeJong, Paul 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.048 0.056 0.044 66.9 41\nLopez, Nicky 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.971 1.039 0.89 75.3 19\nKuhl, Chad 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.087 0.099 0.078 75.6 -6\nJulks, Corey 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nLee, Korey 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.626 2.374 1.208 101.6 32\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.503 0.679 0.505 94.4 18'}, {'pitcher_id': '681892', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFunderburk, Kody 12 11 2 0 0 1 2 16.7 15.8 0 0 0.182 0.455 0.308 0.281 0.472 0.358 84 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSheets, Gavin 2 2 0 0 0 0 2 100 0 0 0 0 0 0 0\nMendick, Danny 2 2 1 0 0 1 0 0 40 0 0 0.5 2 1.043 0.59 1.443 0.849 93.4 20.5\nVaughn, Andrew 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.529 0.62 0.511 89.7 8\nMaldonado, Martín 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.044 0.033 71.2 33\nLopez, Nicky 2 1 0 0 0 0 0 0 50 0 0 0 0 0.362 0.057 0.07 0.389 67.9 -25\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.603 0.716 0.586 97.5 7\nDeJong, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.08 0.126 0.093 77.4 49\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.571 0.691 0.565 94.7 7'}, {'pitcher_id': '676961', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '686973', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarland, Louie 24 23 4 0 0 1 9 37.5 38.1 1 4.2 0.174 0.304 0.223 0.213 0.354 0.259 95.2 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 8 7 2 0 0 0 2 25 42.9 1 12.5 0.286 0.286 0.307 0.182 0.202 0.235 92.5 3.8\nSheets, Gavin 7 7 1 0 0 0 2 28.6 10 0 0 0.143 0.143 0.126 0.271 0.316 0.254 98 20.8\nBenintendi, Andrew 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.064 0.105 0.068 86.9 26.5\nRobert Jr., Luis 3 3 1 0 0 1 1 33.3 16.7 0 0 0.333 1.333 0.668 0.514 1.399 0.784 103.3 19.5\nMaldonado, Martín 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nLee, Korey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663978', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPaddack, Chris 33 33 9 1 0 0 10 30.3 30.2 0 0 0.273 0.303 0.252 0.219 0.273 0.213 84.1 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 9 9 2 1 0 0 3 33.3 37.5 0 0 0.222 0.333 0.236 0.279 0.416 0.296 84.3 20\nBenintendi, Andrew 4 4 2 0 0 0 0 0 16.7 0 0 0.5 0.5 0.438 0.3 0.32 0.273 79.1 33.8\nLopez, Nicky 5 5 2 0 0 0 1 20 22.2 0 0 0.4 0.4 0.355 0.174 0.192 0.161 81.9 -2.5\nLee, Korey 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.074 0.075 0.066 68.3 -47\nSheets, Gavin 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.296 0.131 0.156 0.12 88.7 15.5\nMendick, Danny 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.593 0.334 0.388 0.313 88.8 13.3\nMaldonado, Martín 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nVaughn, Andrew 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.237 0.266 0.221 99 -4\nFlexen, Chris 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.323 0.375 0.3 74.3 11'}, {'pitcher_id': '650960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuarte, Daniel 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJulks, Corey 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDeJong, Paul 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '661395', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuran, Jhoan 30 27 5 0 0 2 13 43.3 34.3 2 6.7 0.185 0.407 0.278 0.142 0.262 0.206 87 -4.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 8 7 0 0 0 0 6 75 46.7 1 12.5 0 0 0.087 0.016 0.016 0.1 61 -3\nBenintendi, Andrew 6 6 0 0 0 0 3 50 47.4 0 0 0 0 0 0.114 0.136 0.104 78.7 11.7\nSheets, Gavin 5 5 0 0 0 0 2 40 18.8 0 0 0 0 0 0.169 0.18 0.155 106.3 -13\nLopez, Nicky 3 1 1 0 0 0 0 0 16.7 1 33.3 1 1 0.79 0.231 0.274 0.453 64 -9\nRobert Jr., Luis 3 3 1 0 0 1 2 66.7 33.3 0 0 0.333 1.333 0.668 0.217 0.666 0.356 102.3 24\nMaldonado, Martín 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.027 0.027 0.025 76.8 -26\nSosa, Lenyn 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.569 2.197 1.109 103.2 34\nPham, Tommy 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.348 0.385 0.325 98.8 -3\nColás, Oscar 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.228 0.228 0.205 86.5 -57\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.132 0.132 0.126 93.1 -13'}, {'pitcher_id': '663485', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSands, Cole 14 12 3 1 0 1 2 14.3 17.4 2 14.3 0.25 0.583 0.401 0.266 0.511 0.385 89.8 13.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 4 4 0 0 0 0 0 0 33.3 0 0 0 0 0 0.289 0.52 0.346 93.3 4.3\nDeJong, Paul 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.346 0.346\nBenintendi, Andrew 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.216 0.753 0.396 94.2 32\nSheets, Gavin 2 1 1 0 0 0 0 0 0 1 50 1 1 0.786 0.163 0.163 0.419 95.4 -14\nMaldonado, Martín 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLee, Korey 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.786 0.835 0.71 64.3 31\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.184 0.186 0.17 81.6 -1\nMendick, Danny 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.491 1.372 0.759 98.5 26'}, {'pitcher_id': '641927', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOber, Bailey 88 86 22 2 0 3 25 28.4 34.1 2 2.3 0.256 0.384 0.285 0.242 0.38 0.276 86.8 22.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSheets, Gavin 19 19 5 1 0 0 3 15.8 31.7 0 0 0.263 0.316 0.251 0.282 0.384 0.289 86.5 16.8\nVaughn, Andrew 17 17 4 0 0 2 8 47.1 41.9 0 0 0.235 0.588 0.344 0.246 0.476 0.305 88 13.8\nBenintendi, Andrew 17 16 3 0 0 0 3 17.6 20.7 1 5.9 0.188 0.188 0.196 0.212 0.293 0.246 88.5 29.2\nLopez, Nicky 8 8 4 0 0 0 1 12.5 38.5 0 0 0.5 0.5 0.441 0.43 0.504 0.415 74.1 25\nRobert Jr., Luis 5 5 0 0 0 0 3 60 33.3 0 0 0 0 0 0.007 0.012 0.008 89.7 49\nPham, Tommy 5 5 2 1 0 1 1 20 20 0 0 0.4 1.2 0.671 0.203 0.534 0.303 91 26\nMaldonado, Martín 6 5 2 0 0 0 1 16.7 36.4 1 16.7 0.4 0.4 0.409 0.232 0.648 0.415 88.1 39\nDeJong, Paul 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.296 0.22 0.247 0.204 100.6 8\nLee, Korey 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.262 0.299 0.244 97.1 -6.7\nColás, Oscar 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.173 0.19 0.155 76.1 29\nMendick, Danny 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.08 0.132 0.09 86.1 25'}, {'pitcher_id': '678692', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHenriquez, Ronny 6 6 2 0 0 0 2 33.3 31.3 0 0 0.333 0.333 0.296 0.262 0.381 0.279 97.9 32', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMendick, Danny 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.166 0.651 0.331 100.2 39\nVaughn, Andrew 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.469 0.535 0.436 105.9 2\nLopez, Nicky 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.099 0.905 93.1 13\nSheets, Gavin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLee, Korey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 92.3 74\nDeJong, Paul 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '657746', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRyan, Joe 83 80 19 3 1 2 18 21.7 24.8 2 2.4 0.237 0.375 0.278 0.256 0.431 0.306 91 22.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 10 10 2 1 0 0 4 40 29.2 0 0 0.2 0.3 0.216 0.225 0.329 0.239 91.3 36\nLopez, Nicky 14 14 3 0 0 0 2 14.3 20 0 0 0.214 0.214 0.19 0.239 0.358 0.256 85.6 27.6\nBenintendi, Andrew 17 17 7 2 1 1 0 0 13.3 0 0 0.412 0.824 0.517 0.376 0.716 0.462 95.7 14\nSheets, Gavin 13 13 3 0 0 0 1 7.7 5 0 0 0.231 0.231 0.204 0.271 0.368 0.277 87.8 16.3\nRobert Jr., Luis 7 6 0 0 0 0 4 57.1 39.1 1 14.3 0 0 0.098 0.148 0.167 0.219 87.3 34.5\nMendick, Danny 5 5 1 0 0 0 2 40 27.3 0 0 0.2 0.2 0.178 0.117 0.156 0.115 88.5 25\nLee, Korey 4 3 1 0 0 1 0 0 28.6 1 25 0.333 1.333 0.694 0.312 1.2 0.633 93.8 38\nPham, Tommy 3 2 1 0 0 0 1 33.3 25 0 0 0.5 0.5 0.535 0.313 0.442 0.454 101.5 15\nDeJong, Paul 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.208 0.373 0.238 99.3 21\nMaldonado, Martín 4 4 1 0 0 0 2 50 42.9 0 0 0.25 0.25 0.221 0.193 0.281 0.201 95.2 24\nJulks, Corey 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.136 0.158 0.122 87.2 36'}, {'pitcher_id': '573204', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nThielbar, Caleb 39 34 5 2 0 0 10 25.6 29.8 3 7.7 0.147 0.206 0.203 0.208 0.293 0.265 84.4 27', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 10 7 3 1 0 0 3 30 44.8 1 10 0.429 0.571 0.442 0.399 0.533 0.448 93 18.4\nSheets, Gavin 6 6 0 0 0 0 2 33.3 30.8 0 0 0 0 0 0.109 0.169 0.116 83.7 38.3\nLopez, Nicky 8 8 1 0 0 0 0 0 0 0 0 0.125 0.125 0.11 0.271 0.341 0.267 75.7 13.1\nVaughn, Andrew 4 3 1 1 0 0 2 50 20 1 25 0.333 0.667 0.483 0.109 0.363 0.315 96.8 31\nMendick, Danny 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.009 0.03 0.355 93.9 43\nPham, Tommy 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.212 0.249 0.193 89.1 32.5\nRobert Jr., Luis 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.235 0.306 0.23 82 28.5\nDeJong, Paul 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLee, Korey 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.002 0.002 0.002 79 77\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.01 0.008 92.5 53'}, {'pitcher_id': '643377', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJax, Griffin 68 57 9 2 0 1 9 13.2 24.4 8 11.8 0.158 0.246 0.24 0.174 0.266 0.259 83.2 9.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 17 13 1 0 0 0 2 11.8 33.3 3 17.6 0.077 0.077 0.174 0.178 0.281 0.292 90.5 9.4\nBenintendi, Andrew 11 10 4 0 0 1 1 9.1 25.9 1 9.1 0.4 0.7 0.486 0.197 0.424 0.297 80.7 16.8\nLopez, Nicky 10 7 1 1 0 0 0 0 0 2 20 0.143 0.286 0.292 0.144 0.274 0.294 82.2 3.4\nSheets, Gavin 9 7 0 0 0 0 2 22.2 21.4 1 11.1 0 0 0.157 0.075 0.095 0.214 81.6 6.8\nRobert Jr., Luis 8 8 2 1 0 0 1 12.5 22.2 0 0 0.25 0.375 0.266 0.225 0.249 0.207 80 0\nMaldonado, Martín 5 4 0 0 0 0 2 40 23.1 1 20 0 0 0.138 0.08 0.096 0.2 82.6 -11.5\nDeJong, Paul 3 3 0 0 0 0 1 33.3 22.2 0 0 0 0 0 0.025 0.03 0.027 80.2 21\nLee, Korey 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.444 0.609 0.695 0.565 84.3 10.5\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.031 0.086 0.05 91.6 38\nMendick, Danny 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.546 0.55 0.491 60.7 34\nColás, Oscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.134 0.176 0.139 76.2 44'}, {'pitcher_id': '680573', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWoods Richardson, Simeon 32 30 12 3 0 0 7 21.9 16.7 2 6.3 0.4 0.5 0.412 0.299 0.476 0.354 87.2 14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 4 4 2 1 0 0 1 25 23.1 0 0 0.5 0.75 0.54 0.39 1.054 0.595 94.7 10\nMendick, Danny 5 5 0 0 0 0 4 80 44.4 0 0 0 0 0 0.039 0.039 0.035 53.7 -65\nSheets, Gavin 5 5 3 0 0 0 1 20 10 0 0 0.6 0.6 0.533 0.311 0.349 0.289 96.9 17.5\nLopez, Nicky 6 5 1 0 0 0 0 0 14.3 1 16.7 0.2 0.2 0.264 0.164 0.203 0.246 82 25.6\nVaughn, Andrew 4 4 3 1 0 0 0 0 0 0 0 0.75 1 0.762 0.517 0.615 0.492 87.6 21.5\nMaldonado, Martín 2 1 1 0 0 0 0 0 50 1 50 1 1 0.791 0.934 1.119 0.796 95.8 13\nBenintendi, Andrew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.265 0.477 0.319 85.4 30\nLee, Korey 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.354 0.4 0.34 89.9 11\nPham, Tommy 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.296 0.891 0.488 81.7 -2.5'}]}, 143: {141: [{'pitcher_id': '554430', 'player_data': '', 'team_data': ''}, {'pitcher_id': '656793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Nick 14 12 1 0 0 0 4 28.6 33.3 2 14.3 0.083 0.083 0.162 0.243 0.409 0.337 93.4 15.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKiermaier, Kevin 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.288 0.358 0.283 92.9 5\nGuerrero Jr., Vladimir 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.349 0.427 0.513 0.558 109.3 -3\nVarsho, Daulton 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.134 0.146 0.122 88.9 17.5\nKirk, Alejandro 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.467 1.266 0.707 90.5 37.5\nBichette, Bo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTurner, Justin 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nJansen, Danny 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '621237', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarado, José 24 17 6 1 0 0 4 16.7 18.4 7 29.2 0.353 0.412 0.439 0.372 0.438 0.454 93.6 2.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJansen, Danny 6 3 0 0 0 0 1 16.7 12.5 3 50 0 0 0.348 0.408 0.533 0.557 84.4 25\nGuerrero Jr., Vladimir 5 3 1 0 0 0 0 0 0 2 40 0.333 0.333 0.455 0.452 0.56 0.536 104.3 -6\nSpringer, George 4 3 1 0 0 0 1 25 33.3 1 25 0.333 0.333 0.395 0.368 0.388 0.432 80.3 5.5\nBichette, Bo 4 3 1 0 0 0 2 50 33.3 1 25 0.333 0.333 0.394 0.311 0.343 0.389 78.5 20\nTurner, Justin 3 3 2 1 0 0 0 0 20 0 0 0.667 1 0.715 0.419 0.494 0.393 100.6 -0.3\nKirk, Alejandro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.33 0.33 0.299 109.2 -8\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.123 0.153 0.111 84.9 -16'}, {'pitcher_id': '650911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Cristopher 27 25 6 2 0 0 5 18.5 23.1 2 7.4 0.24 0.32 0.277 0.274 0.354 0.302 87.9 -5.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchneider, Davis 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.061 0.076 0.059 81.6 -15\nJansen, Danny 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.719 0.546 0.811 0.581 100.3 10\nGuerrero Jr., Vladimir 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.171 0.171 0.333 75.7 -56\nBichette, Bo 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.112 0.122 0.102 106.2 -7\nKiermaier, Kevin 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.14 0.152 0.318 74 -17.5\nTurner, Justin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.352 0.462 0.352 84.9 8.7\nKirk, Alejandro 3 3 1 1 0 0 1 33.3 33.3 0 0 0.333 0.667 0.423 0.292 0.415 0.297 97.6 7\nSpringer, George 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.393 0.483 0.375 92.8 -4.7\nKiner-Falefa, Isiah 3 3 1 0 0 0 1 33.3 33.3 0 0 0.333 0.333 0.296 0.319 0.368 0.3 80.2 13'}, {'pitcher_id': '621381', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrahm, Matt 35 30 9 0 0 1 8 22.9 20.5 4 11.4 0.3 0.4 0.359 0.24 0.379 0.327 88.5 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 10 8 4 0 0 0 2 20 27.8 1 10 0.5 0.5 0.49 0.279 0.355 0.358 88.3 19.5\nSpringer, George 5 3 1 0 0 0 0 0 13.3 2 40 0.333 0.333 0.451 0.165 0.185 0.369 69 15.3\nKiner-Falefa, Isiah 4 3 0 0 0 0 0 0 9.1 1 25 0 0 0.172 0.21 0.234 0.319 87.4 12.3\nKiermaier, Kevin 5 5 1 0 0 0 3 60 37.5 0 0 0.2 0.2 0.178 0.154 0.164 0.141 103.2 -6\nRichards, Trevor 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.203 0.209 0.183 83.7 8\nKirk, Alejandro 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.37 0.885 0.53 94.5 31.3\nVarsho, Daulton 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.227 0.248 0.206 88.7 10\nJansen, Danny 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.871 2.651 1.484 108.3 21\nGuerrero Jr., Vladimir 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBichette, Bo 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0'}, {'pitcher_id': '592836', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Taijuan 58 53 19 3 0 3 11 19 22.9 4 6.9 0.358 0.585 0.415 0.32 0.604 0.406 90 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 25 23 8 0 0 2 7 28 33.3 1 4 0.348 0.609 0.404 0.317 0.592 0.387 87.1 22.4\nTurner, Justin 18 17 5 2 0 1 3 16.7 12.5 1 5.6 0.294 0.588 0.385 0.283 0.708 0.419 91.7 26.1\nKiner-Falefa, Isiah 8 7 3 0 0 0 0 0 10 1 12.5 0.429 0.429 0.419 0.391 0.447 0.413 88.1 5.7\nBichette, Bo 2 1 1 0 0 0 0 0 20 1 50 1 1 0.785 0.103 0.11 0.4 87.2 -18\nGuerrero Jr., Vladimir 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.707 0.529 0.894 0.603 105 3.3\nKiermaier, Kevin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.203 0.204 0.183 82.3 7'}, {'pitcher_id': '624133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Ranger 14 13 2 0 0 0 3 21.4 18.5 0 0 0.154 0.154 0.177 0.196 0.217 0.223 74.9 -10.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 7 7 2 0 0 0 2 28.6 28.6 0 0 0.286 0.286 0.25 0.265 0.297 0.244 76.7 -0.2\nVarsho, Daulton 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.167 0.168 0.157 60.3 -30\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 50 0 0 0 0 0\nJansen, Danny 1 0 0 0 0 0 0 0 0 0 0 0.722 0.722\nGausman, Kevin 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBichette, Bo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.096 0.079 84.9 -18'}, {'pitcher_id': '614179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, José 16 14 1 0 0 0 2 12.5 15.4 2 12.5 0.071 0.071 0.141 0.18 0.208 0.233 80.7 15.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 3 2 0 0 0 0 0 0 27.3 1 33.3 0 0 0.23 0.142 0.173 0.319 77.5 33\nJansen, Danny 2 2 0 0 0 0 0 0 12.5 0 0 0 0 0 0.069 0.079 0.063 68 17\nKirk, Alejandro 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.034 0.037 0.36 69.6 59\nKiermaier, Kevin 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nKiner-Falefa, Isiah 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.243 0.275 0.225 91.7 -8.5\nClement, Ernie 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.447 0.517 0.417 89.8 14.7\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.098 0.1 0.088 84.6 -46\nBichette, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.17 0.133 70.1 50'}, {'pitcher_id': '628708', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Yunior 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.11 0.29 0.171 92.8 27.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarsho, Daulton 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.11 0.29 0.171 92.8 27.5'}, {'pitcher_id': '605513', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurnbull, Spencer 29 25 5 0 0 1 6 20.7 17.2 3 10.3 0.2 0.32 0.285 0.216 0.3 0.286 87.1 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 6 6 2 0 0 1 3 50 14.3 0 0 0.333 0.833 0.47 0.19 0.381 0.234 89.8 5.7\nGuerrero Jr., Vladimir 4 3 1 0 0 0 0 0 9.1 1 25 0.333 0.333 0.395 0.24 0.287 0.342 89.9 45.3\nBichette, Bo 4 4 2 0 0 0 0 0 11.1 0 0 0.5 0.5 0.441 0.51 0.632 0.495 100.1 11.5\nTurner, Justin 4 3 0 0 0 0 0 0 0 0 0 0 0 0.182 0.296 0.38 0.399 83.6 3\nJansen, Danny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.154 0.163 0.141 76.3 -15.5\nKirk, Alejandro 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.08 0.097 0.072 94.4 17\nKiermaier, Kevin 2 1 0 0 0 0 1 50 40 1 50 0 0 0.345 0.345\nVarsho, Daulton 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.023 0.023 0.022 61.3 15\nSchneider, Davis 1 0 0 0 0 0 0 0 33.3 1 100 0.692 0.692'}, {'pitcher_id': '656546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoffman, Jeff 19 15 4 1 0 0 3 15.8 19.4 4 21.1 0.267 0.333 0.349 0.239 0.385 0.353 94.4 17.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 14 10 3 0 0 0 2 14.3 15 4 28.6 0.3 0.3 0.385 0.185 0.234 0.326 91.6 22.1\nKiner-Falefa, Isiah 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.333 0.83 0.488 102.6 22\nKirk, Alejandro 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.353 0.509 0.364 99.6 -5\nSpringer, George 1 1 1 1 0 0 0 0 0 0 0 1 2 1.238 0.367 0.756 0.445 98.3 22'}, {'pitcher_id': '605400', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Aaron 86 81 21 6 0 3 19 22.1 21.4 5 5.8 0.259 0.444 0.321 0.249 0.417 0.307 87.6 3.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 17 17 3 2 0 0 2 11.8 14.3 0 0 0.176 0.294 0.197 0.241 0.352 0.253 87.4 4.1\nVarsho, Daulton 13 12 3 0 0 1 5 38.5 28.6 1 7.7 0.25 0.5 0.344 0.12 0.294 0.212 83 10.3\nGuerrero Jr., Vladimir 12 8 5 0 0 1 0 0 14.3 4 33.3 0.625 1 0.695 0.493 0.944 0.636 89.4 -6.1\nBichette, Bo 10 10 4 1 0 1 3 30 30 0 0 0.4 0.8 0.502 0.362 0.664 0.435 96 2.9\nSpringer, George 6 6 1 0 0 0 3 50 26.7 0 0 0.167 0.167 0.148 0.168 0.185 0.155 96.1 -3.3\nKiner-Falefa, Isiah 5 5 0 0 0 0 1 20 0 0 0 0 0 0 0.177 0.197 0.164 68.8 26.5\nKiermaier, Kevin 7 7 2 1 0 0 1 14.3 18.2 0 0 0.286 0.429 0.305 0.311 0.501 0.348 96.5 8.2\nKirk, Alejandro 5 5 0 0 0 0 0 0 22.2 0 0 0 0 0 0.133 0.321 0.19 88.1 -12.2\nSchneider, Davis 3 3 2 1 0 0 1 33.3 20 0 0 0.667 1 0.719 0.33 0.423 0.313 91.1 19.5\nJansen, Danny 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.424 0.52 0.409 88.6 17.7\nRichards, Trevor 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.022 0.022 0.024 64.9 -23\nGausman, Kevin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.04 0.04 0.053 65.8 -49\nCabrera, Génesis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '689147', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerkering, Orion 6 5 2 0 0 0 1 16.7 33.3 1 16.7 0.4 0.4 0.412 0.115 0.136 0.207 68.2 31.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchneider, Davis 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nVarsho, Daulton 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nKiermaier, Kevin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.171 0.171 0.154 51.8 30\nSpringer, George 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.157 0.212 0.16 71.7 49\nKiner-Falefa, Isiah 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.227 0.247 0.204 65 10\nBichette, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.051 0.031 84.2 36'}, {'pitcher_id': '622554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDomínguez, Seranthony 20 15 2 0 0 1 2 10 18.2 3 15 0.133 0.333 0.324 0.231 0.396 0.38 86.2 -3.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.222 0.269 0.216 92.9 -2.3\nGuerrero Jr., Vladimir 4 2 1 0 0 1 1 25 14.3 1 25 0.5 2 0.873 0.466 1.528 0.774 110.6 21\nJansen, Danny 3 2 0 0 0 0 1 33.3 25 0 0 0 0 0.24 0.025 0.05 0.262 79.3 35\nVarsho, Daulton 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.469 0.503 0.564 109.9 -3\nBichette, Bo 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.127 0.128 0.12 78.7 -12\nKirk, Alejandro 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nSpringer, George 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.128 0.129 0.117 63.5 -19.5\nKiermaier, Kevin 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.569 0.636 0.519 105.8 3\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.05 0.048 59.8 -27'}, {'pitcher_id': '656814', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis F. 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.09 0.079 77.4 16.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.09 0.079 77.4 16.3'}, {'pitcher_id': '642397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Gregory 32 30 3 0 0 1 8 25 29.3 1 3.1 0.1 0.2 0.137 0.187 0.369 0.245 90.6 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBichette, Bo 6 6 1 0 0 0 0 0 14.3 0 0 0.167 0.167 0.147 0.342 0.681 0.428 95.4 23.3\nSpringer, George 5 5 1 0 0 0 2 40 30 0 0 0.2 0.2 0.177 0.163 0.197 0.159 91.9 -18.7\nGuerrero Jr., Vladimir 4 4 1 0 0 1 1 25 33.3 0 0 0.25 1 0.485 0.275 1.029 0.523 99.4 20.7\nKiermaier, Kevin 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.232 0.09 0.103 0.299 92.9 -28.5\nKiner-Falefa, Isiah 4 4 0 0 0 0 1 25 42.9 0 0 0 0 0 0.238 0.265 0.224 90.1 -6\nJansen, Danny 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.17 0.191 0.161 81 -12\nKirk, Alejandro 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.053 0.103 0.086 83.4 15\nTurner, Justin 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.057 0.063 0.052 76 0.5\nClement, Ernie 2 2 0 0 0 0 2 100 33.3 0 0 0 0 0 0\nSchneider, Davis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVarsho, Daulton 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}], 142: [{'pitcher_id': '554430', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWheeler, Zack 32 25 6 1 0 0 7 21.9 15.8 6 18.8 0.24 0.28 0.306 0.173 0.225 0.263 81 -3.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 12 5 3 0 0 0 1 8.3 18.2 6 50 0.6 0.6 0.566 0.284 0.342 0.502 97.9 1\nMargot, Manuel 12 12 3 1 0 0 2 16.7 9.1 0 0 0.25 0.333 0.25 0.201 0.285 0.212 81 -7\nVázquez, Christian 5 5 0 0 0 0 1 20 0 0 0 0 0 0 0.12 0.124 0.11 68.5 0\nPaddack, Chris 2 2 0 0 0 0 2 100 20 0 0 0 0 0 0\nLópez, Pablo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '656793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Nick 7 5 1 0 0 0 0 0 18.2 2 28.6 0.2 0.2 0.324 0.263 0.289 0.372 81.9 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 4 3 0 0 0 0 0 0 25 1 25 0 0 0.173 0.077 0.086 0.228 85 5\nCastro, Willi 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nFarmer, Kyle 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.884 0.903 0.976 0.841 66.8 25\nVázquez, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.179 0.211 0.161 87.6 -6'}, {'pitcher_id': '621237', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarado, José 23 19 2 0 0 0 7 30.4 27.8 3 13 0.105 0.105 0.167 0.151 0.22 0.239 86.7 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 13 11 1 0 0 0 4 30.8 18.2 1 7.7 0.091 0.091 0.121 0.154 0.234 0.224 87.6 17.9\nKepler, Max 3 3 0 0 0 0 2 66.7 62.5 0 0 0 0 0 0.122 0.246 0.151 96.2 21\nSantana, Carlos 5 3 1 0 0 0 0 0 25 2 40 0.333 0.333 0.453 0.264 0.277 0.425 80.1 -9.3\nMargot, Manuel 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.006 0.016 0.011 90.2 43'}, {'pitcher_id': '650911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Cristopher 22 20 4 1 0 2 3 13.6 25 2 9.1 0.2 0.55 0.345 0.227 0.51 0.344 88 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Kyle 6 6 1 0 0 1 1 16.7 25 0 0 0.167 0.667 0.345 0.249 0.78 0.429 85.1 12.6\nKepler, Max 3 2 1 0 0 1 0 0 25 1 33.3 0.5 2 0.9 0.565 1.346 0.758 89.3 15.5\nCorrea, Carlos 3 2 1 1 0 0 1 33.3 28.6 1 33.3 0.5 1 0.647 0.18 0.354 0.381 99 20\nJeffers, Ryan 3 3 0 0 0 0 0 0 40 0 0 0 0 0 0.17 0.282 0.189 88 -1\nVázquez, Christian 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.44 0.352 0.399 0.327 96.4 34.5\nCastro, Willi 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.062 0.063 0.057 86 25\nMargot, Manuel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.075 0.142 0.097 82.2 8.5'}, {'pitcher_id': '621381', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrahm, Matt 27 24 5 1 0 0 6 22.2 25 2 7.4 0.208 0.25 0.255 0.208 0.306 0.275 78.5 0.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 9 8 3 0 0 0 1 11.1 6.7 0 0 0.375 0.375 0.374 0.269 0.348 0.323 72.2 -27.4\nSantana, Carlos 6 5 0 0 0 0 1 16.7 27.3 1 16.7 0 0 0.115 0.259 0.284 0.316 80.6 21.8\nCorrea, Carlos 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.098 0.122 0.091 87.7 19.5\nMargot, Manuel 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.18 0.214 0.163 98.1 -3\nVázquez, Christian 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.021 0.022 0.018 81.4 -24\nMiranda, Jose 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nBuxton, Byron 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nLarnach, Trevor 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.62 2.053 1.108 101.4 27\nCastro, Willi 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 49.5 69\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.218 0.252 0.201 78.8 1'}, {'pitcher_id': '592836', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Taijuan 58 48 10 3 0 2 9 15.5 18.8 7 12.1 0.208 0.396 0.331 0.278 0.483 0.389 88.8 4.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 17 13 2 1 0 1 4 23.5 27.3 3 17.6 0.154 0.462 0.358 0.269 0.624 0.45 82.7 2.7\nMargot, Manuel 10 10 1 0 0 0 1 10 10.5 0 0 0.1 0.1 0.088 0.196 0.242 0.19 85.5 -10.6\nSantana, Carlos 9 7 2 2 0 0 2 22.2 14.3 1 11.1 0.286 0.571 0.434 0.16 0.337 0.318 92.7 11.8\nVázquez, Christian 8 7 0 0 0 0 1 12.5 21.4 1 12.5 0 0 0.087 0.282 0.446 0.351 93.5 14\nKepler, Max 6 5 2 0 0 1 1 16.7 16.7 1 16.7 0.4 1 0.598 0.421 0.699 0.512 93.3 3.5\nJeffers, Ryan 3 1 0 0 0 0 0 0 0 1 33.3 0 0 0.474 0.154 0.173 0.52 83.5 -4\nFarmer, Kyle 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.589 0.568 0.72 0.567 93.1 30.7\nBuxton, Byron 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.439 0.369 0.554 0.392 94.1 -5'}, {'pitcher_id': '624133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Ranger 23 19 4 0 0 0 8 34.8 35.5 4 17.4 0.211 0.211 0.273 0.184 0.276 0.284 81.6 6.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Kyle 9 7 0 0 0 0 2 22.2 30.8 2 22.2 0 0 0.154 0.078 0.116 0.217 82.6 28.4\nVázquez, Christian 5 5 3 0 0 0 2 40 28.6 0 0 0.6 0.6 0.527 0.378 0.656 0.437 80.6 -4\nJeffers, Ryan 3 2 0 0 0 0 2 66.7 40 1 33.3 0 0 0.232 0.232\nMargot, Manuel 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.434 0.483 0.509 90.5 4\nSantana, Carlos 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.095 0.095 0.086 62.2 -65\nLópez, Pablo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '614179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, José 51 44 13 3 0 3 8 15.7 13.6 7 13.7 0.295 0.568 0.406 0.356 0.554 0.43 91 10.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 9 9 4 0 0 0 1 11.1 10.5 0 0 0.444 0.444 0.391 0.428 0.505 0.41 89.9 17.8\nKepler, Max 9 8 1 0 0 1 0 0 0 1 11.1 0.125 0.5 0.3 0.387 0.566 0.437 93.7 2\nCastro, Willi 5 4 0 0 0 0 3 60 25 1 20 0 0 0.138 0.08 0.088 0.195 70.8 9\nJeffers, Ryan 7 5 1 0 0 0 1 14.3 16.7 2 28.6 0.2 0.2 0.322 0.236 0.295 0.364 80.4 12\nSantana, Carlos 5 4 2 0 0 1 1 20 14.3 1 20 0.5 1.25 0.714 0.263 0.763 0.471 89.8 4.3\nCorrea, Carlos 6 5 2 0 0 1 0 0 16.7 1 16.7 0.4 1 0.597 0.438 0.815 0.558 90.3 16.2\nBuxton, Byron 4 4 1 1 0 0 2 50 33.3 0 0 0.25 0.5 0.304 0.239 0.355 0.25 103.1 0\nLarnach, Trevor 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.621 0.611 1.267 0.791 94.7 26\nMiranda, Jose 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nFarmer, Kyle 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.674 1.053 0.727 105.1 15\nMargot, Manuel 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.555 0.673 0.534 95.9 4.5'}, {'pitcher_id': '628708', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Yunior 6 6 1 0 0 0 1 16.7 10 0 0 0.167 0.167 0.147 0.23 0.252 0.221 63.5 -21.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nFarmer, Kyle 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.427 0.493 0.422 64.2 4.5\nJeffers, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.051 0.049 69.2 -8\nMargot, Manuel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.223 0.227 0.205 77.8 -51\nVázquez, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.25 0.25 0.227 42.4 -56'}, {'pitcher_id': '605513', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurnbull, Spencer 41 37 10 3 0 1 8 19.5 20.7 4 9.8 0.27 0.432 0.333 0.323 0.646 0.43 92.4 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 28 26 9 3 0 1 5 17.9 22.4 2 7.1 0.346 0.577 0.408 0.373 0.699 0.463 91.9 11.5\nKepler, Max 11 9 0 0 0 0 2 18.2 13.6 2 18.2 0 0 0.125 0.182 0.531 0.367 92 29.6\nVázquez, Christian 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.435 0.296 0.477 0.315 105.7 15'}, {'pitcher_id': '656546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoffman, Jeff 8 7 1 0 0 0 2 25 23.5 1 12.5 0.143 0.143 0.196 0.204 0.234 0.252 92.2 -14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.54 0.603 0.502 100 5\nCastro, Willi 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nCorrea, Carlos 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.083 0.078 78.7 -38\nVázquez, Christian 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.028 0.055 0.025 87.6 -30\nMargot, Manuel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.213 0.239 0.195 94.8 -13'}, {'pitcher_id': '605400', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Aaron 32 26 7 2 0 1 4 12.5 16.3 3 9.4 0.269 0.462 0.359 0.243 0.359 0.326 87.8 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 10 8 3 2 0 0 2 20 29.4 1 10 0.375 0.625 0.473 0.296 0.527 0.416 96 15.2\nMargot, Manuel 7 7 1 0 0 0 0 0 0 0 0 0.143 0.143 0.125 0.177 0.194 0.166 79.8 6.6\nFarmer, Kyle 8 7 1 0 0 0 2 25 22.2 0 0 0.143 0.143 0.2 0.132 0.152 0.197 84.6 3.8\nSantana, Carlos 3 2 1 0 0 1 0 0 12.5 1 33.3 0.5 2 0.9 0.488 0.871 0.607 93.2 16.5\nBuxton, Byron 2 0 0 0 0 0 0 0 0 1 50 0.345 0.436 86.3 24\nKepler, Max 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.439 0.567 0.588 0.51 102.3 3'}, {'pitcher_id': '689147', 'player_data': '', 'team_data': 'Orion Kerkering has not faced anyone on this team.'}, {'pitcher_id': '622554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDomínguez, Seranthony 5 4 1 0 0 0 2 40 50 1 20 0.25 0.25 0.315 0.261 0.345 0.347 97.8 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 2 1 0 0 0 0 1 50 50 1 50 0 0 0.345 0.345\nVázquez, Christian 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.442 0.418 0.514 0.411 103.4 10\nBuxton, Byron 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.207 0.35 0.223 92.1 22'}, {'pitcher_id': '656814', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis F. 2 2 1 0 0 1 1 50 0 0 0 0.5 2 0.97 0.5 2 1 106.6 28', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 2 2 1 0 0 1 1 50 0 0 0 0.5 2 0.97 0.5 2 1 106.6 28'}, {'pitcher_id': '642397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Gregory 50 45 13 3 0 2 10 20 13.4 4 8 0.289 0.489 0.348 0.272 0.451 0.332 89 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 13 12 4 1 0 0 2 15.4 8 0 0 0.333 0.417 0.298 0.324 0.516 0.332 88.6 23.2\nVázquez, Christian 6 6 1 0 0 1 1 16.7 0 0 0 0.167 0.667 0.323 0.248 0.506 0.313 82.8 34.8\nSantana, Carlos 7 6 2 1 0 0 1 14.3 10 1 14.3 0.333 0.5 0.399 0.282 0.526 0.391 100.7 -3\nCorrea, Carlos 6 5 2 0 0 1 2 33.3 22.2 1 16.7 0.4 1 0.595 0.371 0.781 0.514 96.2 18.7\nJeffers, Ryan 5 4 1 0 0 0 2 40 22.2 1 20 0.25 0.25 0.316 0.189 0.216 0.283 97.6 -1\nBuxton, Byron 5 5 1 1 0 0 1 20 20 0 0 0.2 0.4 0.243 0.332 0.435 0.332 88.6 -22.8\nLarnach, Trevor 4 3 0 0 0 0 1 25 33.3 1 25 0 0 0.172 0.029 0.031 0.192 69.5 17\nMargot, Manuel 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.283 0.316 0.263 94.1 26.5\nFarmer, Kyle 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.126 0.127 0.116 72.8 -9.5'}], 144: [{'pitcher_id': '554430', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWheeler, Zack 305 282 62 14 1 10 82 26.9 28.7 17 5.6 0.22 0.383 0.281 0.223 0.355 0.277 88.9 11.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 48 46 11 5 0 1 15 31.3 23.9 2 4.2 0.239 0.413 0.293 0.244 0.394 0.292 88 2.2\nOzuna, Marcell 55 53 9 1 1 1 20 36.4 32.7 2 3.6 0.17 0.283 0.21 0.196 0.266 0.224 89.6 14.7\nAlbies, Ozzie 52 50 8 2 0 2 11 21.2 27.4 2 3.8 0.16 0.32 0.218 0.226 0.402 0.278 90.3 19.3\nOlson, Matt 28 26 7 1 0 2 9 32.1 37.7 2 7.1 0.269 0.538 0.367 0.194 0.327 0.254 91.8 9.8\nAnderson, Brian 28 22 6 2 0 0 5 17.9 27.5 5 17.9 0.273 0.364 0.338 0.277 0.371 0.372 93.1 12.9\nd'Arnaud, Travis 26 24 9 1 0 2 4 15.4 25.9 2 7.7 0.375 0.667 0.462 0.322 0.537 0.39 96.2 13.9\nArcia, Orlando 21 20 4 0 0 1 5 23.8 32.5 1 4.8 0.2 0.35 0.258 0.157 0.315 0.223 81.8 4.5\nDuvall, Adam 21 18 3 0 0 1 3 14.3 26.7 0 0 0.167 0.333 0.248 0.178 0.293 0.244 78.4 11.8\nFried, Max 7 5 1 1 0 0 2 28.6 21.4 1 14.3 0.2 0.4 0.318 0.255 0.316 0.319 91.8 3.7\nMurphy, Sean 7 7 1 1 0 0 3 42.9 33.3 0 0 0.143 0.286 0.178 0.187 0.233 0.183 89.2 25.5\nMorton, Charlie 5 4 1 0 0 0 3 60 42.9 0 0 0.25 0.25 0.22 0.078 0.09 0.07 75.4 -30.5\nLaureano, Ramón 3 3 1 0 0 0 2 66.7 30 0 0 0.333 0.333 0.294 0.324 0.363 0.305 79.6 19\nKelenic, Jarred 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.273 0.318 0.258 91.3 -11.5\nShort, Zack 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.038 0.051 0.038 80.3 28"}, {'pitcher_id': '656793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Nick 18 16 3 1 0 0 10 55.6 47.4 2 11.1 0.188 0.25 0.246 0.099 0.113 0.16 89 3.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 3 2 0 0 0 0 2 66.7 66.7 1 33.3 0 0 0.231 0.231\nRiley, Austin 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.295 0.139 0.154 0.131 86.1 -45.5\nOlson, Matt 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nAlbies, Ozzie 3 3 2 1 0 0 0 0 42.9 0 0 0.667 1 0.715 0.387 0.444 0.365 91.5 21.7\nd'Arnaud, Travis 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nAnderson, Brian 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.349 0.349\nDuvall, Adam 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nShort, Zack 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nArcia, Orlando 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.009 0.006 87.2 47"}, {'pitcher_id': '621237', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarado, José 68 56 11 3 0 3 21 30.9 33.6 10 14.7 0.196 0.411 0.323 0.234 0.413 0.342 92.7 19.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArcia, Orlando 9 8 1 1 0 0 6 66.7 50 1 11.1 0.125 0.25 0.216 0.176 0.212 0.228 103.7 6.5\nRiley, Austin 11 9 1 0 0 1 3 27.3 44.4 1 9.1 0.111 0.444 0.317 0.29 0.637 0.452 101.8 5.5\nOzuna, Marcell 12 11 2 1 0 0 4 33.3 30 1 8.3 0.182 0.273 0.234 0.288 0.381 0.318 92.8 25\nOlson, Matt 10 9 2 1 0 0 1 10 16 1 10 0.222 0.333 0.284 0.229 0.265 0.267 85 24.3\nd'Arnaud, Travis 7 4 2 0 0 1 2 28.6 18.2 2 28.6 0.5 1.25 0.619 0.297 0.961 0.515 96.3 18.3\nDuvall, Adam 6 2 1 0 0 1 0 0 0 4 66.7 0.5 2 0.795 0.459 1.639 0.745 95.3 43.5\nAlbies, Ozzie 7 7 2 0 0 0 0 0 25 0 0 0.286 0.286 0.252 0.159 0.181 0.152 85.9 18\nMurphy, Sean 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.319 0.354 0.293 108.1 4\nAnderson, Brian 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nLaureano, Ramón 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0"}, {'pitcher_id': '650911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Cristopher 38 35 11 3 0 1 7 18.4 27.7 2 5.3 0.314 0.486 0.35 0.289 0.462 0.342 92.7 10.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 5 4 2 1 0 0 1 20 45.5 1 20 0.5 0.75 0.564 0.321 0.57 0.436 99.5 23\nAlbies, Ozzie 7 6 3 1 0 0 0 0 27.3 1 14.3 0.5 0.667 0.529 0.324 0.422 0.377 88.7 4.8\nRiley, Austin 7 6 3 0 0 1 1 14.3 20 0 0 0.5 1 0.538 0.336 0.862 0.489 99 30\nArcia, Orlando 5 5 0 0 0 0 1 20 12.5 0 0 0 0 0 0.093 0.105 0.086 87.7 -21.5\nOlson, Matt 6 6 1 0 0 0 1 16.7 12.5 0 0 0.167 0.167 0.147 0.304 0.37 0.292 87.7 15\nAnderson, Brian 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.442 0.307 0.395 0.309 89.4 19\nd'Arnaud, Travis 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.622 0.49 0.712 0.506 93.4 11\nMurphy, Sean 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.336 0.404 0.317 99.2 3\nLaureano, Ramón 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0"}, {'pitcher_id': '621381', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrahm, Matt 45 43 11 2 0 1 9 20 18.6 1 2.2 0.256 0.372 0.272 0.256 0.422 0.298 91.8 21.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 7 7 0 0 0 0 2 28.6 6.3 0 0 0 0 0 0.103 0.302 0.164 78.9 26.8\nAlbies, Ozzie 8 7 1 0 0 0 2 25 25 0 0 0.143 0.143 0.109 0.253 0.286 0.253 87.7 8\nOzuna, Marcell 7 7 3 0 0 1 1 14.3 31.3 0 0 0.429 0.857 0.539 0.373 0.82 0.496 92.8 26.8\nAnderson, Brian 5 5 2 0 0 0 0 0 11.1 0 0 0.4 0.4 0.348 0.301 0.345 0.283 94.3 9.8\nMurphy, Sean 3 3 1 0 0 0 2 66.7 40 0 0 0.333 0.333 0.294 0.262 0.356 0.267 101.2 14\nArcia, Orlando 5 5 2 1 0 0 0 0 0 0 0 0.4 0.6 0.43 0.348 0.553 0.386 95.2 26\nRiley, Austin 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.243 0.36 0.259 98.4 40.5\nKelenic, Jarred 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.215 0.284 0.214 98.2 21.5\nLaureano, Ramón 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nDuvall, Adam 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.635 0.173 0.27 0.175 98 19\nd'Arnaud, Travis 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.187 0.212 0.17 96.2 31"}, {'pitcher_id': '592836', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Taijuan 104 93 26 7 1 2 20 19.2 24.4 8 7.7 0.28 0.441 0.341 0.24 0.442 0.331 88 16.6', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Brian 20 19 4 0 0 1 7 35 36.4 0 0 0.211 0.368 0.235 0.209 0.454 0.298 86 8.8\nOlson, Matt 14 11 3 2 0 0 2 14.3 19 3 21.4 0.273 0.455 0.391 0.224 0.359 0.342 91.9 35.3\nOzuna, Marcell 14 11 3 1 0 0 1 7.1 4.3 3 21.4 0.273 0.364 0.363 0.227 0.507 0.391 92.6 31.5\nAlbies, Ozzie 12 10 2 1 0 0 3 25 33.3 2 16.7 0.2 0.3 0.292 0.146 0.348 0.285 87.7 25.7\nRiley, Austin 14 12 5 0 1 1 2 14.3 16.7 0 0 0.417 0.833 0.547 0.381 0.777 0.519 92 21.2\nd'Arnaud, Travis 7 7 3 1 0 0 0 0 9.1 0 0 0.429 0.571 0.43 0.443 0.632 0.456 94.2 -0.4\nArcia, Orlando 7 7 3 1 0 0 0 0 11.1 0 0 0.429 0.571 0.43 0.269 0.33 0.262 74.4 -5.9\nDuvall, Adam 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.17 0.109 83.5 21.8\nShort, Zack 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.294 0.325 0.351 0.301 75.7 36\nLaureano, Ramón 3 3 1 1 0 0 1 33.3 33.3 0 0 0.333 0.667 0.413 0.267 0.409 0.284 94.3 0.5\nMurphy, Sean 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nKelenic, Jarred 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.035 0.036 0.034 77.3 -38\nFried, Max 1 1 1 0 0 0 0 0 50 0 0 1 1 0.879 0.159 0.183 0.143 95.3 -10"}, {'pitcher_id': '624133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Ranger 147 130 34 7 0 7 28 19 23.8 17 11.6 0.262 0.477 0.356 0.239 0.427 0.331 87.9 12.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 24 21 6 2 0 0 4 16.7 20 3 12.5 0.286 0.381 0.338 0.22 0.357 0.302 89.5 21.8\nRiley, Austin 26 22 8 3 0 2 4 15.4 25 4 15.4 0.364 0.773 0.508 0.294 0.582 0.417 92.9 22.7\nAlbies, Ozzie 21 20 4 0 0 2 4 19 20 1 4.8 0.2 0.5 0.309 0.187 0.352 0.249 86.9 11\nOzuna, Marcell 19 18 3 0 0 0 5 26.3 26.3 1 5.3 0.167 0.167 0.176 0.233 0.315 0.262 88.3 -0.4\nArcia, Orlando 14 10 1 1 0 0 3 21.4 23.8 4 28.6 0.1 0.2 0.287 0.17 0.222 0.316 86.4 14.9\nDuvall, Adam 13 11 4 0 0 2 3 23.1 23.1 2 15.4 0.364 0.909 0.55 0.366 0.879 0.545 94.2 16.4\nd'Arnaud, Travis 16 14 2 0 0 1 3 18.8 31.3 2 12.5 0.143 0.357 0.272 0.176 0.383 0.298 85.7 1.7\nAnderson, Brian 8 8 4 1 0 0 0 0 16.7 0 0 0.5 0.625 0.489 0.241 0.381 0.272 79.9 0.9\nKelenic, Jarred 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.118 0.119 0.11 39.1 -21\nMurphy, Sean 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.294 0.282 0.317 0.259 84 8.5\nFried, Max 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.847 0.948 0.78 72.6 17"}, {'pitcher_id': '614179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, José 14 12 2 0 0 1 5 35.7 46.2 2 14.3 0.167 0.417 0.305 0.17 0.35 0.287 84.9 6.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 4 3 0 0 0 0 1 25 33.3 1 25 0 0 0.173 0.111 0.124 0.249 85.6 -7\nShort, Zack 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nLaureano, Ramón 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nArcia, Orlando 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.857 2.872 1.508 105.3 23\nKelenic, Jarred 1 1 1 0 0 0 0 0 50 0 0 1 1 0.884 0.609 0.703 0.594 74.1 29\nMurphy, Sean 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nAnderson, Brian 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.113 0.113 0.104 77.8 -14\nOzuna, Marcell 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.023 0.014 88.3 38\nd'Arnaud, Travis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0"}, {'pitcher_id': '628708', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Yunior 27 24 6 1 0 1 8 29.6 34 3 11.1 0.25 0.417 0.329 0.234 0.405 0.317 85.8 3.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArcia, Orlando 6 5 1 1 0 0 1 16.7 12.5 1 16.7 0.2 0.4 0.328 0.18 0.284 0.28 84.7 -13.8\nd'Arnaud, Travis 4 4 0 0 0 0 2 50 50 0 0 0 0 0 0.01 0.015 0.01 71.2 17.5\nOzuna, Marcell 5 5 3 0 0 1 0 0 0 0 0 0.6 1.2 0.754 0.487 1.14 0.67 85.4 12.6\nAlbies, Ozzie 2 1 0 0 0 0 1 50 20 1 50 0 0 0.348 0.348\nOlson, Matt 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.227 0.263 0.217 107.7 5\nKelenic, Jarred 1 0 0 0 0 0 0 0 40 1 100 0.692 0.692\nMurphy, Sean 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.441 0.437 0.499 0.415 82.6 14\nRiley, Austin 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.197 0.215 0.178 91.5 8\nDuvall, Adam 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.102 0.118 0.095 91.7 -26"}, {'pitcher_id': '605513', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurnbull, Spencer 29 25 3 1 0 0 5 17.2 23.1 2 6.9 0.12 0.16 0.174 0.227 0.358 0.297 87 7.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.329 0.374 0.306 79.2 3.8\nArcia, Orlando 6 6 0 0 0 0 1 16.7 27.3 0 0 0 0 0 0.198 0.316 0.223 89.5 8.2\nAnderson, Brian 5 4 1 1 0 0 1 20 25 0 0 0.25 0.5 0.243 0.473 0.996 0.535 94.9 29.3\nRiley, Austin 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.23 0.099 0.1 0.289 69.6 -53\nLaureano, Ramón 3 2 0 0 0 0 2 66.7 71.4 0 0 0 0 0.24 0.24\nKelenic, Jarred 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.189 0.206 0.301 86.9 -7.3\nAlbies, Ozzie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.16 0.172 0.167 84.1 -10\nDuvall, Adam 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.103 0.201 0.122 92.3 42.5'}, {'pitcher_id': '656546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoffman, Jeff 56 54 14 3 1 2 10 17.9 26.6 2 3.6 0.259 0.463 0.316 0.248 0.462 0.313 89.6 16.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 11 11 3 0 0 1 3 27.3 36 0 0 0.273 0.545 0.343 0.239 0.433 0.284 92.9 29.5\nAlbies, Ozzie 10 10 3 1 1 1 0 0 18.2 0 0 0.3 0.9 0.48 0.283 0.647 0.383 85.9 12.6\nArcia, Orlando 9 8 3 0 0 0 0 0 11.1 1 11.1 0.375 0.375 0.37 0.377 0.531 0.419 93.6 7.3\nOlson, Matt 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.591 1.282 0.799 107 5.3\nd'Arnaud, Travis 7 7 2 1 0 0 2 28.6 22.2 0 0 0.286 0.429 0.302 0.148 0.172 0.141 78.9 5.8\nOzuna, Marcell 6 5 0 0 0 0 1 16.7 54.5 1 16.7 0 0 0.116 0.038 0.041 0.144 77.3 28\nAnderson, Brian 3 3 1 1 0 0 2 66.7 50 0 0 0.333 0.667 0.411 0.28 0.766 0.425 107.4 19\nDuvall, Adam 4 4 0 0 0 0 2 50 37.5 0 0 0 0 0 0.01 0.024 0.011 94.4 42\nMurphy, Sean 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.236 0.275 0.217 80.6 21.5"}, {'pitcher_id': '605400', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Aaron 381 360 98 20 1 17 108 28.3 30.2 18 4.7 0.272 0.475 0.335 0.24 0.42 0.301 89.8 11.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlbies, Ozzie 69 65 18 3 1 1 13 18.8 27.5 4 5.8 0.277 0.4 0.312 0.211 0.311 0.249 87.1 19.3\nRiley, Austin 63 60 22 5 0 5 18 28.6 25.6 3 4.8 0.367 0.7 0.457 0.313 0.652 0.415 92.2 10.4\nOzuna, Marcell 59 57 14 1 0 4 21 35.6 37.4 2 3.4 0.246 0.474 0.313 0.266 0.467 0.322 95.5 10.2\nd'Arnaud, Travis 42 41 7 4 0 2 10 23.8 28.4 1 2.4 0.171 0.415 0.252 0.197 0.39 0.258 87 6.3\nDuvall, Adam 38 38 11 2 0 3 13 34.2 36.1 0 0 0.289 0.579 0.363 0.257 0.549 0.338 90.9 20.7\nAnderson, Brian 34 32 11 1 0 0 11 32.4 30 1 2.9 0.344 0.375 0.336 0.237 0.287 0.257 89 4\nOlson, Matt 33 27 7 2 0 2 7 21.2 32.1 5 15.2 0.259 0.556 0.409 0.162 0.292 0.29 87.7 19.4\nArcia, Orlando 24 23 4 1 0 0 9 37.5 24.5 1 4.2 0.174 0.217 0.192 0.172 0.211 0.189 86.5 -4.1\nMurphy, Sean 8 8 2 1 0 0 4 50 46.7 0 0 0.25 0.375 0.268 0.293 0.411 0.302 96.7 10.8\nFried, Max 6 5 0 0 0 0 2 33.3 28.6 0 0 0 0 0 0.251 0.279 0.234 85.8 -16.3\nKelenic, Jarred 5 4 2 0 0 0 0 0 0 1 20 0.5 0.5 0.493 0.367 0.406 0.409 82.7 22.3"}, {'pitcher_id': '689147', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerkering, Orion 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.354 0.452 0.339 94.5 1.4', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOzuna, Marcell 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.346 0.417 0.314 89.6 3\nArcia, Orlando 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.496 0.577 0.464 95.9 6\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.331 0.572 0.363 96.8 19\nMurphy, Sean 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.119 0.091 94.6 -27"}, {'pitcher_id': '622554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDomínguez, Seranthony 57 51 13 2 0 1 18 31.6 35.5 6 10.5 0.255 0.353 0.308 0.227 0.357 0.296 90.7 12.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRiley, Austin 10 10 3 1 0 0 5 50 50 0 0 0.3 0.4 0.303 0.227 0.321 0.234 89.8 23.6\nAlbies, Ozzie 11 9 4 0 0 0 0 0 0 2 18.2 0.444 0.444 0.447 0.383 0.504 0.442 88.8 16.2\nAnderson, Brian 7 6 1 0 0 0 3 42.9 43.8 1 14.3 0.167 0.167 0.223 0.141 0.285 0.249 88.2 11\nOlson, Matt 7 7 1 0 0 0 3 42.9 31.3 0 0 0.143 0.143 0.126 0.236 0.363 0.253 98.4 19.3\nArcia, Orlando 7 5 1 0 0 0 1 14.3 27.3 2 28.6 0.2 0.2 0.323 0.218 0.3 0.355 92.3 12.5\nd'Arnaud, Travis 4 3 0 0 0 0 1 25 38.5 1 25 0 0 0.172 0.122 0.191 0.271 89.7 26\nOzuna, Marcell 4 4 1 0 0 1 1 25 50 0 0 0.25 1 0.521 0.113 0.324 0.182 87.9 -3.3\nMurphy, Sean 3 3 0 0 0 0 2 66.7 33.3 0 0 0 0 0 0.137 0.143 0.124 107.2 -6\nKelenic, Jarred 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.635 0.425 1.111 0.636 107.4 21\nDuvall, Adam 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nShort, Zack 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.215 0.217 0.198 59.9 -68"}, {'pitcher_id': '656814', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis F. 6 6 3 0 0 0 1 16.7 33.3 0 0 0.5 0.5 0.444 0.363 0.735 0.462 96.7 9.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nd'Arnaud, Travis 2 2 1 0 0 0 0 0 42.9 0 0 0.5 0.5 0.442 0.399 0.625 0.431 105 1.5\nKelenic, Jarred 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.285 0.307 0.26 104.6 3\nRiley, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.609 2.333 1.208 102.6 32\nArcia, Orlando 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.198 0.212 0.18 66.4 8"}, {'pitcher_id': '642397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Gregory 36 27 10 5 0 0 5 13.9 25.5 8 22.2 0.37 0.556 0.45 0.264 0.467 0.384 95.9 8.8', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 11 9 1 1 0 0 0 0 14.3 1 9.1 0.111 0.222 0.176 0.147 0.172 0.177 95 16.6\nKelenic, Jarred 4 2 1 0 0 0 0 0 0 2 50 0.5 0.5 0.567 0.29 0.297 0.478 98.6 -9.5\nArcia, Orlando 4 3 0 0 0 0 3 75 83.3 1 25 0 0 0.174 0.174\nMurphy, Sean 4 2 1 0 0 0 1 25 50 2 50 0.5 0.5 0.566 0.117 0.127 0.399 61.2 -49\nOzuna, Marcell 4 3 2 2 0 0 0 0 0 1 25 0.667 1.333 0.796 0.271 0.339 0.366 91.1 21.3\nLaureano, Ramón 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.71 0.608 1.726 0.935 104.2 9.7\nRiley, Austin 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.635 0.415 1.128 0.64 105.5 21\nd'Arnaud, Travis 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.517 0.548 0.582 110.8 -1\nAlbies, Ozzie 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.441 0.508 0.602 0.487 98 -4.5"}], 145: [{'pitcher_id': '554430', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWheeler, Zack 79 67 16 3 0 3 15 19 24.7 6 7.6 0.239 0.418 0.337 0.261 0.446 0.36 85.5 4.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 18 14 5 0 0 1 3 16.7 23.1 4 22.2 0.357 0.571 0.462 0.317 0.59 0.453 94.5 8.1\nDeJong, Paul 14 12 4 1 0 2 4 28.6 39.4 0 0 0.333 0.917 0.537 0.328 0.806 0.503 91.5 12.1\nBenintendi, Andrew 6 6 2 1 0 0 2 33.3 23.5 0 0 0.333 0.5 0.354 0.172 0.292 0.198 85.5 12\nLopez, Nicky 6 5 0 0 0 0 1 16.7 6.7 1 16.7 0 0 0.115 0.188 0.215 0.26 91.6 10.8\nMaldonado, Martín 9 7 0 0 0 0 1 11.1 20 0 0 0 0 0.161 0.234 0.24 0.382 62.8 -6\nVaughn, Andrew 6 5 1 0 0 0 3 50 40 0 0 0.2 0.2 0.268 0.184 0.212 0.266 81.9 -18\nSheets, Gavin 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.144 0.16 0.318 83 -13\nRobert Jr., Luis 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.534 0.812 0.579 90 18.3\nSosa, Lenyn 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.419 0.504 0.405 85.1 6.7\nJulks, Corey 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.298 0.334 0.275 69.3 40.5\nLee, Korey 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.059 0.067 0.055 77.1 9.7\nColás, Oscar 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.122 0.122 0.112 71 -51\nFedde, Erick 1 0 0 0 0 0 0 0 0 0 0\nSoroka, Michael 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.125 0.117 74.4 -34.7'}, {'pitcher_id': '656793', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNelson, Nick 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.233 0.152 0.163 0.34 91.2 -11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.233 0.152 0.163 0.34 91.2 -11'}, {'pitcher_id': '621237', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAlvarado, José 17 14 1 0 0 0 3 17.6 26.9 1 5.9 0.071 0.071 0.093 0.226 0.301 0.263 86.7 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 8 7 0 0 0 0 1 12.5 30.8 0 0 0 0 0 0.154 0.173 0.127 79 4\nMaldonado, Martín 4 2 0 0 0 0 1 25 50 1 25 0 0 0.175 0.043 0.059 0.341 88.4 30.5\nPham, Tommy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.592 0.867 0.631 103.5 15.3\nDeJong, Paul 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSosa, Lenyn 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.224 0.284 0.205 87 -4'}, {'pitcher_id': '650911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Cristopher 5 5 0 0 0 0 2 40 10 0 0 0 0 0 0.138 0.169 0.132 93.4 24', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 3 3 0 0 0 0 2 66.7 0 0 0 0 0 0 0.001 0.001 0.001 91.8 47\nMendick, Danny 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.343 0.421 0.328 94.2 12.5'}, {'pitcher_id': '621381', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrahm, Matt 14 13 2 1 0 0 5 35.7 30.8 1 7.1 0.154 0.231 0.2 0.153 0.385 0.253 93.6 29.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 6 6 1 1 0 0 1 16.7 38.5 0 0 0.167 0.333 0.203 0.236 0.636 0.351 93.6 38.2\nMaldonado, Martín 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.231 0.067 0.258 0.319 92.9 31\nMendick, Danny 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.12 0.225 0.145 94.4 21\nPham, Tommy 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSosa, Lenyn 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLopez, Nicky 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.206 0.232 0.185 93 -7'}, {'pitcher_id': '592836', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWalker, Taijuan 43 38 8 1 0 1 4 9.3 10.8 4 9.3 0.211 0.316 0.263 0.275 0.474 0.345 85.4 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 12 10 1 0 0 0 1 8.3 10.5 2 16.7 0.1 0.1 0.189 0.184 0.21 0.259 85.6 15.4\nDeJong, Paul 11 9 1 0 0 0 1 9.1 14.3 1 9.1 0.111 0.111 0.143 0.301 0.483 0.341 83.2 23.4\nBenintendi, Andrew 6 6 1 0 0 0 1 16.7 18.2 0 0 0.167 0.167 0.147 0.146 0.231 0.158 90 11.8\nVaughn, Andrew 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.962 0.764 2.326 1.255 106.1 23\nMendick, Danny 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.179 0.203 0.34 74.4 10\nRobert Jr., Luis 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.505 0.539 0.464 72.5 15.7\nSosa, Lenyn 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.03 0.032 0.03 67.8 29\nColás, Oscar 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.228 0.257 0.205 104.2 -3\nMaldonado, Martín 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.36 0.584 0.399 94.4 20'}, {'pitcher_id': '624133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSuárez, Ranger 13 11 2 0 0 1 8 61.5 35 2 15.4 0.182 0.455 0.334 0.117 0.331 0.266 97.3 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 8 6 0 0 0 0 5 62.5 46.2 2 25 0 0 0.174 0.007 0.008 0.179 78.8 49\nMaldonado, Martín 4 4 2 0 0 1 2 50 20 0 0 0.5 1.25 0.739 0.313 0.899 0.506 106.5 10\nBenintendi, Andrew 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '614179', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRuiz, José 19 19 5 0 0 0 3 15.8 22.2 0 0 0.263 0.263 0.231 0.228 0.278 0.223 88.2 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 6 6 2 0 0 0 1 16.7 16.7 0 0 0.333 0.333 0.294 0.384 0.464 0.374 90.8 3\nMaldonado, Martín 7 7 0 0 0 0 0 0 20 0 0 0 0 0 0.029 0.035 0.03 84.1 37.3\nJulks, Corey 2 2 1 0 0 0 1 50 60 0 0 0.5 0.5 0.441 0.481 0.593 0.47 97 13\nBenintendi, Andrew 2 2 2 0 0 0 0 0 0 0 0 1 1 0.87 0.38 0.481 0.372 95.8 14.5\nPham, Tommy 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.054 0.055 0.051 80.2 -45'}, {'pitcher_id': '628708', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarte, Yunior 11 10 2 0 0 0 1 9.1 13.3 1 9.1 0.2 0.2 0.224 0.243 0.265 0.267 88.7 4.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.526 0.578 0.488 95.7 6.3\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.022 0.014 86.9 55\nSheets, Gavin 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nRobert Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.088 0.082 69.8 -7\nVaughn, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.274 0.281 0.248 100.1 -7.5\nJulks, Corey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.122 0.156 0.11 84.4 -6'}, {'pitcher_id': '605513', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurnbull, Spencer 49 41 10 0 1 0 8 16.3 21.1 5 10.2 0.244 0.293 0.291 0.261 0.365 0.324 85.3 3.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 13 11 4 0 1 0 3 23.1 25 0 0 0.364 0.545 0.381 0.167 0.19 0.153 79.8 -19.3\nMaldonado, Martín 10 9 0 0 0 0 2 20 23.8 1 10 0 0 0.069 0.183 0.27 0.245 86.5 6.6\nBenintendi, Andrew 11 10 4 0 0 0 1 9.1 20 0 0 0.4 0.4 0.384 0.447 0.651 0.493 87.2 11\nMendick, Danny 4 3 1 0 0 0 0 0 7.7 1 25 0.333 0.333 0.393 0.29 0.333 0.38 98.4 17.7\nDeJong, Paul 5 3 0 0 0 0 1 20 30 2 40 0 0 0.278 0.197 0.473 0.444 76.9 26.5\nVaughn, Andrew 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.025 0.03 0.246 80.9 51\nSheets, Gavin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.411 0.493 0.393 89.3 6.7'}, {'pitcher_id': '656546', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoffman, Jeff 17 15 3 0 0 1 7 41.2 37.5 1 5.9 0.2 0.4 0.26 0.138 0.315 0.223 86 10.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 6 6 1 0 0 1 3 50 25 0 0 0.167 0.667 0.33 0.157 0.572 0.29 85.7 24.7\nPham, Tommy 5 3 1 0 0 0 0 0 20 1 20 0.333 0.333 0.314 0.344 0.398 0.394 86.5 -6\nMaldonado, Martín 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.293 0.026 0.026 0.023 90.3 -17\nVaughn, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRobert Jr., Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.016 0.013 80.9 59'}, {'pitcher_id': '605400', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNola, Aaron 70 69 10 1 0 2 17 24.3 25.2 0 0 0.145 0.246 0.165 0.192 0.289 0.206 83.8 7.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 13 13 1 0 0 0 6 46.2 25.7 0 0 0.077 0.077 0.068 0.084 0.093 0.077 81.2 6.6\nPham, Tommy 14 14 2 0 0 1 4 28.6 17.4 0 0 0.143 0.357 0.206 0.227 0.399 0.264 87.5 7.9\nDeJong, Paul 14 14 3 0 0 1 4 28.6 37 0 0 0.214 0.429 0.266 0.278 0.44 0.303 86 7\nBenintendi, Andrew 10 10 0 0 0 0 1 10 14.3 0 0 0 0 0 0.132 0.173 0.134 79 13.1\nJulks, Corey 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.294 0.117 0.169 0.119 80.4 18.3\nLopez, Nicky 6 6 2 0 0 0 0 0 14.3 0 0 0.333 0.333 0.295 0.374 0.497 0.38 83.6 -5.3\nSosa, Lenyn 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.024 0.025 0.022 82 -39\nSheets, Gavin 3 3 1 1 0 0 0 0 33.3 0 0 0.333 0.667 0.423 0.104 0.236 0.142 87.1 13\nKuhl, Chad 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.204 0.162 74.1 43\nFedde, Erick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.497 0.414 98.7 0\nSoroka, Michael 1 0 0 0 0 0 0 0 0 0 0'}, {'pitcher_id': '689147', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerkering, Orion 5 5 1 0 0 0 2 40 20 0 0 0.2 0.2 0.177 0.158 0.25 0.176 89.6 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.059 0.063 0.053 81.5 -22\nPham, Tommy 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.303 0.412 0.309 93.7 17\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.123 0.364 0.206 93.7 28\nVaughn, Andrew 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '622554', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDomínguez, Seranthony 10 8 0 0 0 0 2 20 22.7 1 10 0 0 0.142 0.096 0.108 0.233 73.7 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 2 2 0 0 0 0 1 50 14.3 0 0 0 0 0 0.025 0.03 0.029 61 -19\nPham, Tommy 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.069 0.07 0.069 55.3 -40\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 86.1 72\nMaldonado, Martín 2 1 0 0 0 0 0 0 50 0 0 0 0 0.363 0.726\nJulks, Corey 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.426 0.501 0.408 83.3 8\nLopez, Nicky 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.055 0.057 0.073 82.9 -21'}, {'pitcher_id': '656814', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOrtiz, Luis F. 4 4 0 0 0 0 2 50 25 0 0 0 0 0 0.027 0.097 0.05 89.3 54.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0 0 0 80 69\nRobert Jr., Luis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVaughn, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.387 0.198 98.6 40'}, {'pitcher_id': '642397', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Gregory 30 26 4 1 0 1 9 30 31.7 4 13.3 0.154 0.308 0.262 0.153 0.318 0.264 92 10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 7 4 1 0 0 0 1 14.3 14.3 3 42.9 0.25 0.25 0.421 0.1 0.112 0.35 87.7 -27.7\nBenintendi, Andrew 5 4 1 0 0 0 1 20 18.2 1 20 0.25 0.25 0.314 0.265 0.314 0.341 91.5 11.3\nMendick, Danny 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.109 0.114 0.099 88.4 -24.5\nVaughn, Andrew 4 4 1 0 0 1 1 25 20 0 0 0.25 1 0.518 0.311 1.147 0.607 98.3 29\nRobert Jr., Luis 3 3 1 1 0 0 1 33.3 50 0 0 0.333 0.667 0.42 0.102 0.25 0.139 87 21\nMaldonado, Martín 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.193 0.227 0.177 94.4 34\nSosa, Lenyn 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.03 0.1 0.051 96.7 42\nDeJong, Paul 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nPham, Tommy 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nColás, Oscar 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}]}, 144: {141: [{'pitcher_id': '642216', 'player_data': '', 'team_data': ''}, {'pitcher_id': '675911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrider, Spencer 26 24 4 1 0 0 12 46.2 50 2 7.7 0.167 0.208 0.203 0.115 0.165 0.164 89.4 27.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarsho, Daulton 6 5 0 0 0 0 5 83.3 71.4 1 16.7 0 0 0.115 0.115\nTurner, Justin 6 6 1 0 0 0 2 33.3 46.2 0 0 0.167 0.167 0.147 0.193 0.331 0.224 91.3 35.8\nBichette, Bo 3 3 0 0 0 0 2 66.7 38.5 0 0 0 0 0 0.122 0.169 0.123 89.6 20\nGuerrero Jr., Vladimir 3 3 1 1 0 0 1 33.3 66.7 0 0 0.333 0.667 0.415 0.017 0.049 0.028 88.1 54\nSpringer, George 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.294 0.19 0.212 0.172 99 5\nKiermaier, Kevin 2 1 1 0 0 0 0 0 25 1 50 1 1 0.789 0.342 0.389 0.505 103.3 -10\nKirk, Alejandro 3 3 0 0 0 0 0 0 28.6 0 0 0 0 0 0.092 0.101 0.083 79.6 22'}, {'pitcher_id': '519242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSale, Chris 94 86 22 3 0 5 33 35.1 33.5 7 7.4 0.256 0.465 0.327 0.227 0.448 0.311 87.8 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 31 28 7 1 0 1 10 32.3 28.6 2 6.5 0.25 0.393 0.292 0.296 0.595 0.383 90.2 22.7\nKiermaier, Kevin 23 22 6 0 0 1 8 34.8 36.8 1 4.3 0.273 0.409 0.307 0.185 0.348 0.246 82.5 11.5\nGuerrero Jr., Vladimir 11 8 2 0 0 1 4 36.4 31.6 3 27.3 0.25 0.625 0.451 0.162 0.262 0.322 88.1 5.7\nKiner-Falefa, Isiah 6 5 1 1 0 0 2 33.3 22.2 1 16.7 0.2 0.4 0.322 0.148 0.163 0.232 77.3 -26\nJansen, Danny 10 10 2 0 0 1 2 20 44.4 0 0 0.2 0.5 0.281 0.273 0.583 0.354 88.2 29.5\nTurner, Justin 6 6 3 1 0 1 3 50 30.8 0 0 0.5 1.167 0.673 0.279 0.751 0.413 97.4 17\nBichette, Bo 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.157 0.173 0.144 102 0\nSchneider, Davis 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nKirk, Alejandro 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.141 0.233 0.16 95 4.5'}, {'pitcher_id': '450203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorton, Charlie 110 98 26 4 0 5 27 24.5 22.2 5 4.5 0.265 0.459 0.336 0.238 0.37 0.294 87.3 9.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 32 26 8 2 0 2 4 12.5 14.6 3 9.4 0.308 0.615 0.423 0.331 0.61 0.418 88.2 20.1\nSpringer, George 18 17 4 0 0 1 3 16.7 14.7 0 0 0.235 0.412 0.297 0.27 0.394 0.315 88.7 4.4\nBichette, Bo 16 16 5 1 0 1 3 18.8 25.6 0 0 0.313 0.563 0.362 0.271 0.429 0.299 89.7 10.2\nGuerrero Jr., Vladimir 14 13 4 1 0 0 4 28.6 31 0 0 0.308 0.385 0.327 0.182 0.233 0.218 83.3 0\nKiermaier, Kevin 13 12 2 0 0 0 8 61.5 42.9 0 0 0.167 0.167 0.135 0.072 0.078 0.069 92.7 -25.8\nKiner-Falefa, Isiah 7 7 1 0 0 1 5 71.4 20 0 0 0.143 0.571 0.29 0.089 0.193 0.116 83.9 35\nVarsho, Daulton 4 2 1 0 0 0 0 0 16.7 1 25 0.5 0.5 0.572 0.42 0.464 0.548 80.1 34.5\nJansen, Danny 4 4 1 0 0 0 0 0 11.1 0 0 0.25 0.25 0.221 0.324 0.368 0.294 82.4 24.3\nKirk, Alejandro 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.035 0.035 0.371 77.1 -22'}, {'pitcher_id': '678061', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerr, Ray 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.337 0.396 0.324 67 44', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.337 0.396 0.324 67 44'}, {'pitcher_id': '554431', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatzek, Tyler 13 12 3 2 0 0 2 15.4 30.4 1 7.7 0.25 0.417 0.316 0.11 0.188 0.178 89.2 40.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 9 8 3 2 0 0 0 0 21.4 1 11.1 0.375 0.625 0.457 0.195 0.336 0.295 89.2 25.8\nJansen, Danny 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSpringer, George 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nKirk, Alejandro 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.009 0.01 0.008 94.8 70\nVarsho, Daulton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 83.8 82'}, {'pitcher_id': '623474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Jimmy 22 20 3 0 0 1 5 22.7 23.5 1 4.5 0.15 0.3 0.236 0.202 0.366 0.283 83.9 20.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKiner-Falefa, Isiah 5 4 1 0 0 0 1 20 20 0 0 0.25 0.25 0.321 0.168 0.185 0.267 82.2 32\nSpringer, George 4 4 1 0 0 0 2 50 28.6 0 0 0.25 0.25 0.221 0.158 0.174 0.147 81.8 22.5\nGuerrero Jr., Vladimir 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.211 0.238 0.202 86.9 -2\nTurner, Justin 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.03 0.03 0.028 87.4 25\nVarsho, Daulton 1 0 0 0 0 0 0 0 0 1 100 0.699 0.699\nBichette, Bo 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.668 0.435 1.442 0.76 98.9 12.3\nKirk, Alejandro 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.222 0.24 0.2 67.9 33.5\nKiermaier, Kevin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.285 0.308 0.262 66.9 10'}, {'pitcher_id': '572955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJohnson, Pierce 17 14 2 1 0 0 4 23.5 21.9 3 17.6 0.143 0.214 0.247 0.185 0.259 0.28 82.4 21.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 8 7 1 1 0 0 2 25 27.3 1 12.5 0.143 0.286 0.242 0.121 0.228 0.213 86.3 39.2\nVarsho, Daulton 4 3 0 0 0 0 2 50 37.5 1 25 0 0 0.172 0.17 0.191 0.291 93.7 5\nKiner-Falefa, Isiah 4 3 1 0 0 0 0 0 11.1 1 25 0.333 0.333 0.395 0.313 0.386 0.405 86 10.7\nSpringer, George 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.297 0.298 0.267 41 -18'}, {'pitcher_id': '621345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMinter, A.J. 20 18 11 3 0 0 1 5 20 1 5 0.611 0.778 0.573 0.444 0.637 0.496 91.6 12.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 6 6 3 2 0 0 0 0 36.4 0 0 0.5 0.833 0.561 0.466 0.864 0.57 97.1 16.5\nGuerrero Jr., Vladimir 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.293 0.556 0.699 0.626 93.3 21.3\nKiermaier, Kevin 4 3 3 1 0 0 0 0 33.3 1 25 1 1.333 0.924 0.436 0.492 0.478 85.5 11\nVarsho, Daulton 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.295 0.338 0.392 0.329 73.2 -38.5\nBichette, Bo 2 2 2 0 0 0 0 0 0 0 0 1 1 0.881 0.661 0.88 0.658 106.6 10\nJansen, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 89 71\nSpringer, George 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.443 0.476 0.402 80.7 10'}, {'pitcher_id': '678226', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Daysbel 1 0 0 0 0 0 0 0 1 100 0.696 0.696', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '700363', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '628452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIglesias, Raisel 17 17 4 0 0 0 8 47.1 47.2 0 0 0.235 0.235 0.208 0.167 0.307 0.203 87.5 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 4 4 1 0 0 0 3 75 60 0 0 0.25 0.25 0.221 0.234 0.78 0.412 107.8 22\nTurner, Justin 3 3 0 0 0 0 1 33.3 57.1 0 0 0 0 0 0.062 0.078 0.075 87.5 13.5\nKiner-Falefa, Isiah 3 3 1 0 0 0 1 33.3 28.6 0 0 0.333 0.333 0.293 0.246 0.283 0.229 88.9 0\nKiermaier, Kevin 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nJansen, Danny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.332 0.353 0.299 85.1 3\nKirk, Alejandro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.346 0.346 0.311 66.7 13\nBichette, Bo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSpringer, George 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.289 0.311 0.274 90 1\nVarsho, Daulton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 85 77'}, {'pitcher_id': '625643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Reynaldo 50 47 14 3 0 0 9 18 22.6 3 6 0.298 0.362 0.309 0.301 0.417 0.332 89.7 6.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKiermaier, Kevin 11 9 4 0 0 0 2 18.2 18.2 2 18.2 0.444 0.444 0.444 0.379 0.463 0.425 94.5 -7.6\nSpringer, George 10 10 1 1 0 0 1 10 17.4 0 0 0.1 0.2 0.125 0.154 0.245 0.174 83.3 15.9\nTurner, Justin 6 6 3 1 0 0 2 33.3 54.5 0 0 0.5 0.667 0.501 0.374 0.662 0.434 98.1 7.3\nKiner-Falefa, Isiah 5 5 2 0 0 0 1 20 18.2 0 0 0.4 0.4 0.353 0.303 0.338 0.282 85.5 -3\nJansen, Danny 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.217 0.349 0.441 0.338 102 16.7\nGuerrero Jr., Vladimir 6 6 1 0 0 0 1 16.7 41.7 0 0 0.167 0.167 0.145 0.315 0.454 0.327 82.8 2.2\nBichette, Bo 4 4 2 1 0 0 1 25 10 0 0 0.5 0.75 0.532 0.375 0.427 0.356 87.6 -4.7\nKirk, Alejandro 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.254 0.734 0.54 95.7 28\nClement, Ernie 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.199 0.2 0.18 93 31.5'}, {'pitcher_id': '669276', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLee, Dylan 3 2 0 0 0 0 1 33.3 16.7 1 33.3 0 0 0.232 0.005 0.011 0.237 79.5 64', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nKirk, Alejandro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.022 0.015 79.5 64\nVarsho, Daulton 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '608331', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFried, Max 48 45 13 2 0 2 7 14.6 16.7 3 6.3 0.289 0.467 0.341 0.243 0.397 0.299 85.9 3.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 26 25 9 1 0 2 4 15.4 17.9 1 3.8 0.36 0.64 0.428 0.267 0.462 0.322 89.3 7.8\nBichette, Bo 6 5 1 0 0 0 0 0 15.4 1 16.7 0.2 0.2 0.262 0.306 0.545 0.423 82.6 -7\nJansen, Danny 4 4 0 0 0 0 2 50 33.3 0 0 0 0 0 0.025 0.052 0.03 69.2 48\nGuerrero Jr., Vladimir 5 5 1 1 0 0 0 0 0 0 0 0.2 0.4 0.248 0.153 0.249 0.175 85.6 -6.4\nVarsho, Daulton 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.344 0.344\nKiner-Falefa, Isiah 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.588 0.51 0.58 0.476 94.5 4.7\nGausman, Kevin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.17 0.188 0.156 62.7 -35.5'}, {'pitcher_id': '670241', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641729', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Joe 19 19 4 1 0 0 7 36.8 26.8 0 0 0.211 0.263 0.205 0.168 0.224 0.171 89.3 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJansen, Danny 3 3 1 1 0 0 1 33.3 20 0 0 0.333 0.667 0.415 0.32 0.395 0.314 86.4 -7.5\nClement, Ernie 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.176 0.178 0.162 91.2 18\nSpringer, George 3 3 1 0 0 0 1 33.3 14.3 0 0 0.333 0.333 0.295 0.175 0.219 0.174 75.5 6.5\nGuerrero Jr., Vladimir 3 3 1 0 0 0 2 66.7 57.1 0 0 0.333 0.333 0.293 0.235 0.281 0.225 116.5 5\nBichette, Bo 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.345 0.211 92.5 13.3\nKiermaier, Kevin 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0 0 0 80 69\nKirk, Alejandro 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nKiner-Falefa, Isiah 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '693821', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nElder, Bryce 18 17 4 3 0 0 3 16.7 16.7 1 5.6 0.235 0.412 0.295 0.388 0.638 0.45 89.6 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 3 2 1 1 0 0 0 0 11.1 1 33.3 0.5 1 0.647 0.75 1.19 0.774 111.8 13\nBichette, Bo 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.593 1.067 0.703 90.9 20.7\nKiner-Falefa, Isiah 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.127 0.143 0.118 95.9 -15\nVarsho, Daulton 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.405 1.264 0.682 104.1 23\nSpringer, George 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.202 0.218 0.182 92 -3\nJansen, Danny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.264 0.264 0.238 71.1 3.5\nKiermaier, Kevin 2 2 2 2 0 0 0 0 0 0 0 1 2 1.244 0.495 0.565 0.461 68.3 -2.5'}, {'pitcher_id': '607481', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBummer, Aaron 23 22 12 0 0 0 6 26.1 39.5 1 4.3 0.545 0.545 0.489 0.256 0.286 0.257 86.6 -17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKiermaier, Kevin 6 6 2 0 0 0 3 50 63.6 0 0 0.333 0.333 0.291 0.156 0.168 0.146 81.1 -23.3\nGuerrero Jr., Vladimir 4 3 1 0 0 0 1 25 50 1 25 0.333 0.333 0.394 0.152 0.179 0.276 91.6 -8.5\nKirk, Alejandro 3 3 2 0 0 0 1 33.3 16.7 0 0 0.667 0.667 0.588 0.506 0.577 0.473 90.3 12.5\nJansen, Danny 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.589 0.319 0.352 0.295 79.3 -35\nTurner, Justin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBichette, Bo 2 2 2 0 0 0 0 0 33.3 0 0 1 1 0.881 0.353 0.391 0.325 98 -7.5\nSpringer, George 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.474 0.53 0.439 102.6 0\nKiner-Falefa, Isiah 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.147 0.168 0.14 101.7 -20\nVarsho, Daulton 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.3 0.302 0.274 44.6 -52\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.146 0.166 0.131 96.2 -19'}, {'pitcher_id': '445926', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChavez, Jesse 53 45 12 2 0 2 15 28.3 23.1 8 15.1 0.267 0.444 0.357 0.291 0.527 0.395 88.5 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 28 24 6 1 0 2 10 35.7 31.7 4 14.3 0.25 0.542 0.377 0.264 0.564 0.389 94.4 19.9\nKiermaier, Kevin 8 8 3 0 0 0 2 25 0 0 0 0.375 0.375 0.329 0.373 0.418 0.342 85.2 2.5\nGuerrero Jr., Vladimir 4 4 0 0 0 0 2 50 14.3 0 0 0 0 0 0.148 0.19 0.138 89.7 -3\nTurner, Justin 5 4 0 0 0 0 1 20 20 1 20 0 0 0.137 0.059 0.064 0.187 72.8 -17.3\nVarsho, Daulton 4 2 0 0 0 0 0 0 16.7 2 50 0 0 0.345 0.482 0.55 0.571 71.5 -0.5\nBichette, Bo 3 2 2 1 0 0 0 0 20 1 33.3 1 1.5 0.932 0.557 1.909 0.882 101.5 3\nJansen, Danny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.833 0.985 0.784 89.2 16'}], 142: [{'pitcher_id': '642216', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '675911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrider, Spencer 21 18 0 0 0 0 7 33.3 47.2 2 9.5 0 0 0.1 0.142 0.19 0.223 81.1 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFarmer, Kyle 6 4 0 0 0 0 3 50 50 1 16.7 0 0 0.235 0.001 0.001 0.235 75.9 71\nCorrea, Carlos 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.193 0.331 0.224 72.7 24.5\nKepler, Max 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.232 0.149 0.213 0.336 94.7 14.5\nCastro, Willi 3 3 0 0 0 0 0 0 50 0 0 0 0 0 0.311 0.354 0.291 76.6 6\nVázquez, Christian 2 2 0 0 0 0 2 100 60 0 0 0 0 0 0\nMargot, Manuel 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.002 0.002 0.002 61.4 76\nJeffers, Ryan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.365 0.466 0.359 95.1 19'}, {'pitcher_id': '519242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSale, Chris 129 118 31 4 0 4 36 27.9 29.1 8 6.2 0.263 0.398 0.312 0.204 0.361 0.276 89 7.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 51 45 14 2 0 2 8 15.7 20 6 11.8 0.311 0.489 0.382 0.218 0.49 0.356 98 2.4\nCorrea, Carlos 33 30 7 0 0 1 8 24.2 24.2 2 6.1 0.233 0.333 0.261 0.236 0.425 0.306 90.4 15.3\nBuxton, Byron 18 18 3 1 0 1 9 50 42.4 0 0 0.167 0.389 0.23 0.166 0.326 0.206 91.9 7.9\nMargot, Manuel 8 8 3 0 0 0 2 25 27.3 0 0 0.375 0.375 0.33 0.256 0.281 0.238 82.6 7.3\nKepler, Max 6 4 1 1 0 0 3 50 33.3 0 0 0.25 0.5 0.448 0.163 0.168 0.339 60.7 29\nVázquez, Christian 6 6 1 0 0 0 2 33.3 45.5 0 0 0.167 0.167 0.147 0.215 0.24 0.2 92.9 9\nMiranda, Jose 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.019 0.019 0.018 95.8 -33\nCastro, Willi 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.17 0.171 0.155 32.8 -44\nJeffers, Ryan 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.44 0.105 0.105 0.095 40.5 -50'}, {'pitcher_id': '450203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorton, Charlie 58 52 9 4 1 3 21 36.2 26.8 6 10.3 0.173 0.462 0.304 0.179 0.344 0.271 88 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 24 22 3 2 0 1 9 37.5 26 2 8.3 0.136 0.364 0.244 0.143 0.256 0.215 86 3.1\nSantana, Carlos 12 11 4 2 0 2 4 33.3 36.4 1 8.3 0.364 1.091 0.607 0.303 0.795 0.478 96.5 19.4\nCorrea, Carlos 7 6 0 0 0 0 5 71.4 28.6 1 14.3 0 0 0.099 0.118 0.127 0.19 96.2 10\nKepler, Max 6 5 0 0 0 0 1 16.7 20 1 16.7 0 0 0.115 0.176 0.244 0.263 82.1 -11.5\nBuxton, Byron 4 3 1 0 1 0 1 25 25 1 25 0.333 1 0.556 0.04 0.053 0.206 83.6 11.5\nMargot, Manuel 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.293 0.327 0.381 0.316 90.1 22.3\nFarmer, Kyle 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.022 0.022 0.022 67.4 -21'}, {'pitcher_id': '678061', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '554431', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatzek, Tyler 4 4 0 0 0 0 1 25 42.9 0 0 0 0 0 0.159 0.19 0.162 83.9 -1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.108 0.122 0.113 90.8 -22\nLópez, Pablo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.42 0.513 0.421 70.1 41\nFarmer, Kyle 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '623474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Jimmy 18 16 3 2 0 0 4 22.2 30.8 1 5.6 0.188 0.313 0.265 0.241 0.325 0.296 83.8 -0.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.134 0.143 0.125 62.6 -46\nCorrea, Carlos 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.523 0.556 0.631 0.58 72.9 -11.5\nMiranda, Jose 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nMargot, Manuel 2 2 1 1 0 0 0 0 50 0 0 0.5 1 0.622 0.376 0.616 0.418 89.7 30.5\nKepler, Max 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.187 0.23 0.169 93.7 -10\nCastro, Willi 2 1 1 1 0 0 0 0 0 0 0 1 2 0.982 0.771 1.293 0.797 110.8 14\nVázquez, Christian 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.206 0.156 78.7 0\nLarnach, Trevor 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFarmer, Kyle 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nJeffers, Ryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.021 0.014 89.9 49\nBuxton, Byron 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.447 0.472 0.408 103.6 -4'}, {'pitcher_id': '572955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJohnson, Pierce 8 7 1 1 0 0 2 25 26.7 1 12.5 0.143 0.286 0.242 0.201 0.277 0.27 79.1 -11.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 2 1 0 0 0 0 1 50 40 1 50 0 0 0.345 0.345\nFarmer, Kyle 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.624 0.223 0.465 0.288 77.5 36.5\nCorrea, Carlos 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.257 0.259 0.245 67.5 -64.5\nSantana, Carlos 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.224 0.245 0.202 105.4 0'}, {'pitcher_id': '621345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMinter, A.J. 17 15 4 0 0 0 4 23.5 37.8 2 11.8 0.267 0.267 0.288 0.325 0.468 0.383 89.9 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 6 5 2 0 0 0 0 0 33.3 1 16.7 0.4 0.4 0.408 0.33 0.606 0.445 89.3 -11.4\nVázquez, Christian 4 4 0 0 0 0 2 50 45.5 0 0 0 0 0 0.14 0.229 0.153 94.5 33\nMargot, Manuel 3 2 0 0 0 0 1 33.3 66.7 1 33.3 0 0 0.23 0.38 0.458 0.473 82.3 21\nCorrea, Carlos 2 2 1 0 0 0 0 0 16.7 0 0 0.5 0.5 0.44 0.518 0.601 0.494 99.8 3.5\nFarmer, Kyle 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nCastro, Willi 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.869 0.96 0.82 71.8 18'}, {'pitcher_id': '678226', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Daysbel 1 0 0 0 0 0 0 0 66.7 1 100 0.696 0.696', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 1 0 0 0 0 0 0 0 66.7 1 100 0.696 0.696'}, {'pitcher_id': '700363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith-Shawver, AJ 3 2 1 0 0 1 0 0 0 1 33.3 0.5 2 0.9 0.45 1.545 0.77 95.2 38', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 3 2 1 0 0 1 0 0 0 1 33.3 0.5 2 0.9 0.45 1.545 0.77 95.2 38'}, {'pitcher_id': '628452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIglesias, Raisel 28 26 7 2 0 2 8 28.6 37.5 1 3.6 0.269 0.577 0.352 0.251 0.449 0.299 86.9 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 7 7 1 1 0 0 4 57.1 53.3 0 0 0.143 0.286 0.179 0.136 0.179 0.136 87.2 -20.3\nCorrea, Carlos 5 4 1 0 0 1 0 0 12.5 0 0 0.25 1 0.401 0.359 0.723 0.363 81.9 -0.4\nKepler, Max 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.293 0.127 0.15 0.115 78.7 32.3\nMargot, Manuel 4 3 1 0 0 1 0 0 25 1 25 0.333 1.333 0.674 0.423 1.371 0.718 96.4 29\nVázquez, Christian 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.442 0.32 0.368 0.306 101.2 5\nLewis, Royce 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nJeffers, Ryan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.343 0.461 0.343 89.2 22.5\nLarnach, Trevor 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBuxton, Byron 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nCastro, Willi 1 1 1 1 0 0 0 0 50 0 0 1 2 1.242 0.597 0.694 0.569 93 7\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.577 0.603 0.519 82.3 12'}, {'pitcher_id': '625643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Reynaldo 101 92 21 4 0 5 15 14.9 23 7 6.9 0.228 0.435 0.299 0.245 0.434 0.317 88.6 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 30 28 4 1 0 2 0 0 9.3 1 3.3 0.143 0.393 0.222 0.258 0.462 0.322 90.1 22.1\nSantana, Carlos 21 20 3 0 0 0 2 9.5 27 1 4.8 0.15 0.15 0.158 0.164 0.272 0.205 85.5 35.8\nCorrea, Carlos 13 11 2 0 0 0 3 23.1 20 1 7.7 0.182 0.182 0.189 0.215 0.308 0.272 87.6 9.8\nCastro, Willi 10 9 2 0 0 0 3 30 30 1 10 0.222 0.222 0.245 0.229 0.261 0.263 80.4 16\nJeffers, Ryan 6 4 1 0 0 0 3 50 38.5 2 33.3 0.25 0.25 0.38 0.046 0.05 0.261 97.1 -14\nBuxton, Byron 7 7 4 1 0 3 0 0 25 0 0 0.571 2 1.026 0.607 1.707 0.937 101.5 6\nLarnach, Trevor 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nMiranda, Jose 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.295 0.247 0.263 0.223 95.8 -1\nFarmer, Kyle 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.023 0.025 0.025 75.4 23.5\nMargot, Manuel 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.442 0.499 0.563 0.478 85.2 -1\nVázquez, Christian 3 3 3 2 0 0 0 0 0 0 0 1 1.667 1.13 0.45 0.572 0.444 84.6 25.3'}, {'pitcher_id': '669276', 'player_data': '', 'team_data': 'Dylan Lee has not faced anyone on this team.'}, {'pitcher_id': '608331', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFried, Max 32 30 8 1 0 0 8 25 33.9 2 6.3 0.267 0.3 0.275 0.308 0.393 0.333 84.5 -0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 8 7 1 0 0 0 2 25 43.8 1 12.5 0.143 0.143 0.197 0.365 0.545 0.429 91.8 3\nFarmer, Kyle 9 9 3 0 0 0 1 11.1 21.4 0 0 0.333 0.333 0.295 0.319 0.389 0.316 82 5.5\nCorrea, Carlos 6 6 2 0 0 0 3 50 54.5 0 0 0.333 0.333 0.293 0.193 0.229 0.188 82.8 13.3\nVázquez, Christian 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.233 0.106 0.106 0.298 93.6 -43.5\nLópez, Pablo 4 4 1 1 0 0 2 50 37.5 0 0 0.25 0.5 0.31 0.256 0.336 0.258 77.5 -32.5\nMargot, Manuel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.706 0.772 0.654 76.8 18'}, {'pitcher_id': '670241', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641729', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Joe 53 46 8 2 0 2 17 32.1 32 5 9.4 0.174 0.348 0.281 0.155 0.287 0.256 89.6 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 12 10 2 0 0 1 2 16.7 15.4 2 16.7 0.2 0.5 0.357 0.159 0.375 0.303 88.2 33.1\nBuxton, Byron 9 7 1 1 0 0 4 44.4 50 0 0 0.143 0.286 0.296 0.078 0.158 0.237 93.7 30.7\nSantana, Carlos 7 5 1 1 0 0 2 28.6 33.3 2 28.6 0.2 0.4 0.371 0.092 0.204 0.289 89.1 17.3\nJeffers, Ryan 6 6 1 0 0 1 4 66.7 38.5 0 0 0.167 0.667 0.334 0.136 0.37 0.207 95.1 13\nLarnach, Trevor 6 6 0 0 0 0 3 50 50 0 0 0 0 0 0.001 0.001 0.001 82.6 51\nCorrea, Carlos 4 4 1 0 0 0 0 0 14.3 0 0 0.25 0.25 0.221 0.486 0.712 0.517 91.1 13.8\nMiranda, Jose 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.181 0.2 0.167 106.5 3\nFarmer, Kyle 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.045 0.132 0.277 95.7 43.5\nVázquez, Christian 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.588 0.385 0.472 0.379 81.7 2.3'}, {'pitcher_id': '693821', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nElder, Bryce 18 16 1 0 0 0 4 22.2 39.3 1 5.6 0.063 0.063 0.128 0.169 0.237 0.231 88.2 0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBuxton, Byron 3 3 0 0 0 0 1 33.3 66.7 0 0 0 0 0 0.056 0.072 0.056 89.2 10\nVázquez, Christian 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.172 0.192 0.335 99.2 -3\nLewis, Royce 3 2 0 0 0 0 1 33.3 60 0 0 0 0 0.242 0.255 0.534 0.457 102.7 20\nCorrea, Carlos 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.294 0.173 0.21 0.167 81.6 19\nKepler, Max 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.231 0.266 0.21 81.3 -6.3\nSantana, Carlos 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.155 0.232 0.163 90.2 -15.3'}, {'pitcher_id': '607481', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBummer, Aaron 64 50 13 1 0 2 16 25 30.8 10 15.6 0.26 0.4 0.357 0.209 0.311 0.319 88.2 -6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 16 12 3 0 0 1 3 18.8 26.1 3 18.8 0.25 0.5 0.365 0.201 0.366 0.334 86.8 -10.4\nKepler, Max 14 10 2 0 0 0 4 28.6 43.5 3 21.4 0.2 0.2 0.294 0.213 0.251 0.316 87.9 3.8\nCorrea, Carlos 7 4 1 0 0 0 1 14.3 10 3 42.9 0.25 0.25 0.421 0.22 0.253 0.415 83.4 2.7\nBuxton, Byron 5 5 1 0 0 1 4 80 54.5 0 0 0.2 0.8 0.414 0.141 0.384 0.22 102.6 23\nMargot, Manuel 6 6 2 0 0 0 0 0 14.3 0 0 0.333 0.333 0.294 0.289 0.384 0.3 94.9 -13.8\nCastro, Willi 5 3 1 1 0 0 0 0 14.3 1 20 0.333 0.667 0.533 0.423 0.584 0.543 101.9 -8\nVázquez, Christian 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.072 0.072 0.067 62 -30\nLarnach, Trevor 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nMiranda, Jose 2 2 0 0 0 0 1 50 14.3 0 0 0 0 0 0.073 0.206 0.117 94.4 29\nFarmer, Kyle 2 1 0 0 0 0 1 50 50 0 0 0 0 0.361 0.361\nJeffers, Ryan 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.45 0.496 0.42 71.3 -6.5\nLewis, Royce 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.187 0.189 0.173 109.7 -21'}, {'pitcher_id': '445926', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChavez, Jesse 45 40 11 4 0 1 14 31.1 23.4 5 11.1 0.275 0.45 0.347 0.271 0.466 0.338 92.2 21.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 14 12 2 1 0 0 5 35.7 18.2 2 14.3 0.167 0.25 0.248 0.116 0.188 0.126 89.6 31\nCorrea, Carlos 14 12 5 2 0 1 5 35.7 30.4 2 14.3 0.417 0.833 0.545 0.395 0.693 0.489 99.4 16.7\nMargot, Manuel 4 3 1 0 0 0 1 25 37.5 1 25 0.333 0.333 0.392 0.204 0.216 0.312 100.1 -11\nVázquez, Christian 3 3 1 1 0 0 1 33.3 25 0 0 0.333 0.667 0.406 0.316 1.136 0.57 95.8 29\nKepler, Max 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.098 0.098 0.088 85.2 50\nBuxton, Byron 3 3 1 0 0 0 1 33.3 20 0 0 0.333 0.333 0.29 0.429 0.5 0.4 87.5 12\nCastro, Willi 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.153 0.175 0.142 70.2 4\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.087 0.042 93.5 39\nMiranda, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.686 0.749 0.635 76.9 15'}], 143: [{'pitcher_id': '642216', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '675911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrider, Spencer 181 168 35 4 0 8 71 39.2 38.1 13 7.2 0.208 0.375 0.279 0.223 0.403 0.296 89.6 17.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStott, Bryson 28 28 6 2 0 0 8 28.6 23.1 0 0 0.214 0.286 0.216 0.182 0.221 0.176 86.6 24.6\nSchwarber, Kyle 32 28 5 1 0 1 14 43.8 50.8 4 12.5 0.179 0.321 0.273 0.243 0.527 0.369 91.7 13.9\nHarper, Bryce 24 20 6 0 0 2 7 29.2 36.7 4 16.7 0.3 0.6 0.43 0.288 0.532 0.405 93.8 15.4\nCastellanos, Nick 26 25 5 0 0 2 10 38.5 49.1 1 3.8 0.2 0.44 0.283 0.235 0.552 0.341 88.1 14.1\nMarsh, Brandon 20 17 6 1 0 2 9 45 39.5 3 15 0.353 0.765 0.503 0.322 0.606 0.438 94.3 18.1\nBohm, Alec 27 27 5 0 0 1 12 44.4 27.1 0 0 0.185 0.296 0.208 0.172 0.261 0.185 86.5 10.2\nRojas, Johan 12 11 1 0 0 0 5 41.7 37.5 1 8.3 0.091 0.091 0.131 0.267 0.333 0.295 93.8 2.3\nStubbs, Garrett 5 5 0 0 0 0 1 20 22.2 0 0 0 0 0 0.061 0.075 0.058 88.9 40.8\nSosa, Edmundo 4 4 0 0 0 0 3 75 54.5 0 0 0 0 0 0.002 0.002 0.001 65.3 77\nMerrifield, Whit 3 3 1 0 0 0 2 66.7 60 0 0 0.333 0.333 0.294 0.222 0.289 0.216 96.3 16'}, {'pitcher_id': '519242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSale, Chris 105 97 22 6 1 1 39 37.1 32.7 5 4.8 0.227 0.34 0.27 0.246 0.364 0.29 87.1 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 46 43 9 4 0 0 19 41.3 39.4 2 4.3 0.209 0.302 0.234 0.253 0.381 0.284 87.1 21.4\nMerrifield, Whit 33 32 8 2 1 0 10 30.3 25.4 1 3 0.25 0.375 0.277 0.251 0.368 0.276 88.6 21.9\nSosa, Edmundo 5 4 1 0 0 0 2 40 23.5 0 0 0.25 0.25 0.323 0.218 0.241 0.31 76.7 42\nBohm, Alec 6 5 2 0 0 0 1 16.7 0 1 16.7 0.4 0.4 0.411 0.368 0.388 0.396 88.7 -9.7\nSchwarber, Kyle 6 6 1 0 0 1 3 50 44.4 0 0 0.167 0.667 0.348 0.247 0.586 0.348 84.4 17.7\nDahl, David 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.21 0.228 0.195 101.7 -1\nPache, Cristian 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nHarper, Bryce 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.294 0.139 0.141 0.127 74.5 -52\nRojas, Johan 2 1 0 0 0 0 1 50 50 0 0 0 0 0.362 0.362'}, {'pitcher_id': '450203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorton, Charlie 165 141 41 4 3 4 49 29.7 30.9 19 11.5 0.291 0.447 0.353 0.257 0.412 0.336 89.6 15.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 36 27 11 1 1 1 9 25 32.5 8 22.2 0.407 0.63 0.486 0.347 0.608 0.464 94.2 11.6\nSchwarber, Kyle 28 21 6 0 0 2 6 21.4 23.1 5 17.9 0.286 0.571 0.423 0.161 0.343 0.34 91.4 24.9\nStott, Bryson 19 17 4 1 0 0 5 26.3 16.7 1 5.3 0.235 0.294 0.241 0.18 0.28 0.227 83.6 12.9\nBohm, Alec 30 28 13 1 0 0 8 26.7 28.3 1 3.3 0.464 0.5 0.418 0.42 0.495 0.398 90.9 16\nCastellanos, Nick 21 18 1 1 0 0 6 28.6 39.1 3 14.3 0.056 0.111 0.157 0.217 0.337 0.299 92.8 16.8\nMarsh, Brandon 11 11 3 0 0 1 7 63.6 48 0 0 0.273 0.545 0.349 0.196 0.43 0.265 97.9 26\nMerrifield, Whit 6 5 2 0 2 0 1 16.7 27.3 1 16.7 0.4 1.2 0.625 0.224 0.411 0.337 93 22.8\nWheeler, Zack 6 6 1 0 0 0 3 50 28.6 0 0 0.167 0.167 0.147 0.17 0.18 0.158 63 -43.7\nStubbs, Garrett 5 5 0 0 0 0 3 60 66.7 0 0 0 0 0 0.03 0.032 0.028 65.4 43.5\nDahl, David 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.235 0.863 0.431 85.1 8'}, {'pitcher_id': '678061', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerr, Ray 8 8 2 0 0 0 3 37.5 28.6 0 0 0.25 0.25 0.221 0.257 0.307 0.249 87.5 28.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.226 0.244 0.206 105.1 0\nHarper, Bryce 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nStubbs, Garrett 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.026 0.019 88.5 63\nCastellanos, Nick 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.368 0.505 0.379 86.7 31\nSosa, Edmundo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.857 0.933 0.803 70.3 19\nMarsh, Brandon 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '554431', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatzek, Tyler 37 31 7 4 1 0 11 29.7 27.4 3 8.1 0.226 0.419 0.312 0.19 0.254 0.257 87.2 18.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 12 11 2 2 0 0 4 33.3 35.7 0 0 0.182 0.364 0.21 0.271 0.343 0.258 88.8 21.8\nBohm, Alec 7 5 1 1 0 0 3 42.9 22.2 1 14.3 0.2 0.4 0.382 0.127 0.198 0.3 93.5 24.5\nSchwarber, Kyle 7 6 1 0 0 0 1 14.3 0 1 14.3 0.167 0.167 0.224 0.143 0.197 0.223 92.4 32.4\nStott, Bryson 5 4 1 0 0 0 1 20 18.2 1 20 0.25 0.25 0.315 0.156 0.24 0.273 86.6 6.7\nCastellanos, Nick 3 3 2 1 1 0 1 33.3 37.5 0 0 0.667 1.667 0.965 0.029 0.034 0.026 82.1 54\nSosa, Edmundo 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.316 0.347 0.289 88.8 10\nNola, Aaron 1 0 0 0 0 0 0 0 0 0 0 43.8 -76'}, {'pitcher_id': '623474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Jimmy 5 5 3 1 0 1 0 0 0 0 0 0.6 1.4 0.832 0.467 0.796 0.54 84.4 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStott, Bryson 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.463 1.277 0.718 99.3 26\nMarsh, Brandon 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.277 0.281 0.262 50.7 -53\nSosa, Edmundo 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.51 0.579 0.482 84.7 20.5\nCastellanos, Nick 1 1 1 1 0 0 0 0 0 0 0 1 2 1.217 0.573 1.265 0.754 102.5 20'}, {'pitcher_id': '572955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJohnson, Pierce 42 35 7 2 0 1 12 28.6 39 7 16.7 0.2 0.343 0.307 0.233 0.398 0.339 88.8 3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStott, Bryson 6 4 1 0 0 0 1 16.7 25 2 33.3 0.25 0.25 0.379 0.318 0.36 0.426 99 -6\nMarsh, Brandon 6 6 1 1 0 0 4 66.7 60 0 0 0.167 0.333 0.207 0.113 0.171 0.121 91.3 6\nCastellanos, Nick 8 7 2 0 0 0 1 12.5 20 1 12.5 0.286 0.286 0.308 0.385 0.607 0.46 81.4 10.3\nSchwarber, Kyle 4 3 0 0 0 0 2 50 46.2 1 25 0 0 0.172 0.11 0.119 0.25 106.3 -11\nHarper, Bryce 4 3 1 0 0 1 1 25 50 1 25 0.333 1.333 0.674 0.382 1.519 0.746 107.9 34.5\nBohm, Alec 6 5 0 0 0 0 2 33.3 44.4 1 16.7 0 0 0.116 0.085 0.093 0.181 91.9 -14\nRojas, Johan 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.153 0.193 0.152 74.5 -14.3\nDahl, David 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.44 0.506 0.561 0.474 88.3 1\nSosa, Edmundo 2 2 1 1 0 0 1 50 60 0 0 0.5 1 0.621 0.06 0.074 0.057 76.2 38\nMerrifield, Whit 1 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '621345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMinter, A.J. 90 77 14 4 0 1 25 27.8 36.6 12 13.3 0.182 0.273 0.267 0.184 0.289 0.272 87.8 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 21 15 1 1 0 0 4 19 34.4 5 23.8 0.067 0.133 0.258 0.137 0.252 0.314 87.5 19.4\nHarper, Bryce 22 17 2 1 0 0 7 31.8 36.2 5 22.7 0.118 0.176 0.255 0.142 0.293 0.301 88 18\nMarsh, Brandon 8 7 1 0 0 0 4 50 50 1 12.5 0.143 0.143 0.197 0.055 0.086 0.14 92.5 34\nBohm, Alec 12 12 4 0 0 0 1 8.3 25 0 0 0.333 0.333 0.294 0.338 0.38 0.315 89.9 1\nCastellanos, Nick 7 7 2 0 0 0 3 42.9 35.3 0 0 0.286 0.286 0.252 0.129 0.147 0.119 90.1 4.3\nStott, Bryson 7 7 2 1 0 1 3 42.9 28.6 0 0 0.286 0.857 0.474 0.213 0.326 0.229 85.4 22\nSosa, Edmundo 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.138 0.15 0.127 60.9 -16.5\nRojas, Johan 3 2 0 0 0 0 0 0 40 1 33.3 0 0 0.232 0.031 0.045 0.253 81.8 20.5\nMerrifield, Whit 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.223 0.264 0.22 86 26.5\nPache, Cristian 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nDahl, David 2 2 2 1 0 0 0 0 50 0 0 1 1.5 1.048 0.79 1.658 0.993 98.3 19\nHoffman, Jeff 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.354 0.631 0.405 99.1 19'}, {'pitcher_id': '678226', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHernández, Daysbel 3 2 0 0 0 0 0 0 33.3 0 0 0 0 0.242 0.205 0.227 0.371 81.2 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.271 0.309 0.255 72.3 9\nSchwarber, Kyle 1 0 0 0 0 0 0 0 50 0 0 0.726 0.726\nRojas, Johan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.138 0.145 0.131 90 -17'}, {'pitcher_id': '700363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith-Shawver, AJ 9 9 2 0 0 2 2 22.2 14.3 0 0 0.222 0.889 0.445 0.28 0.908 0.479 99.1 24.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.619 2.386 1.207 104.1 27.5\nMarsh, Brandon 2 2 1 0 0 1 1 50 33.3 0 0 0.5 2 1.002 0.364 1.26 0.654 102 26\nSchwarber, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.4 0.72 0.449 102.5 18\nStott, Bryson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.003 92.8 51\nRojas, Johan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBohm, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.136 0.136 0.126 98.4 -19\nHarper, Bryce 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.017 0.01 89.8 38'}, {'pitcher_id': '628452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIglesias, Raisel 68 67 21 4 0 5 13 19.1 26.9 1 1.5 0.313 0.597 0.388 0.308 0.637 0.401 89.2 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 16 16 4 1 0 1 6 37.5 33.3 0 0 0.25 0.5 0.31 0.156 0.286 0.187 93.2 22.2\nHarper, Bryce 10 10 7 0 0 4 1 10 24 0 0 0.7 1.9 1.078 0.646 1.96 1.068 101.2 23.1\nStott, Bryson 8 8 3 2 0 0 1 12.5 7.1 0 0 0.375 0.625 0.425 0.252 0.311 0.246 79.5 12.3\nCastellanos, Nick 8 7 0 0 0 0 1 12.5 40 1 12.5 0 0 0.086 0.243 0.732 0.439 82.7 18.3\nMarsh, Brandon 6 6 3 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.481 0.592 0.467 93.4 14.3\nBohm, Alec 6 6 3 0 0 0 1 16.7 16.7 0 0 0.5 0.5 0.442 0.564 0.715 0.558 89.1 18.2\nStubbs, Garrett 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.414 0.16 0.325 0.198 81.7 37.7\nDahl, David 4 4 0 0 0 0 1 25 57.1 0 0 0 0 0 0.08 0.094 0.073 78.3 -5.7\nRojas, Johan 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.01 0.015 0.011 72.9 58\nMerrifield, Whit 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.176 0.147 88.2 36.3\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.38 1.152 0.637 98.3 28'}, {'pitcher_id': '625643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Reynaldo 100 92 26 7 1 2 20 20 24 7 7 0.283 0.446 0.338 0.26 0.411 0.319 87.7 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 46 42 14 1 1 2 8 17.4 17 3 6.5 0.333 0.548 0.399 0.288 0.422 0.338 88.8 13.9\nCastellanos, Nick 33 33 11 6 0 0 5 15.2 30.4 0 0 0.333 0.515 0.357 0.268 0.441 0.302 88.6 22.4\nSchwarber, Kyle 6 6 0 0 0 0 3 50 46.2 0 0 0 0 0 0.108 0.118 0.103 75 -24.7\nMarsh, Brandon 6 5 1 0 0 0 3 50 28.6 1 16.7 0.2 0.2 0.263 0.192 0.239 0.273 90.2 -5\nHarper, Bryce 4 1 0 0 0 0 1 25 100 3 75 0 0 0.519 0.519\nPache, Cristian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.052 0.054 65.8 -14\nStott, Bryson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.546 1.571 0.877 98.8 23\nBohm, Alec 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.325 0.168 94.4 33\nRojas, Johan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.129 0.116 51.9 39'}, {'pitcher_id': '669276', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLee, Dylan 35 33 5 1 0 2 6 17.1 31.1 2 5.7 0.152 0.364 0.245 0.23 0.524 0.341 89.1 18.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 13 12 2 0 0 1 3 23.1 34.8 1 7.7 0.167 0.417 0.281 0.201 0.527 0.336 93.5 18.9\nHarper, Bryce 9 9 2 1 0 1 2 22.2 14.3 0 0 0.222 0.667 0.37 0.319 0.838 0.49 94.1 2.4\nBohm, Alec 5 4 0 0 0 0 0 0 33.3 1 20 0 0 0.138 0.213 0.271 0.309 84.7 34\nMarsh, Brandon 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.037 0.045 0.034 84.1 46.5\nStott, Bryson 2 2 0 0 0 0 0 0 40 0 0 0 0 0 0.041 0.045 0.039 73.6 9.5\nCastellanos, Nick 3 3 1 0 0 0 0 0 40 0 0 0.333 0.333 0.295 0.429 0.716 0.49 83.7 23.7\nSosa, Edmundo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '608331', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFried, Max 160 136 35 7 0 4 42 26.3 27.7 19 11.9 0.257 0.397 0.328 0.257 0.408 0.334 85.6 2.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 31 26 4 0 0 1 10 32.3 20 5 16.1 0.154 0.269 0.264 0.261 0.449 0.365 92.3 8.9\nHarper, Bryce 40 33 10 2 0 2 11 27.5 43.1 6 15 0.303 0.545 0.405 0.302 0.618 0.435 89.8 -10.7\nBohm, Alec 28 23 6 2 0 0 5 17.9 23.2 3 10.7 0.261 0.348 0.29 0.285 0.334 0.302 82.9 13\nCastellanos, Nick 16 14 9 1 0 1 1 6.3 28.6 2 12.5 0.643 0.929 0.677 0.382 0.589 0.456 81.3 11.9\nStott, Bryson 11 10 2 1 0 0 3 27.3 17.6 1 9.1 0.2 0.3 0.258 0.222 0.246 0.251 85.6 -3.4\nSosa, Edmundo 10 9 2 1 0 0 1 10 13.6 1 10 0.222 0.333 0.283 0.201 0.229 0.238 81.3 4.3\nNola, Aaron 5 4 0 0 0 0 2 40 14.3 0 0 0 0 0 0.1 0.1 0.097 52.5 -48\nPache, Cristian 4 4 0 0 0 0 2 50 62.5 0 0 0 0 0 0.061 0.131 0.074 100.7 1.5\nRojas, Johan 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.395 0.421 0.522 0.482 79.8 25.3\nWheeler, Zack 6 5 0 0 0 0 3 50 22.2 0 0 0 0 0.12 0.025 0.025 0.142 86.6 -23\nDahl, David 3 3 1 0 0 0 2 66.7 57.1 0 0 0.333 0.333 0.29 0.049 0.055 0.049 77.2 -4\nWalker, Taijuan 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSuárez, Ranger 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '670241', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641729', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Joe 45 40 16 5 0 3 11 24.4 26.6 4 8.9 0.4 0.75 0.489 0.292 0.576 0.392 89.7 19.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 15 13 6 3 0 0 3 20 17.6 1 6.7 0.462 0.692 0.469 0.334 0.578 0.407 83.4 19.8\nHarper, Bryce 7 7 1 0 0 1 4 57.1 71.4 0 0 0.143 0.571 0.286 0.153 0.574 0.292 96.3 40.3\nStott, Bryson 4 3 1 0 0 1 1 25 7.7 1 25 0.333 1.333 0.674 0.21 0.747 0.462 89.5 44\nBohm, Alec 5 3 1 0 0 0 0 0 37.5 2 40 0.333 0.333 0.456 0.329 0.398 0.468 92.6 -9.3\nCastellanos, Nick 5 5 2 0 0 0 0 0 0 0 0 0.4 0.4 0.353 0.375 0.587 0.403 95.3 15.8\nSchwarber, Kyle 4 4 2 0 0 1 1 25 14.3 0 0 0.5 1.25 0.715 0.375 0.876 0.516 88 26.3\nDahl, David 3 3 2 2 0 0 1 33.3 50 0 0 0.667 1.333 0.828 0.27 0.379 0.273 93 13.5\nPache, Cristian 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMarsh, Brandon 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.46 0.496 0.417 102.7 -1'}, {'pitcher_id': '693821', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nElder, Bryce 44 39 10 0 0 4 7 15.9 27.3 5 11.4 0.256 0.564 0.382 0.269 0.506 0.369 91.3 12.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStott, Bryson 7 5 0 0 0 0 1 14.3 33.3 2 28.6 0 0 0.199 0.207 0.35 0.368 95.5 15.3\nSchwarber, Kyle 7 6 0 0 0 0 2 28.6 35.7 1 14.3 0 0 0.099 0.061 0.07 0.151 81.8 -18\nHarper, Bryce 7 6 1 0 0 1 1 14.3 9.1 1 14.3 0.167 0.667 0.386 0.316 0.78 0.49 98.3 11.2\nMarsh, Brandon 5 4 3 0 0 0 0 0 33.3 1 20 0.75 0.75 0.669 0.507 0.635 0.536 96.6 7.8\nBohm, Alec 7 7 3 0 0 0 1 14.3 36.4 0 0 0.429 0.429 0.378 0.47 0.545 0.446 93.2 1\nCastellanos, Nick 6 6 3 0 0 3 2 33.3 55.6 0 0 0.5 2 1.002 0.282 1.042 0.534 92.8 18.3\nMerrifield, Whit 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.072 0.105 0.07 76.1 23.5\nStubbs, Garrett 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.01 0.022 0.013 88.4 54.5\nRojas, Johan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.005 0.004 74.6 77'}, {'pitcher_id': '607481', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBummer, Aaron 26 26 6 1 0 1 8 30.8 36.7 0 0 0.231 0.385 0.26 0.275 0.429 0.302 87.8 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 9 9 1 0 0 0 3 33.3 31.6 0 0 0.111 0.111 0.098 0.178 0.196 0.163 80.7 1\nCastellanos, Nick 4 4 1 0 0 0 0 0 25 0 0 0.25 0.25 0.222 0.174 0.205 0.169 90.7 -12\nSchwarber, Kyle 4 4 2 0 0 1 1 25 50 0 0 0.5 1.25 0.717 0.541 1.346 0.784 100 25.3\nMarsh, Brandon 3 3 1 1 0 0 2 66.7 75 0 0 0.333 0.667 0.415 0.258 0.303 0.244 100.1 10\nBohm, Alec 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.555 0.66 0.532 81.8 0.3\nStott, Bryson 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nHarper, Bryce 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRojas, Johan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.265 0.295 0.239 87.9 -3'}, {'pitcher_id': '445926', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChavez, Jesse 77 71 22 3 0 1 13 16.9 19 4 5.2 0.31 0.394 0.317 0.305 0.489 0.348 90.7 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 25 23 8 1 0 0 3 12 17.1 1 4 0.348 0.391 0.325 0.386 0.613 0.415 95.1 18.1\nMerrifield, Whit 10 9 2 0 0 0 1 10 15 1 10 0.222 0.222 0.244 0.167 0.185 0.209 76.4 3.9\nHarper, Bryce 10 10 6 1 0 1 1 10 26.7 0 0 0.6 1 0.676 0.356 0.918 0.53 94.1 21.2\nStott, Bryson 7 7 2 0 0 0 1 14.3 12.5 0 0 0.286 0.286 0.253 0.424 0.492 0.403 91.6 2.8\nBohm, Alec 7 7 2 0 0 0 3 42.9 27.3 0 0 0.286 0.286 0.252 0.289 0.351 0.28 88.9 3\nDahl, David 3 3 1 0 0 0 0 0 11.1 0 0 0.333 0.333 0.293 0.275 0.395 0.279 95.5 31.7\nSchwarber, Kyle 5 4 0 0 0 0 1 20 14.3 1 20 0 0 0.138 0.209 0.223 0.29 86 -26.3\nMarsh, Brandon 4 3 1 1 0 0 2 50 33.3 1 25 0.333 0.667 0.489 0.14 0.412 0.348 97.7 27\nSosa, Edmundo 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.162 0.198 0.081 92.6 14\nRojas, Johan 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.139 0.157 0.125 87.3 -3\nStubbs, Garrett 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.299 0.339 0.281 92.5 -4.5'}], 145: [{'pitcher_id': '642216', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '675911', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStrider, Spencer 23 22 4 2 0 0 11 47.8 45.1 1 4.3 0.182 0.273 0.215 0.182 0.212 0.198 82.3 20.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 6 6 0 0 0 0 5 83.3 56.3 0 0 0 0 0 0.033 0.033 0.031 106.2 -20\nMendick, Danny 3 2 0 0 0 0 0 0 40 1 33.3 0 0 0.232 0.044 0.071 0.266 87.8 5.5\nBenintendi, Andrew 3 3 2 1 0 0 1 33.3 33.3 0 0 0.667 1 0.709 0.553 0.642 0.536 76.4 25.5\nColás, Oscar 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.417 0.534 0.416 100.8 10\nVaughn, Andrew 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.161 0.18 0.149 98.2 40.5\nDeJong, Paul 3 3 0 0 0 0 2 66.7 80 0 0 0 0 0 0.089 0.09 0.08 30.9 6\nRobert Jr., Luis 3 3 1 1 0 0 1 33.3 40 0 0 0.333 0.667 0.415 0.161 0.174 0.145 71.2 44.5'}, {'pitcher_id': '519242', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSale, Chris 38 34 5 1 0 1 18 47.4 33.8 2 5.3 0.147 0.265 0.228 0.099 0.163 0.179 89.8 15.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 20 18 3 0 0 1 11 55 56.3 0 0 0.167 0.333 0.26 0.061 0.134 0.158 87.6 25.2\nPham, Tommy 6 5 2 1 0 0 2 33.3 5.9 1 16.7 0.4 0.6 0.463 0.207 0.278 0.284 95.8 14.3\nRobert Jr., Luis 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.155 0.295 0.182 97.9 20\nVaughn, Andrew 2 1 0 0 0 0 1 50 25 1 50 0 0 0.348 0.348\nLee, Korey 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.28 0.391 0.286 90.1 7.5\nDeJong, Paul 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.001 0.001 0.001 88 44\nLopez, Nicky 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.084 0.091 0.083 82.6 -13'}, {'pitcher_id': '450203', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMorton, Charlie 99 86 18 4 0 4 28 28.3 29 10 10.1 0.209 0.395 0.313 0.181 0.395 0.297 89.4 12.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 33 31 7 2 0 2 7 21.2 23.3 1 3 0.226 0.484 0.318 0.242 0.584 0.364 87.3 11\nMaldonado, Martín 27 20 8 2 0 0 5 18.5 27.9 6 22.2 0.4 0.5 0.47 0.19 0.258 0.323 94.8 8.2\nPham, Tommy 10 9 2 0 0 2 5 50 30.4 1 10 0.222 0.889 0.476 0.22 0.767 0.434 99.4 7\nVaughn, Andrew 5 4 1 0 0 0 0 0 22.2 1 20 0.25 0.25 0.315 0.335 0.424 0.401 86.8 25.3\nMendick, Danny 4 4 0 0 0 0 2 50 36.4 0 0 0 0 0 0.032 0.049 0.03 91.3 16.5\nLopez, Nicky 5 4 0 0 0 0 2 40 42.9 1 20 0 0 0.138 0.074 0.153 0.216 85.8 -5.5\nRobert Jr., Luis 6 6 0 0 0 0 3 50 37.5 0 0 0 0 0 0.063 0.076 0.06 76.6 11.3\nDeJong, Paul 5 5 0 0 0 0 2 40 62.5 0 0 0 0 0 0.058 0.074 0.056 91 29\nColás, Oscar 2 2 0 0 0 0 2 100 28.6 0 0 0 0 0 0\nSheets, Gavin 2 1 0 0 0 0 0 0 0 0 0 0 0 0.362 0.003 0.009 0.364 92.8 43'}, {'pitcher_id': '678061', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKerr, Ray 7 6 1 0 0 1 3 42.9 44.4 1 14.3 0.167 0.667 0.386 0.134 0.48 0.308 95.3 38.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSosa, Lenyn 2 1 1 0 0 1 0 0 0 1 50 1 4 1.35 0.606 2.207 0.906 100.9 29\nDeJong, Paul 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.007 0.012 0.009 87.4 51\nVaughn, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSheets, Gavin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.186 0.652 0.327 97.6 36'}, {'pitcher_id': '554431', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMatzek, Tyler 8 8 0 0 0 0 6 75 52.4 0 0 0 0 0 0.051 0.055 0.046 88.1 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 3 3 0 0 0 0 2 66.7 60 0 0 0 0 0 0.099 0.106 0.09 100.1 -8\nDeJong, Paul 2 2 0 0 0 0 2 100 37.5 0 0 0 0 0 0\nBenintendi, Andrew 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nRobert Jr., Luis 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.114 0.123 0.103 76 38'}, {'pitcher_id': '623474', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Jimmy 12 11 2 0 0 1 3 25 38.7 1 8.3 0.182 0.455 0.296 0.2 0.417 0.292 82.3 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 5 5 1 0 0 1 2 40 50 0 0 0.2 0.8 0.396 0.219 0.672 0.36 92.5 31\nRobert Jr., Luis 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.44 0.443 0.495 0.399 80.1 8\nVaughn, Andrew 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.203 0.203 0.19 55.2 -59\nBenintendi, Andrew 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nPham, Tommy 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nJulks, Corey 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.014 0.031 0.02 90.5 39\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 74.8 73'}, {'pitcher_id': '572955', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJohnson, Pierce 12 10 4 1 0 0 3 25 38.1 1 8.3 0.4 0.5 0.445 0.182 0.256 0.274 84.8 32.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 4 3 1 0 0 0 1 25 33.3 0 0 0.333 0.333 0.402 0.034 0.086 0.22 78.5 54\nSheets, Gavin 1 0 0 0 0 0 0 0 33.3 1 100 0.692 0.692\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.02 0.023 0.018 74.9 62\nJulks, Corey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.749 0.904 0.712 106.4 9\nVaughn, Andrew 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.411 0.492 0.394 69.4 43\nMendick, Danny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRobert Jr., Luis 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.434 0.751 0.487 102.3 18\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.132 0.1 83.7 -10'}, {'pitcher_id': '621345', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMinter, A.J. 15 12 4 0 0 1 2 13.3 16.7 3 20 0.333 0.583 0.453 0.125 0.154 0.235 83.1 26.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 4 2 1 0 0 0 1 25 20 2 50 0.5 0.5 0.566 0.057 0.068 0.372 85.1 -13\nMaldonado, Martín 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.231 0.03 0.031 0.249 62.4 -10\nDeJong, Paul 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.695 0.021 0.078 0.04 87.7 63.7\nBenintendi, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.384 0.421 0.35 86.8 30\nJulks, Corey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.371 0.381 0.334 77.5 10\nLopez, Nicky 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.066 0.098 0.066 84.6 14'}, {'pitcher_id': '678226', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '700363', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '628452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nIglesias, Raisel 42 35 3 0 0 0 16 38.1 34.4 7 16.7 0.086 0.086 0.178 0.1 0.124 0.199 85.7 17.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 13 10 0 0 0 0 6 46.2 39.3 3 23.1 0 0 0.16 0.086 0.098 0.234 86.7 31.7\nPham, Tommy 11 8 2 0 0 0 3 27.3 17.6 3 27.3 0.25 0.25 0.348 0.193 0.24 0.327 89.5 0\nMaldonado, Martín 5 5 0 0 0 0 3 60 53.8 0 0 0 0 0 0.054 0.094 0.061 87.4 1\nRobert Jr., Luis 7 7 1 0 0 0 2 28.6 46.2 0 0 0.143 0.143 0.126 0.053 0.059 0.051 81.2 32\nBenintendi, Andrew 2 1 0 0 0 0 1 50 9.1 1 50 0 0 0.346 0.346\nSheets, Gavin 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.182 0.211 0.164 88.3 3\nLopez, Nicky 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.083 0.096 0.076 81.7 -16\nVaughn, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.004 0.003 84 65'}, {'pitcher_id': '625643', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLópez, Reynaldo 63 57 14 2 1 2 9 14.3 16.8 5 7.9 0.246 0.421 0.311 0.26 0.467 0.337 83.9 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 19 16 4 1 1 0 1 5.3 7.9 2 10.5 0.25 0.438 0.329 0.224 0.314 0.281 83.1 -2.6\nBenintendi, Andrew 13 12 3 1 0 0 2 15.4 21.7 1 7.7 0.25 0.333 0.284 0.255 0.518 0.351 81.1 15.7\nMaldonado, Martín 13 13 3 0 0 0 2 15.4 11.8 0 0 0.231 0.231 0.201 0.282 0.413 0.295 83.3 28.3\nPham, Tommy 7 6 3 0 0 2 1 14.3 20 1 14.3 0.5 1.5 0.778 0.272 0.879 0.492 87.4 7.8\nSheets, Gavin 3 2 1 0 0 0 1 33.3 25 1 33.3 0.5 0.5 0.527 0.358 0.388 0.448 68.3 37\nVaughn, Andrew 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.448 0.507 0.423 94 -18\nRobert Jr., Luis 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.197 0.743 0.38 94.1 42.5\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.239 0.241 0.217 76 6\nDeJong, Paul 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '669276', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLee, Dylan 7 5 1 0 0 1 2 28.6 15 2 28.6 0.2 0.8 0.494 0.232 0.77 0.498 83.4 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 2 1 0 0 0 0 1 50 12.5 1 50 0 0 0.344 0.344\nPham, Tommy 2 1 1 0 0 1 0 0 0 1 50 1 4 1.384 0.909 3.577 1.28 105.6 28\nLopez, Nicky 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.233 0.234 0.213 48.2 -41\nSheets, Gavin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.04 0.026 96.3 52'}, {'pitcher_id': '608331', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFried, Max 31 28 5 1 0 0 10 32.3 26.8 2 6.5 0.179 0.214 0.204 0.217 0.277 0.248 84.4 -6.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 14 12 2 0 0 0 3 21.4 28 2 14.3 0.167 0.167 0.225 0.298 0.343 0.345 89.7 -3.9\nMaldonado, Martín 6 6 2 0 0 0 3 50 20 0 0 0.333 0.333 0.293 0.185 0.236 0.183 71.7 3.7\nDeJong, Paul 8 8 1 1 0 0 2 25 30.8 0 0 0.125 0.25 0.152 0.172 0.279 0.19 89.2 -2.2\nFedde, Erick 2 1 0 0 0 0 1 50 50 0 0 0 0 0 0 0 0 46 -86\nBrebbia, John 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '670241', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '641729', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJiménez, Joe 30 28 8 2 0 0 10 33.3 28.8 2 6.7 0.286 0.357 0.306 0.256 0.388 0.302 94.8 21.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 7 6 1 0 0 0 0 0 6.3 1 14.3 0.167 0.167 0.225 0.233 0.343 0.306 92.8 32.3\nMaldonado, Martín 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.118 0.262 0.158 91.5 33.3\nBenintendi, Andrew 6 5 4 1 0 0 1 16.7 16.7 1 16.7 0.8 1 0.764 0.556 0.679 0.56 90.8 16.3\nMendick, Danny 4 4 0 0 0 0 4 100 71.4 0 0 0 0 0 0\nPham, Tommy 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.179 0.202 0.164 109.6 -11\nVaughn, Andrew 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.095 0.095 0.087 96.4 -15\nRobert Jr., Luis 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.377 0.522 0.389 101.5 14\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.403 1.349 0.718 99.4 30\nColás, Oscar 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.694 1.122 0.776 105.4 15'}, {'pitcher_id': '693821', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nElder, Bryce 5 4 1 0 0 0 2 40 27.3 1 20 0.25 0.25 0.316 0.331 0.506 0.426 98.5 16.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJulks, Corey 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.447 0.576 0.448 96.1 14\nLopez, Nicky 3 2 0 0 0 0 1 33.3 20 1 33.3 0 0 0.232 0.214 0.436 0.411 100.8 19'}, {'pitcher_id': '607481', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBummer, Aaron 22 19 2 0 0 0 6 27.3 33.3 3 13.6 0.105 0.105 0.174 0.133 0.169 0.211 83.3 2.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 13 11 2 0 0 0 3 23.1 30.4 2 15.4 0.182 0.182 0.242 0.153 0.206 0.24 82 12.5\nLopez, Nicky 4 3 0 0 0 0 2 50 75 1 25 0 0 0.173 0.03 0.031 0.194 56.8 -11\nPham, Tommy 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.032 0.036 0.033 86.8 -30\nMaldonado, Martín 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.128 0.144 0.132 91.2 -12\nDeJong, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.493 0.412 101.7 -4'}, {'pitcher_id': '445926', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChavez, Jesse 29 25 9 0 0 1 6 20.7 16.7 4 13.8 0.36 0.48 0.404 0.241 0.385 0.314 88.4 12.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 10 8 1 0 0 1 3 30 26.3 2 20 0.125 0.5 0.333 0.149 0.476 0.337 93.6 16.6\nMaldonado, Martín 10 9 3 0 0 0 1 10 11.1 1 10 0.333 0.333 0.333 0.283 0.334 0.269 86.3 15.1\nDeJong, Paul 4 4 2 0 0 0 1 25 9.1 0 0 0.5 0.5 0.437 0.39 0.458 0.371 95.9 0.3\nLopez, Nicky 2 2 2 0 0 0 0 0 16.7 0 0 1 1 0.87 0.192 0.209 0.172 71.3 12\nBenintendi, Andrew 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.44 0.21 0.23 0.189 89.8 6\nJulks, Corey 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696'}]}, 145: {141: [{'pitcher_id': '621051', 'player_data': '', 'team_data': ''}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 24 23 6 1 0 1 7 29.2 45.6 1 4.2 0.261 0.435 0.31 0.228 0.359 0.268 80.4 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 8 8 4 1 0 1 1 12.5 23.5 0 0 0.5 1 0.621 0.419 0.758 0.495 88.6 15.3\nKiermaier, Kevin 4 4 0 0 0 0 2 50 63.6 0 0 0 0 0 0.003 0.003 0.002 59.6 70\nKiner-Falefa, Isiah 4 4 0 0 0 0 2 50 40 0 0 0 0 0 0.202 0.254 0.193 87.7 34.5\nGuerrero Jr., Vladimir 3 2 1 0 0 0 1 33.3 50 1 33.3 0.5 0.5 0.526 0.28 0.325 0.406 96 4\nTurner, Justin 2 2 0 0 0 0 1 50 83.3 0 0 0 0 0 0.059 0.059 0.062 52 -48\nBichette, Bo 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.05 0.051 0.045 70 -15\nSchneider, Davis 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.144 0.144 0.13 49.9 -76\nKirk, Alejandro 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.206 0.211 0.185 102.9 -11'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 26 25 2 1 0 0 5 19.2 27.5 1 3.8 0.08 0.12 0.11 0.175 0.294 0.218 84.9 7.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 3 3 1 1 0 0 1 33.3 18.2 0 0 0.333 0.667 0.423 0.065 0.147 0.09 83.5 12.5\nJansen, Danny 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.231 0.327 0.381 0.438 106.7 8\nSchneider, Davis 3 3 0 0 0 0 1 33.3 42.9 0 0 0 0 0 0.023 0.028 0.024 85.4 14.5\nKiermaier, Kevin 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.183 0.286 0.2 78.6 18.3\nKiner-Falefa, Isiah 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.318 0.558 0.365 87.6 -3\nGuerrero Jr., Vladimir 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.162 0.202 0.152 87.9 8.3\nSpringer, George 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.075 0.075 0.068 60.3 -83\nVarsho, Daulton 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.001 0.001 0 67.8 71\nClement, Ernie 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.408 0.893 0.542 96.2 8.5\nBichette, Bo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.247 0.497 0.32 85.9 4.5'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 31 30 7 1 0 0 6 19.4 32.7 1 3.2 0.233 0.267 0.234 0.233 0.306 0.248 89.4 27.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 14 13 4 0 0 0 2 14.3 18.2 1 7.1 0.308 0.308 0.301 0.252 0.353 0.291 90 24.7\nSpringer, George 5 5 0 0 0 0 3 60 72.7 0 0 0 0 0 0.23 0.248 0.208 103.2 -1\nKiermaier, Kevin 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.219 0.072 0.09 0.069 78.4 36.7\nKiner-Falefa, Isiah 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.194 0.227 0.183 83.6 22.3\nSchneider, Davis 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.27 0.934 1.204 0.921 103.4 13\nVarsho, Daulton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 86.8 74\nBichette, Bo 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.023 0.045 0.031 91 56\nGuerrero Jr., Vladimir 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.686 0.808 0.647 105.4 8\nKirk, Alejandro 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.07 0.04 90.8 37'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 42 38 10 1 0 1 9 21.4 19.1 3 7.1 0.263 0.368 0.313 0.296 0.472 0.364 90 16.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 8 8 1 1 0 0 3 37.5 22.7 0 0 0.125 0.25 0.155 0.239 0.417 0.276 92.1 10\nBichette, Bo 7 5 3 0 0 0 1 14.3 14.3 2 28.6 0.6 0.6 0.575 0.415 0.459 0.472 94.3 4.3\nKiner-Falefa, Isiah 7 7 4 0 0 0 0 0 16.7 0 0 0.571 0.571 0.503 0.446 0.51 0.423 83.6 21.7\nJansen, Danny 5 5 1 0 0 1 2 40 11.1 0 0 0.2 0.8 0.414 0.287 0.863 0.478 98 26\nVarsho, Daulton 4 3 0 0 0 0 1 25 22.2 1 25 0 0 0.174 0.012 0.021 0.184 83.4 52.5\nKirk, Alejandro 5 5 1 0 0 0 0 0 16.7 0 0 0.2 0.2 0.176 0.418 0.646 0.454 88.7 -2\nTurner, Justin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.01 0.011 0.009 75.5 62\nSchneider, Davis 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.087 0.344 0.168 98.3 36\nSpringer, George 2 1 0 0 0 0 0 0 40 0 0 0 0 0.363 0.397 0.4 0.542 108.3 -4'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 104 92 27 3 1 7 18 17.3 22.6 10 9.6 0.293 0.576 0.4 0.264 0.518 0.368 90.7 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 18 16 4 0 0 1 1 5.6 12.5 2 11.1 0.25 0.438 0.334 0.328 0.692 0.454 93.1 18.9\nBichette, Bo 13 13 4 1 0 1 2 15.4 14.3 0 0 0.308 0.615 0.391 0.307 0.419 0.318 92.2 -0.7\nGuerrero Jr., Vladimir 16 12 5 1 0 0 2 12.5 38.1 3 18.8 0.417 0.5 0.473 0.235 0.327 0.359 95.1 4.7\nVarsho, Daulton 12 11 2 0 0 1 4 33.3 36 1 8.3 0.182 0.455 0.299 0.124 0.373 0.246 84.5 43.6\nKiner-Falefa, Isiah 15 14 5 0 0 1 3 20 20.8 1 6.7 0.357 0.571 0.415 0.261 0.419 0.318 89.8 -2.7\nKiermaier, Kevin 7 5 2 1 1 0 2 28.6 17.6 1 14.3 0.4 1 0.584 0.39 0.57 0.457 82.5 -8\nSchneider, Davis 6 5 0 0 0 0 1 16.7 40 1 16.7 0 0 0.115 0.016 0.019 0.13 80.2 40.8\nJansen, Danny 6 6 2 0 0 1 2 33.3 12.5 0 0 0.333 0.833 0.481 0.317 0.87 0.488 86.8 0.8\nTurner, Justin 4 3 1 0 0 1 0 0 25 1 25 0.333 1.333 0.681 0.44 1.336 0.719 99.5 19.7\nKirk, Alejandro 5 5 1 0 0 0 1 20 14.3 0 0 0.2 0.2 0.178 0.206 0.398 0.254 93.4 15.3\nClement, Ernie 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.441 1.528 0.793 95 53'}, {'pitcher_id': '686563', 'player_data': '', 'team_data': 'Jonathan Cannon has not faced anyone on this team.'}, {'pitcher_id': '672860', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerroa, Prelander 3 2 0 0 0 0 1 33.3 75 1 33.3 0 0 0.232 0.05 0.076 0.267 77 40', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nBichette, Bo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.152 0.106 77 40'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 33 29 12 2 0 0 7 21.2 17.6 4 12.1 0.414 0.483 0.428 0.213 0.244 0.26 80.4 -2.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVarsho, Daulton 9 9 4 1 0 0 2 22.2 16.7 0 0 0.444 0.556 0.433 0.239 0.286 0.23 69.9 3.9\nTurner, Justin 7 6 2 0 0 0 2 28.6 25 1 14.3 0.333 0.333 0.352 0.183 0.214 0.251 83.1 -9.5\nKiner-Falefa, Isiah 3 3 1 1 0 0 1 33.3 12.5 0 0 0.333 0.667 0.416 0.046 0.046 0.041 81.9 -6\nSpringer, George 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.151 0.161 0.324 77.5 -10\nGuerrero Jr., Vladimir 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.23 0.011 0.014 0.237 96.1 57\nJansen, Danny 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.433 0.487 0.405 96.3 2.5\nKiermaier, Kevin 3 2 0 0 0 0 0 0 25 1 33.3 0 0 0.231 0.197 0.205 0.348 65.7 -43\nSchneider, Davis 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.444 0.294 0.329 0.274 101 5\nBichette, Bo 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.606 0.673 0.552 107 5'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 20 18 2 0 0 1 1 5 8.6 2 10 0.111 0.278 0.218 0.237 0.351 0.296 86.5 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 6 5 0 0 0 0 1 16.7 16.7 1 16.7 0 0 0.115 0.229 0.388 0.331 89.2 25.5\nSchneider, Davis 2 1 1 0 0 1 0 0 20 1 50 1 4 1.389 0.171 0.598 0.507 93.6 32\nSpringer, George 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.432 0.554 0.424 88 35\nKiermaier, Kevin 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.16 0.122 75.7 15.3\nJansen, Danny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.085 0.102 0.078 79.6 49\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.563 0.625 0.526 109.7 0\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.117 0.243 0.161 84.1 27\nVarsho, Daulton 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.289 0.332 0.27 89.6 0\nBichette, Bo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.163 0.243 0.165 87.3 24'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 60 52 15 6 0 2 6 10 17.1 8 13.3 0.288 0.519 0.391 0.268 0.421 0.348 85.2 14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 10 7 2 0 0 0 1 10 38.1 3 30 0.286 0.286 0.384 0.207 0.242 0.348 92.5 -9.8\nBichette, Bo 11 11 6 4 0 0 2 18.2 18.2 0 0 0.545 0.909 0.617 0.331 0.468 0.342 91.3 15.3\nVarsho, Daulton 9 7 3 0 0 2 1 11.1 9.5 2 22.2 0.429 1.286 0.713 0.499 1.129 0.689 84.4 13.2\nJansen, Danny 7 5 1 1 0 0 0 0 8.3 2 28.6 0.2 0.4 0.381 0.153 0.313 0.339 88.4 19.2\nSpringer, George 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0.206 0.261 0.196 76.8 15.5\nTurner, Justin 6 6 1 0 0 0 0 0 10 0 0 0.167 0.167 0.147 0.295 0.348 0.281 87.4 16.2\nGausman, Kevin 3 2 1 0 0 0 1 33.3 16.7 1 33.3 0.5 0.5 0.524 0.203 0.22 0.354 79.3 10\nSchneider, Davis 3 3 0 0 0 0 0 0 11.1 0 0 0 0 0 0.094 0.156 0.106 83.6 30.7\nKiner-Falefa, Isiah 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.021 0.021 0.019 45.9 50\nKiermaier, Kevin 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.4 0.44 0.365 84.4 2\nRichards, Trevor 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.08 0.09 0.075 64.6 53'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 64 57 18 3 1 3 14 21.9 21 6 9.4 0.316 0.561 0.405 0.235 0.387 0.313 87.3 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSpringer, George 16 13 6 1 0 2 5 31.3 34.8 3 18.8 0.462 1 0.626 0.195 0.375 0.326 91 -1.4\nKiner-Falefa, Isiah 11 10 4 1 0 0 1 9.1 13.3 1 9.1 0.4 0.5 0.417 0.457 0.66 0.499 88.1 22.6\nVarsho, Daulton 11 11 0 0 0 0 2 18.2 15.8 0 0 0 0 0 0.044 0.047 0.044 75.6 18.6\nTurner, Justin 10 8 1 0 0 0 2 20 20 2 20 0.125 0.125 0.226 0.205 0.343 0.326 85.1 16\nGuerrero Jr., Vladimir 6 6 3 1 0 0 1 16.7 14.3 0 0 0.5 0.667 0.497 0.409 0.733 0.481 94.5 21.2\nJansen, Danny 5 4 2 0 0 1 2 40 33.3 0 0 0.5 1.25 0.721 0.209 0.389 0.342 92.6 17\nBichette, Bo 3 3 1 0 0 0 1 33.3 16.7 0 0 0.333 0.333 0.294 0.128 0.141 0.12 95.9 10.5\nKiermaier, Kevin 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.784 0.248 0.474 0.299 95.3 25'}, {'pitcher_id': '694363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShuster, Jared 10 8 0 0 0 0 1 10 29.4 2 20 0 0 0.138 0.15 0.279 0.281 92.7 14.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 2 2 0 0 0 0 1 50 16.7 0 0 0 0 0 0.036 0.036 0.033 81.4 -11\nSchneider, Davis 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nSpringer, George 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nBichette, Bo 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.182 0.195 0.167 93.7 -10\nVarsho, Daulton 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.297 0.651 0.383 95.4 25\nJansen, Danny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.114 0.388 0.203 95.5 37\nGuerrero Jr., Vladimir 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.209 0.223 0.19 105.4 -21\nKiner-Falefa, Isiah 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.05 0.035 82.2 55'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 35 31 6 0 0 1 9 25.7 27.4 3 8.6 0.194 0.29 0.243 0.179 0.359 0.262 85.9 15.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKiermaier, Kevin 6 6 0 0 0 0 4 66.7 58.3 0 0 0 0 0 0.054 0.055 0.049 70.9 -4.5\nKiner-Falefa, Isiah 4 3 2 0 0 0 0 0 25 1 25 0.667 0.667 0.616 0.555 0.605 0.553 81.5 6.3\nKirk, Alejandro 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.216 0.79 0.414 93.9 -1\nVarsho, Daulton 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.443 0.167 0.203 0.162 72.6 30.3\nTurner, Justin 2 1 0 0 0 0 0 0 20 1 50 0 0 0.348 0.095 0.114 0.392 69.9 1\nBichette, Bo 4 4 0 0 0 0 2 50 16.7 0 0 0 0 0 0.15 0.402 0.231 97.6 14.5\nJansen, Danny 3 3 1 0 0 1 1 33.3 20 0 0 0.333 1.333 0.668 0.215 0.837 0.425 95.8 34.5\nGuerrero Jr., Vladimir 4 4 0 0 0 0 1 25 40 0 0 0 0 0 0.2 0.346 0.234 97.9 8.7\nSchneider, Davis 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.027 0.047 94.4 42.5\nSpringer, George 2 1 1 0 0 0 0 0 50 1 50 1 1 0.787 0.081 0.141 0.384 87.1 -26\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.007 0.005 85.5 45'}, {'pitcher_id': '680897', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNastrini, Nick 39 31 10 1 1 3 4 10.3 26.2 8 20.5 0.323 0.71 0.49 0.315 0.617 0.454 87.7 18.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGuerrero Jr., Vladimir 6 4 1 0 0 0 1 16.7 42.9 2 33.3 0.25 0.25 0.379 0.242 0.255 0.376 99.2 -10\nVarsho, Daulton 5 1 1 0 1 0 0 0 40 4 80 1 3 0.876 0.397 0.48 0.625 90.1 20\nSchneider, Davis 6 6 1 0 0 0 1 16.7 14.3 0 0 0.167 0.167 0.148 0.205 0.23 0.191 82.9 25.8\nTurner, Justin 3 1 0 0 0 0 0 0 11.1 2 66.7 0 0 0.461 0.206 0.206 0.523 58.1 -53\nBichette, Bo 6 6 3 0 0 2 1 16.7 30 0 0 0.5 1.5 0.843 0.471 1.218 0.7 89.6 28\nJansen, Danny 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.095 0.103 0.086 71.5 2.5\nKiner-Falefa, Isiah 4 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.317 0.242 0.609 0.355 93.1 28.5\nKiermaier, Kevin 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.444 0.542 0.75 0.555 95.5 15.5\nKirk, Alejandro 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.706 1.521 0.941 96.2 20.5\nSpringer, George 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.199 0.714 0.372 80.2 55.5'}, {'pitcher_id': '673929', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeasure, Jordan 18 13 3 1 0 1 4 22.2 32.4 5 27.8 0.231 0.538 0.428 0.227 0.514 0.416 87.1 29.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurner, Justin 3 1 0 0 0 0 1 33.3 50 2 66.7 0 0 0.461 0.461\nGuerrero Jr., Vladimir 3 1 0 0 0 0 0 0 16.7 2 66.7 0 0 0.461 0.045 0.045 0.475 74.1 -36\nVarsho, Daulton 3 3 0 0 0 0 2 66.7 54.5 0 0 0 0 0 0.005 0.006 0.004 86.1 54\nBichette, Bo 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.719 0.578 0.997 0.67 92.8 20\nSpringer, George 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.346 0.126 0.167 0.409 73 52\nKiner-Falefa, Isiah 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.08 0.132 0.09 85.9 25\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 83.2 87\nJansen, Danny 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.877 3.194 1.656 103.7 26'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 27 23 6 0 0 0 2 7.4 12.2 3 11.1 0.261 0.261 0.281 0.304 0.39 0.347 87 5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBichette, Bo 6 5 2 0 0 0 0 0 0 1 16.7 0.4 0.4 0.405 0.333 0.397 0.377 83.4 9.6\nTurner, Justin 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.39 0.435 0.532 0.481 93.3 30\nKirk, Alejandro 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.423 0.867 0.542 98.7 20.5\nGuerrero Jr., Vladimir 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.507 0.612 0.484 91.1 16.5\nKiermaier, Kevin 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.074 0.084 0.079 83.2 -10\nJansen, Danny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.174 0.183 0.173 93.4 -19\nSchneider, Davis 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nKiner-Falefa, Isiah 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.374 0.375 0.347 62.6 -55\nRichards, Trevor 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.194 0.194 0.175 69.9 -55\nVarsho, Daulton 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nClement, Ernie 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.012 0.006 84 40'}], 142: [{'pitcher_id': '621051', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Steven 28 22 6 3 0 3 7 25 24.4 5 17.9 0.273 0.818 0.508 0.213 0.469 0.372 87.5 27.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 5 2 1 0 0 1 0 0 20 3 60 0.5 2 0.833 0.443 1.536 0.739 93.9 43.5\nCastro, Willi 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.329 0.352 0.299 71.7 43\nKepler, Max 3 2 0 0 0 0 1 33.3 28.6 1 33.3 0 0 0.231 0.005 0.006 0.234 77.2 62\nFarmer, Kyle 3 3 1 1 0 0 1 33.3 25 0 0 0.333 0.667 0.42 0.141 0.215 0.144 94.7 51\nCorrea, Carlos 3 3 1 1 0 0 1 33.3 20 0 0 0.333 0.667 0.415 0.216 0.236 0.196 100.2 -8\nMiranda, Jose 3 3 1 0 0 1 0 0 20 0 0 0.333 1.333 0.695 0.147 0.382 0.212 91.9 31.7\nMartin, Austin 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.118 0.118 0.106 52.1 -66\nLarnach, Trevor 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nBuxton, Byron 2 1 1 0 0 1 0 0 0 0 0 1 4 1.405 0.957 3.313 1.232 110.5 21\nMargot, Manuel 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nJeffers, Ryan 1 1 1 1 0 0 0 0 50 0 0 1 2 1.27 0.103 0.129 0.099 75.8 44\nVázquez, Christian 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0'}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 10 8 1 0 0 0 4 40 45.5 2 20 0.125 0.125 0.225 0.119 0.131 0.226 77.8 -20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.026 0.032 0.024 86.4 -25\nBuxton, Byron 2 2 0 0 0 0 2 100 37.5 0 0 0 0 0 0\nSantana, Carlos 1 0 0 0 0 0 0 0 50 1 100 0.69 0.69\nMargot, Manuel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.307 0.347 0.29 90.7 -9.5\nKepler, Max 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nVázquez, Christian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.255 0.255 0.23 43.3 -36'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 59 54 13 3 0 2 23 39 26.4 4 6.8 0.241 0.407 0.314 0.197 0.296 0.252 85.3 19.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJeffers, Ryan 7 5 1 0 0 0 2 28.6 33.3 2 28.6 0.2 0.2 0.325 0.028 0.036 0.217 74 33.3\nMargot, Manuel 6 5 1 1 0 0 2 33.3 14.3 1 16.7 0.2 0.4 0.327 0.193 0.27 0.279 81.3 25.3\nSantana, Carlos 8 7 2 0 0 1 4 50 43.8 1 12.5 0.286 0.714 0.457 0.243 0.662 0.413 89.1 40\nLarnach, Trevor 6 6 1 0 0 0 4 66.7 16.7 0 0 0.167 0.167 0.147 0.132 0.186 0.137 95.6 29.5\nVázquez, Christian 4 4 2 0 0 0 2 50 16.7 0 0 0.5 0.5 0.444 0.427 0.477 0.395 85.2 20.5\nFarmer, Kyle 4 4 0 0 0 0 2 50 30.8 0 0 0 0 0 0.1 0.118 0.093 78.4 24\nCastro, Willi 6 6 4 1 0 1 1 16.7 23.1 0 0 0.667 1.333 0.854 0.355 0.512 0.377 89.6 -11.4\nBuxton, Byron 5 4 0 0 0 0 3 60 33.3 0 0 0 0 0.145 0.001 0.001 0.145 92.3 71\nCorrea, Carlos 4 4 1 0 0 0 0 0 20 0 0 0.25 0.25 0.22 0.142 0.163 0.131 74.8 7.5\nMiranda, Jose 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.202 0.224 0.186 86.9 27.7\nKepler, Max 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.172 0.203 0.16 95.9 15\nMartin, Austin 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.635 0.453 0.551 0.432 100.6 13'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 19 18 2 0 0 0 9 47.4 52.3 1 5.3 0.111 0.111 0.13 0.093 0.111 0.124 85 8.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVázquez, Christian 7 7 0 0 0 0 3 42.9 50 0 0 0 0 0 0.021 0.023 0.02 76.3 -0.5\nSantana, Carlos 3 3 1 0 0 0 0 0 28.6 0 0 0.333 0.333 0.296 0.135 0.149 0.124 86.6 15.7\nCorrea, Carlos 2 1 0 0 0 0 1 50 60 1 50 0 0 0.346 0.346\nCastro, Willi 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.295 0.329 0.406 0.318 91.1 8\nFarmer, Kyle 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nLarnach, Trevor 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBuxton, Byron 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 91 76 20 2 1 5 18 19.8 20.6 13 14.3 0.263 0.513 0.373 0.228 0.442 0.341 90.7 18.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 16 14 2 1 0 0 2 12.5 10 2 12.5 0.143 0.214 0.22 0.199 0.291 0.27 86.1 12.5\nKepler, Max 15 13 3 0 0 1 6 40 16.7 1 6.7 0.231 0.462 0.302 0.185 0.308 0.247 89.8 22.4\nSantana, Carlos 14 12 4 0 0 2 3 21.4 26.1 2 14.3 0.333 0.833 0.511 0.345 0.853 0.523 94.4 30\nMiranda, Jose 8 8 1 0 0 1 1 12.5 18.2 0 0 0.125 0.5 0.259 0.222 0.582 0.335 92.5 8.6\nLarnach, Trevor 8 5 3 1 0 1 1 12.5 25 2 25 0.6 1.4 0.702 0.292 0.917 0.531 81.8 25.4\nBuxton, Byron 10 6 2 0 0 0 1 10 22.2 4 40 0.333 0.333 0.453 0.17 0.208 0.377 97.7 14\nCastro, Willi 5 4 2 0 1 0 1 20 50 1 20 0.5 1 0.628 0.245 0.281 0.322 94.5 16.7\nJeffers, Ryan 3 3 1 0 0 0 1 33.3 37.5 0 0 0.333 0.333 0.294 0.087 0.118 0.086 89 38.5\nVázquez, Christian 5 4 1 0 0 0 0 0 0 1 20 0.25 0.25 0.316 0.453 0.637 0.507 88.9 11.3\nMargot, Manuel 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.296 0.156 0.173 0.144 101.2 1\nFarmer, Kyle 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.069 0.08 0.062 87.1 31.5\nLewis, Royce 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 89 79 21 5 1 3 12 13.5 18.8 9 10.1 0.266 0.468 0.345 0.262 0.436 0.338 91.5 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 19 16 5 0 0 0 3 15.8 22.9 3 15.8 0.313 0.313 0.341 0.269 0.396 0.349 92.8 23\nLarnach, Trevor 8 6 0 0 0 0 4 50 50 2 25 0 0 0.173 0.095 0.12 0.241 84.1 18.5\nBuxton, Byron 10 9 7 3 0 2 0 0 19 1 10 0.778 1.778 1.03 0.569 1.504 0.848 102.6 29.3\nKepler, Max 11 9 1 1 0 0 0 0 11.1 1 9.1 0.111 0.222 0.175 0.166 0.195 0.209 91.9 29.4\nMargot, Manuel 8 7 3 1 0 1 1 12.5 13.6 1 12.5 0.429 1 0.597 0.163 0.258 0.244 92.3 -4.5\nSantana, Carlos 10 9 0 0 0 0 0 0 7.7 1 10 0 0 0.069 0.187 0.247 0.236 92.4 19.9\nCastro, Willi 8 8 2 0 1 0 1 12.5 14.3 0 0 0.25 0.5 0.312 0.325 0.438 0.338 85.2 10\nMiranda, Jose 5 5 2 0 0 0 1 20 25 0 0 0.4 0.4 0.355 0.31 0.359 0.298 80.7 29\nJeffers, Ryan 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.141 0.186 0.142 87.5 0.3\nVázquez, Christian 4 4 1 0 0 0 0 0 20 0 0 0.25 0.25 0.221 0.313 0.367 0.305 83.8 -5.8\nPaddack, Chris 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nMartin, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.429 0.53 0.427 99.2 1'}, {'pitcher_id': '686563', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCannon, Jonathan 14 13 6 3 0 0 4 28.6 14.3 1 7.1 0.462 0.692 0.512 0.399 0.617 0.455 96.3 10.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiranda, Jose 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.036 0.036 0.033 81.4 -11\nMartin, Austin 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.18 0.196 0.163 100.1 -4\nLarnach, Trevor 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.391 0.726 0.582 100.8 18\nSantana, Carlos 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.47 0.55 0.447 98.1 12\nCastro, Willi 2 2 1 1 0 0 1 50 0 0 0 0.5 1 0.635 0.385 1.115 0.625 103.4 23\nKepler, Max 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.867 1.255 0.909 108.8 12\nJeffers, Ryan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.46 0.497 0.426 82.5 16'}, {'pitcher_id': '672860', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBerroa, Prelander 2 1 0 0 0 0 0 0 0 0 0 0 0 0.362 0.133 0.157 0.422 81.1 -4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastro, Willi 1 0 0 0 0 0 0 0 0 0 0 0.724 0.724\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.133 0.157 0.12 81.1 -4'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 25 24 6 1 0 0 1 4 11.4 1 4 0.25 0.292 0.255 0.232 0.265 0.24 89 -3.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 8 8 3 1 0 0 0 0 0 0 0 0.375 0.5 0.379 0.222 0.259 0.211 83.6 7.1\nSantana, Carlos 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.177 0.271 0.309 0.266 96.1 -17.6\nJeffers, Ryan 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.442 0.493 0.507 95 2\nMiranda, Jose 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.145 0.163 0.135 84.9 37.5\nCorrea, Carlos 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.209 0.243 0.193 87.4 -13\nVázquez, Christian 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.17 0.192 0.153 91.8 -13\nLarnach, Trevor 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nCastro, Willi 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.27 0.303 0.254 97.9 -19.5\nBuxton, Byron 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.138 0.145 0.131 74.7 -41'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 34 33 9 4 0 0 12 35.3 37.5 0 0 0.273 0.394 0.301 0.255 0.405 0.295 92 24.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCorrea, Carlos 4 4 1 0 0 0 1 25 42.9 0 0 0.25 0.25 0.221 0.146 0.157 0.132 91.1 12.3\nMargot, Manuel 5 5 1 0 0 0 2 40 50 0 0 0.2 0.2 0.178 0.298 0.604 0.385 85.8 20\nSantana, Carlos 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.048 0.174 0.09 87.2 51.3\nJeffers, Ryan 3 2 1 1 0 0 1 33.3 28.6 0 0 0.5 1 0.664 0.223 0.332 0.395 92.9 19\nCastro, Willi 4 4 2 1 0 0 1 25 12.5 0 0 0.5 0.75 0.538 0.438 0.579 0.445 94.8 9.7\nKepler, Max 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.444 0.467 0.559 0.45 95.9 13\nBuxton, Byron 3 3 1 1 0 0 2 66.7 60 0 0 0.333 0.667 0.423 0.276 0.371 0.287 104.7 10\nFarmer, Kyle 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.353 0.417 0.329 89.4 28.7\nVázquez, Christian 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nLarnach, Trevor 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMiranda, Jose 2 2 1 1 0 0 0 0 50 0 0 0.5 1 0.635 0.379 0.877 0.517 97.5 39.5\nMartin, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.371 0.791 0.483 98.8 22'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 37 33 10 3 0 0 8 21.6 24.6 2 5.4 0.303 0.394 0.335 0.257 0.317 0.288 84.9 2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 15 13 3 0 0 0 4 26.7 24 2 13.3 0.231 0.231 0.268 0.297 0.384 0.351 91.1 4.6\nFarmer, Kyle 6 5 2 1 0 0 1 16.7 22.2 0 0 0.4 0.6 0.475 0.298 0.342 0.353 80.6 5\nCastro, Willi 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.068 0.082 0.062 55.4 8\nLarnach, Trevor 3 3 1 1 0 0 0 0 20 0 0 0.333 0.667 0.423 0.428 0.54 0.42 90.6 27.7\nMargot, Manuel 4 4 2 0 0 0 0 0 20 0 0 0.5 0.5 0.44 0.21 0.24 0.191 82.6 -12.5\nVázquez, Christian 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.002 0.002 0.002 86.4 45\nKepler, Max 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.21 0.244 0.199 97.7 -1\nBuxton, Byron 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.635 0.228 0.258 0.213 93.8 -17.5\nPaddack, Chris 1 0 0 0 0 0 0 0 0 0 0 34.7 -51'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 80 71 18 3 0 4 14 17.5 31.3 9 11.3 0.254 0.465 0.345 0.262 0.47 0.352 86.3 23.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 34 31 6 1 0 3 6 17.6 31.1 3 8.8 0.194 0.516 0.323 0.284 0.583 0.391 87.6 25.7\nCorrea, Carlos 12 9 1 0 0 0 2 16.7 30.8 3 25 0.111 0.111 0.246 0.162 0.233 0.3 87.5 26.4\nBuxton, Byron 11 9 1 0 0 0 4 36.4 50 2 18.2 0.111 0.111 0.205 0.111 0.138 0.214 89.2 30\nSantana, Carlos 9 8 4 0 0 1 0 0 18.2 1 11.1 0.5 0.875 0.602 0.461 0.932 0.6 95.4 25.9\nCastro, Willi 8 8 2 1 0 0 2 25 33.3 0 0 0.25 0.375 0.261 0.292 0.377 0.295 67.3 37.7\nVázquez, Christian 4 4 3 1 0 0 0 0 0 0 0 0.75 1 0.746 0.226 0.253 0.206 81.6 -14.8\nMargot, Manuel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.204 0.22 0.187 87.7 -9'}, {'pitcher_id': '694363', 'player_data': '', 'team_data': 'Jared Shuster has not faced anyone on this team.'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 35 30 14 3 0 0 4 11.4 13.8 4 11.4 0.467 0.567 0.467 0.324 0.407 0.35 84.2 18.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKepler, Max 7 7 3 0 0 0 2 28.6 21.4 0 0 0.429 0.429 0.38 0.17 0.25 0.18 80.4 1.6\nFarmer, Kyle 4 2 1 0 0 0 1 25 28.6 1 25 0.5 0.5 0.396 0.298 0.337 0.315 94.2 35.5\nCastro, Willi 7 7 2 1 0 0 1 14.3 16.7 0 0 0.286 0.429 0.308 0.243 0.348 0.253 80.7 34.3\nJeffers, Ryan 3 2 2 1 0 0 0 0 16.7 1 33.3 1 1.5 0.949 0.698 0.857 0.673 100.4 12\nVázquez, Christian 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.569 0.682 0.542 87.9 23\nMiranda, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.267 0.165 91.6 27\nMargot, Manuel 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.397 0.435 0.362 81.7 3.5\nBuxton, Byron 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.116 0.143 0.106 66.2 41.5\nCorrea, Carlos 2 1 1 0 0 0 0 0 0 1 50 1 1 0.792 0.44 0.484 0.552 105.6 -1\nLarnach, Trevor 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.874 0.926 0.791 75.3 17\nSantana, Carlos 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.076 0.076 0.068 77.9 -37\nLewis, Royce 1 0 0 0 0 0 0 0 1 100 0.696 0.696'}, {'pitcher_id': '680897', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '673929', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeasure, Jordan 12 10 4 1 0 0 0 0 16.7 1 8.3 0.4 0.5 0.446 0.41 0.506 0.451 85.7 1.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Carlos 3 2 1 1 0 0 0 0 25 0 0 0.5 1 0.664 0.565 0.649 0.594 86.1 4.5\nKepler, Max 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.089 0.103 0.081 90.6 -27.5\nJeffers, Ryan 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nCastro, Willi 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.137 0.392 0.217 93.3 31\nBuxton, Byron 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.94 0.999 0.86 67.6 23\nMiranda, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.203 0.203 0.183 45 -29\nLarnach, Trevor 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.56 0.65 0.528 95.7 4\nFarmer, Kyle 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.111 0.26 0.163 91.1 27\nCorrea, Carlos 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.837 1.055 0.821 110.8 9'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 39 37 8 1 1 2 3 7.7 12.7 2 5.1 0.216 0.459 0.303 0.262 0.453 0.324 87 10.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMargot, Manuel 7 7 1 0 0 0 1 14.3 18.8 0 0 0.143 0.143 0.124 0.162 0.197 0.157 92 -2.2\nKepler, Max 7 5 1 0 0 1 0 0 14.3 2 28.6 0.2 0.8 0.475 0.303 0.531 0.445 99.2 21\nCastro, Willi 4 4 1 0 1 0 0 0 12.5 0 0 0.25 0.75 0.403 0.186 0.587 0.316 91.1 21.3\nCorrea, Carlos 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.083 0.095 0.076 73.1 -26.5\nSantana, Carlos 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.294 0.307 0.265 70.5 -6\nJeffers, Ryan 5 5 1 0 0 1 1 20 33.3 0 0 0.2 0.8 0.417 0.221 0.571 0.328 79.2 18.5\nMiranda, Jose 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.52 0.589 0.489 86.9 0.5\nLarnach, Trevor 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.559 1.034 0.681 97.1 19.5\nMartin, Austin 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.002 0.002 0.002 69.5 76\nFarmer, Kyle 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.277 0.407 0.306 84.9 20.5'}], 143: [{'pitcher_id': '621051', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Steven 7 6 2 1 0 0 0 0 8.3 1 14.3 0.333 0.5 0.403 0.227 0.286 0.292 82.7 4.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBohm, Alec 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.441 0.517 0.621 0.5 81 -15.5\nStott, Bryson 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nRojas, Johan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.163 0.17 0.154 91.2 -18\nMarsh, Brandon 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.103 0.22 0.135 91.9 27\nHarper, Bryce 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.079 0.054 76.8 -30\nCastellanos, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 74.2 79'}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 5 5 2 0 1 0 3 60 53.3 0 0 0.4 0.8 0.48 0.268 0.313 0.249 79.3 25.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 3 3 0 0 0 0 3 100 66.7 0 0 0 0 0 0\nMerrifield, Whit 1 1 1 0 1 0 0 0 0 0 0 1 3 1.529 0.693 0.901 0.664 95.7 16\nSchwarber, Kyle 1 1 1 0 0 0 0 0 0 0 0 1 1 0.87 0.647 0.665 0.582 62.9 35'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 19 15 3 0 0 3 4 21.1 34.2 4 21.1 0.2 0.8 0.475 0.144 0.427 0.328 92.6 24.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 5 5 0 0 0 0 4 80 54.5 0 0 0 0 0 0.003 0.006 0.004 88.5 38\nMerrifield, Whit 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.695 0.138 0.248 0.157 93.8 -2.3\nBohm, Alec 2 2 2 0 0 2 0 0 16.7 0 0 1 4 2.085 0.55 2 1.03 100.8 29.5\nHarper, Bryce 2 0 0 0 0 0 0 0 100 2 100 0.692 0.692\nDahl, David 2 1 0 0 0 0 0 0 66.7 1 50 0 0 0.346 0.011 0.016 0.351 85.4 63\nSosa, Edmundo 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.346 0.046 0.065 0.366 78.7 34\nCastellanos, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.127 0.266 0.16 93.2 28\nPache, Cristian 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.32 1.016 0.542 96.9 31'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 36 31 7 2 0 0 6 16.7 32.4 5 13.9 0.226 0.29 0.289 0.283 0.401 0.341 85.2 12.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCastellanos, Nick 7 3 2 0 0 0 0 0 33.3 4 57.1 0.667 0.667 0.649 0.636 0.682 0.641 66.1 22.7\nHarper, Bryce 7 6 0 0 0 0 1 14.3 41.7 1 14.3 0 0 0.099 0.162 0.325 0.275 84.7 7.6\nSchwarber, Kyle 6 6 0 0 0 0 4 66.7 50 0 0 0 0 0 0.136 0.155 0.13 93.5 -0.5\nStott, Bryson 5 5 1 1 0 0 0 0 33.3 0 0 0.2 0.4 0.252 0.314 0.657 0.406 88.9 27\nMerrifield, Whit 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.092 0.098 0.096 56 -41\nBohm, Alec 3 3 1 1 0 0 0 0 20 0 0 0.333 0.667 0.415 0.581 0.722 0.563 102.2 4\nRojas, Johan 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.134 0.183 0.128 79.4 37\nMarsh, Brandon 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.603 0.699 0.56 105.1 5\nPache, Cristian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.614 0.693 0.572 103.3 6\nDahl, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 90.4 55'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 30 29 8 1 0 3 8 26.7 28.9 0 0 0.276 0.621 0.386 0.222 0.502 0.315 88.4 20.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 14 14 3 0 0 1 1 7.1 14.3 0 0 0.214 0.429 0.274 0.187 0.36 0.231 82.9 16.7\nMarsh, Brandon 3 3 1 0 0 1 2 66.7 80 0 0 0.333 1.333 0.669 0.295 0.868 0.475 108.9 20\nCastellanos, Nick 5 4 3 1 0 0 1 20 20 0 0 0.75 1 0.745 0.55 1.207 0.725 99.1 19.7\nSchwarber, Kyle 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.003 0.003 0.003 92.1 84\nSosa, Edmundo 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.048 0.063 0.056 90.3 7.5\nDahl, David 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.003 0.308 0.981 0.523 99.7 27\nHarper, Bryce 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBohm, Alec 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 49 46 9 3 0 0 8 16.3 20.2 3 6.1 0.196 0.261 0.228 0.248 0.387 0.301 88.5 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 11 11 3 0 0 0 1 9.1 9.1 0 0 0.273 0.273 0.24 0.261 0.339 0.262 90.6 16.9\nMarsh, Brandon 8 7 2 1 0 0 1 12.5 18.2 1 12.5 0.286 0.429 0.354 0.294 0.455 0.379 84.4 -1.5\nSchwarber, Kyle 6 5 0 0 0 0 2 33.3 27.3 1 16.7 0 0 0.115 0.201 0.355 0.311 99 30\nBohm, Alec 4 4 1 0 0 0 1 25 15.4 0 0 0.25 0.25 0.221 0.087 0.133 0.093 80.7 25.7\nHarper, Bryce 5 4 1 0 0 0 0 0 14.3 1 20 0.25 0.25 0.315 0.244 0.295 0.327 87.7 39.5\nCastellanos, Nick 4 4 2 2 0 0 2 50 50 0 0 0.5 1 0.631 0.415 0.936 0.577 103.5 17\nDahl, David 5 5 0 0 0 0 0 0 33.3 0 0 0 0 0 0.281 0.387 0.292 81.3 -4\nSosa, Edmundo 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.038 0.124 0.067 93.8 36\nStott, Bryson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.083 0.077 87.4 -29\nPache, Cristian 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.427 0.659 0.462 88.5 8.5'}, {'pitcher_id': '686563', 'player_data': '', 'team_data': 'Jonathan Cannon has not faced anyone on this team.'}, {'pitcher_id': '672860', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 35 30 9 0 0 0 3 8.6 13.1 3 8.6 0.3 0.3 0.295 0.219 0.275 0.259 83.7 -9.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarsh, Brandon 8 8 3 0 0 0 1 12.5 13.3 0 0 0.375 0.375 0.331 0.259 0.287 0.244 79.4 -25.4\nSchwarber, Kyle 6 5 2 0 0 0 0 0 11.1 0 0 0.4 0.4 0.294 0.399 0.471 0.359 84.9 7.5\nCastellanos, Nick 5 4 1 0 0 0 1 20 22.2 1 20 0.25 0.25 0.315 0.153 0.245 0.277 96.7 4.3\nHarper, Bryce 4 3 2 0 0 0 0 0 14.3 1 25 0.667 0.667 0.616 0.126 0.135 0.26 80.8 -40.7\nStott, Bryson 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.133 0.151 0.123 84.5 9.7\nBohm, Alec 5 5 1 0 0 0 0 0 12.5 0 0 0.2 0.2 0.177 0.167 0.29 0.192 95 -10.2\nSosa, Edmundo 2 1 0 0 0 0 1 50 33.3 1 50 0 0 0.348 0.348\nDahl, David 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.29 0.324 0.261 69.8 9\nRojas, Johan 1 0 0 0 0 0 0 0 0 0 0 29.8 -21'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 23 20 7 2 1 1 6 26.1 36.7 2 8.7 0.35 0.7 0.441 0.238 0.408 0.297 90.8 16.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 5 5 1 0 1 0 2 40 50 0 0 0.2 0.6 0.32 0.28 0.512 0.338 99.4 21.3\nHarper, Bryce 3 3 2 1 0 0 1 33.3 20 0 0 0.667 1 0.705 0.262 0.548 0.335 94.9 36.5\nCastellanos, Nick 3 2 1 1 0 0 0 0 20 1 33.3 0.5 1 0.651 0.293 0.389 0.423 83.1 8\nBohm, Alec 4 3 1 0 0 0 0 0 20 1 25 0.333 0.333 0.393 0.249 0.304 0.35 94 6\nMerrifield, Whit 2 1 0 0 0 0 0 0 25 0 0 0 0 0 0.44 0.878 0.273 94.7 29.5\nSosa, Edmundo 2 2 2 0 0 1 0 0 33.3 0 0 1 2.5 1.445 0.391 0.673 0.445 75.6 -22\nStott, Bryson 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.01 0.018 0.011 84.7 55\nMarsh, Brandon 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMarchán, Rafael 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 79 68 24 5 0 7 10 12.7 18.7 9 11.4 0.353 0.735 0.469 0.304 0.625 0.422 91.6 7.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 26 22 11 1 0 6 2 7.7 15 4 15.4 0.5 1.364 0.753 0.498 1.25 0.716 102.3 13.4\nSchwarber, Kyle 9 7 3 1 0 1 2 22.2 18.2 2 22.2 0.429 1 0.611 0.22 0.423 0.359 98.2 16.2\nBohm, Alec 12 10 2 0 0 0 0 0 5.9 1 8.3 0.2 0.2 0.204 0.204 0.221 0.233 86.2 -9.2\nCastellanos, Nick 6 5 1 0 0 0 1 16.7 50 0 0 0.2 0.2 0.147 0.197 0.216 0.244 69.2 8.8\nDahl, David 4 4 1 1 0 0 2 50 33.3 0 0 0.25 0.5 0.304 0.252 0.744 0.393 99.4 25.5\nStott, Bryson 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.393 0.091 0.123 0.236 85.3 15\nMarsh, Brandon 6 5 3 2 0 0 0 0 16.7 1 16.7 0.6 1 0.683 0.306 0.472 0.389 97.2 8.2\nStubbs, Garrett 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.196 0.314 0.215 93.5 10\nWheeler, Zack 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.217 0.15 0.157 0.135 66.2 4.5\nNola, Aaron 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.44 0.085 0.087 0.079 101.3 -17\nSosa, Edmundo 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.215 0.24 0.199 68.8 -1.5\nPache, Cristian 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.287 0.328 0.273 74.2 29'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 88 82 19 4 0 1 17 19.3 20.7 6 6.8 0.232 0.317 0.266 0.274 0.424 0.325 87.8 20.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 42 40 10 1 0 0 5 11.9 13.4 2 4.8 0.25 0.275 0.25 0.277 0.361 0.296 86.1 18.8\nCastellanos, Nick 29 25 4 2 0 0 8 27.6 35 4 13.8 0.16 0.24 0.241 0.247 0.38 0.325 90 25.5\nSchwarber, Kyle 6 6 1 0 0 0 2 33.3 22.2 0 0 0.167 0.167 0.147 0.192 0.356 0.233 82.3 25\nStott, Bryson 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.188 0.207 0.173 82.6 30.3\nBohm, Alec 4 4 1 0 0 0 1 25 0 0 0 0.25 0.25 0.221 0.374 0.48 0.371 102 5\nMarsh, Brandon 2 2 2 0 0 1 0 0 20 0 0 1 2.5 1.443 0.479 1.822 0.926 84.3 3\nHarper, Bryce 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.837 2.285 1.313 111 19'}, {'pitcher_id': '694363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShuster, Jared 16 14 3 1 1 0 3 18.8 29.6 2 12.5 0.214 0.429 0.318 0.246 0.623 0.399 89.3 21.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.232 0.364 1.319 0.682 91.3 31.5\nStott, Bryson 3 2 1 0 0 0 0 0 20 1 33.3 0.5 0.5 0.526 0.243 0.382 0.405 91.4 28.5\nCastellanos, Nick 3 3 1 0 1 0 0 0 20 0 0 0.333 1 0.523 0.382 1.144 0.623 87.8 21.7\nMarsh, Brandon 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nSchwarber, Kyle 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.229 0.264 0.214 100.8 31.5\nBohm, Alec 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.198 0.551 0.311 76.3 -3.5'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 20 18 6 2 0 0 5 25 23.9 1 5 0.333 0.444 0.338 0.277 0.462 0.331 90.8 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMerrifield, Whit 3 3 0 0 0 0 0 0 7.7 0 0 0 0 0 0.298 0.385 0.293 72.1 11.7\nStott, Bryson 4 4 2 1 0 0 1 25 20 0 0 0.5 0.75 0.532 0.343 0.416 0.328 91 -4\nMarsh, Brandon 3 2 0 0 0 0 2 66.7 75 1 33.3 0 0 0.231 0.231\nCastellanos, Nick 3 3 1 0 0 0 0 0 12.5 0 0 0.333 0.333 0.296 0.49 1.108 0.672 98.5 21\nSchwarber, Kyle 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.121 0.477 0.243 105.4 43\nBohm, Alec 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.635 0.08 0.16 0.1 94.4 24\nHarper, Bryce 2 1 1 0 0 0 0 0 0 0 0 1 1 0.444 0.449 0.496 0.332 102.1 21\nRojas, Johan 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.395 0.4 0.364 83.2 -59'}, {'pitcher_id': '680897', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNastrini, Nick 15 11 4 0 0 0 2 13.3 24 4 26.7 0.364 0.364 0.422 0.295 0.501 0.433 85.8 26.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchwarber, Kyle 3 0 0 0 0 0 0 0 100 3 100 0.692 0.692\nMarsh, Brandon 2 1 1 0 0 0 0 0 12.5 1 50 1 1 0.791 0.906 1.14 0.789 100 13\nCastellanos, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.073 0.102 0.074 76.7 32\nStott, Bryson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.171 0.119 77.5 41\nRojas, Johan 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.28 0.308 0.257 64.6 38\nBohm, Alec 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.444 1.278 0.702 95.2 16\nHarper, Bryce 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.275 0.325 0.268 92.9 20.5'}, {'pitcher_id': '673929', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeasure, Jordan 4 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.444 0.445 0.53 0.322 92 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStott, Bryson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.146 0.91 101.5 12\nSchwarber, Kyle 1 0 0 0 0 0 0 0 0 0 0 0 0.005 97.9 55\nRojas, Johan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.351 0.395 0.33 84.8 1\nCastellanos, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.046 0.05 0.041 83.9 -28'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 32 28 11 0 1 1 5 15.6 25 4 12.5 0.393 0.571 0.449 0.323 0.496 0.394 82.6 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHarper, Bryce 11 10 4 0 0 0 2 18.2 29.6 1 9.1 0.4 0.4 0.381 0.312 0.37 0.33 74.9 17.4\nCastellanos, Nick 5 5 2 0 1 0 1 20 36.4 0 0 0.4 0.8 0.5 0.246 0.436 0.3 95.5 24\nSchwarber, Kyle 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.231 0.167 0.197 0.331 85.2 4\nStott, Bryson 2 2 0 0 0 0 0 0 12.5 0 0 0 0 0 0.102 0.107 0.093 67.2 12.5\nBohm, Alec 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.532 0.702 0.591 93.2 -17\nWheeler, Zack 2 1 1 0 0 0 0 0 33.3 1 50 1 1 0.78 0.553 0.553 0.594 66.9 18\nMarsh, Brandon 2 2 1 0 0 1 0 0 25 0 0 0.5 2 1.043 0.661 2.05 1.115 95.2 17\nNola, Aaron 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.435 0.363 0.396 0.33 89 11\nRojas, Johan 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.242 0.278 0.228 83.4 38'}], 144: [{'pitcher_id': '621051', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Steven 18 15 3 1 0 1 3 16.7 23.3 2 11.1 0.2 0.467 0.352 0.262 0.57 0.409 89.9 15.2', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMurphy, Sean 3 2 0 0 0 0 0 0 16.7 1 33.3 0 0 0.232 0.39 0.45 0.471 88.9 21\nAlbies, Ozzie 4 3 1 0 0 0 0 0 16.7 1 25 0.333 0.333 0.395 0.343 0.718 0.508 97 27.7\nOzuna, Marcell 2 2 1 0 0 1 1 50 50 0 0 0.5 2 1.036 0.35 1.38 0.723 106.1 35\nRiley, Austin 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.42 0.333 0.747 0.454 81.5 30\nd'Arnaud, Travis 2 1 0 0 0 0 1 50 100 0 0 0 0 0.36 0.36\nOlson, Matt 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.106 0.097 83.7 -32.5\nAnderson, Brian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.212 0.283 0.195 92.2 -3\nKelenic, Jarred 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0"}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 4 3 2 1 0 0 1 25 40 1 25 0.667 1 0.697 0.243 0.278 0.382 107.9 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 3 2 2 1 0 0 0 0 25 1 33.3 1 1.5 0.929 0.487 0.556 0.573 107.9 -2\nOzuna, Marcell 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 28 25 4 0 0 1 9 32.1 35.7 2 7.1 0.16 0.28 0.219 0.172 0.255 0.245 88.8 19.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 6 4 0 0 0 0 2 33.3 27.3 2 33.3 0 0 0.231 0.125 0.13 0.306 101.8 21\nDuvall, Adam 4 3 1 0 0 0 1 25 25 0 0 0.333 0.333 0.222 0.313 0.645 0.534 97.6 22.3\nLaureano, Ramón 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.296 0.174 0.197 0.162 93.5 6\nShort, Zack 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.082 0.108 0.081 77.7 34\nAlbies, Ozzie 3 3 0 0 0 0 0 0 28.6 0 0 0 0 0 0.157 0.177 0.146 70.4 -6.3\nRiley, Austin 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.071 0.083 0.066 85.7 33\nArcia, Orlando 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.444 0.434 0.546 0.416 104.6 13\nOzuna, Marcell 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.695 0.149 0.341 0.205 99.5 31'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 30 25 5 0 0 1 4 13.3 28.6 2 6.7 0.2 0.32 0.281 0.18 0.312 0.287 87.6 17.7', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMurphy, Sean 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.295 0.178 0.255 0.186 94.8 31.3\nDuvall, Adam 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.446 0.579 0.443 95.6 11.7\nArcia, Orlando 5 5 2 0 0 0 0 0 0 0 0 0.4 0.4 0.352 0.136 0.194 0.135 85.1 15.6\nRiley, Austin 4 4 1 0 0 1 1 25 50 0 0 0.25 1 0.521 0.216 0.711 0.376 92.2 26.3\nOlson, Matt 4 3 0 0 0 0 0 0 16.7 0 0 0 0 0.18 0.088 0.092 0.24 66.4 -10\nOzuna, Marcell 2 1 0 0 0 0 1 50 50 1 50 0 0 0.345 0\nLaureano, Ramón 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nd'Arnaud, Travis 3 2 0 0 0 0 0 0 0 0 0 0 0 0.24 0.33 0.362 0.512 96.9 -3\nAlbies, Ozzie 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.154 0.268 0.181 86 45\nMartínez, J.P. 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nAnderson, Brian 1 0 0 0 0 0 0 0 66.7 0 0 0 0.916 98.8 28"}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 20 13 5 0 0 3 3 15 22.2 6 30 0.385 1.077 0.638 0.359 0.992 0.605 93.7 25.5', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArcia, Orlando 2 1 1 0 0 0 0 0 16.7 1 50 1 1 0.792 0.19 0.204 0.433 96.6 -12\nKelenic, Jarred 3 1 1 0 0 0 0 0 0 2 66.7 1 1 0.758 0.906 0.957 0.74 74.5 18\nShort, Zack 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.002 0.003 0.002 92 51\nAlbies, Ozzie 2 0 0 0 0 0 0 0 0 1 50 0.709 0.709\nOlson, Matt 3 3 2 0 0 2 0 0 16.7 0 0 0.667 2.667 1.337 0.783 2.753 1.43 98.3 26.3\nOzuna, Marcell 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.452 1.567 0.819 99.5 31.5\nMurphy, Sean 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nRiley, Austin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.302 0.33 0.484 97.7 -6\nd'Arnaud, Travis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.01 0.008 82.1 62\nLaureano, Ramón 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nDuvall, Adam 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0"}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 69 58 16 5 0 1 10 14.5 14.3 9 13 0.276 0.414 0.338 0.286 0.462 0.369 91.4 19.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 18 18 6 2 0 0 2 11.1 6.9 0 0 0.333 0.444 0.334 0.328 0.428 0.33 87.9 21.1\nMurphy, Sean 14 14 2 1 0 0 5 35.7 12 0 0 0.143 0.214 0.151 0.236 0.289 0.231 93.9 8.3\nOzuna, Marcell 9 4 1 0 0 0 0 0 18.8 4 44.4 0.25 0.25 0.406 0.399 0.866 0.593 95.6 31.2\nAlbies, Ozzie 6 4 1 0 0 1 0 0 14.3 2 33.3 0.25 1 0.561 0.525 1.511 0.789 96.9 10.3\nAnderson, Brian 4 3 2 0 0 0 1 25 28.6 1 25 0.667 0.667 0.615 0.418 0.501 0.473 99.9 9\nDuvall, Adam 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.083 0.132 0.088 94.1 14\nLaureano, Ramón 5 5 1 0 0 0 1 20 16.7 0 0 0.2 0.2 0.176 0.186 0.235 0.191 97.1 25.3\nRiley, Austin 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.039 0.04 0.255 84.7 10.5\nd'Arnaud, Travis 2 1 1 1 0 0 0 0 0 1 50 1 2 0.981 0.494 1.06 0.674 102 20\nArcia, Orlando 2 2 1 1 0 0 0 0 33.3 0 0 0.5 1 0.635 0.253 0.559 0.345 90.9 48.5\nKelenic, Jarred 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.09 0.105 0.081 72.1 34\nShort, Zack 1 0 0 0 0 0 0 0 50 0 0 0 0.126 79.2 31"}, {'pitcher_id': '686563', 'player_data': '', 'team_data': 'Jonathan Cannon has not faced anyone on this team.'}, {'pitcher_id': '672860', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 18 18 7 2 0 0 4 22.2 28.6 0 0 0.389 0.5 0.383 0.274 0.377 0.28 90 5.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nOlson, Matt 9 9 3 1 0 0 2 22.2 27.8 0 0 0.333 0.444 0.334 0.217 0.266 0.207 95.5 16.6\nRiley, Austin 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.136 0.137 0.123 73.4 -52\nKelenic, Jarred 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.377 1.403 0.721 101 36\nAnderson, Brian 1 1 1 0 0 0 0 0 66.7 0 0 1 1 0.883 0.954 1.173 0.929 98.1 13\nd'Arnaud, Travis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.48 0.528 0.437 94 4\nAlbies, Ozzie 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.306 0.382 0.294 73 38\nDuvall, Adam 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nArcia, Orlando 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.454 0.504 0.418 98.9 2\nFried, Max 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.133 0.133 0.133 53.4 -75"}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 38 33 4 1 0 1 11 28.9 29.7 3 7.9 0.121 0.242 0.203 0.192 0.334 0.268 88.3 16.1', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nArcia, Orlando 12 11 1 0 0 0 2 16.7 23.1 0 0 0.091 0.091 0.073 0.165 0.215 0.161 84 9.2\nAnderson, Brian 7 6 1 0 0 1 2 28.6 33.3 1 14.3 0.167 0.667 0.376 0.242 0.678 0.414 96.5 28.8\nAlbies, Ozzie 6 6 1 0 0 0 1 16.7 25 0 0 0.167 0.167 0.147 0.391 0.478 0.382 88.5 21.2\nRiley, Austin 4 3 0 0 0 0 2 50 36.4 1 25 0 0 0.172 0.082 0.277 0.277 95.2 33\nDuvall, Adam 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.046 0.087 0.056 83.5 26\nd'Arnaud, Travis 2 2 1 1 0 0 1 50 42.9 0 0 0.5 1 0.616 0.146 0.288 0.176 95.5 21\nKelenic, Jarred 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nOlson, Matt 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nFried, Max 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.1 0.134 0.09 87.9 -22\nOzuna, Marcell 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nLaureano, Ramón 1 0 0 0 0 0 0 0 0 0 0.722 0.722"}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 118 102 39 11 0 6 9 7.6 12.7 12 10.2 0.382 0.667 0.468 0.357 0.65 0.455 90 14.9', 'team_data': "Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuvall, Adam 20 17 7 2 0 2 1 5 13.8 3 15 0.412 0.882 0.555 0.331 0.664 0.455 92.8 15.1\nAlbies, Ozzie 17 15 4 2 0 0 1 5.9 11.4 1 5.9 0.267 0.4 0.288 0.246 0.49 0.325 85.6 5.6\nRiley, Austin 20 20 7 2 0 0 3 15 13.5 0 0 0.35 0.45 0.343 0.342 0.42 0.333 87.1 17.8\nOzuna, Marcell 17 12 7 2 0 2 1 5.9 13 5 29.4 0.583 1.25 0.744 0.549 1.196 0.72 91.5 22.5\nAnderson, Brian 14 14 4 2 0 0 0 0 4.2 0 0 0.286 0.429 0.305 0.397 0.691 0.462 91.1 18.3\nArcia, Orlando 7 5 3 0 0 0 0 0 0 2 28.6 0.6 0.6 0.572 0.373 0.431 0.439 89.6 5.4\nd'Arnaud, Travis 10 9 3 1 0 1 1 10 25 0 0 0.333 0.778 0.483 0.42 1.059 0.615 96.5 18.9\nFried, Max 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.233 0.252 0.22 70.5 -41\nLaureano, Ramón 3 1 0 0 0 0 1 33.3 50 1 33.3 0 0 0.472 0.472\nOlson, Matt 5 5 1 0 0 1 1 20 16.7 0 0 0.2 0.8 0.414 0.053 0.187 0.101 99.2 51\nMurphy, Sean 2 2 2 0 0 0 0 0 25 0 0 1 1 0.884 0.855 1.04 0.831 94.5 12.5"}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 17 15 5 3 0 1 4 23.5 36.4 1 5.9 0.333 0.733 0.474 0.378 0.955 0.574 96.4 13.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDuvall, Adam 6 5 1 0 0 1 2 33.3 53.8 0 0 0.2 0.8 0.455 0.234 0.756 0.459 84.7 -7.7\nOlson, Matt 3 2 1 1 0 0 0 0 20 1 33.3 0.5 1 0.646 0.56 1.607 0.824 100.7 23\nOzuna, Marcell 3 3 2 1 0 0 1 33.3 16.7 0 0 0.667 1 0.709 0.509 0.898 0.596 106.6 14.5\nAnderson, Brian 3 3 1 1 0 0 0 0 28.6 0 0 0.333 0.667 0.42 0.421 1.313 0.727 96.1 30.7\nLaureano, Ramón 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.291 0.349 0.283 103.5 4'}], 146: [{'pitcher_id': '621051', 'player_data': '', 'team_data': ''}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.257 0.299 0.235 75.4 -12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.257 0.299 0.235 75.4 -12'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRivera, Emmanuel 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nGordon, Nick 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 18 16 4 1 0 0 7 38.9 37.1 1 5.6 0.25 0.313 0.27 0.182 0.266 0.224 83.9 14.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 6 6 4 1 0 0 1 16.7 10 0 0 0.667 0.833 0.651 0.425 0.516 0.416 100.3 2.4\nAnderson, Tim 4 4 0 0 0 0 2 50 42.9 0 0 0 0 0 0.016 0.017 0.018 77.2 13\nBethancourt, Christian 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.147 0.545 0.275 99.3 35\nChisholm Jr., Jazz 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.344 0.344\nRivera, Emmanuel 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBurger, Jake 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSánchez, Jesús 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.008 0.008 65.9 71\nGordon, Nick 1 0 0 0 0 0 0 0 0 0 0 17.6 -3'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 19 19 1 0 0 0 8 42.1 31.8 0 0 0.053 0.053 0.046 0.118 0.137 0.112 90.1 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 6 6 0 0 0 0 1 16.7 20 0 0 0 0 0 0.233 0.264 0.219 95.3 -11.2\nGordon, Nick 4 4 0 0 0 0 2 50 21.4 0 0 0 0 0 0.019 0.032 0.02 82.9 50\nDe La Cruz, Bryan 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.112 0.118 0.103 83.9 -7\nSánchez, Jesús 3 3 0 0 0 0 3 100 87.5 0 0 0 0 0 0\nBethancourt, Christian 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.186 0.199 0.171 95.3 -1\nRivera, Emmanuel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.071 0.145 0.087 85.6 28'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 9 9 1 0 0 0 1 11.1 25 0 0 0.111 0.111 0.098 0.297 0.394 0.299 85.9 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 5 5 1 0 0 0 1 20 27.3 0 0 0.2 0.2 0.177 0.283 0.368 0.284 83.6 17.8\nGordon, Nick 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.338 0.391 0.314 85.6 10\nDe La Cruz, Bryan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.246 0.532 0.327 96.3 23'}, {'pitcher_id': '686563', 'player_data': '', 'team_data': ''}, {'pitcher_id': '672860', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 8 8 8 3 0 1 0 0 11.8 0 0 1 1.75 1.161 0.696 1.2 0.812 96.1 11.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 3 3 3 2 0 0 0 0 28.6 0 0 1 1.667 1.125 0.82 1.177 0.87 85.7 21.7\nDe La Cruz, Bryan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.463 0.502 0.419 103.2 -2\nGordon, Nick 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.072 0.733 0.908 0.712 99.5 9.5\nBell, Josh 2 2 2 0 0 1 0 0 0 0 0 1 2.5 1.443 0.591 1.875 1.022 104.7 4'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 13 12 4 1 0 1 2 15.4 27 1 7.7 0.333 0.667 0.435 0.291 0.733 0.441 92.6 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 7 7 2 0 0 1 1 14.3 18.8 0 0 0.286 0.714 0.403 0.292 0.714 0.408 90.1 14\nSánchez, Jesús 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nAnderson, Tim 2 2 1 1 0 0 0 0 40 0 0 0.5 1 0.631 0.427 0.916 0.561 100.5 20.5\nGordon, Nick 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.46 1.778 0.938 105.3 39\nDe La Cruz, Bryan 0 0 0 0 0 0 0 50 0\nRivera, Emmanuel 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.139 0.18 0.127 79.5 -4'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 45 41 12 2 0 4 11 24.4 24.1 4 8.9 0.293 0.634 0.415 0.26 0.575 0.383 88.3 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChisholm Jr., Jazz 14 12 4 0 0 2 1 7.1 13 2 14.3 0.333 0.833 0.512 0.31 0.819 0.503 88.7 26.3\nDe La Cruz, Bryan 13 12 6 2 0 1 2 15.4 22.2 1 7.7 0.5 0.917 0.61 0.338 0.693 0.458 97.1 13\nSánchez, Jesús 11 10 2 0 0 1 6 54.5 37.5 1 9.1 0.2 0.5 0.325 0.166 0.407 0.282 79.2 -1\nFortes, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.364 0.41 0.345 77.1 21.5\nLuzardo, Jesús 4 4 0 0 0 0 1 25 25 0 0 0 0 0 0.12 0.132 0.114 77.4 -24\nLopez, Otto 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 25 24 4 2 0 0 6 24 25 1 4 0.167 0.25 0.197 0.252 0.53 0.339 88.7 7.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 16 16 3 1 0 0 4 25 25.8 0 0 0.188 0.25 0.187 0.233 0.459 0.29 85.3 1.4\nBell, Josh 5 5 1 1 0 0 1 20 22.2 0 0 0.2 0.4 0.249 0.446 1.045 0.61 102.3 5.8\nFortes, Nick 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.026 0.057 0.362 82 36\nBethancourt, Christian 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.025 0.044 0.033 81.8 56'}, {'pitcher_id': '694363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShuster, Jared 8 8 4 1 0 0 1 12.5 31.3 0 0 0.5 0.625 0.486 0.299 0.382 0.296 73.6 5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Jesús 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.132 0.134 0.121 57.7 15\nBell, Josh 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.441 0.316 0.349 0.289 97.6 6\nChisholm Jr., Jazz 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.283 0.286 0.257 59.3 -13\nBurger, Jake 2 2 2 1 0 0 0 0 25 0 0 1 1.5 1.063 0.464 0.759 0.516 91.8 13.5'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 4 4 0 0 0 0 2 50 42.9 0 0 0 0 0 0.024 0.066 0.035 89.5 45.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRivera, Emmanuel 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.01 0.013 0.009 85.2 57\nBruján, Vidal 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBell, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.077 0.24 0.123 93.8 34'}, {'pitcher_id': '680897', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '673929', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 18 18 5 1 0 1 5 27.8 17.6 0 0 0.278 0.5 0.327 0.272 0.496 0.322 93.7 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 6 6 1 0 0 0 2 33.3 12.5 0 0 0.167 0.167 0.145 0.132 0.152 0.121 87.8 16.5\nSánchez, Jesús 2 2 1 0 0 1 1 50 0 0 0 0.5 2 1.002 0.45 1.76 0.89 104.7 29\nChisholm Jr., Jazz 3 3 1 1 0 0 0 0 25 0 0 0.333 0.667 0.415 0.234 0.416 0.268 84.6 16\nDe La Cruz, Bryan 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.294 0.358 0.465 0.361 102.5 -3.5\nRivera, Emmanuel 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.653 0.852 0.64 105.4 4\nAnderson, Tim 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBurger, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.121 0.145 0.114 92.7 -23'}], 147: [{'pitcher_id': '621051', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Steven 11 10 1 0 0 0 3 27.3 31.8 1 9.1 0.1 0.1 0.143 0.2 0.34 0.271 88.9 25.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 2 1 0 0 0 0 1 50 75 1 50 0 0 0.346 0.346\nSoto, Juan 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.406 0.456 0.377 84.6 5\nVerdugo, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.308 0.452 0.32 94.5 32.5\nVolpe, Anthony 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nStanton, Giancarlo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nRizzo, Anthony 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.449 0.463 0.404 75 13\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.42 1.465 0.773 98.2 30\nLeMahieu, DJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.104 0.104 0.095 82.2 -9\nTorres, Gleyber 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 93.1 76'}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 10 8 2 0 0 0 4 40 40 2 20 0.25 0.25 0.312 0.293 0.564 0.427 97.6 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTorres, Gleyber 3 3 2 0 0 0 1 33.3 28.6 0 0 0.667 0.667 0.58 0.455 0.57 0.438 100.9 7\nJudge, Aaron 2 1 0 0 0 0 1 50 75 1 50 0 0 0.345 0.345\nStanton, Giancarlo 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.326 0.935 0.525 94.3 9\nVerdugo, Alex 1 0 0 0 0 0 0 0 0 1 100 0.69 0.69\nLeMahieu, DJ 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 8 7 1 0 0 0 3 37.5 42.9 1 12.5 0.143 0.143 0.197 0.284 0.704 0.441 97.8 11.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.68 2.419 0.97 101.4 28\nJudge, Aaron 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.44 0.415 0.784 0.504 117.4 14\nRizzo, Anthony 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.024 0.033 0.022 71.4 -15\nTorres, Gleyber 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.429 0.873 0.536 100.9 19\nStanton, Giancarlo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 23 19 2 0 0 0 9 39.1 31 3 13 0.105 0.105 0.166 0.152 0.173 0.248 96.1 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrisham, Trent 5 4 0 0 0 0 2 40 8.3 1 20 0 0 0.138 0.157 0.169 0.258 100.2 -8\nRizzo, Anthony 5 5 2 0 0 0 1 20 18.2 0 0 0.4 0.4 0.35 0.319 0.372 0.296 92.7 12.5\nJudge, Aaron 3 3 0 0 0 0 2 66.7 57.1 0 0 0 0 0 0.002 0.002 0.002 98.7 62\nStanton, Giancarlo 3 2 0 0 0 0 2 66.7 60 0 0 0 0 0 0 0 0.415 112.1 15\nSoto, Juan 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.204 0.212 0.441 98.2 -16\nLeMahieu, DJ 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nTorres, Gleyber 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.294 0.351 0.481 82.2 4\nVolpe, Anthony 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 95.2 53'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 35 30 2 0 0 1 7 20 22.8 5 14.3 0.067 0.167 0.181 0.156 0.271 0.257 88.9 21.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeMahieu, DJ 7 5 0 0 0 0 1 14.3 12.5 2 28.6 0 0 0.197 0.109 0.116 0.269 90.1 15.3\nJudge, Aaron 6 6 0 0 0 0 2 33.3 30 0 0 0 0 0 0.187 0.211 0.177 98.1 1.8\nStanton, Giancarlo 7 6 0 0 0 0 1 14.3 20 1 14.3 0 0 0.098 0.161 0.223 0.252 84.2 13\nRizzo, Anthony 6 6 0 0 0 0 1 16.7 22.2 0 0 0 0 0 0.123 0.419 0.225 86.8 49.8\nTorres, Gleyber 4 3 1 0 0 1 1 25 40 1 25 0.333 1.333 0.674 0.145 0.483 0.363 96.9 42\nTrevino, Jose 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.23 0.127 0.152 0.313 74.2 5\nVerdugo, Alex 2 2 1 0 0 0 1 50 20 0 0 0.5 0.5 0.44 0.307 0.32 0.277 89.6 10'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 69 61 22 5 0 3 9 13 23.4 8 11.6 0.361 0.59 0.438 0.265 0.475 0.357 87.4 10', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanton, Giancarlo 10 8 2 0 0 0 1 10 18.2 2 20 0.25 0.25 0.314 0.32 0.65 0.46 90.8 33.3\nJudge, Aaron 7 6 3 1 0 2 2 28.6 42.9 1 14.3 0.5 1.667 0.861 0.435 1.197 0.673 106 18.3\nLeMahieu, DJ 8 8 4 1 0 0 2 25 7.7 0 0 0.5 0.625 0.484 0.301 0.357 0.289 93.3 0.2\nRizzo, Anthony 8 7 2 0 0 0 2 25 42.9 1 12.5 0.286 0.286 0.309 0.048 0.057 0.13 76.5 0.2\nSoto, Juan 6 4 1 1 0 0 1 16.7 27.3 2 33.3 0.25 0.5 0.437 0.302 0.364 0.426 79.7 -27.7\nTrevino, Jose 7 7 3 0 0 0 0 0 8.3 0 0 0.429 0.429 0.378 0.288 0.434 0.309 88.1 -5.4\nVerdugo, Alex 6 5 1 0 0 0 0 0 20 1 16.7 0.2 0.2 0.261 0.405 0.491 0.442 84.9 8.4\nTorres, Gleyber 5 5 2 1 0 0 0 0 7.7 0 0 0.4 0.6 0.432 0.317 0.788 0.455 90 14.2\nGrisham, Trent 5 4 2 0 0 1 0 0 30 1 20 0.5 1.25 0.716 0.108 0.306 0.273 84 16.3\nVolpe, Anthony 4 4 2 1 0 0 0 0 18.2 0 0 0.5 0.75 0.538 0.2 0.235 0.191 80.1 19\nCabrera, Oswaldo 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.066 0.099 0.072 78.2 40'}, {'pitcher_id': '686563', 'player_data': '', 'team_data': 'Jonathan Cannon has not faced anyone on this team.'}, {'pitcher_id': '672860', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 15 14 4 1 0 0 6 40 26.9 1 6.7 0.286 0.357 0.307 0.14 0.191 0.178 86.3 4.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTorres, Gleyber 4 3 1 0 0 0 1 25 22.2 1 25 0.333 0.333 0.395 0.249 0.319 0.353 92.2 -8.5\nRizzo, Anthony 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.533 0.128 0.233 0.151 88.9 18.8\nSoto, Juan 3 3 0 0 0 0 3 100 57.1 0 0 0 0 0 0\nVerdugo, Alex 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.325 0.364 0.3 93.5 10\nStanton, Giancarlo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.056 0.057 0.05 56.7 -33'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 24 23 6 0 0 1 7 29.2 33.3 1 4.2 0.261 0.391 0.296 0.286 0.464 0.334 90.4 21.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 6 5 1 0 0 1 1 16.7 16.7 1 16.7 0.2 0.8 0.449 0.256 0.712 0.443 88.3 2.8\nRizzo, Anthony 5 5 0 0 0 0 2 40 33.3 0 0 0 0 0 0.262 0.414 0.287 91.1 25\nGrisham, Trent 3 3 0 0 0 0 2 66.7 44.4 0 0 0 0 0 0.029 0.084 0.049 96.5 41\nLeMahieu, DJ 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.586 0.538 0.651 0.513 94.6 15.3\nJudge, Aaron 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.441 0.367 0.483 0.377 84.9 41\nVerdugo, Alex 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0 0 0 81.5 64\nCabrera, Oswaldo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVolpe, Anthony 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nStanton, Giancarlo 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.877 0.627 0.658 0.564 83.4 11\nTrevino, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.92 1.215 0.927 105.4 11'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 46 41 14 3 0 2 7 15.2 20.8 3 6.5 0.341 0.561 0.394 0.349 0.623 0.433 85.8 -2.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 8 6 1 1 0 0 4 50 37.5 1 12.5 0.167 0.333 0.242 0.236 0.666 0.425 103.1 18\nGrisham, Trent 8 6 2 0 0 0 0 0 7.1 2 25 0.333 0.333 0.393 0.356 0.429 0.427 83.5 6.8\nStanton, Giancarlo 7 7 5 0 0 1 1 14.3 25 0 0 0.714 1.143 0.785 0.411 0.873 0.542 103 -2.3\nLeMahieu, DJ 8 8 3 1 0 0 0 0 8.3 0 0 0.375 0.5 0.374 0.401 0.493 0.389 86.8 -13\nTorres, Gleyber 7 7 2 0 0 1 1 14.3 25 0 0 0.286 0.714 0.416 0.457 0.926 0.577 87.7 6.2\nWeaver, Luke 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.068 0.074 0.061 91.7 -15\nRizzo, Anthony 3 3 0 0 0 0 0 0 40 0 0 0 0 0 0.179 0.182 0.163 54.3 -4.7\nStroman, Marcus 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0.198 0.198 0.203 39.3 -56\nVerdugo, Alex 1 1 1 1 0 0 0 0 0 0 0 1 2 1.242 0.666 1.41 0.857 107.2 18'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 70 64 16 5 0 4 18 25.7 23.3 6 8.6 0.25 0.516 0.353 0.277 0.575 0.385 89.4 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 16 12 5 0 0 2 5 31.3 30.4 4 25 0.417 0.917 0.592 0.385 0.82 0.548 100.3 19.7\nTorres, Gleyber 12 12 2 0 0 1 5 41.7 27.3 0 0 0.167 0.417 0.241 0.231 0.418 0.275 94 18.9\nLeMahieu, DJ 5 5 0 0 0 0 2 40 25 0 0 0 0 0 0.07 0.076 0.065 83.2 5.3\nRizzo, Anthony 7 7 2 0 0 1 0 0 14.3 0 0 0.286 0.714 0.415 0.364 0.799 0.483 85.5 23.1\nSoto, Juan 7 6 3 3 0 0 1 14.3 30 1 14.3 0.5 1 0.632 0.387 1.257 0.663 95.3 15.4\nStanton, Giancarlo 7 7 2 1 0 0 2 28.6 27.3 0 0 0.286 0.429 0.307 0.322 0.73 0.446 95.4 16.2\nCabrera, Oswaldo 4 3 0 0 0 0 1 25 22.2 1 25 0 0 0.174 0.009 0.013 0.181 73.6 53.5\nVolpe, Anthony 5 5 0 0 0 0 2 40 28.6 0 0 0 0 0 0.08 0.087 0.075 67.7 2.7\nTrevino, Jose 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.338 0.398 0.322 83.3 -3\nVerdugo, Alex 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.68 0.784 0.656 90.1 10\nWells, Austin 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.1 0.129 0.092 80.4 46\nGrisham, Trent 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.236 0.24 0.212 92 -7'}, {'pitcher_id': '694363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShuster, Jared 11 10 0 0 0 0 3 27.3 25 1 9.1 0 0 0.063 0.245 0.536 0.356 92.7 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.232 0.355 1.254 0.657 97.5 33\nTrevino, Jose 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nGrisham, Trent 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.051 0.053 0.046 83.3 -12\nRizzo, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.729 0.855 0.696 101.7 9\nTorres, Gleyber 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nStanton, Giancarlo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.049 0.063 0.044 86.8 31\nJudge, Aaron 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.469 1.384 0.757 98.1 28\nVerdugo, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.389 0.442 0.352 84 5'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 21 19 9 0 0 2 5 23.8 9.3 1 4.8 0.474 0.789 0.523 0.369 0.803 0.501 96.6 21.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVolpe, Anthony 3 3 1 0 0 0 1 33.3 0 0 0 0.333 0.333 0.296 0.09 0.106 0.089 90 18\nVerdugo, Alex 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.589 0.458 0.589 0.458 97.3 17\nSoto, Juan 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.469 1.559 0.829 108.2 22\nTorres, Gleyber 3 3 2 0 0 0 1 33.3 28.6 0 0 0.667 0.667 0.589 0.479 0.569 0.462 89.9 13.5\nJudge, Aaron 2 1 1 0 0 1 0 0 0 0 0 1 4 1.002 0.646 2.595 0.888 105 28.5\nCabrera, Oswaldo 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRizzo, Anthony 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.015 0.031 0.018 88.2 33\nGrisham, Trent 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nLeMahieu, DJ 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.477 0.546 0.457 71.9 35\nWells, Austin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.102 0.058 85.8 30\nStanton, Giancarlo 2 2 2 0 0 1 0 0 33.3 0 0 1 2.5 1.485 0.897 2.518 1.408 116.5 16.5'}, {'pitcher_id': '680897', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '673929', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 32 27 7 1 0 2 12 37.5 32.1 5 15.6 0.259 0.519 0.378 0.251 0.545 0.383 99.4 23.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 14 10 6 0 0 2 1 7.1 11.8 4 28.6 0.6 1.2 0.724 0.543 1.256 0.721 103.7 14.2\nWeaver, Luke 4 4 0 0 0 0 4 100 54.5 0 0 0 0 0 0\nStanton, Giancarlo 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.883 1.333 0.818 112.8 12\nVolpe, Anthony 2 2 1 1 0 0 1 50 16.7 0 0 0.5 1 0.635 0.137 0.205 0.144 89.5 21\nTorres, Gleyber 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nJudge, Aaron 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nRizzo, Anthony 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.082 0.169 0.103 94.5 23\nVerdugo, Alex 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.001 0.001 0 83.7 65\nTrevino, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.016 0.031 0.022 88.9 55'}], 158: [{'pitcher_id': '621051', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Steven 11 9 0 0 0 0 0 0 15 2 18.2 0 0 0.127 0.29 0.612 0.434 93.3 27.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 3 2 0 0 0 0 0 0 12.5 1 33.3 0 0 0.232 0.099 0.277 0.334 87.5 34\nContreras, William 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.224 0.782 0.404 91 21.3\nYelich, Christian 3 3 0 0 0 0 0 0 14.3 0 0 0 0 0 0.581 0.87 0.623 102.4 15.3\nMonasterio, Andruw 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nBauers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 84.9 70'}, {'pitcher_id': '605121', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Justin 11 8 2 0 0 0 2 18.2 18.8 3 27.3 0.25 0.25 0.35 0.236 0.544 0.427 83.2 6.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 2 1 0 0 0 0 0 0 28.6 1 50 0 0 0.346 0.851 3.15 1.16 102.7 27\nSánchez, Gary 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.031 0.031 0.028 69.4 16.5\nAdames, Willy 1 0 0 0 0 0 0 0 1 100 0.69 0.69\nFrelick, Sal 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nTurang, Brice 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.563 0.693 0.627 96.6 5\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.318 0.344 0.286 85 2\nChourio, Jackson 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.061 0.07 0.058 76.4 -29'}, {'pitcher_id': '676979', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCrochet, Garrett 28 26 6 1 0 0 9 32.1 34.4 1 3.6 0.231 0.269 0.237 0.23 0.306 0.249 90.1 8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.296 0.168 0.174 0.152 98.4 -11.5\nAdames, Willy 3 3 0 0 0 0 2 66.7 62.5 0 0 0 0 0 0.037 0.102 0.058 91.7 29\nSánchez, Gary 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.071 0.091 0.277 72.5 52\nTurang, Brice 2 2 1 0 0 0 1 50 16.7 0 0 0.5 0.5 0.444 0.42 0.503 0.398 92.2 16\nBauers, Jake 3 2 1 0 0 0 1 33.3 33.3 0 0 0.5 0.5 0.44 0.273 0.33 0.269 96.3 5\nYelich, Christian 3 3 1 0 0 0 1 33.3 50 0 0 0.333 0.333 0.296 0.314 0.352 0.294 101.6 -1\nHoskins, Rhys 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.296 0.287 0.337 0.271 76.5 34.3\nPerkins, Blake 3 3 0 0 0 0 1 33.3 42.9 0 0 0 0 0 0.14 0.15 0.13 74 -40.5\nChourio, Jackson 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.086 0.299 0.161 93.6 32\nOrtiz, Joey 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.481 0.72 0.508 102.8 5'}, {'pitcher_id': '608678', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeone, Dominic 20 19 4 0 0 2 7 35 40 0 0 0.211 0.526 0.322 0.266 0.519 0.348 86.6 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 7 6 2 0 0 1 1 14.3 27.3 0 0 0.333 0.833 0.517 0.452 0.922 0.598 91.8 30.3\nSánchez, Gary 4 4 0 0 0 0 2 50 55.6 0 0 0 0 0 0.151 0.166 0.138 65.7 -14\nHoskins, Rhys 2 2 1 0 0 1 0 0 25 0 0 0.5 2 0.97 0.594 1.651 0.905 101.8 25.5\nPerkins, Blake 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nContreras, William 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBauers, Jake 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.006 0.009 0.005 86.3 38\nAdames, Willy 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.29 0.296 0.261 67.8 17\nMonasterio, Andruw 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTurang, Brice 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.149 0.153 0.14 96.1 -21'}, {'pitcher_id': '656629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKopech, Michael 30 24 6 0 0 0 8 26.7 28.6 6 20 0.25 0.25 0.316 0.224 0.308 0.324 90.8 14.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Gary 6 5 2 0 0 0 1 16.7 37.5 1 16.7 0.4 0.4 0.409 0.34 0.388 0.386 86.7 2\nAdames, Willy 5 3 2 0 0 0 0 0 0 2 40 0.667 0.667 0.632 0.322 0.416 0.467 85.9 20\nContreras, William 4 4 1 0 0 0 3 75 57.1 0 0 0.25 0.25 0.221 0.164 0.196 0.155 113.1 4\nYelich, Christian 4 3 0 0 0 0 0 0 0 1 25 0 0 0.174 0.253 0.356 0.371 95.9 -3.3\nTurang, Brice 3 3 0 0 0 0 2 66.7 37.5 0 0 0 0 0 0.031 0.032 0.029 86.1 -22\nBauers, Jake 2 2 0 0 0 0 2 100 66.7 0 0 0 0 0 0\nFrelick, Sal 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.344 0.405 0.448 93.8 45.5\nMonasterio, Andruw 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.001 0.001 0.348 73.3 77\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.523 1.438 0.821 99.8 24'}, {'pitcher_id': '623167', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFlexen, Chris 52 46 14 4 0 3 9 17.3 20 6 11.5 0.304 0.587 0.411 0.288 0.54 0.387 91.4 14.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 8 7 2 0 0 1 2 25 20 1 12.5 0.286 0.714 0.455 0.112 0.225 0.211 84.1 21.4\nYelich, Christian 10 10 4 1 0 0 1 10 10.5 0 0 0.4 0.5 0.388 0.273 0.477 0.317 95.7 -2.1\nContreras, William 6 5 0 0 0 0 2 33.3 36.4 1 16.7 0 0 0.116 0.308 0.695 0.46 103.8 15.7\nAdames, Willy 6 5 2 0 0 1 1 16.7 37.5 1 16.7 0.4 1 0.597 0.362 0.944 0.564 84.8 7.5\nSánchez, Gary 6 6 1 1 0 0 1 16.7 25 0 0 0.167 0.333 0.21 0.31 0.493 0.341 95.8 29\nBauers, Jake 5 3 1 1 0 0 0 0 0 2 40 0.333 0.667 0.527 0.476 0.825 0.606 102.5 26.3\nTurang, Brice 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.526 0.361 0.386 0.449 87.5 28.5\nMonasterio, Andruw 3 3 2 0 0 1 1 33.3 14.3 0 0 0.667 1.667 0.962 0.352 0.907 0.514 100.9 6\nFrelick, Sal 3 3 1 1 0 0 0 0 33.3 0 0 0.333 0.667 0.415 0.432 0.464 0.392 71.4 2.3\nPerkins, Blake 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.001 0.001 0 71.5 75'}, {'pitcher_id': '686563', 'player_data': '', 'team_data': 'Jonathan Cannon has not faced anyone on this team.'}, {'pitcher_id': '672860', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '657612', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHill, Tim 20 18 6 0 0 0 1 5 13.3 2 10 0.333 0.333 0.335 0.284 0.336 0.309 88.3 -9.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBauers, Jake 6 5 1 0 0 0 1 16.7 25 1 16.7 0.2 0.2 0.26 0.354 0.454 0.398 93.1 -2.3\nYelich, Christian 7 7 3 0 0 0 0 0 16.7 0 0 0.429 0.429 0.38 0.18 0.209 0.168 91.9 -12.1\nContreras, William 2 1 1 0 0 0 0 0 0 1 50 1 1 0.792 0.569 0.612 0.604 111.7 1\nAdames, Willy 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.505 0.589 0.475 91 -4\nHoskins, Rhys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.074 0.074 0.068 67.4 -37\nTurang, Brice 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.081 0.087 0.076 88.6 -32\nFrelick, Sal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.354 0.36 0.322 35.3 15'}, {'pitcher_id': '605154', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBrebbia, John 29 27 10 3 0 1 9 31 34 2 6.9 0.37 0.593 0.428 0.266 0.408 0.314 89.3 14.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 9 9 3 2 0 1 4 44.4 35.3 0 0 0.333 0.889 0.495 0.185 0.477 0.268 95.1 17.8\nHoskins, Rhys 7 7 2 0 0 0 2 28.6 27.3 0 0 0.286 0.286 0.253 0.219 0.311 0.226 82.7 11.2\nAdames, Willy 5 5 2 1 0 0 0 0 30 0 0 0.4 0.6 0.425 0.403 0.476 0.382 90.9 15.2\nBauers, Jake 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.444 0.21 0.227 0.189 91.4 4\nContreras, William 2 1 1 0 0 0 0 0 50 1 50 1 1 0.792 0.563 0.663 0.618 103.5 5\nTurang, Brice 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nOrtiz, Joey 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nSánchez, Gary 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.991 1.035 0.894 69.4 25\nChourio, Jackson 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '607200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFedde, Erick 82 68 25 4 0 5 11 13.4 14.9 13 15.9 0.368 0.647 0.473 0.31 0.524 0.412 89.2 9.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 35 29 9 1 0 3 2 5.7 13.9 6 17.1 0.31 0.655 0.454 0.313 0.548 0.422 87.9 13.6\nYelich, Christian 13 13 6 2 0 0 2 15.4 14.3 0 0 0.462 0.615 0.465 0.342 0.393 0.327 93 7.6\nContreras, William 10 8 5 0 0 2 1 10 13.3 2 20 0.625 1.375 0.811 0.535 1.373 0.777 101.3 13.3\nAdames, Willy 6 4 0 0 0 0 1 16.7 37.5 2 33.3 0 0 0.23 0.022 0.033 0.245 81.6 25\nBauers, Jake 4 3 0 0 0 0 2 50 20 1 25 0 0 0.173 0.023 0.024 0.189 88.3 -39\nPerkins, Blake 3 3 1 0 0 0 2 66.7 22.2 0 0 0.333 0.333 0.296 0.152 0.184 0.15 99.2 2\nFrelick, Sal 3 1 1 0 0 0 0 0 0 2 66.7 1 1 0.758 0.21 0.235 0.527 93.2 -10\nTurang, Brice 3 2 1 0 0 0 1 33.3 16.7 0 0 0.5 0.5 0.444 0.222 0.249 0.206 62.5 -15.5\nOrtiz, Joey 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.719 0.407 0.451 0.382 83.1 -16\nSánchez, Gary 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.393 0.397 0.354 83.4 21'}, {'pitcher_id': '605182', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nClevinger, Mike 17 15 3 1 0 1 8 47.1 33.3 2 11.8 0.2 0.467 0.328 0.161 0.306 0.257 86.5 18.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Gary 6 6 0 0 0 0 6 100 47.1 0 0 0 0 0 0\nBauers, Jake 6 5 2 1 0 0 1 16.7 12.5 1 16.7 0.4 0.6 0.47 0.327 0.425 0.386 87.9 34.8\nHoskins, Rhys 3 3 1 0 0 1 1 33.3 0 0 0 0.333 1.333 0.691 0.241 0.793 0.436 86.6 11\nYelich, Christian 2 1 0 0 0 0 0 0 60 1 50 0 0 0.344 0.06 0.083 0.375 80.4 -33'}, {'pitcher_id': '694363', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nShuster, Jared 10 9 5 2 0 0 1 10 11.8 1 10 0.556 0.778 0.59 0.479 0.547 0.473 93.3 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrelick, Sal 2 2 1 1 0 0 0 0 20 0 0 0.5 1 0.635 0.604 0.662 0.552 92.2 5.5\nContreras, William 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nPerkins, Blake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.037 0.116 0.061 91.5 37\nOrtiz, Joey 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nTurang, Brice 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.443 0.52 0.429 100.9 1\nYelich, Christian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.646 0.704 0.588 96.3 9\nAdames, Willy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.434 0.463 0.391 101.3 0\nDunn, Oliver 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.969 1.156 0.924 93.7 14\nBauers, Jake 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.574 0.636 0.54 78.6 13'}, {'pitcher_id': '621383', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBanks, Tanner 8 8 1 0 0 0 2 25 25 0 0 0.125 0.125 0.111 0.314 0.39 0.305 91.1 9.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBauers, Jake 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.193 0.332 0.217 94.9 18.5\nYelich, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.428 0.464 0.385 106.4 -4\nTurang, Brice 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFrelick, Sal 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.166 0.194 0.153 64.9 4\nAdames, Willy 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nContreras, William 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.583 0.698 0.566 102.6 4\nOrtiz, Joey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.951 1.096 0.901 83.1 15'}, {'pitcher_id': '680897', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNastrini, Nick 18 14 3 1 0 1 1 5.6 22.2 3 16.7 0.214 0.5 0.391 0.301 0.526 0.426 92.2 16.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 2 1 0 0 0 0 1 50 50 1 50 0 0 0.346 0.346\nBauers, Jake 2 1 1 1 0 0 0 0 50 1 50 1 2 0.981 0.363 0.755 0.581 97.9 22\nContreras, William 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.388 0.6 0.409 95.9 8.5\nYelich, Christian 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.196 0.224 0.181 87.4 34\nTurang, Brice 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.1 0.115 0.394 81.2 -15\nOrtiz, Joey 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.468 0.564 0.452 90 31\nFrelick, Sal 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.347 0.388 0.318 93.6 -2\nChourio, Jackson 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.043 0.382 1.111 0.608 93.9 16.5\nSánchez, Gary 2 1 0 0 0 0 0 0 0 0 0 0 0 0.362 0.197 0.724 0.549 97.5 37'}, {'pitcher_id': '673929', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeasure, Jordan 6 5 1 0 0 0 2 33.3 33.3 1 16.7 0.2 0.2 0.264 0.113 0.136 0.21 94.6 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nPerkins, Blake 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSánchez, Gary 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.009 0.007 92.3 44\nAdames, Willy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.369 0.458 0.381 101.1 -1\nFrelick, Sal 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHoskins, Rhys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.19 0.212 0.182 90.3 -4'}, {'pitcher_id': '647336', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoroka, Michael 33 30 10 2 0 1 11 33.3 29.7 3 9.1 0.333 0.5 0.386 0.274 0.418 0.332 86.5 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 9 8 1 0 0 0 7 77.8 31.8 1 11.1 0.125 0.125 0.173 0.017 0.019 0.09 90.3 -17\nAdames, Willy 6 5 2 0 0 1 1 16.7 50 1 16.7 0.4 1 0.596 0.368 0.748 0.503 87.5 11.5\nTurang, Brice 3 3 2 0 0 0 0 0 18.2 0 0 0.667 0.667 0.59 0.401 0.452 0.38 72.4 2.3\nPerkins, Blake 3 2 1 0 0 0 1 33.3 60 1 33.3 0.5 0.5 0.528 0.126 0.147 0.311 90.8 -5\nMonasterio, Andruw 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.155 0.274 0.178 92.8 47.3\nYelich, Christian 3 3 1 1 0 0 1 33.3 33.3 0 0 0.333 0.667 0.415 0.402 0.456 0.377 86.6 6.5\nFrelick, Sal 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.986 1.017 0.887 70.8 25\nContreras, William 3 3 2 1 0 0 0 0 25 0 0 0.667 1 0.709 0.59 1.076 0.704 93.4 21.3\nBauers, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.357 0.562 0.375 92.9 20'}]}, 146: {145: [{'pitcher_id': '676534', 'player_data': '', 'team_data': ''}, {'pitcher_id': '572143', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Burch 15 15 4 1 0 1 2 13.3 14.3 0 0 0.267 0.533 0.336 0.356 0.703 0.446 91.3 15.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 4 4 0 0 0 0 1 25 0 0 0 0 0 0 0.283 0.666 0.402 89 23.7\nPham, Tommy 4 4 1 1 0 0 0 0 28.6 0 0 0.25 0.5 0.312 0.37 0.491 0.366 95.7 1.5\nVaughn, Andrew 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.378 0.401 0.351 74.8 11\nBenintendi, Andrew 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.44 0.257 0.297 0.246 105.4 1\nDeJong, Paul 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.015 0.5 2 1 92.7 48.5\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.466 0.516 0.427 97.4 3'}, {'pitcher_id': '677960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeathers, Ryan 5 5 0 0 0 0 1 20 28.6 0 0 0 0 0 0.168 0.182 0.154 93.8 3.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.326 0.351 0.296 99.1 -12\nFedde, Erick 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.078 0.081 0.07 97.3 -18\nDeJong, Paul 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.034 0.046 0.037 79.5 57'}, {'pitcher_id': '669622', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBender, Anthony 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '666200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLuzardo, Jesús 31 27 3 1 0 2 10 32.3 29.4 3 9.7 0.111 0.37 0.259 0.205 0.459 0.331 82.8 15.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 12 11 1 0 0 1 6 50 50 1 8.3 0.091 0.364 0.22 0.159 0.351 0.251 72.7 36.4\nDeJong, Paul 7 4 1 1 0 0 2 28.6 21.4 2 28.6 0.25 0.5 0.483 0.211 0.515 0.479 97.2 -5\nFedde, Erick 5 5 0 0 0 0 1 20 10 0 0 0 0 0 0.193 0.204 0.174 85.5 10\nPham, Tommy 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.257 0.702 0.393 78.2 -20\nRobert Jr., Luis 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.176 0.213 0.17 72.4 37\nBenintendi, Andrew 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.003 0.431 1.46 0.764 100.2 39'}, {'pitcher_id': '664350', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '665795', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Edward 4 4 0 0 0 0 2 50 42.9 0 0 0 0 0 0.035 0.035 0.032 94.4 30.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.069 0.07 0.064 95.5 -21\nDeJong, Paul 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.001 0.001 0 93.3 82'}, {'pitcher_id': '676974', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '677053', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNardi, Andrew 1 1 1 1 0 0 0 0 50 0 0 1 2 1.244 0.403 0.917 0.526 99.4 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 1 1 1 1 0 0 0 0 50 0 0 1 2 1.244 0.403 0.917 0.526 99.4 23'}, {'pitcher_id': '682610', 'player_data': '', 'team_data': 'Roddery Muñoz has not faced anyone on this team.'}, {'pitcher_id': '666277', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '663773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoeing, Bryan 6 6 1 0 0 0 1 16.7 22.2 0 0 0.167 0.167 0.147 0.291 0.393 0.298 82.1 29.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 4 4 1 0 0 0 0 0 0 0 0 0.25 0.25 0.221 0.399 0.534 0.405 85 24.3\nMaldonado, Martín 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.146 0.22 0.167 70.6 51'}, {'pitcher_id': '642629', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '669432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Trevor 11 10 3 0 0 1 4 36.4 42.9 1 9.1 0.3 0.6 0.406 0.297 0.629 0.417 91.2 9.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 7 7 3 0 0 1 2 28.6 36.4 0 0 0.429 0.857 0.539 0.403 0.875 0.532 94.7 11\nDeJong, Paul 4 3 0 0 0 0 2 50 50 1 25 0 0 0.173 0.05 0.055 0.214 73.5 3'}, {'pitcher_id': '640462', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPuk, A.J. 14 13 2 0 0 0 8 57.1 44.8 0 0 0.154 0.154 0.136 0.127 0.145 0.121 75 -4.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVaughn, Andrew 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.295 0.096 0.102 0.088 87.9 -12\nBenintendi, Andrew 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.149 0.149 0.135 72.2 -12.5\nLopez, Nicky 2 1 0 0 0 0 1 50 16.7 0 0 0 0 0 0 0 0 31.7 8\nPham, Tommy 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRobert Jr., Luis 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLee, Korey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDeJong, Paul 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSheets, Gavin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.92 1.132 0.898 98 13'}, {'pitcher_id': '686539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronin, Declan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.167 0.153 96.6 -17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.165 0.167 0.153 96.6 -17'}, {'pitcher_id': '666129', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Braxton 29 26 8 1 0 0 7 24.1 32 2 6.9 0.308 0.346 0.329 0.235 0.293 0.28 86.5 4.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 13 13 5 0 0 0 4 30.8 17.6 0 0 0.385 0.385 0.339 0.257 0.275 0.237 86.8 -10.4\nMaldonado, Martín 4 3 1 0 0 0 1 25 50 1 25 0.333 0.333 0.395 0.128 0.331 0.318 73 36\nDeJong, Paul 4 2 0 0 0 0 0 0 60 1 25 0 0 0.354 0.067 0.084 0.385 81.5 27.5\nRobert Jr., Luis 2 2 1 1 0 0 1 50 42.9 0 0 0.5 1 0.622 0.018 0.048 0.024 101 45\nVaughn, Andrew 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.486 0.558 0.455 102.2 0\nLopez, Nicky 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.154 0.298 0.194 77.8 -5\nBenintendi, Andrew 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.474 0.54 0.449 91.5 13'}, {'pitcher_id': '681871', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tanner 23 18 6 0 0 0 9 39.1 40.9 3 13 0.333 0.333 0.383 0.228 0.267 0.323 86.2 4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 8 6 3 0 0 0 2 25 26.3 2 25 0.5 0.5 0.502 0.291 0.331 0.375 84.4 10.3\nPham, Tommy 6 4 1 0 0 0 1 16.7 16.7 1 16.7 0.25 0.25 0.383 0.319 0.407 0.447 91.3 -4\nLopez, Nicky 3 3 1 0 0 0 2 66.7 66.7 0 0 0.333 0.333 0.293 0.033 0.038 0.036 88.4 -12\nDeJong, Paul 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.294 0.328 0.357 0.303 75.8 20\nVaughn, Andrew 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nMendick, Danny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMaldonado, Martín 1 0 0 0 0 0 0 0 0 0 0.72 0.72'}, {'pitcher_id': '687424', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 147: [{'pitcher_id': '676534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFaucher, Calvin 16 13 3 0 0 1 3 18.8 24.1 3 18.8 0.231 0.462 0.369 0.245 0.449 0.369 87.8 10.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 3 3 1 0 0 0 0 0 12.5 0 0 0.333 0.333 0.294 0.37 0.428 0.343 89.1 3.3\nTorres, Gleyber 3 3 0 0 0 0 1 33.3 14.3 0 0 0 0 0 0.003 0.005 0.003 90.7 61.5\nStanton, Giancarlo 3 1 0 0 0 0 0 0 0 2 66.7 0 0 0.459 0.646 0.783 0.67 81.8 22\nRizzo, Anthony 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.442 0.202 0.311 0.213 96.6 18\nCabrera, Oswaldo 2 2 1 0 0 1 1 50 50 0 0 0.5 2 1.036 0.37 1.423 0.739 103.8 30\nJudge, Aaron 1 0 0 0 0 0 0 0 66.7 1 100 0.689 0.689\nLeMahieu, DJ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.121 0.121 0.112 76.4 -45\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.159 0.149 71.2 -51'}, {'pitcher_id': '572143', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Burch 28 28 8 1 0 0 5 17.9 20 0 0 0.286 0.321 0.265 0.281 0.387 0.286 98.1 7.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTorres, Gleyber 4 4 0 0 0 0 2 50 30.8 0 0 0 0 0 0.209 0.357 0.235 99.5 17.5\nStanton, Giancarlo 4 4 1 0 0 0 0 0 27.3 0 0 0.25 0.25 0.22 0.398 0.542 0.405 103.4 6.3\nLeMahieu, DJ 5 5 3 1 0 0 1 20 8.3 0 0 0.6 0.8 0.608 0.322 0.355 0.292 98.5 -9.5\nJudge, Aaron 4 4 2 0 0 0 1 25 20 0 0 0.5 0.5 0.44 0.4 0.493 0.389 100.2 -6.3\nTrevino, Jose 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0.113 0.233 0.143 91.4 14\nRizzo, Anthony 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.293 0.388 0.429 0.357 96.5 -5\nGrisham, Trent 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.358 0.575 0.392 100.2 20.5\nSoto, Juan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nVerdugo, Alex 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.009 0.018 0.012 94.8 46'}, {'pitcher_id': '677960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeathers, Ryan 30 26 10 0 0 1 3 10 18.2 4 13.3 0.385 0.5 0.424 0.328 0.583 0.428 95.1 16.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 8 8 4 0 0 1 1 12.5 26.7 0 0 0.5 0.875 0.582 0.41 1.064 0.613 94.8 18.4\nRizzo, Anthony 7 7 5 0 0 0 0 0 15.4 0 0 0.714 0.714 0.629 0.482 0.619 0.477 100.4 21\nTrevino, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.151 0.288 0.178 94.5 30\nVolpe, Anthony 3 3 1 0 0 0 0 0 14.3 0 0 0.333 0.333 0.296 0.255 0.284 0.237 86.2 -4\nGrisham, Trent 2 1 0 0 0 0 1 50 66.7 1 50 0 0 0.348 0.348\nTorres, Gleyber 2 0 0 0 0 0 0 0 0 2 100 0.692 0.692\nVerdugo, Alex 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.119 0.13 0.107 78.5 5\nJudge, Aaron 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.411 0.46 0.542 105.1 -2\nStanton, Giancarlo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.084 0.089 0.076 94.6 22.5'}, {'pitcher_id': '669622', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBender, Anthony 15 14 2 1 0 0 5 33.3 24 1 6.7 0.143 0.214 0.189 0.206 0.276 0.239 96 -1.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrisham, Trent 4 4 0 0 0 0 2 50 37.5 0 0 0 0 0 0.166 0.22 0.167 94.2 21\nSoto, Juan 4 4 2 1 0 0 0 0 0 0 0 0.5 0.75 0.537 0.47 0.652 0.48 102.6 1.8\nTorres, Gleyber 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.031 0.032 0.029 77.9 -30\nVolpe, Anthony 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nVerdugo, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.092 0.103 0.083 91.4 -26\nJudge, Aaron 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nRizzo, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.184 0.211 0.167 96 -9\nWells, Austin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '666200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLuzardo, Jesús 63 55 19 5 0 4 13 20.6 23.3 7 11.1 0.345 0.655 0.458 0.291 0.496 0.381 90.6 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 16 13 5 1 0 1 2 12.5 21.7 2 12.5 0.385 0.692 0.506 0.347 0.552 0.445 92.6 13.2\nTrevino, Jose 8 6 1 1 0 0 1 12.5 14.3 2 25 0.167 0.333 0.325 0.165 0.22 0.296 83.3 29.8\nVolpe, Anthony 5 4 2 0 0 2 2 40 30 1 20 0.5 2 0.956 0.35 1.333 0.685 103.7 31.5\nGrisham, Trent 6 6 2 1 0 0 2 33.3 28.6 0 0 0.333 0.5 0.353 0.338 0.408 0.323 92.7 -0.5\nJudge, Aaron 5 4 1 0 0 1 0 0 14.3 1 20 0.25 1 0.54 0.299 1.065 0.579 96.6 32.8\nVerdugo, Alex 6 5 3 1 0 0 1 16.7 37.5 1 16.7 0.6 0.8 0.622 0.401 0.464 0.429 87.5 -3\nTorres, Gleyber 5 5 0 0 0 0 1 20 22.2 0 0 0 0 0 0.119 0.153 0.117 93 33.3\nRizzo, Anthony 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.296 0.029 0.032 0.026 78.7 -21\nStanton, Giancarlo 5 5 3 1 0 0 0 0 0 0 0 0.6 0.8 0.608 0.516 0.57 0.482 85.6 -11\nStroman, Marcus 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nCabrera, Oswaldo 2 2 1 0 0 0 1 50 25 0 0 0.5 0.5 0.441 0.307 0.36 0.299 98.2 5'}, {'pitcher_id': '664350', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Sixto 13 12 2 0 0 0 2 15.4 13.6 1 7.7 0.167 0.167 0.19 0.211 0.262 0.245 93 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 6 6 2 0 0 0 0 0 12.5 0 0 0.333 0.333 0.295 0.339 0.37 0.318 96.2 -6.7\nRizzo, Anthony 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.091 0.149 0.097 92.7 21\nVolpe, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.014 0.013 82.6 52\nWells, Austin 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nTorres, Gleyber 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.109 0.334 0.185 93.2 30\nVerdugo, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.107 0.125 0.1 84.3 -8'}, {'pitcher_id': '665795', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Edward 10 8 1 0 0 0 1 10 8.3 1 10 0.125 0.125 0.229 0.141 0.159 0.247 79.6 7.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 8 6 0 0 0 0 1 12.5 11.1 1 12.5 0 0 0.176 0.139 0.158 0.275 84.3 0.6\nGrisham, Trent 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.145 0.164 0.136 67.9 26'}, {'pitcher_id': '676974', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '677053', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNardi, Andrew 8 7 1 0 0 0 3 37.5 21.1 1 12.5 0.143 0.143 0.197 0.17 0.338 0.273 93.6 13.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.057 0.068 0.051 87.1 25\nSoto, Juan 2 1 0 0 0 0 1 50 50 1 50 0 0 0.344 0.344\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.397 1.427 0.744 99 31\nGrisham, Trent 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nVolpe, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.22 0.252 0.201 89.6 -5\nRizzo, Anthony 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.457 0.552 0.451 98.6 2'}, {'pitcher_id': '682610', 'player_data': '', 'team_data': 'Roddery Muñoz has not faced anyone on this team.'}, {'pitcher_id': '666277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoriano, George 11 9 2 1 0 0 4 36.4 43.8 2 18.2 0.222 0.333 0.319 0.286 0.395 0.366 95.8 5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 3 2 0 0 0 0 2 66.7 50 1 33.3 0 0 0.231 0.231\nVolpe, Anthony 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.006 0.011 0.351 87.5 41\nStanton, Giancarlo 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.316 0.417 0.316 92.8 -19.5\nTorres, Gleyber 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.063 0.808 1.147 0.843 102.9 13\nCabrera, Oswaldo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '663773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoeing, Bryan 9 8 2 0 0 1 3 33.3 20 1 11.1 0.25 0.625 0.407 0.264 0.646 0.412 96.4 13.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.173 0.186 0.156 100.1 -12.5\nGrisham, Trent 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.56 0.593 0.6 93.4 8\nTorres, Gleyber 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nRizzo, Anthony 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.043 0.05 0.039 77.7 56\nJudge, Aaron 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nStanton, Giancarlo 1 1 1 0 0 1 0 0 0 0 0 1 4 2.085 0.991 3.963 2.002 110.5 30'}, {'pitcher_id': '642629', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '669432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Trevor 37 28 8 2 0 0 5 13.5 20.6 8 21.6 0.286 0.357 0.379 0.314 0.417 0.409 90.3 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 16 11 3 0 0 0 2 12.5 22.2 5 31.3 0.273 0.273 0.381 0.319 0.419 0.436 84 2\nVerdugo, Alex 5 5 2 0 0 0 1 20 14.3 0 0 0.4 0.4 0.353 0.219 0.249 0.206 77.5 7.3\nStanton, Giancarlo 3 2 1 1 0 0 0 0 0 1 33.3 0.5 1 0.644 0.649 1.187 0.749 111.8 7.5\nJudge, Aaron 4 3 0 0 0 0 1 25 25 1 25 0 0 0.175 0.154 0.162 0.279 107.7 23\nTorres, Gleyber 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.137 0.212 0.155 100 21\nLeMahieu, DJ 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.06 0.78 0.934 0.738 102.7 10.5\nRizzo, Anthony 2 0 0 0 0 0 0 0 0 1 50 0.707 0.707\nGrisham, Trent 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.222 0.235 0.208 83.3 -26'}, {'pitcher_id': '640462', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPuk, A.J. 38 30 5 2 0 1 6 15.8 19.6 5 13.2 0.167 0.333 0.305 0.219 0.364 0.334 88.2 12.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 5 3 1 1 0 0 1 20 33.3 2 40 0.333 0.667 0.531 0.306 0.431 0.468 101.3 16\nStanton, Giancarlo 5 4 1 1 0 0 0 0 0 0 0 0.25 0.5 0.317 0.395 0.732 0.472 86.6 0\nRizzo, Anthony 6 4 2 0 0 0 1 16.7 14.3 0 0 0.5 0.5 0.535 0.452 0.546 0.531 88.7 5\nSoto, Juan 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.203 0.504 0.392 92.4 -9.7\nTorres, Gleyber 4 4 0 0 0 0 0 0 16.7 0 0 0 0 0 0.013 0.026 0.017 85.6 40.3\nLeMahieu, DJ 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.23 0.162 0.169 0.328 107.2 -12\nVolpe, Anthony 3 3 0 0 0 0 1 33.3 16.7 0 0 0 0 0 0.136 0.283 0.179 99.5 9\nVerdugo, Alex 3 2 1 0 0 1 0 0 25 1 33.3 0.5 2 0.926 0.226 0.605 0.464 77.4 -6.5\nWells, Austin 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.122 0.138 0.111 71.2 9\nTrevino, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.09 0.118 0.084 75.1 57.5\nCabrera, Oswaldo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '686539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronin, Declan 5 4 2 0 0 0 2 40 22.2 1 20 0.5 0.5 0.492 0.447 0.525 0.48 90.4 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nVolpe, Anthony 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.917 1.021 0.852 86.8 17\nTorres, Gleyber 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nStanton, Giancarlo 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nGrisham, Trent 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.869 1.081 0.853 93.9 15'}, {'pitcher_id': '666129', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Braxton 13 11 4 0 0 0 3 23.1 28.6 1 7.7 0.364 0.364 0.38 0.374 0.446 0.413 93.5 0.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrisham, Trent 7 6 3 0 0 0 1 14.3 22.2 0 0 0.5 0.5 0.48 0.577 0.703 0.582 95.2 10.4\nSoto, Juan 4 3 1 0 0 0 1 25 33.3 1 25 0.333 0.333 0.395 0.2 0.212 0.311 104 -8.5\nVerdugo, Alex 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.026 0.026 0.025 63.9 -31'}, {'pitcher_id': '681871', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tanner 55 44 11 2 0 1 10 18.2 30.9 11 20 0.25 0.364 0.348 0.257 0.408 0.367 92.1 17.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 13 10 4 1 0 0 1 7.7 31.6 3 23.1 0.4 0.5 0.459 0.356 0.499 0.446 92.7 23.7\nJudge, Aaron 9 6 2 0 0 1 3 33.3 31.3 3 33.3 0.333 0.833 0.554 0.279 0.798 0.523 102.1 24.7\nStanton, Giancarlo 10 9 1 0 0 0 3 30 33.3 1 10 0.111 0.111 0.157 0.191 0.222 0.238 93.7 16\nVerdugo, Alex 7 6 0 0 0 0 1 14.3 25 1 14.3 0 0 0.1 0.169 0.292 0.265 84.9 4.4\nLeMahieu, DJ 5 4 2 0 0 0 1 20 50 1 20 0.5 0.5 0.49 0.325 0.371 0.384 89.9 14.3\nTorres, Gleyber 5 5 1 1 0 0 1 20 40 0 0 0.2 0.4 0.248 0.28 0.439 0.306 95.4 7.2\nTrevino, Jose 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.435 0.271 0.308 0.256 93.3 38\nRizzo, Anthony 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.231 0.054 0.064 0.267 83.6 20\nVolpe, Anthony 1 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '687424', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}], 158: [{'pitcher_id': '676534', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFaucher, Calvin 5 4 1 0 0 0 1 20 37.5 1 20 0.25 0.25 0.316 0.277 0.316 0.344 90.8 -5.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nPerkins, Blake 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.469 0.538 0.433 94.9 2\nTurang, Brice 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nContreras, William 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.586 0.669 0.548 91.8 9\nAdames, Willy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.052 0.056 0.047 85.6 -28'}, {'pitcher_id': '572143', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSmith, Burch 12 11 4 0 0 1 3 25 35 1 8.3 0.364 0.636 0.448 0.297 0.413 0.34 89.3 10.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Gary 3 2 1 0 0 1 0 0 0 1 33.3 0.5 2 0.908 0.334 0.762 0.535 95.8 39\nBauers, Jake 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.065 0.066 0.059 91.4 -17\nAdames, Willy 3 3 2 0 0 0 1 33.3 33.3 0 0 0.667 0.667 0.586 0.336 0.385 0.324 79.6 -1\nYelich, Christian 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.607 0.762 0.599 78.1 24\nOrtiz, Joey 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.446 0.527 0.425 93.5 3\nChourio, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.338 0.377 0.307 100.3 -5'}, {'pitcher_id': '677960', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeathers, Ryan 24 20 3 1 0 2 7 29.2 33.3 3 12.5 0.15 0.5 0.339 0.174 0.46 0.334 87.2 3.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 5 5 1 1 0 0 2 40 20 0 0 0.2 0.4 0.248 0.195 0.357 0.231 93.3 10.3\nSánchez, Gary 3 3 0 0 0 0 1 33.3 40 0 0 0 0 0 0.051 0.052 0.046 75.5 -30.5\nOrtiz, Joey 3 1 0 0 0 0 1 33.3 100 1 33.3 0 0 0.472 0.472\nContreras, William 3 3 0 0 0 0 2 66.7 42.9 0 0 0 0 0 0.057 0.061 0.051 110.3 -18\nMonasterio, Andruw 3 3 1 0 0 1 1 33.3 20 0 0 0.333 1.333 0.695 0.243 0.879 0.456 94.7 30\nYelich, Christian 3 2 0 0 0 0 0 0 33.3 1 33.3 0 0 0.231 0.064 0.077 0.272 89 18\nChourio, Jackson 3 2 0 0 0 0 0 0 50 1 33.3 0 0 0.231 0.179 0.209 0.343 57.9 -15.5\nHoskins, Rhys 1 1 1 0 0 1 0 0 0 0 0 1 4 2.007 0.977 3.874 1.947 109 27'}, {'pitcher_id': '669622', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBender, Anthony 8 7 0 0 0 0 3 37.5 62.5 0 0 0 0 0 0.181 0.31 0.187 87 22.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 3 2 0 0 0 0 0 0 25 0 0 0 0 0 0.157 0.17 0.106 75.9 30.7\nContreras, William 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nSánchez, Gary 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.471 1.321 0.733 97.5 27\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.483 0.511 0.447 109.8 -5\nChourio, Jackson 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '666200', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLuzardo, Jesús 64 59 13 3 0 1 14 21.9 30.4 5 7.8 0.22 0.322 0.268 0.231 0.373 0.292 87.2 24.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 14 12 1 0 0 0 5 35.7 35.5 2 14.3 0.083 0.083 0.163 0.136 0.179 0.215 90.8 27.6\nAdames, Willy 9 9 3 2 0 0 2 22.2 43.8 0 0 0.333 0.556 0.375 0.388 0.689 0.459 88.7 24.7\nMonasterio, Andruw 8 7 1 0 0 0 0 0 10 1 12.5 0.143 0.143 0.197 0.197 0.225 0.247 89.1 24.1\nYelich, Christian 6 6 1 1 0 0 2 33.3 35.7 0 0 0.167 0.333 0.207 0.191 0.349 0.228 85 -5.3\nSánchez, Gary 7 7 3 0 0 1 1 14.3 20 0 0 0.429 0.857 0.54 0.274 0.426 0.295 95.7 35.5\nFrelick, Sal 5 5 3 0 0 0 1 20 11.1 0 0 0.6 0.6 0.53 0.406 0.481 0.389 81.5 -7.8\nHoskins, Rhys 3 2 0 0 0 0 1 33.3 57.1 1 33.3 0 0 0.23 0.197 0.217 0.348 89.7 5\nPerkins, Blake 4 3 0 0 0 0 1 25 25 1 25 0 0 0.174 0.03 0.034 0.195 51.2 61\nChourio, Jackson 3 3 0 0 0 0 1 33.3 37.5 0 0 0 0 0 0.103 0.114 0.095 95.4 22\nOrtiz, Joey 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.014 0.016 0.012 76.1 60\nBauers, Jake 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.607 1.837 1.012 96.1 19'}, {'pitcher_id': '664350', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Sixto 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.152 0.183 0.144 90.8 7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 3 3 1 0 0 0 1 33.3 40 0 0 0.333 0.333 0.294 0.152 0.183 0.144 90.8 7'}, {'pitcher_id': '665795', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCabrera, Edward 34 29 4 1 0 0 9 26.5 32.8 4 11.8 0.138 0.172 0.218 0.113 0.148 0.199 83.7 -0.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 7 7 3 1 0 0 1 14.3 30 0 0 0.429 0.571 0.43 0.178 0.252 0.183 94.6 27.3\nAdames, Willy 6 3 0 0 0 0 2 33.3 42.9 3 50 0 0 0.348 0.091 0.106 0.39 89.2 -1\nYelich, Christian 4 4 0 0 0 0 2 50 40 0 0 0 0 0 0.053 0.06 0.05 79.2 -10\nHoskins, Rhys 5 4 0 0 0 0 1 20 50 0 0 0 0 0.144 0.064 0.064 0.191 88 4.3\nMonasterio, Andruw 2 1 0 0 0 0 1 50 37.5 1 50 0 0 0.348 0.348\nFrelick, Sal 4 4 0 0 0 0 0 0 14.3 0 0 0 0 0 0.152 0.161 0.141 77 -31.5\nTurang, Brice 4 4 1 0 0 0 1 25 25 0 0 0.25 0.25 0.221 0.118 0.218 0.136 86.1 -11\nWilson, Bryse 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.105 0.105 0.098 28.2 1'}, {'pitcher_id': '676974', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMeyer, Max 3 3 1 0 0 1 1 33.3 12.5 0 0 0.333 1.333 0.691 0.378 1.328 0.713 101.8 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 3 3 1 0 0 1 1 33.3 12.5 0 0 0.333 1.333 0.691 0.378 1.328 0.713 101.8 8.5'}, {'pitcher_id': '677053', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nNardi, Andrew 11 9 3 0 0 1 3 27.3 22.7 2 18.2 0.333 0.667 0.474 0.235 0.506 0.38 95.5 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 3 2 1 0 0 0 0 0 11.1 1 33.3 0.5 0.5 0.524 0.466 0.888 0.608 99.7 22\nAdames, Willy 4 4 1 0 0 0 2 50 33.3 0 0 0.25 0.25 0.221 0.145 0.155 0.132 95.8 -12.5\nFrelick, Sal 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.025 0.044 0.033 80.3 56\nPerkins, Blake 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nYelich, Christian 1 1 1 0 0 1 0 0 0 0 0 1 4 2.072 0.549 2.067 1.073 101.6 32'}, {'pitcher_id': '682610', 'player_data': '', 'team_data': 'Roddery Muñoz has not faced anyone on this team.'}, {'pitcher_id': '666277', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoriano, George 14 9 2 0 0 1 2 14.3 33.3 4 28.6 0.222 0.556 0.405 0.405 0.776 0.521 83 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 3 1 0 0 0 0 1 33.3 75 1 33.3 0 0 0.232 0 0 0.237 86.2 37\nAdames, Willy 3 1 0 0 0 0 0 0 50 2 66.7 0 0 0.464 0.651 0.725 0.662 96.2 9\nBauers, Jake 2 1 0 0 0 0 1 50 50 1 50 0 0 0.348 0.348\nYelich, Christian 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 1 4 2 106.8 25\nTurang, Brice 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.641 0.712 0.613 67.9 26.5\nFrelick, Sal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.129 0.176 0.121 87.6 -14\nPerkins, Blake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.474 0.537 0.449 75 29\nMonasterio, Andruw 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.111 0.127 0.105 76.1 45'}, {'pitcher_id': '663773', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoeing, Bryan 17 15 7 2 0 1 2 11.8 10 2 11.8 0.467 0.8 0.553 0.425 0.744 0.521 88.8 14.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 4 3 1 0 0 0 1 25 10 1 25 0.333 0.333 0.395 0.51 0.547 0.525 90.7 11.5\nContreras, William 4 3 1 0 0 0 0 0 0 1 25 0.333 0.333 0.393 0.106 0.137 0.251 90.3 24.3\nPerkins, Blake 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.622 0.501 0.614 0.49 86.2 27.5\nYelich, Christian 2 2 2 1 0 1 0 0 0 0 0 1 3 1.624 0.865 2.944 1.541 105 23.5\nSánchez, Gary 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFrelick, Sal 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.498 0.55 0.474 66.2 0.5\nTurang, Brice 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.398 0.444 0.37 94 -3'}, {'pitcher_id': '642629', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRamirez, Emmanuel 4 4 0 0 0 0 1 25 20 0 0 0 0 0 0.181 0.284 0.193 97 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBauers, Jake 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nSánchez, Gary 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.254 0.294 0.23 94 -5\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 93.6 46\nChourio, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.463 0.838 0.537 103.4 17'}, {'pitcher_id': '669432', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRogers, Trevor 55 48 17 6 0 4 15 27.3 23.6 6 10.9 0.354 0.729 0.486 0.299 0.605 0.422 92.7 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 22 19 8 2 0 3 4 18.2 19.6 2 9.1 0.421 1 0.605 0.367 0.908 0.553 92.7 17.5\nAdames, Willy 8 8 2 1 0 1 5 62.5 31.3 0 0 0.25 0.75 0.418 0.235 0.591 0.35 87.4 12.3\nYelich, Christian 6 5 1 0 0 0 2 33.3 40 1 16.7 0.2 0.2 0.263 0.269 0.307 0.329 99.3 -1.3\nSánchez, Gary 5 4 1 1 0 0 1 20 12.5 1 20 0.25 0.5 0.387 0.145 0.182 0.249 97.7 26.7\nContreras, William 5 5 3 2 0 0 1 20 37.5 0 0 0.6 1 0.684 0.302 0.477 0.332 93.1 9.5\nMonasterio, Andruw 3 2 1 0 0 0 0 0 0 1 33.3 0.5 0.5 0.527 0.363 0.428 0.467 77.8 -22\nPerkins, Blake 2 1 1 0 0 0 0 0 0 1 50 1 1 0.791 0.609 0.727 0.641 106.2 5\nOrtiz, Joey 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.01 0.012 0.009 79.6 39\nChourio, Jackson 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.355 0.395 0.325 101.5 8'}, {'pitcher_id': '640462', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPuk, A.J. 12 11 4 3 1 0 4 33.3 20 1 8.3 0.364 0.818 0.507 0.252 0.433 0.325 89.5 12.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFrelick, Sal 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.046 0.049 0.045 52 -26\nBauers, Jake 2 2 1 1 0 0 1 50 50 0 0 0.5 1 0.635 0.189 0.35 0.222 99 20\nAdames, Willy 2 2 1 1 0 0 1 50 40 0 0 0.5 1 0.635 0.407 0.507 0.402 102.7 10\nMonasterio, Andruw 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.514 1.738 0.916 99.2 28\nTurang, Brice 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.004 0.006 0.005 77.2 63\nContreras, William 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nYelich, Christian 1 1 1 0 1 0 0 0 0 0 0 1 3 1.612 0.837 1.055 0.821 111 9\nPerkins, Blake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.131 0.145 0.122 85.7 -15'}, {'pitcher_id': '686539', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCronin, Declan 14 11 4 1 0 0 2 14.3 27.3 2 14.3 0.364 0.455 0.431 0.362 0.477 0.435 88.4 6.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurang, Brice 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nSánchez, Gary 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nContreras, William 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.175 0.433 0.255 88.2 38\nFrelick, Sal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.156 0.16 0.146 73.6 -44\nOrtiz, Joey 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nChourio, Jackson 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.093 0.078 52.4 -15\nMonasterio, Andruw 2 1 1 0 0 0 0 0 0 0 0 1 1 0.807 0.834 1.1 0.786 106.8 10\nAdames, Willy 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBauers, Jake 2 2 2 1 0 0 0 0 33.3 0 0 1 1.5 1.076 0.789 0.945 0.746 92.3 18\nYelich, Christian 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.486 0.568 0.456 100.9 -3.5'}, {'pitcher_id': '666129', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGarrett, Braxton 28 27 11 3 0 1 6 21.4 22.4 1 3.6 0.407 0.63 0.451 0.296 0.516 0.357 91.4 5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 9 8 2 0 0 0 2 22.2 21.4 1 11.1 0.25 0.25 0.274 0.183 0.207 0.229 90.7 -3\nAdames, Willy 5 5 2 0 0 0 2 40 33.3 0 0 0.4 0.4 0.353 0.365 0.433 0.354 79.1 11.3\nHoskins, Rhys 5 5 2 2 0 0 1 20 37.5 0 0 0.4 0.8 0.501 0.24 0.523 0.315 98.6 13.3\nMonasterio, Andruw 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.415 0.209 0.676 0.355 88 7\nSánchez, Gary 2 2 1 0 0 1 0 0 33.3 0 0 0.5 2 1.002 0.539 1.722 0.913 102 2.5\nYelich, Christian 2 2 2 0 0 0 0 0 0 0 0 1 1 0.884 0.476 0.532 0.442 96.1 4.5\nFrelick, Sal 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.425 0.473 0.405 86.2 10.5'}, {'pitcher_id': '681871', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '656945', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nScott, Tanner 31 25 3 0 0 0 13 41.9 39.4 4 12.9 0.12 0.12 0.187 0.187 0.24 0.257 86.5 5.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 8 5 1 0 0 0 3 37.5 60 3 37.5 0.2 0.2 0.369 0.282 0.353 0.432 100.7 9.5\nAdames, Willy 5 4 0 0 0 0 3 60 50 1 20 0 0 0.138 0.009 0.024 0.149 90.7 35\nSánchez, Gary 5 5 0 0 0 0 2 40 20 0 0 0 0 0 0.213 0.242 0.202 84.3 6.3\nMonasterio, Andruw 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.323 0.541 0.375 101.5 12.5\nBauers, Jake 4 3 0 0 0 0 2 50 25 0 0 0 0 0 0.244 0.276 0.228 85.1 12\nContreras, William 2 2 2 0 0 0 0 0 25 0 0 1 1 0.886 0.217 0.217 0.197 85.4 -23.5\nHoskins, Rhys 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFrelick, Sal 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.029 0.057 0.037 81.8 36\nPerkins, Blake 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nOrtiz, Joey 1 0 0 0 0 0 0 0 0 0 0 38.4 -30'}, {'pitcher_id': '687424', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}]}, 147: {145: [{'pitcher_id': '543859', 'player_data': '', 'team_data': ''}, {'pitcher_id': '664776', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCousins, Jake 7 6 1 0 0 0 1 14.3 38.5 1 14.3 0.167 0.167 0.225 0.306 0.347 0.347 94.3 -3.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 2 1 1 0 0 0 0 0 0 1 50 1 1 0.789 0.666 0.75 0.66 97.6 7\nKopech, Michael 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.1 0.114 0.1 80.4 -41\nDeJong, Paul 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.429 0.502 0.413 100.3 0\nBenintendi, Andrew 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.387 0.389 0.352 108.1 -8\nSheets, Gavin 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.254 0.328 0.241 84.9 23'}, {'pitcher_id': '690925', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '677076', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '573186', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStroman, Marcus 74 64 19 3 0 2 15 20.3 18.3 8 10.8 0.297 0.438 0.359 0.272 0.427 0.348 88 2.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 21 17 7 1 0 0 2 9.5 10.7 4 19 0.412 0.471 0.441 0.315 0.375 0.373 85 -2.9\nPham, Tommy 14 9 5 1 0 0 3 21.4 27.8 4 28.6 0.556 0.667 0.589 0.247 0.359 0.419 100.1 1.7\nDeJong, Paul 15 15 1 0 0 1 4 26.7 30.8 0 0 0.067 0.267 0.134 0.271 0.552 0.347 91.3 8.2\nMaldonado, Martín 5 5 1 0 0 1 3 60 16.7 0 0 0.2 0.8 0.388 0.177 0.64 0.324 99 40.5\nVaughn, Andrew 5 5 1 0 0 0 1 20 22.2 0 0 0.2 0.2 0.177 0.129 0.156 0.117 93.3 -13.5\nKuhl, Chad 2 1 0 0 0 0 1 50 0 0 0 0 0 0 0 0 0 26.9 8\nColás, Oscar 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.553 0.626 0.518 94.1 -1\nLopez, Nicky 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.435 0\nSheets, Gavin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.416 0.527 0.412 85.2 32\nRobert Jr., Luis 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.415 0.422 0.632 0.447 100.5 0\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.271 0.271 0.244 22.9 -19\nFedde, Erick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.052 0.058 0.047 64.4 -28'}, {'pitcher_id': '607074', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodón, Carlos 70 62 9 1 1 1 19 27.1 32.3 6 8.6 0.145 0.242 0.215 0.194 0.356 0.287 89.3 20.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 15 14 0 0 0 0 5 33.3 31.3 1 6.7 0 0 0.046 0.095 0.213 0.165 87.7 25.1\nPham, Tommy 15 11 2 0 0 0 4 26.7 30.4 3 20 0.182 0.182 0.255 0.151 0.228 0.312 97.9 23.6\nBenintendi, Andrew 14 13 0 0 0 0 3 21.4 29.6 1 7.1 0 0 0.049 0.153 0.206 0.194 86.1 17.2\nLopez, Nicky 7 7 3 0 1 0 1 14.3 6.3 0 0 0.429 0.714 0.475 0.318 0.569 0.372 91.7 11.2\nDeJong, Paul 6 5 1 0 0 0 2 33.3 54.5 0 0 0.2 0.2 0.267 0.141 0.159 0.229 76.2 6\nMendick, Danny 4 4 1 1 0 0 1 25 28.6 0 0 0.25 0.5 0.311 0.325 0.4 0.318 82.1 35.3\nVaughn, Andrew 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.388 0.977 0.567 91.2 26\nLee, Korey 3 2 0 0 0 0 2 66.7 75 1 33.3 0 0 0.231 0.231\nJulks, Corey 3 3 2 0 0 1 1 33.3 50 0 0 0.667 1.667 0.991 0.555 1.536 0.861 98 24'}, {'pitcher_id': '592454', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKahnle, Tommy 21 19 5 1 0 1 5 23.8 38.9 1 4.8 0.263 0.474 0.312 0.283 0.528 0.344 93.4 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 9 7 2 0 0 0 2 22.2 36.4 1 11.1 0.286 0.286 0.271 0.276 0.44 0.322 87 11.8\nPham, Tommy 4 4 0 0 0 0 1 25 54.5 0 0 0 0 0 0.071 0.159 0.091 91.9 22.7\nVaughn, Andrew 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.092 0.101 0.094 92.8 -27\nRobert Jr., Luis 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.622 0.483 0.525 0.447 99.3 -1.5\nMaldonado, Martín 2 2 1 0 0 1 0 0 0 0 0 0.5 2 0.99 0.574 1.987 1.022 105.1 29.5\nDeJong, Paul 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nSheets, Gavin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.954 1.198 0.937 102 12'}, {'pitcher_id': '641482', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCortes, Nestor 87 80 13 3 0 1 24 27.6 23.2 7 8 0.163 0.237 0.213 0.198 0.327 0.26 91.6 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 19 15 3 0 0 0 3 15.8 7.1 4 21.1 0.2 0.2 0.285 0.288 0.344 0.359 90.2 14.2\nVaughn, Andrew 12 12 3 0 0 1 2 16.7 7.7 0 0 0.25 0.5 0.315 0.253 0.534 0.331 93 28.2\nMaldonado, Martín 11 10 1 1 0 0 4 36.4 40.9 1 9.1 0.1 0.2 0.175 0.173 0.401 0.277 86.4 20.5\nBenintendi, Andrew 13 13 2 1 0 0 3 23.1 25 0 0 0.154 0.231 0.164 0.204 0.358 0.236 93.8 17.5\nRobert Jr., Luis 9 9 1 0 0 0 5 55.6 41.7 0 0 0.111 0.111 0.098 0.157 0.214 0.16 96.9 14.5\nLopez, Nicky 10 9 1 0 0 0 3 30 17.6 1 10 0.111 0.111 0.157 0.188 0.257 0.242 92.2 24\nDeJong, Paul 5 4 0 0 0 0 2 40 27.3 1 20 0 0 0.138 0.014 0.048 0.158 96 53\nMendick, Danny 5 5 1 0 0 0 0 0 16.7 0 0 0.2 0.2 0.176 0.133 0.148 0.124 86.6 18.4\nJulks, Corey 3 3 1 1 0 0 2 66.7 50 0 0 0.333 0.667 0.423 0.096 0.25 0.144 96.4 27'}, {'pitcher_id': '641656', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHamilton, Ian 8 7 4 2 0 0 1 12.5 23.1 1 12.5 0.571 0.857 0.626 0.372 0.503 0.414 94.5 20', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 2 2 1 1 0 0 0 0 25 0 0 0.5 1 0.635 0.304 0.476 0.33 88.3 26\nSheets, Gavin 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.903 1.191 0.896 105.6 12\nRobert Jr., Luis 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nMaldonado, Martín 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nVaughn, Andrew 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.651 0.89 0.649 98.5 16\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.017 0.036 0.021 91.3 39\nLopez, Nicky 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.425 0.449 0.388 95 1'}, {'pitcher_id': '595897', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBurdi, Nick 9 8 1 0 0 0 2 22.2 21.4 1 11.1 0.125 0.125 0.176 0.14 0.187 0.202 87.6 36.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.006 0.006 0.006 81.2 40\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.069 0.265 0.135 96 38\nSheets, Gavin 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.974 1.061 0.894 76.7 21\nLee, Korey 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nMaldonado, Martín 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nVaughn, Andrew 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.009 0.017 0.01 84.5 41\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.06 0.034 91.8 39\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.078 0.04 95.3 42'}, {'pitcher_id': '661563', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGil, Luis 25 23 5 2 0 0 14 56 44.9 2 8 0.217 0.304 0.263 0.16 0.228 0.21 83.1 11.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSheets, Gavin 5 4 1 0 0 0 1 20 33.3 1 20 0.25 0.25 0.316 0.228 0.256 0.309 68.9 -2.3\nPham, Tommy 3 3 2 1 0 0 1 33.3 42.9 0 0 0.667 1 0.719 0.443 0.89 0.563 96.4 19.5\nDeJong, Paul 3 2 0 0 0 0 2 66.7 40 1 33.3 0 0 0.231 0.231\nVaughn, Andrew 3 3 0 0 0 0 3 100 66.7 0 0 0 0 0 0\nMendick, Danny 2 2 0 0 0 0 2 100 40 0 0 0 0 0 0\nLee, Korey 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nRobert Jr., Luis 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.442 0.378 0.389 0.345 103.8 -6\nBenintendi, Andrew 3 3 1 1 0 0 1 33.3 25 0 0 0.333 0.667 0.423 0.23 0.255 0.213 70.6 42\nLopez, Nicky 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0'}, {'pitcher_id': '657571', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFerguson, Caleb 10 8 1 0 0 0 4 40 34.5 2 20 0.125 0.125 0.227 0.129 0.172 0.244 82.6 -7.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 3 1 0 0 0 0 0 0 8.3 2 66.7 0 0 0.462 0.076 0.079 0.485 81.9 -21\nMaldonado, Martín 2 2 0 0 0 0 2 100 71.4 0 0 0 0 0 0\nDeJong, Paul 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBenintendi, Andrew 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nLopez, Nicky 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.326 0.328 0.293 58.8 -52\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.491 0.671 0.514 95.4 17\nVaughn, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.14 0.299 0.176 94.2 27'}, {'pitcher_id': '676760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarinaccio, Ron 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.077 0.122 0.083 88.7 23', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.154 0.244 0.166 88.7 23'}, {'pitcher_id': '672782', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolmes, Clay 32 28 7 3 0 0 8 25 33.3 4 12.5 0.25 0.357 0.312 0.234 0.326 0.298 88.5 -8.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 7 6 1 0 0 0 3 42.9 50 1 14.3 0.167 0.167 0.225 0.1 0.149 0.196 84.1 2.7\nDeJong, Paul 8 7 5 3 0 0 0 0 0 1 12.5 0.714 1.143 0.767 0.513 0.812 0.572 95.5 7.9\nRobert Jr., Luis 3 2 1 0 0 0 1 33.3 16.7 1 33.3 0.5 0.5 0.527 0.052 0.052 0.264 93.3 -36\nBenintendi, Andrew 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.231 0.152 0.164 0.326 99.5 -4\nLopez, Nicky 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.31 0.336 0.288 83.5 -22.3\nVaughn, Andrew 4 4 0 0 0 0 1 25 40 0 0 0 0 0 0.092 0.095 0.086 76.4 -33.3\nLee, Korey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSheets, Gavin 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.609 0.689 0.573 97.1 8\nMaldonado, Martín 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.051 0.046 79.2 -37'}, {'pitcher_id': '624647', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzález, Victor 4 4 2 0 0 0 1 25 37.5 0 0 0.5 0.5 0.441 0.407 0.51 0.396 93 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMaldonado, Martín 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nLopez, Nicky 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.777 0.84 0.704 92.4 11\nBenintendi, Andrew 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.253 0.297 0.233 88.1 -3\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.597 0.903 0.649 98.6 16'}, {'pitcher_id': '596133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeaver, Luke 40 35 7 3 0 1 13 32.5 22.5 3 7.5 0.2 0.371 0.275 0.248 0.599 0.38 88.1 17.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 8 8 2 1 0 0 2 25 13.3 0 0 0.25 0.375 0.268 0.212 0.384 0.255 93.9 21.7\nMaldonado, Martín 9 7 2 1 0 0 2 22.2 30.8 2 22.2 0.286 0.429 0.39 0.289 0.597 0.442 91.6 35\nFedde, Erick 3 2 0 0 0 0 2 66.7 11.1 0 0 0 0 0 0 0 0 43.9 -71\nRobert Jr., Luis 2 2 0 0 0 0 1 50 30 0 0 0 0 0 0.456 1.726 0.909 104.5 26\nJulks, Corey 3 2 1 0 0 1 0 0 20 1 33.3 0.5 2 0.9 0.458 1.595 0.786 88.3 48.5\nSoroka, Michael 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.045 0.06 0.041 86.7 -12\nDeJong, Paul 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.171 0.497 0.275 73.2 26\nBenintendi, Andrew 3 3 2 1 0 0 0 0 20 0 0 0.667 1 0.695 0.54 0.903 0.601 96.7 12.7\nKuhl, Chad 2 1 0 0 0 0 1 50 0 0 0 0 0 0 0\nVaughn, Andrew 1 1 0 0 0 0 0 0 20 0 0 0 0 0 0.239 0.239 0.218 55 -57\nSosa, Lenyn 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.322 1.224 0.636 102.8 31\nSheets, Gavin 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0'}, {'pitcher_id': '642528', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLoáisiga, Jonathan 20 18 5 0 0 0 2 10 23.8 2 10 0.278 0.278 0.29 0.293 0.348 0.324 86.4 -2.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 5 4 1 0 0 0 0 0 10 1 20 0.25 0.25 0.315 0.153 0.17 0.254 89.1 -21.5\nVaughn, Andrew 4 4 1 0 0 0 0 0 20 0 0 0.25 0.25 0.221 0.26 0.285 0.241 76.7 4.8\nMaldonado, Martín 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.23 0.051 0.069 0.26 89.1 -12\nSheets, Gavin 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0\nLopez, Nicky 2 2 1 0 0 0 0 0 50 0 0 0.5 0.5 0.44 0.52 0.679 0.517 95.8 18\nBenintendi, Andrew 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.505 0.621 0.484 86.5 -1.5\nRobert Jr., Luis 2 2 1 0 0 0 0 0 40 0 0 0.5 0.5 0.442 0.59 0.678 0.565 90 6'}, {'pitcher_id': '657376', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchmidt, Clarke 31 30 8 1 0 1 6 19.4 21.9 0 0 0.267 0.4 0.299 0.204 0.291 0.229 88.5 11.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 6 5 1 1 0 0 1 16.7 21.4 0 0 0.2 0.4 0.328 0.106 0.138 0.209 84.3 7.8\nVaughn, Andrew 5 5 0 0 0 0 3 60 30.8 0 0 0 0 0 0.041 0.045 0.037 72.7 42\nPham, Tommy 4 4 0 0 0 0 1 25 28.6 0 0 0 0 0 0.105 0.135 0.104 89.8 23.7\nRobert Jr., Luis 7 7 3 0 0 1 0 0 0 0 0 0.429 0.857 0.538 0.276 0.558 0.35 87.9 15.3\nSheets, Gavin 4 4 0 0 0 0 1 25 12.5 0 0 0 0 0 0.129 0.134 0.118 97.3 -16.3\nLopez, Nicky 3 3 2 0 0 0 0 0 44.4 0 0 0.667 0.667 0.588 0.46 0.518 0.422 89.5 5.7\nMaldonado, Martín 2 2 2 0 0 0 0 0 0 0 0 1 1 0.883 0.563 0.645 0.529 97.7 3.5'}, {'pitcher_id': '547001', 'player_data': '', 'team_data': 'Cody Poteet has not faced anyone on this team.'}, {'pitcher_id': '642701', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Dennis 18 14 3 1 0 0 3 16.7 17.2 4 22.2 0.214 0.286 0.32 0.197 0.286 0.314 82.9 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 4 3 2 1 0 0 1 25 20 1 25 0.667 1 0.702 0.426 0.738 0.539 106 11\nMaldonado, Martín 3 2 0 0 0 0 1 33.3 14.3 1 33.3 0 0 0.231 0.021 0.022 0.243 89.2 -34\nBenintendi, Andrew 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.854 0.908 0.733 72.2 18\nVaughn, Andrew 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.442 0.08 0.1 0.072 102.2 -22\nSheets, Gavin 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.059 0.072 0.378 83.5 -25\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.058 0.059 68.9 -11\nLee, Korey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.093 0.106 0.089 59 -11\nDeJong, Paul 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.026 0.042 0.03 80.7 41\nRobert Jr., Luis 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.026 0.023 50.4 57\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.169 0.33 0.206 93.8 25'}], 146: [{'pitcher_id': '543859', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTonkin, Michael 12 12 4 0 0 1 3 25 28 0 0 0.333 0.583 0.386 0.232 0.496 0.307 95.1 20.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Jesús 3 3 0 0 0 0 3 100 50 0 0 0 0 0 0\nAnderson, Tim 3 3 3 0 0 0 0 0 25 0 0 1 1 0.878 0.384 0.412 0.352 87.8 0.3\nChisholm Jr., Jazz 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.002 0.459 1.786 0.907 103.7 47\nFortes, Nick 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.194 0.62 0.334 95.4 30\nBurger, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 87 74\nRivera, Emmanuel 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.27 0.27 0.247 100.8 -7\nBell, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.253 0.253 0.232 102.1 -11'}, {'pitcher_id': '664776', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCousins, Jake 3 2 1 0 0 0 1 33.3 37.5 1 33.3 0.5 0.5 0.524 0.455 0.509 0.515 78.5 21', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBethancourt, Christian 1 1 0 0 0 0 1 100 40 0 0 0 0 0 0\nBell, Josh 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nAnderson, Tim 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.879 0.909 1.019 0.853 78.5 21'}, {'pitcher_id': '690925', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '677076', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '573186', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStroman, Marcus 102 92 19 2 0 4 30 29.4 27.7 9 8.8 0.207 0.359 0.278 0.245 0.386 0.306 91.4 5.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 22 18 2 0 0 0 6 27.3 28.6 3 13.6 0.111 0.111 0.174 0.227 0.258 0.275 97.4 11.7\nAnderson, Tim 22 21 5 0 0 1 7 31.8 34.2 1 4.5 0.238 0.381 0.284 0.264 0.335 0.281 86.1 -1.7\nChisholm Jr., Jazz 14 13 4 0 0 0 4 28.6 23.3 1 7.1 0.308 0.308 0.301 0.353 0.443 0.372 92.1 -13.4\nDe La Cruz, Bryan 11 11 1 1 0 0 5 45.5 29.4 0 0 0.091 0.182 0.115 0.096 0.185 0.118 88.4 17.7\nSánchez, Jesús 9 6 1 0 0 0 3 33.3 25 3 33.3 0.167 0.167 0.329 0.292 0.373 0.421 97.8 7.7\nBurger, Jake 8 8 2 0 0 2 2 25 41.7 0 0 0.25 1 0.511 0.302 0.9 0.495 96.2 8.8\nGordon, Nick 4 3 1 1 0 0 1 25 9.1 1 25 0.333 0.667 0.484 0.246 0.821 0.493 92.3 3\nFortes, Nick 8 8 3 0 0 1 0 0 0 0 0 0.375 0.75 0.48 0.266 0.443 0.302 85.6 21.4\nBethancourt, Christian 3 3 0 0 0 0 1 33.3 44.4 0 0 0 0 0 0.072 0.187 0.105 93.1 -1\nLuzardo, Jesús 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '607074', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodón, Carlos 35 32 6 2 0 0 8 22.9 25 3 8.6 0.188 0.25 0.233 0.156 0.214 0.204 89.4 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBethancourt, Christian 9 8 1 1 0 0 1 11.1 28.6 1 11.1 0.125 0.25 0.217 0.177 0.31 0.262 86.9 22.6\nRivera, Emmanuel 7 7 0 0 0 0 3 42.9 33.3 0 0 0 0 0 0.118 0.138 0.111 93.8 18.8\nBell, Josh 5 3 1 0 0 0 1 20 42.9 2 40 0.333 0.333 0.453 0.268 0.303 0.425 97.3 1\nSánchez, Jesús 2 2 0 0 0 0 2 100 20 0 0 0 0 0 0\nAnderson, Tim 3 3 2 0 0 0 0 0 16.7 0 0 0.667 0.667 0.593 0.188 0.212 0.172 92.4 -11.7\nDe La Cruz, Bryan 3 3 0 0 0 0 1 33.3 12.5 0 0 0 0 0 0.212 0.253 0.197 88.8 10.5\nChisholm Jr., Jazz 3 3 2 1 0 0 0 0 0 0 0 0.667 1 0.719 0.161 0.276 0.18 86.7 22.7\nBurger, Jake 3 3 0 0 0 0 0 0 28.6 0 0 0 0 0 0.089 0.093 0.081 84.3 31.7'}, {'pitcher_id': '592454', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKahnle, Tommy 9 7 2 0 0 0 3 33.3 37.5 1 11.1 0.286 0.286 0.273 0.403 0.664 0.435 97.1 21.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.329 0.756 0.46 100.5 19\nBruján, Vidal 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nDe La Cruz, Bryan 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nChisholm Jr., Jazz 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nBethancourt, Christian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.911 1.059 0.863 94.9 12\nBurger, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.926 1.324 0.971 106.7 12\nRivera, Emmanuel 1 0 0 0 0 0 0 0 0 0 0 0 0.005 83 46'}, {'pitcher_id': '641482', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCortes, Nestor 50 49 9 0 0 0 10 20 22.5 0 0 0.184 0.184 0.173 0.201 0.258 0.212 87.3 15.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 14 14 5 0 0 0 2 14.3 20.7 0 0 0.357 0.357 0.314 0.265 0.309 0.257 86.5 -6.8\nBruján, Vidal 10 10 1 0 0 0 1 10 20 0 0 0.1 0.1 0.088 0.14 0.235 0.16 88.9 31.1\nRivera, Emmanuel 6 6 1 0 0 0 2 33.3 25 0 0 0.167 0.167 0.147 0.251 0.295 0.237 90.4 13.8\nGordon, Nick 6 5 0 0 0 0 2 33.3 27.3 0 0 0 0 0.12 0.101 0.205 0.231 94.7 27.7\nBurger, Jake 3 3 0 0 0 0 3 100 27.3 0 0 0 0 0 0\nDe La Cruz, Bryan 3 3 2 0 0 0 0 0 33.3 0 0 0.667 0.667 0.593 0.545 0.597 0.498 77.5 27.3\nBethancourt, Christian 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.286 0.301 0.261 82.2 -9.3\nBell, Josh 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.075 0.05 87 44\nFortes, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.061 0.124 0.077 89.9 40.5'}, {'pitcher_id': '641656', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHamilton, Ian 20 18 5 2 1 0 5 25 40.5 1 5 0.278 0.5 0.326 0.212 0.264 0.222 87.3 -2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBethancourt, Christian 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.221 0.199 0.221 0.185 87 -20.3\nBell, Josh 4 4 0 0 0 0 2 50 44.4 0 0 0 0 0 0.048 0.053 0.046 87.1 -23.5\nDe La Cruz, Bryan 2 1 0 0 0 0 0 0 66.7 1 50 0 0 0.346 0.003 0.003 0.348 65 77\nBurger, Jake 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.575 0.898 0.619 104.8 6.5\nSánchez, Jesús 2 2 1 0 1 0 0 0 0 0 0 0.5 1.5 0.784 0.295 0.309 0.266 86.1 -6.5\nGordon, Nick 2 1 0 0 0 0 1 50 66.7 0 0 0 0 0 0 0 0.013 87.8 36\nChisholm Jr., Jazz 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.086 0.09 0.083 87.7 -25\nRivera, Emmanuel 1 1 1 1 0 0 0 0 66.7 0 0 1 2 1.244 0.937 1.095 0.894 91.9 15\nAnderson, Tim 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nFortes, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.057 0.059 0.056 72.6 -23'}, {'pitcher_id': '595897', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661563', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGil, Luis 3 3 0 0 0 0 2 66.7 75 0 0 0 0 0 0.04 0.06 0.037 86.8 25', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 3 3 0 0 0 0 2 66.7 75 0 0 0 0 0 0.04 0.06 0.037 86.8 25'}, {'pitcher_id': '657571', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFerguson, Caleb 8 7 3 0 0 0 2 25 35 1 12.5 0.429 0.429 0.417 0.302 0.503 0.389 91.2 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 4 3 2 0 0 0 0 0 0 1 25 0.667 0.667 0.613 0.248 0.33 0.361 96.6 -2\nBurger, Jake 2 2 0 0 0 0 1 50 42.9 0 0 0 0 0 0.241 0.765 0.407 99.4 27\nChisholm Jr., Jazz 1 1 1 0 0 0 0 0 50 0 0 1 1 0.883 0.886 1.002 0.857 66.9 32\nSánchez, Jesús 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '676760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarinaccio, Ron 8 7 1 1 0 0 2 25 20 0 0 0.143 0.286 0.18 0.238 0.325 0.248 77 -1.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBethancourt, Christian 3 2 1 1 0 0 0 0 0 0 0 0.5 1 0.631 0.6 0.736 0.594 64.5 -1.7\nBell, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.234 0.55 0.328 94.8 24\nGordon, Nick 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nBruján, Vidal 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.116 0.128 0.11 86.7 -13\nBurger, Jake 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '672782', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolmes, Clay 23 22 1 0 0 0 5 21.7 28.2 1 4.3 0.045 0.045 0.069 0.179 0.309 0.229 92.6 -4.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBruján, Vidal 4 4 0 0 0 0 1 25 10 0 0 0 0 0 0.121 0.165 0.123 82.7 4.3\nChisholm Jr., Jazz 3 2 0 0 0 0 1 33.3 50 1 33.3 0 0 0.232 0.208 0.249 0.367 83.5 8\nBurger, Jake 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.433 0.729 0.49 107.7 1.5\nDe La Cruz, Bryan 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.203 0.235 0.196 100.1 -16.5\nAnderson, Tim 3 3 0 0 0 0 1 33.3 50 0 0 0 0 0 0.055 0.062 0.051 90.2 -24\nGordon, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.044 0.062 0.048 89.2 4.5\nSánchez, Jesús 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.462 1.376 0.757 103.1 8.5\nBell, Josh 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.087 0.087 0.083 85.8 -16\nBethancourt, Christian 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nFortes, Nick 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.205 0.229 0.184 90 -6'}, {'pitcher_id': '624647', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzález, Victor 13 13 1 0 0 0 2 15.4 12.5 0 0 0.077 0.077 0.068 0.156 0.165 0.143 80.2 -7.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChisholm Jr., Jazz 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.044 0.048 0.042 74.2 -8\nBurger, Jake 3 3 0 0 0 0 1 33.3 25 0 0 0 0 0 0.242 0.257 0.225 78.9 -36.5\nBell, Josh 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.294 0.3 0.312 0.27 78.8 6.3\nDe La Cruz, Bryan 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.026 0.03 0.026 81.7 16.5\nSánchez, Jesús 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.106 0.118 0.097 88.2 -20.5'}, {'pitcher_id': '596133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeaver, Luke 53 50 13 4 0 1 14 26.4 29.7 2 3.8 0.26 0.4 0.296 0.268 0.445 0.32 87.3 6.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 25 24 5 0 0 1 7 28 35.7 1 4 0.208 0.333 0.246 0.275 0.483 0.338 93.5 6\nDe La Cruz, Bryan 6 6 2 2 0 0 3 50 22.7 0 0 0.333 0.667 0.415 0.179 0.234 0.179 84.6 32.3\nSánchez, Jesús 3 2 0 0 0 0 2 66.7 33.3 1 33.3 0 0 0.231 0.231\nFortes, Nick 5 4 1 0 0 0 0 0 12.5 0 0 0.25 0.25 0.221 0.263 0.431 0.295 74.6 -14.8\nChisholm Jr., Jazz 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.304 0.344 0.291 75 -25\nGordon, Nick 4 4 3 0 0 0 1 25 44.4 0 0 0.75 0.75 0.664 0.611 0.773 0.6 97.2 14.3\nAnderson, Tim 4 4 0 0 0 0 1 25 33.3 0 0 0 0 0 0.032 0.037 0.033 72.9 9.3\nBurger, Jake 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.622 0.277 0.472 0.308 88.4 51\nRivera, Emmanuel 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.649 2.305 1.238 100.8 28'}, {'pitcher_id': '642528', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLoáisiga, Jonathan 9 8 1 0 0 0 1 11.1 31.6 1 11.1 0.125 0.125 0.175 0.213 0.257 0.26 88.7 -9.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 3 2 0 0 0 0 0 0 50 1 33.3 0 0 0.23 0.119 0.155 0.309 95.9 -9\nDe La Cruz, Bryan 2 2 0 0 0 0 0 0 20 0 0 0 0 0 0.147 0.165 0.139 81.1 -5\nFortes, Nick 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.062 0.064 0.061 78.1 -37\nBurger, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.168 0.212 0.156 94.9 -14\nBell, Josh 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nGordon, Nick 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.884 0.937 1.142 0.923 94.3 13'}, {'pitcher_id': '657376', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchmidt, Clarke 19 18 5 1 0 0 4 21.1 25 1 5.3 0.278 0.333 0.288 0.237 0.36 0.28 90.2 3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 4 3 2 0 0 0 0 0 10 1 25 0.667 0.667 0.615 0.492 0.54 0.514 93.9 5.3\nAnderson, Tim 4 4 1 0 0 0 0 0 22.2 0 0 0.25 0.25 0.221 0.251 0.258 0.229 94.5 -6.8\nBethancourt, Christian 3 3 0 0 0 0 3 100 50 0 0 0 0 0 0\nRivera, Emmanuel 2 2 0 0 0 0 0 0 16.7 0 0 0 0 0 0.072 0.122 0.078 79.4 13.5\nBruján, Vidal 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.082 0.162 0.098 93.8 24\nGordon, Nick 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.442 0.138 0.149 0.127 83.5 -13.5\nChisholm Jr., Jazz 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.619 0.603 1.48 0.87 91.5 14.5'}, {'pitcher_id': '547001', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoteet, Cody 3 3 0 0 0 0 0 0 20 0 0 0 0 0 0.137 0.137 0.127 80.9 -6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.142 0.142 0.133 81.7 -6.5\nBruján, Vidal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.128 0.129 0.117 79.3 -5'}, {'pitcher_id': '642701', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Dennis 14 12 4 0 0 0 4 28.6 32 1 7.1 0.333 0.333 0.302 0.291 0.347 0.313 87.2 11', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Bryan 3 3 2 0 0 0 0 0 20 0 0 0.667 0.667 0.589 0.651 0.816 0.64 85.7 25\nFortes, Nick 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.065 0.07 0.058 98.4 -12\nSánchez, Jesús 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.193 0.222 0.182 83.7 6\nBurger, Jake 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.064 0.075 0.059 77.1 -2\nBell, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.377 0.403 0.339 90.9 4\nGordon, Nick 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nBruján, Vidal 1 0 0 0 0 0 0 0 0 0 0 0 0.362 91 20\nAnderson, Tim 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.451 0.504 0.412 87 8'}], 158: [{'pitcher_id': '543859', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTonkin, Michael 27 26 7 1 0 0 6 22.2 26.9 1 3.7 0.269 0.308 0.269 0.275 0.376 0.296 90 16.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurang, Brice 4 4 0 0 0 0 3 75 30.8 0 0 0 0 0 0.064 0.078 0.06 79.2 27\nAdames, Willy 4 4 2 0 0 0 0 0 37.5 0 0 0.5 0.5 0.444 0.393 0.466 0.382 90.9 20\nYelich, Christian 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.18 0.637 0.328 98.7 32\nOrtiz, Joey 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.3 0.316 0.276 70.5 5.3\nHoskins, Rhys 2 1 0 0 0 0 0 0 20 1 50 0 0 0.346 0.174 0.689 0.514 97.8 36\nContreras, William 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nPerkins, Blake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.218 0.243 0.197 96.4 -19\nSánchez, Gary 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.438 0.602 0.693 0.559 89.7 7\nFrelick, Sal 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.202 0.215 0.184 96.7 35.5\nBauers, Jake 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.181 0.332 0.21 93.9 37.5\nChourio, Jackson 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.854 1.017 0.826 97.1 11\nMonasterio, Andruw 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.631 0.699 0.578 98.3 6'}, {'pitcher_id': '664776', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '690925', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '677076', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '573186', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStroman, Marcus 104 88 22 2 1 4 24 23.1 29.5 15 14.4 0.25 0.432 0.352 0.261 0.51 0.38 91.4 1.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 23 19 9 1 0 2 4 17.4 28.9 4 17.4 0.474 0.842 0.583 0.378 0.815 0.529 93.9 4.5\nHoskins, Rhys 19 17 1 0 0 0 6 31.6 39.5 2 10.5 0.059 0.059 0.119 0.214 0.3 0.269 92.7 6.7\nYelich, Christian 15 12 3 0 0 1 2 13.3 25 3 20 0.25 0.5 0.39 0.33 0.525 0.431 94.4 -10.5\nSánchez, Gary 14 11 3 0 0 0 4 28.6 26.1 2 14.3 0.273 0.273 0.339 0.221 0.464 0.374 92.1 4.4\nBauers, Jake 12 11 2 1 0 1 4 33.3 42.3 1 8.3 0.182 0.545 0.333 0.249 0.692 0.41 91.4 11.6\nTurang, Brice 6 6 1 0 0 0 0 0 0 0 0 0.167 0.167 0.147 0.1 0.111 0.093 86 -10.3\nContreras, William 6 6 1 0 0 0 3 50 37.5 0 0 0.167 0.167 0.148 0.073 0.117 0.084 75.9 11.3\nFrelick, Sal 3 2 1 0 1 0 0 0 0 1 33.3 0.5 1.5 0.768 0.487 1.378 0.745 86.9 -10.5\nOrtiz, Joey 2 0 0 0 0 0 0 0 0 2 100 0.692 0.692\nPerkins, Blake 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.281 0.31 0.259 102.5 5\nChourio, Jackson 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.239 0.271 0.221 85.8 -4'}, {'pitcher_id': '607074', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRodón, Carlos 58 53 10 4 0 1 19 32.8 36.3 5 8.6 0.189 0.321 0.257 0.239 0.46 0.326 93.6 19.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 14 13 3 1 0 0 3 21.4 39.3 1 7.1 0.231 0.308 0.261 0.258 0.635 0.389 91.3 16.5\nHoskins, Rhys 9 9 3 1 0 1 2 22.2 19 0 0 0.333 0.778 0.47 0.247 0.612 0.355 96.8 38.9\nContreras, William 8 6 1 0 0 0 2 25 42.9 2 25 0.167 0.167 0.283 0.273 0.337 0.368 97 0.5\nYelich, Christian 7 7 1 1 0 0 5 71.4 50 0 0 0.143 0.286 0.177 0.27 0.564 0.349 88.3 22.5\nBauers, Jake 7 5 1 0 0 0 2 28.6 45 2 28.6 0.2 0.2 0.323 0.061 0.07 0.239 93.6 7\nSánchez, Gary 7 7 0 0 0 0 2 28.6 30.8 0 0 0 0 0 0.321 0.451 0.326 99.8 23.6\nOrtiz, Joey 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.635 0.449 0.488 0.416 65.1 29\nChourio, Jackson 2 2 0 0 0 0 1 50 20 0 0 0 0 0 0.007 0.015 0.01 91.3 45\nPerkins, Blake 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.056 0.061 0.052 91.7 -29'}, {'pitcher_id': '592454', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKahnle, Tommy 12 10 3 1 0 0 3 25 27.3 2 16.7 0.3 0.4 0.365 0.242 0.288 0.313 87.4 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 6 4 0 0 0 0 3 50 42.9 2 33.3 0 0 0.231 0.016 0.017 0.24 88.3 -34\nMonasterio, Andruw 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.091 0.113 0.085 86.1 17\nYelich, Christian 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.493 0.533 0.453 98.1 2\nTurang, Brice 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.957 1.048 0.897 67.3 27\nSánchez, Gary 1 1 1 1 0 0 0 0 0 0 0 1 2 1.232 0.487 0.719 0.483 98.4 17'}, {'pitcher_id': '641482', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nCortes, Nestor 12 12 4 2 0 0 3 25 23.8 0 0 0.333 0.5 0.353 0.203 0.441 0.266 88.8 26.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 6 6 2 1 0 0 1 16.7 33.3 0 0 0.333 0.5 0.348 0.317 0.754 0.437 89.2 21\nSánchez, Gary 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.631 0.127 0.243 0.155 97.5 21\nBauers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 80 69\nOrtiz, Joey 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.441 0.125 0.126 0.115 109.8 -15\nHoskins, Rhys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.029 0.026 65.4 57'}, {'pitcher_id': '641656', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHamilton, Ian 6 3 1 0 0 0 0 0 44.4 2 33.3 0.333 0.333 0.501 0.59 0.739 0.634 89.1 8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPerkins, Blake 1 0 0 0 0 0 0 0 100 1 100 0.692 0.692\nYelich, Christian 1 0 0 0 0 0 0 0 100 1 100 0.699 0.699\nAdames, Willy 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.109 0.135 0.1 69.6 -3\nTurang, Brice 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.717 0.928 0.688 95 16\nBauers, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.943 1.154 0.902 102.8 11\nSánchez, Gary 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '595897', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '661563', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGil, Luis 26 24 7 1 0 2 7 26.9 37.3 2 7.7 0.292 0.583 0.399 0.228 0.471 0.324 92.6 17.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nContreras, William 3 3 0 0 0 0 0 0 33.3 0 0 0 0 0 0.049 0.061 0.046 82.4 33\nHoskins, Rhys 3 3 2 0 0 0 1 33.3 25 0 0 0.667 0.667 0.593 0.312 0.364 0.292 89.9 -0.5\nAdames, Willy 3 2 0 0 0 0 0 0 20 1 33.3 0 0 0.231 0.042 0.047 0.257 90.8 14.5\nPerkins, Blake 3 3 1 0 0 1 1 33.3 25 0 0 0.333 1.333 0.695 0.371 1.225 0.649 99.5 28\nSánchez, Gary 3 3 1 1 0 0 2 66.7 50 0 0 0.333 0.667 0.423 0.304 0.414 0.311 112.3 10\nFrelick, Sal 3 2 0 0 0 0 1 33.3 40 1 33.3 0 0 0.231 0.004 0.008 0.234 85.4 42\nBauers, Jake 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.16 0.176 0.146 99 -4\nOrtiz, Joey 2 2 1 0 0 1 0 0 40 0 0 0.5 2 1.043 0.487 1.784 0.922 98.3 39.5\nChourio, Jackson 2 2 0 0 0 0 1 50 80 0 0 0 0 0 0.03 0.034 0.029 73.6 -16\nTurang, Brice 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.464 0.511 0.427 100.4 0'}, {'pitcher_id': '657571', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFerguson, Caleb 20 17 3 1 0 0 5 25 32.1 2 10 0.176 0.235 0.221 0.3 0.374 0.321 97.6 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 4 3 1 1 0 0 0 0 33.3 0 0 0.333 0.667 0.315 0.264 0.321 0.191 90.8 36\nYelich, Christian 6 6 1 0 0 0 1 16.7 33.3 0 0 0.167 0.167 0.147 0.436 0.575 0.446 103.2 -1\nSánchez, Gary 2 1 0 0 0 0 0 0 50 1 50 0 0 0.348 0.167 0.179 0.425 103.5 -23\nPerkins, Blake 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.061 0.066 0.056 97 -23\nTurang, Brice 2 1 1 0 0 0 0 0 0 1 50 1 1 0.787 0.957 1.164 0.811 97.5 12\nDunn, Oliver 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBauers, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.443 0.479 0.399 91.4 5\nOrtiz, Joey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHoskins, Rhys 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0'}, {'pitcher_id': '676760', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMarinaccio, Ron 15 12 1 1 0 0 5 33.3 36 2 13.3 0.083 0.167 0.177 0.102 0.144 0.179 87.9 17', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTurang, Brice 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.061 0.065 0.058 73.2 -26\nOrtiz, Joey 2 0 0 0 0 0 0 0 33.3 1 50 0.346 0.346 95.7 46\nFrelick, Sal 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.255 0.289 0.241 91.7 8\nMonasterio, Andruw 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nChourio, Jackson 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.088 0.289 0.157 93.8 31\nSánchez, Gary 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nAdames, Willy 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.004 0.005 0.004 92.6 64\nPerkins, Blake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.029 0.037 0.027 80.5 46\nContreras, William 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.319 0.333 0.287 102.1 -7'}, {'pitcher_id': '672782', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '605280', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHolmes, Clay 18 15 4 3 0 0 4 22.2 33.3 3 16.7 0.267 0.467 0.371 0.156 0.208 0.249 91.8 -8.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 7 5 3 2 0 0 0 0 18.2 2 28.6 0.6 1 0.677 0.231 0.253 0.355 91.1 -10\nContreras, William 4 4 0 0 0 0 2 50 40 0 0 0 0 0 0.096 0.099 0.087 89.6 -34.5\nSánchez, Gary 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.107 0.107 0.097 96.5 -8\nAdames, Willy 2 1 1 1 0 0 0 0 50 1 50 1 2 0.967 0.526 1.183 0.698 100.8 22\nDunn, Oliver 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nHoskins, Rhys 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 92.8 45\nFrelick, Sal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.064 0.067 0.058 85.1 -37'}, {'pitcher_id': '624647', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGonzález, Victor 4 3 1 0 0 0 0 0 16.7 1 25 0.333 0.333 0.395 0.366 0.468 0.44 101.2 -9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nChourio, Jackson 1 1 1 0 0 0 0 0 25 0 0 1 1 0.889 0.757 1.042 0.754 102.9 14\nTurang, Brice 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.197 0.215 0.18 98.7 -17\nOrtiz, Joey 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nYelich, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.143 0.147 0.132 102.1 -24'}, {'pitcher_id': '596133', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWeaver, Luke 34 33 10 2 1 5 8 23.5 17.5 1 2.9 0.303 0.879 0.49 0.299 0.698 0.422 91.5 10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 8 8 3 1 0 2 2 25 15.4 0 0 0.375 1.25 0.663 0.438 1.149 0.657 104.2 13\nHoskins, Rhys 7 7 2 0 0 0 3 42.9 33.3 0 0 0.286 0.286 0.251 0.267 0.32 0.255 91.4 38.5\nSánchez, Gary 4 4 1 0 0 1 1 25 25 0 0 0.25 1 0.518 0.374 1.134 0.614 92.6 6.3\nPerkins, Blake 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.231 0.025 0.051 0.252 83.7 30\nContreras, William 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.104 0.114 0.097 83.9 -29\nTurang, Brice 2 2 1 0 1 0 0 0 20 0 0 0.5 1.5 0.784 0.154 0.31 0.189 82.1 38.5\nAdames, Willy 2 2 2 0 0 2 0 0 0 0 0 1 4 2.004 0.659 2.46 1.248 102 30\nChourio, Jackson 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.434 0.463 0.391 100.7 0\nBauers, Jake 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.164 0.166 0.151 64.4 -40\nDunn, Oliver 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nFrelick, Sal 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.104 0.116 0.098 86.9 -16\nOrtiz, Joey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.264 0.294 0.238 83 2'}, {'pitcher_id': '642528', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLoáisiga, Jonathan 24 21 6 1 0 1 5 20.8 26.4 3 12.5 0.286 0.476 0.368 0.238 0.413 0.325 90.1 7.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAdames, Willy 9 7 3 1 0 1 1 11.1 31.8 2 22.2 0.429 1 0.609 0.37 0.84 0.536 90 22.5\nHoskins, Rhys 4 3 0 0 0 0 2 50 33.3 1 25 0 0 0.175 0.013 0.021 0.186 76.9 59\nContreras, William 3 3 2 0 0 0 0 0 28.6 0 0 0.667 0.667 0.588 0.51 0.593 0.48 97.5 2.3\nBauers, Jake 4 4 0 0 0 0 2 50 33.3 0 0 0 0 0 0.044 0.049 0.042 81.3 -12.5\nFrelick, Sal 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.155 0.189 0.144 82.6 -9\nMonasterio, Andruw 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.159 0.161 0.143 98 -21\nYelich, Christian 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.206 0.229 0.191 105.8 -21'}, {'pitcher_id': '657376', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSchmidt, Clarke 3 3 2 1 0 0 1 33.3 0 0 0 0.667 1 0.715 0.366 0.585 0.403 107.9 8.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nYelich, Christian 1 1 0 0 0 0 1 100 0 0 0 0 0 0\nSánchez, Gary 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.477 0.499 0.429 108.6 -1\nAdames, Willy 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.62 1.257 0.781 107.2 18'}, {'pitcher_id': '547001', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPoteet, Cody 5 5 3 0 0 1 0 0 12.5 0 0 0.6 1.2 0.767 0.422 1.017 0.603 96.8 34.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHoskins, Rhys 4 4 2 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.304 0.376 0.288 94.3 35.5\nContreras, William 1 1 1 0 0 1 0 0 50 0 0 1 4 2.072 0.894 3.581 1.864 106.6 31'}, {'pitcher_id': '642701', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSantana, Dennis 9 8 2 1 0 0 4 44.4 43.8 1 11.1 0.25 0.375 0.316 0.127 0.162 0.187 96.1 24.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Gary 2 1 0 0 0 0 1 50 100 1 50 0 0 0.344 0.344\nYelich, Christian 2 2 0 0 0 0 2 100 25 0 0 0 0 0 0\nHoskins, Rhys 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.442 0.138 0.176 0.124 93.1 -1\nContreras, William 1 1 1 1 0 0 0 0 0 0 0 1 2 1.27 0.737 0.929 0.735 107.6 9\nTurang, Brice 1 1 0 0 0 0 0 0 66.7 0 0 0 0 0 0.003 0.003 0.003 88.5 42\nAdames, Willy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.015 0.006 95.1 48'}]}, 158: {145: [{'pitcher_id': '657649', 'player_data': '', 'team_data': ''}, {'pitcher_id': '489119', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiley, Wade 42 37 9 1 0 2 7 16.7 12.9 5 11.9 0.243 0.432 0.332 0.266 0.417 0.34 84.6 7.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 12 10 2 0 0 1 2 16.7 15 2 16.7 0.2 0.5 0.354 0.27 0.486 0.38 80.6 11.8\nPham, Tommy 16 15 4 1 0 1 4 25 20 1 6.3 0.267 0.533 0.354 0.259 0.45 0.323 94.3 18.3\nBenintendi, Andrew 8 6 2 0 0 0 1 12.5 8.3 2 25 0.333 0.333 0.392 0.39 0.421 0.438 74.9 2\nLopez, Nicky 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.097 0.106 0.093 77.5 -32.3\nMaldonado, Martín 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.108 0.147 0.1 80 -5'}, {'pitcher_id': '656730', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Trevor 18 15 4 1 0 1 6 33.3 21.6 3 16.7 0.267 0.533 0.397 0.249 0.371 0.338 92.8 8.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 3 2 0 0 0 0 1 33.3 33.3 1 33.3 0 0 0.23 0.134 0.155 0.311 97.9 -9\nVaughn, Andrew 3 3 1 0 0 1 2 66.7 28.6 0 0 0.333 1.333 0.691 0.235 0.64 0.367 103 23\nPham, Tommy 2 1 0 0 0 0 1 50 50 1 50 0 0 0.344 0.344\nRobert Jr., Luis 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.293 0.377 0.495 0.379 92.3 18.5\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.286 0.307 0.261 95.8 -3\nBenintendi, Andrew 2 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.442 0.285 0.314 0.266 94.7 -6\nLee, Korey 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nJulks, Corey 1 1 1 1 0 0 0 0 0 0 0 1 2 1.244 0.494 0.605 0.488 68.6 40\nDeJong, Paul 1 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '607067', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRea, Colin 12 11 1 0 0 1 5 41.7 26.9 1 8.3 0.091 0.364 0.222 0.164 0.264 0.223 90.9 12.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 7 6 0 0 0 0 2 28.6 25 1 14.3 0 0 0.098 0.213 0.25 0.271 92.7 13\nJulks, Corey 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.076 0.098 0.07 78.6 -4\nRobert Jr., Luis 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nMendick, Danny 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nDeJong, Paul 1 1 1 0 0 1 0 0 0 0 0 1 4 1.979 0.373 1.207 0.639 95.9 29'}, {'pitcher_id': '682842', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUribe, Abner 4 4 1 0 0 0 1 25 33.3 0 0 0.25 0.25 0.221 0.21 0.227 0.192 97.1 -6.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 2 2 1 0 0 0 1 50 40 0 0 0.5 0.5 0.441 0.192 0.214 0.174 82.8 7\nSheets, Gavin 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.171 0.171 0.157 109.5 -22\nColás, Oscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.288 0.309 0.263 99.1 -5'}, {'pitcher_id': '663542', 'player_data': '', 'team_data': 'Bryan Hudson has not faced anyone on this team.'}, {'pitcher_id': '676879', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAshby, Aaron 6 6 0 0 0 0 3 50 50 0 0 0 0 0 0.089 0.096 0.084 74.3 -8.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 5 5 0 0 0 0 2 40 44.4 0 0 0 0 0 0.107 0.116 0.101 74.3 -8.3\nPham, Tommy 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '660600', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Enoli 9 8 1 1 0 0 1 11.1 8.3 0 0 0.125 0.25 0.221 0.24 0.465 0.349 86.4 25.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJulks, Corey 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.009 0.02 0.013 95.9 52\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0\nColás, Oscar 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.07 0.057 73.3 -13\nLopez, Nicky 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.496 1.127 0.676 96.7 15\nLee, Korey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.001 0.001 0 54.3 69\nSheets, Gavin 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.611 0.892 0.631 101.8 16\nDeJong, Paul 1 0 0 0 0 0 0 0 0 0 0.724 0.724'}, {'pitcher_id': '606303', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPayamps, Joel 25 22 4 0 0 0 6 24 18.6 2 8 0.182 0.182 0.204 0.114 0.121 0.154 79.8 8.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 4 4 1 0 0 0 2 50 37.5 0 0 0.25 0.25 0.221 0.192 0.207 0.174 89 -0.5\nBenintendi, Andrew 5 4 1 0 0 0 1 20 11.1 1 20 0.25 0.25 0.315 0.079 0.082 0.196 91.6 43.3\nVaughn, Andrew 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.021 0.023 0.021 87.8 15\nDeJong, Paul 4 4 0 0 0 0 2 50 57.1 0 0 0 0 0 0.135 0.147 0.127 90.6 -22\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.009 0.007 80.2 65\nMaldonado, Martín 2 1 0 0 0 0 0 0 50 1 50 0 0 0.346 0.035 0.037 0.363 73.2 -19\nRobert Jr., Luis 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.219 0.224 0.2 74 1\nMendick, Danny 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.141 0.158 0.137 69.9 4\nJulks, Corey 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.188 0.189 0.169 60.2 -45\nColás, Oscar 1 0 0 0 0 0 0 0 0 0 0 35.7 -44'}, {'pitcher_id': '669084', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHall, DL 8 8 0 0 0 0 3 37.5 33.3 0 0 0 0 0 0.2 0.229 0.186 81.3 -5.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nColás, Oscar 2 2 0 0 0 0 1 50 60 0 0 0 0 0 0.138 0.167 0.124 81.7 3\nVaughn, Andrew 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.211 0.232 0.193 92.2 9.5\nPham, Tommy 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nSosa, Lenyn 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.449 0.518 0.427 70.2 -25\nLee, Korey 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '669952', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWhite, Mitch 12 12 2 0 0 0 2 16.7 17.6 0 0 0.167 0.167 0.147 0.253 0.302 0.242 90.7 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMendick, Danny 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.045 0.045 0.043 91.2 20.5\nVaughn, Andrew 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.26 0.401 0.281 97.2 17\nRobert Jr., Luis 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.059 0.07 0.057 87.3 -7\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.167 0.199 0.151 86.6 -7.5\nBenintendi, Andrew 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.502 0.56 0.467 87.9 4.7\nPham, Tommy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.469 0.519 0.441 103.5 0'}, {'pitcher_id': '665625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeguero, Elvis 14 11 3 0 0 0 2 14.3 29.4 2 14.3 0.273 0.273 0.339 0.266 0.3 0.347 84.4 -7.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.04 0.085 0.053 89 27\nMaldonado, Martín 2 2 2 0 0 0 0 0 25 0 0 1 1 0.883 0.768 0.873 0.728 92.2 13.5\nVaughn, Andrew 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.162 0.162 0.418 88.3 -54\nJulks, Corey 2 2 0 0 0 0 1 50 66.7 0 0 0 0 0 0.028 0.04 0.027 72.5 -12\nSheets, Gavin 1 0 0 0 0 0 0 0 1 100 0.689 0.689\nRobert Jr., Luis 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.617 0.658 0.555 78.6 14\nSosa, Lenyn 1 0 0 0 0 0 0 0 0 0 0 0.724 0.724\nLopez, Nicky 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.101 0.102 0.094 77.6 -38\nPham, Tommy 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.333 0.335 0.301 97.6 -4\nMendick, Danny 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.045 0.045 0.04 71.2 -25'}, {'pitcher_id': '688107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGasser, Robert 19 19 7 0 0 2 3 15.8 19.4 0 0 0.368 0.684 0.453 0.309 0.529 0.353 91.4 21.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJulks, Corey 3 3 2 0 0 0 0 0 33.3 0 0 0.667 0.667 0.593 0.434 0.624 0.447 98.9 17\nPham, Tommy 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.225 0.302 0.227 97.6 13\nLee, Korey 3 3 0 0 0 0 1 33.3 42.9 0 0 0 0 0 0.023 0.026 0.022 84 12.5\nMendick, Danny 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.689 0.81 0.646 90.3 17\nSheets, Gavin 2 2 1 0 0 1 1 50 25 0 0 0.5 2 1.043 0.407 1.25 0.679 104.4 23\nSosa, Lenyn 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.579 0.81 0.587 96.5 16.5\nDeJong, Paul 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.043 0.059 0.046 74.9 37.3\nBenintendi, Andrew 1 1 1 0 0 1 0 0 25 0 0 1 4 2.085 0.349 1.269 0.654 99.4 33'}, {'pitcher_id': '669060', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Bryse 20 19 4 1 0 0 4 20 23.5 1 5 0.211 0.263 0.229 0.222 0.397 0.281 92.7 12.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 8 7 2 0 0 0 2 25 30.8 1 12.5 0.286 0.286 0.307 0.272 0.335 0.315 90.9 16.2\nPham, Tommy 5 5 1 0 0 0 1 20 11.1 0 0 0.2 0.2 0.177 0.174 0.25 0.182 96.1 -5\nVaughn, Andrew 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.301 1.102 0.57 99.9 30\nRobert Jr., Luis 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.227 0.13 86.9 18.7\nBenintendi, Andrew 1 1 1 1 0 0 0 0 33.3 0 0 1 2 1.247 0.447 0.628 0.428 96.4 17\nMendick, Danny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.154 0.443 0.235 94.3 30'}, {'pitcher_id': '676083', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunk, Janson 8 8 2 1 0 0 2 25 8.3 0 0 0.25 0.375 0.266 0.192 0.222 0.181 91 18.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.048 0.048 0.044 91.5 -23\nSheets, Gavin 2 2 1 1 0 0 1 50 33.3 0 0 0.5 1 0.621 0.38 0.468 0.37 102.2 9\nBenintendi, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.331 0.354 0.3 83 5\nRobert Jr., Luis 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.009 0.017 0.011 93.1 57.5'}, {'pitcher_id': '643361', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Kevin 17 16 1 0 0 0 6 35.3 26.1 1 5.9 0.063 0.063 0.093 0.228 0.413 0.296 90.8 15.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLee, Korey 2 2 0 0 0 0 2 100 50 0 0 0 0 0 0\nSheets, Gavin 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.385 1.115 0.625 103.3 23\nDeJong, Paul 3 3 0 0 0 0 1 33.3 0 0 0 0 0 0 0.096 0.376 0.191 92.5 55.5\nPham, Tommy 4 3 0 0 0 0 0 0 0 1 25 0 0 0.173 0.18 0.216 0.296 88.7 -8\nSosa, Lenyn 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.703 0.919 0.703 105.5 11\nJulks, Corey 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nMendick, Danny 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.503 0.599 0.484 76.4 27\nColás, Oscar 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nBenintendi, Andrew 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.141 0.161 0.132 66.5 -1'}, {'pitcher_id': '642547', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Freddy 63 55 12 1 0 2 20 31.7 38 4 6.3 0.218 0.345 0.288 0.238 0.367 0.305 85.5 9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 19 14 4 0 0 2 3 15.8 27 1 5.3 0.286 0.714 0.455 0.312 0.709 0.466 90.5 27.4\nPham, Tommy 11 10 1 0 0 0 6 54.5 59.1 1 9.1 0.1 0.1 0.143 0.142 0.198 0.198 83.9 -23.3\nSheets, Gavin 5 5 1 0 0 0 0 0 22.2 0 0 0.2 0.2 0.178 0.142 0.165 0.138 81.2 -1.8\nColás, Oscar 5 4 0 0 0 0 2 40 30 1 20 0 0 0.139 0.05 0.056 0.175 81 21\nVaughn, Andrew 5 5 3 1 0 0 1 20 10 0 0 0.6 0.8 0.601 0.502 0.557 0.467 83.3 11.3\nLopez, Nicky 3 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.296 0.336 0.388 0.314 86.3 -12.7\nLee, Korey 2 1 0 0 0 0 0 0 37.5 1 50 0 0 0.346 0.951 1.029 0.783 72.7 19\nKuhl, Chad 3 3 0 0 0 0 3 100 100 0 0 0 0 0 0\nJulks, Corey 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.444 0.458 0.599 0.45 94 12.5\nRobert Jr., Luis 2 2 0 0 0 0 2 100 80 0 0 0 0 0 0\nSosa, Lenyn 2 2 0 0 0 0 2 100 57.1 0 0 0 0 0 0\nBenintendi, Andrew 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.294 0.328 0.353 0.299 78.9 1\nMendick, Danny 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '571948', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMilner, Hoby 16 13 1 0 0 0 7 43.8 27.6 2 12.5 0.077 0.077 0.142 0.111 0.154 0.179 91.4 7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 4 4 0 0 0 0 3 75 27.3 0 0 0 0 0 0.096 0.19 0.122 100 20\nMaldonado, Martín 2 1 0 0 0 0 1 50 100 1 50 0 0 0.349 0.349\nMendick, Danny 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.088 0.088 0.08 95.6 -11\nPham, Tommy 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.048 0.049 0.366 67.4 -17\nBenintendi, Andrew 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.035 0.086 0.049 91.2 29\nLee, Korey 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.385 0.425 0.347 98.9 -5\nColás, Oscar 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nSheets, Gavin 1 0 0 0 0 0 0 0 0 0 0 0 0 87.7 43'}, {'pitcher_id': '668964', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMyers, Tobias 21 19 7 2 0 1 4 19 19.4 1 4.8 0.368 0.632 0.457 0.381 0.655 0.465 92.5 21.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLopez, Nicky 3 2 1 0 0 0 1 33.3 14.3 1 33.3 0.5 0.5 0.527 0.484 0.567 0.536 90.3 15\nPham, Tommy 3 3 2 0 0 0 1 33.3 20 0 0 0.667 0.667 0.593 0.586 0.736 0.574 101.6 11.5\nJulks, Corey 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.423 0.551 1.251 0.751 95.4 27\nMaldonado, Martín 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.006 0.009 0.006 89.1 59.5\nLee, Korey 2 2 0 0 0 0 2 100 100 0 0 0 0 0 0\nDeJong, Paul 2 2 2 1 0 1 0 0 0 0 0 1 3 1.677 0.816 1.969 1.155 102.3 21\nSheets, Gavin 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.19 0.225 0.181 88.9 23.5\nMendick, Danny 2 1 0 0 0 0 0 0 33.3 0 0 0 0 0.362 0.072 0.078 0.395 67.7 -6\nBenintendi, Andrew 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.378 0.438 0.358 89.5 2.5'}, {'pitcher_id': '656266', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBukauskas, J.B. 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.811 1.071 0.801 105.5 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPham, Tommy 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.811 1.071 0.801 105.5 13'}, {'pitcher_id': '605452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRoss, Joe 5 4 2 0 0 2 0 0 14.3 1 20 0.5 2 0.941 0.512 1.406 0.769 94.8 11.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDeJong, Paul 3 2 2 0 0 2 0 0 25 1 33.3 1 4 1.568 0.67 2.406 1.061 104.6 31.5\nMaldonado, Martín 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.355 0.407 0.331 84.9 -9'}, {'pitcher_id': '596001', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunis, Jakob 47 39 10 2 0 1 7 14.9 22.1 4 8.5 0.256 0.385 0.33 0.227 0.39 0.323 89.4 19.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBenintendi, Andrew 11 11 5 1 0 0 2 18.2 5 0 0 0.455 0.545 0.428 0.283 0.44 0.309 86.8 20.1\nPham, Tommy 10 9 1 0 0 0 1 10 17.6 1 10 0.111 0.111 0.156 0.167 0.213 0.215 83.8 10.3\nDeJong, Paul 8 7 0 0 0 0 2 25 23.5 0 0 0 0 0.09 0.131 0.199 0.21 93.1 30.4\nVaughn, Andrew 4 3 3 1 0 1 0 0 20 1 25 1 2.333 1.207 0.691 1.826 0.952 90.9 27\nMendick, Danny 3 2 0 0 0 0 0 0 16.7 1 33.3 0 0 0.233 0.11 0.129 0.299 94.3 -25\nRobert Jr., Luis 4 4 0 0 0 0 2 50 60 0 0 0 0 0 0.031 0.089 0.046 98.7 40.5\nSheets, Gavin 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.443 0.478 0.547 98.2 2\nMaldonado, Martín 4 2 1 0 0 0 0 0 20 0 0 0.5 0.5 0.58 0.234 0.234 0.466 100.3 28.5\nLopez, Nicky 1 0 0 0 0 0 0 0 33.3 0 0 0 0.122 76.9 44'}], 146: [{'pitcher_id': '657649', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKoenig, Jared 11 10 1 0 0 0 0 0 8.7 1 9.1 0.1 0.1 0.144 0.311 0.412 0.344 95.8 1.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGordon, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.115 0.143 0.104 89.4 -13\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.21 0.235 0.197 93.3 -10\nDe La Cruz, Bryan 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nRivera, Emmanuel 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.234 0.388 0.266 93.7 9.5\nBruján, Vidal 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.463 0.838 0.537 102.6 17\nSánchez, Jesús 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.514 0.569 0.475 109.3 0\nFortes, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.16 0.18 0.144 100.1 -18\nBell, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.44 0.49 0.406 99.6 1\nBurger, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.345 0.393 0.318 93.7 -1\nLopez, Otto 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.394 0.496 0.383 83 23'}, {'pitcher_id': '489119', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiley, Wade 39 36 11 3 0 1 8 20.5 25.4 2 5.1 0.306 0.472 0.36 0.326 0.494 0.381 89.7 6.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 9 8 0 0 0 0 3 33.3 47.1 1 11.1 0 0 0.077 0.272 0.419 0.341 97.4 9.8\nDe La Cruz, Bryan 7 6 3 2 0 1 1 14.3 35.7 1 14.3 0.5 1.333 0.75 0.396 0.936 0.578 85.2 19.4\nAnderson, Tim 9 9 6 1 0 0 0 0 0 0 0 0.667 0.778 0.622 0.591 0.706 0.559 93.7 9.4\nChisholm Jr., Jazz 6 5 0 0 0 0 3 50 27.3 0 0 0 0 0.12 0.037 0.043 0.15 81.2 -14.5\nSánchez, Jesús 3 3 0 0 0 0 1 33.3 20 0 0 0 0 0 0.152 0.168 0.147 91 12.5\nBethancourt, Christian 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.297\nFortes, Nick 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.118 0.13 0.11 70.3 -27'}, {'pitcher_id': '656730', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Trevor 22 22 8 0 0 0 8 36.4 39 0 0 0.364 0.364 0.321 0.282 0.389 0.29 89.5 8.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 3 3 0 0 0 0 2 66.7 25 0 0 0 0 0 0.228 0.286 0.222 100.2 15\nRivera, Emmanuel 4 4 2 0 0 0 1 25 33.3 0 0 0.5 0.5 0.442 0.395 0.409 0.361 76.6 -11.3\nGordon, Nick 3 3 2 0 0 0 1 33.3 60 0 0 0.667 0.667 0.589 0.345 0.431 0.336 96.1 15\nBurger, Jake 3 3 0 0 0 0 2 66.7 71.4 0 0 0 0 0 0.137 0.525 0.27 104.1 39\nChisholm Jr., Jazz 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.361 0.552 0.382 97.9 -4\nDe La Cruz, Bryan 2 2 2 0 0 0 0 0 0 0 0 1 1 0.884 0.663 0.766 0.622 95 7.5\nBruján, Vidal 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBell, Josh 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.03 0.03 0.029 81.3 -40\nLopez, Otto 1 1 1 0 0 0 0 0 50 0 0 1 1 0.889 0.377 0.487 0.384 68.9 44\nSánchez, Jesús 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.01 0.007 91.5 52'}, {'pitcher_id': '607067', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRea, Colin 12 11 2 0 0 0 3 25 26.3 1 8.3 0.182 0.182 0.205 0.267 0.308 0.288 91.3 5.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.075 0.096 0.068 87.6 -12\nSánchez, Jesús 2 2 1 0 0 0 1 50 33.3 0 0 0.5 0.5 0.441 0.475 0.573 0.458 95.8 13\nDe La Cruz, Bryan 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.002 0.002 0.002 78.9 73\nBurger, Jake 2 1 0 0 0 0 0 0 0 1 50 0 0 0.348 0.62 0.711 0.642 108.2 3\nAnderson, Tim 1 1 0 0 0 0 0 0 33.3 0 0 0 0 0 0.433 0.492 0.398 95.5 2\nFortes, Nick 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.105 0.106 0.098 81.2 -19.5\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.571 0.635 0.525 101.8 4'}, {'pitcher_id': '682842', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUribe, Abner 3 2 1 0 0 0 1 33.3 20 1 33.3 0.5 0.5 0.526 0.393 0.534 0.499 100.9 14', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMyers, Dane 1 0 0 0 0 0 0 0 100 1 100 0.696 0.696\nChisholm Jr., Jazz 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nBell, Josh 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.786 1.069 0.801 100.9 14'}, {'pitcher_id': '663542', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Bryan 5 4 1 0 0 0 2 40 50 0 0 0.25 0.25 0.323 0.235 0.27 0.321 87.1 39', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Bryan 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nChisholm Jr., Jazz 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBruján, Vidal 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.076 0.879 94.7 13\nMyers, Dane 1 0 0 0 0 0 0 0 0 0 0.724 0.724\nBell, Josh 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.003 0.003 79.4 65'}, {'pitcher_id': '676879', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAshby, Aaron 12 12 1 0 1 0 6 50 39.1 0 0 0.083 0.25 0.133 0.078 0.14 0.093 88.7 2.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nDe La Cruz, Bryan 4 4 0 0 0 0 3 75 30 0 0 0 0 0 0.034 0.04 0.032 84.1 -11\nBell, Josh 5 5 1 0 1 0 1 20 42.9 0 0 0.2 0.6 0.32 0.063 0.086 0.066 86.7 1.3\nFortes, Nick 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.241 0.545 0.329 101 22\nSánchez, Jesús 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0'}, {'pitcher_id': '660600', 'player_data': '', 'team_data': 'Enoli Paredes has not faced anyone on this team.'}, {'pitcher_id': '606303', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPayamps, Joel 21 17 3 1 0 0 8 38.1 41.7 4 19 0.176 0.235 0.276 0.172 0.231 0.274 89.1 0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 4 2 0 0 0 0 1 25 57.1 2 50 0 0 0.347 0.116 0.116 0.4 57.1 -52\nGordon, Nick 3 3 1 1 0 0 2 66.7 50 0 0 0.333 0.667 0.42 0.186 0.411 0.248 104 20\nBurger, Jake 3 2 1 0 0 0 0 0 16.7 1 33.3 0.5 0.5 0.526 0.54 0.661 0.577 89.2 20\nSánchez, Jesús 3 3 0 0 0 0 2 66.7 66.7 0 0 0 0 0 0.029 0.03 0.028 96 -31\nLopez, Otto 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.203 0.234 0.19 82.4 26\nBruján, Vidal 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.179 0.199 0.172 94.9 -13\nFortes, Nick 1 0 0 0 0 0 0 0 0 1 100 0.696 0.696\nRivera, Emmanuel 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBell, Josh 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nChisholm Jr., Jazz 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nDe La Cruz, Bryan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.389 0.391 0.35 106.8 -9'}, {'pitcher_id': '669084', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHall, DL 3 3 1 0 0 0 2 66.7 50 0 0 0.333 0.333 0.295 0.126 0.126 0.113 72.9 13', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBethancourt, Christian 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nLopez, Otto 1 1 1 0 0 0 0 0 0 0 0 1 1 0.884 0.377 0.377 0.339 72.9 13'}, {'pitcher_id': '669952', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWhite, Mitch 21 19 9 3 0 1 3 14.3 25.6 0 0 0.474 0.789 0.546 0.274 0.585 0.38 87.7 -2.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Jesús 3 3 1 0 0 1 2 66.7 20 0 0 0.333 1.333 0.669 0.322 1.212 0.618 108.1 24\nBethancourt, Christian 6 5 2 2 0 0 0 0 33.3 0 0 0.4 0.8 0.505 0.198 0.535 0.31 81.5 -8.8\nChisholm Jr., Jazz 3 3 1 0 0 0 0 0 20 0 0 0.333 0.333 0.293 0.337 0.365 0.311 94.9 -7.7\nBell, Josh 4 4 3 1 0 0 0 0 14.3 0 0 0.75 1 0.759 0.465 0.806 0.551 87.5 -4.5\nGordon, Nick 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.442 0.093 0.137 0.087 83.8 17\nDe La Cruz, Bryan 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.184 0.208 0.171 92.4 -10\nBurger, Jake 2 1 0 0 0 0 1 50 33.3 0 0 0 0 0.36 0.36'}, {'pitcher_id': '665625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeguero, Elvis 15 14 3 0 0 0 3 20 40.6 1 6.7 0.214 0.214 0.223 0.273 0.329 0.289 89.1 -0.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRivera, Emmanuel 2 2 1 0 0 0 1 50 30 0 0 0.5 0.5 0.441 0.367 0.506 0.369 102.9 14\nSánchez, Jesús 2 2 1 0 0 0 1 50 50 0 0 0.5 0.5 0.442 0.41 0.521 0.404 104.8 10\nBurger, Jake 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nAnderson, Tim 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.281 0.293 0.262 81 -34.5\nDe La Cruz, Bryan 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.106 0.12 0.098 79.9 18\nGordon, Nick 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.889 0.27 0.33 0.245 96.7 -8\nLopez, Otto 2 2 0 0 0 0 0 0 50 0 0 0 0 0 0.134 0.148 0.122 83.3 -9.5\nFortes, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.394 0.496 0.383 82.9 23\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.557 0.602 0.513 104.4 4'}, {'pitcher_id': '688107', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGasser, Robert 29 24 10 1 0 0 0 0 18.6 0 0 0.417 0.458 0.383 0.369 0.474 0.365 84.7 1.4', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 3 3 2 0 0 0 0 0 50 0 0 0.667 0.667 0.593 0.294 0.352 0.279 77.4 4\nMyers, Dane 4 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.403 0.439 0.566 0.402 88.4 31\nBell, Josh 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0.278 0.305 0.174 88.1 15.3\nBurger, Jake 3 3 0 0 0 0 0 0 25 0 0 0 0 0 0.129 0.142 0.12 66.2 -16\nDe La Cruz, Bryan 4 3 1 0 0 0 0 0 16.7 0 0 0.333 0.333 0.403 0.684 0.792 0.675 105.5 6.5\nRivera, Emmanuel 3 3 2 1 0 0 0 0 25 0 0 0.667 1 0.719 0.579 0.876 0.62 89.9 1.7\nLopez, Otto 3 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.307 0.375 0.297 66.6 -22\nChisholm Jr., Jazz 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.593 0.517 0.689 0.522 97.8 5\nBethancourt, Christian 3 3 1 0 0 0 0 0 25 0 0 0.333 0.333 0.296 0.089 0.103 0.081 83 -16.7'}, {'pitcher_id': '669060', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Bryse 23 20 5 0 0 1 3 13 15.4 2 8.7 0.25 0.4 0.314 0.362 0.526 0.41 87.4 10.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 8 6 1 0 0 0 0 0 7.1 1 12.5 0.167 0.167 0.224 0.295 0.321 0.33 88.6 15.7\nChisholm Jr., Jazz 4 4 1 0 0 0 0 0 11.1 0 0 0.25 0.25 0.22 0.564 0.642 0.527 95.2 24.8\nDe La Cruz, Bryan 3 3 0 0 0 0 1 33.3 28.6 0 0 0 0 0 0.273 0.285 0.249 64.6 -0.5\nSánchez, Jesús 3 2 0 0 0 0 1 33.3 0 1 33.3 0 0 0.232 0.027 0.03 0.255 67.6 -26\nFortes, Nick 3 3 2 0 0 1 1 33.3 50 0 0 0.667 1.667 0.962 0.471 1.361 0.746 106 14.5\nBurger, Jake 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.062 0.064 0.061 80 -24\nBethancourt, Christian 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.851 0.956 0.792 85.6 13'}, {'pitcher_id': '676083', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunk, Janson 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.085 0.165 0.103 94 25', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRivera, Emmanuel 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.085 0.165 0.103 94 25'}, {'pitcher_id': '643361', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Kevin 3 3 0 0 0 0 1 33.3 44.4 0 0 0 0 0 0.054 0.187 0.095 96.6 39', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nFortes, Nick 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.066 0.227 0.113 95.1 38\nBethancourt, Christian 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.097 0.335 0.173 98.1 40\nBruján, Vidal 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '642547', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Freddy 70 67 13 1 0 2 21 30 36.2 3 4.3 0.194 0.299 0.232 0.177 0.269 0.214 85.7 13.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 23 21 4 0 0 1 6 26.1 34.1 2 8.7 0.19 0.333 0.262 0.131 0.187 0.187 87.6 11.6\nDe La Cruz, Bryan 9 9 2 0 0 0 3 33.3 37.5 0 0 0.222 0.222 0.196 0.236 0.262 0.218 81.4 30\nSánchez, Jesús 8 7 0 0 0 0 3 37.5 50 1 12.5 0 0 0.087 0.059 0.122 0.153 83.9 46\nBurger, Jake 8 8 2 0 0 0 4 50 33.3 0 0 0.25 0.25 0.221 0.208 0.265 0.207 88 5.3\nChisholm Jr., Jazz 7 7 2 1 0 1 2 28.6 52.9 0 0 0.286 0.857 0.476 0.288 0.712 0.417 85.1 -1\nAnderson, Tim 5 5 1 0 0 0 1 20 12.5 0 0 0.2 0.2 0.178 0.134 0.155 0.13 91 1.3\nLopez, Otto 2 2 0 0 0 0 0 0 14.3 0 0 0 0 0 0.174 0.463 0.271 93.1 27.5\nGordon, Nick 3 3 0 0 0 0 1 33.3 33.3 0 0 0 0 0 0.042 0.06 0.045 76.1 -1.5\nFortes, Nick 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.437 0.479 0.401 74.6 -3\nBethancourt, Christian 2 2 1 0 0 0 1 50 66.7 0 0 0.5 0.5 0.442 0.222 0.245 0.2 99.9 2'}, {'pitcher_id': '571948', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMilner, Hoby 20 15 0 0 0 0 4 20 26.7 4 20 0 0 0.146 0.1 0.115 0.22 77.1 7.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 7 4 0 0 0 0 1 14.3 22.2 3 42.9 0 0 0.296 0.086 0.098 0.342 77 18.3\nDe La Cruz, Bryan 4 3 0 0 0 0 0 0 42.9 1 25 0 0 0.174 0.183 0.234 0.306 86.1 9.3\nSánchez, Jesús 4 3 0 0 0 0 1 25 14.3 0 0 0 0 0 0.082 0.083 0.078 59.8 -42.3\nChisholm Jr., Jazz 2 2 0 0 0 0 1 50 50 0 0 0 0 0 0.174 0.189 0.162 89.9 0\nFortes, Nick 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.001 0.001 0 83.3 69\nAnderson, Tim 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 83.1 62'}, {'pitcher_id': '668964', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMyers, Tobias 4 4 1 0 0 0 2 50 44.4 0 0 0.25 0.25 0.222 0.135 0.136 0.121 82.3 31.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nBell, Josh 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.001 0.001 0 56.4 60\nBurger, Jake 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.537 0.544 0.483 108.1 3'}, {'pitcher_id': '656266', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBukauskas, J.B. 4 4 0 0 0 0 1 25 10 0 0 0 0 0 0.267 0.294 0.246 103 -7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBell, Josh 2 2 0 0 0 0 1 50 12.5 0 0 0 0 0 0.097 0.097 0.089 107.1 -19\nChisholm Jr., Jazz 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.571 0.655 0.53 104.1 4\nSánchez, Jesús 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.303 0.327 0.275 97.9 -8'}, {'pitcher_id': '605452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRoss, Joe 14 12 5 0 0 1 1 7.1 7.7 2 14.3 0.417 0.667 0.494 0.481 0.59 0.504 90.4 5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSánchez, Jesús 4 3 1 0 0 0 0 0 9.1 1 25 0.333 0.333 0.393 0.722 0.904 0.705 92.3 17\nBell, Josh 5 5 3 0 0 1 0 0 0 0 0 0.6 1.2 0.755 0.372 0.499 0.375 99.8 12.7\nChisholm Jr., Jazz 4 4 1 0 0 0 1 25 14.3 0 0 0.25 0.25 0.22 0.382 0.423 0.353 78.9 -14.7\nDe La Cruz, Bryan 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692'}, {'pitcher_id': '596001', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunis, Jakob 42 38 7 1 1 1 11 26.2 29.7 3 7.1 0.184 0.342 0.265 0.18 0.34 0.266 87.8 6.5', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAnderson, Tim 18 16 3 1 1 0 4 22.2 28.6 2 11.1 0.188 0.375 0.282 0.129 0.237 0.215 84.4 0.3\nRivera, Emmanuel 5 4 0 0 0 0 2 40 40 0 0 0 0 0.144 0.164 0.183 0.267 97.7 26\nBell, Josh 4 3 1 0 0 0 1 25 40 1 25 0.333 0.333 0.393 0.154 0.18 0.277 96.8 -7.5\nDe La Cruz, Bryan 4 4 1 0 0 0 1 25 20 0 0 0.25 0.25 0.221 0.317 0.383 0.305 82.4 5.7\nGordon, Nick 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.289 0.439 0.298 96.5 11.5\nFortes, Nick 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.003 0.003 0.003 88.6 43\nSánchez, Jesús 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.089 0.096 0.085 88.3 23.5\nBethancourt, Christian 3 3 0 0 0 0 1 33.3 60 0 0 0 0 0 0.308 0.82 0.477 76.6 -17.5\nChisholm Jr., Jazz 2 2 1 0 0 1 1 50 0 0 0 0.5 2 1.002 0.345 1.397 0.711 109.5 39'}], 147: [{'pitcher_id': '657649', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nKoenig, Jared 20 16 2 1 0 0 1 5 6.9 3 15 0.125 0.188 0.247 0.187 0.275 0.297 90 19.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeMahieu, DJ 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.023 0.046 0.028 90.1 43.5\nVerdugo, Alex 4 3 0 0 0 0 0 0 20 1 25 0 0 0.172 0.066 0.075 0.218 76.7 10.3\nStanton, Giancarlo 3 2 1 1 0 0 0 0 0 1 33.3 0.5 1 0.653 0.466 0.682 0.559 92.8 42.5\nTorres, Gleyber 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.295 0.213 0.381 0.25 91.9 21.3\nJudge, Aaron 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.626 0.733 0.639 105.9 7\nCabrera, Oswaldo 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.245 0.378 0.265 96.6 0\nRizzo, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.051 0.06 0.047 89.1 -36\nVolpe, Anthony 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTrevino, Jose 1 0 0 0 0 0 0 0 0 0 0 0.72 0.72\nSoto, Juan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.011 0.022 0.015 90.9 51'}, {'pitcher_id': '489119', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMiley, Wade 113 96 19 4 0 1 16 14.2 19.5 13 11.5 0.198 0.271 0.277 0.221 0.358 0.337 87.2 0', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRizzo, Anthony 39 31 6 2 0 1 4 10.3 17.6 4 10.3 0.194 0.355 0.329 0.181 0.364 0.347 82.6 2.8\nLeMahieu, DJ 28 26 8 2 0 0 3 10.7 12.2 2 7.1 0.308 0.385 0.329 0.213 0.26 0.29 89.7 -12\nJudge, Aaron 15 10 3 0 0 0 2 13.3 21.7 5 33.3 0.3 0.3 0.407 0.305 0.416 0.44 96.3 9.5\nStanton, Giancarlo 12 11 1 0 0 0 3 25 35.3 1 8.3 0.091 0.091 0.132 0.138 0.261 0.251 87.6 18.7\nGrisham, Trent 6 6 0 0 0 0 2 33.3 41.7 0 0 0 0 0 0.137 0.158 0.134 80.7 -12.5\nVerdugo, Alex 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.662 1.438 0.875 93.2 27\nSoto, Juan 3 3 0 0 0 0 2 66.7 50 0 0 0 0 0 0.058 0.065 0.053 97.4 -13\nTorres, Gleyber 3 2 0 0 0 0 0 0 0 1 33.3 0 0 0.232 0.449 0.587 0.53 101.7 -11.5\nVolpe, Anthony 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.441 0.128 0.143 0.12 79.8 -22\nWells, Austin 2 2 0 0 0 0 0 0 25 0 0 0 0 0 0.136 0.141 0.124 64.8 -7.5'}, {'pitcher_id': '656730', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMegill, Trevor 21 21 7 1 0 1 3 14.3 16.3 0 0 0.333 0.524 0.369 0.299 0.582 0.377 89.9 15', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeMahieu, DJ 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.213 0.337 0.23 90 23.5\nStanton, Giancarlo 3 3 0 0 0 0 1 33.3 11.1 0 0 0 0 0 0.066 0.111 0.076 86.2 16.5\nTrevino, Jose 3 3 2 0 0 0 0 0 33.3 0 0 0.667 0.667 0.589 0.438 0.49 0.413 82.6 8.7\nJudge, Aaron 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.036 0.707 1.99 1.139 110.2 18\nCabrera, Oswaldo 2 2 1 0 0 0 0 0 14.3 0 0 0.5 0.5 0.442 0.341 0.368 0.315 63.9 -13\nSoto, Juan 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.441 0.212 0.233 0.194 90.2 -22.5\nGrisham, Trent 2 2 0 0 0 0 1 50 40 0 0 0 0 0 0.145 0.537 0.28 100.4 36\nVolpe, Anthony 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTorres, Gleyber 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.346 0.384 0.318 96.4 40\nVerdugo, Alex 1 1 1 1 0 0 0 0 0 0 0 1 2 1.261 0.823 2.716 1.487 103.6 24\nRizzo, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 93.3 59'}, {'pitcher_id': '607067', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRea, Colin 51 45 9 1 0 3 9 17.6 16.5 6 11.8 0.2 0.422 0.315 0.228 0.435 0.33 91.6 14.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeMahieu, DJ 8 7 3 1 0 0 1 12.5 12.5 1 12.5 0.429 0.571 0.462 0.305 0.371 0.352 101.4 2.4\nJudge, Aaron 5 4 1 0 0 0 2 40 16.7 1 20 0.25 0.25 0.316 0.179 0.267 0.292 98.8 33\nRizzo, Anthony 6 5 1 0 0 0 1 16.7 16.7 1 16.7 0.2 0.2 0.262 0.137 0.294 0.263 88.7 16.8\nGrisham, Trent 5 4 1 0 0 1 0 0 0 1 20 0.25 1 0.556 0.262 0.893 0.512 94.5 36.3\nVolpe, Anthony 5 5 0 0 0 0 3 60 36.4 0 0 0 0 0 0.105 0.256 0.149 87.8 -5.5\nCabrera, Oswaldo 5 5 1 0 0 0 0 0 0 0 0 0.2 0.2 0.178 0.181 0.193 0.163 88.2 16\nWells, Austin 4 3 0 0 0 0 0 0 25 1 25 0 0 0.173 0.479 0.555 0.519 93.3 22.3\nSoto, Juan 5 4 1 0 0 1 1 20 0 1 20 0.25 1 0.556 0.296 0.801 0.505 81.5 -14.7\nTorres, Gleyber 5 5 0 0 0 0 1 20 37.5 0 0 0 0 0 0.061 0.112 0.072 82.1 21.3\nVerdugo, Alex 3 3 1 0 0 1 0 0 0 0 0 0.333 1.333 0.695 0.469 1.043 0.634 99.8 14.7'}, {'pitcher_id': '682842', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nUribe, Abner 16 10 2 0 0 0 3 18.8 42.9 5 31.3 0.2 0.2 0.373 0.161 0.29 0.379 89.1 22.9', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTorres, Gleyber 3 3 1 0 0 0 1 33.3 25 0 0 0.333 0.333 0.296 0.079 0.099 0.073 93.1 17.5\nStanton, Giancarlo 2 1 0 0 0 0 0 0 33.3 1 50 0 0 0.348 0.001 0.001 0.348 60.7 65\nJudge, Aaron 2 1 0 0 0 0 1 50 100 1 50 0 0 0.346 0.346\nWells, Austin 1 0 0 0 0 0 0 0 50 1 100 0.696 0.696\nGrisham, Trent 2 1 0 0 0 0 1 50 100 0 0 0 0 0.363 0.363\nRizzo, Anthony 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nCabrera, Oswaldo 1 0 0 0 0 0 0 0 50 1 100 0.692 0.692\nTrevino, Jose 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.751 0.879 0.707 88.1 18\nVolpe, Anthony 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.256 0.8 0.426 96.3 33\nVerdugo, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.107 0.118 0.098 96.3 -24'}, {'pitcher_id': '663542', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHudson, Bryan 7 5 1 0 0 0 1 14.3 22.2 2 28.6 0.2 0.2 0.324 0.324 0.4 0.422 79.2 -10.3', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 2 1 0 0 0 0 0 0 0 1 50 0 0 0.346 0.267 0.267 0.467 65.7 -51\nTorres, Gleyber 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nTrevino, Jose 1 1 0 0 0 0 0 0 40 0 0 0 0 0 0.039 0.04 0.035 62.9 -14\nCabrera, Oswaldo 1 0 0 0 0 0 0 0 0 1 100 0.692 0.692\nSoto, Juan 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.471 0.493 0.424 81.2 11\nGrisham, Trent 1 1 1 0 0 0 0 0 0 0 0 1 1 0.883 0.843 1.2 0.869 107 13'}, {'pitcher_id': '676879', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nAshby, Aaron 8 7 3 0 0 0 2 25 37.5 1 12.5 0.429 0.429 0.418 0.245 0.277 0.289 80.6 -3.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 4 3 1 0 0 0 1 25 30 1 25 0.333 0.333 0.394 0.343 0.387 0.42 81.8 -8.5\nVerdugo, Alex 2 2 0 0 0 0 1 50 75 0 0 0 0 0 0.037 0.037 0.036 75.4 -8\nGrisham, Trent 2 2 2 0 0 0 0 0 0 0 0 1 1 0.884 0.305 0.351 0.279 81.9 3'}, {'pitcher_id': '660600', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nParedes, Enoli 3 2 1 1 0 0 0 0 16.7 1 33.3 0.5 1 0.647 0.186 0.531 0.432 82.1 12', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nTrevino, Jose 2 1 1 1 0 0 0 0 25 1 50 1 2 0.97 0.294 0.976 0.604 97.1 32\nGrisham, Trent 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.079 0.087 0.087 67 -8'}, {'pitcher_id': '606303', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPayamps, Joel 26 23 6 1 0 1 5 19.2 21.3 3 11.5 0.261 0.435 0.341 0.251 0.426 0.334 88.1 5.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nLeMahieu, DJ 5 4 1 0 0 0 0 0 0 1 20 0.25 0.25 0.316 0.366 0.49 0.432 89.5 -13.5\nJudge, Aaron 4 3 1 0 0 0 1 25 50 1 25 0.333 0.333 0.394 0.191 0.209 0.304 95.3 19\nTorres, Gleyber 4 4 1 0 0 0 1 25 37.5 0 0 0.25 0.25 0.221 0.258 0.293 0.247 84.3 -6\nCabrera, Oswaldo 2 2 1 1 0 0 0 0 16.7 0 0 0.5 1 0.622 0.474 0.773 0.521 99.8 17\nStanton, Giancarlo 3 3 0 0 0 0 2 66.7 40 0 0 0 0 0 0.02 0.024 0.018 77.3 43\nTrevino, Jose 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0.067 0.068 0.062 71 -2.7\nVerdugo, Alex 1 0 0 0 0 0 0 0 0 1 100 0.689 0.689\nRizzo, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.14 0.28 0.174 92.1 26\nJones, Jahmai 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nGrisham, Trent 1 1 1 0 0 0 0 0 33.3 0 0 1 1 0.883 0.48 0.53 0.447 98.3 3\nSoto, Juan 1 1 1 0 0 1 0 0 0 0 0 1 4 2.004 0.877 3.415 1.732 103.7 29'}, {'pitcher_id': '669084', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHall, DL 6 6 0 0 0 0 2 33.3 18.8 0 0 0 0 0 0.016 0.017 0.016 78.3 16', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 4 4 0 0 0 0 1 25 14.3 0 0 0 0 0 0.022 0.023 0.022 77.6 -1.3\nCabrera, Oswaldo 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0\nStanton, Giancarlo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.006 0.006 0.005 80.4 68'}, {'pitcher_id': '669952', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWhite, Mitch 43 38 10 1 0 2 6 14 14.5 5 11.6 0.263 0.447 0.349 0.27 0.435 0.347 87 5.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJudge, Aaron 7 5 3 0 0 0 0 0 8.3 2 28.6 0.6 0.6 0.577 0.343 0.368 0.424 98.1 9\nCabrera, Oswaldo 6 5 0 0 0 0 0 0 6.7 1 16.7 0 0 0.115 0.32 0.415 0.373 90.4 10.2\nTorres, Gleyber 4 3 0 0 0 0 1 25 50 1 25 0 0 0.173 0.127 0.15 0.262 85.9 -3\nSoto, Juan 5 4 0 0 0 0 0 0 0 1 20 0 0 0.139 0.167 0.169 0.261 67.5 -26.8\nRizzo, Anthony 4 4 1 0 0 1 1 25 10 0 0 0.25 1 0.521 0.197 0.354 0.234 83.9 37.7\nLeMahieu, DJ 3 3 1 0 0 0 2 66.7 42.9 0 0 0.333 0.333 0.295 0.32 0.37 0.309 89.2 15\nWells, Austin 2 2 0 0 0 0 1 50 25 0 0 0 0 0 0.431 1.663 0.846 103.6 28\nStanton, Giancarlo 2 2 2 0 0 0 0 0 20 0 0 1 1 0.889 0.64 0.774 0.619 107 5.5\nVolpe, Anthony 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.037 0.043 0.035 88.8 -28\nTrevino, Jose 3 3 1 1 0 0 0 0 0 0 0 0.333 0.667 0.42 0.241 0.452 0.297 81.2 0\nGrisham, Trent 3 3 2 0 0 1 0 0 0 0 0 0.667 1.667 0.963 0.345 0.824 0.484 77.1 18.7\nVerdugo, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.086 0.098 0.083 89.3 3.5'}, {'pitcher_id': '665625', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeguero, Elvis 14 12 4 1 0 0 3 21.4 22.7 2 14.3 0.333 0.417 0.378 0.207 0.259 0.274 95.3 -10.6', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.082 0.096 0.078 89.2 -27.5\nRizzo, Anthony 2 2 1 1 0 0 1 50 25 0 0 0.5 1 0.621 0.169 0.287 0.189 92.9 20\nStanton, Giancarlo 1 1 0 0 0 0 1 100 50 0 0 0 0 0 0\nJudge, Aaron 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.216 0.224 0.197 96.9 -12.7\nLeMahieu, DJ 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.198 0.211 0.183 93.8 -16.5\nGrisham, Trent 1 1 0 0 0 0 1 100 66.7 0 0 0 0 0 0\nTorres, Gleyber 1 0 0 0 0 0 0 0 1 100 0.696 0.696\nVolpe, Anthony 1 0 0 0 0 0 0 0 1 100 0.692 0.692\nSoto, Juan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.937 1.252 0.953 107.7 11'}, {'pitcher_id': '688107', 'player_data': '', 'team_data': 'has not faced anyone on this team.'}, {'pitcher_id': '669060', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nWilson, Bryse 31 22 9 2 0 4 2 6.5 12.1 9 29 0.409 1.045 0.626 0.352 0.841 0.553 91.7 17.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 9 5 1 0 0 1 1 11.1 0 4 44.4 0.2 0.8 0.538 0.347 1.12 0.647 87.9 8.5\nRizzo, Anthony 8 7 5 1 0 2 0 0 12.5 1 12.5 0.714 1.714 0.959 0.352 0.749 0.486 89.7 30.4\nGrisham, Trent 5 4 2 1 0 0 1 20 18.2 1 20 0.5 0.75 0.563 0.233 0.312 0.324 97.6 31.7\nJudge, Aaron 2 1 1 0 0 1 0 0 0 1 50 1 4 1.338 0.967 3.547 1.253 113.1 22\nTorres, Gleyber 2 1 0 0 0 0 0 0 0 1 50 0 0 0.349 0.098 0.098 0.395 54.9 -40\nVolpe, Anthony 1 1 0 0 0 0 0 0 25 0 0 0 0 0 0.329 0.376 0.301 96.1 -3\nLeMahieu, DJ 1 0 0 0 0 0 0 0 0 1 100 0.699 0.699\nVerdugo, Alex 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.128 0.128 0.116 89.3 -10\nWells, Austin 2 2 0 0 0 0 0 0 33.3 0 0 0 0 0 0.553 1.13 0.703 104.4 22'}, {'pitcher_id': '676083', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunk, Janson 19 18 11 2 0 1 2 10.5 14 0 0 0.611 0.889 0.617 0.429 0.563 0.409 89.5 18.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanton, Giancarlo 3 3 1 0 0 0 1 33.3 12.5 0 0 0.333 0.333 0.296 0.375 0.4 0.346 85.4 0.5\nRizzo, Anthony 2 2 2 1 0 1 0 0 25 0 0 1 3 1.677 0.524 1.151 0.683 93.3 25\nTrevino, Jose 3 2 1 0 0 0 1 33.3 50 0 0 0.5 0.5 0.293 0.473 0.518 0.313 80.2 25.5\nJudge, Aaron 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.694 0.829 0.671 106.3 6.5\nVolpe, Anthony 1 1 1 0 0 0 0 0 0 0 0 1 1 0.889 0.731 0.85 0.677 86.7 19\nTorres, Gleyber 2 2 1 1 0 0 0 0 0 0 0 0.5 1 0.635 0.189 0.33 0.22 86.6 52\nSoto, Juan 2 2 1 0 0 0 0 0 25 0 0 0.5 0.5 0.444 0.214 0.245 0.198 85.5 33.5\nVerdugo, Alex 2 2 2 0 0 0 0 0 0 0 0 1 1 0.889 0.421 0.475 0.396 90.7 4.5\nCabrera, Oswaldo 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.444 0.413 0.492 0.395 90 -3.5'}, {'pitcher_id': '643361', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nHerget, Kevin 8 7 2 0 0 0 1 12.5 17.6 1 12.5 0.286 0.286 0.308 0.189 0.27 0.262 83.2 -7.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nVerdugo, Alex 4 3 1 0 0 0 0 0 20 1 25 0.333 0.333 0.395 0.361 0.535 0.467 87.6 5.7\nRizzo, Anthony 1 1 1 0 0 0 0 0 14.3 0 0 1 1 0.883 0.047 0.047 0.046 69.8 -14\nTorres, Gleyber 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.048 0.048 0.047 67.2 -29\nVolpe, Anthony 1 1 0 0 0 0 1 100 0 0 0 0 0 0 0\nLeMahieu, DJ 1 1 0 0 0 0 0 0 50 0 0 0 0 0 0.145 0.186 0.134 99.4 -20'}, {'pitcher_id': '642547', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nPeralta, Freddy 40 31 5 2 0 2 11 27.5 28.2 8 20 0.161 0.419 0.342 0.176 0.357 0.331 93.6 26.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 14 10 1 0 0 1 1 7.1 15 4 28.6 0.1 0.4 0.343 0.27 0.515 0.435 90.4 18\nRizzo, Anthony 12 8 0 0 0 0 3 25 45.5 3 25 0 0 0.233 0.049 0.153 0.287 92.5 50.4\nLeMahieu, DJ 6 5 1 0 0 0 3 50 28.6 1 16.7 0.2 0.2 0.262 0.033 0.039 0.145 91.8 5.5\nGrisham, Trent 5 5 1 1 0 0 3 60 44.4 0 0 0.2 0.4 0.249 0.201 0.43 0.266 106.1 28.5\nVerdugo, Alex 3 3 2 1 0 1 1 33.3 7.7 0 0 0.667 2 1.083 0.401 0.786 0.5 99.9 20.5'}, {'pitcher_id': '571948', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMilner, Hoby 24 20 5 2 0 1 6 25 17.9 3 12.5 0.25 0.5 0.38 0.274 0.454 0.375 90.3 7.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 6 5 1 0 0 0 1 16.7 22.2 1 16.7 0.2 0.2 0.264 0.192 0.232 0.271 90.2 -2.8\nGrisham, Trent 3 3 1 1 0 0 1 33.3 0 0 0 0.333 0.667 0.42 0.245 0.282 0.232 86.7 -11\nCabrera, Oswaldo 2 1 0 0 0 0 1 50 20 1 50 0 0 0.344 0.344\nVerdugo, Alex 3 2 1 1 0 0 0 0 0 0 0 0.5 1 0.662 0.478 0.542 0.543 95.9 -2.5\nJudge, Aaron 2 1 0 0 0 0 0 0 0 1 50 0 0 0.344 0.109 0.366 0.44 97.4 39\nTorres, Gleyber 1 1 0 0 0 0 1 100 25 0 0 0 0 0 0\nRizzo, Anthony 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.442 0.524 0.573 0.48 79.5 7.5\nVolpe, Anthony 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.003 0.01 0.006 87.8 41\nLeMahieu, DJ 1 1 0 0 0 0 1 100 33.3 0 0 0 0 0 0\nTrevino, Jose 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.751 0.879 0.707 88.2 18\nStanton, Giancarlo 1 1 1 0 0 1 0 0 0 0 0 1 4 2.031 0.92 3.58 1.818 105.9 26\nWells, Austin 1 1 0 0 0 0 1 100 100 0 0 0 0 0 0'}, {'pitcher_id': '668964', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nMyers, Tobias 20 20 5 1 0 2 5 25 18.2 0 0 0.25 0.6 0.361 0.279 0.597 0.366 90.8 11.8', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 3 3 0 0 0 0 0 0 16.7 0 0 0 0 0 0.394 0.444 0.369 96.6 -5.7\nJudge, Aaron 2 2 1 0 0 1 0 0 0 0 0 0.5 2 1.043 0.545 2.099 1.062 101.7 23\nVolpe, Anthony 3 3 1 0 0 1 0 0 16.7 0 0 0.333 1.333 0.695 0.355 1.179 0.629 90.7 20\nVerdugo, Alex 2 2 0 0 0 0 1 50 33.3 0 0 0 0 0 0.091 0.227 0.132 95.1 27\nStanton, Giancarlo 2 2 0 0 0 0 2 100 75 0 0 0 0 0 0\nTorres, Gleyber 2 2 0 0 0 0 1 50 0 0 0 0 0 0 0.026 0.029 0.025 73.4 -17\nCabrera, Oswaldo 2 2 1 0 0 0 1 50 0 0 0 0.5 0.5 0.444 0.452 0.494 0.414 85.2 18\nRizzo, Anthony 2 2 2 1 0 0 0 0 0 0 0 1 1.5 1.079 0.52 0.644 0.505 88.5 19\nTrevino, Jose 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0.031 0.037 0.028 83.4 11'}, {'pitcher_id': '656266', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nBukauskas, J.B. 3 3 2 0 0 0 0 0 0 0 0 0.667 0.667 0.586 0.356 0.426 0.342 92.3 24.7', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nGrisham, Trent 2 2 1 0 0 0 0 0 0 0 0 0.5 0.5 0.44 0.424 0.522 0.413 92.5 41.5\nSoto, Juan 1 1 1 0 0 0 0 0 0 0 0 1 1 0.879 0.218 0.234 0.199 91.9 -9'}, {'pitcher_id': '605452', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nRoss, Joe 73 63 19 4 0 3 12 16.4 24.8 8 11 0.302 0.508 0.389 0.294 0.534 0.394 91.1 17.1', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nStanton, Giancarlo 16 12 3 0 0 1 4 25 46.4 4 25 0.25 0.5 0.409 0.19 0.474 0.379 94.7 9.1\nRizzo, Anthony 19 16 7 2 0 0 2 10.5 15.6 2 10.5 0.438 0.563 0.462 0.413 0.563 0.451 89.9 15.6\nLeMahieu, DJ 9 9 3 1 0 0 0 0 8.3 0 0 0.333 0.444 0.332 0.392 0.722 0.472 91.9 4.1\nTorres, Gleyber 6 6 2 1 0 1 1 16.7 26.7 0 0 0.333 1 0.546 0.382 0.926 0.543 89.4 15.6\nVerdugo, Alex 6 6 1 0 0 1 2 33.3 20 0 0 0.167 0.667 0.348 0.138 0.404 0.223 90.6 26\nVolpe, Anthony 4 3 0 0 0 0 2 50 50 0 0 0 0 0.181 0.006 0.009 0.186 95.2 61\nJudge, Aaron 3 2 1 0 0 0 1 33.3 50 1 33.3 0.5 0.5 0.527 0.488 0.55 0.534 86.4 16\nSoto, Juan 3 2 0 0 0 0 0 0 25 1 33.3 0 0 0.231 0.01 0.013 0.237 88.8 61\nCabrera, Oswaldo 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.188 0.206 0.171 89.9 20\nWells, Austin 3 3 1 0 0 0 0 0 0 0 0 0.333 0.333 0.296 0.388 0.803 0.5 95.1 31.7\nStroman, Marcus 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.27 0.287 0.251 77.3 8'}, {'pitcher_id': '596001', 'player_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nJunis, Jakob 68 60 18 4 0 2 11 16.2 26.9 4 5.9 0.3 0.467 0.349 0.279 0.468 0.36 86.3 8.2', 'team_data': 'Statcast\nPlayer PA AB H 2B 3B HR SO K% Whiff% BB BB% BA SLG wOBA xBA xSLG xwOBA EV LA\nSoto, Juan 12 11 5 1 0 1 3 25 20 0 0 0.455 0.818 0.553 0.397 0.742 0.502 96.3 11.7\nJudge, Aaron 12 11 4 1 0 1 1 8.3 22.7 0 0 0.364 0.727 0.473 0.364 0.847 0.524 94.3 4.1\nTorres, Gleyber 12 11 3 0 0 0 2 16.7 31.3 1 8.3 0.273 0.273 0.277 0.287 0.34 0.31 77 1.4\nGrisham, Trent 9 8 0 0 0 0 1 11.1 29.4 1 11.1 0 0 0.077 0.131 0.158 0.19 71.6 13.4\nStanton, Giancarlo 7 5 2 0 0 0 1 14.3 41.7 1 14.3 0.4 0.4 0.35 0.148 0.157 0.262 76.1 -3.4\nLeMahieu, DJ 7 6 2 1 0 0 2 28.6 18.2 0 0 0.333 0.5 0.303 0.181 0.2 0.234 97.1 1.8\nRizzo, Anthony 4 3 1 1 0 0 0 0 25 1 25 0.333 0.667 0.483 0.35 0.699 0.503 90.9 29.7\nVolpe, Anthony 2 2 0 0 0 0 1 50 28.6 0 0 0 0 0 0.033 0.042 0.032 76.1 51\nTrevino, Jose 2 2 1 0 0 0 0 0 33.3 0 0 0.5 0.5 0.44 0.47 0.555 0.447 92.4 23\nCabrera, Oswaldo 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0.306 0.35 0.279 96.1 -3'}]}}